# Homework 3 Solutions

**Name**:

**ID**:

> **Due Date**
>
> Thursday, 10/03/23, 9:00pm

## Overview

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

In [1]:
using Random
using CSV
using DataFrames
using Plots
using LaTeXStrings
using Distributions

## Problems (Total: 50/60 Points)

### Problem 1 (30 points)

The first step is to write function(s) to implement the dissolved oxygen
simulation. With two releases, we can turn this into a two-box model,
with the first box from the initial waste release ($x=0\ \text{km}$) to
the second release ($x=15\ \text{km}$), and the second from the second
release to the end of the domain ($x=50\ \text{km}$). As a result, our
lives will be easiest if we write a function to simulate each box with
appropriate initial conditions, which we can then call for each river
segment. An example of how this might look is below. Note that we need
to compute B and N as well to get the appropriate initial conditions at
the transition between boxes (and this might also help with debugging).

``` julia
# mix_concentration: function to compute initial conditions by mixing inflow and new waste stream concentrations
# inputs:
#   - arguments ending in "_in" are inflow, those ending in "_st" are from the stream
#   - V is the volume (L/d), Q is the relevant concentration (mg/L); these should be Floats
# outputs:
#   - mixed concentration (a Float) in mg/L
function mix_concentration(V_in, Q_in, V_st, Q_st)
    Q_mix = ((V_in * Q_in) + (V_st * Q_st)) / (V_in + V_st)
    return Q_mix
end

# dissolved_oxygen: function to simulate dissolved oxygen concentrations for a given segment
# inputs:
#   - x: vector or range of downstream distances to simulate over
#   - C₀, B₀, N₀: initial conditions for DO, CBOD, and NBOD, respectively (mg/L)
#   - U: velocity of river (km/d)
#   - Cₛ = saturation oxygen concentration (mg/L)
#   - ka, kc, kn: reaeration, CBOD decay, and NBOD decay rates, respectively (d^{-1})
function dissolved_oxygen(x, C₀, B₀, N₀, U, Cₛ, ka, kc, kn)

    # initialize vectors for C, B, and N
    # the zeros function lets us define a vector of the appropriate length with values set to zero
    C = zeros(length(x))
    B = zeros(length(x))
    N = zeros(length(x))

    # compute α values for the simulation
    α₁ = exp.(-ka * x / U)
    α₂ = (kc / (ka - kc)) * (exp.(-kc * x / U) .- α₁)
    α₃ = (kn / (ka - kn)) * (exp.(-kn * x / U) .- α₁)

    # loop over values in x to calculate B, N, and C
    for (i, d) in pairs(x)
        B[i] = B₀ * exp(-kb * x[i] / U)
        N[i] = N₀ * exp(-kc * x[i] / U)
        C[i] = Cₛ * (1 - α₁[i]) + (C₀ * α₁[i]) - (B₀ * α₂[i]) - (N₀ * α₃[i])
    end

    return (C, B, N)
end
```

    dissolved_oxygen (generic function with 1 method)

1.  These will be vectors due to the broadcasting of `exp` and `-` over
    the `x` vector. We could also have computed the `α` values in the
    loop below for each value of `x`.
2.  `pairs(x)` lets us directly iterate over indices (`i`) and values
    (`d`) in the vector `x`, rather than only iterating over indices and
    needing to look up the values `d=x[i]`.
3.  While we don’t need `B` and `N` for this solution, returning this
    tuple can be useful for debugging.

Next, let’s simulate the concentrations. Hopefully this is intuitive,
but one critical thing is that we need to compute the initial segment
from $x=0$ to $15\ \text{km}$, not just to 14, as $x=15$ is the inflow
for the initial condition of the segment after the second waste stream.

> **Note**
>
> I initially wrote this out as a script to debug, but then reformulated
> it as a function with an optional parameters for treatment of the two
> waste streams to solve Problems 2 and 3, which means I didn’t have to
> copy and paste everything, possibly introducing new bugs.

``` julia
# do_simulate: function to simulate dissolved oxygen concentrations over the entire river
# inputs:
#   - inflow: tuple with inflow properties: (Volume, DO, CBOD, NBOD)
#   - waste1: tuple with waste stream 1 properties: (Volume, DO, CBOD, NBOD)
#   - waste2: tuple with waste stream 2 properties: (Volume, DO, CBOD, NBOD)
#   - U: velocity of river (km/d)
#   - Cₛ = saturation oxygen concentration (mg/L)
#   - ka, kc, kn: reaeration, CBOD decay, and NBOD decay rates, respectively (d^{-1})
function do_simulate(inflow, waste1, waste2, U, Cₛ, ka, kb, kn)
    # set up ranges for each box/segment
    x₁ = 0:1:15
    x₂ = (15:1:50) .- 15

    V_inflow, C_inflow, B_inflow, N_inflow = inflow
    V_ws1, C_ws1, B_ws1, N_ws1 = waste1
    V_ws2, C_ws2, B_ws2, N_ws2 = waste2

    # initialize storage for final C, B, and N
    # need to store d=1 so the total length should be d+1
    C = zeros(51)
    B = zeros(51)
    N = zeros(51)

    # compute initial conditions for first segment
    C₀ = mix_concentration(V_inflow, C_inflow, V_ws1, C_ws1)
    B₀ = mix_concentration(V_inflow, B_inflow, V_ws1, B_ws1)
    N₀ = mix_concentration(V_inflow, N_inflow, V_ws1, N_ws1)

    # conduct first segment simulation
    (C₁, B₁, N₁) = dissolved_oxygen(x₁, C₀, B₀, N₀, U, Cₛ, ka, kb, kn)
    C[1:15] = C₁[1:end-1]
    B[1:15] = B₁[1:end-1]
    N[1:15] = N₁[1:end-1]

    # compute initial conditions for second segment
    C₀₁ = mix_concentration(V_inflow + V_ws1, C₁[end], V_ws2, C_ws2)
    B₀₁ = mix_concentration(V_inflow + V_ws1, B₁[end], V_ws2, B_ws2)
    N₀₁ = mix_concentration(V_inflow + V_ws1, N₁[end], V_ws2, N_ws2)

    # conduct second segment simulation
    (C₂, B₂, N₂) = dissolved_oxygen(x₂, C₀₁, B₀₁, N₀₁, U, Cₛ, ka, kb, kn)
    C[16:end] = C₂
    B[16:end] = B₂
    N[16:end] = N₂

    return (C, B, N)
end

# set variables for river dynamics
U = 6
Cₛ = 10
ka = 0.55
kb = 0.35
kn = 0.25

# set initial parameters
C_inflow = 7.5 # DO concentration
B_inflow = 5.0 # CBOD
N_inflow = 5.0 # NBOD
V_inflow = 100 * 1_000 # volume converted to L
inflow = (V_inflow, C_inflow, B_inflow, N_inflow)

# set waste stream parameters
C_ws1 = 5.0
B_ws1 = 50.0
N_ws1 = 35.0
V_ws1 = 10 * 1_000
waste1 = (V_ws1, C_ws1, B_ws1, N_ws1)

C_ws2 = 4.0
B_ws2 = 45.0
N_ws2 = 35.0
V_ws2 = 15 * 1_000
waste2 = (V_ws2, C_ws2, B_ws2, N_ws2)

C, B, N = do_simulate(inflow, waste1, waste2, U, Cₛ, ka, kb, kn)
```

    ([7.2727272727272725, 6.718366940001292, 6.252736478441488, 5.865982487427475, 5.549224625451668, 5.294464471079079, 5.094502688374758, 4.942863751725384, 4.8337275512230455, 4.761867260163488  …  6.13037601820527, 6.272760855101071, 6.412340434703638, 6.548872787980461, 6.6821623236453025, 6.812054438906417, 6.938430673374777, 7.061204354693571, 7.1803166891679835, 7.295733254966803], [9.090909090909092, 8.575776817031748, 8.089834281709308, 7.63142746153825, 7.198996057607107, 6.791068184640297, 6.4062553610792135, 6.043247783048131, 5.700809866118693, 5.377776039698319  …  1.916804331604075, 1.8081894764731885, 1.7057292332453498, 1.6090748536058688, 1.5178973509064155, 1.4318863803790853, 1.350749182802631, 1.274209588025372, 1.2020070749530312, 1.1338958848019383], [7.7272727272727275, 7.289410294476985, 6.876359139452911, 6.486713342307512, 6.11914664896604, 5.772407956944252, 5.4453170569173315, 5.136760615590911, 4.845688386200889, 4.571109633743571  …  1.5437012460963393, 1.4562281094565608, 1.3737115987526298, 1.2958708490057462, 1.2224409103247795, 1.1531718460849871, 1.087827882208027, 1.0261866046476376, 0.9680382023494304, 0.9131847531080266])

1.  The colon syntax sets up the range using the syntax
    `initial_value:stepsize:end_value`. In general a stepsize of 1 is
    implicit, but I’ve made it explicit here for illustration.
2.  This starts at 0 because we care about the distance from the initial
    condition, not the “absolute” distance.
3.  Tuples (including multiple outputs from functions) can be unpacked
    into multiple variables this way to make the subsequent code more
    readable (versus just relying on indexing).
4.  We don’t need to store the last value because that occurs at the
    point of mixing with the next waste stream. We will use it to
    compute the mixed concentration at that point.
5.  This was my mistake; I meant for the inflow/waste stream data to not
    include the last thousands digits, *e.g.* to be 100
    m<sup>3</sup>/day. But this should wash out in this problem when you
    compute the mixed concentrations.

Now we can plot the dissolved oxygen concentration, shown in
<a href="#fig-do-untreated" class="quarto-xref">Figure 1</a>.

``` julia
# create plot axis with labels, etc
p = plot(; xlabel="Distance Downriver (km)", ylabel="Dissolved Oxygen Concentration (mg/L)", legend=:top)
plot!(p, 0:50, C, label="Simulated DO")
hline!([4], label="Regulatory Standard")
```

<?xml version="1.0" encoding="utf-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="672" height="480" viewBox="0 0 2688 1920">
<defs>
  <clipPath id="clip810">
    <rect x="0" y="0" width="2688" height="1920"/>
  </clipPath>
</defs>
<path clip-path="url(#clip810)" d="M0 1920 L2688 1920 L2688 -4.26326e-14 L0 -4.26326e-14  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip811">
    <rect x="537" y="0" width="1883" height="1883"/>
  </clipPath>
</defs>
<path clip-path="url(#clip810)" d="M181.719 1734.12 L2640.76 1734.12 L2640.76 47.2441 L181.719 47.2441  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip812">
    <rect x="181" y="47" width="2460" height="1688"/>
  </clipPath>
</defs>
<polyline clip-path="url(#clip812)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="251.315,1734.12 251.315,47.2441 "/>
<polyline clip-path="url(#clip812)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="715.284,1734.12 715.284,47.2441 "/>
<polyline clip-path="url(#clip812)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1179.25,1734.12 1179.25,47.2441 "/>
<polyline clip-path="url(#clip812)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1643.22,1734.12 1643.22,47.2441 "/>
<polyline clip-path="url(#clip812)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="2107.19,1734.12 2107.19,47.2441 "/>
<polyline clip-path="url(#clip812)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="2571.16,1734.12 2571.16,47.2441 "/>
<polyline clip-path="url(#clip810)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,1734.12 2640.76,1734.12 "/>
<polyline clip-path="url(#clip810)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="251.315,1734.12 251.315,1715.22 "/>
<polyline clip-path="url(#clip810)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="715.284,1734.12 715.284,1715.22 "/>
<polyline clip-path="url(#clip810)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1179.25,1734.12 1179.25,1715.22 "/>
<polyline clip-path="url(#clip810)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1643.22,1734.12 1643.22,1715.22 "/>
<polyline clip-path="url(#clip810)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2107.19,1734.12 2107.19,1715.22 "/>
<polyline clip-path="url(#clip810)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2571.16,1734.12 2571.16,1715.22 "/>
<path clip-path="url(#clip810)" d="M251.315 1767.34 Q247.704 1767.34 245.875 1770.9 Q244.07 1774.45 244.07 1781.58 Q244.07 1788.68 245.875 1792.25 Q247.704 1795.79 251.315 1795.79 Q254.949 1795.79 256.755 1792.25 Q258.583 1788.68 258.583 1781.58 Q258.583 1774.45 256.755 1770.9 Q254.949 1767.34 251.315 1767.34 M251.315 1763.64 Q257.125 1763.64 260.181 1768.24 Q263.259 1772.83 263.259 1781.58 Q263.259 1790.3 260.181 1794.91 Q257.125 1799.49 251.315 1799.49 Q245.505 1799.49 242.426 1794.91 Q239.37 1790.3 239.37 1781.58 Q239.37 1772.83 242.426 1768.24 Q245.505 1763.64 251.315 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M689.972 1794.89 L697.61 1794.89 L697.61 1768.52 L689.3 1770.19 L689.3 1765.93 L697.564 1764.26 L702.24 1764.26 L702.24 1794.89 L709.879 1794.89 L709.879 1798.82 L689.972 1798.82 L689.972 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M729.323 1767.34 Q725.712 1767.34 723.883 1770.9 Q722.078 1774.45 722.078 1781.58 Q722.078 1788.68 723.883 1792.25 Q725.712 1795.79 729.323 1795.79 Q732.958 1795.79 734.763 1792.25 Q736.592 1788.68 736.592 1781.58 Q736.592 1774.45 734.763 1770.9 Q732.958 1767.34 729.323 1767.34 M729.323 1763.64 Q735.133 1763.64 738.189 1768.24 Q741.268 1772.83 741.268 1781.58 Q741.268 1790.3 738.189 1794.91 Q735.133 1799.49 729.323 1799.49 Q723.513 1799.49 720.434 1794.91 Q717.379 1790.3 717.379 1781.58 Q717.379 1772.83 720.434 1768.24 Q723.513 1763.64 729.323 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1158.03 1794.89 L1174.35 1794.89 L1174.35 1798.82 L1152.4 1798.82 L1152.4 1794.89 Q1155.06 1792.13 1159.65 1787.5 Q1164.25 1782.85 1165.43 1781.51 Q1167.68 1778.98 1168.56 1777.25 Q1169.46 1775.49 1169.46 1773.8 Q1169.46 1771.04 1167.52 1769.31 Q1165.6 1767.57 1162.49 1767.57 Q1160.29 1767.57 1157.84 1768.34 Q1155.41 1769.1 1152.63 1770.65 L1152.63 1765.93 Q1155.46 1764.79 1157.91 1764.21 Q1160.36 1763.64 1162.4 1763.64 Q1167.77 1763.64 1170.97 1766.32 Q1174.16 1769.01 1174.16 1773.5 Q1174.16 1775.63 1173.35 1777.55 Q1172.56 1779.45 1170.46 1782.04 Q1169.88 1782.71 1166.78 1785.93 Q1163.67 1789.12 1158.03 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1194.16 1767.34 Q1190.55 1767.34 1188.72 1770.9 Q1186.92 1774.45 1186.92 1781.58 Q1186.92 1788.68 1188.72 1792.25 Q1190.55 1795.79 1194.16 1795.79 Q1197.79 1795.79 1199.6 1792.25 Q1201.43 1788.68 1201.43 1781.58 Q1201.43 1774.45 1199.6 1770.9 Q1197.79 1767.34 1194.16 1767.34 M1194.16 1763.64 Q1199.97 1763.64 1203.03 1768.24 Q1206.1 1772.83 1206.1 1781.58 Q1206.1 1790.3 1203.03 1794.91 Q1199.97 1799.49 1194.16 1799.49 Q1188.35 1799.49 1185.27 1794.91 Q1182.22 1790.3 1182.22 1781.58 Q1182.22 1772.83 1185.27 1768.24 Q1188.35 1763.64 1194.16 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1632.06 1780.19 Q1635.42 1780.9 1637.3 1783.17 Q1639.19 1785.44 1639.19 1788.77 Q1639.19 1793.89 1635.68 1796.69 Q1632.16 1799.49 1625.68 1799.49 Q1623.5 1799.49 1621.19 1799.05 Q1618.89 1798.64 1616.44 1797.78 L1616.44 1793.27 Q1618.38 1794.4 1620.7 1794.98 Q1623.01 1795.56 1625.54 1795.56 Q1629.94 1795.56 1632.23 1793.82 Q1634.54 1792.08 1634.54 1788.77 Q1634.54 1785.72 1632.39 1784.01 Q1630.26 1782.27 1626.44 1782.27 L1622.41 1782.27 L1622.41 1778.43 L1626.63 1778.43 Q1630.07 1778.43 1631.9 1777.06 Q1633.73 1775.67 1633.73 1773.08 Q1633.73 1770.42 1631.83 1769.01 Q1629.96 1767.57 1626.44 1767.57 Q1624.52 1767.57 1622.32 1767.99 Q1620.12 1768.4 1617.48 1769.28 L1617.48 1765.12 Q1620.14 1764.38 1622.46 1764.01 Q1624.8 1763.64 1626.86 1763.64 Q1632.18 1763.64 1635.28 1766.07 Q1638.38 1768.47 1638.38 1772.59 Q1638.38 1775.46 1636.74 1777.46 Q1635.1 1779.42 1632.06 1780.19 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1658.06 1767.34 Q1654.45 1767.34 1652.62 1770.9 Q1650.81 1774.45 1650.81 1781.58 Q1650.81 1788.68 1652.62 1792.25 Q1654.45 1795.79 1658.06 1795.79 Q1661.69 1795.79 1663.5 1792.25 Q1665.33 1788.68 1665.33 1781.58 Q1665.33 1774.45 1663.5 1770.9 Q1661.69 1767.34 1658.06 1767.34 M1658.06 1763.64 Q1663.87 1763.64 1666.93 1768.24 Q1670 1772.83 1670 1781.58 Q1670 1790.3 1666.93 1794.91 Q1663.87 1799.49 1658.06 1799.49 Q1652.25 1799.49 1649.17 1794.91 Q1646.12 1790.3 1646.12 1781.58 Q1646.12 1772.83 1649.17 1768.24 Q1652.25 1763.64 1658.06 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M2095.36 1768.34 L2083.56 1786.78 L2095.36 1786.78 L2095.36 1768.34 M2094.14 1764.26 L2100.02 1764.26 L2100.02 1786.78 L2104.95 1786.78 L2104.95 1790.67 L2100.02 1790.67 L2100.02 1798.82 L2095.36 1798.82 L2095.36 1790.67 L2079.76 1790.67 L2079.76 1786.16 L2094.14 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M2122.68 1767.34 Q2119.07 1767.34 2117.24 1770.9 Q2115.43 1774.45 2115.43 1781.58 Q2115.43 1788.68 2117.24 1792.25 Q2119.07 1795.79 2122.68 1795.79 Q2126.31 1795.79 2128.12 1792.25 Q2129.95 1788.68 2129.95 1781.58 Q2129.95 1774.45 2128.12 1770.9 Q2126.31 1767.34 2122.68 1767.34 M2122.68 1763.64 Q2128.49 1763.64 2131.54 1768.24 Q2134.62 1772.83 2134.62 1781.58 Q2134.62 1790.3 2131.54 1794.91 Q2128.49 1799.49 2122.68 1799.49 Q2116.87 1799.49 2113.79 1794.91 Q2110.73 1790.3 2110.73 1781.58 Q2110.73 1772.83 2113.79 1768.24 Q2116.87 1763.64 2122.68 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M2545.86 1764.26 L2564.22 1764.26 L2564.22 1768.2 L2550.14 1768.2 L2550.14 1776.67 Q2551.16 1776.32 2552.18 1776.16 Q2553.2 1775.97 2554.22 1775.97 Q2560 1775.97 2563.38 1779.15 Q2566.76 1782.32 2566.76 1787.73 Q2566.76 1793.31 2563.29 1796.41 Q2559.82 1799.49 2553.5 1799.49 Q2551.32 1799.49 2549.05 1799.12 Q2546.81 1798.75 2544.4 1798.01 L2544.4 1793.31 Q2546.48 1794.45 2548.71 1795 Q2550.93 1795.56 2553.41 1795.56 Q2557.41 1795.56 2559.75 1793.45 Q2562.09 1791.34 2562.09 1787.73 Q2562.09 1784.12 2559.75 1782.02 Q2557.41 1779.91 2553.41 1779.91 Q2551.53 1779.91 2549.66 1780.33 Q2547.8 1780.74 2545.86 1781.62 L2545.86 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M2585.98 1767.34 Q2582.36 1767.34 2580.54 1770.9 Q2578.73 1774.45 2578.73 1781.58 Q2578.73 1788.68 2580.54 1792.25 Q2582.36 1795.79 2585.98 1795.79 Q2589.61 1795.79 2591.42 1792.25 Q2593.24 1788.68 2593.24 1781.58 Q2593.24 1774.45 2591.42 1770.9 Q2589.61 1767.34 2585.98 1767.34 M2585.98 1763.64 Q2591.79 1763.64 2594.84 1768.24 Q2597.92 1772.83 2597.92 1781.58 Q2597.92 1790.3 2594.84 1794.91 Q2591.79 1799.49 2585.98 1799.49 Q2580.17 1799.49 2577.09 1794.91 Q2574.03 1790.3 2574.03 1781.58 Q2574.03 1772.83 2577.09 1768.24 Q2580.17 1763.64 2585.98 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1016.83 1841.06 L1016.83 1878.02 L1024.6 1878.02 Q1034.44 1878.02 1038.99 1873.56 Q1043.57 1869.11 1043.57 1859.49 Q1043.57 1849.94 1038.99 1845.52 Q1034.44 1841.06 1024.6 1841.06 L1016.83 1841.06 M1010.41 1835.78 L1023.61 1835.78 Q1037.43 1835.78 1043.89 1841.54 Q1050.35 1847.27 1050.35 1859.49 Q1050.35 1871.78 1043.86 1877.54 Q1037.36 1883.3 1023.61 1883.3 L1010.41 1883.3 L1010.41 1835.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1060.34 1847.65 L1066.2 1847.65 L1066.2 1883.3 L1060.34 1883.3 L1060.34 1847.65 M1060.34 1833.78 L1066.2 1833.78 L1066.2 1841.19 L1060.34 1841.19 L1060.34 1833.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1101.18 1848.7 L1101.18 1854.24 Q1098.7 1852.97 1096.02 1852.33 Q1093.35 1851.7 1090.49 1851.7 Q1086.13 1851.7 1083.93 1853.03 Q1081.76 1854.37 1081.76 1857.04 Q1081.76 1859.08 1083.32 1860.26 Q1084.88 1861.4 1089.59 1862.45 L1091.6 1862.9 Q1097.84 1864.24 1100.45 1866.69 Q1103.09 1869.11 1103.09 1873.47 Q1103.09 1878.43 1099.14 1881.33 Q1095.23 1884.22 1088.35 1884.22 Q1085.49 1884.22 1082.37 1883.65 Q1079.28 1883.11 1075.84 1882 L1075.84 1875.95 Q1079.09 1877.64 1082.24 1878.49 Q1085.39 1879.32 1088.48 1879.32 Q1092.62 1879.32 1094.85 1877.92 Q1097.07 1876.49 1097.07 1873.91 Q1097.07 1871.52 1095.45 1870.25 Q1093.86 1868.98 1088.42 1867.8 L1086.38 1867.32 Q1080.94 1866.18 1078.52 1863.82 Q1076.1 1861.43 1076.1 1857.3 Q1076.1 1852.27 1079.66 1849.53 Q1083.23 1846.79 1089.79 1846.79 Q1093.03 1846.79 1095.9 1847.27 Q1098.76 1847.75 1101.18 1848.7 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1118.21 1837.53 L1118.21 1847.65 L1130.27 1847.65 L1130.27 1852.2 L1118.21 1852.2 L1118.21 1871.56 Q1118.21 1875.92 1119.39 1877.16 Q1120.6 1878.4 1124.26 1878.4 L1130.27 1878.4 L1130.27 1883.3 L1124.26 1883.3 Q1117.48 1883.3 1114.9 1880.79 Q1112.32 1878.24 1112.32 1871.56 L1112.32 1852.2 L1108.02 1852.2 L1108.02 1847.65 L1112.32 1847.65 L1112.32 1837.53 L1118.21 1837.53 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1154.17 1865.38 Q1147.08 1865.38 1144.34 1867 Q1141.6 1868.63 1141.6 1872.54 Q1141.6 1875.66 1143.64 1877.51 Q1145.71 1879.32 1149.24 1879.32 Q1154.11 1879.32 1157.04 1875.88 Q1160 1872.42 1160 1866.69 L1160 1865.38 L1154.17 1865.38 M1165.86 1862.96 L1165.86 1883.3 L1160 1883.3 L1160 1877.89 Q1157.99 1881.14 1155 1882.7 Q1152.01 1884.22 1147.68 1884.22 Q1142.21 1884.22 1138.96 1881.17 Q1135.75 1878.08 1135.75 1872.92 Q1135.75 1866.91 1139.76 1863.85 Q1143.8 1860.8 1151.79 1860.8 L1160 1860.8 L1160 1860.23 Q1160 1856.18 1157.33 1853.99 Q1154.68 1851.76 1149.88 1851.76 Q1146.82 1851.76 1143.93 1852.49 Q1141.03 1853.22 1138.36 1854.69 L1138.36 1849.28 Q1141.57 1848.03 1144.59 1847.43 Q1147.62 1846.79 1150.48 1846.79 Q1158.22 1846.79 1162.04 1850.8 Q1165.86 1854.81 1165.86 1862.96 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1207.55 1861.78 L1207.55 1883.3 L1201.69 1883.3 L1201.69 1861.98 Q1201.69 1856.92 1199.72 1854.4 Q1197.75 1851.89 1193.8 1851.89 Q1189.06 1851.89 1186.32 1854.91 Q1183.58 1857.93 1183.58 1863.15 L1183.58 1883.3 L1177.7 1883.3 L1177.7 1847.65 L1183.58 1847.65 L1183.58 1853.19 Q1185.68 1849.98 1188.52 1848.39 Q1191.38 1846.79 1195.11 1846.79 Q1201.25 1846.79 1204.4 1850.61 Q1207.55 1854.4 1207.55 1861.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1244.89 1849.02 L1244.89 1854.5 Q1242.4 1853.13 1239.89 1852.46 Q1237.41 1851.76 1234.86 1851.76 Q1229.16 1851.76 1226.01 1855.39 Q1222.86 1858.98 1222.86 1865.51 Q1222.86 1872.03 1226.01 1875.66 Q1229.16 1879.26 1234.86 1879.26 Q1237.41 1879.26 1239.89 1878.59 Q1242.4 1877.89 1244.89 1876.52 L1244.89 1881.93 Q1242.44 1883.08 1239.79 1883.65 Q1237.18 1884.22 1234.22 1884.22 Q1226.17 1884.22 1221.43 1879.16 Q1216.69 1874.1 1216.69 1865.51 Q1216.69 1856.79 1221.46 1851.79 Q1226.27 1846.79 1234.61 1846.79 Q1237.31 1846.79 1239.89 1847.37 Q1242.47 1847.91 1244.89 1849.02 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1285.56 1864.01 L1285.56 1866.88 L1258.64 1866.88 Q1259.02 1872.92 1262.26 1876.11 Q1265.54 1879.26 1271.37 1879.26 Q1274.74 1879.26 1277.89 1878.43 Q1281.07 1877.6 1284.19 1875.95 L1284.19 1881.49 Q1281.04 1882.82 1277.73 1883.52 Q1274.42 1884.22 1271.02 1884.22 Q1262.49 1884.22 1257.49 1879.26 Q1252.52 1874.29 1252.52 1865.83 Q1252.52 1857.07 1257.24 1851.95 Q1261.98 1846.79 1270 1846.79 Q1277.19 1846.79 1281.36 1851.44 Q1285.56 1856.06 1285.56 1864.01 M1279.71 1862.29 Q1279.64 1857.49 1277 1854.62 Q1274.39 1851.76 1270.06 1851.76 Q1265.16 1851.76 1262.2 1854.53 Q1259.27 1857.3 1258.83 1862.33 L1279.71 1862.29 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1322.58 1841.06 L1322.58 1878.02 L1330.35 1878.02 Q1340.18 1878.02 1344.73 1873.56 Q1349.32 1869.11 1349.32 1859.49 Q1349.32 1849.94 1344.73 1845.52 Q1340.18 1841.06 1330.35 1841.06 L1322.58 1841.06 M1316.15 1835.78 L1329.36 1835.78 Q1343.17 1835.78 1349.63 1841.54 Q1356.09 1847.27 1356.09 1859.49 Q1356.09 1871.78 1349.6 1877.54 Q1343.11 1883.3 1329.36 1883.3 L1316.15 1883.3 L1316.15 1835.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1379.9 1851.76 Q1375.19 1851.76 1372.45 1855.45 Q1369.72 1859.11 1369.72 1865.51 Q1369.72 1871.91 1372.42 1875.6 Q1375.16 1879.26 1379.9 1879.26 Q1384.58 1879.26 1387.32 1875.57 Q1390.06 1871.87 1390.06 1865.51 Q1390.06 1859.17 1387.32 1855.48 Q1384.58 1851.76 1379.9 1851.76 M1379.9 1846.79 Q1387.54 1846.79 1391.9 1851.76 Q1396.26 1856.72 1396.26 1865.51 Q1396.26 1874.26 1391.9 1879.26 Q1387.54 1884.22 1379.9 1884.22 Q1372.23 1884.22 1367.87 1879.26 Q1363.54 1874.26 1363.54 1865.51 Q1363.54 1856.72 1367.87 1851.76 Q1372.23 1846.79 1379.9 1846.79 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1402.56 1847.65 L1408.42 1847.65 L1415.74 1875.47 L1423.03 1847.65 L1429.94 1847.65 L1437.26 1875.47 L1444.55 1847.65 L1450.4 1847.65 L1441.08 1883.3 L1434.17 1883.3 L1426.5 1854.08 L1418.8 1883.3 L1411.89 1883.3 L1402.56 1847.65 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1488.92 1861.78 L1488.92 1883.3 L1483.06 1883.3 L1483.06 1861.98 Q1483.06 1856.92 1481.09 1854.4 Q1479.11 1851.89 1475.17 1851.89 Q1470.42 1851.89 1467.69 1854.91 Q1464.95 1857.93 1464.95 1863.15 L1464.95 1883.3 L1459.06 1883.3 L1459.06 1847.65 L1464.95 1847.65 L1464.95 1853.19 Q1467.05 1849.98 1469.88 1848.39 Q1472.75 1846.79 1476.47 1846.79 Q1482.61 1846.79 1485.76 1850.61 Q1488.92 1854.4 1488.92 1861.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1521.25 1853.13 Q1520.27 1852.55 1519.09 1852.3 Q1517.94 1852.01 1516.54 1852.01 Q1511.58 1852.01 1508.9 1855.26 Q1506.26 1858.47 1506.26 1864.52 L1506.26 1883.3 L1500.37 1883.3 L1500.37 1847.65 L1506.26 1847.65 L1506.26 1853.19 Q1508.11 1849.94 1511.07 1848.39 Q1514.03 1846.79 1518.26 1846.79 Q1518.87 1846.79 1519.6 1846.89 Q1520.33 1846.95 1521.22 1847.11 L1521.25 1853.13 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1527.4 1847.65 L1533.25 1847.65 L1533.25 1883.3 L1527.4 1883.3 L1527.4 1847.65 M1527.4 1833.78 L1533.25 1833.78 L1533.25 1841.19 L1527.4 1841.19 L1527.4 1833.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1541.31 1847.65 L1547.51 1847.65 L1558.65 1877.57 L1569.79 1847.65 L1576 1847.65 L1562.63 1883.3 L1554.67 1883.3 L1541.31 1847.65 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1614.57 1864.01 L1614.57 1866.88 L1587.65 1866.88 Q1588.03 1872.92 1591.28 1876.11 Q1594.55 1879.26 1600.38 1879.26 Q1603.75 1879.26 1606.9 1878.43 Q1610.09 1877.6 1613.21 1875.95 L1613.21 1881.49 Q1610.05 1882.82 1606.74 1883.52 Q1603.43 1884.22 1600.03 1884.22 Q1591.5 1884.22 1586.5 1879.26 Q1581.54 1874.29 1581.54 1865.83 Q1581.54 1857.07 1586.25 1851.95 Q1590.99 1846.79 1599.01 1846.79 Q1606.2 1846.79 1610.37 1851.44 Q1614.57 1856.06 1614.57 1864.01 M1608.72 1862.29 Q1608.65 1857.49 1606.01 1854.62 Q1603.4 1851.76 1599.07 1851.76 Q1594.17 1851.76 1591.21 1854.53 Q1588.28 1857.3 1587.84 1862.33 L1608.72 1862.29 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1644.84 1853.13 Q1643.86 1852.55 1642.68 1852.3 Q1641.53 1852.01 1640.13 1852.01 Q1635.17 1852.01 1632.49 1855.26 Q1629.85 1858.47 1629.85 1864.52 L1629.85 1883.3 L1623.96 1883.3 L1623.96 1847.65 L1629.85 1847.65 L1629.85 1853.19 Q1631.7 1849.94 1634.66 1848.39 Q1637.62 1846.79 1641.85 1846.79 Q1642.46 1846.79 1643.19 1846.89 Q1643.92 1846.95 1644.81 1847.11 L1644.84 1853.13 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1685.77 1833.84 Q1681.51 1841.16 1679.44 1848.32 Q1677.37 1855.48 1677.37 1862.84 Q1677.37 1870.19 1679.44 1877.41 Q1681.54 1884.61 1685.77 1891.89 L1680.68 1891.89 Q1675.91 1884.41 1673.52 1877.19 Q1671.17 1869.96 1671.17 1862.84 Q1671.17 1855.74 1673.52 1848.54 Q1675.88 1841.35 1680.68 1833.84 L1685.77 1833.84 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1696.91 1833.78 L1702.8 1833.78 L1702.8 1863.03 L1720.28 1847.65 L1727.76 1847.65 L1708.85 1864.33 L1728.55 1883.3 L1720.91 1883.3 L1702.8 1865.89 L1702.8 1883.3 L1696.91 1883.3 L1696.91 1833.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1762.64 1854.5 Q1764.84 1850.55 1767.89 1848.67 Q1770.95 1846.79 1775.09 1846.79 Q1780.66 1846.79 1783.68 1850.71 Q1786.7 1854.59 1786.7 1861.78 L1786.7 1883.3 L1780.81 1883.3 L1780.81 1861.98 Q1780.81 1856.85 1779 1854.37 Q1777.19 1851.89 1773.46 1851.89 Q1768.91 1851.89 1766.27 1854.91 Q1763.63 1857.93 1763.63 1863.15 L1763.63 1883.3 L1757.74 1883.3 L1757.74 1861.98 Q1757.74 1856.82 1755.92 1854.37 Q1754.11 1851.89 1750.32 1851.89 Q1745.84 1851.89 1743.19 1854.94 Q1740.55 1857.97 1740.55 1863.15 L1740.55 1883.3 L1734.66 1883.3 L1734.66 1847.65 L1740.55 1847.65 L1740.55 1853.19 Q1742.56 1849.91 1745.36 1848.35 Q1748.16 1846.79 1752.01 1846.79 Q1755.89 1846.79 1758.6 1848.77 Q1761.34 1850.74 1762.64 1854.5 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1797.46 1833.84 L1802.55 1833.84 Q1807.33 1841.35 1809.68 1848.54 Q1812.07 1855.74 1812.07 1862.84 Q1812.07 1869.96 1809.68 1877.19 Q1807.33 1884.41 1802.55 1891.89 L1797.46 1891.89 Q1801.69 1884.61 1803.76 1877.41 Q1805.86 1870.19 1805.86 1862.84 Q1805.86 1855.48 1803.76 1848.32 Q1801.69 1841.16 1797.46 1833.84 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip812)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,1613.71 2640.76,1613.71 "/>
<polyline clip-path="url(#clip812)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,1152.9 2640.76,1152.9 "/>
<polyline clip-path="url(#clip812)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,692.081 2640.76,692.081 "/>
<polyline clip-path="url(#clip812)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,231.265 2640.76,231.265 "/>
<polyline clip-path="url(#clip810)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,1734.12 181.719,47.2441 "/>
<polyline clip-path="url(#clip810)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,1613.71 200.617,1613.71 "/>
<polyline clip-path="url(#clip810)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,1152.9 200.617,1152.9 "/>
<polyline clip-path="url(#clip810)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,692.081 200.617,692.081 "/>
<polyline clip-path="url(#clip810)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,231.265 200.617,231.265 "/>
<path clip-path="url(#clip810)" d="M131.816 1600.51 L120.011 1618.96 L131.816 1618.96 L131.816 1600.51 M130.589 1596.43 L136.469 1596.43 L136.469 1618.96 L141.399 1618.96 L141.399 1622.85 L136.469 1622.85 L136.469 1630.99 L131.816 1630.99 L131.816 1622.85 L116.214 1622.85 L116.214 1618.33 L130.589 1596.43 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M120.497 1135.62 L138.853 1135.62 L138.853 1139.55 L124.779 1139.55 L124.779 1148.03 Q125.798 1147.68 126.816 1147.52 Q127.835 1147.33 128.853 1147.33 Q134.64 1147.33 138.02 1150.5 Q141.399 1153.67 141.399 1159.09 Q141.399 1164.67 137.927 1167.77 Q134.455 1170.85 128.136 1170.85 Q125.96 1170.85 123.691 1170.48 Q121.446 1170.11 119.039 1169.37 L119.039 1164.67 Q121.122 1165.8 123.344 1166.36 Q125.566 1166.91 128.043 1166.91 Q132.048 1166.91 134.386 1164.81 Q136.724 1162.7 136.724 1159.09 Q136.724 1155.48 134.386 1153.37 Q132.048 1151.27 128.043 1151.27 Q126.168 1151.27 124.293 1151.68 Q122.441 1152.1 120.497 1152.98 L120.497 1135.62 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M129.872 690.218 Q126.724 690.218 124.872 692.371 Q123.043 694.523 123.043 698.273 Q123.043 702 124.872 704.176 Q126.724 706.329 129.872 706.329 Q133.02 706.329 134.849 704.176 Q136.7 702 136.7 698.273 Q136.7 694.523 134.849 692.371 Q133.02 690.218 129.872 690.218 M139.154 675.565 L139.154 679.824 Q137.395 678.991 135.589 678.551 Q133.807 678.111 132.048 678.111 Q127.418 678.111 124.964 681.236 Q122.534 684.361 122.187 690.681 Q123.552 688.667 125.613 687.602 Q127.673 686.514 130.15 686.514 Q135.358 686.514 138.367 689.685 Q141.399 692.833 141.399 698.273 Q141.399 703.597 138.251 706.815 Q135.103 710.032 129.872 710.032 Q123.876 710.032 120.705 705.449 Q117.534 700.843 117.534 692.116 Q117.534 683.921 121.423 679.06 Q125.312 674.176 131.863 674.176 Q133.622 674.176 135.404 674.523 Q137.21 674.871 139.154 675.565 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M119.177 213.985 L141.399 213.985 L141.399 215.975 L128.853 248.545 L123.969 248.545 L135.775 217.92 L119.177 217.92 L119.177 213.985 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M19.402 1529.67 L56.3549 1529.67 L56.3549 1521.9 Q56.3549 1512.07 51.8989 1507.52 Q47.4429 1502.93 37.8307 1502.93 Q28.2821 1502.93 23.858 1507.52 Q19.402 1512.07 19.402 1521.9 L19.402 1529.67 M14.1184 1536.1 L14.1184 1522.89 Q14.1184 1509.08 19.8794 1502.62 Q25.6085 1496.15 37.8307 1496.15 Q50.1165 1496.15 55.8775 1502.65 Q61.6384 1509.14 61.6384 1522.89 L61.6384 1536.1 L14.1184 1536.1 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M25.9905 1486.16 L25.9905 1480.3 L61.6384 1480.3 L61.6384 1486.16 L25.9905 1486.16 M12.1132 1486.16 L12.1132 1480.3 L19.5293 1480.3 L19.5293 1486.16 L12.1132 1486.16 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M27.0408 1445.32 L32.579 1445.32 Q31.3059 1447.81 30.6693 1450.48 Q30.0327 1453.15 30.0327 1456.02 Q30.0327 1460.38 31.3695 1462.58 Q32.7063 1464.74 35.3799 1464.74 Q37.4169 1464.74 38.5946 1463.18 Q39.7404 1461.62 40.7908 1456.91 L41.2364 1454.9 Q42.5732 1448.67 45.0239 1446.06 Q47.4429 1443.41 51.8034 1443.41 Q56.7687 1443.41 59.6651 1447.36 Q62.5615 1451.28 62.5615 1458.15 Q62.5615 1461.02 61.9886 1464.14 Q61.4475 1467.22 60.3335 1470.66 L54.2861 1470.66 Q55.973 1467.41 56.8323 1464.26 Q57.6599 1461.11 57.6599 1458.02 Q57.6599 1453.89 56.2594 1451.66 Q54.8271 1449.43 52.249 1449.43 Q49.8619 1449.43 48.5887 1451.05 Q47.3156 1452.65 46.1379 1458.09 L45.6605 1460.12 Q44.5147 1465.57 42.1594 1467.99 Q39.7722 1470.41 35.6345 1470.41 Q30.6056 1470.41 27.8684 1466.84 Q25.1311 1463.28 25.1311 1456.72 Q25.1311 1453.47 25.6085 1450.61 Q26.086 1447.74 27.0408 1445.32 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M27.0408 1411.36 L32.579 1411.36 Q31.3059 1413.85 30.6693 1416.52 Q30.0327 1419.19 30.0327 1422.06 Q30.0327 1426.42 31.3695 1428.61 Q32.7063 1430.78 35.3799 1430.78 Q37.4169 1430.78 38.5946 1429.22 Q39.7404 1427.66 40.7908 1422.95 L41.2364 1420.94 Q42.5732 1414.71 45.0239 1412.1 Q47.4429 1409.45 51.8034 1409.45 Q56.7687 1409.45 59.6651 1413.4 Q62.5615 1417.32 62.5615 1424.19 Q62.5615 1427.05 61.9886 1430.17 Q61.4475 1433.26 60.3335 1436.7 L54.2861 1436.7 Q55.973 1433.45 56.8323 1430.3 Q57.6599 1427.15 57.6599 1424.06 Q57.6599 1419.93 56.2594 1417.7 Q54.8271 1415.47 52.249 1415.47 Q49.8619 1415.47 48.5887 1417.09 Q47.3156 1418.68 46.1379 1424.13 L45.6605 1426.16 Q44.5147 1431.61 42.1594 1434.03 Q39.7722 1436.44 35.6345 1436.44 Q30.6056 1436.44 27.8684 1432.88 Q25.1311 1429.31 25.1311 1422.76 Q25.1311 1419.51 25.6085 1416.65 Q26.086 1413.78 27.0408 1411.36 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M30.0964 1386.31 Q30.0964 1391.03 33.7885 1393.76 Q37.4488 1396.5 43.8463 1396.5 Q50.2438 1396.5 53.9359 1393.79 Q57.5962 1391.06 57.5962 1386.31 Q57.5962 1381.64 53.9041 1378.9 Q50.212 1376.16 43.8463 1376.16 Q37.5124 1376.16 33.8203 1378.9 Q30.0964 1381.64 30.0964 1386.31 M25.1311 1386.31 Q25.1311 1378.68 30.0964 1374.32 Q35.0616 1369.95 43.8463 1369.95 Q52.5991 1369.95 57.5962 1374.32 Q62.5615 1378.68 62.5615 1386.31 Q62.5615 1393.99 57.5962 1398.35 Q52.5991 1402.67 43.8463 1402.67 Q35.0616 1402.67 30.0964 1398.35 Q25.1311 1393.99 25.1311 1386.31 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M12.1132 1360.25 L12.1132 1354.39 L61.6384 1354.39 L61.6384 1360.25 L12.1132 1360.25 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M25.9905 1346.34 L25.9905 1340.13 L55.9093 1328.99 L25.9905 1317.85 L25.9905 1311.64 L61.6384 1325.01 L61.6384 1332.97 L25.9905 1346.34 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M42.3504 1273.07 L45.2149 1273.07 L45.2149 1300 Q51.2623 1299.61 54.4452 1296.37 Q57.5962 1293.09 57.5962 1287.26 Q57.5962 1283.89 56.7687 1280.74 Q55.9411 1277.56 54.2861 1274.44 L59.8242 1274.44 Q61.161 1277.59 61.8612 1280.9 Q62.5615 1284.21 62.5615 1287.61 Q62.5615 1296.14 57.5962 1301.14 Q52.631 1306.11 44.1646 1306.11 Q35.4117 1306.11 30.2873 1301.4 Q25.1311 1296.65 25.1311 1288.63 Q25.1311 1281.44 29.7781 1277.27 Q34.3932 1273.07 42.3504 1273.07 M40.6316 1278.92 Q35.8255 1278.99 32.9609 1281.63 Q30.0964 1284.24 30.0964 1288.57 Q30.0964 1293.47 32.8654 1296.43 Q35.6345 1299.36 40.6634 1299.8 L40.6316 1278.92 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M31.4013 1240 L12.1132 1240 L12.1132 1234.14 L61.6384 1234.14 L61.6384 1240 L56.2912 1240 Q59.4741 1241.84 61.0337 1244.68 Q62.5615 1247.48 62.5615 1251.43 Q62.5615 1257.89 57.4052 1261.96 Q52.249 1266 43.8463 1266 Q35.4436 1266 30.2873 1261.96 Q25.1311 1257.89 25.1311 1251.43 Q25.1311 1247.48 26.6907 1244.68 Q28.2185 1241.84 31.4013 1240 M43.8463 1259.96 Q50.3075 1259.96 53.9996 1257.31 Q57.6599 1254.64 57.6599 1249.99 Q57.6599 1245.35 53.9996 1242.67 Q50.3075 1240 43.8463 1240 Q37.3851 1240 33.7248 1242.67 Q30.0327 1245.35 30.0327 1249.99 Q30.0327 1254.64 33.7248 1257.31 Q37.3851 1259.96 43.8463 1259.96 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M18.479 1181.82 Q18.479 1188.82 23.6988 1192.96 Q28.9187 1197.06 37.9262 1197.06 Q46.9018 1197.06 52.1217 1192.96 Q57.3416 1188.82 57.3416 1181.82 Q57.3416 1174.81 52.1217 1170.74 Q46.9018 1166.63 37.9262 1166.63 Q28.9187 1166.63 23.6988 1170.74 Q18.479 1174.81 18.479 1181.82 M13.2591 1181.82 Q13.2591 1171.82 19.9749 1165.84 Q26.6589 1159.85 37.9262 1159.85 Q49.1617 1159.85 55.8775 1165.84 Q62.5615 1171.82 62.5615 1181.82 Q62.5615 1191.84 55.8775 1197.86 Q49.1935 1203.84 37.9262 1203.84 Q26.6589 1203.84 19.9749 1197.86 Q13.2591 1191.84 13.2591 1181.82 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M25.9905 1120.42 L43.337 1133.31 L61.6384 1119.75 L61.6384 1126.66 L47.6339 1137.03 L61.6384 1147.41 L61.6384 1154.32 L42.9869 1140.47 L25.9905 1153.14 L25.9905 1146.23 L38.6901 1136.78 L25.9905 1127.33 L25.9905 1120.42 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M64.9486 1096.64 Q71.3143 1099.13 73.2559 1101.48 Q75.1974 1103.84 75.1974 1107.78 L75.1974 1112.46 L70.2958 1112.46 L70.2958 1109.02 Q70.2958 1106.61 69.15 1105.27 Q68.0041 1103.93 63.7391 1102.31 L61.0655 1101.26 L25.9905 1115.68 L25.9905 1109.47 L53.8723 1098.33 L25.9905 1087.19 L25.9905 1080.98 L64.9486 1096.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M43.4007 1049.44 Q37.035 1049.44 33.5338 1052.08 Q30.0327 1054.69 30.0327 1059.44 Q30.0327 1064.15 33.5338 1066.79 Q37.035 1069.4 43.4007 1069.4 Q49.7346 1069.4 53.2357 1066.79 Q56.7368 1064.15 56.7368 1059.44 Q56.7368 1054.69 53.2357 1052.08 Q49.7346 1049.44 43.4007 1049.44 M57.2143 1043.58 Q66.3172 1043.58 70.7414 1047.63 Q75.1974 1051.67 75.1974 1060.01 Q75.1974 1063.1 74.72 1065.83 Q74.2744 1068.57 73.3195 1071.15 L67.6222 1071.15 Q69.0227 1068.57 69.6911 1066.06 Q70.3595 1063.54 70.3595 1060.93 Q70.3595 1055.17 67.3358 1052.31 Q64.3439 1049.44 58.2646 1049.44 L55.3682 1049.44 Q58.5192 1051.26 60.0788 1054.09 Q61.6384 1056.92 61.6384 1060.87 Q61.6384 1067.42 56.6414 1071.43 Q51.6443 1075.45 43.4007 1075.45 Q35.1253 1075.45 30.1282 1071.43 Q25.1311 1067.42 25.1311 1060.87 Q25.1311 1056.92 26.6907 1054.09 Q28.2503 1051.26 31.4013 1049.44 L25.9905 1049.44 L25.9905 1043.58 L57.2143 1043.58 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M42.3504 1001.03 L45.2149 1001.03 L45.2149 1027.96 Q51.2623 1027.58 54.4452 1024.33 Q57.5962 1021.05 57.5962 1015.23 Q57.5962 1011.85 56.7687 1008.7 Q55.9411 1005.52 54.2861 1002.4 L59.8242 1002.4 Q61.161 1005.55 61.8612 1008.86 Q62.5615 1012.17 62.5615 1015.58 Q62.5615 1024.11 57.5962 1029.1 Q52.631 1034.07 44.1646 1034.07 Q35.4117 1034.07 30.2873 1029.36 Q25.1311 1024.62 25.1311 1016.59 Q25.1311 1009.4 29.7781 1005.23 Q34.3932 1001.03 42.3504 1001.03 M40.6316 1006.89 Q35.8255 1006.95 32.9609 1009.59 Q30.0964 1012.2 30.0964 1016.53 Q30.0964 1021.43 32.8654 1024.39 Q35.6345 1027.32 40.6634 1027.77 L40.6316 1006.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M40.1224 961.785 L61.6384 961.785 L61.6384 967.642 L40.3133 967.642 Q35.2526 967.642 32.7381 969.615 Q30.2237 971.589 30.2237 975.535 Q30.2237 980.278 33.2474 983.015 Q36.2711 985.752 41.491 985.752 L61.6384 985.752 L61.6384 991.641 L25.9905 991.641 L25.9905 985.752 L31.5287 985.752 Q28.314 983.652 26.7225 980.819 Q25.1311 977.954 25.1311 974.23 Q25.1311 968.088 28.9505 964.937 Q32.7381 961.785 40.1224 961.785 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M17.7787 893.545 L24.5582 893.545 Q21.5345 896.792 20.0386 900.484 Q18.5426 904.144 18.5426 908.282 Q18.5426 916.43 23.5397 920.759 Q28.5049 925.087 37.9262 925.087 Q47.3156 925.087 52.3127 920.759 Q57.2779 916.43 57.2779 908.282 Q57.2779 904.144 55.782 900.484 Q54.2861 896.792 51.2623 893.545 L57.9782 893.545 Q60.2698 896.919 61.4156 900.707 Q62.5615 904.462 62.5615 908.664 Q62.5615 919.454 55.973 925.66 Q49.3526 931.867 37.9262 931.867 Q26.4679 931.867 19.8794 925.66 Q13.2591 919.454 13.2591 908.664 Q13.2591 904.399 14.4049 900.643 Q15.5189 896.855 17.7787 893.545 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M30.0964 870.056 Q30.0964 874.766 33.7885 877.504 Q37.4488 880.241 43.8463 880.241 Q50.2438 880.241 53.9359 877.535 Q57.5962 874.798 57.5962 870.056 Q57.5962 865.377 53.9041 862.64 Q50.212 859.902 43.8463 859.902 Q37.5124 859.902 33.8203 862.64 Q30.0964 865.377 30.0964 870.056 M25.1311 870.056 Q25.1311 862.417 30.0964 858.056 Q35.0616 853.696 43.8463 853.696 Q52.5991 853.696 57.5962 858.056 Q62.5615 862.417 62.5615 870.056 Q62.5615 877.726 57.5962 882.087 Q52.5991 886.416 43.8463 886.416 Q35.0616 886.416 30.0964 882.087 Q25.1311 877.726 25.1311 870.056 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M40.1224 814.356 L61.6384 814.356 L61.6384 820.212 L40.3133 820.212 Q35.2526 820.212 32.7381 822.186 Q30.2237 824.159 30.2237 828.106 Q30.2237 832.848 33.2474 835.585 Q36.2711 838.323 41.491 838.323 L61.6384 838.323 L61.6384 844.211 L25.9905 844.211 L25.9905 838.323 L31.5287 838.323 Q28.314 836.222 26.7225 833.389 Q25.1311 830.525 25.1311 826.801 Q25.1311 820.658 28.9505 817.507 Q32.7381 814.356 40.1224 814.356 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M27.3591 777.021 L32.8336 777.021 Q31.465 779.503 30.7966 782.018 Q30.0964 784.501 30.0964 787.047 Q30.0964 792.744 33.7248 795.895 Q37.3214 799.046 43.8463 799.046 Q50.3711 799.046 53.9996 795.895 Q57.5962 792.744 57.5962 787.047 Q57.5962 784.501 56.9278 782.018 Q56.2276 779.503 54.859 777.021 L60.2698 777.021 Q61.4156 779.472 61.9886 782.113 Q62.5615 784.723 62.5615 787.683 Q62.5615 795.736 57.5007 800.478 Q52.44 805.221 43.8463 805.221 Q35.1253 805.221 30.1282 800.447 Q25.1311 795.641 25.1311 787.301 Q25.1311 784.596 25.704 782.018 Q26.2451 779.44 27.3591 777.021 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M42.3504 736.344 L45.2149 736.344 L45.2149 763.271 Q51.2623 762.889 54.4452 759.642 Q57.5962 756.364 57.5962 750.54 Q57.5962 747.166 56.7687 744.015 Q55.9411 740.832 54.2861 737.713 L59.8242 737.713 Q61.161 740.864 61.8612 744.174 Q62.5615 747.484 62.5615 750.89 Q62.5615 759.42 57.5962 764.417 Q52.631 769.382 44.1646 769.382 Q35.4117 769.382 30.2873 764.671 Q25.1311 759.929 25.1311 751.908 Q25.1311 744.715 29.7781 740.545 Q34.3932 736.344 42.3504 736.344 M40.6316 742.2 Q35.8255 742.264 32.9609 744.906 Q30.0964 747.516 30.0964 751.844 Q30.0964 756.746 32.8654 759.706 Q35.6345 762.634 40.6634 763.08 L40.6316 742.2 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M40.1224 697.099 L61.6384 697.099 L61.6384 702.956 L40.3133 702.956 Q35.2526 702.956 32.7381 704.929 Q30.2237 706.903 30.2237 710.849 Q30.2237 715.592 33.2474 718.329 Q36.2711 721.066 41.491 721.066 L61.6384 721.066 L61.6384 726.955 L25.9905 726.955 L25.9905 721.066 L31.5287 721.066 Q28.314 718.966 26.7225 716.133 Q25.1311 713.268 25.1311 709.544 Q25.1311 703.401 28.9505 700.25 Q32.7381 697.099 40.1224 697.099 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M15.869 679.626 L25.9905 679.626 L25.9905 667.563 L30.542 667.563 L30.542 679.626 L49.8937 679.626 Q54.2542 679.626 55.4955 678.448 Q56.7368 677.238 56.7368 673.578 L56.7368 667.563 L61.6384 667.563 L61.6384 673.578 Q61.6384 680.358 59.124 682.936 Q56.5777 685.514 49.8937 685.514 L30.542 685.514 L30.542 689.811 L25.9905 689.811 L25.9905 685.514 L15.869 685.514 L15.869 679.626 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M31.465 639.203 Q30.8921 640.19 30.6375 641.368 Q30.351 642.513 30.351 643.914 Q30.351 648.879 33.5975 651.553 Q36.8122 654.195 42.8596 654.195 L61.6384 654.195 L61.6384 660.083 L25.9905 660.083 L25.9905 654.195 L31.5287 654.195 Q28.2821 652.348 26.7225 649.388 Q25.1311 646.428 25.1311 642.195 Q25.1311 641.59 25.2266 640.858 Q25.2903 640.126 25.4494 639.235 L31.465 639.203 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M43.719 616.86 Q43.719 623.957 45.3422 626.695 Q46.9655 629.432 50.8804 629.432 Q53.9996 629.432 55.8457 627.395 Q57.6599 625.326 57.6599 621.793 Q57.6599 616.923 54.2224 613.995 Q50.7531 611.035 45.0239 611.035 L43.719 611.035 L43.719 616.86 M41.3 605.179 L61.6384 605.179 L61.6384 611.035 L56.2276 611.035 Q59.4741 613.04 61.0337 616.032 Q62.5615 619.024 62.5615 623.353 Q62.5615 628.827 59.5059 632.074 Q56.4186 635.288 51.2623 635.288 Q45.2467 635.288 42.1912 631.278 Q39.1357 627.236 39.1357 619.247 L39.1357 611.035 L38.5628 611.035 Q34.5205 611.035 32.3244 613.709 Q30.0964 616.35 30.0964 621.157 Q30.0964 624.212 30.8284 627.108 Q31.5605 630.005 33.0246 632.678 L27.6137 632.678 Q26.3724 629.464 25.7677 626.44 Q25.1311 623.416 25.1311 620.552 Q25.1311 612.817 29.1415 608.998 Q33.1519 605.179 41.3 605.179 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M15.869 587.323 L25.9905 587.323 L25.9905 575.26 L30.542 575.26 L30.542 587.323 L49.8937 587.323 Q54.2542 587.323 55.4955 586.145 Q56.7368 584.936 56.7368 581.275 L56.7368 575.26 L61.6384 575.26 L61.6384 581.275 Q61.6384 588.055 59.124 590.633 Q56.5777 593.211 49.8937 593.211 L30.542 593.211 L30.542 597.508 L25.9905 597.508 L25.9905 593.211 L15.869 593.211 L15.869 587.323 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M25.9905 567.557 L25.9905 561.701 L61.6384 561.701 L61.6384 567.557 L25.9905 567.557 M12.1132 567.557 L12.1132 561.701 L19.5293 561.701 L19.5293 567.557 L12.1132 567.557 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M30.0964 535.633 Q30.0964 540.344 33.7885 543.081 Q37.4488 545.818 43.8463 545.818 Q50.2438 545.818 53.9359 543.113 Q57.5962 540.376 57.5962 535.633 Q57.5962 530.954 53.9041 528.217 Q50.212 525.48 43.8463 525.48 Q37.5124 525.48 33.8203 528.217 Q30.0964 530.954 30.0964 535.633 M25.1311 535.633 Q25.1311 527.994 30.0964 523.634 Q35.0616 519.273 43.8463 519.273 Q52.5991 519.273 57.5962 523.634 Q62.5615 527.994 62.5615 535.633 Q62.5615 543.304 57.5962 547.664 Q52.5991 551.993 43.8463 551.993 Q35.0616 551.993 30.0964 547.664 Q25.1311 543.304 25.1311 535.633 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M40.1224 479.933 L61.6384 479.933 L61.6384 485.79 L40.3133 485.79 Q35.2526 485.79 32.7381 487.763 Q30.2237 489.737 30.2237 493.683 Q30.2237 498.426 33.2474 501.163 Q36.2711 503.9 41.491 503.9 L61.6384 503.9 L61.6384 509.788 L25.9905 509.788 L25.9905 503.9 L31.5287 503.9 Q28.314 501.8 26.7225 498.967 Q25.1311 496.102 25.1311 492.378 Q25.1311 486.235 28.9505 483.084 Q32.7381 479.933 40.1224 479.933 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M12.1769 433.464 Q19.4975 437.729 26.6589 439.798 Q33.8203 441.866 41.1727 441.866 Q48.5251 441.866 55.7502 439.798 Q62.9434 437.697 70.2321 433.464 L70.2321 438.556 Q62.7524 443.331 55.5274 445.718 Q48.3023 448.073 41.1727 448.073 Q34.0749 448.073 26.8817 445.718 Q19.6884 443.362 12.1769 438.556 L12.1769 433.464 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M32.8336 394.346 Q28.8869 392.15 27.009 389.095 Q25.1311 386.039 25.1311 381.901 Q25.1311 376.331 29.046 373.308 Q32.9291 370.284 40.1224 370.284 L61.6384 370.284 L61.6384 376.172 L40.3133 376.172 Q35.1889 376.172 32.7063 377.987 Q30.2237 379.801 30.2237 383.525 Q30.2237 388.076 33.2474 390.718 Q36.2711 393.36 41.491 393.36 L61.6384 393.36 L61.6384 399.248 L40.3133 399.248 Q35.1571 399.248 32.7063 401.062 Q30.2237 402.876 30.2237 406.664 Q30.2237 411.152 33.2792 413.794 Q36.3029 416.435 41.491 416.435 L61.6384 416.435 L61.6384 422.324 L25.9905 422.324 L25.9905 416.435 L31.5287 416.435 Q28.2503 414.43 26.6907 411.629 Q25.1311 408.828 25.1311 404.977 Q25.1311 401.094 27.1045 398.389 Q29.0779 395.651 32.8336 394.346 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M43.4007 335.145 Q37.035 335.145 33.5338 337.787 Q30.0327 340.397 30.0327 345.139 Q30.0327 349.85 33.5338 352.492 Q37.035 355.102 43.4007 355.102 Q49.7346 355.102 53.2357 352.492 Q56.7368 349.85 56.7368 345.139 Q56.7368 340.397 53.2357 337.787 Q49.7346 335.145 43.4007 335.145 M57.2143 329.289 Q66.3172 329.289 70.7414 333.331 Q75.1974 337.373 75.1974 345.712 Q75.1974 348.8 74.72 351.537 Q74.2744 354.274 73.3195 356.852 L67.6222 356.852 Q69.0227 354.274 69.6911 351.76 Q70.3595 349.245 70.3595 346.635 Q70.3595 340.874 67.3358 338.01 Q64.3439 335.145 58.2646 335.145 L55.3682 335.145 Q58.5192 336.96 60.0788 339.792 Q61.6384 342.625 61.6384 346.572 Q61.6384 353.128 56.6414 357.139 Q51.6443 361.149 43.4007 361.149 Q35.1253 361.149 30.1282 357.139 Q25.1311 353.128 25.1311 346.572 Q25.1311 342.625 26.6907 339.792 Q28.2503 336.96 31.4013 335.145 L25.9905 335.145 L25.9905 329.289 L57.2143 329.289 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M14.1184 306.818 L14.1184 301.407 L67.6859 317.958 L67.6859 323.369 L14.1184 306.818 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M14.1184 295.01 L14.1184 288.58 L56.2276 288.58 L56.2276 265.441 L61.6384 265.441 L61.6384 295.01 L14.1184 295.01 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M12.1769 259.871 L12.1769 254.778 Q19.6884 250.004 26.8817 247.649 Q34.0749 245.262 41.1727 245.262 Q48.3023 245.262 55.5274 247.649 Q62.7524 250.004 70.2321 254.778 L70.2321 259.871 Q62.9434 255.638 55.7502 253.569 Q48.5251 251.468 41.1727 251.468 Q33.8203 251.468 26.6589 253.569 Q19.4975 255.638 12.1769 259.871 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip812)" style="stroke:#009af9; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="251.315,105.587 297.712,361.046 344.109,575.616 390.506,753.839 436.903,899.806 483.299,1017.2 529.696,1109.35 576.093,1179.23 622.49,1229.52 668.887,1262.63 715.284,1280.73 761.681,1285.75 808.078,1279.43 854.475,1263.31 900.872,1238.79 947.269,1255.9 993.665,1389.28 1040.06,1493.48 1086.46,1571.95 1132.86,1627.83 1179.25,1663.89 1225.65,1682.66 1272.05,1686.38 1318.44,1677.05 1364.84,1656.48 1411.24,1626.28 1457.63,1587.87 1504.03,1542.54 1550.43,1491.43 1596.83,1435.53 1643.22,1375.74 1689.62,1312.86 1736.02,1247.58 1782.41,1180.52 1828.81,1112.21 1875.21,1043.13 1921.6,973.686 1968,904.241 2014.4,835.102 2060.79,766.537 2107.19,698.772 2153.59,632.002 2199.99,566.388 2246.38,502.068 2292.78,439.151 2339.18,377.729 2385.57,317.873 2431.97,259.637 2478.37,203.061 2524.76,148.172 2571.16,94.9858 "/>
<polyline clip-path="url(#clip812)" style="stroke:#e26f46; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="-2277.32,1613.71 5099.79,1613.71 "/>
<path clip-path="url(#clip810)" d="M1048.66 258.993 L1773.82 258.993 L1773.82 103.473 L1048.66 103.473  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip810)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1048.66,258.993 1773.82,258.993 1773.82,103.473 1048.66,103.473 1048.66,258.993 "/>
<polyline clip-path="url(#clip810)" style="stroke:#009af9; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1075.98,155.313 1239.92,155.313 "/>
<path clip-path="url(#clip810)" d="M1289.48 139.167 L1289.48 143.728 Q1286.82 142.454 1284.46 141.829 Q1282.1 141.204 1279.9 141.204 Q1276.08 141.204 1274 142.686 Q1271.94 144.167 1271.94 146.899 Q1271.94 149.191 1273.3 150.371 Q1274.69 151.528 1278.53 152.246 L1281.36 152.825 Q1286.59 153.82 1289.07 156.343 Q1291.57 158.843 1291.57 163.056 Q1291.57 168.079 1288.19 170.672 Q1284.83 173.264 1278.33 173.264 Q1275.87 173.264 1273.09 172.709 Q1270.34 172.153 1267.38 171.065 L1267.38 166.251 Q1270.22 167.848 1272.96 168.658 Q1275.69 169.468 1278.33 169.468 Q1282.33 169.468 1284.51 167.894 Q1286.68 166.32 1286.68 163.403 Q1286.68 160.857 1285.11 159.422 Q1283.56 157.987 1279.99 157.269 L1277.15 156.714 Q1271.91 155.672 1269.58 153.45 Q1267.24 151.228 1267.24 147.269 Q1267.24 142.686 1270.46 140.047 Q1273.7 137.408 1279.37 137.408 Q1281.8 137.408 1284.32 137.848 Q1286.84 138.288 1289.48 139.167 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1298.67 146.667 L1302.93 146.667 L1302.93 172.593 L1298.67 172.593 L1298.67 146.667 M1298.67 136.575 L1302.93 136.575 L1302.93 141.968 L1298.67 141.968 L1298.67 136.575 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1332.03 151.644 Q1333.63 148.774 1335.85 147.408 Q1338.07 146.042 1341.08 146.042 Q1345.13 146.042 1347.33 148.89 Q1349.53 151.714 1349.53 156.945 L1349.53 172.593 L1345.25 172.593 L1345.25 157.084 Q1345.25 153.357 1343.93 151.552 Q1342.61 149.746 1339.9 149.746 Q1336.59 149.746 1334.67 151.945 Q1332.75 154.144 1332.75 157.94 L1332.75 172.593 L1328.47 172.593 L1328.47 157.084 Q1328.47 153.334 1327.15 151.552 Q1325.83 149.746 1323.07 149.746 Q1319.81 149.746 1317.89 151.968 Q1315.97 154.167 1315.97 157.94 L1315.97 172.593 L1311.68 172.593 L1311.68 146.667 L1315.97 146.667 L1315.97 150.695 Q1317.42 148.311 1319.46 147.177 Q1321.5 146.042 1324.3 146.042 Q1327.12 146.042 1329.09 147.478 Q1331.08 148.913 1332.03 151.644 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1357.59 162.362 L1357.59 146.667 L1361.84 146.667 L1361.84 162.2 Q1361.84 165.88 1363.28 167.732 Q1364.71 169.561 1367.59 169.561 Q1371.03 169.561 1373.03 167.362 Q1375.04 165.163 1375.04 161.366 L1375.04 146.667 L1379.3 146.667 L1379.3 172.593 L1375.04 172.593 L1375.04 168.612 Q1373.49 170.973 1371.43 172.13 Q1369.39 173.264 1366.68 173.264 Q1362.21 173.264 1359.9 170.487 Q1357.59 167.709 1357.59 162.362 M1368.3 146.042 L1368.3 146.042 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1388.07 136.575 L1392.33 136.575 L1392.33 172.593 L1388.07 172.593 L1388.07 136.575 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1413.02 159.561 Q1407.86 159.561 1405.87 160.741 Q1403.88 161.922 1403.88 164.769 Q1403.88 167.038 1405.36 168.38 Q1406.87 169.7 1409.44 169.7 Q1412.98 169.7 1415.11 167.2 Q1417.26 164.677 1417.26 160.51 L1417.26 159.561 L1413.02 159.561 M1421.52 157.802 L1421.52 172.593 L1417.26 172.593 L1417.26 168.658 Q1415.8 171.019 1413.63 172.153 Q1411.45 173.264 1408.3 173.264 Q1404.32 173.264 1401.96 171.042 Q1399.62 168.797 1399.62 165.047 Q1399.62 160.672 1402.54 158.45 Q1405.48 156.228 1411.29 156.228 L1417.26 156.228 L1417.26 155.811 Q1417.26 152.871 1415.32 151.274 Q1413.4 149.653 1409.9 149.653 Q1407.68 149.653 1405.57 150.186 Q1403.46 150.718 1401.52 151.783 L1401.52 147.848 Q1403.86 146.945 1406.06 146.505 Q1408.26 146.042 1410.34 146.042 Q1415.96 146.042 1418.74 148.959 Q1421.52 151.876 1421.52 157.802 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1434.51 139.306 L1434.51 146.667 L1443.28 146.667 L1443.28 149.978 L1434.51 149.978 L1434.51 164.052 Q1434.51 167.223 1435.36 168.126 Q1436.24 169.028 1438.9 169.028 L1443.28 169.028 L1443.28 172.593 L1438.9 172.593 Q1433.97 172.593 1432.1 170.764 Q1430.22 168.913 1430.22 164.052 L1430.22 149.978 L1427.1 149.978 L1427.1 146.667 L1430.22 146.667 L1430.22 139.306 L1434.51 139.306 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1471.06 158.565 L1471.06 160.649 L1451.47 160.649 Q1451.75 165.047 1454.11 167.362 Q1456.5 169.653 1460.73 169.653 Q1463.19 169.653 1465.48 169.052 Q1467.79 168.45 1470.06 167.246 L1470.06 171.274 Q1467.77 172.246 1465.36 172.755 Q1462.96 173.264 1460.48 173.264 Q1454.27 173.264 1450.64 169.653 Q1447.03 166.042 1447.03 159.885 Q1447.03 153.519 1450.46 149.792 Q1453.9 146.042 1459.74 146.042 Q1464.97 146.042 1468 149.422 Q1471.06 152.778 1471.06 158.565 M1466.8 157.315 Q1466.75 153.82 1464.83 151.737 Q1462.93 149.653 1459.78 149.653 Q1456.22 149.653 1454.07 151.667 Q1451.94 153.681 1451.61 157.339 L1466.8 157.315 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1495.11 150.603 L1495.11 136.575 L1499.37 136.575 L1499.37 172.593 L1495.11 172.593 L1495.11 168.704 Q1493.77 171.019 1491.7 172.153 Q1489.67 173.264 1486.8 173.264 Q1482.1 173.264 1479.14 169.515 Q1476.2 165.765 1476.2 159.653 Q1476.2 153.542 1479.14 149.792 Q1482.1 146.042 1486.8 146.042 Q1489.67 146.042 1491.7 147.177 Q1493.77 148.288 1495.11 150.603 M1480.59 159.653 Q1480.59 164.352 1482.52 167.038 Q1484.46 169.7 1487.84 169.7 Q1491.22 169.7 1493.16 167.038 Q1495.11 164.352 1495.11 159.653 Q1495.11 154.954 1493.16 152.292 Q1491.22 149.607 1487.84 149.607 Q1484.46 149.607 1482.52 152.292 Q1480.59 154.954 1480.59 159.653 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1528.07 141.876 L1528.07 168.751 L1533.72 168.751 Q1540.87 168.751 1544.18 165.51 Q1547.51 162.269 1547.51 155.278 Q1547.51 148.334 1544.18 145.116 Q1540.87 141.876 1533.72 141.876 L1528.07 141.876 M1523.39 138.033 L1533 138.033 Q1543.05 138.033 1547.75 142.223 Q1552.45 146.39 1552.45 155.278 Q1552.45 164.214 1547.72 168.403 Q1543 172.593 1533 172.593 L1523.39 172.593 L1523.39 138.033 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1573.93 141.204 Q1568.83 141.204 1565.82 145.001 Q1562.84 148.797 1562.84 155.348 Q1562.84 161.876 1565.82 165.672 Q1568.83 169.468 1573.93 169.468 Q1579.02 169.468 1581.98 165.672 Q1584.97 161.876 1584.97 155.348 Q1584.97 148.797 1581.98 145.001 Q1579.02 141.204 1573.93 141.204 M1573.93 137.408 Q1581.2 137.408 1585.55 142.292 Q1589.9 147.154 1589.9 155.348 Q1589.9 163.519 1585.55 168.403 Q1581.2 173.264 1573.93 173.264 Q1566.64 173.264 1562.26 168.403 Q1557.91 163.542 1557.91 155.348 Q1557.91 147.154 1562.26 142.292 Q1566.64 137.408 1573.93 137.408 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip810)" style="stroke:#e26f46; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1075.98,207.153 1239.92,207.153 "/>
<path clip-path="url(#clip810)" d="M1283.63 208.23 Q1285.13 208.739 1286.54 210.405 Q1287.98 212.072 1289.41 214.989 L1294.16 224.433 L1289.14 224.433 L1284.72 215.567 Q1283 212.095 1281.38 210.961 Q1279.78 209.827 1277.01 209.827 L1271.91 209.827 L1271.91 224.433 L1267.24 224.433 L1267.24 189.873 L1277.79 189.873 Q1283.72 189.873 1286.64 192.35 Q1289.55 194.827 1289.55 199.827 Q1289.55 203.091 1288.03 205.243 Q1286.52 207.396 1283.63 208.23 M1271.91 193.716 L1271.91 205.984 L1277.79 205.984 Q1281.17 205.984 1282.89 204.433 Q1284.62 202.859 1284.62 199.827 Q1284.62 196.794 1282.89 195.267 Q1281.17 193.716 1277.79 193.716 L1271.91 193.716 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1320.04 210.405 L1320.04 212.489 L1300.46 212.489 Q1300.73 216.887 1303.09 219.202 Q1305.48 221.493 1309.72 221.493 Q1312.17 221.493 1314.46 220.892 Q1316.78 220.29 1319.04 219.086 L1319.04 223.114 Q1316.75 224.086 1314.34 224.595 Q1311.94 225.104 1309.46 225.104 Q1303.26 225.104 1299.62 221.493 Q1296.01 217.882 1296.01 211.725 Q1296.01 205.359 1299.44 201.632 Q1302.89 197.882 1308.72 197.882 Q1313.95 197.882 1316.98 201.262 Q1320.04 204.618 1320.04 210.405 M1315.78 209.155 Q1315.73 205.66 1313.81 203.577 Q1311.91 201.493 1308.77 201.493 Q1305.2 201.493 1303.05 203.507 Q1300.92 205.521 1300.59 209.179 L1315.78 209.155 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1344.09 211.169 Q1344.09 206.54 1342.17 203.993 Q1340.27 201.447 1336.82 201.447 Q1333.4 201.447 1331.47 203.993 Q1329.58 206.54 1329.58 211.169 Q1329.58 215.776 1331.47 218.322 Q1333.4 220.868 1336.82 220.868 Q1340.27 220.868 1342.17 218.322 Q1344.09 215.776 1344.09 211.169 M1348.35 221.216 Q1348.35 227.836 1345.41 231.054 Q1342.47 234.294 1336.4 234.294 Q1334.16 234.294 1332.17 233.947 Q1330.18 233.623 1328.3 232.929 L1328.3 228.785 Q1330.18 229.804 1332.01 230.29 Q1333.84 230.776 1335.73 230.776 Q1339.92 230.776 1342.01 228.577 Q1344.09 226.401 1344.09 221.98 L1344.09 219.873 Q1342.77 222.165 1340.71 223.299 Q1338.65 224.433 1335.78 224.433 Q1331.01 224.433 1328.09 220.799 Q1325.18 217.165 1325.18 211.169 Q1325.18 205.151 1328.09 201.517 Q1331.01 197.882 1335.78 197.882 Q1338.65 197.882 1340.71 199.017 Q1342.77 200.151 1344.09 202.443 L1344.09 198.507 L1348.35 198.507 L1348.35 221.216 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1356.68 214.202 L1356.68 198.507 L1360.94 198.507 L1360.94 214.04 Q1360.94 217.72 1362.38 219.572 Q1363.81 221.401 1366.68 221.401 Q1370.13 221.401 1372.12 219.202 Q1374.14 217.003 1374.14 213.206 L1374.14 198.507 L1378.4 198.507 L1378.4 224.433 L1374.14 224.433 L1374.14 220.452 Q1372.59 222.813 1370.53 223.97 Q1368.49 225.104 1365.78 225.104 Q1361.31 225.104 1359 222.327 Q1356.68 219.549 1356.68 214.202 M1367.4 197.882 L1367.4 197.882 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1387.17 188.415 L1391.43 188.415 L1391.43 224.433 L1387.17 224.433 L1387.17 188.415 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1412.12 211.401 Q1406.96 211.401 1404.97 212.581 Q1402.98 213.762 1402.98 216.609 Q1402.98 218.878 1404.46 220.22 Q1405.96 221.54 1408.53 221.54 Q1412.08 221.54 1414.21 219.04 Q1416.36 216.517 1416.36 212.35 L1416.36 211.401 L1412.12 211.401 M1420.62 209.642 L1420.62 224.433 L1416.36 224.433 L1416.36 220.498 Q1414.9 222.859 1412.72 223.993 Q1410.55 225.104 1407.4 225.104 Q1403.42 225.104 1401.06 222.882 Q1398.72 220.637 1398.72 216.887 Q1398.72 212.512 1401.64 210.29 Q1404.58 208.068 1410.39 208.068 L1416.36 208.068 L1416.36 207.651 Q1416.36 204.711 1414.41 203.114 Q1412.49 201.493 1409 201.493 Q1406.77 201.493 1404.67 202.026 Q1402.56 202.558 1400.62 203.623 L1400.62 199.688 Q1402.96 198.785 1405.15 198.345 Q1407.35 197.882 1409.44 197.882 Q1415.06 197.882 1417.84 200.799 Q1420.62 203.716 1420.62 209.642 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1433.6 191.146 L1433.6 198.507 L1442.38 198.507 L1442.38 201.818 L1433.6 201.818 L1433.6 215.892 Q1433.6 219.063 1434.46 219.966 Q1435.34 220.868 1438 220.868 L1442.38 220.868 L1442.38 224.433 L1438 224.433 Q1433.07 224.433 1431.2 222.604 Q1429.32 220.753 1429.32 215.892 L1429.32 201.818 L1426.2 201.818 L1426.2 198.507 L1429.32 198.507 L1429.32 191.146 L1433.6 191.146 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1458.02 201.493 Q1454.6 201.493 1452.61 204.179 Q1450.62 206.841 1450.62 211.493 Q1450.62 216.146 1452.58 218.831 Q1454.58 221.493 1458.02 221.493 Q1461.43 221.493 1463.42 218.808 Q1465.41 216.123 1465.41 211.493 Q1465.41 206.887 1463.42 204.202 Q1461.43 201.493 1458.02 201.493 M1458.02 197.882 Q1463.58 197.882 1466.75 201.493 Q1469.92 205.105 1469.92 211.493 Q1469.92 217.859 1466.75 221.493 Q1463.58 225.104 1458.02 225.104 Q1452.45 225.104 1449.27 221.493 Q1446.13 217.859 1446.13 211.493 Q1446.13 205.105 1449.27 201.493 Q1452.45 197.882 1458.02 197.882 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1492.01 202.489 Q1491.29 202.072 1490.43 201.887 Q1489.6 201.679 1488.58 201.679 Q1484.97 201.679 1483.02 204.04 Q1481.1 206.378 1481.1 210.776 L1481.1 224.433 L1476.82 224.433 L1476.82 198.507 L1481.1 198.507 L1481.1 202.535 Q1482.45 200.174 1484.6 199.04 Q1486.75 197.882 1489.83 197.882 Q1490.27 197.882 1490.8 197.952 Q1491.33 197.998 1491.98 198.114 L1492.01 202.489 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1507.26 226.841 Q1505.45 231.47 1503.74 232.882 Q1502.03 234.294 1499.16 234.294 L1495.76 234.294 L1495.76 230.729 L1498.26 230.729 Q1500.02 230.729 1500.99 229.896 Q1501.96 229.063 1503.14 225.961 L1503.9 224.017 L1493.42 198.507 L1497.93 198.507 L1506.03 218.785 L1514.14 198.507 L1518.65 198.507 L1507.26 226.841 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1560.5 191.007 L1560.5 195.568 Q1557.84 194.294 1555.48 193.669 Q1553.12 193.044 1550.92 193.044 Q1547.1 193.044 1545.01 194.526 Q1542.95 196.007 1542.95 198.739 Q1542.95 201.031 1544.32 202.211 Q1545.71 203.368 1549.55 204.086 L1552.38 204.665 Q1557.61 205.66 1560.08 208.183 Q1562.58 210.683 1562.58 214.896 Q1562.58 219.919 1559.2 222.512 Q1555.85 225.104 1549.34 225.104 Q1546.89 225.104 1544.11 224.549 Q1541.36 223.993 1538.39 222.905 L1538.39 218.091 Q1541.24 219.688 1543.97 220.498 Q1546.7 221.308 1549.34 221.308 Q1553.35 221.308 1555.52 219.734 Q1557.7 218.16 1557.7 215.243 Q1557.7 212.697 1556.13 211.262 Q1554.57 209.827 1551.01 209.109 L1548.16 208.554 Q1542.93 207.512 1540.59 205.29 Q1538.26 203.068 1538.26 199.109 Q1538.26 194.526 1541.47 191.887 Q1544.71 189.248 1550.39 189.248 Q1552.82 189.248 1555.34 189.688 Q1557.86 190.128 1560.5 191.007 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1573.9 191.146 L1573.9 198.507 L1582.68 198.507 L1582.68 201.818 L1573.9 201.818 L1573.9 215.892 Q1573.9 219.063 1574.76 219.966 Q1575.64 220.868 1578.3 220.868 L1582.68 220.868 L1582.68 224.433 L1578.3 224.433 Q1573.37 224.433 1571.5 222.604 Q1569.62 220.753 1569.62 215.892 L1569.62 201.818 L1566.5 201.818 L1566.5 198.507 L1569.62 198.507 L1569.62 191.146 L1573.9 191.146 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1600.06 211.401 Q1594.9 211.401 1592.91 212.581 Q1590.92 213.762 1590.92 216.609 Q1590.92 218.878 1592.4 220.22 Q1593.9 221.54 1596.47 221.54 Q1600.01 221.54 1602.14 219.04 Q1604.3 216.517 1604.3 212.35 L1604.3 211.401 L1600.06 211.401 M1608.56 209.642 L1608.56 224.433 L1604.3 224.433 L1604.3 220.498 Q1602.84 222.859 1600.66 223.993 Q1598.49 225.104 1595.34 225.104 Q1591.36 225.104 1589 222.882 Q1586.66 220.637 1586.66 216.887 Q1586.66 212.512 1589.57 210.29 Q1592.51 208.068 1598.32 208.068 L1604.3 208.068 L1604.3 207.651 Q1604.3 204.711 1602.35 203.114 Q1600.43 201.493 1596.94 201.493 Q1594.71 201.493 1592.61 202.026 Q1590.5 202.558 1588.56 203.623 L1588.56 199.688 Q1590.89 198.785 1593.09 198.345 Q1595.29 197.882 1597.38 197.882 Q1603 197.882 1605.78 200.799 Q1608.56 203.716 1608.56 209.642 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1638.88 208.785 L1638.88 224.433 L1634.62 224.433 L1634.62 208.924 Q1634.62 205.243 1633.19 203.415 Q1631.75 201.586 1628.88 201.586 Q1625.43 201.586 1623.44 203.785 Q1621.45 205.984 1621.45 209.78 L1621.45 224.433 L1617.17 224.433 L1617.17 198.507 L1621.45 198.507 L1621.45 202.535 Q1622.98 200.197 1625.04 199.04 Q1627.12 197.882 1629.83 197.882 Q1634.3 197.882 1636.59 200.66 Q1638.88 203.415 1638.88 208.785 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1664.44 202.443 L1664.44 188.415 L1668.69 188.415 L1668.69 224.433 L1664.44 224.433 L1664.44 220.544 Q1663.09 222.859 1661.03 223.993 Q1659 225.104 1656.13 225.104 Q1651.43 225.104 1648.46 221.355 Q1645.52 217.605 1645.52 211.493 Q1645.52 205.382 1648.46 201.632 Q1651.43 197.882 1656.13 197.882 Q1659 197.882 1661.03 199.017 Q1663.09 200.128 1664.44 202.443 M1649.92 211.493 Q1649.92 216.192 1651.84 218.878 Q1653.79 221.54 1657.17 221.54 Q1660.55 221.54 1662.49 218.878 Q1664.44 216.192 1664.44 211.493 Q1664.44 206.794 1662.49 204.132 Q1660.55 201.447 1657.17 201.447 Q1653.79 201.447 1651.84 204.132 Q1649.92 206.794 1649.92 211.493 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1689.25 211.401 Q1684.09 211.401 1682.1 212.581 Q1680.11 213.762 1680.11 216.609 Q1680.11 218.878 1681.59 220.22 Q1683.09 221.54 1685.66 221.54 Q1689.2 221.54 1691.33 219.04 Q1693.49 216.517 1693.49 212.35 L1693.49 211.401 L1689.25 211.401 M1697.75 209.642 L1697.75 224.433 L1693.49 224.433 L1693.49 220.498 Q1692.03 222.859 1689.85 223.993 Q1687.68 225.104 1684.53 225.104 Q1680.55 225.104 1678.19 222.882 Q1675.85 220.637 1675.85 216.887 Q1675.85 212.512 1678.76 210.29 Q1681.7 208.068 1687.51 208.068 L1693.49 208.068 L1693.49 207.651 Q1693.49 204.711 1691.54 203.114 Q1689.62 201.493 1686.13 201.493 Q1683.9 201.493 1681.8 202.026 Q1679.69 202.558 1677.75 203.623 L1677.75 199.688 Q1680.08 198.785 1682.28 198.345 Q1684.48 197.882 1686.57 197.882 Q1692.19 197.882 1694.97 200.799 Q1697.75 203.716 1697.75 209.642 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1721.54 202.489 Q1720.82 202.072 1719.97 201.887 Q1719.13 201.679 1718.12 201.679 Q1714.5 201.679 1712.56 204.04 Q1710.64 206.378 1710.64 210.776 L1710.64 224.433 L1706.36 224.433 L1706.36 198.507 L1710.64 198.507 L1710.64 202.535 Q1711.98 200.174 1714.13 199.04 Q1716.29 197.882 1719.37 197.882 Q1719.81 197.882 1720.34 197.952 Q1720.87 197.998 1721.52 198.114 L1721.54 202.489 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip810)" d="M1742.24 202.443 L1742.24 188.415 L1746.5 188.415 L1746.5 224.433 L1742.24 224.433 L1742.24 220.544 Q1740.89 222.859 1738.83 223.993 Q1736.8 225.104 1733.93 225.104 Q1729.23 225.104 1726.26 221.355 Q1723.32 217.605 1723.32 211.493 Q1723.32 205.382 1726.26 201.632 Q1729.23 197.882 1733.93 197.882 Q1736.8 197.882 1738.83 199.017 Q1740.89 200.128 1742.24 202.443 M1727.72 211.493 Q1727.72 216.192 1729.64 218.878 Q1731.59 221.54 1734.97 221.54 Q1738.35 221.54 1740.29 218.878 Q1742.24 216.192 1742.24 211.493 Q1742.24 206.794 1740.29 204.132 Q1738.35 201.447 1734.97 201.447 Q1731.59 201.447 1729.64 204.132 Q1727.72 206.794 1727.72 211.493 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /></svg>

Figure 1: Simulated dissolved oxygen concentration for Problem 1 without
treatment.

1.  These lines of code separate the creation of the plot axis (with
    labels, legend positions, etc) using `p = plot(...)` from the
    plotting of the data with `plot!(p, ...)`. You can do this all in a
    single `plot()` call, but this may sometimes make things more
    readable when there are a lot of style arguments for the axes.

We can see that the DO concentration falls below the regulatory standard
of 4 mg/L before 20 km downstream. To find the minimum value, we can use
the `minimum()` function.

``` julia
@show minimum(C);
```

    minimum(C) = 3.8423213306662958

So the minimum value is 3.8 mg/L.

To determine the minimum treatment of the waste streams needed to
maintain compliance, we will write a function which will apply treatment
levels `eff1` and `eff2` and evaluate our function above, returning the
minimum DO concentration.

``` julia
# waste_treat: function to simulate DO with treated discharges; treatments apply to CBOD and NBOD released
# inputs:
#   - eff1: efficiency of treatment for waste stream 1 (as a decimal)
#   - eff2: efficiency of treatment for waste stream 2 (as a decimal)
#   - inflow: tuple with inflow properties: (Volume, DO, CBOD, NBOD)
#   - waste1: tuple with waste stream 1 properties: (Volume, DO, CBOD, NBOD)
#   - waste2: tuple with waste stream 2 properties: (Volume, DO, CBOD, NBOD)
#   - U: velocity of river (km/d)
#   - Cₛ = saturation oxygen concentration (mg/L)
#   - ka, kc, kn: reaeration, CBOD decay, and NBOD decay rates, respectively (d^{-1})
function waste_treat(eff1, eff2, inflow, waste1, waste2, U, Cₛ, ka, kb, kn)
    waste1_treated = (waste1[1], waste1[2], (1 - eff1) * waste1[3], (1 - eff1) * waste1[4])    
    waste2_treated = (waste2[1], waste2[2], (1 - eff2) * waste2[3], (1 - eff2) * waste2[4])    
    C, B, N = do_simulate(inflow, waste1_treated, waste2_treated, U, Cₛ, ka, kb, kn)
    return minimum(C)
end
```

    waste_treat (generic function with 1 method)

Now we can evaluate this function over a range of treatment
efficiencies. There are a number of ways to do this, but we’ll use a
trick which Julia makes easy: broadcasting using an anonymous function.

``` julia
# evaluate treatment efficiencies between 0 and 1
effs = 0:0.01:1
treat1 = (e1 -> waste_treat(e1, 0, inflow, waste1, waste2, U, Cₛ, ka, kb, kn)).(effs)
treat2 = (e2 -> waste_treat(0, e2, inflow, waste1, waste2, U, Cₛ, ka, kb, kn)).(effs)
```

    101-element Vector{Float64}:
     3.8423213306662958
     3.8631158474631597
     3.883910364260024
     3.9047048810568867
     3.925499397853751
     3.9460612716757266
     3.965197323117371
     3.984333374559016
     4.0034694260006605
     4.022605477442305
     ⋮
     4.711700098308851
     4.711700098308851
     4.711700098308851
     4.711700098308851
     4.711700098308851
     4.711700098308851
     4.711700098308851
     4.711700098308851
     4.711700098308851

To find the minimum treatment level which ensures compliance, we can now
find the position of the first value where the minimum value is at least
4 mg/L, and look up the associated efficiency. Julia provides the
`findfirst()` function which lets you find the index of the first value
satisfying a Boolean condition.

``` julia
# find indices and treatment values
idx1 = findfirst(treat1 .>= 4.0)
idx2 = findfirst(treat2 .>= 4.0)
@show effs[idx1];
@show effs[idx2];
```

    effs[idx1] = 0.11
    effs[idx2] = 0.08

So the minimum treatment level for waste stream 1 is 11% and the minimum
treatment level for waste stream 2 is 8%.

There is no “right” answer to the question of which treatment option you
would pick, so long as your solution is thoughtful and justified.

-   For example, one could argue that as waste stream 1 on its own does
    not result in a lack of compliance (which we can see from
    <a href="#fig-do-untreated" class="quarto-xref">Figure 1</a>, as the
    initial “sag” has started to recover prior to waste stream 2), waste
    stream 2 ought to be treated.
-   On the other hand, waste stream 1 has a much more negative impact on
    the inflow dissolved oxygen levels, and waste stream 2 might not
    cause a lack of compliance without that effect, which might suggest
    that waste stream 1 should be treated.
-   Another consideration might be the relative cost of treating each
    waste stream, which we have no information on, or whether these
    waste streams are from municipal, residual, or industrial sources
    (in other words, non-profit vs. profit).

### Problem 2 (20 points)

Loading the data:

``` julia
# Dataset from https://zenodo.org/record/3973015
# The CSV is read into a DataFrame object, and we specify that it is comma delimited
forcings_all = CSV.read("data/ERF_ssp585_1750-2500.csv", DataFrame, delim=",")

# Separate out the individual components
# Get total aerosol forcings
forcing_aerosol_rad = forcings_all[!,"aerosol-radiation_interactions"]
forcing_aerosol_cloud = forcings_all[!,"aerosol-cloud_interactions"]
forcing_aerosol = forcing_aerosol_rad + forcing_aerosol_cloud
# Calculate non-aerosol forcings from the total.
forcing_total = forcings_all[!,"total"]
forcing_non_aerosol = forcing_total - forcing_aerosol
t = Int64.(forcings_all[!,"year"])
idx_2100 = findfirst(t .== 2100) # find the index which corresponds to the year 2100
```

    351

The energy balance model is given by
$$cd \frac{dT}{dt} = F - \lambda T.$$ Discretizing with Forward Euler,
we get $$
\begin{aligned}
cd\frac{T(t + \Delta t) - T(t)}{\Delta t} &= F(t) - \lambda T(t) \\
T(t + \Delta t) &= T(t) + \frac{\Delta t}{cd} \left(F(t) - \lambda T(t)\right).
\end{aligned}
$$

In Julia code (and with the assumed parameter values), this becomes the
following function (replacing
$F(t) = F_\text{nonaerosol} + \alpha F_\text{aerosol}$):

``` julia
# ebm: function to simulate the energy balance model
# inputs:
#   - t: vector of time values.
#   - F_nonaerosol: vector of non-aerosol radiative forcing values
#   - F_aerosol: vector of aerosol radiative forcing values
#   - c, d, λ, α, Δt: model parameters, see below.
function ebm(t, F_nonaerosol, F_aerosol, c, d, λ, α, Δt)
    T = zeros(length(t))
    F = F_nonaerosol + α * F_aerosol
    for s in 1:length(t)-1
        T[s+1] = T[s] + Δt * (F[s] - λ * T[s]) / (c * d)
    end
    return T
end

c = 4.184e6 # specific heat of water, J/K/m^2
d = 86 # ocean mixing depth, m
λ = 2.1 # climate feedback factor
α = 0.8 # aerosol scaling factor
Δt = 31_558_152 # annual time step, s

temps = ebm(t, forcing_non_aerosol, forcing_aerosol, c, d, λ, α, Δt)
```

    751-element Vector{Float64}:
     0.0
     0.022731483349329427
     0.040278289494741326
     0.05240826578450054
     0.05895069636157351
     0.06063584224928685
     0.058278631700672184
     0.036311110702349894
     0.03060499874837628
     0.03778068117354878
     ⋮
     6.0349064449781515
     6.033928949865114
     6.032952263810847
     6.031978595303466
     6.0310066214578395
     6.030034177683138
     6.029061472805756
     6.028090703835366
     6.027123866872981

1.  The `zeros` function initializes a vector with zero values of the
    given length. This is a good way to initialize storage when you
    aren’t worried about distinguishing non-set values from “true”
    zeroes.
2.  While normally `eachindex(t)` is preferred instead of `1:length(t)`,
    in this case we need to iterate only until the second-to-last index
    of `t` as we set the next value `T[s+1]` in the loop.

The simulated temperatures are plotted in
<a href="#fig-temps" class="quarto-xref">Figure 2</a>:

``` julia
p = plot(; xlabel="Year", ylabel="Global Mean Temperature Anomaly (°C)", legend=false)
plot!(p, t, temps)
xlims!(p, (1750, 2100))
```

<?xml version="1.0" encoding="utf-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="672" height="480" viewBox="0 0 2688 1920">
<defs>
  <clipPath id="clip870">
    <rect x="0" y="0" width="2688" height="1920"/>
  </clipPath>
</defs>
<path clip-path="url(#clip870)" d="M0 1920 L2688 1920 L2688 -4.26326e-14 L0 -4.26326e-14  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip871">
    <rect x="537" y="0" width="1883" height="1883"/>
  </clipPath>
</defs>
<path clip-path="url(#clip870)" d="M181.719 1734.12 L2640.76 1734.12 L2640.76 47.2441 L181.719 47.2441  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip872">
    <rect x="181" y="47" width="2460" height="1688"/>
  </clipPath>
</defs>
<polyline clip-path="url(#clip872)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,1734.12 181.719,47.2441 "/>
<polyline clip-path="url(#clip872)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="533.01,1734.12 533.01,47.2441 "/>
<polyline clip-path="url(#clip872)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="884.301,1734.12 884.301,47.2441 "/>
<polyline clip-path="url(#clip872)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1235.59,1734.12 1235.59,47.2441 "/>
<polyline clip-path="url(#clip872)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1586.88,1734.12 1586.88,47.2441 "/>
<polyline clip-path="url(#clip872)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1938.17,1734.12 1938.17,47.2441 "/>
<polyline clip-path="url(#clip872)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="2289.46,1734.12 2289.46,47.2441 "/>
<polyline clip-path="url(#clip872)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="2640.76,1734.12 2640.76,47.2441 "/>
<polyline clip-path="url(#clip870)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,1734.12 2640.76,1734.12 "/>
<polyline clip-path="url(#clip870)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,1734.12 181.719,1715.22 "/>
<polyline clip-path="url(#clip870)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="533.01,1734.12 533.01,1715.22 "/>
<polyline clip-path="url(#clip870)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="884.301,1734.12 884.301,1715.22 "/>
<polyline clip-path="url(#clip870)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1235.59,1734.12 1235.59,1715.22 "/>
<polyline clip-path="url(#clip870)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1586.88,1734.12 1586.88,1715.22 "/>
<polyline clip-path="url(#clip870)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1938.17,1734.12 1938.17,1715.22 "/>
<polyline clip-path="url(#clip870)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2289.46,1734.12 2289.46,1715.22 "/>
<polyline clip-path="url(#clip870)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2640.76,1734.12 2640.76,1715.22 "/>
<path clip-path="url(#clip870)" d="M126.245 1794.89 L133.884 1794.89 L133.884 1768.52 L125.574 1770.19 L125.574 1765.93 L133.838 1764.26 L138.514 1764.26 L138.514 1794.89 L146.153 1794.89 L146.153 1798.82 L126.245 1798.82 L126.245 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M154.416 1764.26 L176.639 1764.26 L176.639 1766.25 L164.092 1798.82 L159.208 1798.82 L171.014 1768.2 L154.416 1768.2 L154.416 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M185.805 1764.26 L204.162 1764.26 L204.162 1768.2 L190.088 1768.2 L190.088 1776.67 Q191.106 1776.32 192.125 1776.16 Q193.143 1775.97 194.162 1775.97 Q199.949 1775.97 203.328 1779.15 Q206.708 1782.32 206.708 1787.73 Q206.708 1793.31 203.236 1796.41 Q199.763 1799.49 193.444 1799.49 Q191.268 1799.49 189 1799.12 Q186.754 1798.75 184.347 1798.01 L184.347 1793.31 Q186.43 1794.45 188.652 1795 Q190.875 1795.56 193.351 1795.56 Q197.356 1795.56 199.694 1793.45 Q202.032 1791.34 202.032 1787.73 Q202.032 1784.12 199.694 1782.02 Q197.356 1779.91 193.351 1779.91 Q191.476 1779.91 189.601 1780.33 Q187.75 1780.74 185.805 1781.62 L185.805 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M225.921 1767.34 Q222.31 1767.34 220.481 1770.9 Q218.675 1774.45 218.675 1781.58 Q218.675 1788.68 220.481 1792.25 Q222.31 1795.79 225.921 1795.79 Q229.555 1795.79 231.36 1792.25 Q233.189 1788.68 233.189 1781.58 Q233.189 1774.45 231.36 1770.9 Q229.555 1767.34 225.921 1767.34 M225.921 1763.64 Q231.731 1763.64 234.786 1768.24 Q237.865 1772.83 237.865 1781.58 Q237.865 1790.3 234.786 1794.91 Q231.731 1799.49 225.921 1799.49 Q220.11 1799.49 217.032 1794.91 Q213.976 1790.3 213.976 1781.58 Q213.976 1772.83 217.032 1768.24 Q220.11 1763.64 225.921 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M477.536 1794.89 L485.175 1794.89 L485.175 1768.52 L476.865 1770.19 L476.865 1765.93 L485.129 1764.26 L489.805 1764.26 L489.805 1794.89 L497.443 1794.89 L497.443 1798.82 L477.536 1798.82 L477.536 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M516.888 1782.41 Q513.554 1782.41 511.633 1784.19 Q509.735 1785.97 509.735 1789.1 Q509.735 1792.22 511.633 1794.01 Q513.554 1795.79 516.888 1795.79 Q520.221 1795.79 522.142 1794.01 Q524.064 1792.2 524.064 1789.1 Q524.064 1785.97 522.142 1784.19 Q520.244 1782.41 516.888 1782.41 M512.212 1780.42 Q509.203 1779.68 507.513 1777.62 Q505.846 1775.56 505.846 1772.59 Q505.846 1768.45 508.786 1766.04 Q511.749 1763.64 516.888 1763.64 Q522.05 1763.64 524.99 1766.04 Q527.929 1768.45 527.929 1772.59 Q527.929 1775.56 526.24 1777.62 Q524.573 1779.68 521.587 1780.42 Q524.966 1781.21 526.841 1783.5 Q528.74 1785.79 528.74 1789.1 Q528.74 1794.12 525.661 1796.81 Q522.605 1799.49 516.888 1799.49 Q511.17 1799.49 508.092 1796.81 Q505.036 1794.12 505.036 1789.1 Q505.036 1785.79 506.934 1783.5 Q508.832 1781.21 512.212 1780.42 M510.499 1773.03 Q510.499 1775.72 512.166 1777.22 Q513.855 1778.73 516.888 1778.73 Q519.897 1778.73 521.587 1777.22 Q523.3 1775.72 523.3 1773.03 Q523.3 1770.35 521.587 1768.84 Q519.897 1767.34 516.888 1767.34 Q513.855 1767.34 512.166 1768.84 Q510.499 1770.35 510.499 1773.03 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M547.05 1767.34 Q543.439 1767.34 541.61 1770.9 Q539.804 1774.45 539.804 1781.58 Q539.804 1788.68 541.61 1792.25 Q543.439 1795.79 547.05 1795.79 Q550.684 1795.79 552.489 1792.25 Q554.318 1788.68 554.318 1781.58 Q554.318 1774.45 552.489 1770.9 Q550.684 1767.34 547.05 1767.34 M547.05 1763.64 Q552.86 1763.64 555.915 1768.24 Q558.994 1772.83 558.994 1781.58 Q558.994 1790.3 555.915 1794.91 Q552.86 1799.49 547.05 1799.49 Q541.24 1799.49 538.161 1794.91 Q535.105 1790.3 535.105 1781.58 Q535.105 1772.83 538.161 1768.24 Q541.24 1763.64 547.05 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M577.212 1767.34 Q573.6 1767.34 571.772 1770.9 Q569.966 1774.45 569.966 1781.58 Q569.966 1788.68 571.772 1792.25 Q573.6 1795.79 577.212 1795.79 Q580.846 1795.79 582.651 1792.25 Q584.48 1788.68 584.48 1781.58 Q584.48 1774.45 582.651 1770.9 Q580.846 1767.34 577.212 1767.34 M577.212 1763.64 Q583.022 1763.64 586.077 1768.24 Q589.156 1772.83 589.156 1781.58 Q589.156 1790.3 586.077 1794.91 Q583.022 1799.49 577.212 1799.49 Q571.401 1799.49 568.323 1794.91 Q565.267 1790.3 565.267 1781.58 Q565.267 1772.83 568.323 1768.24 Q571.401 1763.64 577.212 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M828.827 1794.89 L836.466 1794.89 L836.466 1768.52 L828.156 1770.19 L828.156 1765.93 L836.42 1764.26 L841.096 1764.26 L841.096 1794.89 L848.734 1794.89 L848.734 1798.82 L828.827 1798.82 L828.827 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M868.179 1782.41 Q864.845 1782.41 862.924 1784.19 Q861.026 1785.97 861.026 1789.1 Q861.026 1792.22 862.924 1794.01 Q864.845 1795.79 868.179 1795.79 Q871.512 1795.79 873.433 1794.01 Q875.355 1792.2 875.355 1789.1 Q875.355 1785.97 873.433 1784.19 Q871.535 1782.41 868.179 1782.41 M863.503 1780.42 Q860.494 1779.68 858.804 1777.62 Q857.137 1775.56 857.137 1772.59 Q857.137 1768.45 860.077 1766.04 Q863.04 1763.64 868.179 1763.64 Q873.341 1763.64 876.281 1766.04 Q879.22 1768.45 879.22 1772.59 Q879.22 1775.56 877.531 1777.62 Q875.864 1779.68 872.878 1780.42 Q876.257 1781.21 878.132 1783.5 Q880.031 1785.79 880.031 1789.1 Q880.031 1794.12 876.952 1796.81 Q873.896 1799.49 868.179 1799.49 Q862.461 1799.49 859.382 1796.81 Q856.327 1794.12 856.327 1789.1 Q856.327 1785.79 858.225 1783.5 Q860.123 1781.21 863.503 1780.42 M861.79 1773.03 Q861.79 1775.72 863.457 1777.22 Q865.146 1778.73 868.179 1778.73 Q871.188 1778.73 872.878 1777.22 Q874.591 1775.72 874.591 1773.03 Q874.591 1770.35 872.878 1768.84 Q871.188 1767.34 868.179 1767.34 Q865.146 1767.34 863.457 1768.84 Q861.79 1770.35 861.79 1773.03 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M888.387 1764.26 L906.743 1764.26 L906.743 1768.2 L892.669 1768.2 L892.669 1776.67 Q893.688 1776.32 894.706 1776.16 Q895.725 1775.97 896.743 1775.97 Q902.53 1775.97 905.91 1779.15 Q909.29 1782.32 909.29 1787.73 Q909.29 1793.31 905.817 1796.41 Q902.345 1799.49 896.026 1799.49 Q893.85 1799.49 891.581 1799.12 Q889.336 1798.75 886.929 1798.01 L886.929 1793.31 Q889.012 1794.45 891.234 1795 Q893.456 1795.56 895.933 1795.56 Q899.938 1795.56 902.276 1793.45 Q904.614 1791.34 904.614 1787.73 Q904.614 1784.12 902.276 1782.02 Q899.938 1779.91 895.933 1779.91 Q894.058 1779.91 892.183 1780.33 Q890.331 1780.74 888.387 1781.62 L888.387 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M928.502 1767.34 Q924.891 1767.34 923.063 1770.9 Q921.257 1774.45 921.257 1781.58 Q921.257 1788.68 923.063 1792.25 Q924.891 1795.79 928.502 1795.79 Q932.137 1795.79 933.942 1792.25 Q935.771 1788.68 935.771 1781.58 Q935.771 1774.45 933.942 1770.9 Q932.137 1767.34 928.502 1767.34 M928.502 1763.64 Q934.313 1763.64 937.368 1768.24 Q940.447 1772.83 940.447 1781.58 Q940.447 1790.3 937.368 1794.91 Q934.313 1799.49 928.502 1799.49 Q922.692 1799.49 919.614 1794.91 Q916.558 1790.3 916.558 1781.58 Q916.558 1772.83 919.614 1768.24 Q922.692 1763.64 928.502 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M1180.12 1794.89 L1187.76 1794.89 L1187.76 1768.52 L1179.45 1770.19 L1179.45 1765.93 L1187.71 1764.26 L1192.39 1764.26 L1192.39 1794.89 L1200.03 1794.89 L1200.03 1798.82 L1180.12 1798.82 L1180.12 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M1209.61 1798.1 L1209.61 1793.84 Q1211.37 1794.68 1213.17 1795.12 Q1214.98 1795.56 1216.72 1795.56 Q1221.34 1795.56 1223.78 1792.46 Q1226.23 1789.33 1226.58 1782.99 Q1225.23 1784.98 1223.17 1786.04 Q1221.11 1787.11 1218.61 1787.11 Q1213.43 1787.11 1210.4 1783.98 Q1207.39 1780.83 1207.39 1775.4 Q1207.39 1770.07 1210.53 1766.85 Q1213.68 1763.64 1218.91 1763.64 Q1224.91 1763.64 1228.06 1768.24 Q1231.23 1772.83 1231.23 1781.58 Q1231.23 1789.75 1227.34 1794.63 Q1223.47 1799.49 1216.92 1799.49 Q1215.16 1799.49 1213.36 1799.15 Q1211.55 1798.8 1209.61 1798.1 M1218.91 1783.45 Q1222.06 1783.45 1223.89 1781.3 Q1225.74 1779.15 1225.74 1775.4 Q1225.74 1771.67 1223.89 1769.52 Q1222.06 1767.34 1218.91 1767.34 Q1215.77 1767.34 1213.91 1769.52 Q1212.09 1771.67 1212.09 1775.4 Q1212.09 1779.15 1213.91 1781.3 Q1215.77 1783.45 1218.91 1783.45 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M1249.63 1767.34 Q1246.02 1767.34 1244.19 1770.9 Q1242.39 1774.45 1242.39 1781.58 Q1242.39 1788.68 1244.19 1792.25 Q1246.02 1795.79 1249.63 1795.79 Q1253.27 1795.79 1255.07 1792.25 Q1256.9 1788.68 1256.9 1781.58 Q1256.9 1774.45 1255.07 1770.9 Q1253.27 1767.34 1249.63 1767.34 M1249.63 1763.64 Q1255.44 1763.64 1258.5 1768.24 Q1261.58 1772.83 1261.58 1781.58 Q1261.58 1790.3 1258.5 1794.91 Q1255.44 1799.49 1249.63 1799.49 Q1243.82 1799.49 1240.74 1794.91 Q1237.69 1790.3 1237.69 1781.58 Q1237.69 1772.83 1240.74 1768.24 Q1243.82 1763.64 1249.63 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M1279.79 1767.34 Q1276.18 1767.34 1274.35 1770.9 Q1272.55 1774.45 1272.55 1781.58 Q1272.55 1788.68 1274.35 1792.25 Q1276.18 1795.79 1279.79 1795.79 Q1283.43 1795.79 1285.23 1792.25 Q1287.06 1788.68 1287.06 1781.58 Q1287.06 1774.45 1285.23 1770.9 Q1283.43 1767.34 1279.79 1767.34 M1279.79 1763.64 Q1285.6 1763.64 1288.66 1768.24 Q1291.74 1772.83 1291.74 1781.58 Q1291.74 1790.3 1288.66 1794.91 Q1285.6 1799.49 1279.79 1799.49 Q1273.98 1799.49 1270.9 1794.91 Q1267.85 1790.3 1267.85 1781.58 Q1267.85 1772.83 1270.9 1768.24 Q1273.98 1763.64 1279.79 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M1531.41 1794.89 L1539.05 1794.89 L1539.05 1768.52 L1530.74 1770.19 L1530.74 1765.93 L1539 1764.26 L1543.68 1764.26 L1543.68 1794.89 L1551.32 1794.89 L1551.32 1798.82 L1531.41 1798.82 L1531.41 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M1560.9 1798.1 L1560.9 1793.84 Q1562.66 1794.68 1564.46 1795.12 Q1566.27 1795.56 1568.01 1795.56 Q1572.64 1795.56 1575.07 1792.46 Q1577.52 1789.33 1577.87 1782.99 Q1576.52 1784.98 1574.46 1786.04 Q1572.4 1787.11 1569.9 1787.11 Q1564.72 1787.11 1561.69 1783.98 Q1558.68 1780.83 1558.68 1775.4 Q1558.68 1770.07 1561.83 1766.85 Q1564.97 1763.64 1570.21 1763.64 Q1576.2 1763.64 1579.35 1768.24 Q1582.52 1772.83 1582.52 1781.58 Q1582.52 1789.75 1578.63 1794.63 Q1574.77 1799.49 1568.21 1799.49 Q1566.46 1799.49 1564.65 1799.15 Q1562.84 1798.8 1560.9 1798.1 M1570.21 1783.45 Q1573.35 1783.45 1575.18 1781.3 Q1577.03 1779.15 1577.03 1775.4 Q1577.03 1771.67 1575.18 1769.52 Q1573.35 1767.34 1570.21 1767.34 Q1567.06 1767.34 1565.21 1769.52 Q1563.38 1771.67 1563.38 1775.4 Q1563.38 1779.15 1565.21 1781.3 Q1567.06 1783.45 1570.21 1783.45 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M1590.97 1764.26 L1609.33 1764.26 L1609.33 1768.2 L1595.25 1768.2 L1595.25 1776.67 Q1596.27 1776.32 1597.29 1776.16 Q1598.31 1775.97 1599.33 1775.97 Q1605.11 1775.97 1608.49 1779.15 Q1611.87 1782.32 1611.87 1787.73 Q1611.87 1793.31 1608.4 1796.41 Q1604.93 1799.49 1598.61 1799.49 Q1596.43 1799.49 1594.16 1799.12 Q1591.92 1798.75 1589.51 1798.01 L1589.51 1793.31 Q1591.59 1794.45 1593.82 1795 Q1596.04 1795.56 1598.52 1795.56 Q1602.52 1795.56 1604.86 1793.45 Q1607.2 1791.34 1607.2 1787.73 Q1607.2 1784.12 1604.86 1782.02 Q1602.52 1779.91 1598.52 1779.91 Q1596.64 1779.91 1594.77 1780.33 Q1592.91 1780.74 1590.97 1781.62 L1590.97 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M1631.08 1767.34 Q1627.47 1767.34 1625.64 1770.9 Q1623.84 1774.45 1623.84 1781.58 Q1623.84 1788.68 1625.64 1792.25 Q1627.47 1795.79 1631.08 1795.79 Q1634.72 1795.79 1636.52 1792.25 Q1638.35 1788.68 1638.35 1781.58 Q1638.35 1774.45 1636.52 1770.9 Q1634.72 1767.34 1631.08 1767.34 M1631.08 1763.64 Q1636.89 1763.64 1639.95 1768.24 Q1643.03 1772.83 1643.03 1781.58 Q1643.03 1790.3 1639.95 1794.91 Q1636.89 1799.49 1631.08 1799.49 Q1625.27 1799.49 1622.2 1794.91 Q1619.14 1790.3 1619.14 1781.58 Q1619.14 1772.83 1622.2 1768.24 Q1625.27 1763.64 1631.08 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M1886.79 1794.89 L1903.1 1794.89 L1903.1 1798.82 L1881.16 1798.82 L1881.16 1794.89 Q1883.82 1792.13 1888.41 1787.5 Q1893.01 1782.85 1894.19 1781.51 Q1896.44 1778.98 1897.32 1777.25 Q1898.22 1775.49 1898.22 1773.8 Q1898.22 1771.04 1896.28 1769.31 Q1894.35 1767.57 1891.25 1767.57 Q1889.05 1767.57 1886.6 1768.34 Q1884.17 1769.1 1881.39 1770.65 L1881.39 1765.93 Q1884.22 1764.79 1886.67 1764.21 Q1889.12 1763.64 1891.16 1763.64 Q1896.53 1763.64 1899.73 1766.32 Q1902.92 1769.01 1902.92 1773.5 Q1902.92 1775.63 1902.11 1777.55 Q1901.32 1779.45 1899.22 1782.04 Q1898.64 1782.71 1895.54 1785.93 Q1892.43 1789.12 1886.79 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M1922.92 1767.34 Q1919.31 1767.34 1917.48 1770.9 Q1915.67 1774.45 1915.67 1781.58 Q1915.67 1788.68 1917.48 1792.25 Q1919.31 1795.79 1922.92 1795.79 Q1926.55 1795.79 1928.36 1792.25 Q1930.19 1788.68 1930.19 1781.58 Q1930.19 1774.45 1928.36 1770.9 Q1926.55 1767.34 1922.92 1767.34 M1922.92 1763.64 Q1928.73 1763.64 1931.79 1768.24 Q1934.86 1772.83 1934.86 1781.58 Q1934.86 1790.3 1931.79 1794.91 Q1928.73 1799.49 1922.92 1799.49 Q1917.11 1799.49 1914.03 1794.91 Q1910.98 1790.3 1910.98 1781.58 Q1910.98 1772.83 1914.03 1768.24 Q1917.11 1763.64 1922.92 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M1953.08 1767.34 Q1949.47 1767.34 1947.64 1770.9 Q1945.84 1774.45 1945.84 1781.58 Q1945.84 1788.68 1947.64 1792.25 Q1949.47 1795.79 1953.08 1795.79 Q1956.72 1795.79 1958.52 1792.25 Q1960.35 1788.68 1960.35 1781.58 Q1960.35 1774.45 1958.52 1770.9 Q1956.72 1767.34 1953.08 1767.34 M1953.08 1763.64 Q1958.89 1763.64 1961.95 1768.24 Q1965.03 1772.83 1965.03 1781.58 Q1965.03 1790.3 1961.95 1794.91 Q1958.89 1799.49 1953.08 1799.49 Q1947.27 1799.49 1944.19 1794.91 Q1941.14 1790.3 1941.14 1781.58 Q1941.14 1772.83 1944.19 1768.24 Q1947.27 1763.64 1953.08 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M1983.24 1767.34 Q1979.63 1767.34 1977.8 1770.9 Q1976 1774.45 1976 1781.58 Q1976 1788.68 1977.8 1792.25 Q1979.63 1795.79 1983.24 1795.79 Q1986.88 1795.79 1988.68 1792.25 Q1990.51 1788.68 1990.51 1781.58 Q1990.51 1774.45 1988.68 1770.9 Q1986.88 1767.34 1983.24 1767.34 M1983.24 1763.64 Q1989.05 1763.64 1992.11 1768.24 Q1995.19 1772.83 1995.19 1781.58 Q1995.19 1790.3 1992.11 1794.91 Q1989.05 1799.49 1983.24 1799.49 Q1977.43 1799.49 1974.35 1794.91 Q1971.3 1790.3 1971.3 1781.58 Q1971.3 1772.83 1974.35 1768.24 Q1977.43 1763.64 1983.24 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M2238.08 1794.89 L2254.4 1794.89 L2254.4 1798.82 L2232.45 1798.82 L2232.45 1794.89 Q2235.11 1792.13 2239.7 1787.5 Q2244.3 1782.85 2245.48 1781.51 Q2247.73 1778.98 2248.61 1777.25 Q2249.51 1775.49 2249.51 1773.8 Q2249.51 1771.04 2247.57 1769.31 Q2245.65 1767.57 2242.54 1767.57 Q2240.34 1767.57 2237.89 1768.34 Q2235.46 1769.1 2232.68 1770.65 L2232.68 1765.93 Q2235.51 1764.79 2237.96 1764.21 Q2240.41 1763.64 2242.45 1763.64 Q2247.82 1763.64 2251.02 1766.32 Q2254.21 1769.01 2254.21 1773.5 Q2254.21 1775.63 2253.4 1777.55 Q2252.61 1779.45 2250.51 1782.04 Q2249.93 1782.71 2246.83 1785.93 Q2243.72 1789.12 2238.08 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M2274.21 1767.34 Q2270.6 1767.34 2268.77 1770.9 Q2266.97 1774.45 2266.97 1781.58 Q2266.97 1788.68 2268.77 1792.25 Q2270.6 1795.79 2274.21 1795.79 Q2277.84 1795.79 2279.65 1792.25 Q2281.48 1788.68 2281.48 1781.58 Q2281.48 1774.45 2279.65 1770.9 Q2277.84 1767.34 2274.21 1767.34 M2274.21 1763.64 Q2280.02 1763.64 2283.08 1768.24 Q2286.15 1772.83 2286.15 1781.58 Q2286.15 1790.3 2283.08 1794.91 Q2280.02 1799.49 2274.21 1799.49 Q2268.4 1799.49 2265.32 1794.91 Q2262.27 1790.3 2262.27 1781.58 Q2262.27 1772.83 2265.32 1768.24 Q2268.4 1763.64 2274.21 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M2294.42 1764.26 L2312.78 1764.26 L2312.78 1768.2 L2298.7 1768.2 L2298.7 1776.67 Q2299.72 1776.32 2300.74 1776.16 Q2301.76 1775.97 2302.78 1775.97 Q2308.56 1775.97 2311.94 1779.15 Q2315.32 1782.32 2315.32 1787.73 Q2315.32 1793.31 2311.85 1796.41 Q2308.38 1799.49 2302.06 1799.49 Q2299.88 1799.49 2297.61 1799.12 Q2295.37 1798.75 2292.96 1798.01 L2292.96 1793.31 Q2295.04 1794.45 2297.27 1795 Q2299.49 1795.56 2301.96 1795.56 Q2305.97 1795.56 2308.31 1793.45 Q2310.65 1791.34 2310.65 1787.73 Q2310.65 1784.12 2308.31 1782.02 Q2305.97 1779.91 2301.96 1779.91 Q2300.09 1779.91 2298.21 1780.33 Q2296.36 1780.74 2294.42 1781.62 L2294.42 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M2334.53 1767.34 Q2330.92 1767.34 2329.09 1770.9 Q2327.29 1774.45 2327.29 1781.58 Q2327.29 1788.68 2329.09 1792.25 Q2330.92 1795.79 2334.53 1795.79 Q2338.17 1795.79 2339.97 1792.25 Q2341.8 1788.68 2341.8 1781.58 Q2341.8 1774.45 2339.97 1770.9 Q2338.17 1767.34 2334.53 1767.34 M2334.53 1763.64 Q2340.34 1763.64 2343.4 1768.24 Q2346.48 1772.83 2346.48 1781.58 Q2346.48 1790.3 2343.4 1794.91 Q2340.34 1799.49 2334.53 1799.49 Q2328.72 1799.49 2325.65 1794.91 Q2322.59 1790.3 2322.59 1781.58 Q2322.59 1772.83 2325.65 1768.24 Q2328.72 1763.64 2334.53 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M2589.37 1794.89 L2605.69 1794.89 L2605.69 1798.82 L2583.74 1798.82 L2583.74 1794.89 Q2586.4 1792.13 2590.99 1787.5 Q2595.59 1782.85 2596.77 1781.51 Q2599.02 1778.98 2599.9 1777.25 Q2600.8 1775.49 2600.8 1773.8 Q2600.8 1771.04 2598.86 1769.31 Q2596.94 1767.57 2593.83 1767.57 Q2591.64 1767.57 2589.18 1768.34 Q2586.75 1769.1 2583.97 1770.65 L2583.97 1765.93 Q2586.8 1764.79 2589.25 1764.21 Q2591.71 1763.64 2593.74 1763.64 Q2599.11 1763.64 2602.31 1766.32 Q2605.5 1769.01 2605.5 1773.5 Q2605.5 1775.63 2604.69 1777.55 Q2603.9 1779.45 2601.8 1782.04 Q2601.22 1782.71 2598.12 1785.93 Q2595.02 1789.12 2589.37 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M2616.31 1794.89 L2623.95 1794.89 L2623.95 1768.52 L2615.64 1770.19 L2615.64 1765.93 L2623.9 1764.26 L2628.58 1764.26 L2628.58 1794.89 L2636.22 1794.89 L2636.22 1798.82 L2616.31 1798.82 L2616.31 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M2655.66 1767.34 Q2652.05 1767.34 2650.22 1770.9 Q2648.42 1774.45 2648.42 1781.58 Q2648.42 1788.68 2650.22 1792.25 Q2652.05 1795.79 2655.66 1795.79 Q2659.3 1795.79 2661.1 1792.25 Q2662.93 1788.68 2662.93 1781.58 Q2662.93 1774.45 2661.1 1770.9 Q2659.3 1767.34 2655.66 1767.34 M2655.66 1763.64 Q2661.47 1763.64 2664.53 1768.24 Q2667.61 1772.83 2667.61 1781.58 Q2667.61 1790.3 2664.53 1794.91 Q2661.47 1799.49 2655.66 1799.49 Q2649.85 1799.49 2646.77 1794.91 Q2643.72 1790.3 2643.72 1781.58 Q2643.72 1772.83 2646.77 1768.24 Q2649.85 1763.64 2655.66 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M2685.83 1767.34 Q2682.21 1767.34 2680.39 1770.9 Q2678.58 1774.45 2678.58 1781.58 Q2678.58 1788.68 2680.39 1792.25 Q2682.21 1795.79 2685.83 1795.79 Q2689.46 1795.79 2691.26 1792.25 Q2693.09 1788.68 2693.09 1781.58 Q2693.09 1774.45 2691.26 1770.9 Q2689.46 1767.34 2685.83 1767.34 M2685.83 1763.64 Q2691.64 1763.64 2694.69 1768.24 Q2697.77 1772.83 2697.77 1781.58 Q2697.77 1790.3 2694.69 1794.91 Q2691.64 1799.49 2685.83 1799.49 Q2680.01 1799.49 2676.94 1794.91 Q2673.88 1790.3 2673.88 1781.58 Q2673.88 1772.83 2676.94 1768.24 Q2680.01 1763.64 2685.83 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M1342.17 1835.78 L1349.08 1835.78 L1362.25 1855.32 L1375.34 1835.78 L1382.24 1835.78 L1365.44 1860.67 L1365.44 1883.3 L1358.98 1883.3 L1358.98 1860.67 L1342.17 1835.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M1410.09 1864.01 L1410.09 1866.88 L1383.16 1866.88 Q1383.55 1872.92 1386.79 1876.11 Q1390.07 1879.26 1395.9 1879.26 Q1399.27 1879.26 1402.42 1878.43 Q1405.6 1877.6 1408.72 1875.95 L1408.72 1881.49 Q1405.57 1882.82 1402.26 1883.52 Q1398.95 1884.22 1395.55 1884.22 Q1387.02 1884.22 1382.02 1879.26 Q1377.05 1874.29 1377.05 1865.83 Q1377.05 1857.07 1381.76 1851.95 Q1386.51 1846.79 1394.53 1846.79 Q1401.72 1846.79 1405.89 1851.44 Q1410.09 1856.06 1410.09 1864.01 M1404.24 1862.29 Q1404.17 1857.49 1401.53 1854.62 Q1398.92 1851.76 1394.59 1851.76 Q1389.69 1851.76 1386.73 1854.53 Q1383.8 1857.3 1383.36 1862.33 L1404.24 1862.29 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M1435.9 1865.38 Q1428.81 1865.38 1426.07 1867 Q1423.33 1868.63 1423.33 1872.54 Q1423.33 1875.66 1425.37 1877.51 Q1427.44 1879.32 1430.97 1879.32 Q1435.84 1879.32 1438.77 1875.88 Q1441.73 1872.42 1441.73 1866.69 L1441.73 1865.38 L1435.9 1865.38 M1447.59 1862.96 L1447.59 1883.3 L1441.73 1883.3 L1441.73 1877.89 Q1439.72 1881.14 1436.73 1882.7 Q1433.74 1884.22 1429.41 1884.22 Q1423.94 1884.22 1420.69 1881.17 Q1417.48 1878.08 1417.48 1872.92 Q1417.48 1866.91 1421.49 1863.85 Q1425.53 1860.8 1433.52 1860.8 L1441.73 1860.8 L1441.73 1860.23 Q1441.73 1856.18 1439.06 1853.99 Q1436.41 1851.76 1431.61 1851.76 Q1428.55 1851.76 1425.66 1852.49 Q1422.76 1853.22 1420.09 1854.69 L1420.09 1849.28 Q1423.3 1848.03 1426.32 1847.43 Q1429.35 1846.79 1432.21 1846.79 Q1439.95 1846.79 1443.77 1850.8 Q1447.59 1854.81 1447.59 1862.96 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M1480.31 1853.13 Q1479.32 1852.55 1478.14 1852.3 Q1477 1852.01 1475.59 1852.01 Q1470.63 1852.01 1467.96 1855.26 Q1465.31 1858.47 1465.31 1864.52 L1465.31 1883.3 L1459.43 1883.3 L1459.43 1847.65 L1465.31 1847.65 L1465.31 1853.19 Q1467.16 1849.94 1470.12 1848.39 Q1473.08 1846.79 1477.31 1846.79 Q1477.92 1846.79 1478.65 1846.89 Q1479.38 1846.95 1480.27 1847.11 L1480.31 1853.13 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip872)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,1525.16 2640.76,1525.16 "/>
<polyline clip-path="url(#clip872)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,1302.48 2640.76,1302.48 "/>
<polyline clip-path="url(#clip872)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,1079.8 2640.76,1079.8 "/>
<polyline clip-path="url(#clip872)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,857.114 2640.76,857.114 "/>
<polyline clip-path="url(#clip872)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,634.431 2640.76,634.431 "/>
<polyline clip-path="url(#clip872)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,411.748 2640.76,411.748 "/>
<polyline clip-path="url(#clip872)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="181.719,189.065 2640.76,189.065 "/>
<polyline clip-path="url(#clip870)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,1734.12 181.719,47.2441 "/>
<polyline clip-path="url(#clip870)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,1525.16 200.617,1525.16 "/>
<polyline clip-path="url(#clip870)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,1302.48 200.617,1302.48 "/>
<polyline clip-path="url(#clip870)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,1079.8 200.617,1079.8 "/>
<polyline clip-path="url(#clip870)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,857.114 200.617,857.114 "/>
<polyline clip-path="url(#clip870)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,634.431 200.617,634.431 "/>
<polyline clip-path="url(#clip870)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,411.748 200.617,411.748 "/>
<polyline clip-path="url(#clip870)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,189.065 200.617,189.065 "/>
<path clip-path="url(#clip870)" d="M129.455 1510.96 Q125.844 1510.96 124.015 1514.53 Q122.21 1518.07 122.21 1525.2 Q122.21 1532.3 124.015 1535.87 Q125.844 1539.41 129.455 1539.41 Q133.089 1539.41 134.895 1535.87 Q136.724 1532.3 136.724 1525.2 Q136.724 1518.07 134.895 1514.53 Q133.089 1510.96 129.455 1510.96 M129.455 1507.26 Q135.265 1507.26 138.321 1511.86 Q141.399 1516.45 141.399 1525.2 Q141.399 1533.92 138.321 1538.53 Q135.265 1543.11 129.455 1543.11 Q123.645 1543.11 120.566 1538.53 Q117.511 1533.92 117.511 1525.2 Q117.511 1516.45 120.566 1511.86 Q123.645 1507.26 129.455 1507.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M121.492 1315.82 L129.131 1315.82 L129.131 1289.46 L120.821 1291.13 L120.821 1286.87 L129.085 1285.2 L133.761 1285.2 L133.761 1315.82 L141.399 1315.82 L141.399 1319.76 L121.492 1319.76 L121.492 1315.82 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M125.08 1093.14 L141.399 1093.14 L141.399 1097.08 L119.455 1097.08 L119.455 1093.14 Q122.117 1090.39 126.7 1085.76 Q131.307 1081.1 132.488 1079.76 Q134.733 1077.24 135.612 1075.5 Q136.515 1073.74 136.515 1072.05 Q136.515 1069.3 134.571 1067.56 Q132.65 1065.83 129.548 1065.83 Q127.349 1065.83 124.895 1066.59 Q122.464 1067.36 119.687 1068.91 L119.687 1064.18 Q122.511 1063.05 124.964 1062.47 Q127.418 1061.89 129.455 1061.89 Q134.825 1061.89 138.02 1064.58 Q141.214 1067.26 141.214 1071.75 Q141.214 1073.88 140.404 1075.8 Q139.617 1077.7 137.511 1080.29 Q136.932 1080.97 133.83 1084.18 Q130.728 1087.38 125.08 1093.14 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M134.27 855.76 Q137.626 856.478 139.501 858.746 Q141.399 861.015 141.399 864.348 Q141.399 869.464 137.881 872.265 Q134.362 875.065 127.881 875.065 Q125.705 875.065 123.39 874.626 Q121.099 874.209 118.645 873.352 L118.645 868.839 Q120.589 869.973 122.904 870.552 Q125.219 871.13 127.742 871.13 Q132.14 871.13 134.432 869.394 Q136.747 867.658 136.747 864.348 Q136.747 861.292 134.594 859.579 Q132.464 857.843 128.645 857.843 L124.617 857.843 L124.617 854.001 L128.83 854.001 Q132.279 854.001 134.108 852.635 Q135.937 851.246 135.937 848.654 Q135.937 845.992 134.038 844.579 Q132.163 843.144 128.645 843.144 Q126.724 843.144 124.525 843.561 Q122.326 843.978 119.687 844.857 L119.687 840.691 Q122.349 839.95 124.663 839.58 Q127.001 839.209 129.062 839.209 Q134.386 839.209 137.487 841.64 Q140.589 844.047 140.589 848.167 Q140.589 851.038 138.946 853.029 Q137.302 854.996 134.27 855.76 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M131.816 621.225 L120.011 639.674 L131.816 639.674 L131.816 621.225 M130.589 617.151 L136.469 617.151 L136.469 639.674 L141.399 639.674 L141.399 643.563 L136.469 643.563 L136.469 651.711 L131.816 651.711 L131.816 643.563 L116.214 643.563 L116.214 639.049 L130.589 617.151 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M120.497 394.468 L138.853 394.468 L138.853 398.403 L124.779 398.403 L124.779 406.876 Q125.798 406.528 126.816 406.366 Q127.835 406.181 128.853 406.181 Q134.64 406.181 138.02 409.353 Q141.399 412.524 141.399 417.94 Q141.399 423.519 137.927 426.621 Q134.455 429.7 128.136 429.7 Q125.96 429.7 123.691 429.329 Q121.446 428.959 119.039 428.218 L119.039 423.519 Q121.122 424.653 123.344 425.209 Q125.566 425.764 128.043 425.764 Q132.048 425.764 134.386 423.658 Q136.724 421.552 136.724 417.94 Q136.724 414.329 134.386 412.223 Q132.048 410.116 128.043 410.116 Q126.168 410.116 124.293 410.533 Q122.441 410.95 120.497 411.829 L120.497 394.468 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M129.872 187.202 Q126.724 187.202 124.872 189.355 Q123.043 191.508 123.043 195.258 Q123.043 198.984 124.872 201.16 Q126.724 203.313 129.872 203.313 Q133.02 203.313 134.849 201.16 Q136.7 198.984 136.7 195.258 Q136.7 191.508 134.849 189.355 Q133.02 187.202 129.872 187.202 M139.154 172.549 L139.154 176.809 Q137.395 175.975 135.589 175.535 Q133.807 175.096 132.048 175.096 Q127.418 175.096 124.964 178.221 Q122.534 181.346 122.187 187.665 Q123.552 185.651 125.613 184.586 Q127.673 183.498 130.15 183.498 Q135.358 183.498 138.367 186.67 Q141.399 189.818 141.399 195.258 Q141.399 200.582 138.251 203.799 Q135.103 207.017 129.872 207.017 Q123.876 207.017 120.705 202.433 Q117.534 197.827 117.534 189.1 Q117.534 180.906 121.423 176.045 Q125.312 171.16 131.863 171.16 Q133.622 171.16 135.404 171.508 Q137.21 171.855 139.154 172.549 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M54.859 1499.02 L42.0957 1499.02 L42.0957 1509.52 L36.8122 1509.52 L36.8122 1492.65 L57.2143 1492.65 Q59.856 1496.38 61.2247 1500.87 Q62.5615 1505.35 62.5615 1510.45 Q62.5615 1521.59 56.0684 1527.89 Q49.5436 1534.16 37.9262 1534.16 Q26.2769 1534.16 19.7839 1527.89 Q13.2591 1521.59 13.2591 1510.45 Q13.2591 1505.8 14.4049 1501.63 Q15.5507 1497.43 17.7787 1493.89 L24.6219 1493.89 Q21.5981 1497.46 20.0704 1501.47 Q18.5426 1505.48 18.5426 1509.9 Q18.5426 1518.63 23.4124 1523.02 Q28.2821 1527.38 37.9262 1527.38 Q47.5384 1527.38 52.4082 1523.02 Q57.2779 1518.63 57.2779 1509.9 Q57.2779 1506.5 56.705 1503.83 Q56.1003 1501.15 54.859 1499.02 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M12.1132 1481.16 L12.1132 1475.31 L61.6384 1475.31 L61.6384 1481.16 L12.1132 1481.16 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M30.0964 1449.24 Q30.0964 1453.95 33.7885 1456.69 Q37.4488 1459.42 43.8463 1459.42 Q50.2438 1459.42 53.9359 1456.72 Q57.5962 1453.98 57.5962 1449.24 Q57.5962 1444.56 53.9041 1441.82 Q50.212 1439.09 43.8463 1439.09 Q37.5124 1439.09 33.8203 1441.82 Q30.0964 1444.56 30.0964 1449.24 M25.1311 1449.24 Q25.1311 1441.6 30.0964 1437.24 Q35.0616 1432.88 43.8463 1432.88 Q52.5991 1432.88 57.5962 1437.24 Q62.5615 1441.6 62.5615 1449.24 Q62.5615 1456.91 57.5962 1461.27 Q52.5991 1465.6 43.8463 1465.6 Q35.0616 1465.6 30.0964 1461.27 Q25.1311 1456.91 25.1311 1449.24 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M43.8463 1397.58 Q37.3851 1397.58 33.7248 1400.26 Q30.0327 1402.9 30.0327 1407.54 Q30.0327 1412.19 33.7248 1414.86 Q37.3851 1417.51 43.8463 1417.51 Q50.3075 1417.51 53.9996 1414.86 Q57.6599 1412.19 57.6599 1407.54 Q57.6599 1402.9 53.9996 1400.26 Q50.3075 1397.58 43.8463 1397.58 M31.4013 1417.51 Q28.2185 1415.66 26.6907 1412.86 Q25.1311 1410.03 25.1311 1406.11 Q25.1311 1399.62 30.2873 1395.58 Q35.4436 1391.5 43.8463 1391.5 Q52.249 1391.5 57.4052 1395.58 Q62.5615 1399.62 62.5615 1406.11 Q62.5615 1410.03 61.0337 1412.86 Q59.4741 1415.66 56.2912 1417.51 L61.6384 1417.51 L61.6384 1423.39 L12.1132 1423.39 L12.1132 1417.51 L31.4013 1417.51 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M43.719 1365.59 Q43.719 1372.69 45.3422 1375.43 Q46.9655 1378.17 50.8804 1378.17 Q53.9996 1378.17 55.8457 1376.13 Q57.6599 1374.06 57.6599 1370.53 Q57.6599 1365.66 54.2224 1362.73 Q50.7531 1359.77 45.0239 1359.77 L43.719 1359.77 L43.719 1365.59 M41.3 1353.91 L61.6384 1353.91 L61.6384 1359.77 L56.2276 1359.77 Q59.4741 1361.77 61.0337 1364.77 Q62.5615 1367.76 62.5615 1372.09 Q62.5615 1377.56 59.5059 1380.81 Q56.4186 1384.02 51.2623 1384.02 Q45.2467 1384.02 42.1912 1380.01 Q39.1357 1375.97 39.1357 1367.98 L39.1357 1359.77 L38.5628 1359.77 Q34.5205 1359.77 32.3244 1362.44 Q30.0964 1365.08 30.0964 1369.89 Q30.0964 1372.95 30.8284 1375.84 Q31.5605 1378.74 33.0246 1381.41 L27.6137 1381.41 Q26.3724 1378.2 25.7677 1375.17 Q25.1311 1372.15 25.1311 1369.29 Q25.1311 1361.55 29.1415 1357.73 Q33.1519 1353.91 41.3 1353.91 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M12.1132 1341.85 L12.1132 1335.99 L61.6384 1335.99 L61.6384 1341.85 L12.1132 1341.85 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M14.1184 1302.76 L14.1184 1293.18 L46.4562 1281.06 L14.1184 1268.87 L14.1184 1259.29 L61.6384 1259.29 L61.6384 1265.56 L19.9112 1265.56 L52.5037 1277.81 L52.5037 1284.27 L19.9112 1296.53 L61.6384 1296.53 L61.6384 1302.76 L14.1184 1302.76 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M42.3504 1216.29 L45.2149 1216.29 L45.2149 1243.21 Q51.2623 1242.83 54.4452 1239.58 Q57.5962 1236.31 57.5962 1230.48 Q57.5962 1227.11 56.7687 1223.96 Q55.9411 1220.77 54.2861 1217.66 L59.8242 1217.66 Q61.161 1220.81 61.8612 1224.12 Q62.5615 1227.43 62.5615 1230.83 Q62.5615 1239.36 57.5962 1244.36 Q52.631 1249.32 44.1646 1249.32 Q35.4117 1249.32 30.2873 1244.61 Q25.1311 1239.87 25.1311 1231.85 Q25.1311 1224.66 29.7781 1220.49 Q34.3932 1216.29 42.3504 1216.29 M40.6316 1222.14 Q35.8255 1222.21 32.9609 1224.85 Q30.0964 1227.46 30.0964 1231.79 Q30.0964 1236.69 32.8654 1239.65 Q35.6345 1242.58 40.6634 1243.02 L40.6316 1222.14 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M43.719 1190.47 Q43.719 1197.57 45.3422 1200.31 Q46.9655 1203.05 50.8804 1203.05 Q53.9996 1203.05 55.8457 1201.01 Q57.6599 1198.94 57.6599 1195.41 Q57.6599 1190.54 54.2224 1187.61 Q50.7531 1184.65 45.0239 1184.65 L43.719 1184.65 L43.719 1190.47 M41.3 1178.79 L61.6384 1178.79 L61.6384 1184.65 L56.2276 1184.65 Q59.4741 1186.65 61.0337 1189.65 Q62.5615 1192.64 62.5615 1196.97 Q62.5615 1202.44 59.5059 1205.69 Q56.4186 1208.9 51.2623 1208.9 Q45.2467 1208.9 42.1912 1204.89 Q39.1357 1200.85 39.1357 1192.86 L39.1357 1184.65 L38.5628 1184.65 Q34.5205 1184.65 32.3244 1187.32 Q30.0964 1189.96 30.0964 1194.77 Q30.0964 1197.83 30.8284 1200.72 Q31.5605 1203.62 33.0246 1206.29 L27.6137 1206.29 Q26.3724 1203.08 25.7677 1200.05 Q25.1311 1197.03 25.1311 1194.17 Q25.1311 1186.43 29.1415 1182.61 Q33.1519 1178.79 41.3 1178.79 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M40.1224 1137.1 L61.6384 1137.1 L61.6384 1142.95 L40.3133 1142.95 Q35.2526 1142.95 32.7381 1144.93 Q30.2237 1146.9 30.2237 1150.85 Q30.2237 1155.59 33.2474 1158.33 Q36.2711 1161.06 41.491 1161.06 L61.6384 1161.06 L61.6384 1166.95 L25.9905 1166.95 L25.9905 1161.06 L31.5287 1161.06 Q28.314 1158.96 26.7225 1156.13 Q25.1311 1153.27 25.1311 1149.54 Q25.1311 1143.4 28.9505 1140.25 Q32.7381 1137.1 40.1224 1137.1 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M14.1184 1111.03 L14.1184 1070.83 L19.5293 1070.83 L19.5293 1087.7 L61.6384 1087.7 L61.6384 1094.16 L19.5293 1094.16 L19.5293 1111.03 L14.1184 1111.03 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M42.3504 1045.46 L45.2149 1045.46 L45.2149 1072.39 Q51.2623 1072.01 54.4452 1068.76 Q57.5962 1065.48 57.5962 1059.66 Q57.5962 1056.28 56.7687 1053.13 Q55.9411 1049.95 54.2861 1046.83 L59.8242 1046.83 Q61.161 1049.98 61.8612 1053.29 Q62.5615 1056.6 62.5615 1060.01 Q62.5615 1068.54 57.5962 1073.54 Q52.631 1078.5 44.1646 1078.5 Q35.4117 1078.5 30.2873 1073.79 Q25.1311 1069.05 25.1311 1061.03 Q25.1311 1053.83 29.7781 1049.66 Q34.3932 1045.46 42.3504 1045.46 M40.6316 1051.32 Q35.8255 1051.38 32.9609 1054.02 Q30.0964 1056.63 30.0964 1060.96 Q30.0964 1065.86 32.8654 1068.82 Q35.6345 1071.75 40.6634 1072.2 L40.6316 1051.32 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M32.8336 1008.1 Q28.8869 1005.9 27.009 1002.84 Q25.1311 999.789 25.1311 995.651 Q25.1311 990.081 29.046 987.057 Q32.9291 984.034 40.1224 984.034 L61.6384 984.034 L61.6384 989.922 L40.3133 989.922 Q35.1889 989.922 32.7063 991.736 Q30.2237 993.55 30.2237 997.274 Q30.2237 1001.83 33.2474 1004.47 Q36.2711 1007.11 41.491 1007.11 L61.6384 1007.11 L61.6384 1013 L40.3133 1013 Q35.1571 1013 32.7063 1014.81 Q30.2237 1016.63 30.2237 1020.41 Q30.2237 1024.9 33.2792 1027.54 Q36.3029 1030.19 41.491 1030.19 L61.6384 1030.19 L61.6384 1036.07 L25.9905 1036.07 L25.9905 1030.19 L31.5287 1030.19 Q28.2503 1028.18 26.6907 1025.38 Q25.1311 1022.58 25.1311 1018.73 Q25.1311 1014.84 27.1045 1012.14 Q29.0779 1009.4 32.8336 1008.1 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M56.2912 966.687 L75.1974 966.687 L75.1974 972.575 L25.9905 972.575 L25.9905 966.687 L31.4013 966.687 Q28.2185 964.841 26.6907 962.04 Q25.1311 959.207 25.1311 955.292 Q25.1311 948.799 30.2873 944.757 Q35.4436 940.683 43.8463 940.683 Q52.249 940.683 57.4052 944.757 Q62.5615 948.799 62.5615 955.292 Q62.5615 959.207 61.0337 962.04 Q59.4741 964.841 56.2912 966.687 M43.8463 946.762 Q37.3851 946.762 33.7248 949.436 Q30.0327 952.078 30.0327 956.725 Q30.0327 961.372 33.7248 964.045 Q37.3851 966.687 43.8463 966.687 Q50.3075 966.687 53.9996 964.045 Q57.6599 961.372 57.6599 956.725 Q57.6599 952.078 53.9996 949.436 Q50.3075 946.762 43.8463 946.762 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M42.3504 900.484 L45.2149 900.484 L45.2149 927.411 Q51.2623 927.029 54.4452 923.782 Q57.5962 920.504 57.5962 914.679 Q57.5962 911.305 56.7687 908.154 Q55.9411 904.972 54.2861 901.852 L59.8242 901.852 Q61.161 905.003 61.8612 908.314 Q62.5615 911.624 62.5615 915.029 Q62.5615 923.559 57.5962 928.556 Q52.631 933.522 44.1646 933.522 Q35.4117 933.522 30.2873 928.811 Q25.1311 924.069 25.1311 916.048 Q25.1311 908.855 29.7781 904.685 Q34.3932 900.484 42.3504 900.484 M40.6316 906.34 Q35.8255 906.404 32.9609 909.046 Q30.0964 911.656 30.0964 915.984 Q30.0964 920.886 32.8654 923.846 Q35.6345 926.774 40.6634 927.22 L40.6316 906.34 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M31.465 870.215 Q30.8921 871.201 30.6375 872.379 Q30.351 873.525 30.351 874.925 Q30.351 879.891 33.5975 882.564 Q36.8122 885.206 42.8596 885.206 L61.6384 885.206 L61.6384 891.094 L25.9905 891.094 L25.9905 885.206 L31.5287 885.206 Q28.2821 883.36 26.7225 880.4 Q25.1311 877.44 25.1311 873.207 Q25.1311 872.602 25.2266 871.87 Q25.2903 871.138 25.4494 870.247 L31.465 870.215 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M43.719 847.871 Q43.719 854.969 45.3422 857.706 Q46.9655 860.443 50.8804 860.443 Q53.9996 860.443 55.8457 858.406 Q57.6599 856.338 57.6599 852.805 Q57.6599 847.935 54.2224 845.007 Q50.7531 842.047 45.0239 842.047 L43.719 842.047 L43.719 847.871 M41.3 836.19 L61.6384 836.19 L61.6384 842.047 L56.2276 842.047 Q59.4741 844.052 61.0337 847.044 Q62.5615 850.036 62.5615 854.364 Q62.5615 859.839 59.5059 863.085 Q56.4186 866.3 51.2623 866.3 Q45.2467 866.3 42.1912 862.289 Q39.1357 858.247 39.1357 850.258 L39.1357 842.047 L38.5628 842.047 Q34.5205 842.047 32.3244 844.72 Q30.0964 847.362 30.0964 852.168 Q30.0964 855.224 30.8284 858.12 Q31.5605 861.016 33.0246 863.69 L27.6137 863.69 Q26.3724 860.475 25.7677 857.452 Q25.1311 854.428 25.1311 851.563 Q25.1311 843.829 29.1415 840.01 Q33.1519 836.19 41.3 836.19 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M15.869 818.334 L25.9905 818.334 L25.9905 806.271 L30.542 806.271 L30.542 818.334 L49.8937 818.334 Q54.2542 818.334 55.4955 817.157 Q56.7368 815.947 56.7368 812.287 L56.7368 806.271 L61.6384 806.271 L61.6384 812.287 Q61.6384 819.066 59.124 821.644 Q56.5777 824.223 49.8937 824.223 L30.542 824.223 L30.542 828.519 L25.9905 828.519 L25.9905 824.223 L15.869 824.223 L15.869 818.334 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M47.5702 799.174 L25.9905 799.174 L25.9905 793.317 L47.3474 793.317 Q52.4082 793.317 54.9545 791.344 Q57.4689 789.37 57.4689 785.424 Q57.4689 780.681 54.4452 777.944 Q51.4215 775.175 46.2016 775.175 L25.9905 775.175 L25.9905 769.318 L61.6384 769.318 L61.6384 775.175 L56.1639 775.175 Q59.4104 777.307 61.0019 780.14 Q62.5615 782.941 62.5615 786.665 Q62.5615 792.808 58.742 795.991 Q54.9226 799.174 47.5702 799.174 M25.1311 784.437 L25.1311 784.437 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M31.465 736.599 Q30.8921 737.585 30.6375 738.763 Q30.351 739.909 30.351 741.309 Q30.351 746.274 33.5975 748.948 Q36.8122 751.59 42.8596 751.59 L61.6384 751.59 L61.6384 757.478 L25.9905 757.478 L25.9905 751.59 L31.5287 751.59 Q28.2821 749.744 26.7225 746.784 Q25.1311 743.824 25.1311 739.59 Q25.1311 738.986 25.2266 738.254 Q25.2903 737.522 25.4494 736.63 L31.465 736.599 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M42.3504 701.396 L45.2149 701.396 L45.2149 728.323 Q51.2623 727.941 54.4452 724.695 Q57.5962 721.416 57.5962 715.592 Q57.5962 712.218 56.7687 709.067 Q55.9411 705.884 54.2861 702.765 L59.8242 702.765 Q61.161 705.916 61.8612 709.226 Q62.5615 712.536 62.5615 715.942 Q62.5615 724.472 57.5962 729.469 Q52.631 734.434 44.1646 734.434 Q35.4117 734.434 30.2873 729.724 Q25.1311 724.981 25.1311 716.96 Q25.1311 709.767 29.7781 705.598 Q34.3932 701.396 42.3504 701.396 M40.6316 707.253 Q35.8255 707.316 32.9609 709.958 Q30.0964 712.568 30.0964 716.897 Q30.0964 721.798 32.8654 724.758 Q35.6345 727.687 40.6634 728.132 L40.6316 707.253 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M20.4523 654.927 L44.1009 663.648 L44.1009 646.174 L20.4523 654.927 M14.1184 658.555 L14.1184 651.266 L61.6384 633.156 L61.6384 639.84 L49.4481 644.169 L49.4481 665.589 L61.6384 669.918 L61.6384 676.697 L14.1184 658.555 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M40.1224 596.84 L61.6384 596.84 L61.6384 602.696 L40.3133 602.696 Q35.2526 602.696 32.7381 604.669 Q30.2237 606.643 30.2237 610.589 Q30.2237 615.332 33.2474 618.069 Q36.2711 620.806 41.491 620.806 L61.6384 620.806 L61.6384 626.695 L25.9905 626.695 L25.9905 620.806 L31.5287 620.806 Q28.314 618.706 26.7225 615.873 Q25.1311 613.008 25.1311 609.284 Q25.1311 603.142 28.9505 599.991 Q32.7381 596.84 40.1224 596.84 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M30.0964 571.345 Q30.0964 576.055 33.7885 578.793 Q37.4488 581.53 43.8463 581.53 Q50.2438 581.53 53.9359 578.825 Q57.5962 576.087 57.5962 571.345 Q57.5962 566.666 53.9041 563.929 Q50.212 561.192 43.8463 561.192 Q37.5124 561.192 33.8203 563.929 Q30.0964 566.666 30.0964 571.345 M25.1311 571.345 Q25.1311 563.706 30.0964 559.346 Q35.0616 554.985 43.8463 554.985 Q52.5991 554.985 57.5962 559.346 Q62.5615 563.706 62.5615 571.345 Q62.5615 579.016 57.5962 583.376 Q52.5991 587.705 43.8463 587.705 Q35.0616 587.705 30.0964 583.376 Q25.1311 579.016 25.1311 571.345 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M32.8336 517.523 Q28.8869 515.327 27.009 512.271 Q25.1311 509.216 25.1311 505.078 Q25.1311 499.508 29.046 496.484 Q32.9291 493.46 40.1224 493.46 L61.6384 493.46 L61.6384 499.349 L40.3133 499.349 Q35.1889 499.349 32.7063 501.163 Q30.2237 502.977 30.2237 506.701 Q30.2237 511.253 33.2474 513.894 Q36.2711 516.536 41.491 516.536 L61.6384 516.536 L61.6384 522.424 L40.3133 522.424 Q35.1571 522.424 32.7063 524.239 Q30.2237 526.053 30.2237 529.84 Q30.2237 534.328 33.2792 536.97 Q36.3029 539.612 41.491 539.612 L61.6384 539.612 L61.6384 545.5 L25.9905 545.5 L25.9905 539.612 L31.5287 539.612 Q28.2503 537.607 26.6907 534.806 Q25.1311 532.005 25.1311 528.154 Q25.1311 524.27 27.1045 521.565 Q29.0779 518.828 32.8336 517.523 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M43.719 465.579 Q43.719 472.676 45.3422 475.414 Q46.9655 478.151 50.8804 478.151 Q53.9996 478.151 55.8457 476.114 Q57.6599 474.045 57.6599 470.512 Q57.6599 465.642 54.2224 462.714 Q50.7531 459.754 45.0239 459.754 L43.719 459.754 L43.719 465.579 M41.3 453.898 L61.6384 453.898 L61.6384 459.754 L56.2276 459.754 Q59.4741 461.759 61.0337 464.751 Q62.5615 467.743 62.5615 472.072 Q62.5615 477.546 59.5059 480.793 Q56.4186 484.007 51.2623 484.007 Q45.2467 484.007 42.1912 479.997 Q39.1357 475.955 39.1357 467.966 L39.1357 459.754 L38.5628 459.754 Q34.5205 459.754 32.3244 462.428 Q30.0964 465.069 30.0964 469.876 Q30.0964 472.931 30.8284 475.827 Q31.5605 478.724 33.0246 481.397 L27.6137 481.397 Q26.3724 478.183 25.7677 475.159 Q25.1311 472.135 25.1311 469.271 Q25.1311 461.536 29.1415 457.717 Q33.1519 453.898 41.3 453.898 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M12.1132 441.835 L12.1132 435.978 L61.6384 435.978 L61.6384 441.835 L12.1132 441.835 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M64.9486 408.892 Q71.3143 411.375 73.2559 413.73 Q75.1974 416.085 75.1974 420.032 L75.1974 424.711 L70.2958 424.711 L70.2958 421.273 Q70.2958 418.854 69.15 417.518 Q68.0041 416.181 63.7391 414.558 L61.0655 413.507 L25.9905 427.925 L25.9905 421.719 L53.8723 410.579 L25.9905 399.439 L25.9905 393.232 L64.9486 408.892 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M12.1769 350.359 Q19.4975 354.624 26.6589 356.693 Q33.8203 358.762 41.1727 358.762 Q48.5251 358.762 55.7502 356.693 Q62.9434 354.593 70.2321 350.359 L70.2321 355.452 Q62.7524 360.226 55.5274 362.613 Q48.3023 364.969 41.1727 364.969 Q34.0749 364.969 26.8817 362.613 Q19.6884 360.258 12.1769 355.452 L12.1769 350.359 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M17.365 328.843 Q17.365 331.39 19.1474 333.14 Q20.8979 334.891 23.4442 334.891 Q25.9587 334.891 27.7092 333.14 Q29.428 331.39 29.428 328.843 Q29.428 326.297 27.7092 324.546 Q25.9587 322.796 23.4442 322.796 Q20.9297 322.796 19.1474 324.578 Q17.365 326.329 17.365 328.843 M13.2591 328.843 Q13.2591 326.806 14.0548 324.928 Q14.8187 323.05 16.2828 321.682 Q17.7151 320.218 19.5293 319.486 Q21.3435 318.754 23.4442 318.754 Q27.6456 318.754 30.5738 321.714 Q33.4702 324.642 33.4702 328.907 Q33.4702 333.204 30.6056 336.068 Q27.7411 338.933 23.4442 338.933 Q19.1792 338.933 16.2191 336.005 Q13.2591 333.076 13.2591 328.843 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M17.7787 270.565 L24.5582 270.565 Q21.5345 273.812 20.0386 277.504 Q18.5426 281.164 18.5426 285.302 Q18.5426 293.45 23.5397 297.779 Q28.5049 302.107 37.9262 302.107 Q47.3156 302.107 52.3127 297.779 Q57.2779 293.45 57.2779 285.302 Q57.2779 281.164 55.782 277.504 Q54.2861 273.812 51.2623 270.565 L57.9782 270.565 Q60.2698 273.939 61.4156 277.727 Q62.5615 281.482 62.5615 285.684 Q62.5615 296.474 55.973 302.68 Q49.3526 308.887 37.9262 308.887 Q26.4679 308.887 19.8794 302.68 Q13.2591 296.474 13.2591 285.684 Q13.2591 281.419 14.4049 277.663 Q15.5189 273.875 17.7787 270.565 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip870)" d="M12.1769 261.812 L12.1769 256.72 Q19.6884 251.946 26.8817 249.59 Q34.0749 247.203 41.1727 247.203 Q48.3023 247.203 55.5274 249.59 Q62.7524 251.946 70.2321 256.72 L70.2321 261.812 Q62.9434 257.579 55.7502 255.51 Q48.5251 253.41 41.1727 253.41 Q33.8203 253.41 26.6589 255.51 Q19.4975 257.579 12.1769 261.812 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip872)" style="stroke:#009af9; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="181.719,1525.16 188.745,1520.1 195.771,1516.19 202.797,1513.49 209.823,1512.04 216.849,1511.66 223.874,1512.19 230.9,1517.08 237.926,1518.35 244.952,1516.75 251.978,1513.92 259.003,1511.04 266.029,1508.65 273.055,1524.7 280.081,1532.93 287.107,1532.24 294.133,1529.64 301.158,1533.25 308.184,1536.92 315.21,1534.01 322.236,1528.16 329.262,1524.62 336.287,1520.63 343.313,1516.47 350.339,1513.33 357.365,1511.68 364.391,1510.79 371.417,1510.33 378.442,1509.95 385.468,1508.83 392.494,1506.37 399.52,1503.65 406.546,1501.3 413.571,1499.77 420.597,1534.18 427.623,1599.61 434.649,1611.61 441.675,1605.3 448.701,1592.31 455.726,1576.99 462.752,1561.48 469.778,1547.85 476.804,1536.62 483.83,1527.65 490.855,1520.9 497.881,1516.34 504.907,1513.2 511.933,1513.7 518.959,1516.62 525.985,1517.33 533.01,1516.17 540.036,1514.33 547.062,1511.93 554.088,1509.33 561.114,1506.7 568.14,1504.39 575.165,1502.8 582.191,1501.68 589.217,1501.19 596.243,1501.41 603.269,1571.17 610.294,1607.79 617.32,1606.29 624.346,1593.21 631.372,1578.31 638.398,1564.66 645.424,1621.24 652.449,1686.38 659.475,1684.62 666.501,1661.48 673.527,1635.22 680.553,1611.7 687.578,1593.91 694.604,1579.46 701.63,1574.13 708.656,1564.79 715.682,1553.97 722.708,1543.41 729.733,1534.07 736.759,1526.47 743.785,1520.26 750.811,1515.61 757.837,1559.59 764.862,1581.31 771.888,1578.3 778.914,1568.22 785.94,1591.3 792.966,1597.39 799.992,1585.84 807.017,1570.48 814.043,1556.18 821.069,1544.31 828.095,1535.08 835.121,1528.09 842.146,1522.78 849.172,1518.74 856.198,1515.61 863.224,1516.21 870.25,1515.48 877.276,1513.25 884.301,1510.47 891.327,1507.81 898.353,1505.43 905.379,1503.44 912.405,1505.23 919.43,1507.81 926.456,1507.41 933.482,1506.9 940.508,1509.68 947.534,1509.07 954.56,1506.89 961.585,1504.43 968.611,1503.52 975.637,1517.37 982.663,1521.46 989.689,1519.07 996.714,1515.32 1003.74,1511.5 1010.77,1508.33 1017.79,1505.6 1024.82,1502.79 1031.84,1500.54 1038.87,1498.26 1045.9,1496.36 1052.92,1499.19 1059.95,1499.39 1066.97,1499.24 1074,1499.07 1081.02,1498.21 1088.05,1497.3 1095.08,1496.51 1102.1,1496.25 1109.13,1495.67 1116.15,1494.38 1123.18,1503.18 1130.2,1539.03 1137.23,1545.62 1144.26,1540.23 1151.28,1534.85 1158.31,1527.55 1165.33,1520.36 1172.36,1516.72 1179.39,1516.31 1186.41,1511.8 1193.44,1505.58 1200.46,1500.13 1207.49,1495.88 1214.51,1492.75 1221.54,1490.63 1228.57,1489 1235.59,1488.03 1242.62,1487.26 1249.64,1486.66 1256.67,1488.01 1263.7,1504.32 1270.72,1508.41 1277.75,1505.4 1284.77,1500.44 1291.8,1499.13 1298.82,1496.31 1305.85,1492.89 1312.88,1490.43 1319.9,1488.33 1326.93,1495.11 1333.95,1501.89 1340.98,1500.34 1348.01,1495.03 1355.03,1489.57 1362.06,1484.68 1369.08,1480.41 1376.11,1476.71 1383.13,1474.47 1390.16,1471.97 1397.19,1472.12 1404.21,1472.41 1411.24,1470.73 1418.26,1468.89 1425.29,1466.73 1432.32,1464.76 1439.34,1463.82 1446.37,1466.51 1453.39,1465.09 1460.42,1463.11 1467.44,1464.08 1474.47,1462.88 1481.5,1461.44 1488.52,1459.79 1495.55,1457.51 1502.57,1455.51 1509.6,1453.26 1516.62,1451.78 1523.65,1451.04 1530.68,1450.68 1537.7,1450.86 1544.73,1451.19 1551.75,1451.21 1558.78,1449.7 1565.81,1447.79 1572.83,1445.78 1579.86,1444.07 1586.88,1442.75 1593.91,1443.08 1600.93,1443.75 1607.96,1444.37 1614.99,1445.15 1622.01,1445.52 1629.04,1445.56 1636.06,1444.96 1643.09,1443.71 1650.12,1441.76 1657.14,1440.78 1664.17,1440.23 1671.19,1441.98 1678.22,1446.49 1685.24,1465.18 1692.27,1477.78 1699.3,1481.47 1706.32,1475.47 1713.35,1473.44 1720.37,1469.41 1727.4,1463.98 1734.43,1459.25 1741.45,1457.57 1748.48,1454.59 1755.5,1449.26 1762.53,1445.34 1769.55,1447.24 1776.58,1444.32 1783.61,1438.69 1790.63,1431.94 1797.66,1425.03 1804.68,1419.18 1811.71,1413.64 1818.74,1415.07 1825.76,1423.25 1832.79,1422.39 1839.81,1416.82 1846.84,1411.36 1853.86,1405.01 1860.89,1396.82 1867.92,1387.11 1874.94,1378.99 1881.97,1386.26 1888.99,1410.58 1896.02,1413.75 1903.04,1405.15 1910.07,1393.01 1917.1,1380.99 1924.12,1372.96 1931.15,1363.93 1938.17,1352.23 1945.2,1340.92 1952.23,1331.36 1959.25,1324.3 1966.28,1319.58 1973.3,1315.14 1980.33,1312.25 1987.35,1310.11 1994.38,1307.56 2001.41,1303.65 2008.43,1299.88 2015.46,1295.49 2022.48,1290.57 2029.51,1285.74 2036.54,1279.34 2043.56,1274.35 2050.59,1268.41 2057.61,1262.78 2064.64,1256.89 2071.66,1250.71 2078.69,1244.23 2085.72,1237.39 2092.74,1230.07 2099.77,1222.26 2106.79,1214.37 2113.82,1206.65 2120.85,1199.37 2127.87,1192.49 2134.9,1185.92 2141.92,1179.26 2148.95,1172.66 2155.97,1165.91 2163,1159.2 2170.03,1152.27 2177.05,1144.75 2184.08,1136.38 2191.1,1127.96 2198.13,1119.2 2205.16,1110.82 2212.18,1102.39 2219.21,1094.35 2226.23,1086.57 2233.26,1078.93 2240.28,1071.08 2247.31,1062.89 2254.34,1054.2 2261.36,1044.61 2268.39,1034.5 2275.41,1023.88 2282.44,1013.31 2289.46,1003.25 2296.49,993.821 2303.52,984.661 2310.54,975.679 2317.57,966.749 2324.59,957.55 2331.62,947.798 2338.65,937.49 2345.67,927.21 2352.7,916.874 2359.72,906.505 2366.75,896.581 2373.77,886.92 2380.8,877.4 2387.83,867.797 2394.85,858.05 2401.88,848.013 2408.9,837.298 2415.93,826.078 2422.96,814.661 2429.98,803.461 2437.01,792.626 2444.03,782.155 2451.06,771.755 2458.08,761.367 2465.11,751.069 2472.14,740.715 2479.16,730.321 2486.19,719.681 2493.21,708.62 2500.24,697.193 2507.27,685.582 2514.29,674.06 2521.32,662.685 2528.34,651.726 2535.37,640.986 2542.39,630.748 2549.42,620.76 2556.45,610.789 2563.47,600.806 2570.5,590.546 2577.52,579.869 2584.55,569.024 2591.58,557.945 2598.6,547.43 2605.63,537.618 2612.65,528.31 2619.68,519.492 2626.7,510.828 2633.73,502.299 2640.76,493.585 2647.78,484.497 2654.81,475.119 2661.83,466.06 2668.86,457.282 2675.88,448.956 2682.91,441.221 2689.94,433.816 2696.96,426.662 2703.99,419.564 2711.01,412.387 2718.04,404.892 2725.07,396.718 2732.09,388.818 2739.12,381.154 2746.14,374.237 2753.17,367.606 2760.19,361.501 2767.22,355.647 2774.25,349.941 2781.27,344.231 2788.3,338.156 2795.32,331.585 2802.35,324.592 2809.38,318.278 2816.4,312.084 2823.43,306.612 2830.45,301.685 2837.48,296.991 2844.5,292.462 2851.53,287.846 2858.56,282.835 2865.58,276.902 2872.61,271.079 2879.63,265.332 2886.66,260.181 2893.69,255.701 2900.71,251.88 2907.74,248.2 2914.76,244.645 2921.79,241.114 2928.81,237.466 2935.84,233.399 2942.87,228.99 2949.89,224.676 2956.92,220.417 2963.94,216.378 2970.97,212.824 2978,209.36 2985.02,206.182 2992.05,203.141 2999.07,200.152 3006.1,197.149 3013.12,193.801 3020.15,189.855 3027.18,185.595 3034.2,181.634 3041.23,178.003 3048.25,174.881 3055.28,172.327 3062.31,170.166 3069.33,168.161 3076.36,166.065 3083.38,163.576 3090.41,160.423 3097.43,156.761 3104.46,153.593 3111.49,150.519 3118.51,148.296 3125.54,146.588 3132.56,145.176 3139.59,143.87 3146.61,142.518 3153.64,140.888 3160.67,138.634 3167.69,136.133 3174.72,133.478 3181.74,131.009 3188.77,129.032 3195.8,127.603 3202.82,126.45 3209.85,125.558 3216.87,124.767 3223.9,124.045 3230.92,123.291 3237.95,122.38 3244.98,121.15 3252,119.296 3259.03,117.481 3266.05,115.624 3273.08,113.793 3280.11,112.569 3287.13,111.791 3294.16,111.248 3301.18,110.737 3308.21,110.214 3315.23,109.484 3322.26,108.178 3329.29,106.381 3336.31,104.453 3343.34,102.917 3350.36,101.843 3357.39,101.368 3364.42,101.092 3371.44,100.946 3378.47,101.019 3385.49,101.106 3392.52,101.231 3399.54,101.153 3406.57,100.68 3413.6,99.8475 3420.62,98.827 3427.65,97.9569 3434.67,97.2561 3441.7,96.9634 3448.73,96.8752 3455.75,97.2408 3462.78,97.7884 3469.8,98.256 3476.83,98.5953 3483.85,98.5342 3490.88,97.9604 3497.91,97.109 3504.93,95.8558 3511.96,95.1156 3518.98,94.9858 3526.01,95.2983 3533.03,96.0624 3540.06,96.8573 3547.09,97.7547 3554.11,98.3438 3561.14,98.4007 3568.16,98.1934 3575.19,98.0245 3582.22,97.782 3589.24,97.9053 3596.27,98.3045 3603.29,99.1861 3610.32,100.19 3617.34,101.233 3624.37,102.169 3631.4,102.803 3638.42,102.842 3645.45,102.462 3652.47,102.214 3659.5,102.221 3666.53,102.769 3673.55,103.775 3680.58,104.921 3687.6,106.29 3694.63,107.738 3701.65,109.213 3708.68,110.631 3715.71,111.787 3722.73,112.509 3729.76,112.807 3736.78,113.139 3743.81,113.447 3750.84,113.929 3757.86,114.764 3764.89,115.793 3771.91,117.175 3778.94,118.627 3785.96,119.915 3792.99,120.999 3800.02,121.577 3807.04,121.571 3814.07,121.225 3821.09,120.625 3828.12,120.682 3835.15,121.168 3842.17,122.117 3849.2,123.257 3856.22,124.539 3863.25,125.672 3870.27,126.362 3877.3,126.545 3884.33,126.439 3891.35,126.437 3898.38,126.583 3905.4,127.025 3912.43,127.909 3919.45,128.936 3926.48,129.936 3933.51,130.868 3940.53,131.472 3947.56,131.479 3954.58,130.967 3961.61,130.423 3968.64,130.062 3975.66,130.494 3982.69,131.063 3989.71,131.947 3996.74,133.03 4003.76,134.144 4010.79,134.985 4017.82,135.222 4024.84,135.02 4031.87,134.427 4038.89,133.989 4045.92,134.066 4052.95,134.22 4059.97,134.405 4067,134.613 4074.02,134.841 4081.05,135.084 4088.07,135.34 4095.1,135.605 4102.13,135.878 4109.15,136.157 4116.18,136.442 4123.2,136.729 4130.23,137.018 4137.26,137.308 4144.28,137.599 4151.31,137.891 4158.33,138.184 4165.36,138.477 4172.38,138.77 4179.41,139.063 4186.44,139.356 4193.46,139.647 4200.49,139.938 4207.51,140.228 4214.54,140.517 4221.57,140.805 4228.59,141.094 4235.62,141.382 4242.64,141.668 4249.67,141.953 4256.69,142.237 4263.72,142.52 4270.75,142.801 4277.77,143.082 4284.8,143.364 4291.82,143.643 4298.85,143.921 4305.87,144.2 4312.9,144.479 4319.93,144.756 4326.95,145.032 4333.98,145.306 4341,145.58 4348.03,145.852 4355.06,146.123 4362.08,146.392 4369.11,146.66 4376.13,146.928 4383.16,147.195 4390.18,147.461 4397.21,147.726 4404.24,147.99 4411.26,148.254 4418.29,148.516 4425.31,148.778 4432.34,149.039 4439.37,149.299 4446.39,149.559 4453.42,149.818 4460.44,150.077 4467.47,150.335 4474.49,150.593 4481.52,150.849 4488.55,151.105 4495.57,151.361 4502.6,151.616 4509.62,151.87 4516.65,152.124 4523.68,152.378 4530.7,152.632 4537.73,152.884 4544.75,153.137 4551.78,153.388 4558.8,153.639 4565.83,153.89 4572.86,154.139 4579.88,154.389 4586.91,154.639 4593.93,154.888 4600.96,155.136 4607.99,155.384 4615.01,155.632 4622.04,155.88 4629.06,156.128 4636.09,156.375 4643.11,156.62 4650.14,156.866 4657.17,157.112 4664.19,157.358 4671.22,157.603 4678.24,157.847 4685.27,158.091 4692.29,158.336 4699.32,158.579 4706.35,158.822 4713.37,159.064 4720.4,159.307 4727.42,159.55 4734.45,159.792 4741.48,160.033 4748.5,160.274 4755.53,160.513 4762.55,160.754 4769.58,160.994 4776.6,161.232 4783.63,161.471 4790.66,161.71 4797.68,161.949 4804.71,162.188 4811.73,162.426 4818.76,162.664 4825.79,162.902 4832.81,163.139 4839.84,163.377 4846.86,163.613 4853.89,163.849 4860.91,164.086 4867.94,164.322 4874.97,164.557 4881.99,164.793 4889.02,165.028 4896.04,165.262 4903.07,165.497 4910.1,165.731 4917.12,165.965 4924.15,166.199 4931.17,166.432 4938.2,166.665 4945.22,166.897 4952.25,167.13 4959.28,167.363 4966.3,167.595 4973.33,167.826 4980.35,168.057 4987.38,168.288 4994.41,168.519 5001.43,168.749 5008.46,168.98 5015.48,169.211 5022.51,169.442 5029.53,169.672 5036.56,169.902 5043.59,170.132 5050.61,170.361 5057.64,170.591 5064.66,170.819 5071.69,171.048 5078.71,171.276 5085.74,171.504 5092.77,171.731 5099.79,171.958 5106.82,172.184 5113.84,172.411 5120.87,172.637 5127.9,172.863 5134.92,173.089 5141.95,173.314 5148.97,173.54 5156,173.766 5163.02,173.991 5170.05,174.216 5177.08,174.44 5184.1,174.664 5191.13,174.889 5198.15,175.113 5205.18,175.337 5212.21,175.561 5219.23,175.783 5226.26,176.006 5233.28,176.229 5240.31,176.452 5247.33,176.674 5254.36,176.896 5261.39,177.118 5268.41,177.34 5275.44,177.562 5282.46,177.784 5289.49,178.006 5296.52,178.227 5303.54,178.448 5310.57,178.669 5317.59,178.889 5324.62,179.108 5331.64,179.327 5338.67,179.547 5345.7,179.764 5352.72,179.983 5359.75,180.202 5366.77,180.42 5373.8,180.639 5380.83,180.858 5387.85,181.075 5394.88,181.292 5401.9,181.51 5408.93,181.728 5415.95,181.944 5422.98,182.161 5430.01,182.377 5437.03,182.594 5444.06,182.81 5451.08,183.025 "/>
</svg>

Figure 2: Global mean temperature anomalies (in °C, relative to 1750)
from 1750 to 2100 simulated using the energy balance model and RCP 8.5
forcings and parameters given in the problem statement.

1.  This syntax creates an empty plot object with the right axis labels,
    legend settings, etc; this is not necessary but can make the code
    more readable.
2.  `plot!` is different than `plot` because it adds elements to an
    existing plot object (in this case, `p`) rather than creating a new
    one. Since `p` is the last plot object created, we could have just
    written `plot!(t, temps)`, but adding `p` explicitly minimizes the
    chances of accidentally plotting onto the wrong axis.
3.  Similarly, `xlims!` changes the x limits for an existing plot object
    (analogously, there is also `ylims!`).

For the Monte Carlo analysis, we’ll start with 10,000 samples of
$\lambda$ and see how the simulation looks.

``` julia
n_samples = 10_000
λ_dist = LogNormal(log(2.1), log(2) / 4)
λ_samples = rand(λ_dist, n_samples)
h = histogram(λ_samples; xlabel="Climate Feedback Factor (°C/W/m²)", ylabel="Count", legend=false)
vline!(h, [2.1], color=:red, linestyle=:dash) # the original value for context
```

<?xml version="1.0" encoding="utf-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="672" height="480" viewBox="0 0 2688 1920">
<defs>
  <clipPath id="clip930">
    <rect x="0" y="0" width="2688" height="1920"/>
  </clipPath>
</defs>
<path clip-path="url(#clip930)" d="M0 1920 L2688 1920 L2688 -4.26326e-14 L0 -4.26326e-14  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip931">
    <rect x="537" y="0" width="1883" height="1883"/>
  </clipPath>
</defs>
<path clip-path="url(#clip930)" d="M241.557 1734.12 L2640.76 1734.12 L2640.76 47.2441 L241.557 47.2441  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip932">
    <rect x="241" y="47" width="2400" height="1688"/>
  </clipPath>
</defs>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="373.517,1734.12 373.517,47.2441 "/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1073.61,1734.12 1073.61,47.2441 "/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1773.7,1734.12 1773.7,47.2441 "/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="2473.79,1734.12 2473.79,47.2441 "/>
<polyline clip-path="url(#clip930)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="241.557,1734.12 2640.76,1734.12 "/>
<polyline clip-path="url(#clip930)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="373.517,1734.12 373.517,1715.22 "/>
<polyline clip-path="url(#clip930)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1073.61,1734.12 1073.61,1715.22 "/>
<polyline clip-path="url(#clip930)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1773.7,1734.12 1773.7,1715.22 "/>
<polyline clip-path="url(#clip930)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2473.79,1734.12 2473.79,1715.22 "/>
<path clip-path="url(#clip930)" d="M363.899 1794.89 L371.538 1794.89 L371.538 1768.52 L363.228 1770.19 L363.228 1765.93 L371.492 1764.26 L376.168 1764.26 L376.168 1794.89 L383.807 1794.89 L383.807 1798.82 L363.899 1798.82 L363.899 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M1068.26 1794.89 L1084.58 1794.89 L1084.58 1798.82 L1062.64 1798.82 L1062.64 1794.89 Q1065.3 1792.13 1069.88 1787.5 Q1074.49 1782.85 1075.67 1781.51 Q1077.91 1778.98 1078.79 1777.25 Q1079.7 1775.49 1079.7 1773.8 Q1079.7 1771.04 1077.75 1769.31 Q1075.83 1767.57 1072.73 1767.57 Q1070.53 1767.57 1068.08 1768.34 Q1065.65 1769.1 1062.87 1770.65 L1062.87 1765.93 Q1065.69 1764.79 1068.15 1764.21 Q1070.6 1763.64 1072.64 1763.64 Q1078.01 1763.64 1081.2 1766.32 Q1084.4 1769.01 1084.4 1773.5 Q1084.4 1775.63 1083.59 1777.55 Q1082.8 1779.45 1080.69 1782.04 Q1080.11 1782.71 1077.01 1785.93 Q1073.91 1789.12 1068.26 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M1777.95 1780.19 Q1781.3 1780.9 1783.18 1783.17 Q1785.08 1785.44 1785.08 1788.77 Q1785.08 1793.89 1781.56 1796.69 Q1778.04 1799.49 1771.56 1799.49 Q1769.38 1799.49 1767.07 1799.05 Q1764.78 1798.64 1762.32 1797.78 L1762.32 1793.27 Q1764.27 1794.4 1766.58 1794.98 Q1768.9 1795.56 1771.42 1795.56 Q1775.82 1795.56 1778.11 1793.82 Q1780.42 1792.08 1780.42 1788.77 Q1780.42 1785.72 1778.27 1784.01 Q1776.14 1782.27 1772.32 1782.27 L1768.29 1782.27 L1768.29 1778.43 L1772.51 1778.43 Q1775.96 1778.43 1777.79 1777.06 Q1779.61 1775.67 1779.61 1773.08 Q1779.61 1770.42 1777.72 1769.01 Q1775.84 1767.57 1772.32 1767.57 Q1770.4 1767.57 1768.2 1767.99 Q1766 1768.4 1763.36 1769.28 L1763.36 1765.12 Q1766.03 1764.38 1768.34 1764.01 Q1770.68 1763.64 1772.74 1763.64 Q1778.06 1763.64 1781.17 1766.07 Q1784.27 1768.47 1784.27 1772.59 Q1784.27 1775.46 1782.62 1777.46 Q1780.98 1779.42 1777.95 1780.19 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M2476.8 1768.34 L2464.99 1786.78 L2476.8 1786.78 L2476.8 1768.34 M2475.57 1764.26 L2481.45 1764.26 L2481.45 1786.78 L2486.38 1786.78 L2486.38 1790.67 L2481.45 1790.67 L2481.45 1798.82 L2476.8 1798.82 L2476.8 1790.67 L2461.2 1790.67 L2461.2 1786.16 L2475.57 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M909.143 1839.44 L909.143 1846.22 Q905.896 1843.2 902.204 1841.7 Q898.544 1840.21 894.406 1840.21 Q886.258 1840.21 881.929 1845.2 Q877.601 1850.17 877.601 1859.59 Q877.601 1868.98 881.929 1873.98 Q886.258 1878.94 894.406 1878.94 Q898.544 1878.94 902.204 1877.44 Q905.896 1875.95 909.143 1872.92 L909.143 1879.64 Q905.769 1881.93 901.981 1883.08 Q898.225 1884.22 894.024 1884.22 Q883.234 1884.22 877.028 1877.64 Q870.821 1871.02 870.821 1859.59 Q870.821 1848.13 877.028 1841.54 Q883.234 1834.92 894.024 1834.92 Q898.289 1834.92 902.045 1836.07 Q905.832 1837.18 909.143 1839.44 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M918.818 1833.78 L924.675 1833.78 L924.675 1883.3 L918.818 1883.3 L918.818 1833.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M936.929 1847.65 L942.785 1847.65 L942.785 1883.3 L936.929 1883.3 L936.929 1847.65 M936.929 1833.78 L942.785 1833.78 L942.785 1841.19 L936.929 1841.19 L936.929 1833.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M982.794 1854.5 Q984.99 1850.55 988.046 1848.67 Q991.101 1846.79 995.239 1846.79 Q1000.81 1846.79 1003.83 1850.71 Q1006.86 1854.59 1006.86 1861.78 L1006.86 1883.3 L1000.97 1883.3 L1000.97 1861.98 Q1000.97 1856.85 999.154 1854.37 Q997.339 1851.89 993.616 1851.89 Q989.064 1851.89 986.422 1854.91 Q983.78 1857.93 983.78 1863.15 L983.78 1883.3 L977.892 1883.3 L977.892 1861.98 Q977.892 1856.82 976.078 1854.37 Q974.264 1851.89 970.476 1851.89 Q965.988 1851.89 963.347 1854.94 Q960.705 1857.97 960.705 1863.15 L960.705 1883.3 L954.817 1883.3 L954.817 1847.65 L960.705 1847.65 L960.705 1853.19 Q962.71 1849.91 965.511 1848.35 Q968.312 1846.79 972.163 1846.79 Q976.046 1846.79 978.752 1848.77 Q981.489 1850.74 982.794 1854.5 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M1034.74 1865.38 Q1027.64 1865.38 1024.9 1867 Q1022.17 1868.63 1022.17 1872.54 Q1022.17 1875.66 1024.2 1877.51 Q1026.27 1879.32 1029.8 1879.32 Q1034.67 1879.32 1037.6 1875.88 Q1040.56 1872.42 1040.56 1866.69 L1040.56 1865.38 L1034.74 1865.38 M1046.42 1862.96 L1046.42 1883.3 L1040.56 1883.3 L1040.56 1877.89 Q1038.56 1881.14 1035.57 1882.7 Q1032.57 1884.22 1028.24 1884.22 Q1022.77 1884.22 1019.52 1881.17 Q1016.31 1878.08 1016.31 1872.92 Q1016.31 1866.91 1020.32 1863.85 Q1024.36 1860.8 1032.35 1860.8 L1040.56 1860.8 L1040.56 1860.23 Q1040.56 1856.18 1037.89 1853.99 Q1035.25 1851.76 1030.44 1851.76 Q1027.39 1851.76 1024.49 1852.49 Q1021.59 1853.22 1018.92 1854.69 L1018.92 1849.28 Q1022.13 1848.03 1025.16 1847.43 Q1028.18 1846.79 1031.05 1846.79 Q1038.78 1846.79 1042.6 1850.8 Q1046.42 1854.81 1046.42 1862.96 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M1064.27 1837.53 L1064.27 1847.65 L1076.34 1847.65 L1076.34 1852.2 L1064.27 1852.2 L1064.27 1871.56 Q1064.27 1875.92 1065.45 1877.16 Q1066.66 1878.4 1070.32 1878.4 L1076.34 1878.4 L1076.34 1883.3 L1070.32 1883.3 Q1063.54 1883.3 1060.96 1880.79 Q1058.39 1878.24 1058.39 1871.56 L1058.39 1852.2 L1054.09 1852.2 L1054.09 1847.65 L1058.39 1847.65 L1058.39 1837.53 L1064.27 1837.53 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M1114.53 1864.01 L1114.53 1866.88 L1087.61 1866.88 Q1087.99 1872.92 1091.23 1876.11 Q1094.51 1879.26 1100.34 1879.26 Q1103.71 1879.26 1106.86 1878.43 Q1110.04 1877.6 1113.16 1875.95 L1113.16 1881.49 Q1110.01 1882.82 1106.7 1883.52 Q1103.39 1884.22 1099.99 1884.22 Q1091.46 1884.22 1086.46 1879.26 Q1081.49 1874.29 1081.49 1865.83 Q1081.49 1857.07 1086.2 1851.95 Q1090.95 1846.79 1098.97 1846.79 Q1106.16 1846.79 1110.33 1851.44 Q1114.53 1856.06 1114.53 1864.01 M1108.68 1862.29 Q1108.61 1857.49 1105.97 1854.62 Q1103.36 1851.76 1099.03 1851.76 Q1094.13 1851.76 1091.17 1854.53 Q1088.24 1857.3 1087.8 1862.33 L1108.68 1862.29 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M1145.12 1835.78 L1172.43 1835.78 L1172.43 1841.19 L1151.55 1841.19 L1151.55 1855.2 L1170.39 1855.2 L1170.39 1860.61 L1151.55 1860.61 L1151.55 1883.3 L1145.12 1883.3 L1145.12 1835.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M1209.29 1864.01 L1209.29 1866.88 L1182.36 1866.88 Q1182.74 1872.92 1185.99 1876.11 Q1189.27 1879.26 1195.09 1879.26 Q1198.46 1879.26 1201.61 1878.43 Q1204.8 1877.6 1207.92 1875.95 L1207.92 1881.49 Q1204.77 1882.82 1201.46 1883.52 Q1198.15 1884.22 1194.74 1884.22 Q1186.21 1884.22 1181.21 1879.26 Q1176.25 1874.29 1176.25 1865.83 Q1176.25 1857.07 1180.96 1851.95 Q1185.7 1846.79 1193.72 1846.79 Q1200.91 1846.79 1205.08 1851.44 Q1209.29 1856.06 1209.29 1864.01 M1203.43 1862.29 Q1203.37 1857.49 1200.72 1854.62 Q1198.11 1851.76 1193.79 1851.76 Q1188.88 1851.76 1185.92 1854.53 Q1183 1857.3 1182.55 1862.33 L1203.43 1862.29 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M1249.39 1864.01 L1249.39 1866.88 L1222.46 1866.88 Q1222.84 1872.92 1226.09 1876.11 Q1229.37 1879.26 1235.19 1879.26 Q1238.57 1879.26 1241.72 1878.43 Q1244.9 1877.6 1248.02 1875.95 L1248.02 1881.49 Q1244.87 1882.82 1241.56 1883.52 Q1238.25 1884.22 1234.84 1884.22 Q1226.31 1884.22 1221.32 1879.26 Q1216.35 1874.29 1216.35 1865.83 Q1216.35 1857.07 1221.06 1851.95 Q1225.8 1846.79 1233.83 1846.79 Q1241.02 1846.79 1245.19 1851.44 Q1249.39 1856.06 1249.39 1864.01 M1243.53 1862.29 Q1243.47 1857.49 1240.83 1854.62 Q1238.22 1851.76 1233.89 1851.76 Q1228.99 1851.76 1226.03 1854.53 Q1223.1 1857.3 1222.65 1862.33 L1243.53 1862.29 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M1282.46 1853.06 L1282.46 1833.78 L1288.32 1833.78 L1288.32 1883.3 L1282.46 1883.3 L1282.46 1877.95 Q1280.61 1881.14 1277.78 1882.7 Q1274.98 1884.22 1271.03 1884.22 Q1264.57 1884.22 1260.5 1879.07 Q1256.46 1873.91 1256.46 1865.51 Q1256.46 1857.11 1260.5 1851.95 Q1264.57 1846.79 1271.03 1846.79 Q1274.98 1846.79 1277.78 1848.35 Q1280.61 1849.88 1282.46 1853.06 M1262.5 1865.51 Q1262.5 1871.97 1265.14 1875.66 Q1267.82 1879.32 1272.47 1879.32 Q1277.11 1879.32 1279.79 1875.66 Q1282.46 1871.97 1282.46 1865.51 Q1282.46 1859.05 1279.79 1855.39 Q1277.11 1851.7 1272.47 1851.7 Q1267.82 1851.7 1265.14 1855.39 Q1262.5 1859.05 1262.5 1865.51 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M1325.97 1865.51 Q1325.97 1859.05 1323.3 1855.39 Q1320.65 1851.7 1316.01 1851.7 Q1311.36 1851.7 1308.69 1855.39 Q1306.04 1859.05 1306.04 1865.51 Q1306.04 1871.97 1308.69 1875.66 Q1311.36 1879.32 1316.01 1879.32 Q1320.65 1879.32 1323.3 1875.66 Q1325.97 1871.97 1325.97 1865.51 M1306.04 1853.06 Q1307.89 1849.88 1310.69 1848.35 Q1313.52 1846.79 1317.44 1846.79 Q1323.93 1846.79 1327.97 1851.95 Q1332.05 1857.11 1332.05 1865.51 Q1332.05 1873.91 1327.97 1879.07 Q1323.93 1884.22 1317.44 1884.22 Q1313.52 1884.22 1310.69 1882.7 Q1307.89 1881.14 1306.04 1877.95 L1306.04 1883.3 L1300.16 1883.3 L1300.16 1833.78 L1306.04 1833.78 L1306.04 1853.06 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M1357.96 1865.38 Q1350.86 1865.38 1348.12 1867 Q1345.38 1868.63 1345.38 1872.54 Q1345.38 1875.66 1347.42 1877.51 Q1349.49 1879.32 1353.02 1879.32 Q1357.89 1879.32 1360.82 1875.88 Q1363.78 1872.42 1363.78 1866.69 L1363.78 1865.38 L1357.96 1865.38 M1369.64 1862.96 L1369.64 1883.3 L1363.78 1883.3 L1363.78 1877.89 Q1361.78 1881.14 1358.78 1882.7 Q1355.79 1884.22 1351.46 1884.22 Q1345.99 1884.22 1342.74 1881.17 Q1339.53 1878.08 1339.53 1872.92 Q1339.53 1866.91 1343.54 1863.85 Q1347.58 1860.8 1355.57 1860.8 L1363.78 1860.8 L1363.78 1860.23 Q1363.78 1856.18 1361.11 1853.99 Q1358.47 1851.76 1353.66 1851.76 Q1350.6 1851.76 1347.71 1852.49 Q1344.81 1853.22 1342.14 1854.69 L1342.14 1849.28 Q1345.35 1848.03 1348.38 1847.43 Q1351.4 1846.79 1354.26 1846.79 Q1362 1846.79 1365.82 1850.8 Q1369.64 1854.81 1369.64 1862.96 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M1407.35 1849.02 L1407.35 1854.5 Q1404.87 1853.13 1402.36 1852.46 Q1399.87 1851.76 1397.33 1851.76 Q1391.63 1851.76 1388.48 1855.39 Q1385.33 1858.98 1385.33 1865.51 Q1385.33 1872.03 1388.48 1875.66 Q1391.63 1879.26 1397.33 1879.26 Q1399.87 1879.26 1402.36 1878.59 Q1404.87 1877.89 1407.35 1876.52 L1407.35 1881.93 Q1404.9 1883.08 1402.26 1883.65 Q1399.65 1884.22 1396.69 1884.22 Q1388.64 1884.22 1383.9 1879.16 Q1379.15 1874.1 1379.15 1865.51 Q1379.15 1856.79 1383.93 1851.79 Q1388.73 1846.79 1397.07 1846.79 Q1399.78 1846.79 1402.36 1847.37 Q1404.94 1847.91 1407.35 1849.02 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M1417.32 1833.78 L1423.21 1833.78 L1423.21 1863.03 L1440.68 1847.65 L1448.16 1847.65 L1429.25 1864.33 L1448.95 1883.3 L1441.32 1883.3 L1423.21 1865.89 L1423.21 1883.3 L1417.32 1883.3 L1417.32 1833.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M1476.26 1835.78 L1503.57 1835.78 L1503.57 1841.19 L1482.69 1841.19 L1482.69 1855.2 L1501.54 1855.2 L1501.54 1860.61 L1482.69 1860.61 L1482.69 1883.3 L1476.26 1883.3 L1476.26 1835.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M1523.72 1865.38 Q1516.62 1865.38 1513.88 1867 Q1511.15 1868.63 1511.15 1872.54 Q1511.15 1875.66 1513.18 1877.51 Q1515.25 1879.32 1518.79 1879.32 Q1523.66 1879.32 1526.58 1875.88 Q1529.54 1872.42 1529.54 1866.69 L1529.54 1865.38 L1523.72 1865.38 M1535.4 1862.96 L1535.4 1883.3 L1529.54 1883.3 L1529.54 1877.89 Q1527.54 1881.14 1524.55 1882.7 Q1521.56 1884.22 1517.23 1884.22 Q1511.75 1884.22 1508.51 1881.17 Q1505.29 1878.08 1505.29 1872.92 Q1505.29 1866.91 1509.3 1863.85 Q1513.34 1860.8 1521.33 1860.8 L1529.54 1860.8 L1529.54 1860.23 Q1529.54 1856.18 1526.87 1853.99 Q1524.23 1851.76 1519.42 1851.76 Q1516.37 1851.76 1513.47 1852.49 Q1510.57 1853.22 1507.9 1854.69 L1507.9 1849.28 Q1511.12 1848.03 1514.14 1847.43 Q1517.16 1846.79 1520.03 1846.79 Q1527.76 1846.79 1531.58 1850.8 Q1535.4 1854.81 1535.4 1862.96 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M1573.12 1849.02 L1573.12 1854.5 Q1570.63 1853.13 1568.12 1852.46 Q1565.64 1851.76 1563.09 1851.76 Q1557.39 1851.76 1554.24 1855.39 Q1551.09 1858.98 1551.09 1865.51 Q1551.09 1872.03 1554.24 1875.66 Q1557.39 1879.26 1563.09 1879.26 Q1565.64 1879.26 1568.12 1878.59 Q1570.63 1877.89 1573.12 1876.52 L1573.12 1881.93 Q1570.67 1883.08 1568.03 1883.65 Q1565.42 1884.22 1562.46 1884.22 Q1554.4 1884.22 1549.66 1879.16 Q1544.92 1874.1 1544.92 1865.51 Q1544.92 1856.79 1549.69 1851.79 Q1554.5 1846.79 1562.84 1846.79 Q1565.54 1846.79 1568.12 1847.37 Q1570.7 1847.91 1573.12 1849.02 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M1589.1 1837.53 L1589.1 1847.65 L1601.16 1847.65 L1601.16 1852.2 L1589.1 1852.2 L1589.1 1871.56 Q1589.1 1875.92 1590.27 1877.16 Q1591.48 1878.4 1595.14 1878.4 L1601.16 1878.4 L1601.16 1883.3 L1595.14 1883.3 Q1588.36 1883.3 1585.79 1880.79 Q1583.21 1878.24 1583.21 1871.56 L1583.21 1852.2 L1578.91 1852.2 L1578.91 1847.65 L1583.21 1847.65 L1583.21 1837.53 L1589.1 1837.53 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M1622.67 1851.76 Q1617.96 1851.76 1615.23 1855.45 Q1612.49 1859.11 1612.49 1865.51 Q1612.49 1871.91 1615.19 1875.6 Q1617.93 1879.26 1622.67 1879.26 Q1627.35 1879.26 1630.09 1875.57 Q1632.83 1871.87 1632.83 1865.51 Q1632.83 1859.17 1630.09 1855.48 Q1627.35 1851.76 1622.67 1851.76 M1622.67 1846.79 Q1630.31 1846.79 1634.67 1851.76 Q1639.03 1856.72 1639.03 1865.51 Q1639.03 1874.26 1634.67 1879.26 Q1630.31 1884.22 1622.67 1884.22 Q1615 1884.22 1610.64 1879.26 Q1606.31 1874.26 1606.31 1865.51 Q1606.31 1856.72 1610.64 1851.76 Q1615 1846.79 1622.67 1846.79 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M1669.4 1853.13 Q1668.41 1852.55 1667.23 1852.3 Q1666.09 1852.01 1664.69 1852.01 Q1659.72 1852.01 1657.05 1855.26 Q1654.41 1858.47 1654.41 1864.52 L1654.41 1883.3 L1648.52 1883.3 L1648.52 1847.65 L1654.41 1847.65 L1654.41 1853.19 Q1656.25 1849.94 1659.21 1848.39 Q1662.17 1846.79 1666.41 1846.79 Q1667.01 1846.79 1667.74 1846.89 Q1668.48 1846.95 1669.37 1847.11 L1669.4 1853.13 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M1710.33 1833.84 Q1706.07 1841.16 1704 1848.32 Q1701.93 1855.48 1701.93 1862.84 Q1701.93 1870.19 1704 1877.41 Q1706.1 1884.61 1710.33 1891.89 L1705.24 1891.89 Q1700.46 1884.41 1698.08 1877.19 Q1695.72 1869.96 1695.72 1862.84 Q1695.72 1855.74 1698.08 1848.54 Q1700.43 1841.35 1705.24 1833.84 L1710.33 1833.84 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M1731.85 1839.03 Q1729.3 1839.03 1727.55 1840.81 Q1725.8 1842.56 1725.8 1845.11 Q1725.8 1847.62 1727.55 1849.37 Q1729.3 1851.09 1731.85 1851.09 Q1734.39 1851.09 1736.14 1849.37 Q1737.89 1847.62 1737.89 1845.11 Q1737.89 1842.59 1736.11 1840.81 Q1734.36 1839.03 1731.85 1839.03 M1731.85 1834.92 Q1733.88 1834.92 1735.76 1835.72 Q1737.64 1836.48 1739.01 1837.95 Q1740.47 1839.38 1741.2 1841.19 Q1741.94 1843.01 1741.94 1845.11 Q1741.94 1849.31 1738.98 1852.24 Q1736.05 1855.13 1731.78 1855.13 Q1727.49 1855.13 1724.62 1852.27 Q1721.76 1849.4 1721.76 1845.11 Q1721.76 1840.84 1724.69 1837.88 Q1727.61 1834.92 1731.85 1834.92 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M1790.12 1839.44 L1790.12 1846.22 Q1786.88 1843.2 1783.19 1841.7 Q1779.53 1840.21 1775.39 1840.21 Q1767.24 1840.21 1762.91 1845.2 Q1758.58 1850.17 1758.58 1859.59 Q1758.58 1868.98 1762.91 1873.98 Q1767.24 1878.94 1775.39 1878.94 Q1779.53 1878.94 1783.19 1877.44 Q1786.88 1875.95 1790.12 1872.92 L1790.12 1879.64 Q1786.75 1881.93 1782.96 1883.08 Q1779.21 1884.22 1775.01 1884.22 Q1764.22 1884.22 1758.01 1877.64 Q1751.8 1871.02 1751.8 1859.59 Q1751.8 1848.13 1758.01 1841.54 Q1764.22 1834.92 1775.01 1834.92 Q1779.27 1834.92 1783.03 1836.07 Q1786.81 1837.18 1790.12 1839.44 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M1810.21 1835.78 L1815.62 1835.78 L1799.07 1889.35 L1793.66 1889.35 L1810.21 1835.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M1817.78 1835.78 L1824.28 1835.78 L1834.27 1875.95 L1844.23 1835.78 L1851.46 1835.78 L1861.45 1875.95 L1871.41 1835.78 L1877.94 1835.78 L1866 1883.3 L1857.92 1883.3 L1847.89 1842.05 L1837.77 1883.3 L1829.69 1883.3 L1817.78 1835.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M1896.62 1835.78 L1902.03 1835.78 L1885.48 1889.35 L1880.07 1889.35 L1896.62 1835.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M1935.93 1854.5 Q1938.13 1850.55 1941.18 1848.67 Q1944.24 1846.79 1948.38 1846.79 Q1953.95 1846.79 1956.97 1850.71 Q1959.99 1854.59 1959.99 1861.78 L1959.99 1883.3 L1954.11 1883.3 L1954.11 1861.98 Q1954.11 1856.85 1952.29 1854.37 Q1950.48 1851.89 1946.75 1851.89 Q1942.2 1851.89 1939.56 1854.91 Q1936.92 1857.93 1936.92 1863.15 L1936.92 1883.3 L1931.03 1883.3 L1931.03 1861.98 Q1931.03 1856.82 1929.22 1854.37 Q1927.4 1851.89 1923.61 1851.89 Q1919.13 1851.89 1916.48 1854.94 Q1913.84 1857.97 1913.84 1863.15 L1913.84 1883.3 L1907.95 1883.3 L1907.95 1847.65 L1913.84 1847.65 L1913.84 1853.19 Q1915.85 1849.91 1918.65 1848.35 Q1921.45 1846.79 1925.3 1846.79 Q1929.18 1846.79 1931.89 1848.77 Q1934.63 1850.74 1935.93 1854.5 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M1974.06 1858.41 L1987.56 1858.41 L1987.56 1862.04 L1968.52 1862.04 L1968.52 1858.54 Q1969.61 1857.55 1971.61 1855.77 Q1982.56 1846.06 1982.56 1843.07 Q1982.56 1840.97 1980.9 1839.7 Q1979.25 1838.39 1976.54 1838.39 Q1974.89 1838.39 1972.95 1838.96 Q1971.01 1839.5 1968.71 1840.62 L1968.71 1836.7 Q1971.17 1835.81 1973.27 1835.37 Q1975.4 1834.92 1977.21 1834.92 Q1981.83 1834.92 1984.6 1837.02 Q1987.37 1839.12 1987.37 1842.56 Q1987.37 1846.98 1976.83 1856.02 Q1975.05 1857.55 1974.06 1858.41 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M1996.88 1833.84 L2001.98 1833.84 Q2006.75 1841.35 2009.1 1848.54 Q2011.49 1855.74 2011.49 1862.84 Q2011.49 1869.96 2009.1 1877.19 Q2006.75 1884.41 2001.98 1891.89 L1996.88 1891.89 Q2001.12 1884.61 2003.18 1877.41 Q2005.29 1870.19 2005.29 1862.84 Q2005.29 1855.48 2003.18 1848.32 Q2001.12 1841.16 1996.88 1833.84 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="241.557,1734.12 2640.76,1734.12 "/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="241.557,1436.61 2640.76,1436.61 "/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="241.557,1139.1 2640.76,1139.1 "/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="241.557,841.592 2640.76,841.592 "/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="241.557,544.083 2640.76,544.083 "/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="241.557,246.575 2640.76,246.575 "/>
<polyline clip-path="url(#clip930)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="241.557,1734.12 241.557,47.2441 "/>
<polyline clip-path="url(#clip930)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="241.557,1734.12 260.455,1734.12 "/>
<polyline clip-path="url(#clip930)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="241.557,1436.61 260.455,1436.61 "/>
<polyline clip-path="url(#clip930)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="241.557,1139.1 260.455,1139.1 "/>
<polyline clip-path="url(#clip930)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="241.557,841.592 260.455,841.592 "/>
<polyline clip-path="url(#clip930)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="241.557,544.083 260.455,544.083 "/>
<polyline clip-path="url(#clip930)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="241.557,246.575 260.455,246.575 "/>
<path clip-path="url(#clip930)" d="M189.293 1719.92 Q185.682 1719.92 183.853 1723.48 Q182.047 1727.02 182.047 1734.15 Q182.047 1741.26 183.853 1744.82 Q185.682 1748.36 189.293 1748.36 Q192.927 1748.36 194.733 1744.82 Q196.561 1741.26 196.561 1734.15 Q196.561 1727.02 194.733 1723.48 Q192.927 1719.92 189.293 1719.92 M189.293 1716.21 Q195.103 1716.21 198.158 1720.82 Q201.237 1725.4 201.237 1734.15 Q201.237 1742.88 198.158 1747.48 Q195.103 1752.07 189.293 1752.07 Q183.483 1752.07 180.404 1747.48 Q177.348 1742.88 177.348 1734.15 Q177.348 1725.4 180.404 1720.82 Q183.483 1716.21 189.293 1716.21 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M119.779 1449.95 L127.418 1449.95 L127.418 1423.59 L119.108 1425.25 L119.108 1421 L127.372 1419.33 L132.048 1419.33 L132.048 1449.95 L139.687 1449.95 L139.687 1453.89 L119.779 1453.89 L119.779 1449.95 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M159.131 1422.41 Q155.52 1422.41 153.691 1425.97 Q151.886 1429.51 151.886 1436.64 Q151.886 1443.75 153.691 1447.31 Q155.52 1450.86 159.131 1450.86 Q162.765 1450.86 164.571 1447.31 Q166.399 1443.75 166.399 1436.64 Q166.399 1429.51 164.571 1425.97 Q162.765 1422.41 159.131 1422.41 M159.131 1418.7 Q164.941 1418.7 167.997 1423.31 Q171.075 1427.89 171.075 1436.64 Q171.075 1445.37 167.997 1449.98 Q164.941 1454.56 159.131 1454.56 Q153.321 1454.56 150.242 1449.98 Q147.186 1445.37 147.186 1436.64 Q147.186 1427.89 150.242 1423.31 Q153.321 1418.7 159.131 1418.7 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M189.293 1422.41 Q185.682 1422.41 183.853 1425.97 Q182.047 1429.51 182.047 1436.64 Q182.047 1443.75 183.853 1447.31 Q185.682 1450.86 189.293 1450.86 Q192.927 1450.86 194.733 1447.31 Q196.561 1443.75 196.561 1436.64 Q196.561 1429.51 194.733 1425.97 Q192.927 1422.41 189.293 1422.41 M189.293 1418.7 Q195.103 1418.7 198.158 1423.31 Q201.237 1427.89 201.237 1436.64 Q201.237 1445.37 198.158 1449.98 Q195.103 1454.56 189.293 1454.56 Q183.483 1454.56 180.404 1449.98 Q177.348 1445.37 177.348 1436.64 Q177.348 1427.89 180.404 1423.31 Q183.483 1418.7 189.293 1418.7 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M122.997 1152.44 L139.316 1152.44 L139.316 1156.38 L117.372 1156.38 L117.372 1152.44 Q120.034 1149.69 124.617 1145.06 Q129.224 1140.41 130.404 1139.07 Q132.65 1136.54 133.529 1134.81 Q134.432 1133.05 134.432 1131.36 Q134.432 1128.6 132.488 1126.87 Q130.566 1125.13 127.464 1125.13 Q125.265 1125.13 122.812 1125.89 Q120.381 1126.66 117.603 1128.21 L117.603 1123.49 Q120.427 1122.35 122.881 1121.77 Q125.335 1121.2 127.372 1121.2 Q132.742 1121.2 135.937 1123.88 Q139.131 1126.57 139.131 1131.06 Q139.131 1133.19 138.321 1135.11 Q137.534 1137.01 135.427 1139.6 Q134.849 1140.27 131.747 1143.49 Q128.645 1146.68 122.997 1152.44 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M159.131 1124.9 Q155.52 1124.9 153.691 1128.46 Q151.886 1132.01 151.886 1139.13 Q151.886 1146.24 153.691 1149.81 Q155.52 1153.35 159.131 1153.35 Q162.765 1153.35 164.571 1149.81 Q166.399 1146.24 166.399 1139.13 Q166.399 1132.01 164.571 1128.46 Q162.765 1124.9 159.131 1124.9 M159.131 1121.2 Q164.941 1121.2 167.997 1125.8 Q171.075 1130.38 171.075 1139.13 Q171.075 1147.86 167.997 1152.47 Q164.941 1157.05 159.131 1157.05 Q153.321 1157.05 150.242 1152.47 Q147.186 1147.86 147.186 1139.13 Q147.186 1130.38 150.242 1125.8 Q153.321 1121.2 159.131 1121.2 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M189.293 1124.9 Q185.682 1124.9 183.853 1128.46 Q182.047 1132.01 182.047 1139.13 Q182.047 1146.24 183.853 1149.81 Q185.682 1153.35 189.293 1153.35 Q192.927 1153.35 194.733 1149.81 Q196.561 1146.24 196.561 1139.13 Q196.561 1132.01 194.733 1128.46 Q192.927 1124.9 189.293 1124.9 M189.293 1121.2 Q195.103 1121.2 198.158 1125.8 Q201.237 1130.38 201.237 1139.13 Q201.237 1147.86 198.158 1152.47 Q195.103 1157.05 189.293 1157.05 Q183.483 1157.05 180.404 1152.47 Q177.348 1147.86 177.348 1139.13 Q177.348 1130.38 180.404 1125.8 Q183.483 1121.2 189.293 1121.2 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M133.136 840.237 Q136.492 840.955 138.367 843.224 Q140.265 845.492 140.265 848.825 Q140.265 853.941 136.747 856.742 Q133.228 859.543 126.747 859.543 Q124.571 859.543 122.256 859.103 Q119.964 858.686 117.511 857.83 L117.511 853.316 Q119.455 854.45 121.77 855.029 Q124.085 855.608 126.608 855.608 Q131.006 855.608 133.298 853.872 Q135.612 852.136 135.612 848.825 Q135.612 845.77 133.46 844.057 Q131.33 842.321 127.511 842.321 L123.483 842.321 L123.483 838.478 L127.696 838.478 Q131.145 838.478 132.974 837.113 Q134.802 835.724 134.802 833.131 Q134.802 830.469 132.904 829.057 Q131.029 827.622 127.511 827.622 Q125.589 827.622 123.39 828.038 Q121.191 828.455 118.552 829.335 L118.552 825.168 Q121.214 824.427 123.529 824.057 Q125.867 823.687 127.927 823.687 Q133.251 823.687 136.353 826.117 Q139.455 828.525 139.455 832.645 Q139.455 835.515 137.812 837.506 Q136.168 839.474 133.136 840.237 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M159.131 827.39 Q155.52 827.39 153.691 830.955 Q151.886 834.497 151.886 841.626 Q151.886 848.733 153.691 852.298 Q155.52 855.839 159.131 855.839 Q162.765 855.839 164.571 852.298 Q166.399 848.733 166.399 841.626 Q166.399 834.497 164.571 830.955 Q162.765 827.39 159.131 827.39 M159.131 823.687 Q164.941 823.687 167.997 828.293 Q171.075 832.876 171.075 841.626 Q171.075 850.353 167.997 854.96 Q164.941 859.543 159.131 859.543 Q153.321 859.543 150.242 854.96 Q147.186 850.353 147.186 841.626 Q147.186 832.876 150.242 828.293 Q153.321 823.687 159.131 823.687 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M189.293 827.39 Q185.682 827.39 183.853 830.955 Q182.047 834.497 182.047 841.626 Q182.047 848.733 183.853 852.298 Q185.682 855.839 189.293 855.839 Q192.927 855.839 194.733 852.298 Q196.561 848.733 196.561 841.626 Q196.561 834.497 194.733 830.955 Q192.927 827.39 189.293 827.39 M189.293 823.687 Q195.103 823.687 198.158 828.293 Q201.237 832.876 201.237 841.626 Q201.237 850.353 198.158 854.96 Q195.103 859.543 189.293 859.543 Q183.483 859.543 180.404 854.96 Q177.348 850.353 177.348 841.626 Q177.348 832.876 180.404 828.293 Q183.483 823.687 189.293 823.687 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M131.816 530.877 L120.011 549.326 L131.816 549.326 L131.816 530.877 M130.589 526.803 L136.469 526.803 L136.469 549.326 L141.399 549.326 L141.399 553.215 L136.469 553.215 L136.469 561.363 L131.816 561.363 L131.816 553.215 L116.214 553.215 L116.214 548.701 L130.589 526.803 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M159.131 529.882 Q155.52 529.882 153.691 533.447 Q151.886 536.988 151.886 544.118 Q151.886 551.224 153.691 554.789 Q155.52 558.331 159.131 558.331 Q162.765 558.331 164.571 554.789 Q166.399 551.224 166.399 544.118 Q166.399 536.988 164.571 533.447 Q162.765 529.882 159.131 529.882 M159.131 526.178 Q164.941 526.178 167.997 530.785 Q171.075 535.368 171.075 544.118 Q171.075 552.845 167.997 557.451 Q164.941 562.034 159.131 562.034 Q153.321 562.034 150.242 557.451 Q147.186 552.845 147.186 544.118 Q147.186 535.368 150.242 530.785 Q153.321 526.178 159.131 526.178 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M189.293 529.882 Q185.682 529.882 183.853 533.447 Q182.047 536.988 182.047 544.118 Q182.047 551.224 183.853 554.789 Q185.682 558.331 189.293 558.331 Q192.927 558.331 194.733 554.789 Q196.561 551.224 196.561 544.118 Q196.561 536.988 194.733 533.447 Q192.927 529.882 189.293 529.882 M189.293 526.178 Q195.103 526.178 198.158 530.785 Q201.237 535.368 201.237 544.118 Q201.237 552.845 198.158 557.451 Q195.103 562.034 189.293 562.034 Q183.483 562.034 180.404 557.451 Q177.348 552.845 177.348 544.118 Q177.348 535.368 180.404 530.785 Q183.483 526.178 189.293 526.178 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M119.015 229.295 L137.372 229.295 L137.372 233.23 L123.298 233.23 L123.298 241.702 Q124.316 241.355 125.335 241.193 Q126.353 241.008 127.372 241.008 Q133.159 241.008 136.538 244.179 Q139.918 247.35 139.918 252.767 Q139.918 258.346 136.446 261.447 Q132.974 264.526 126.654 264.526 Q124.478 264.526 122.21 264.156 Q119.964 263.785 117.557 263.045 L117.557 258.346 Q119.64 259.48 121.863 260.035 Q124.085 260.591 126.562 260.591 Q130.566 260.591 132.904 258.484 Q135.242 256.378 135.242 252.767 Q135.242 249.156 132.904 247.049 Q130.566 244.943 126.562 244.943 Q124.687 244.943 122.812 245.359 Q120.96 245.776 119.015 246.656 L119.015 229.295 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M159.131 232.373 Q155.52 232.373 153.691 235.938 Q151.886 239.48 151.886 246.609 Q151.886 253.716 153.691 257.281 Q155.52 260.822 159.131 260.822 Q162.765 260.822 164.571 257.281 Q166.399 253.716 166.399 246.609 Q166.399 239.48 164.571 235.938 Q162.765 232.373 159.131 232.373 M159.131 228.67 Q164.941 228.67 167.997 233.276 Q171.075 237.86 171.075 246.609 Q171.075 255.336 167.997 259.943 Q164.941 264.526 159.131 264.526 Q153.321 264.526 150.242 259.943 Q147.186 255.336 147.186 246.609 Q147.186 237.86 150.242 233.276 Q153.321 228.67 159.131 228.67 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M189.293 232.373 Q185.682 232.373 183.853 235.938 Q182.047 239.48 182.047 246.609 Q182.047 253.716 183.853 257.281 Q185.682 260.822 189.293 260.822 Q192.927 260.822 194.733 257.281 Q196.561 253.716 196.561 246.609 Q196.561 239.48 194.733 235.938 Q192.927 232.373 189.293 232.373 M189.293 228.67 Q195.103 228.67 198.158 233.276 Q201.237 237.86 201.237 246.609 Q201.237 255.336 198.158 259.943 Q195.103 264.526 189.293 264.526 Q183.483 264.526 180.404 259.943 Q177.348 255.336 177.348 246.609 Q177.348 237.86 180.404 233.276 Q183.483 228.67 189.293 228.67 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M17.7787 946.54 L24.5582 946.54 Q21.5345 949.786 20.0386 953.478 Q18.5426 957.139 18.5426 961.276 Q18.5426 969.424 23.5397 973.753 Q28.5049 978.082 37.9262 978.082 Q47.3156 978.082 52.3127 973.753 Q57.2779 969.424 57.2779 961.276 Q57.2779 957.139 55.782 953.478 Q54.2861 949.786 51.2623 946.54 L57.9782 946.54 Q60.2698 949.913 61.4156 953.701 Q62.5615 957.457 62.5615 961.658 Q62.5615 972.448 55.973 978.655 Q49.3526 984.861 37.9262 984.861 Q26.4679 984.861 19.8794 978.655 Q13.2591 972.448 13.2591 961.658 Q13.2591 957.393 14.4049 953.637 Q15.5189 949.85 17.7787 946.54 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M30.0964 923.05 Q30.0964 927.761 33.7885 930.498 Q37.4488 933.235 43.8463 933.235 Q50.2438 933.235 53.9359 930.53 Q57.5962 927.793 57.5962 923.05 Q57.5962 918.371 53.9041 915.634 Q50.212 912.897 43.8463 912.897 Q37.5124 912.897 33.8203 915.634 Q30.0964 918.371 30.0964 923.05 M25.1311 923.05 Q25.1311 915.411 30.0964 911.051 Q35.0616 906.69 43.8463 906.69 Q52.5991 906.69 57.5962 911.051 Q62.5615 915.411 62.5615 923.05 Q62.5615 930.721 57.5962 935.081 Q52.5991 939.41 43.8463 939.41 Q35.0616 939.41 30.0964 935.081 Q25.1311 930.721 25.1311 923.05 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M47.5702 897.587 L25.9905 897.587 L25.9905 891.731 L47.3474 891.731 Q52.4082 891.731 54.9545 889.758 Q57.4689 887.784 57.4689 883.837 Q57.4689 879.095 54.4452 876.358 Q51.4215 873.589 46.2016 873.589 L25.9905 873.589 L25.9905 867.732 L61.6384 867.732 L61.6384 873.589 L56.1639 873.589 Q59.4104 875.721 61.0019 878.554 Q62.5615 881.355 62.5615 885.079 Q62.5615 891.222 58.742 894.404 Q54.9226 897.587 47.5702 897.587 M25.1311 882.851 L25.1311 882.851 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M40.1224 826.037 L61.6384 826.037 L61.6384 831.893 L40.3133 831.893 Q35.2526 831.893 32.7381 833.867 Q30.2237 835.84 30.2237 839.787 Q30.2237 844.529 33.2474 847.266 Q36.2711 850.004 41.491 850.004 L61.6384 850.004 L61.6384 855.892 L25.9905 855.892 L25.9905 850.004 L31.5287 850.004 Q28.314 847.903 26.7225 845.07 Q25.1311 842.206 25.1311 838.482 Q25.1311 832.339 28.9505 829.188 Q32.7381 826.037 40.1224 826.037 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip930)" d="M15.869 808.563 L25.9905 808.563 L25.9905 796.5 L30.542 796.5 L30.542 808.563 L49.8937 808.563 Q54.2542 808.563 55.4955 807.385 Q56.7368 806.176 56.7368 802.516 L56.7368 796.5 L61.6384 796.5 L61.6384 802.516 Q61.6384 809.295 59.124 811.873 Q56.5777 814.451 49.8937 814.451 L30.542 814.451 L30.542 818.748 L25.9905 818.748 L25.9905 814.451 L15.869 814.451 L15.869 808.563 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip932)" d="M373.517 1731.14 L373.517 1734.12 L408.522 1734.12 L408.522 1731.14 L373.517 1731.14 L373.517 1731.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="373.517,1731.14 373.517,1734.12 408.522,1734.12 408.522,1731.14 373.517,1731.14 "/>
<path clip-path="url(#clip932)" d="M408.522 1731.14 L408.522 1734.12 L443.526 1734.12 L443.526 1731.14 L408.522 1731.14 L408.522 1731.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="408.522,1731.14 408.522,1734.12 443.526,1734.12 443.526,1731.14 408.522,1731.14 "/>
<path clip-path="url(#clip932)" d="M443.526 1731.14 L443.526 1734.12 L478.531 1734.12 L478.531 1731.14 L443.526 1731.14 L443.526 1731.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="443.526,1731.14 443.526,1734.12 478.531,1734.12 478.531,1731.14 443.526,1731.14 "/>
<path clip-path="url(#clip932)" d="M478.531 1722.22 L478.531 1734.12 L513.536 1734.12 L513.536 1722.22 L478.531 1722.22 L478.531 1722.22  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="478.531,1722.22 478.531,1734.12 513.536,1734.12 513.536,1722.22 478.531,1722.22 "/>
<path clip-path="url(#clip932)" d="M513.536 1704.37 L513.536 1734.12 L548.54 1734.12 L548.54 1704.37 L513.536 1704.37 L513.536 1704.37  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="513.536,1704.37 513.536,1734.12 548.54,1734.12 548.54,1704.37 513.536,1704.37 "/>
<path clip-path="url(#clip932)" d="M548.54 1689.49 L548.54 1734.12 L583.545 1734.12 L583.545 1689.49 L548.54 1689.49 L548.54 1689.49  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="548.54,1689.49 548.54,1734.12 583.545,1734.12 583.545,1689.49 548.54,1689.49 "/>
<path clip-path="url(#clip932)" d="M583.545 1653.79 L583.545 1734.12 L618.549 1734.12 L618.549 1653.79 L583.545 1653.79 L583.545 1653.79  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="583.545,1653.79 583.545,1734.12 618.549,1734.12 618.549,1653.79 583.545,1653.79 "/>
<path clip-path="url(#clip932)" d="M618.549 1588.34 L618.549 1734.12 L653.554 1734.12 L653.554 1588.34 L618.549 1588.34 L618.549 1588.34  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="618.549,1588.34 618.549,1734.12 653.554,1734.12 653.554,1588.34 618.549,1588.34 "/>
<path clip-path="url(#clip932)" d="M653.554 1543.71 L653.554 1734.12 L688.558 1734.12 L688.558 1543.71 L653.554 1543.71 L653.554 1543.71  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="653.554,1543.71 653.554,1734.12 688.558,1734.12 688.558,1543.71 653.554,1543.71 "/>
<path clip-path="url(#clip932)" d="M688.558 1403.88 L688.558 1734.12 L723.563 1734.12 L723.563 1403.88 L688.558 1403.88 L688.558 1403.88  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="688.558,1403.88 688.558,1734.12 723.563,1734.12 723.563,1403.88 688.558,1403.88 "/>
<path clip-path="url(#clip932)" d="M723.563 1335.46 L723.563 1734.12 L758.568 1734.12 L758.568 1335.46 L723.563 1335.46 L723.563 1335.46  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="723.563,1335.46 723.563,1734.12 758.568,1734.12 758.568,1335.46 723.563,1335.46 "/>
<path clip-path="url(#clip932)" d="M758.568 1112.32 L758.568 1734.12 L793.572 1734.12 L793.572 1112.32 L758.568 1112.32 L758.568 1112.32  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="758.568,1112.32 758.568,1734.12 793.572,1734.12 793.572,1112.32 758.568,1112.32 "/>
<path clip-path="url(#clip932)" d="M793.572 1046.87 L793.572 1734.12 L828.577 1734.12 L828.577 1046.87 L793.572 1046.87 L793.572 1046.87  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="793.572,1046.87 793.572,1734.12 828.577,1734.12 828.577,1046.87 793.572,1046.87 "/>
<path clip-path="url(#clip932)" d="M828.577 942.745 L828.577 1734.12 L863.581 1734.12 L863.581 942.745 L828.577 942.745 L828.577 942.745  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="828.577,942.745 828.577,1734.12 863.581,1734.12 863.581,942.745 828.577,942.745 "/>
<path clip-path="url(#clip932)" d="M863.581 710.688 L863.581 1734.12 L898.586 1734.12 L898.586 710.688 L863.581 710.688 L863.581 710.688  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="863.581,710.688 863.581,1734.12 898.586,1734.12 898.586,710.688 863.581,710.688 "/>
<path clip-path="url(#clip932)" d="M898.586 526.233 L898.586 1734.12 L933.59 1734.12 L933.59 526.233 L898.586 526.233 L898.586 526.233  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="898.586,526.233 898.586,1734.12 933.59,1734.12 933.59,526.233 898.586,526.233 "/>
<path clip-path="url(#clip932)" d="M933.59 309.052 L933.59 1734.12 L968.595 1734.12 L968.595 309.052 L933.59 309.052 L933.59 309.052  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="933.59,309.052 933.59,1734.12 968.595,1734.12 968.595,309.052 933.59,309.052 "/>
<path clip-path="url(#clip932)" d="M968.595 267.4 L968.595 1734.12 L1003.6 1734.12 L1003.6 267.4 L968.595 267.4 L968.595 267.4  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="968.595,267.4 968.595,1734.12 1003.6,1734.12 1003.6,267.4 968.595,267.4 "/>
<path clip-path="url(#clip932)" d="M1003.6 225.749 L1003.6 1734.12 L1038.6 1734.12 L1038.6 225.749 L1003.6 225.749 L1003.6 225.749  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1003.6,225.749 1003.6,1734.12 1038.6,1734.12 1038.6,225.749 1003.6,225.749 "/>
<path clip-path="url(#clip932)" d="M1038.6 166.247 L1038.6 1734.12 L1073.61 1734.12 L1073.61 166.247 L1038.6 166.247 L1038.6 166.247  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1038.6,166.247 1038.6,1734.12 1073.61,1734.12 1073.61,166.247 1038.6,166.247 "/>
<path clip-path="url(#clip932)" d="M1073.61 47.2441 L1073.61 1734.12 L1108.61 1734.12 L1108.61 47.2441 L1073.61 47.2441 L1073.61 47.2441  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1073.61,47.2441 1073.61,1734.12 1108.61,1734.12 1108.61,47.2441 1073.61,47.2441 "/>
<path clip-path="url(#clip932)" d="M1108.61 133.522 L1108.61 1734.12 L1143.62 1734.12 L1143.62 133.522 L1108.61 133.522 L1108.61 133.522  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1108.61,133.522 1108.61,1734.12 1143.62,1734.12 1143.62,133.522 1108.61,133.522 "/>
<path clip-path="url(#clip932)" d="M1143.62 79.97 L1143.62 1734.12 L1178.62 1734.12 L1178.62 79.97 L1143.62 79.97 L1143.62 79.97  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1143.62,79.97 1143.62,1734.12 1178.62,1734.12 1178.62,79.97 1143.62,79.97 "/>
<path clip-path="url(#clip932)" d="M1178.62 97.8205 L1178.62 1734.12 L1213.63 1734.12 L1213.63 97.8205 L1178.62 97.8205 L1178.62 97.8205  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1178.62,97.8205 1178.62,1734.12 1213.63,1734.12 1213.63,97.8205 1178.62,97.8205 "/>
<path clip-path="url(#clip932)" d="M1213.63 353.678 L1213.63 1734.12 L1248.63 1734.12 L1248.63 353.678 L1213.63 353.678 L1213.63 353.678  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1213.63,353.678 1213.63,1734.12 1248.63,1734.12 1248.63,353.678 1213.63,353.678 "/>
<path clip-path="url(#clip932)" d="M1248.63 383.429 L1248.63 1734.12 L1283.64 1734.12 L1283.64 383.429 L1248.63 383.429 L1248.63 383.429  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1248.63,383.429 1248.63,1734.12 1283.64,1734.12 1283.64,383.429 1248.63,383.429 "/>
<path clip-path="url(#clip932)" d="M1283.64 517.307 L1283.64 1734.12 L1318.64 1734.12 L1318.64 517.307 L1283.64 517.307 L1283.64 517.307  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1283.64,517.307 1283.64,1734.12 1318.64,1734.12 1318.64,517.307 1283.64,517.307 "/>
<path clip-path="url(#clip932)" d="M1318.64 624.41 L1318.64 1734.12 L1353.65 1734.12 L1353.65 624.41 L1318.64 624.41 L1318.64 624.41  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1318.64,624.41 1318.64,1734.12 1353.65,1734.12 1353.65,624.41 1318.64,624.41 "/>
<path clip-path="url(#clip932)" d="M1353.65 686.887 L1353.65 1734.12 L1388.65 1734.12 L1388.65 686.887 L1353.65 686.887 L1353.65 686.887  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1353.65,686.887 1353.65,1734.12 1388.65,1734.12 1388.65,686.887 1353.65,686.887 "/>
<path clip-path="url(#clip932)" d="M1388.65 907.044 L1388.65 1734.12 L1423.65 1734.12 L1423.65 907.044 L1388.65 907.044 L1388.65 907.044  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1388.65,907.044 1388.65,1734.12 1423.65,1734.12 1423.65,907.044 1388.65,907.044 "/>
<path clip-path="url(#clip932)" d="M1423.65 945.72 L1423.65 1734.12 L1458.66 1734.12 L1458.66 945.72 L1423.65 945.72 L1423.65 945.72  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1423.65,945.72 1423.65,1734.12 1458.66,1734.12 1458.66,945.72 1423.65,945.72 "/>
<path clip-path="url(#clip932)" d="M1458.66 1034.97 L1458.66 1734.12 L1493.66 1734.12 L1493.66 1034.97 L1458.66 1034.97 L1458.66 1034.97  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1458.66,1034.97 1458.66,1734.12 1493.66,1734.12 1493.66,1034.97 1458.66,1034.97 "/>
<path clip-path="url(#clip932)" d="M1493.66 1213.48 L1493.66 1734.12 L1528.67 1734.12 L1528.67 1213.48 L1493.66 1213.48 L1493.66 1213.48  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1493.66,1213.48 1493.66,1734.12 1528.67,1734.12 1528.67,1213.48 1493.66,1213.48 "/>
<path clip-path="url(#clip932)" d="M1528.67 1198.6 L1528.67 1734.12 L1563.67 1734.12 L1563.67 1198.6 L1528.67 1198.6 L1528.67 1198.6  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1528.67,1198.6 1528.67,1734.12 1563.67,1734.12 1563.67,1198.6 1528.67,1198.6 "/>
<path clip-path="url(#clip932)" d="M1563.67 1287.85 L1563.67 1734.12 L1598.68 1734.12 L1598.68 1287.85 L1563.67 1287.85 L1563.67 1287.85  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1563.67,1287.85 1563.67,1734.12 1598.68,1734.12 1598.68,1287.85 1563.67,1287.85 "/>
<path clip-path="url(#clip932)" d="M1598.68 1442.56 L1598.68 1734.12 L1633.68 1734.12 L1633.68 1442.56 L1598.68 1442.56 L1598.68 1442.56  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1598.68,1442.56 1598.68,1734.12 1633.68,1734.12 1633.68,1442.56 1598.68,1442.56 "/>
<path clip-path="url(#clip932)" d="M1633.68 1439.58 L1633.68 1734.12 L1668.69 1734.12 L1668.69 1439.58 L1633.68 1439.58 L1633.68 1439.58  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1633.68,1439.58 1633.68,1734.12 1668.69,1734.12 1668.69,1439.58 1633.68,1439.58 "/>
<path clip-path="url(#clip932)" d="M1668.69 1588.34 L1668.69 1734.12 L1703.69 1734.12 L1703.69 1588.34 L1668.69 1588.34 L1668.69 1588.34  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1668.69,1588.34 1668.69,1734.12 1703.69,1734.12 1703.69,1588.34 1668.69,1588.34 "/>
<path clip-path="url(#clip932)" d="M1703.69 1510.99 L1703.69 1734.12 L1738.7 1734.12 L1738.7 1510.99 L1703.69 1510.99 L1703.69 1510.99  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1703.69,1510.99 1703.69,1734.12 1738.7,1734.12 1738.7,1510.99 1703.69,1510.99 "/>
<path clip-path="url(#clip932)" d="M1738.7 1591.31 L1738.7 1734.12 L1773.7 1734.12 L1773.7 1591.31 L1738.7 1591.31 L1738.7 1591.31  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1738.7,1591.31 1738.7,1734.12 1773.7,1734.12 1773.7,1591.31 1738.7,1591.31 "/>
<path clip-path="url(#clip932)" d="M1773.7 1585.36 L1773.7 1734.12 L1808.7 1734.12 L1808.7 1585.36 L1773.7 1585.36 L1773.7 1585.36  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1773.7,1585.36 1773.7,1734.12 1808.7,1734.12 1808.7,1585.36 1773.7,1585.36 "/>
<path clip-path="url(#clip932)" d="M1808.7 1656.76 L1808.7 1734.12 L1843.71 1734.12 L1843.71 1656.76 L1808.7 1656.76 L1808.7 1656.76  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1808.7,1656.76 1808.7,1734.12 1843.71,1734.12 1843.71,1656.76 1808.7,1656.76 "/>
<path clip-path="url(#clip932)" d="M1843.71 1671.64 L1843.71 1734.12 L1878.71 1734.12 L1878.71 1671.64 L1843.71 1671.64 L1843.71 1671.64  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1843.71,1671.64 1843.71,1734.12 1878.71,1734.12 1878.71,1671.64 1843.71,1671.64 "/>
<path clip-path="url(#clip932)" d="M1878.71 1653.79 L1878.71 1734.12 L1913.72 1734.12 L1913.72 1653.79 L1878.71 1653.79 L1878.71 1653.79  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1878.71,1653.79 1878.71,1734.12 1913.72,1734.12 1913.72,1653.79 1878.71,1653.79 "/>
<path clip-path="url(#clip932)" d="M1913.72 1668.67 L1913.72 1734.12 L1948.72 1734.12 L1948.72 1668.67 L1913.72 1668.67 L1913.72 1668.67  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1913.72,1668.67 1913.72,1734.12 1948.72,1734.12 1948.72,1668.67 1913.72,1668.67 "/>
<path clip-path="url(#clip932)" d="M1948.72 1686.52 L1948.72 1734.12 L1983.73 1734.12 L1983.73 1686.52 L1948.72 1686.52 L1948.72 1686.52  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1948.72,1686.52 1948.72,1734.12 1983.73,1734.12 1983.73,1686.52 1948.72,1686.52 "/>
<path clip-path="url(#clip932)" d="M1983.73 1713.29 L1983.73 1734.12 L2018.73 1734.12 L2018.73 1713.29 L1983.73 1713.29 L1983.73 1713.29  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1983.73,1713.29 1983.73,1734.12 2018.73,1734.12 2018.73,1713.29 1983.73,1713.29 "/>
<path clip-path="url(#clip932)" d="M2018.73 1716.27 L2018.73 1734.12 L2053.74 1734.12 L2053.74 1716.27 L2018.73 1716.27 L2018.73 1716.27  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2018.73,1716.27 2018.73,1734.12 2053.74,1734.12 2053.74,1716.27 2018.73,1716.27 "/>
<path clip-path="url(#clip932)" d="M2053.74 1713.29 L2053.74 1734.12 L2088.74 1734.12 L2088.74 1713.29 L2053.74 1713.29 L2053.74 1713.29  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2053.74,1713.29 2053.74,1734.12 2088.74,1734.12 2088.74,1713.29 2053.74,1713.29 "/>
<path clip-path="url(#clip932)" d="M2088.74 1716.27 L2088.74 1734.12 L2123.75 1734.12 L2123.75 1716.27 L2088.74 1716.27 L2088.74 1716.27  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2088.74,1716.27 2088.74,1734.12 2123.75,1734.12 2123.75,1716.27 2088.74,1716.27 "/>
<path clip-path="url(#clip932)" d="M2123.75 1728.17 L2123.75 1734.12 L2158.75 1734.12 L2158.75 1728.17 L2123.75 1728.17 L2123.75 1728.17  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2123.75,1728.17 2123.75,1734.12 2158.75,1734.12 2158.75,1728.17 2123.75,1728.17 "/>
<path clip-path="url(#clip932)" d="M2158.75 1716.27 L2158.75 1734.12 L2193.75 1734.12 L2193.75 1716.27 L2158.75 1716.27 L2158.75 1716.27  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2158.75,1716.27 2158.75,1734.12 2193.75,1734.12 2193.75,1716.27 2158.75,1716.27 "/>
<path clip-path="url(#clip932)" d="M2193.75 1722.22 L2193.75 1734.12 L2228.76 1734.12 L2228.76 1722.22 L2193.75 1722.22 L2193.75 1722.22  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2193.75,1722.22 2193.75,1734.12 2228.76,1734.12 2228.76,1722.22 2193.75,1722.22 "/>
<path clip-path="url(#clip932)" d="M2228.76 1731.14 L2228.76 1734.12 L2263.76 1734.12 L2263.76 1731.14 L2228.76 1731.14 L2228.76 1731.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2228.76,1731.14 2228.76,1734.12 2263.76,1734.12 2263.76,1731.14 2228.76,1731.14 "/>
<path clip-path="url(#clip932)" d="M2263.76 1728.17 L2263.76 1734.12 L2298.77 1734.12 L2298.77 1728.17 L2263.76 1728.17 L2263.76 1728.17  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2263.76,1728.17 2263.76,1734.12 2298.77,1734.12 2298.77,1728.17 2263.76,1728.17 "/>
<path clip-path="url(#clip932)" d="M2298.77 1731.14 L2298.77 1734.12 L2333.77 1734.12 L2333.77 1731.14 L2298.77 1731.14 L2298.77 1731.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2298.77,1731.14 2298.77,1734.12 2333.77,1734.12 2333.77,1731.14 2298.77,1731.14 "/>
<path clip-path="url(#clip932)" d="M2333.77 1734.12 L2333.77 1734.12 L2368.78 1734.12 L2368.78 1734.12 L2333.77 1734.12 L2333.77 1734.12  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2333.77,1734.12 2333.77,1734.12 2368.78,1734.12 2333.77,1734.12 "/>
<path clip-path="url(#clip932)" d="M2368.78 1734.12 L2368.78 1734.12 L2403.78 1734.12 L2403.78 1734.12 L2368.78 1734.12 L2368.78 1734.12  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2368.78,1734.12 2368.78,1734.12 2403.78,1734.12 2368.78,1734.12 "/>
<path clip-path="url(#clip932)" d="M2403.78 1731.14 L2403.78 1734.12 L2438.79 1734.12 L2438.79 1731.14 L2403.78 1731.14 L2403.78 1731.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2403.78,1731.14 2403.78,1734.12 2438.79,1734.12 2438.79,1731.14 2403.78,1731.14 "/>
<path clip-path="url(#clip932)" d="M2438.79 1734.12 L2438.79 1734.12 L2473.79 1734.12 L2473.79 1734.12 L2438.79 1734.12 L2438.79 1734.12  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2438.79,1734.12 2438.79,1734.12 2473.79,1734.12 2438.79,1734.12 "/>
<path clip-path="url(#clip932)" d="M2473.79 1731.14 L2473.79 1734.12 L2508.8 1734.12 L2508.8 1731.14 L2473.79 1731.14 L2473.79 1731.14  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="1"/>
<polyline clip-path="url(#clip932)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2473.79,1731.14 2473.79,1734.12 2508.8,1734.12 2508.8,1731.14 2473.79,1731.14 "/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="391.02" cy="1731.14" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="426.024" cy="1731.14" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="461.029" cy="1731.14" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="496.033" cy="1722.22" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="531.038" cy="1704.37" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="566.042" cy="1689.49" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="601.047" cy="1653.79" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="636.052" cy="1588.34" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="671.056" cy="1543.71" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="706.061" cy="1403.88" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="741.065" cy="1335.46" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="776.07" cy="1112.32" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="811.074" cy="1046.87" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="846.079" cy="942.745" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="881.084" cy="710.688" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="916.088" cy="526.233" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="951.093" cy="309.052" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="986.097" cy="267.4" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1021.1" cy="225.749" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1056.11" cy="166.247" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1091.11" cy="47.2441" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1126.12" cy="133.522" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1161.12" cy="79.97" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1196.12" cy="97.8205" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1231.13" cy="353.678" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1266.13" cy="383.429" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1301.14" cy="517.307" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1336.14" cy="624.41" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1371.15" cy="686.887" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1406.15" cy="907.044" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1441.16" cy="945.72" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1476.16" cy="1034.97" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1511.17" cy="1213.48" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1546.17" cy="1198.6" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1581.17" cy="1287.85" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1616.18" cy="1442.56" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1651.18" cy="1439.58" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1686.19" cy="1588.34" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1721.19" cy="1510.99" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1756.2" cy="1591.31" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1791.2" cy="1585.36" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1826.21" cy="1656.76" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1861.21" cy="1671.64" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1896.22" cy="1653.79" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1931.22" cy="1668.67" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="1966.22" cy="1686.52" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2001.23" cy="1713.29" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2036.23" cy="1716.27" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2071.24" cy="1713.29" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2106.24" cy="1716.27" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2141.25" cy="1728.17" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2176.25" cy="1716.27" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2211.26" cy="1722.22" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2246.26" cy="1731.14" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2281.27" cy="1728.17" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2316.27" cy="1731.14" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2351.28" cy="1734.12" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2386.28" cy="1734.12" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2421.28" cy="1731.14" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2456.29" cy="1734.12" r="2"/>
<circle clip-path="url(#clip932)" style="fill:#009af9; stroke:none; fill-opacity:0" cx="2491.29" cy="1731.14" r="2"/>
<polyline clip-path="url(#clip932)" style="stroke:#ff0000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" stroke-dasharray="16, 10" points="1143.62,3420.99 1143.62,-1639.63 "/>
</svg>

Figure 3: Samples of the climate feedback factor $\lambda$. The red line
is the deterministic value we used in the original simulation above.

1.  Creating a variable for the number of samples which can then be used
    later makes it easier to change the number of samples down the road
    without possibly creating bugs (if *e.g.* we forget to change the
    value everywhere).

Now, simulate:

``` julia
temp_samples = [ebm(t, forcing_non_aerosol, forcing_aerosol, c, d, λ, α, Δt)[idx_2100] for λ in λ_samples]
```

    10000-element Vector{Float64}:
     4.437377443659359
     4.531743066138032
     4.0265497608808865
     5.472549832040345
     4.343101882803982
     4.19573801220097
     3.062665575467207
     5.7106325045374415
     5.697294616208039
     3.8178738159189893
     ⋮
     4.276748498380077
     3.1186438710186954
     4.31699202560427
     5.123016615230876
     4.394836819222925
     4.496289517748488
     5.3515402746558145
     5.360250154137332
     3.913043231193262

1.  We directly get the values of `λ` from `λ_samples` instead of
    looping over the indices (which is slightly slower and makes the
    line longer).

The expected value after this many simulations is the mean, or 4.69, and
the 95% confidence interval is (4.68, 4.71) (using the formula for the
standard error). The confidence interval around the expected value is
pretty tight, but let’s look at how the estimates of the mean and the
confidence interval evolve over the course of the simulations to see
what an “efficient” sample size might have been:

``` julia
# compute running estimate of the expected value and standard deviation
# first, pre-allocate memory for better efficiency
T_est = zeros(n_samples)
T_se = zeros(n_samples)
for i = 1:n_samples
    T_est[i] = mean(temp_samples[1:i])
    if i > 1
        T_se[i] = std(temp_samples[1:i]) / sqrt(i)
    end    
end

p = plot(; xlabel="Number of Samples", ylabel="Global Temperature Anomaly (°C)", legend=false)
plot!(p, 1:n_samples, T_est, ribbon=1.96 * T_se)
ylims!(p, (4.5, 5.0))
```

<?xml version="1.0" encoding="utf-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="672" height="480" viewBox="0 0 2688 1920">
<defs>
  <clipPath id="clip990">
    <rect x="0" y="0" width="2688" height="1920"/>
  </clipPath>
</defs>
<path clip-path="url(#clip990)" d="M0 1920 L2688 1920 L2688 -4.26326e-14 L0 -4.26326e-14  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip991">
    <rect x="537" y="0" width="1883" height="1883"/>
  </clipPath>
</defs>
<path clip-path="url(#clip990)" d="M226.627 1734.12 L2640.76 1734.12 L2640.76 47.2441 L226.627 47.2441  Z" fill="#ffffff" fill-rule="evenodd" fill-opacity="1"/>
<defs>
  <clipPath id="clip992">
    <rect x="226" y="47" width="2415" height="1688"/>
  </clipPath>
</defs>
<polyline clip-path="url(#clip992)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="294.723,1734.12 294.723,47.2441 "/>
<polyline clip-path="url(#clip992)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="864.15,1734.12 864.15,47.2441 "/>
<polyline clip-path="url(#clip992)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="1433.58,1734.12 1433.58,47.2441 "/>
<polyline clip-path="url(#clip992)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="2003,1734.12 2003,47.2441 "/>
<polyline clip-path="url(#clip992)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="2572.43,1734.12 2572.43,47.2441 "/>
<polyline clip-path="url(#clip990)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.627,1734.12 2640.76,1734.12 "/>
<polyline clip-path="url(#clip990)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="294.723,1734.12 294.723,1715.22 "/>
<polyline clip-path="url(#clip990)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="864.15,1734.12 864.15,1715.22 "/>
<polyline clip-path="url(#clip990)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="1433.58,1734.12 1433.58,1715.22 "/>
<polyline clip-path="url(#clip990)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2003,1734.12 2003,1715.22 "/>
<polyline clip-path="url(#clip990)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="2572.43,1734.12 2572.43,1715.22 "/>
<path clip-path="url(#clip990)" d="M294.723 1767.34 Q291.112 1767.34 289.284 1770.9 Q287.478 1774.45 287.478 1781.58 Q287.478 1788.68 289.284 1792.25 Q291.112 1795.79 294.723 1795.79 Q298.358 1795.79 300.163 1792.25 Q301.992 1788.68 301.992 1781.58 Q301.992 1774.45 300.163 1770.9 Q298.358 1767.34 294.723 1767.34 M294.723 1763.64 Q300.533 1763.64 303.589 1768.24 Q306.668 1772.83 306.668 1781.58 Q306.668 1790.3 303.589 1794.91 Q300.533 1799.49 294.723 1799.49 Q288.913 1799.49 285.834 1794.91 Q282.779 1790.3 282.779 1781.58 Q282.779 1772.83 285.834 1768.24 Q288.913 1763.64 294.723 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M812.762 1794.89 L829.081 1794.89 L829.081 1798.82 L807.137 1798.82 L807.137 1794.89 Q809.799 1792.13 814.382 1787.5 Q818.989 1782.85 820.169 1781.51 Q822.414 1778.98 823.294 1777.25 Q824.197 1775.49 824.197 1773.8 Q824.197 1771.04 822.252 1769.31 Q820.331 1767.57 817.229 1767.57 Q815.03 1767.57 812.577 1768.34 Q810.146 1769.1 807.368 1770.65 L807.368 1765.93 Q810.192 1764.79 812.646 1764.21 Q815.1 1763.64 817.137 1763.64 Q822.507 1763.64 825.701 1766.32 Q828.896 1769.01 828.896 1773.5 Q828.896 1775.63 828.086 1777.55 Q827.299 1779.45 825.192 1782.04 Q824.614 1782.71 821.512 1785.93 Q818.41 1789.12 812.762 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M838.942 1764.26 L857.299 1764.26 L857.299 1768.2 L843.225 1768.2 L843.225 1776.67 Q844.243 1776.32 845.262 1776.16 Q846.28 1775.97 847.299 1775.97 Q853.086 1775.97 856.465 1779.15 Q859.845 1782.32 859.845 1787.73 Q859.845 1793.31 856.373 1796.41 Q852.9 1799.49 846.581 1799.49 Q844.405 1799.49 842.137 1799.12 Q839.891 1798.75 837.484 1798.01 L837.484 1793.31 Q839.567 1794.45 841.789 1795 Q844.012 1795.56 846.488 1795.56 Q850.493 1795.56 852.831 1793.45 Q855.169 1791.34 855.169 1787.73 Q855.169 1784.12 852.831 1782.02 Q850.493 1779.91 846.488 1779.91 Q844.613 1779.91 842.738 1780.33 Q840.887 1780.74 838.942 1781.62 L838.942 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M879.058 1767.34 Q875.447 1767.34 873.618 1770.9 Q871.812 1774.45 871.812 1781.58 Q871.812 1788.68 873.618 1792.25 Q875.447 1795.79 879.058 1795.79 Q882.692 1795.79 884.497 1792.25 Q886.326 1788.68 886.326 1781.58 Q886.326 1774.45 884.497 1770.9 Q882.692 1767.34 879.058 1767.34 M879.058 1763.64 Q884.868 1763.64 887.923 1768.24 Q891.002 1772.83 891.002 1781.58 Q891.002 1790.3 887.923 1794.91 Q884.868 1799.49 879.058 1799.49 Q873.248 1799.49 870.169 1794.91 Q867.113 1790.3 867.113 1781.58 Q867.113 1772.83 870.169 1768.24 Q873.248 1763.64 879.058 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M909.22 1767.34 Q905.608 1767.34 903.78 1770.9 Q901.974 1774.45 901.974 1781.58 Q901.974 1788.68 903.78 1792.25 Q905.608 1795.79 909.22 1795.79 Q912.854 1795.79 914.659 1792.25 Q916.488 1788.68 916.488 1781.58 Q916.488 1774.45 914.659 1770.9 Q912.854 1767.34 909.22 1767.34 M909.22 1763.64 Q915.03 1763.64 918.085 1768.24 Q921.164 1772.83 921.164 1781.58 Q921.164 1790.3 918.085 1794.91 Q915.03 1799.49 909.22 1799.49 Q903.409 1799.49 900.331 1794.91 Q897.275 1790.3 897.275 1781.58 Q897.275 1772.83 900.331 1768.24 Q903.409 1763.64 909.22 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M1378.11 1764.26 L1396.47 1764.26 L1396.47 1768.2 L1382.4 1768.2 L1382.4 1776.67 Q1383.42 1776.32 1384.43 1776.16 Q1385.45 1775.97 1386.47 1775.97 Q1392.26 1775.97 1395.64 1779.15 Q1399.02 1782.32 1399.02 1787.73 Q1399.02 1793.31 1395.55 1796.41 Q1392.07 1799.49 1385.75 1799.49 Q1383.58 1799.49 1381.31 1799.12 Q1379.06 1798.75 1376.66 1798.01 L1376.66 1793.31 Q1378.74 1794.45 1380.96 1795 Q1383.18 1795.56 1385.66 1795.56 Q1389.67 1795.56 1392 1793.45 Q1394.34 1791.34 1394.34 1787.73 Q1394.34 1784.12 1392 1782.02 Q1389.67 1779.91 1385.66 1779.91 Q1383.79 1779.91 1381.91 1780.33 Q1380.06 1780.74 1378.11 1781.62 L1378.11 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M1418.23 1767.34 Q1414.62 1767.34 1412.79 1770.9 Q1410.98 1774.45 1410.98 1781.58 Q1410.98 1788.68 1412.79 1792.25 Q1414.62 1795.79 1418.23 1795.79 Q1421.86 1795.79 1423.67 1792.25 Q1425.5 1788.68 1425.5 1781.58 Q1425.5 1774.45 1423.67 1770.9 Q1421.86 1767.34 1418.23 1767.34 M1418.23 1763.64 Q1424.04 1763.64 1427.1 1768.24 Q1430.17 1772.83 1430.17 1781.58 Q1430.17 1790.3 1427.1 1794.91 Q1424.04 1799.49 1418.23 1799.49 Q1412.42 1799.49 1409.34 1794.91 Q1406.29 1790.3 1406.29 1781.58 Q1406.29 1772.83 1409.34 1768.24 Q1412.42 1763.64 1418.23 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M1448.39 1767.34 Q1444.78 1767.34 1442.95 1770.9 Q1441.15 1774.45 1441.15 1781.58 Q1441.15 1788.68 1442.95 1792.25 Q1444.78 1795.79 1448.39 1795.79 Q1452.03 1795.79 1453.83 1792.25 Q1455.66 1788.68 1455.66 1781.58 Q1455.66 1774.45 1453.83 1770.9 Q1452.03 1767.34 1448.39 1767.34 M1448.39 1763.64 Q1454.2 1763.64 1457.26 1768.24 Q1460.34 1772.83 1460.34 1781.58 Q1460.34 1790.3 1457.26 1794.91 Q1454.2 1799.49 1448.39 1799.49 Q1442.58 1799.49 1439.5 1794.91 Q1436.45 1790.3 1436.45 1781.58 Q1436.45 1772.83 1439.5 1768.24 Q1442.58 1763.64 1448.39 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M1478.55 1767.34 Q1474.94 1767.34 1473.11 1770.9 Q1471.31 1774.45 1471.31 1781.58 Q1471.31 1788.68 1473.11 1792.25 Q1474.94 1795.79 1478.55 1795.79 Q1482.19 1795.79 1483.99 1792.25 Q1485.82 1788.68 1485.82 1781.58 Q1485.82 1774.45 1483.99 1770.9 Q1482.19 1767.34 1478.55 1767.34 M1478.55 1763.64 Q1484.36 1763.64 1487.42 1768.24 Q1490.5 1772.83 1490.5 1781.58 Q1490.5 1790.3 1487.42 1794.91 Q1484.36 1799.49 1478.55 1799.49 Q1472.74 1799.49 1469.67 1794.91 Q1466.61 1790.3 1466.61 1781.58 Q1466.61 1772.83 1469.67 1768.24 Q1472.74 1763.64 1478.55 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M1946.2 1764.26 L1968.42 1764.26 L1968.42 1766.25 L1955.88 1798.82 L1950.99 1798.82 L1962.8 1768.2 L1946.2 1768.2 L1946.2 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M1977.59 1764.26 L1995.94 1764.26 L1995.94 1768.2 L1981.87 1768.2 L1981.87 1776.67 Q1982.89 1776.32 1983.91 1776.16 Q1984.93 1775.97 1985.94 1775.97 Q1991.73 1775.97 1995.11 1779.15 Q1998.49 1782.32 1998.49 1787.73 Q1998.49 1793.31 1995.02 1796.41 Q1991.55 1799.49 1985.23 1799.49 Q1983.05 1799.49 1980.78 1799.12 Q1978.54 1798.75 1976.13 1798.01 L1976.13 1793.31 Q1978.21 1794.45 1980.44 1795 Q1982.66 1795.56 1985.13 1795.56 Q1989.14 1795.56 1991.48 1793.45 Q1993.81 1791.34 1993.81 1787.73 Q1993.81 1784.12 1991.48 1782.02 Q1989.14 1779.91 1985.13 1779.91 Q1983.26 1779.91 1981.38 1780.33 Q1979.53 1780.74 1977.59 1781.62 L1977.59 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M2017.7 1767.34 Q2014.09 1767.34 2012.26 1770.9 Q2010.46 1774.45 2010.46 1781.58 Q2010.46 1788.68 2012.26 1792.25 Q2014.09 1795.79 2017.7 1795.79 Q2021.34 1795.79 2023.14 1792.25 Q2024.97 1788.68 2024.97 1781.58 Q2024.97 1774.45 2023.14 1770.9 Q2021.34 1767.34 2017.7 1767.34 M2017.7 1763.64 Q2023.51 1763.64 2026.57 1768.24 Q2029.65 1772.83 2029.65 1781.58 Q2029.65 1790.3 2026.57 1794.91 Q2023.51 1799.49 2017.7 1799.49 Q2011.89 1799.49 2008.81 1794.91 Q2005.76 1790.3 2005.76 1781.58 Q2005.76 1772.83 2008.81 1768.24 Q2011.89 1763.64 2017.7 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M2047.87 1767.34 Q2044.25 1767.34 2042.43 1770.9 Q2040.62 1774.45 2040.62 1781.58 Q2040.62 1788.68 2042.43 1792.25 Q2044.25 1795.79 2047.87 1795.79 Q2051.5 1795.79 2053.31 1792.25 Q2055.13 1788.68 2055.13 1781.58 Q2055.13 1774.45 2053.31 1770.9 Q2051.5 1767.34 2047.87 1767.34 M2047.87 1763.64 Q2053.68 1763.64 2056.73 1768.24 Q2059.81 1772.83 2059.81 1781.58 Q2059.81 1790.3 2056.73 1794.91 Q2053.68 1799.49 2047.87 1799.49 Q2042.06 1799.49 2038.98 1794.91 Q2035.92 1790.3 2035.92 1781.58 Q2035.92 1772.83 2038.98 1768.24 Q2042.06 1763.64 2047.87 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M2501.88 1794.89 L2509.52 1794.89 L2509.52 1768.52 L2501.21 1770.19 L2501.21 1765.93 L2509.47 1764.26 L2514.14 1764.26 L2514.14 1794.89 L2521.78 1794.89 L2521.78 1798.82 L2501.88 1798.82 L2501.88 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M2541.23 1767.34 Q2537.62 1767.34 2535.79 1770.9 Q2533.98 1774.45 2533.98 1781.58 Q2533.98 1788.68 2535.79 1792.25 Q2537.62 1795.79 2541.23 1795.79 Q2544.86 1795.79 2546.67 1792.25 Q2548.5 1788.68 2548.5 1781.58 Q2548.5 1774.45 2546.67 1770.9 Q2544.86 1767.34 2541.23 1767.34 M2541.23 1763.64 Q2547.04 1763.64 2550.09 1768.24 Q2553.17 1772.83 2553.17 1781.58 Q2553.17 1790.3 2550.09 1794.91 Q2547.04 1799.49 2541.23 1799.49 Q2535.42 1799.49 2532.34 1794.91 Q2529.28 1790.3 2529.28 1781.58 Q2529.28 1772.83 2532.34 1768.24 Q2535.42 1763.64 2541.23 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M2571.39 1767.34 Q2567.78 1767.34 2565.95 1770.9 Q2564.14 1774.45 2564.14 1781.58 Q2564.14 1788.68 2565.95 1792.25 Q2567.78 1795.79 2571.39 1795.79 Q2575.02 1795.79 2576.83 1792.25 Q2578.66 1788.68 2578.66 1781.58 Q2578.66 1774.45 2576.83 1770.9 Q2575.02 1767.34 2571.39 1767.34 M2571.39 1763.64 Q2577.2 1763.64 2580.26 1768.24 Q2583.33 1772.83 2583.33 1781.58 Q2583.33 1790.3 2580.26 1794.91 Q2577.2 1799.49 2571.39 1799.49 Q2565.58 1799.49 2562.5 1794.91 Q2559.45 1790.3 2559.45 1781.58 Q2559.45 1772.83 2562.5 1768.24 Q2565.58 1763.64 2571.39 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M2601.55 1767.34 Q2597.94 1767.34 2596.11 1770.9 Q2594.31 1774.45 2594.31 1781.58 Q2594.31 1788.68 2596.11 1792.25 Q2597.94 1795.79 2601.55 1795.79 Q2605.19 1795.79 2606.99 1792.25 Q2608.82 1788.68 2608.82 1781.58 Q2608.82 1774.45 2606.99 1770.9 Q2605.19 1767.34 2601.55 1767.34 M2601.55 1763.64 Q2607.36 1763.64 2610.42 1768.24 Q2613.5 1772.83 2613.5 1781.58 Q2613.5 1790.3 2610.42 1794.91 Q2607.36 1799.49 2601.55 1799.49 Q2595.74 1799.49 2592.66 1794.91 Q2589.61 1790.3 2589.61 1781.58 Q2589.61 1772.83 2592.66 1768.24 Q2595.74 1763.64 2601.55 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M2631.71 1767.34 Q2628.1 1767.34 2626.27 1770.9 Q2624.47 1774.45 2624.47 1781.58 Q2624.47 1788.68 2626.27 1792.25 Q2628.1 1795.79 2631.71 1795.79 Q2635.35 1795.79 2637.15 1792.25 Q2638.98 1788.68 2638.98 1781.58 Q2638.98 1774.45 2637.15 1770.9 Q2635.35 1767.34 2631.71 1767.34 M2631.71 1763.64 Q2637.52 1763.64 2640.58 1768.24 Q2643.66 1772.83 2643.66 1781.58 Q2643.66 1790.3 2640.58 1794.91 Q2637.52 1799.49 2631.71 1799.49 Q2625.9 1799.49 2622.82 1794.91 Q2619.77 1790.3 2619.77 1781.58 Q2619.77 1772.83 2622.82 1768.24 Q2625.9 1763.64 2631.71 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M1116.23 1835.78 L1124.89 1835.78 L1145.96 1875.53 L1145.96 1835.78 L1152.2 1835.78 L1152.2 1883.3 L1143.54 1883.3 L1122.47 1843.55 L1122.47 1883.3 L1116.23 1883.3 L1116.23 1835.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M1164.14 1869.23 L1164.14 1847.65 L1169.99 1847.65 L1169.99 1869.01 Q1169.99 1874.07 1171.97 1876.62 Q1173.94 1879.13 1177.89 1879.13 Q1182.63 1879.13 1185.37 1876.11 Q1188.13 1873.08 1188.13 1867.86 L1188.13 1847.65 L1193.99 1847.65 L1193.99 1883.3 L1188.13 1883.3 L1188.13 1877.83 Q1186 1881.07 1183.17 1882.66 Q1180.37 1884.22 1176.64 1884.22 Q1170.5 1884.22 1167.32 1880.4 Q1164.14 1876.59 1164.14 1869.23 M1178.87 1846.79 L1178.87 1846.79 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M1233.81 1854.5 Q1236 1850.55 1239.06 1848.67 Q1242.12 1846.79 1246.25 1846.79 Q1251.82 1846.79 1254.85 1850.71 Q1257.87 1854.59 1257.87 1861.78 L1257.87 1883.3 L1251.98 1883.3 L1251.98 1861.98 Q1251.98 1856.85 1250.17 1854.37 Q1248.35 1851.89 1244.63 1851.89 Q1240.08 1851.89 1237.44 1854.91 Q1234.79 1857.93 1234.79 1863.15 L1234.79 1883.3 L1228.91 1883.3 L1228.91 1861.98 Q1228.91 1856.82 1227.09 1854.37 Q1225.28 1851.89 1221.49 1851.89 Q1217 1851.89 1214.36 1854.94 Q1211.72 1857.97 1211.72 1863.15 L1211.72 1883.3 L1205.83 1883.3 L1205.83 1847.65 L1211.72 1847.65 L1211.72 1853.19 Q1213.72 1849.91 1216.53 1848.35 Q1219.33 1846.79 1223.18 1846.79 Q1227.06 1846.79 1229.77 1848.77 Q1232.5 1850.74 1233.81 1854.5 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M1295.14 1865.51 Q1295.14 1859.05 1292.47 1855.39 Q1289.83 1851.7 1285.18 1851.7 Q1280.53 1851.7 1277.86 1855.39 Q1275.22 1859.05 1275.22 1865.51 Q1275.22 1871.97 1277.86 1875.66 Q1280.53 1879.32 1285.18 1879.32 Q1289.83 1879.32 1292.47 1875.66 Q1295.14 1871.97 1295.14 1865.51 M1275.22 1853.06 Q1277.06 1849.88 1279.86 1848.35 Q1282.7 1846.79 1286.61 1846.79 Q1293.1 1846.79 1297.15 1851.95 Q1301.22 1857.11 1301.22 1865.51 Q1301.22 1873.91 1297.15 1879.07 Q1293.1 1884.22 1286.61 1884.22 Q1282.7 1884.22 1279.86 1882.7 Q1277.06 1881.14 1275.22 1877.95 L1275.22 1883.3 L1269.33 1883.3 L1269.33 1833.78 L1275.22 1833.78 L1275.22 1853.06 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M1341.42 1864.01 L1341.42 1866.88 L1314.49 1866.88 Q1314.88 1872.92 1318.12 1876.11 Q1321.4 1879.26 1327.22 1879.26 Q1330.6 1879.26 1333.75 1878.43 Q1336.93 1877.6 1340.05 1875.95 L1340.05 1881.49 Q1336.9 1882.82 1333.59 1883.52 Q1330.28 1884.22 1326.87 1884.22 Q1318.34 1884.22 1313.35 1879.26 Q1308.38 1874.29 1308.38 1865.83 Q1308.38 1857.07 1313.09 1851.95 Q1317.84 1846.79 1325.86 1846.79 Q1333.05 1846.79 1337.22 1851.44 Q1341.42 1856.06 1341.42 1864.01 M1335.56 1862.29 Q1335.5 1857.49 1332.86 1854.62 Q1330.25 1851.76 1325.92 1851.76 Q1321.02 1851.76 1318.06 1854.53 Q1315.13 1857.3 1314.68 1862.33 L1335.56 1862.29 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M1371.69 1853.13 Q1370.7 1852.55 1369.52 1852.3 Q1368.38 1852.01 1366.98 1852.01 Q1362.01 1852.01 1359.34 1855.26 Q1356.7 1858.47 1356.7 1864.52 L1356.7 1883.3 L1350.81 1883.3 L1350.81 1847.65 L1356.7 1847.65 L1356.7 1853.19 Q1358.54 1849.94 1361.5 1848.39 Q1364.46 1846.79 1368.7 1846.79 Q1369.3 1846.79 1370.03 1846.89 Q1370.77 1846.95 1371.66 1847.11 L1371.69 1853.13 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M1412.37 1851.76 Q1407.66 1851.76 1404.92 1855.45 Q1402.18 1859.11 1402.18 1865.51 Q1402.18 1871.91 1404.89 1875.6 Q1407.62 1879.26 1412.37 1879.26 Q1417.04 1879.26 1419.78 1875.57 Q1422.52 1871.87 1422.52 1865.51 Q1422.52 1859.17 1419.78 1855.48 Q1417.04 1851.76 1412.37 1851.76 M1412.37 1846.79 Q1420.01 1846.79 1424.37 1851.76 Q1428.73 1856.72 1428.73 1865.51 Q1428.73 1874.26 1424.37 1879.26 Q1420.01 1884.22 1412.37 1884.22 Q1404.7 1884.22 1400.33 1879.26 Q1396.01 1874.26 1396.01 1865.51 Q1396.01 1856.72 1400.33 1851.76 Q1404.7 1846.79 1412.37 1846.79 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M1456.48 1833.78 L1456.48 1838.65 L1450.88 1838.65 Q1447.73 1838.65 1446.49 1839.92 Q1445.28 1841.19 1445.28 1844.5 L1445.28 1847.65 L1454.92 1847.65 L1454.92 1852.2 L1445.28 1852.2 L1445.28 1883.3 L1439.39 1883.3 L1439.39 1852.2 L1433.79 1852.2 L1433.79 1847.65 L1439.39 1847.65 L1439.39 1845.17 Q1439.39 1839.22 1442.16 1836.51 Q1444.93 1833.78 1450.94 1833.78 L1456.48 1833.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M1510.84 1837.34 L1510.84 1843.61 Q1507.18 1841.86 1503.94 1841 Q1500.69 1840.14 1497.67 1840.14 Q1492.41 1840.14 1489.55 1842.18 Q1486.72 1844.22 1486.72 1847.97 Q1486.72 1851.12 1488.6 1852.75 Q1490.51 1854.34 1495.79 1855.32 L1499.67 1856.12 Q1506.87 1857.49 1510.27 1860.96 Q1513.71 1864.39 1513.71 1870.19 Q1513.71 1877.09 1509.06 1880.66 Q1504.45 1884.22 1495.5 1884.22 Q1492.13 1884.22 1488.31 1883.46 Q1484.52 1882.7 1480.45 1881.2 L1480.45 1874.58 Q1484.36 1876.78 1488.12 1877.89 Q1491.87 1879 1495.5 1879 Q1501.01 1879 1504 1876.84 Q1506.99 1874.68 1506.99 1870.67 Q1506.99 1867.16 1504.83 1865.19 Q1502.7 1863.22 1497.79 1862.23 L1493.88 1861.47 Q1486.69 1860.03 1483.47 1856.98 Q1480.26 1853.92 1480.26 1848.48 Q1480.26 1842.18 1484.68 1838.55 Q1489.14 1834.92 1496.93 1834.92 Q1500.28 1834.92 1503.75 1835.53 Q1507.22 1836.13 1510.84 1837.34 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M1539.68 1865.38 Q1532.58 1865.38 1529.85 1867 Q1527.11 1868.63 1527.11 1872.54 Q1527.11 1875.66 1529.15 1877.51 Q1531.21 1879.32 1534.75 1879.32 Q1539.62 1879.32 1542.54 1875.88 Q1545.5 1872.42 1545.5 1866.69 L1545.5 1865.38 L1539.68 1865.38 M1551.36 1862.96 L1551.36 1883.3 L1545.5 1883.3 L1545.5 1877.89 Q1543.5 1881.14 1540.51 1882.7 Q1537.52 1884.22 1533.19 1884.22 Q1527.71 1884.22 1524.47 1881.17 Q1521.25 1878.08 1521.25 1872.92 Q1521.25 1866.91 1525.26 1863.85 Q1529.3 1860.8 1537.29 1860.8 L1545.5 1860.8 L1545.5 1860.23 Q1545.5 1856.18 1542.83 1853.99 Q1540.19 1851.76 1535.38 1851.76 Q1532.33 1851.76 1529.43 1852.49 Q1526.54 1853.22 1523.86 1854.69 L1523.86 1849.28 Q1527.08 1848.03 1530.1 1847.43 Q1533.12 1846.79 1535.99 1846.79 Q1543.72 1846.79 1547.54 1850.8 Q1551.36 1854.81 1551.36 1862.96 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M1591.18 1854.5 Q1593.38 1850.55 1596.43 1848.67 Q1599.49 1846.79 1603.62 1846.79 Q1609.19 1846.79 1612.22 1850.71 Q1615.24 1854.59 1615.24 1861.78 L1615.24 1883.3 L1609.35 1883.3 L1609.35 1861.98 Q1609.35 1856.85 1607.54 1854.37 Q1605.72 1851.89 1602 1851.89 Q1597.45 1851.89 1594.81 1854.91 Q1592.17 1857.93 1592.17 1863.15 L1592.17 1883.3 L1586.28 1883.3 L1586.28 1861.98 Q1586.28 1856.82 1584.46 1854.37 Q1582.65 1851.89 1578.86 1851.89 Q1574.37 1851.89 1571.73 1854.94 Q1569.09 1857.97 1569.09 1863.15 L1569.09 1883.3 L1563.2 1883.3 L1563.2 1847.65 L1569.09 1847.65 L1569.09 1853.19 Q1571.1 1849.91 1573.9 1848.35 Q1576.7 1846.79 1580.55 1846.79 Q1584.43 1846.79 1587.14 1848.77 Q1589.87 1850.74 1591.18 1854.5 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M1632.59 1877.95 L1632.59 1896.86 L1626.7 1896.86 L1626.7 1847.65 L1632.59 1847.65 L1632.59 1853.06 Q1634.43 1849.88 1637.23 1848.35 Q1640.07 1846.79 1643.98 1846.79 Q1650.48 1846.79 1654.52 1851.95 Q1658.59 1857.11 1658.59 1865.51 Q1658.59 1873.91 1654.52 1879.07 Q1650.48 1884.22 1643.98 1884.22 Q1640.07 1884.22 1637.23 1882.7 Q1634.43 1881.14 1632.59 1877.95 M1652.51 1865.51 Q1652.51 1859.05 1649.84 1855.39 Q1647.2 1851.7 1642.55 1851.7 Q1637.9 1851.7 1635.23 1855.39 Q1632.59 1859.05 1632.59 1865.51 Q1632.59 1871.97 1635.23 1875.66 Q1637.9 1879.32 1642.55 1879.32 Q1647.2 1879.32 1649.84 1875.66 Q1652.51 1871.97 1652.51 1865.51 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M1668.3 1833.78 L1674.16 1833.78 L1674.16 1883.3 L1668.3 1883.3 L1668.3 1833.78 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M1716.9 1864.01 L1716.9 1866.88 L1689.97 1866.88 Q1690.36 1872.92 1693.6 1876.11 Q1696.88 1879.26 1702.71 1879.26 Q1706.08 1879.26 1709.23 1878.43 Q1712.41 1877.6 1715.53 1875.95 L1715.53 1881.49 Q1712.38 1882.82 1709.07 1883.52 Q1705.76 1884.22 1702.36 1884.22 Q1693.83 1884.22 1688.83 1879.26 Q1683.86 1874.29 1683.86 1865.83 Q1683.86 1857.07 1688.57 1851.95 Q1693.32 1846.79 1701.34 1846.79 Q1708.53 1846.79 1712.7 1851.44 Q1716.9 1856.06 1716.9 1864.01 M1711.05 1862.29 Q1710.98 1857.49 1708.34 1854.62 Q1705.73 1851.76 1701.4 1851.76 Q1696.5 1851.76 1693.54 1854.53 Q1690.61 1857.3 1690.17 1862.33 L1711.05 1862.29 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M1749.24 1848.7 L1749.24 1854.24 Q1746.76 1852.97 1744.08 1852.33 Q1741.41 1851.7 1738.54 1851.7 Q1734.18 1851.7 1731.99 1853.03 Q1729.82 1854.37 1729.82 1857.04 Q1729.82 1859.08 1731.38 1860.26 Q1732.94 1861.4 1737.65 1862.45 L1739.66 1862.9 Q1745.9 1864.24 1748.51 1866.69 Q1751.15 1869.11 1751.15 1873.47 Q1751.15 1878.43 1747.2 1881.33 Q1743.29 1884.22 1736.41 1884.22 Q1733.55 1884.22 1730.43 1883.65 Q1727.34 1883.11 1723.9 1882 L1723.9 1875.95 Q1727.15 1877.64 1730.3 1878.49 Q1733.45 1879.32 1736.54 1879.32 Q1740.68 1879.32 1742.91 1877.92 Q1745.13 1876.49 1745.13 1873.91 Q1745.13 1871.52 1743.51 1870.25 Q1741.92 1868.98 1736.48 1867.8 L1734.44 1867.32 Q1729 1866.18 1726.58 1863.82 Q1724.16 1861.43 1724.16 1857.3 Q1724.16 1852.27 1727.72 1849.53 Q1731.29 1846.79 1737.84 1846.79 Q1741.09 1846.79 1743.96 1847.27 Q1746.82 1847.75 1749.24 1848.7 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><polyline clip-path="url(#clip992)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="226.627,1734.12 2640.76,1734.12 "/>
<polyline clip-path="url(#clip992)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="226.627,1396.74 2640.76,1396.74 "/>
<polyline clip-path="url(#clip992)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="226.627,1059.37 2640.76,1059.37 "/>
<polyline clip-path="url(#clip992)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="226.627,721.993 2640.76,721.993 "/>
<polyline clip-path="url(#clip992)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="226.627,384.619 2640.76,384.619 "/>
<polyline clip-path="url(#clip992)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:2; stroke-opacity:0.1; fill:none" points="226.627,47.2441 2640.76,47.2441 "/>
<polyline clip-path="url(#clip990)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.627,1734.12 226.627,47.2441 "/>
<polyline clip-path="url(#clip990)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.627,1734.12 245.524,1734.12 "/>
<polyline clip-path="url(#clip990)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.627,1396.74 245.524,1396.74 "/>
<polyline clip-path="url(#clip990)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.627,1059.37 245.524,1059.37 "/>
<polyline clip-path="url(#clip990)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.627,721.993 245.524,721.993 "/>
<polyline clip-path="url(#clip990)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.627,384.619 245.524,384.619 "/>
<polyline clip-path="url(#clip990)" style="stroke:#000000; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="226.627,47.2441 245.524,47.2441 "/>
<path clip-path="url(#clip990)" d="M132.974 1720.91 L121.168 1739.36 L132.974 1739.36 L132.974 1720.91 M131.747 1716.84 L137.626 1716.84 L137.626 1739.36 L142.557 1739.36 L142.557 1743.25 L137.626 1743.25 L137.626 1751.4 L132.974 1751.4 L132.974 1743.25 L117.372 1743.25 L117.372 1738.74 L131.747 1716.84 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M150.288 1745.52 L155.173 1745.52 L155.173 1751.4 L150.288 1751.4 L150.288 1745.52 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M165.404 1716.84 L183.76 1716.84 L183.76 1720.77 L169.686 1720.77 L169.686 1729.24 Q170.705 1728.9 171.723 1728.74 Q172.742 1728.55 173.76 1728.55 Q179.547 1728.55 182.927 1731.72 Q186.307 1734.89 186.307 1740.31 Q186.307 1745.89 182.834 1748.99 Q179.362 1752.07 173.043 1752.07 Q170.867 1752.07 168.598 1751.7 Q166.353 1751.33 163.946 1750.59 L163.946 1745.89 Q166.029 1747.02 168.251 1747.58 Q170.473 1748.13 172.95 1748.13 Q176.955 1748.13 179.293 1746.03 Q181.631 1743.92 181.631 1740.31 Q181.631 1736.7 179.293 1734.59 Q176.955 1732.49 172.95 1732.49 Q171.075 1732.49 169.2 1732.9 Q167.348 1733.32 165.404 1734.2 L165.404 1716.84 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M131.816 1383.54 L120.011 1401.99 L131.816 1401.99 L131.816 1383.54 M130.589 1379.46 L136.469 1379.46 L136.469 1401.99 L141.399 1401.99 L141.399 1405.87 L136.469 1405.87 L136.469 1414.02 L131.816 1414.02 L131.816 1405.87 L116.214 1405.87 L116.214 1401.36 L130.589 1379.46 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M149.131 1408.14 L154.015 1408.14 L154.015 1414.02 L149.131 1414.02 L149.131 1408.14 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M174.779 1394.88 Q171.631 1394.88 169.779 1397.03 Q167.95 1399.18 167.95 1402.93 Q167.95 1406.66 169.779 1408.84 Q171.631 1410.99 174.779 1410.99 Q177.927 1410.99 179.756 1408.84 Q181.608 1406.66 181.608 1402.93 Q181.608 1399.18 179.756 1397.03 Q177.927 1394.88 174.779 1394.88 M184.061 1380.23 L184.061 1384.49 Q182.302 1383.65 180.497 1383.21 Q178.714 1382.77 176.955 1382.77 Q172.325 1382.77 169.872 1385.9 Q167.441 1389.02 167.094 1395.34 Q168.46 1393.33 170.52 1392.26 Q172.58 1391.18 175.057 1391.18 Q180.265 1391.18 183.274 1394.35 Q186.307 1397.49 186.307 1402.93 Q186.307 1408.26 183.159 1411.48 Q180.01 1414.69 174.779 1414.69 Q168.784 1414.69 165.612 1410.11 Q162.441 1405.5 162.441 1396.78 Q162.441 1388.58 166.33 1383.72 Q170.219 1378.84 176.77 1378.84 Q178.529 1378.84 180.311 1379.18 Q182.117 1379.53 184.061 1380.23 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M132.881 1046.16 L121.076 1064.61 L132.881 1064.61 L132.881 1046.16 M131.654 1042.09 L137.534 1042.09 L137.534 1064.61 L142.464 1064.61 L142.464 1068.5 L137.534 1068.5 L137.534 1076.65 L132.881 1076.65 L132.881 1068.5 L117.279 1068.5 L117.279 1063.99 L131.654 1042.09 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M150.196 1070.77 L155.08 1070.77 L155.08 1076.65 L150.196 1076.65 L150.196 1070.77 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M164.085 1042.09 L186.307 1042.09 L186.307 1044.08 L173.76 1076.65 L168.876 1076.65 L180.682 1046.02 L164.085 1046.02 L164.085 1042.09 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M132.071 708.787 L120.265 727.236 L132.071 727.236 L132.071 708.787 M130.844 704.713 L136.724 704.713 L136.724 727.236 L141.654 727.236 L141.654 731.125 L136.724 731.125 L136.724 739.273 L132.071 739.273 L132.071 731.125 L116.469 731.125 L116.469 726.611 L130.844 704.713 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M149.386 733.394 L154.27 733.394 L154.27 739.273 L149.386 739.273 L149.386 733.394 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M174.455 722.861 Q171.122 722.861 169.2 724.644 Q167.302 726.426 167.302 729.551 Q167.302 732.676 169.2 734.458 Q171.122 736.241 174.455 736.241 Q177.788 736.241 179.709 734.458 Q181.631 732.653 181.631 729.551 Q181.631 726.426 179.709 724.644 Q177.811 722.861 174.455 722.861 M169.779 720.871 Q166.77 720.13 165.08 718.07 Q163.413 716.009 163.413 713.047 Q163.413 708.903 166.353 706.496 Q169.316 704.088 174.455 704.088 Q179.617 704.088 182.557 706.496 Q185.496 708.903 185.496 713.047 Q185.496 716.009 183.807 718.07 Q182.14 720.13 179.154 720.871 Q182.534 721.658 184.409 723.949 Q186.307 726.241 186.307 729.551 Q186.307 734.574 183.228 737.259 Q180.172 739.945 174.455 739.945 Q168.737 739.945 165.659 737.259 Q162.603 734.574 162.603 729.551 Q162.603 726.241 164.501 723.949 Q166.399 721.658 169.779 720.871 M168.066 713.486 Q168.066 716.172 169.733 717.676 Q171.422 719.181 174.455 719.181 Q177.464 719.181 179.154 717.676 Q180.867 716.172 180.867 713.486 Q180.867 710.801 179.154 709.297 Q177.464 707.792 174.455 707.792 Q171.422 707.792 169.733 709.297 Q168.066 710.801 168.066 713.486 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M132.163 371.413 L120.358 389.862 L132.163 389.862 L132.163 371.413 M130.937 367.339 L136.816 367.339 L136.816 389.862 L141.747 389.862 L141.747 393.751 L136.816 393.751 L136.816 401.899 L132.163 401.899 L132.163 393.751 L116.562 393.751 L116.562 389.237 L130.937 367.339 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M149.478 396.019 L154.362 396.019 L154.362 401.899 L149.478 401.899 L149.478 396.019 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M164.686 401.181 L164.686 396.922 Q166.446 397.755 168.251 398.195 Q170.057 398.635 171.793 398.635 Q176.422 398.635 178.853 395.533 Q181.307 392.408 181.654 386.065 Q180.311 388.056 178.251 389.121 Q176.191 390.186 173.691 390.186 Q168.506 390.186 165.473 387.061 Q162.464 383.913 162.464 378.473 Q162.464 373.149 165.612 369.931 Q168.76 366.714 173.992 366.714 Q179.987 366.714 183.135 371.32 Q186.307 375.903 186.307 384.653 Q186.307 392.825 182.418 397.709 Q178.552 402.57 172.001 402.57 Q170.242 402.57 168.436 402.223 Q166.631 401.876 164.686 401.181 M173.992 386.528 Q177.14 386.528 178.969 384.376 Q180.821 382.223 180.821 378.473 Q180.821 374.746 178.969 372.593 Q177.14 370.417 173.992 370.417 Q170.844 370.417 168.992 372.593 Q167.163 374.746 167.163 378.473 Q167.163 382.223 168.992 384.376 Q170.844 386.528 173.992 386.528 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M119.177 29.9641 L137.534 29.9641 L137.534 33.8993 L123.46 33.8993 L123.46 42.3714 Q124.478 42.0242 125.497 41.8622 Q126.515 41.677 127.534 41.677 Q133.321 41.677 136.7 44.8483 Q140.08 48.0196 140.08 53.4362 Q140.08 59.0149 136.608 62.1167 Q133.136 65.1954 126.816 65.1954 Q124.64 65.1954 122.372 64.825 Q120.126 64.4547 117.719 63.7139 L117.719 59.0149 Q119.802 60.1491 122.025 60.7047 Q124.247 61.2602 126.724 61.2602 Q130.728 61.2602 133.066 59.1538 Q135.404 57.0473 135.404 53.4362 Q135.404 49.8251 133.066 47.7186 Q130.728 45.6122 126.724 45.6122 Q124.849 45.6122 122.974 46.0288 Q121.122 46.4455 119.177 47.3251 L119.177 29.9641 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M149.293 58.6445 L154.177 58.6445 L154.177 64.5241 L149.293 64.5241 L149.293 58.6445 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M174.362 33.0428 Q170.751 33.0428 168.922 36.6076 Q167.117 40.1492 167.117 47.2788 Q167.117 54.3853 168.922 57.9501 Q170.751 61.4917 174.362 61.4917 Q177.997 61.4917 179.802 57.9501 Q181.631 54.3853 181.631 47.2788 Q181.631 40.1492 179.802 36.6076 Q177.997 33.0428 174.362 33.0428 M174.362 29.3391 Q180.172 29.3391 183.228 33.9456 Q186.307 38.5289 186.307 47.2788 Q186.307 56.0056 183.228 60.6121 Q180.172 65.1954 174.362 65.1954 Q168.552 65.1954 165.473 60.6121 Q162.418 56.0056 162.418 47.2788 Q162.418 38.5289 165.473 33.9456 Q168.552 29.3391 174.362 29.3391 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M54.859 1399.86 L42.0957 1399.86 L42.0957 1410.36 L36.8122 1410.36 L36.8122 1393.49 L57.2143 1393.49 Q59.856 1397.22 61.2247 1401.7 Q62.5615 1406.19 62.5615 1411.28 Q62.5615 1422.42 56.0684 1428.73 Q49.5436 1435 37.9262 1435 Q26.2769 1435 19.7839 1428.73 Q13.2591 1422.42 13.2591 1411.28 Q13.2591 1406.64 14.4049 1402.47 Q15.5507 1398.27 17.7787 1394.73 L24.6219 1394.73 Q21.5981 1398.3 20.0704 1402.31 Q18.5426 1406.32 18.5426 1410.74 Q18.5426 1419.46 23.4124 1423.86 Q28.2821 1428.22 37.9262 1428.22 Q47.5384 1428.22 52.4082 1423.86 Q57.2779 1419.46 57.2779 1410.74 Q57.2779 1407.34 56.705 1404.66 Q56.1003 1401.99 54.859 1399.86 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M12.1132 1382 L12.1132 1376.15 L61.6384 1376.15 L61.6384 1382 L12.1132 1382 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M30.0964 1350.08 Q30.0964 1354.79 33.7885 1357.53 Q37.4488 1360.26 43.8463 1360.26 Q50.2438 1360.26 53.9359 1357.56 Q57.5962 1354.82 57.5962 1350.08 Q57.5962 1345.4 53.9041 1342.66 Q50.212 1339.92 43.8463 1339.92 Q37.5124 1339.92 33.8203 1342.66 Q30.0964 1345.4 30.0964 1350.08 M25.1311 1350.08 Q25.1311 1342.44 30.0964 1338.08 Q35.0616 1333.72 43.8463 1333.72 Q52.5991 1333.72 57.5962 1338.08 Q62.5615 1342.44 62.5615 1350.08 Q62.5615 1357.75 57.5962 1362.11 Q52.5991 1366.44 43.8463 1366.44 Q35.0616 1366.44 30.0964 1362.11 Q25.1311 1357.75 25.1311 1350.08 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M43.8463 1298.42 Q37.3851 1298.42 33.7248 1301.09 Q30.0327 1303.74 30.0327 1308.38 Q30.0327 1313.03 33.7248 1315.7 Q37.3851 1318.34 43.8463 1318.34 Q50.3075 1318.34 53.9996 1315.7 Q57.6599 1313.03 57.6599 1308.38 Q57.6599 1303.74 53.9996 1301.09 Q50.3075 1298.42 43.8463 1298.42 M31.4013 1318.34 Q28.2185 1316.5 26.6907 1313.7 Q25.1311 1310.86 25.1311 1306.95 Q25.1311 1300.46 30.2873 1296.41 Q35.4436 1292.34 43.8463 1292.34 Q52.249 1292.34 57.4052 1296.41 Q62.5615 1300.46 62.5615 1306.95 Q62.5615 1310.86 61.0337 1313.7 Q59.4741 1316.5 56.2912 1318.34 L61.6384 1318.34 L61.6384 1324.23 L12.1132 1324.23 L12.1132 1318.34 L31.4013 1318.34 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M43.719 1266.43 Q43.719 1273.53 45.3422 1276.27 Q46.9655 1279 50.8804 1279 Q53.9996 1279 55.8457 1276.97 Q57.6599 1274.9 57.6599 1271.37 Q57.6599 1266.5 54.2224 1263.57 Q50.7531 1260.61 45.0239 1260.61 L43.719 1260.61 L43.719 1266.43 M41.3 1254.75 L61.6384 1254.75 L61.6384 1260.61 L56.2276 1260.61 Q59.4741 1262.61 61.0337 1265.6 Q62.5615 1268.6 62.5615 1272.93 Q62.5615 1278.4 59.5059 1281.65 Q56.4186 1284.86 51.2623 1284.86 Q45.2467 1284.86 42.1912 1280.85 Q39.1357 1276.81 39.1357 1268.82 L39.1357 1260.61 L38.5628 1260.61 Q34.5205 1260.61 32.3244 1263.28 Q30.0964 1265.92 30.0964 1270.73 Q30.0964 1273.78 30.8284 1276.68 Q31.5605 1279.58 33.0246 1282.25 L27.6137 1282.25 Q26.3724 1279.04 25.7677 1276.01 Q25.1311 1272.99 25.1311 1270.12 Q25.1311 1262.39 29.1415 1258.57 Q33.1519 1254.75 41.3 1254.75 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M12.1132 1242.69 L12.1132 1236.83 L61.6384 1236.83 L61.6384 1242.69 L12.1132 1242.69 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M14.1184 1210.19 L14.1184 1169.99 L19.5293 1169.99 L19.5293 1186.86 L61.6384 1186.86 L61.6384 1193.32 L19.5293 1193.32 L19.5293 1210.19 L14.1184 1210.19 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M42.3504 1144.62 L45.2149 1144.62 L45.2149 1171.55 Q51.2623 1171.17 54.4452 1167.92 Q57.5962 1164.64 57.5962 1158.82 Q57.5962 1155.45 56.7687 1152.3 Q55.9411 1149.11 54.2861 1145.99 L59.8242 1145.99 Q61.161 1149.14 61.8612 1152.45 Q62.5615 1155.76 62.5615 1159.17 Q62.5615 1167.7 57.5962 1172.7 Q52.631 1177.66 44.1646 1177.66 Q35.4117 1177.66 30.2873 1172.95 Q25.1311 1168.21 25.1311 1160.19 Q25.1311 1153 29.7781 1148.83 Q34.3932 1144.62 42.3504 1144.62 M40.6316 1150.48 Q35.8255 1150.54 32.9609 1153.19 Q30.0964 1155.8 30.0964 1160.12 Q30.0964 1165.03 32.8654 1167.99 Q35.6345 1170.91 40.6634 1171.36 L40.6316 1150.48 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M32.8336 1107.26 Q28.8869 1105.06 27.009 1102.01 Q25.1311 1098.95 25.1311 1094.81 Q25.1311 1089.24 29.046 1086.22 Q32.9291 1083.2 40.1224 1083.2 L61.6384 1083.2 L61.6384 1089.08 L40.3133 1089.08 Q35.1889 1089.08 32.7063 1090.9 Q30.2237 1092.71 30.2237 1096.44 Q30.2237 1100.99 33.2474 1103.63 Q36.2711 1106.27 41.491 1106.27 L61.6384 1106.27 L61.6384 1112.16 L40.3133 1112.16 Q35.1571 1112.16 32.7063 1113.97 Q30.2237 1115.79 30.2237 1119.58 Q30.2237 1124.06 33.2792 1126.71 Q36.3029 1129.35 41.491 1129.35 L61.6384 1129.35 L61.6384 1135.24 L25.9905 1135.24 L25.9905 1129.35 L31.5287 1129.35 Q28.2503 1127.34 26.6907 1124.54 Q25.1311 1121.74 25.1311 1117.89 Q25.1311 1114.01 27.1045 1111.3 Q29.0779 1108.56 32.8336 1107.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M56.2912 1065.85 L75.1974 1065.85 L75.1974 1071.74 L25.9905 1071.74 L25.9905 1065.85 L31.4013 1065.85 Q28.2185 1064 26.6907 1061.2 Q25.1311 1058.37 25.1311 1054.45 Q25.1311 1047.96 30.2873 1043.92 Q35.4436 1039.84 43.8463 1039.84 Q52.249 1039.84 57.4052 1043.92 Q62.5615 1047.96 62.5615 1054.45 Q62.5615 1058.37 61.0337 1061.2 Q59.4741 1064 56.2912 1065.85 M43.8463 1045.92 Q37.3851 1045.92 33.7248 1048.6 Q30.0327 1051.24 30.0327 1055.89 Q30.0327 1060.53 33.7248 1063.21 Q37.3851 1065.85 43.8463 1065.85 Q50.3075 1065.85 53.9996 1063.21 Q57.6599 1060.53 57.6599 1055.89 Q57.6599 1051.24 53.9996 1048.6 Q50.3075 1045.92 43.8463 1045.92 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M42.3504 999.646 L45.2149 999.646 L45.2149 1026.57 Q51.2623 1026.19 54.4452 1022.94 Q57.5962 1019.67 57.5962 1013.84 Q57.5962 1010.47 56.7687 1007.32 Q55.9411 1004.13 54.2861 1001.01 L59.8242 1001.01 Q61.161 1004.17 61.8612 1007.48 Q62.5615 1010.79 62.5615 1014.19 Q62.5615 1022.72 57.5962 1027.72 Q52.631 1032.68 44.1646 1032.68 Q35.4117 1032.68 30.2873 1027.97 Q25.1311 1023.23 25.1311 1015.21 Q25.1311 1008.02 29.7781 1003.85 Q34.3932 999.646 42.3504 999.646 M40.6316 1005.5 Q35.8255 1005.57 32.9609 1008.21 Q30.0964 1010.82 30.0964 1015.15 Q30.0964 1020.05 32.8654 1023.01 Q35.6345 1025.94 40.6634 1026.38 L40.6316 1005.5 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M31.465 969.377 Q30.8921 970.363 30.6375 971.541 Q30.351 972.687 30.351 974.087 Q30.351 979.052 33.5975 981.726 Q36.8122 984.368 42.8596 984.368 L61.6384 984.368 L61.6384 990.256 L25.9905 990.256 L25.9905 984.368 L31.5287 984.368 Q28.2821 982.522 26.7225 979.562 Q25.1311 976.602 25.1311 972.368 Q25.1311 971.764 25.2266 971.032 Q25.2903 970.3 25.4494 969.408 L31.465 969.377 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M43.719 947.033 Q43.719 954.131 45.3422 956.868 Q46.9655 959.605 50.8804 959.605 Q53.9996 959.605 55.8457 957.568 Q57.6599 955.499 57.6599 951.966 Q57.6599 947.097 54.2224 944.168 Q50.7531 941.208 45.0239 941.208 L43.719 941.208 L43.719 947.033 M41.3 935.352 L61.6384 935.352 L61.6384 941.208 L56.2276 941.208 Q59.4741 943.214 61.0337 946.205 Q62.5615 949.197 62.5615 953.526 Q62.5615 959 59.5059 962.247 Q56.4186 965.462 51.2623 965.462 Q45.2467 965.462 42.1912 961.451 Q39.1357 957.409 39.1357 949.42 L39.1357 941.208 L38.5628 941.208 Q34.5205 941.208 32.3244 943.882 Q30.0964 946.524 30.0964 951.33 Q30.0964 954.385 30.8284 957.282 Q31.5605 960.178 33.0246 962.852 L27.6137 962.852 Q26.3724 959.637 25.7677 956.613 Q25.1311 953.59 25.1311 950.725 Q25.1311 942.991 29.1415 939.171 Q33.1519 935.352 41.3 935.352 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M15.869 917.496 L25.9905 917.496 L25.9905 905.433 L30.542 905.433 L30.542 917.496 L49.8937 917.496 Q54.2542 917.496 55.4955 916.318 Q56.7368 915.109 56.7368 911.449 L56.7368 905.433 L61.6384 905.433 L61.6384 911.449 Q61.6384 918.228 59.124 920.806 Q56.5777 923.384 49.8937 923.384 L30.542 923.384 L30.542 927.681 L25.9905 927.681 L25.9905 923.384 L15.869 923.384 L15.869 917.496 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M47.5702 898.335 L25.9905 898.335 L25.9905 892.479 L47.3474 892.479 Q52.4082 892.479 54.9545 890.505 Q57.4689 888.532 57.4689 884.585 Q57.4689 879.843 54.4452 877.106 Q51.4215 874.337 46.2016 874.337 L25.9905 874.337 L25.9905 868.48 L61.6384 868.48 L61.6384 874.337 L56.1639 874.337 Q59.4104 876.469 61.0019 879.302 Q62.5615 882.103 62.5615 885.827 Q62.5615 891.97 58.742 895.152 Q54.9226 898.335 47.5702 898.335 M25.1311 883.599 L25.1311 883.599 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M31.465 835.76 Q30.8921 836.747 30.6375 837.925 Q30.351 839.071 30.351 840.471 Q30.351 845.436 33.5975 848.11 Q36.8122 850.752 42.8596 850.752 L61.6384 850.752 L61.6384 856.64 L25.9905 856.64 L25.9905 850.752 L31.5287 850.752 Q28.2821 848.906 26.7225 845.946 Q25.1311 842.985 25.1311 838.752 Q25.1311 838.148 25.2266 837.415 Q25.2903 836.683 25.4494 835.792 L31.465 835.76 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M42.3504 800.558 L45.2149 800.558 L45.2149 827.485 Q51.2623 827.103 54.4452 823.857 Q57.5962 820.578 57.5962 814.754 Q57.5962 811.38 56.7687 808.229 Q55.9411 805.046 54.2861 801.927 L59.8242 801.927 Q61.161 805.078 61.8612 808.388 Q62.5615 811.698 62.5615 815.104 Q62.5615 823.634 57.5962 828.631 Q52.631 833.596 44.1646 833.596 Q35.4117 833.596 30.2873 828.885 Q25.1311 824.143 25.1311 816.122 Q25.1311 808.929 29.7781 804.759 Q34.3932 800.558 42.3504 800.558 M40.6316 806.414 Q35.8255 806.478 32.9609 809.12 Q30.0964 811.73 30.0964 816.059 Q30.0964 820.96 32.8654 823.92 Q35.6345 826.848 40.6634 827.294 L40.6316 806.414 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M20.4523 754.088 L44.1009 762.809 L44.1009 745.336 L20.4523 754.088 M14.1184 757.717 L14.1184 750.428 L61.6384 732.318 L61.6384 739.002 L49.4481 743.33 L49.4481 764.751 L61.6384 769.08 L61.6384 775.859 L14.1184 757.717 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M40.1224 696.001 L61.6384 696.001 L61.6384 701.858 L40.3133 701.858 Q35.2526 701.858 32.7381 703.831 Q30.2237 705.805 30.2237 709.751 Q30.2237 714.494 33.2474 717.231 Q36.2711 719.968 41.491 719.968 L61.6384 719.968 L61.6384 725.856 L25.9905 725.856 L25.9905 719.968 L31.5287 719.968 Q28.314 717.868 26.7225 715.035 Q25.1311 712.17 25.1311 708.446 Q25.1311 702.303 28.9505 699.152 Q32.7381 696.001 40.1224 696.001 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M30.0964 670.507 Q30.0964 675.217 33.7885 677.955 Q37.4488 680.692 43.8463 680.692 Q50.2438 680.692 53.9359 677.986 Q57.5962 675.249 57.5962 670.507 Q57.5962 665.828 53.9041 663.091 Q50.212 660.353 43.8463 660.353 Q37.5124 660.353 33.8203 663.091 Q30.0964 665.828 30.0964 670.507 M25.1311 670.507 Q25.1311 662.868 30.0964 658.507 Q35.0616 654.147 43.8463 654.147 Q52.5991 654.147 57.5962 658.507 Q62.5615 662.868 62.5615 670.507 Q62.5615 678.177 57.5962 682.538 Q52.5991 686.867 43.8463 686.867 Q35.0616 686.867 30.0964 682.538 Q25.1311 678.177 25.1311 670.507 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M32.8336 616.685 Q28.8869 614.488 27.009 611.433 Q25.1311 608.377 25.1311 604.24 Q25.1311 598.67 29.046 595.646 Q32.9291 592.622 40.1224 592.622 L61.6384 592.622 L61.6384 598.511 L40.3133 598.511 Q35.1889 598.511 32.7063 600.325 Q30.2237 602.139 30.2237 605.863 Q30.2237 610.414 33.2474 613.056 Q36.2711 615.698 41.491 615.698 L61.6384 615.698 L61.6384 621.586 L40.3133 621.586 Q35.1571 621.586 32.7063 623.4 Q30.2237 625.215 30.2237 629.002 Q30.2237 633.49 33.2792 636.132 Q36.3029 638.774 41.491 638.774 L61.6384 638.774 L61.6384 644.662 L25.9905 644.662 L25.9905 638.774 L31.5287 638.774 Q28.2503 636.768 26.6907 633.968 Q25.1311 631.167 25.1311 627.315 Q25.1311 623.432 27.1045 620.727 Q29.0779 617.99 32.8336 616.685 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M43.719 564.74 Q43.719 571.838 45.3422 574.575 Q46.9655 577.313 50.8804 577.313 Q53.9996 577.313 55.8457 575.276 Q57.6599 573.207 57.6599 569.674 Q57.6599 564.804 54.2224 561.876 Q50.7531 558.916 45.0239 558.916 L43.719 558.916 L43.719 564.74 M41.3 553.059 L61.6384 553.059 L61.6384 558.916 L56.2276 558.916 Q59.4741 560.921 61.0337 563.913 Q62.5615 566.905 62.5615 571.233 Q62.5615 576.708 59.5059 579.954 Q56.4186 583.169 51.2623 583.169 Q45.2467 583.169 42.1912 579.159 Q39.1357 575.117 39.1357 567.128 L39.1357 558.916 L38.5628 558.916 Q34.5205 558.916 32.3244 561.589 Q30.0964 564.231 30.0964 569.037 Q30.0964 572.093 30.8284 574.989 Q31.5605 577.886 33.0246 580.559 L27.6137 580.559 Q26.3724 577.345 25.7677 574.321 Q25.1311 571.297 25.1311 568.433 Q25.1311 560.698 29.1415 556.879 Q33.1519 553.059 41.3 553.059 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M12.1132 540.996 L12.1132 535.14 L61.6384 535.14 L61.6384 540.996 L12.1132 540.996 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M64.9486 508.054 Q71.3143 510.536 73.2559 512.892 Q75.1974 515.247 75.1974 519.194 L75.1974 523.873 L70.2958 523.873 L70.2958 520.435 Q70.2958 518.016 69.15 516.679 Q68.0041 515.343 63.7391 513.719 L61.0655 512.669 L25.9905 527.087 L25.9905 520.881 L53.8723 509.741 L25.9905 498.601 L25.9905 492.394 L64.9486 508.054 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M12.1769 449.521 Q19.4975 453.786 26.6589 455.855 Q33.8203 457.924 41.1727 457.924 Q48.5251 457.924 55.7502 455.855 Q62.9434 453.754 70.2321 449.521 L70.2321 454.614 Q62.7524 459.388 55.5274 461.775 Q48.3023 464.13 41.1727 464.13 Q34.0749 464.13 26.8817 461.775 Q19.6884 459.42 12.1769 454.614 L12.1769 449.521 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M17.365 428.005 Q17.365 430.551 19.1474 432.302 Q20.8979 434.052 23.4442 434.052 Q25.9587 434.052 27.7092 432.302 Q29.428 430.551 29.428 428.005 Q29.428 425.459 27.7092 423.708 Q25.9587 421.958 23.4442 421.958 Q20.9297 421.958 19.1474 423.74 Q17.365 425.491 17.365 428.005 M13.2591 428.005 Q13.2591 425.968 14.0548 424.09 Q14.8187 422.212 16.2828 420.844 Q17.7151 419.38 19.5293 418.647 Q21.3435 417.915 23.4442 417.915 Q27.6456 417.915 30.5738 420.875 Q33.4702 423.804 33.4702 428.069 Q33.4702 432.366 30.6056 435.23 Q27.7411 438.095 23.4442 438.095 Q19.1792 438.095 16.2191 435.166 Q13.2591 432.238 13.2591 428.005 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M17.7787 369.727 L24.5582 369.727 Q21.5345 372.974 20.0386 376.666 Q18.5426 380.326 18.5426 384.464 Q18.5426 392.612 23.5397 396.94 Q28.5049 401.269 37.9262 401.269 Q47.3156 401.269 52.3127 396.94 Q57.2779 392.612 57.2779 384.464 Q57.2779 380.326 55.782 376.666 Q54.2861 372.974 51.2623 369.727 L57.9782 369.727 Q60.2698 373.101 61.4156 376.888 Q62.5615 380.644 62.5615 384.846 Q62.5615 395.635 55.973 401.842 Q49.3526 408.049 37.9262 408.049 Q26.4679 408.049 19.8794 401.842 Q13.2591 395.635 13.2591 384.846 Q13.2591 380.581 14.4049 376.825 Q15.5189 373.037 17.7787 369.727 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip990)" d="M12.1769 360.974 L12.1769 355.882 Q19.6884 351.107 26.8817 348.752 Q34.0749 346.365 41.1727 346.365 Q48.3023 346.365 55.5274 348.752 Q62.7524 351.107 70.2321 355.882 L70.2321 360.974 Q62.9434 356.741 55.7502 354.672 Q48.5251 352.571 41.1727 352.571 Q33.8203 352.571 26.6589 354.672 Q19.4975 356.741 12.1769 360.974 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip992)" d="M294.951 1945.39 L295.179 2098.21 L295.407 3326.76 L295.634 3359.49 L295.862 3130.35 L296.09 3102.48 L296.318 4210.02 L296.545 3766.26 L296.773 3300.38 L297.001 3440.47 L297.229 3097.36 L297.457 2951.58 L297.684 2766.55 L297.912 2828.18 L298.14 2559.19 L298.368 2301.18 L298.595 2247.68 L298.823 2219.43 L299.051 2118.48 L299.279 2217.8 L299.506 2251.97 L299.734 2190.1 L299.962 2353.2 L300.19 2504.17 L300.418 2332.89 L300.645 2155.26 L300.873 2035.68 L301.101 1986.29 L301.329 1831.33 L301.556 1834.33 L301.784 1876.91 L302.012 1754.19 L302.24 1865.6 L302.468 1833.63 L302.695 1790.16 L302.923 1830.01 L303.151 1824.21 L303.379 1832.46 L303.606 1793.91 L303.834 1871.65 L304.062 1847.15 L304.29 1855.58 L304.517 1986.86 L304.745 1995.08 L304.973 2012.3 L305.201 2092.21 L305.429 2118.22 L305.656 2039.98 L305.884 2052.19 L306.112 2038.55 L306.34 2021.26 L306.567 1976.1 L306.795 1930.5 L307.023 1918.45 L307.251 1873.07 L307.478 1838.75 L307.706 1762.71 L307.934 1800.59 L308.162 1754.67 L308.39 1762.01 L308.617 1748.12 L308.845 1672.84 L309.073 1674.69 L309.301 1664.13 L309.528 1592.91 L309.756 1560.44 L309.984 1546.87 L310.212 1540.59 L310.439 1602.23 L310.667 1570.65 L310.895 1570.97 L311.123 1561.58 L311.351 1539.75 L311.578 1567.19 L311.806 1579.76 L312.034 1570.54 L312.262 1649.13 L312.489 1671.6 L312.717 1653.63 L312.945 1672.61 L313.173 1712.89 L313.401 1727.55 L313.628 1687 L313.856 1722.18 L314.084 1674.81 L314.312 1671.47 L314.539 1646.6 L314.767 1662.95 L314.995 1610.37 L315.223 1607.49 L315.45 1607.48 L315.678 1589.47 L315.906 1533.27 L316.134 1506.68 L316.362 1452.69 L316.589 1436.4 L316.817 1475.38 L317.045 1472.2 L317.273 1471.98 L317.5 1452.58 L317.728 1469.17 L317.956 1496.15 L318.184 1526.3 L318.411 1514.83 L318.639 1521.11 L318.867 1519.72 L319.095 1520.37 L319.323 1496.85 L319.55 1491.5 L319.778 1469.03 L320.006 1463.41 L320.234 1455.78 L320.461 1443.65 L320.689 1431.12 L320.917 1414.91 L321.145 1391.5 L321.372 1381.67 L321.6 1389.51 L321.828 1399.02 L322.056 1379.21 L322.284 1407.31 L322.511 1434.16 L322.739 1472.72 L322.967 1508.19 L323.195 1491.93 L323.422 1449.5 L323.65 1414.5 L323.878 1425.25 L324.106 1372.5 L324.334 1381.7 L324.561 1395.81 L324.789 1384.71 L325.017 1376.09 L325.245 1379.89 L325.472 1383.13 L325.7 1382.79 L325.928 1385.06 L326.156 1357.81 L326.383 1369.28 L326.611 1368.04 L326.839 1378.4 L327.067 1358.11 L327.295 1368.4 L327.522 1381.73 L327.75 1391.67 L327.978 1408.98 L328.206 1401.87 L328.433 1370.82 L328.661 1378.76 L328.889 1380.07 L329.117 1406.34 L329.344 1400.68 L329.572 1416.15 L329.8 1398.76 L330.028 1391.46 L330.256 1371.12 L330.483 1349.98 L330.711 1339.29 L330.939 1357.49 L331.167 1351.54 L331.394 1353.12 L331.622 1327.5 L331.85 1348.06 L332.078 1349.68 L332.305 1337.28 L332.533 1355.56 L332.761 1358.82 L332.989 1383.27 L333.217 1390.66 L333.444 1368.24 L333.672 1366.55 L333.9 1365.81 L334.128 1347.52 L334.355 1354.86 L334.583 1360.02 L334.811 1336.94 L335.039 1327.73 L335.267 1310.62 L335.494 1308.7 L335.722 1290.21 L335.95 1280.69 L336.178 1265.47 L336.405 1253.61 L336.633 1275.75 L336.861 1261.53 L337.089 1260.45 L337.316 1234.37 L337.544 1254.12 L337.772 1248.05 L338 1238.56 L338.228 1254.92 L338.455 1255.16 L338.683 1257.76 L338.911 1256.01 L339.139 1263.11 L339.366 1274.97 L339.594 1281.8 L339.822 1291.43 L340.05 1275.75 L340.277 1278.71 L340.505 1265.85 L340.733 1239.69 L340.961 1238.6 L341.189 1244.77 L341.416 1231.2 L341.644 1248.05 L341.872 1244.2 L342.1 1236.05 L342.327 1247.67 L342.555 1261.68 L342.783 1244.59 L343.011 1237.1 L343.238 1244.01 L343.466 1221 L343.694 1221.63 L343.922 1202.79 L344.15 1182.31 L344.377 1189.31 L344.605 1177.14 L344.833 1181.02 L345.061 1200.28 L345.288 1193.32 L345.516 1205.78 L345.744 1216.42 L345.972 1221.57 L346.2 1224.93 L346.427 1220.77 L346.655 1212.66 L346.883 1205.58 L347.111 1219.19 L347.338 1216.59 L347.566 1254.3 L347.794 1268.18 L348.022 1279.43 L348.249 1287.68 L348.477 1288.62 L348.705 1278.43 L348.933 1290.76 L349.161 1287.2 L349.388 1280.79 L349.616 1261.38 L349.844 1259.52 L350.072 1270.23 L350.299 1262.96 L350.527 1230.96 L350.755 1221.62 L350.983 1215.28 L351.21 1208.51 L351.438 1206.76 L351.666 1221.92 L351.894 1225.28 L352.122 1221.84 L352.349 1219.47 L352.577 1206.05 L352.805 1204.83 L353.033 1209.03 L353.26 1224.8 L353.488 1225.38 L353.716 1226.1 L353.944 1224.38 L354.171 1231.84 L354.399 1234.94 L354.627 1231.21 L354.855 1223.31 L355.083 1226.36 L355.31 1228.84 L355.538 1229.26 L355.766 1232.27 L355.994 1239.51 L356.221 1220.98 L356.449 1218.11 L356.677 1200.16 L356.905 1195.53 L357.133 1191.3 L357.36 1176.22 L357.588 1182.63 L357.816 1166.02 L358.044 1159.21 L358.271 1154.85 L358.499 1164.53 L358.727 1183.09 L358.955 1196.01 L359.182 1182.91 L359.41 1176.66 L359.638 1179.99 L359.866 1190.64 L360.094 1182.98 L360.321 1192.45 L360.549 1189.22 L360.777 1184.65 L361.005 1204.97 L361.232 1206.96 L361.46 1185.13 L361.688 1177.64 L361.916 1162.91 L362.143 1171.15 L362.371 1175.05 L362.599 1168.42 L362.827 1156.27 L363.055 1158.81 L363.282 1173.33 L363.51 1184.19 L363.738 1185.9 L363.966 1166.59 L364.193 1162.33 L364.421 1152.48 L364.649 1154.01 L364.877 1164.06 L365.104 1157.78 L365.332 1161.6 L365.56 1172.57 L365.788 1156.02 L366.016 1154.05 L366.243 1160.08 L366.471 1139.7 L366.699 1143.19 L366.927 1152.96 L367.154 1147.99 L367.382 1152.57 L367.61 1150.7 L367.838 1147.34 L368.066 1152.01 L368.293 1145.22 L368.521 1147.26 L368.749 1148.1 L368.977 1141.04 L369.204 1140.08 L369.432 1147.45 L369.66 1143.05 L369.888 1132.57 L370.115 1123.99 L370.343 1126.26 L370.571 1125.55 L370.799 1110.63 L371.027 1104.03 L371.254 1119.31 L371.482 1119.14 L371.71 1127.48 L371.938 1116.44 L372.165 1103.44 L372.393 1100.25 L372.621 1102.4 L372.849 1105.99 L373.076 1103.68 L373.304 1104.49 L373.532 1094.26 L373.76 1102.93 L373.988 1092.94 L374.215 1095.96 L374.443 1112.03 L374.671 1104.82 L374.899 1106.74 L375.126 1106.44 L375.354 1099.27 L375.582 1103.86 L375.81 1104.32 L376.037 1105.66 L376.265 1096.89 L376.493 1102.43 L376.721 1110.45 L376.949 1104.26 L377.176 1103.51 L377.404 1100.94 L377.632 1113.12 L377.86 1118.44 L378.087 1114.32 L378.315 1113.47 L378.543 1122.3 L378.771 1120.95 L378.999 1125.59 L379.226 1133.76 L379.454 1132.35 L379.682 1125.83 L379.91 1115.58 L380.137 1121.34 L380.365 1124.6 L380.593 1116.51 L380.821 1112.39 L381.048 1118.37 L381.276 1114.96 L381.504 1111.92 L381.732 1115.65 L381.96 1106.7 L382.187 1114.03 L382.415 1107.48 L382.643 1113.22 L382.871 1121.2 L383.098 1123.06 L383.326 1123.96 L383.554 1120.86 L383.782 1130.7 L384.009 1118.62 L384.237 1122.48 L384.465 1119.8 L384.693 1112.61 L384.921 1120.41 L385.148 1115.86 L385.376 1121.76 L385.604 1119.02 L385.832 1126.1 L386.059 1127.42 L386.287 1130.73 L386.515 1123.75 L386.743 1129.25 L386.97 1132.74 L387.198 1128.28 L387.426 1142.91 L387.654 1144.62 L387.882 1139.54 L388.109 1142.58 L388.337 1142.64 L388.565 1146.55 L388.793 1147.72 L389.02 1147.07 L389.248 1146.63 L389.476 1145.97 L389.704 1158.61 L389.932 1164.88 L390.159 1161.65 L390.387 1154.42 L390.615 1164.49 L390.843 1151.16 L391.07 1148.21 L391.298 1155.06 L391.526 1157.48 L391.754 1151.31 L391.981 1152.6 L392.209 1151.37 L392.437 1145.42 L392.665 1138.39 L392.893 1134.01 L393.12 1121.16 L393.348 1125.03 L393.576 1129.21 L393.804 1133.86 L394.031 1134.51 L394.259 1139.67 L394.487 1128.64 L394.715 1133.79 L394.942 1131.45 L395.17 1124.07 L395.398 1110.54 L395.626 1110.16 L395.854 1109.04 L396.081 1110.96 L396.309 1111.99 L396.537 1103.66 L396.765 1101.35 L396.992 1099.2 L397.22 1102.3 L397.448 1107.59 L397.676 1107.29 L397.903 1106.65 L398.131 1109.46 L398.359 1115.04 L398.587 1110.1 L398.815 1110.2 L399.042 1113.34 L399.27 1117.88 L399.498 1120.85 L399.726 1128.29 L399.953 1133.56 L400.181 1136.98 L400.409 1135.63 L400.637 1127.93 L400.865 1134.2 L401.092 1131.41 L401.32 1129.68 L401.548 1126.57 L401.776 1118.01 L402.003 1117.28 L402.231 1107.92 L402.459 1108.93 L402.687 1104.99 L402.914 1106.43 L403.142 1114.47 L403.37 1115.95 L403.598 1112.76 L403.826 1108.92 L404.053 1108.84 L404.281 1102.87 L404.509 1115.3 L404.737 1116.55 L404.964 1118.13 L405.192 1121.65 L405.42 1127.55 L405.648 1123.79 L405.875 1120.1 L406.103 1119.13 L406.331 1123.56 L406.559 1118.8 L406.787 1115 L407.014 1111.27 L407.242 1120.02 L407.47 1115.79 L407.698 1117.66 L407.925 1118.41 L408.153 1113.85 L408.381 1116.34 L408.609 1112.52 L408.836 1115.01 L409.064 1123.12 L409.292 1118.68 L409.52 1117.44 L409.748 1113.54 L409.975 1121.96 L410.203 1121.77 L410.431 1115.9 L410.659 1108.82 L410.886 1111.81 L411.114 1095.36 L411.342 1098.13 L411.57 1102.7 L411.798 1102.15 L412.025 1097.94 L412.253 1098.21 L412.481 1103.12 L412.709 1103.52 L412.936 1103.01 L413.164 1098.83 L413.392 1087.98 L413.62 1092.98 L413.847 1093.26 L414.075 1095.34 L414.303 1096.56 L414.531 1102.45 L414.759 1108.27 L414.986 1113.83 L415.214 1115.92 L415.442 1120.3 L415.67 1116.95 L415.897 1115.56 L416.125 1113.15 L416.353 1106.6 L416.581 1103.03 L416.808 1097.34 L417.036 1093.27 L417.264 1080.66 L417.492 1071.83 L417.72 1068.15 L417.947 1065.93 L418.175 1060.61 L418.403 1060.22 L418.631 1065.71 L418.858 1069.62 L419.086 1070.47 L419.314 1065.04 L419.542 1064.11 L419.769 1066.48 L419.997 1067.46 L420.225 1064.73 L420.453 1072.65 L420.681 1072.07 L420.908 1074.75 L421.136 1074.28 L421.364 1078 L421.592 1079.25 L421.819 1076.69 L422.047 1074.68 L422.275 1076.5 L422.503 1075.35 L422.731 1072.73 L422.958 1080.63 L423.186 1080.95 L423.414 1082.91 L423.642 1082.66 L423.869 1081.67 L424.097 1081.19 L424.325 1080.98 L424.553 1074.43 L424.78 1073.58 L425.008 1072.22 L425.236 1072.8 L425.464 1071.16 L425.692 1078.38 L425.919 1080.3 L426.147 1079.85 L426.375 1083.79 L426.603 1083.6 L426.83 1082.52 L427.058 1083.33 L427.286 1081.67 L427.514 1080.4 L427.741 1079.76 L427.969 1084.5 L428.197 1088.66 L428.425 1088.8 L428.653 1093.47 L428.88 1093.7 L429.108 1101.18 L429.336 1107.23 L429.564 1109.31 L429.791 1106.21 L430.019 1107.59 L430.247 1109.92 L430.475 1114.46 L430.702 1103.83 L430.93 1100.16 L431.158 1105.51 L431.386 1111.57 L431.614 1112.34 L431.841 1113.72 L432.069 1110.12 L432.297 1109.94 L432.525 1105.61 L432.752 1100.93 L432.98 1102.78 L433.208 1103.75 L433.436 1110.71 L433.664 1111.52 L433.891 1114.27 L434.119 1117.23 L434.347 1122.32 L434.575 1126.94 L434.802 1124.76 L435.03 1119.18 L435.258 1115.12 L435.486 1115.54 L435.713 1119.25 L435.941 1119.52 L436.169 1119.51 L436.397 1122.86 L436.625 1122.86 L436.852 1119.65 L437.08 1118.39 L437.308 1115.61 L437.536 1116.05 L437.763 1112.85 L437.991 1105.78 L438.219 1106.73 L438.447 1105.38 L438.674 1108.66 L438.902 1109.74 L439.13 1110.83 L439.358 1110.08 L439.586 1109.83 L439.813 1117.2 L440.041 1111.34 L440.269 1114.99 L440.497 1118.56 L440.724 1118.2 L440.952 1123.87 L441.18 1118.75 L441.408 1120.63 L441.635 1113.55 L441.863 1117.51 L442.091 1108.2 L442.319 1107.57 L442.547 1108.29 L442.774 1112.74 L443.002 1111.99 L443.23 1118.57 L443.458 1113.79 L443.685 1115.02 L443.913 1119.85 L444.141 1115.21 L444.369 1123.22 L444.597 1127.47 L444.824 1129.46 L445.052 1128.28 L445.28 1124.91 L445.508 1127.34 L445.735 1127.44 L445.963 1125.86 L446.191 1121.55 L446.419 1125.61 L446.646 1122.04 L446.874 1121.06 L447.102 1119.74 L447.33 1118.65 L447.558 1127.47 L447.785 1125.3 L448.013 1126.03 L448.241 1130.12 L448.469 1126.75 L448.696 1126.23 L448.924 1120.06 L449.152 1120.31 L449.38 1115.41 L449.607 1117.31 L449.835 1121.58 L450.063 1123.59 L450.291 1121.65 L450.519 1121.02 L450.746 1113.19 L450.974 1107.01 L451.202 1111.66 L451.43 1115.72 L451.657 1115.9 L451.885 1116.64 L452.113 1111.06 L452.341 1109.78 L452.568 1115.68 L452.796 1117.77 L453.024 1119.55 L453.252 1122.99 L453.48 1123.65 L453.707 1121.87 L453.935 1118.5 L454.163 1119.06 L454.391 1119.82 L454.618 1119.64 L454.846 1122.37 L455.074 1121.29 L455.302 1119.14 L455.53 1112.4 L455.757 1108.48 L455.985 1110.96 L456.213 1104.79 L456.441 1104.06 L456.668 1098.08 L456.896 1097.4 L457.124 1103.28 L457.352 1100.67 L457.579 1099.94 L457.807 1098.7 L458.035 1103.15 L458.263 1104.91 L458.491 1096.89 L458.718 1096.08 L458.946 1098.71 L459.174 1096.71 L459.402 1095.16 L459.629 1101.91 L459.857 1098.21 L460.085 1101.3 L460.313 1105.93 L460.54 1106.69 L460.768 1101.82 L460.996 1101.37 L461.224 1100.3 L461.452 1099.19 L461.679 1098.75 L461.907 1099.43 L462.135 1096.93 L462.363 1096.6 L462.59 1097.13 L462.818 1096.67 L463.046 1093.19 L463.274 1087.26 L463.501 1088.37 L463.729 1086.61 L463.957 1086.06 L464.185 1085.09 L464.413 1090.73 L464.64 1090.02 L464.868 1092.39 L465.096 1094.91 L465.324 1097.13 L465.551 1094.68 L465.779 1095.12 L466.007 1100.85 L466.235 1104.57 L466.462 1104.29 L466.69 1105.91 L466.918 1104.34 L467.146 1107.22 L467.374 1112.1 L467.601 1111.4 L467.829 1108.9 L468.057 1115.49 L468.285 1111.89 L468.512 1115.21 L468.74 1119.86 L468.968 1123.48 L469.196 1120.95 L469.424 1122.57 L469.651 1121.98 L469.879 1120.58 L470.107 1125.18 L470.335 1125.06 L470.562 1122.98 L470.79 1118.26 L471.018 1118.13 L471.246 1120.96 L471.473 1120.64 L471.701 1119.3 L471.929 1121.5 L472.157 1123.1 L472.385 1120.13 L472.612 1120.59 L472.84 1120.62 L473.068 1117.46 L473.296 1111.97 L473.523 1113.82 L473.751 1114.88 L473.979 1116.17 L474.207 1118.24 L474.434 1116.29 L474.662 1116.05 L474.89 1110.56 L475.118 1108.43 L475.346 1112.77 L475.573 1114.08 L475.801 1116.4 L476.029 1117.69 L476.257 1117.13 L476.484 1118 L476.712 1116.37 L476.94 1121.11 L477.168 1121.93 L477.395 1123.48 L477.623 1121.3 L477.851 1120.56 L478.079 1120.1 L478.307 1123.35 L478.534 1114.32 L478.762 1115.7 L478.99 1119.68 L479.218 1116.78 L479.445 1118.1 L479.673 1115.58 L479.901 1113.25 L480.129 1110.34 L480.357 1111.37 L480.584 1111.66 L480.812 1112.34 L481.04 1111.96 L481.268 1117.73 L481.495 1114.94 L481.723 1118.51 L481.951 1120 L482.179 1122.67 L482.406 1115.79 L482.634 1119.14 L482.862 1123.37 L483.09 1122.21 L483.318 1120.09 L483.545 1115.75 L483.773 1114.62 L484.001 1116.09 L484.229 1117.48 L484.456 1120.99 L484.684 1118.48 L484.912 1117.71 L485.14 1117.63 L485.367 1117.96 L485.595 1121.42 L485.823 1120.94 L486.051 1119.98 L486.279 1122.19 L486.506 1119.83 L486.734 1119.27 L486.962 1123.44 L487.19 1127.73 L487.417 1129.43 L487.645 1131.21 L487.873 1132.29 L488.101 1126.38 L488.328 1126.39 L488.556 1129.23 L488.784 1118.18 L489.012 1115.17 L489.24 1115.36 L489.467 1108.84 L489.695 1105.23 L489.923 1104.47 L490.151 1107.45 L490.378 1109.35 L490.606 1110.31 L490.834 1109.84 L491.062 1111.05 L491.29 1107.41 L491.517 1111.65 L491.745 1112.55 L491.973 1110.19 L492.201 1110.1 L492.428 1107.36 L492.656 1103.9 L492.884 1101.2 L493.112 1103.54 L493.339 1100.26 L493.567 1100.33 L493.795 1100.69 L494.023 1099.26 L494.251 1101.89 L494.478 1100.75 L494.706 1102.71 L494.934 1104.1 L495.162 1099.46 L495.389 1094.7 L495.617 1096.98 L495.845 1094.84 L496.073 1094.54 L496.3 1095.54 L496.528 1091.62 L496.756 1090.93 L496.984 1091.64 L497.212 1094.51 L497.439 1097.58 L497.667 1094.72 L497.895 1088.27 L498.123 1087.68 L498.35 1089.04 L498.578 1090.4 L498.806 1087.91 L499.034 1087.26 L499.261 1083.51 L499.489 1085.12 L499.717 1088.72 L499.945 1093.86 L500.173 1089.19 L500.4 1093.22 L500.628 1090.85 L500.856 1090.19 L501.084 1094.49 L501.311 1094.44 L501.539 1094.72 L501.767 1089.67 L501.995 1091.9 L502.223 1094.08 L502.45 1091.38 L502.678 1090.29 L502.906 1088.3 L503.134 1087.68 L503.361 1093.09 L503.589 1094.59 L503.817 1096.09 L504.045 1094.96 L504.272 1098.29 L504.5 1099.08 L504.728 1098.93 L504.956 1102.61 L505.184 1102.35 L505.411 1101.45 L505.639 1102.92 L505.867 1103.79 L506.095 1105.41 L506.322 1108.89 L506.55 1108.62 L506.778 1104.09 L507.006 1108.12 L507.233 1110.75 L507.461 1108.24 L507.689 1104.82 L507.917 1108.02 L508.145 1112.01 L508.372 1113.81 L508.6 1115.28 L508.828 1119.49 L509.056 1117.38 L509.283 1117.2 L509.511 1114.21 L509.739 1115.65 L509.967 1116.34 L510.194 1118 L510.422 1117.19 L510.65 1111.83 L510.878 1111.02 L511.106 1110.16 L511.333 1106.57 L511.561 1103.22 L511.789 1105.16 L512.017 1108.38 L512.244 1104.34 L512.472 1108.1 L512.7 1106.53 L512.928 1105.07 L513.156 1102.07 L513.383 1105.7 L513.611 1108.86 L513.839 1105.55 L514.067 1105.47 L514.294 1102.55 L514.522 1103.07 L514.75 1101.12 L514.978 1100.26 L515.205 1098.55 L515.433 1096.81 L515.661 1099.01 L515.889 1097.39 L516.117 1099.78 L516.344 1100.65 L516.572 1100.36 L516.8 1099.67 L517.028 1097.16 L517.255 1098.85 L517.483 1095.45 L517.711 1093.86 L517.939 1091.89 L518.166 1094.07 L518.394 1094.95 L518.622 1094.73 L518.85 1094.96 L519.078 1097.56 L519.305 1098.78 L519.533 1102.08 L519.761 1101.1 L519.989 1102.3 L520.216 1101.48 L520.444 1104.02 L520.672 1103.57 L520.9 1103.56 L521.127 1105.43 L521.355 1109.18 L521.583 1107.26 L521.811 1107.36 L522.039 1107.61 L522.266 1110.04 L522.494 1104.47 L522.722 1100.15 L522.95 1101.37 L523.177 1100.55 L523.405 1097.56 L523.633 1093.21 L523.861 1091.26 L524.089 1086.26 L524.316 1083.35 L524.544 1085.58 L524.772 1086.31 L525 1089.6 L525.227 1090.33 L525.455 1082.82 L525.683 1085.47 L525.911 1085.14 L526.138 1084.27 L526.366 1086.2 L526.594 1086.71 L526.822 1079.08 L527.05 1082.3 L527.277 1082.16 L527.505 1079.07 L527.733 1079.3 L527.961 1083.62 L528.188 1077.32 L528.416 1082.45 L528.644 1083.01 L528.872 1083.66 L529.099 1083.06 L529.327 1079.72 L529.555 1081.22 L529.783 1080.53 L530.011 1081.44 L530.238 1081.76 L530.466 1081.95 L530.694 1082.36 L530.922 1080.13 L531.149 1076.07 L531.377 1075.2 L531.605 1072.2 L531.833 1068.57 L532.06 1065.96 L532.288 1067.06 L532.516 1067.84 L532.744 1068 L532.972 1072.95 L533.199 1074.47 L533.427 1069.73 L533.655 1066.44 L533.883 1063.2 L534.11 1061.43 L534.338 1057.92 L534.566 1058.9 L534.794 1062.51 L535.022 1060.95 L535.249 1064.98 L535.477 1065.37 L535.705 1062.43 L535.933 1062.95 L536.16 1061.1 L536.388 1056.52 L536.616 1056.6 L536.844 1058.29 L537.071 1060.91 L537.299 1067.48 L537.527 1064.29 L537.755 1068.61 L537.983 1067.23 L538.21 1069.19 L538.438 1070.99 L538.666 1073.2 L538.894 1074.56 L539.121 1075.13 L539.349 1075.35 L539.577 1076.72 L539.805 1075.5 L540.032 1076.53 L540.26 1079.73 L540.488 1082.7 L540.716 1085.43 L540.944 1087.72 L541.171 1088.25 L541.399 1092.03 L541.627 1093.17 L541.855 1096.63 L542.082 1096.39 L542.31 1097.76 L542.538 1097.7 L542.766 1096.98 L542.993 1099.1 L543.221 1097.71 L543.449 1095.83 L543.677 1098.03 L543.905 1096.59 L544.132 1099.83 L544.36 1095.88 L544.588 1091.38 L544.816 1092.8 L545.043 1092.3 L545.271 1093.23 L545.499 1092.65 L545.727 1092.33 L545.955 1090.97 L546.182 1092.44 L546.41 1094.95 L546.638 1098.42 L546.866 1096.62 L547.093 1097.26 L547.321 1096.49 L547.549 1096.83 L547.777 1098.01 L548.004 1097.91 L548.232 1095.93 L548.46 1097.61 L548.688 1097.18 L548.916 1099.6 L549.143 1095.57 L549.371 1097.21 L549.599 1098.46 L549.827 1095.08 L550.054 1092.22 L550.282 1092.39 L550.51 1089.96 L550.738 1085.83 L550.965 1086.51 L551.193 1089.16 L551.421 1086.41 L551.649 1087.38 L551.877 1085.11 L552.104 1087.11 L552.332 1090.92 L552.56 1089.69 L552.788 1091.23 L553.015 1090.41 L553.243 1087.16 L553.471 1088.8 L553.699 1089.6 L553.926 1091.67 L554.154 1089.7 L554.382 1089.21 L554.61 1090.51 L554.838 1094.21 L555.065 1095.6 L555.293 1097.24 L555.521 1100.01 L555.749 1101.33 L555.976 1101.47 L556.204 1102.48 L556.432 1104.18 L556.66 1107.64 L556.888 1109.22 L557.115 1109.95 L557.343 1113.28 L557.571 1114.09 L557.799 1115.52 L558.026 1117.41 L558.254 1117.54 L558.482 1119.52 L558.71 1117.73 L558.937 1120.27 L559.165 1115.79 L559.393 1114.89 L559.621 1116.43 L559.849 1112.47 L560.076 1113.96 L560.304 1115.03 L560.532 1120.82 L560.76 1119.78 L560.987 1118.3 L561.215 1120.61 L561.443 1121.04 L561.671 1120.63 L561.898 1121.11 L562.126 1124.37 L562.354 1124.77 L562.582 1125.51 L562.81 1126.44 L563.037 1126.76 L563.265 1126.04 L563.493 1128.79 L563.721 1127.91 L563.948 1127.75 L564.176 1125.06 L564.404 1122.4 L564.632 1123.68 L564.859 1123.59 L565.087 1121.94 L565.315 1120.67 L565.543 1117.33 L565.771 1118.2 L565.998 1118.8 L566.226 1120.22 L566.454 1122.41 L566.682 1124.05 L566.909 1123.81 L567.137 1120.85 L567.365 1120.58 L567.593 1121.67 L567.821 1122.38 L568.048 1121.59 L568.276 1121.06 L568.504 1119.73 L568.732 1117.38 L568.959 1118.82 L569.187 1119.59 L569.415 1122.2 L569.643 1121.03 L569.87 1116.57 L570.098 1117.84 L570.326 1121.18 L570.554 1121.15 L570.782 1122.79 L571.009 1120.87 L571.237 1120.83 L571.465 1120.22 L571.693 1125.1 L571.92 1129.51 L572.148 1130.59 L572.376 1128.97 L572.604 1125.34 L572.831 1125.73 L573.059 1126.25 L573.287 1125.78 L573.515 1130.87 L573.743 1134.74 L573.97 1135.96 L574.198 1133.55 L574.426 1137.49 L574.654 1136.36 L574.881 1135.34 L575.109 1134.33 L575.337 1132.51 L575.565 1132.59 L575.792 1134.96 L576.02 1133.93 L576.248 1135.93 L576.476 1134.79 L576.704 1139.36 L576.931 1139.36 L577.159 1144.31 L577.387 1143.21 L577.615 1143.56 L577.842 1145.83 L578.07 1145.49 L578.298 1147.43 L578.526 1149.64 L578.754 1149.02 L578.981 1150.94 L579.209 1145.62 L579.437 1147.78 L579.665 1146.89 L579.892 1146.71 L580.12 1141.81 L580.348 1137.93 L580.576 1133.9 L580.803 1134.14 L581.031 1134.85 L581.259 1129.51 L581.487 1126.63 L581.715 1125.56 L581.942 1127.9 L582.17 1125.6 L582.398 1128.5 L582.626 1122.51 L582.853 1123.84 L583.081 1124.04 L583.309 1123.97 L583.537 1125.76 L583.764 1126.34 L583.992 1127.29 L584.22 1127.81 L584.448 1129.59 L584.676 1133 L584.903 1130.57 L585.131 1130.85 L585.359 1127.76 L585.587 1126.46 L585.814 1128.93 L586.042 1128.72 L586.27 1128.79 L586.498 1131.19 L586.725 1132.42 L586.953 1134.95 L587.181 1132.43 L587.409 1130.46 L587.637 1133.09 L587.864 1133.59 L588.092 1132.84 L588.32 1133.25 L588.548 1134.92 L588.775 1132.76 L589.003 1130.51 L589.231 1132.88 L589.459 1132.34 L589.687 1128.01 L589.914 1124.63 L590.142 1121.71 L590.37 1120.65 L590.598 1121.25 L590.825 1121.4 L591.053 1124.41 L591.281 1125.1 L591.509 1127.68 L591.736 1127.71 L591.964 1129.92 L592.192 1131.87 L592.42 1133.96 L592.648 1134.64 L592.875 1132.65 L593.103 1132.44 L593.331 1132.73 L593.559 1130.3 L593.786 1131.19 L594.014 1132.65 L594.242 1132.07 L594.47 1134.42 L594.697 1135.87 L594.925 1136.04 L595.153 1137.61 L595.381 1134.43 L595.609 1132.95 L595.836 1132.07 L596.064 1127.6 L596.292 1127.94 L596.52 1128.04 L596.747 1129.53 L596.975 1128.89 L597.203 1124.68 L597.431 1122.49 L597.658 1124.37 L597.886 1125.59 L598.114 1123.66 L598.342 1125.07 L598.57 1123.85 L598.797 1125.14 L599.025 1126.75 L599.253 1127.58 L599.481 1127.55 L599.708 1122.66 L599.936 1123.98 L600.164 1123.21 L600.392 1120.29 L600.62 1118.35 L600.847 1115.81 L601.075 1118.39 L601.303 1118.02 L601.531 1120.07 L601.758 1121.07 L601.986 1121.33 L602.214 1121.26 L602.442 1126.41 L602.669 1123.67 L602.897 1122.36 L603.125 1121.5 L603.353 1119.19 L603.581 1121.82 L603.808 1120.1 L604.036 1121.45 L604.264 1119.88 L604.492 1118.72 L604.719 1118.25 L604.947 1121.77 L605.175 1122.19 L605.403 1123.97 L605.63 1125.03 L605.858 1125.84 L606.086 1122.85 L606.314 1123.55 L606.542 1124.59 L606.769 1124.85 L606.997 1126.83 L607.225 1124.77 L607.453 1126.59 L607.68 1124.29 L607.908 1124.53 L608.136 1123.87 L608.364 1122.53 L608.591 1119.78 L608.819 1118.16 L609.047 1118.35 L609.275 1115.27 L609.503 1115.52 L609.73 1115.06 L609.958 1117.7 L610.186 1118.63 L610.414 1118.8 L610.641 1120.98 L610.869 1121.33 L611.097 1122.14 L611.325 1125.08 L611.553 1124.74 L611.78 1124.3 L612.008 1124.1 L612.236 1125.55 L612.464 1122.73 L612.691 1124.86 L612.919 1127.95 L613.147 1124.15 L613.375 1126.16 L613.602 1127.59 L613.83 1125.23 L614.058 1124.37 L614.286 1124.2 L614.514 1123.01 L614.741 1126.31 L614.969 1127.87 L615.197 1127.34 L615.425 1126.59 L615.652 1126.48 L615.88 1128.62 L616.108 1127.65 L616.336 1124.54 L616.563 1126.18 L616.791 1127.95 L617.019 1128.63 L617.247 1126.76 L617.475 1126.22 L617.702 1128.75 L617.93 1126.7 L618.158 1124.5 L618.386 1125.76 L618.613 1127.21 L618.841 1123.12 L619.069 1124.18 L619.297 1120.38 L619.524 1123.08 L619.752 1121.54 L619.98 1118.95 L620.208 1117.26 L620.436 1115.56 L620.663 1113.81 L620.891 1113.85 L621.119 1114.64 L621.347 1112.11 L621.574 1111.52 L621.802 1113 L622.03 1112.52 L622.258 1113.04 L622.486 1112.21 L622.713 1114.71 L622.941 1113.85 L623.169 1113.16 L623.397 1113.88 L623.624 1114.94 L623.852 1115.41 L624.08 1116.26 L624.308 1117 L624.535 1114.65 L624.763 1114.01 L624.991 1115.51 L625.219 1111.35 L625.447 1111.59 L625.674 1112.25 L625.902 1113.45 L626.13 1112.78 L626.358 1112.02 L626.585 1109.13 L626.813 1103.7 L627.041 1102.22 L627.269 1105.88 L627.496 1099.44 L627.724 1101.89 L627.952 1103.38 L628.18 1100.4 L628.408 1099.02 L628.635 1098.3 L628.863 1097.57 L629.091 1097.76 L629.319 1100.35 L629.546 1096.75 L629.774 1097.09 L630.002 1094.77 L630.23 1091.06 L630.457 1087.68 L630.685 1086.38 L630.913 1086.33 L631.141 1084.73 L631.369 1085.88 L631.596 1084.57 L631.824 1085.05 L632.052 1083.79 L632.28 1084.09 L632.507 1082.9 L632.735 1082.18 L632.963 1079.35 L633.191 1080.01 L633.419 1079.52 L633.646 1079.21 L633.874 1081.1 L634.102 1083.7 L634.33 1085.2 L634.557 1087.38 L634.785 1086.84 L635.013 1086.03 L635.241 1086.43 L635.468 1088 L635.696 1089.57 L635.924 1088.6 L636.152 1085.95 L636.38 1084.51 L636.607 1083.99 L636.835 1081.81 L637.063 1081.62 L637.291 1080.71 L637.518 1081.14 L637.746 1082.74 L637.974 1081.56 L638.202 1080.16 L638.429 1078.88 L638.657 1075.92 L638.885 1076.43 L639.113 1076.96 L639.341 1079.72 L639.568 1078.07 L639.796 1073.43 L640.024 1072.7 L640.252 1072.13 L640.479 1074.85 L640.707 1073.79 L640.935 1069.86 L641.163 1071.7 L641.39 1071.14 L641.618 1071.06 L641.846 1068.58 L642.074 1067.36 L642.302 1065.22 L642.529 1064.13 L642.757 1065.55 L642.985 1068.65 L643.213 1068.69 L643.44 1067.75 L643.668 1065.27 L643.896 1065.04 L644.124 1063.67 L644.352 1064.98 L644.579 1064.17 L644.807 1061.73 L645.035 1060.53 L645.263 1061.73 L645.49 1058.31 L645.718 1057.53 L645.946 1056.47 L646.174 1056.86 L646.401 1056.96 L646.629 1057.09 L646.857 1057.14 L647.085 1056.56 L647.313 1059.1 L647.54 1055.75 L647.768 1055.42 L647.996 1052.5 L648.224 1051.65 L648.451 1049.29 L648.679 1051.32 L648.907 1052.73 L649.135 1049.47 L649.362 1049.33 L649.59 1049.4 L649.818 1049.1 L650.046 1049.63 L650.274 1049.39 L650.501 1048.95 L650.729 1050.65 L650.957 1050.45 L651.185 1052.14 L651.412 1049.78 L651.64 1049.3 L651.868 1050.71 L652.096 1049.75 L652.323 1053.29 L652.551 1052.32 L652.779 1053.33 L653.007 1052.59 L653.235 1053.16 L653.462 1052.1 L653.69 1053.31 L653.918 1054 L654.146 1053.64 L654.373 1051.74 L654.601 1047.5 L654.829 1045.93 L655.057 1044.74 L655.285 1043.8 L655.512 1042.78 L655.74 1040.76 L655.968 1041.4 L656.196 1042.53 L656.423 1040.6 L656.651 1039.34 L656.879 1039.7 L657.107 1040.34 L657.334 1040.65 L657.562 1039.32 L657.79 1040.79 L658.018 1041.38 L658.246 1041.58 L658.473 1039.35 L658.701 1039.1 L658.929 1036.18 L659.157 1038.77 L659.384 1040.07 L659.612 1040.89 L659.84 1042.1 L660.068 1041.67 L660.295 1040.58 L660.523 1038.26 L660.751 1039.26 L660.979 1039.13 L661.207 1038.91 L661.434 1039.72 L661.662 1041.99 L661.89 1043.84 L662.118 1043.74 L662.345 1042.94 L662.573 1043.35 L662.801 1046.22 L663.029 1046.11 L663.256 1044.33 L663.484 1043.09 L663.712 1041.26 L663.94 1039.74 L664.168 1038.05 L664.395 1037.34 L664.623 1036.96 L664.851 1038.51 L665.079 1039.6 L665.306 1042.55 L665.534 1043.1 L665.762 1042.23 L665.99 1043.84 L666.218 1045.47 L666.445 1040.1 L666.673 1038.2 L666.901 1040.61 L667.129 1041.16 L667.356 1042.54 L667.584 1041.01 L667.812 1042.05 L668.04 1044.38 L668.267 1043.05 L668.495 1044.17 L668.723 1042.68 L668.951 1043.26 L669.179 1045.16 L669.406 1045.1 L669.634 1046.06 L669.862 1046.53 L670.09 1047.69 L670.317 1048.6 L670.545 1051.73 L670.773 1051.71 L671.001 1050.24 L671.228 1050.94 L671.456 1050.02 L671.684 1050.21 L671.912 1050.45 L672.14 1047.51 L672.367 1048.26 L672.595 1046.23 L672.823 1047.04 L673.051 1045.55 L673.278 1044.72 L673.506 1042.96 L673.734 1041.14 L673.962 1040.54 L674.189 1039.88 L674.417 1040.32 L674.645 1041.23 L674.873 1041.92 L675.101 1044.38 L675.328 1042.08 L675.556 1043.23 L675.784 1045.09 L676.012 1044.81 L676.239 1045.1 L676.467 1044.32 L676.695 1044.7 L676.923 1045.46 L677.151 1043.44 L677.378 1044.39 L677.606 1045.93 L677.834 1044.91 L678.062 1042.98 L678.289 1043.71 L678.517 1043.17 L678.745 1043.94 L678.973 1044.88 L679.2 1045.9 L679.428 1046.33 L679.656 1047.16 L679.884 1046.31 L680.112 1047.11 L680.339 1049.41 L680.567 1048.35 L680.795 1048.43 L681.023 1050.03 L681.25 1049.88 L681.478 1050.22 L681.706 1050.55 L681.934 1052.75 L682.161 1054.75 L682.389 1052.64 L682.617 1050.17 L682.845 1050.68 L683.073 1051.54 L683.3 1053.58 L683.528 1050.78 L683.756 1051.99 L683.984 1053.56 L684.211 1055.83 L684.439 1056.13 L684.667 1051.92 L684.895 1053.62 L685.122 1054.06 L685.35 1055.28 L685.578 1054.11 L685.806 1051.15 L686.034 1053.02 L686.261 1052.53 L686.489 1054.95 L686.717 1053.14 L686.945 1051.72 L687.172 1051.81 L687.4 1053.12 L687.628 1052 L687.856 1051.98 L688.084 1052.54 L688.311 1051.12 L688.539 1050.44 L688.767 1052.29 L688.995 1053.21 L689.222 1054.02 L689.45 1055.3 L689.678 1055.89 L689.906 1054.53 L690.133 1052.95 L690.361 1053.08 L690.589 1055.21 L690.817 1058.67 L691.045 1059.28 L691.272 1057.91 L691.5 1057.49 L691.728 1058.88 L691.956 1058.45 L692.183 1058.63 L692.411 1060.01 L692.639 1059.38 L692.867 1060.51 L693.094 1060.5 L693.322 1061.22 L693.55 1060 L693.778 1062.88 L694.006 1064.45 L694.233 1064.56 L694.461 1064.58 L694.689 1058.91 L694.917 1055.34 L695.144 1054.12 L695.372 1054.45 L695.6 1055.01 L695.828 1057.33 L696.055 1056.83 L696.283 1056.72 L696.511 1059.01 L696.739 1060.24 L696.967 1061.67 L697.194 1061.79 L697.422 1063.05 L697.65 1061.42 L697.878 1061.45 L698.105 1060.01 L698.333 1061.05 L698.561 1059.62 L698.789 1057.55 L699.017 1055.04 L699.244 1053.79 L699.472 1054.26 L699.7 1054.61 L699.928 1056.43 L700.155 1055.86 L700.383 1054.79 L700.611 1057.04 L700.839 1058.96 L701.066 1061.97 L701.294 1064.39 L701.522 1065.39 L701.75 1064.78 L701.978 1064.72 L702.205 1068.14 L702.433 1068.54 L702.661 1070.75 L702.889 1072.99 L703.116 1072.19 L703.344 1071.77 L703.572 1071.04 L703.8 1069.85 L704.027 1070.52 L704.255 1069.43 L704.483 1068 L704.711 1066.56 L704.939 1064.13 L705.166 1064.28 L705.394 1065.68 L705.622 1063.51 L705.85 1060.31 L706.077 1061.59 L706.305 1062.45 L706.533 1060.78 L706.761 1061 L706.988 1060.51 L707.216 1059.42 L707.444 1059.67 L707.672 1060.53 L707.9 1059.83 L708.127 1059.12 L708.355 1059.69 L708.583 1061.05 L708.811 1060.88 L709.038 1061.63 L709.266 1063.44 L709.494 1062.32 L709.722 1063.16 L709.95 1063.09 L710.177 1061.4 L710.405 1061.48 L710.633 1061.9 L710.861 1061.84 L711.088 1063.16 L711.316 1062.37 L711.544 1063.25 L711.772 1063.39 L711.999 1066.11 L712.227 1066.15 L712.455 1063.58 L712.683 1064.94 L712.911 1065.32 L713.138 1061.17 L713.366 1064.19 L713.594 1063.62 L713.822 1061.93 L714.049 1061.98 L714.277 1063.11 L714.505 1062.78 L714.733 1064.98 L714.96 1064.09 L715.188 1064.77 L715.416 1064.07 L715.644 1064.18 L715.872 1065.73 L716.099 1064.9 L716.327 1065.68 L716.555 1067.83 L716.783 1066.76 L717.01 1067.45 L717.238 1067.66 L717.466 1066.14 L717.694 1065.27 L717.921 1066.26 L718.149 1065.02 L718.377 1067.59 L718.605 1069.55 L718.833 1069.94 L719.06 1070.72 L719.288 1070.46 L719.516 1070.36 L719.744 1066.03 L719.971 1067.37 L720.199 1069.05 L720.427 1068.73 L720.655 1069.73 L720.883 1068.87 L721.11 1069.77 L721.338 1067.42 L721.566 1066.36 L721.794 1066.52 L722.021 1068.33 L722.249 1067.79 L722.477 1068.42 L722.705 1069.23 L722.932 1070.33 L723.16 1070.34 L723.388 1070.94 L723.616 1068.72 L723.844 1068.25 L724.071 1069.88 L724.299 1070.48 L724.527 1068.43 L724.755 1068.66 L724.982 1067.96 L725.21 1067.17 L725.438 1067.15 L725.666 1066.21 L725.893 1063.87 L726.121 1062.68 L726.349 1062.15 L726.577 1060.83 L726.805 1059.08 L727.032 1060.15 L727.26 1059.47 L727.488 1058.07 L727.716 1058.73 L727.943 1058.74 L728.171 1059.96 L728.399 1059.65 L728.627 1059.06 L728.854 1059.17 L729.082 1057.14 L729.31 1058.98 L729.538 1059.64 L729.766 1060.1 L729.993 1059.5 L730.221 1061.63 L730.449 1060.57 L730.677 1060.44 L730.904 1057.76 L731.132 1057.85 L731.36 1058.13 L731.588 1058.29 L731.816 1058.51 L732.043 1058.97 L732.271 1060.56 L732.499 1060.66 L732.727 1061.18 L732.954 1060.42 L733.182 1060.76 L733.41 1060.85 L733.638 1061.35 L733.865 1063.5 L734.093 1063.19 L734.321 1063.11 L734.549 1060.86 L734.777 1061.79 L735.004 1064.2 L735.232 1061.37 L735.46 1061.63 L735.688 1059.66 L735.915 1060.16 L736.143 1059.18 L736.371 1059.05 L736.599 1059.9 L736.826 1059.97 L737.054 1059.95 L737.282 1061.85 L737.51 1062.33 L737.738 1063.25 L737.965 1063.19 L738.193 1060.63 L738.421 1062.01 L738.649 1061.58 L738.876 1061.99 L739.104 1062.25 L739.332 1061.28 L739.56 1063.19 L739.787 1062.3 L740.015 1061.48 L740.243 1057.73 L740.471 1055.52 L740.699 1054 L740.926 1052.52 L741.154 1052.34 L741.382 1053.19 L741.61 1055.06 L741.837 1057.91 L742.065 1057.92 L742.293 1057.2 L742.521 1056.83 L742.749 1057.42 L742.976 1058.68 L743.204 1057.44 L743.432 1056.01 L743.66 1057.28 L743.887 1059.27 L744.115 1058.58 L744.343 1056.33 L744.571 1058.56 L744.798 1057.67 L745.026 1057.36 L745.254 1059.22 L745.482 1060.59 L745.71 1063.15 L745.937 1062.32 L746.165 1061.21 L746.393 1060.55 L746.621 1059.4 L746.848 1060.87 L747.076 1059.73 L747.304 1059.12 L747.532 1058.97 L747.759 1057.5 L747.987 1058.52 L748.215 1058.01 L748.443 1059.87 L748.671 1060.22 L748.898 1058.75 L749.126 1061.46 L749.354 1061.47 L749.582 1060.85 L749.809 1062.46 L750.037 1062.13 L750.265 1060.43 L750.493 1060.3 L750.72 1059.72 L750.948 1057.46 L751.176 1060.7 L751.404 1061.57 L751.632 1061.98 L751.859 1061.21 L752.087 1061.22 L752.315 1060.68 L752.543 1061.1 L752.77 1061.73 L752.998 1063.52 L753.226 1063.87 L753.454 1064.21 L753.682 1065.27 L753.909 1063.28 L754.137 1061.87 L754.365 1061.26 L754.593 1060.06 L754.82 1059.91 L755.048 1059.9 L755.276 1060.28 L755.504 1062.11 L755.731 1062.91 L755.959 1064.13 L756.187 1062.89 L756.415 1063.04 L756.643 1064.52 L756.87 1064.11 L757.098 1064.62 L757.326 1065.43 L757.554 1062.51 L757.781 1064.56 L758.009 1065.64 L758.237 1065.91 L758.465 1066.23 L758.692 1062.39 L758.92 1064.1 L759.148 1063.93 L759.376 1063.79 L759.604 1061.61 L759.831 1061.82 L760.059 1061.5 L760.287 1060.78 L760.515 1061.81 L760.742 1061.86 L760.97 1063.86 L761.198 1064.46 L761.426 1066.7 L761.653 1066.33 L761.881 1066.43 L762.109 1066.54 L762.337 1067.08 L762.565 1066.02 L762.792 1063.95 L763.02 1065.68 L763.248 1062.99 L763.476 1061.58 L763.703 1062.06 L763.931 1063.51 L764.159 1063.73 L764.387 1065.33 L764.615 1065.22 L764.842 1065.15 L765.07 1065.81 L765.298 1067.04 L765.526 1068.17 L765.753 1069.22 L765.981 1068.31 L766.209 1067.24 L766.437 1065.49 L766.664 1066.94 L766.892 1065.61 L767.12 1064.22 L767.348 1063.99 L767.576 1063.36 L767.803 1065.48 L768.031 1065.03 L768.259 1064.36 L768.487 1065.84 L768.714 1062.89 L768.942 1061.33 L769.17 1062.44 L769.398 1060.43 L769.625 1059.29 L769.853 1057.78 L770.081 1056.93 L770.309 1056.83 L770.537 1057.33 L770.764 1057.92 L770.992 1058.39 L771.22 1059.05 L771.448 1060.78 L771.675 1062.31 L771.903 1061.23 L772.131 1062.27 L772.359 1063.6 L772.586 1064.88 L772.814 1063.64 L773.042 1064.86 L773.27 1064.98 L773.498 1066.54 L773.725 1066.31 L773.953 1068.48 L774.181 1070.26 L774.409 1070.89 L774.636 1069.94 L774.864 1070.5 L775.092 1071.04 L775.32 1071.73 L775.548 1070.96 L775.775 1071.74 L776.003 1072.24 L776.231 1073.28 L776.459 1071.05 L776.686 1071.65 L776.914 1072.38 L777.142 1071.83 L777.37 1068.21 L777.597 1067.34 L777.825 1068.25 L778.053 1069.05 L778.281 1068.24 L778.509 1068.18 L778.736 1066.9 L778.964 1067.07 L779.192 1065.73 L779.42 1066.5 L779.647 1066.75 L779.875 1066.22 L780.103 1064.86 L780.331 1064.23 L780.558 1062.57 L780.786 1062.4 L781.014 1060.36 L781.242 1057.49 L781.47 1055.71 L781.697 1055.34 L781.925 1055.77 L782.153 1055.99 L782.381 1056.23 L782.608 1058.14 L782.836 1058.99 L783.064 1058.62 L783.292 1059.97 L783.519 1058.76 L783.747 1057.8 L783.975 1056.25 L784.203 1055.01 L784.431 1053.9 L784.658 1053.48 L784.886 1053.36 L785.114 1053.4 L785.342 1053.72 L785.569 1054.37 L785.797 1052.92 L786.025 1054.04 L786.253 1054.29 L786.481 1053.88 L786.708 1053.47 L786.936 1052 L787.164 1050.53 L787.392 1049.42 L787.619 1049.34 L787.847 1050.91 L788.075 1051.49 L788.303 1050.87 L788.53 1051.27 L788.758 1051.76 L788.986 1053.06 L789.214 1053.85 L789.442 1054.32 L789.669 1055.63 L789.897 1057.3 L790.125 1059.64 L790.353 1060.13 L790.58 1060.08 L790.808 1059.89 L791.036 1057.31 L791.264 1056.35 L791.491 1055.46 L791.719 1056.31 L791.947 1058.25 L792.175 1058.87 L792.403 1060.01 L792.63 1059.54 L792.858 1058.42 L793.086 1058.86 L793.314 1058.77 L793.541 1059.31 L793.769 1059.81 L793.997 1059.97 L794.225 1058.39 L794.452 1058.09 L794.68 1057.69 L794.908 1058.09 L795.136 1056.86 L795.364 1057.12 L795.591 1055.18 L795.819 1054.38 L796.047 1056.01 L796.275 1055.72 L796.502 1057.91 L796.73 1055.65 L796.958 1053.68 L797.186 1051.64 L797.413 1052.04 L797.641 1051.77 L797.869 1049.87 L798.097 1049.5 L798.325 1049.2 L798.552 1050.24 L798.78 1050.93 L799.008 1053.2 L799.236 1051.63 L799.463 1050.65 L799.691 1052.41 L799.919 1051.29 L800.147 1049.61 L800.375 1048.88 L800.602 1049.56 L800.83 1048.31 L801.058 1048.04 L801.286 1049.32 L801.513 1047.75 L801.741 1048.26 L801.969 1049.96 L802.197 1046.58 L802.424 1046.07 L802.652 1047.5 L802.88 1046.76 L803.108 1047.69 L803.336 1046.11 L803.563 1044.18 L803.791 1045.85 L804.019 1046.16 L804.247 1044.76 L804.474 1045.82 L804.702 1044.75 L804.93 1046.26 L805.158 1046.75 L805.385 1046.89 L805.613 1047.05 L805.841 1044.83 L806.069 1045.16 L806.297 1043.66 L806.524 1043.85 L806.752 1045.17 L806.98 1045.87 L807.208 1046.19 L807.435 1044.64 L807.663 1046.16 L807.891 1047.63 L808.119 1048.41 L808.346 1047.34 L808.574 1048.01 L808.802 1048.67 L809.03 1049.85 L809.258 1051.93 L809.485 1049.87 L809.713 1049.23 L809.941 1049.82 L810.169 1049.4 L810.396 1050.02 L810.624 1047.99 L810.852 1049.64 L811.08 1049.11 L811.308 1047.56 L811.535 1047.69 L811.763 1047.41 L811.991 1047.18 L812.219 1046.39 L812.446 1047.42 L812.674 1046.8 L812.902 1046.53 L813.13 1045.23 L813.357 1047.91 L813.585 1049.4 L813.813 1049.32 L814.041 1046 L814.269 1047.24 L814.496 1048.2 L814.724 1047.32 L814.952 1046.8 L815.18 1047.85 L815.407 1047.09 L815.635 1045.73 L815.863 1044.99 L816.091 1044.57 L816.318 1044.58 L816.546 1044.97 L816.774 1046.18 L817.002 1045.58 L817.23 1046.12 L817.457 1046.58 L817.685 1048.69 L817.913 1046.22 L818.141 1047.1 L818.368 1047.59 L818.596 1046.89 L818.824 1046.59 L819.052 1046.27 L819.279 1046.97 L819.507 1046.66 L819.735 1046.94 L819.963 1047.39 L820.191 1046.68 L820.418 1048.05 L820.646 1047.87 L820.874 1046.45 L821.102 1047.95 L821.329 1047.7 L821.557 1045.22 L821.785 1043.6 L822.013 1041.57 L822.241 1040.24 L822.468 1038.96 L822.696 1038.32 L822.924 1039.86 L823.152 1040 L823.379 1040.24 L823.607 1042.65 L823.835 1043.4 L824.063 1043.99 L824.29 1045.17 L824.518 1046.87 L824.746 1044.99 L824.974 1043.73 L825.202 1045.05 L825.429 1046.8 L825.657 1046.11 L825.885 1045.97 L826.113 1046.78 L826.34 1047.93 L826.568 1048.31 L826.796 1047.19 L827.024 1048.63 L827.251 1047.69 L827.479 1047.23 L827.707 1047.8 L827.935 1047.96 L828.163 1049.45 L828.39 1049.68 L828.618 1048.65 L828.846 1050.12 L829.074 1048.31 L829.301 1047.13 L829.529 1046.84 L829.757 1046.05 L829.985 1047.47 L830.212 1047.77 L830.44 1047.37 L830.668 1045.81 L830.896 1044.87 L831.124 1045.79 L831.351 1047.63 L831.579 1048.14 L831.807 1049.64 L832.035 1049.18 L832.262 1049.55 L832.49 1046.1 L832.718 1047.49 L832.946 1047.55 L833.174 1046.67 L833.401 1047.04 L833.629 1047.3 L833.857 1046.48 L834.085 1048.17 L834.312 1050.01 L834.54 1049.25 L834.768 1048.77 L834.996 1048.69 L835.223 1048.92 L835.451 1049.84 L835.679 1049.96 L835.907 1049.61 L836.135 1049.88 L836.362 1049.94 L836.59 1051.48 L836.818 1050.8 L837.046 1051.28 L837.273 1052.52 L837.501 1051.76 L837.729 1050.88 L837.957 1049.83 L838.184 1050.07 L838.412 1049.59 L838.64 1048.72 L838.868 1047.55 L839.096 1048.85 L839.323 1048.43 L839.551 1047.85 L839.779 1048.95 L840.007 1050.11 L840.234 1052 L840.462 1052.71 L840.69 1052.15 L840.918 1052.44 L841.145 1051.73 L841.373 1049.93 L841.601 1050.66 L841.829 1049.22 L842.057 1050.9 L842.284 1050.7 L842.512 1050.93 L842.74 1052.32 L842.968 1052.9 L843.195 1053.22 L843.423 1051.9 L843.651 1052.18 L843.879 1053.75 L844.107 1053.31 L844.334 1053.66 L844.562 1053.34 L844.79 1052.81 L845.018 1052.78 L845.245 1052.76 L845.473 1052.64 L845.701 1052.91 L845.929 1053.61 L846.156 1052.52 L846.384 1053.46 L846.612 1053.4 L846.84 1053.58 L847.068 1054.77 L847.295 1054.97 L847.523 1052.21 L847.751 1053.16 L847.979 1053.48 L848.206 1054.07 L848.434 1052.79 L848.662 1053.02 L848.89 1054.49 L849.117 1053.58 L849.345 1054.51 L849.573 1053.15 L849.801 1053.6 L850.029 1054.13 L850.256 1054.38 L850.484 1054.81 L850.712 1055.48 L850.94 1055.65 L851.167 1056.56 L851.395 1057.34 L851.623 1057.02 L851.851 1057.82 L852.078 1056.38 L852.306 1057.92 L852.534 1058.47 L852.762 1059.37 L852.99 1058.19 L853.217 1058.13 L853.445 1056.05 L853.673 1057.15 L853.901 1055.94 L854.128 1056.54 L854.356 1056 L854.584 1056.14 L854.812 1055.58 L855.04 1055.31 L855.267 1056.86 L855.495 1057.51 L855.723 1058.26 L855.951 1059.7 L856.178 1058.92 L856.406 1058.52 L856.634 1058.49 L856.862 1059.31 L857.089 1057.96 L857.317 1058.25 L857.545 1056.67 L857.773 1054.99 L858.001 1054.9 L858.228 1056.82 L858.456 1057.21 L858.684 1053.38 L858.912 1053.43 L859.139 1053.65 L859.367 1056.58 L859.595 1056.42 L859.823 1057.45 L860.05 1058.63 L860.278 1060.27 L860.506 1060.07 L860.734 1060.45 L860.962 1061.97 L861.189 1062.34 L861.417 1063.07 L861.645 1064.05 L861.873 1062.93 L862.1 1063.04 L862.328 1063.2 L862.556 1062.6 L862.784 1062.72 L863.011 1063.08 L863.239 1062.67 L863.467 1061.21 L863.695 1061.42 L863.923 1061.5 L864.15 1061.47 L864.378 1062.15 L864.606 1061.68 L864.834 1060.72 L865.061 1061.84 L865.289 1059.69 L865.517 1061.33 L865.745 1059.74 L865.973 1060.71 L866.2 1062.02 L866.428 1061.94 L866.656 1061.02 L866.884 1060.77 L867.111 1059.49 L867.339 1060.11 L867.567 1059.31 L867.795 1059.21 L868.022 1060.63 L868.25 1059.34 L868.478 1059.16 L868.706 1058.21 L868.934 1058.29 L869.161 1057.22 L869.389 1057.57 L869.617 1058.75 L869.845 1059.33 L870.072 1059.55 L870.3 1060.47 L870.528 1061.07 L870.756 1060.91 L870.983 1061.59 L871.211 1061.38 L871.439 1061.61 L871.667 1062.67 L871.895 1061.47 L872.122 1060.2 L872.35 1060.27 L872.578 1060.97 L872.806 1060.03 L873.033 1061.12 L873.261 1060.93 L873.489 1059.59 L873.717 1059.17 L873.944 1059.08 L874.172 1060.6 L874.4 1062.84 L874.628 1060.88 L874.856 1060.01 L875.083 1059.72 L875.311 1060.8 L875.539 1060.51 L875.767 1061.53 L875.994 1060.95 L876.222 1060.92 L876.45 1062.04 L876.678 1063.3 L876.906 1065.45 L877.133 1063.69 L877.361 1063.02 L877.589 1063.27 L877.817 1063.14 L878.044 1062.07 L878.272 1062.92 L878.5 1063.84 L878.728 1064.61 L878.955 1065.89 L879.183 1065.56 L879.411 1065.91 L879.639 1067.05 L879.867 1067.28 L880.094 1067.83 L880.322 1066.45 L880.55 1066.29 L880.778 1066.62 L881.005 1066.19 L881.233 1066.28 L881.461 1067.38 L881.689 1069.34 L881.916 1071.17 L882.144 1072.95 L882.372 1073.72 L882.6 1074.5 L882.828 1072.15 L883.055 1072.35 L883.283 1073.01 L883.511 1070.74 L883.739 1070.8 L883.966 1070.33 L884.194 1070.65 L884.422 1071.22 L884.65 1069.06 L884.877 1069.01 L885.105 1069.74 L885.333 1069.94 L885.561 1069.32 L885.789 1068.64 L886.016 1069.45 L886.244 1068.99 L886.472 1066.36 L886.7 1065.92 L886.927 1067.43 L887.155 1069.37 L887.383 1070.08 L887.611 1070.13 L887.839 1069.28 L888.066 1068.7 L888.294 1067.94 L888.522 1068.34 L888.75 1067 L888.977 1067.55 L889.205 1065.4 L889.433 1066.01 L889.661 1064.66 L889.888 1065.26 L890.116 1066.03 L890.344 1064.1 L890.572 1064 L890.8 1061.7 L891.027 1062.91 L891.255 1063.96 L891.483 1064.29 L891.711 1064.09 L891.938 1064.21 L892.166 1063.41 L892.394 1064.71 L892.622 1064.8 L892.849 1061.65 L893.077 1060.71 L893.305 1061.53 L893.533 1059.99 L893.761 1059.33 L893.988 1057.94 L894.216 1058.16 L894.444 1058.39 L894.672 1056.75 L894.899 1055.85 L895.127 1056.77 L895.355 1056.35 L895.583 1055.34 L895.81 1056.92 L896.038 1057.17 L896.266 1058.31 L896.494 1058 L896.722 1058.23 L896.949 1058.96 L897.177 1058.52 L897.405 1056.94 L897.633 1056.39 L897.86 1053.28 L898.088 1054.19 L898.316 1054.01 L898.544 1053.79 L898.772 1055.01 L898.999 1054.5 L899.227 1052.31 L899.455 1051.38 L899.683 1050.85 L899.91 1051.07 L900.138 1051.23 L900.366 1052.53 L900.594 1051.86 L900.821 1052.62 L901.049 1051.55 L901.277 1050.58 L901.505 1047.93 L901.733 1047.96 L901.96 1048.03 L902.188 1048.35 L902.416 1048.07 L902.644 1047.35 L902.871 1045.88 L903.099 1044.55 L903.327 1045.56 L903.555 1045.8 L903.782 1045.26 L904.01 1044.92 L904.238 1045.34 L904.466 1047.06 L904.694 1048.17 L904.921 1047.77 L905.149 1047.93 L905.377 1048.28 L905.605 1050.01 L905.832 1050.19 L906.06 1051.54 L906.288 1052.13 L906.516 1051.54 L906.743 1051.06 L906.971 1051.34 L907.199 1049.71 L907.427 1049.73 L907.655 1050.61 L907.882 1051.01 L908.11 1049.38 L908.338 1049.18 L908.566 1050.97 L908.793 1051.14 L909.021 1052.19 L909.249 1051.98 L909.477 1051.6 L909.705 1052.45 L909.932 1052.76 L910.16 1052.94 L910.388 1052.12 L910.616 1051.79 L910.843 1051.19 L911.071 1051.47 L911.299 1051.9 L911.527 1053.94 L911.754 1055.37 L911.982 1056.69 L912.21 1056.8 L912.438 1057.73 L912.666 1058.22 L912.893 1056.73 L913.121 1054.01 L913.349 1054.59 L913.577 1053.98 L913.804 1053.67 L914.032 1052.19 L914.26 1053.74 L914.488 1055.57 L914.715 1054.82 L914.943 1056.52 L915.171 1056.8 L915.399 1055.68 L915.627 1056.26 L915.854 1057.68 L916.082 1057.9 L916.31 1059.03 L916.538 1058.84 L916.765 1058.86 L916.993 1060.16 L917.221 1060.17 L917.449 1057.77 L917.676 1057.82 L917.904 1058.85 L918.132 1059.5 L918.36 1060.88 L918.588 1061.79 L918.815 1061.37 L919.043 1061.1 L919.271 1061.3 L919.499 1060.06 L919.726 1060.11 L919.954 1059.96 L920.182 1060.18 L920.41 1060.38 L920.638 1061.9 L920.865 1062.71 L921.093 1061.8 L921.321 1062.81 L921.549 1063.59 L921.776 1064.98 L922.004 1064.3 L922.232 1065.36 L922.46 1066.6 L922.687 1064.6 L922.915 1064.59 L923.143 1064.63 L923.371 1063.94 L923.599 1062.36 L923.826 1060.57 L924.054 1059.97 L924.282 1061.48 L924.51 1061.8 L924.737 1061.75 L924.965 1063.01 L925.193 1063.08 L925.421 1063.24 L925.648 1063.49 L925.876 1063.47 L926.104 1061.22 L926.332 1062.78 L926.56 1062.25 L926.787 1062.84 L927.015 1063.81 L927.243 1064.4 L927.471 1063.42 L927.698 1064.67 L927.926 1064.54 L928.154 1063.49 L928.382 1063.69 L928.609 1062.83 L928.837 1063.36 L929.065 1064.39 L929.293 1065.9 L929.521 1066.23 L929.748 1066.94 L929.976 1067.89 L930.204 1068.28 L930.432 1069.79 L930.659 1070.04 L930.887 1070.71 L931.115 1071.97 L931.343 1071.83 L931.571 1072.55 L931.798 1072.19 L932.026 1072.3 L932.254 1071.84 L932.482 1071.57 L932.709 1071.07 L932.937 1072.02 L933.165 1073.27 L933.393 1075.67 L933.62 1074.36 L933.848 1074.14 L934.076 1074.7 L934.304 1075.32 L934.532 1075.65 L934.759 1075.76 L934.987 1074.66 L935.215 1071.95 L935.443 1072.2 L935.67 1071.84 L935.898 1071.84 L936.126 1072.44 L936.354 1072.61 L936.581 1072.32 L936.809 1072.34 L937.037 1073.47 L937.265 1073.82 L937.493 1074.53 L937.72 1073.61 L937.948 1073.34 L938.176 1073.03 L938.404 1073.13 L938.631 1073.03 L938.859 1072.91 L939.087 1074.07 L939.315 1074.98 L939.542 1075.54 L939.77 1076.5 L939.998 1076.25 L940.226 1075.69 L940.454 1075.59 L940.681 1076.74 L940.909 1076.37 L941.137 1076.06 L941.365 1075.65 L941.592 1075.51 L941.82 1077.25 L942.048 1077.5 L942.276 1078.1 L942.504 1077.11 L942.731 1076.64 L942.959 1076.42 L943.187 1077.49 L943.415 1076.38 L943.642 1077.29 L943.87 1076.86 L944.098 1077.16 L944.326 1076.55 L944.553 1075.6 L944.781 1075.09 L945.009 1075.21 L945.237 1074.42 L945.465 1074.31 L945.692 1073.41 L945.92 1072.65 L946.148 1072.39 L946.376 1071.9 L946.603 1071.63 L946.831 1071.25 L947.059 1068.37 L947.287 1068.71 L947.514 1065.72 L947.742 1066.19 L947.97 1067.1 L948.198 1068.2 L948.426 1066.78 L948.653 1067.75 L948.881 1069.53 L949.109 1070.99 L949.337 1070.41 L949.564 1070.19 L949.792 1071.14 L950.02 1070.83 L950.248 1071.92 L950.475 1072.86 L950.703 1074.42 L950.931 1074.99 L951.159 1073.93 L951.387 1072.83 L951.614 1073.66 L951.842 1073.48 L952.07 1072.19 L952.298 1073.12 L952.525 1073.1 L952.753 1074.11 L952.981 1074.59 L953.209 1075.17 L953.437 1076.21 L953.664 1077.19 L953.892 1076.3 L954.12 1075.71 L954.348 1076.21 L954.575 1075.89 L954.803 1074.9 L955.031 1072.27 L955.259 1072.26 L955.486 1071.96 L955.714 1072.13 L955.942 1072.87 L956.17 1074.09 L956.398 1073.4 L956.625 1072.11 L956.853 1072.49 L957.081 1070.92 L957.309 1070.71 L957.536 1070.08 L957.764 1070.26 L957.992 1070.72 L958.22 1070.04 L958.447 1070.64 L958.675 1071.01 L958.903 1071.69 L959.131 1070.64 L959.359 1070.74 L959.586 1070.51 L959.814 1070.51 L960.042 1070.87 L960.27 1071.99 L960.497 1071.68 L960.725 1074.01 L960.953 1072.31 L961.181 1071.9 L961.408 1072.34 L961.636 1071.6 L961.864 1071.57 L962.092 1071.73 L962.32 1072.08 L962.547 1072.49 L962.775 1071.56 L963.003 1070.99 L963.231 1071.63 L963.458 1072.1 L963.686 1073.79 L963.914 1073.53 L964.142 1073.97 L964.37 1074.17 L964.597 1073.28 L964.825 1074.18 L965.053 1074.89 L965.281 1075.89 L965.508 1077.16 L965.736 1078.32 L965.964 1077.85 L966.192 1077.39 L966.419 1077.54 L966.647 1078.01 L966.875 1076.11 L967.103 1076.68 L967.331 1077.66 L967.558 1078.83 L967.786 1079.7 L968.014 1080.15 L968.242 1080.17 L968.469 1081.01 L968.697 1079.92 L968.925 1080.6 L969.153 1080.68 L969.38 1080.53 L969.608 1079.67 L969.836 1080.41 L970.064 1080.66 L970.292 1079.67 L970.519 1080.66 L970.747 1081.45 L970.975 1081.3 L971.203 1082.16 L971.43 1080.91 L971.658 1080.51 L971.886 1081.9 L972.114 1081.11 L972.341 1081.12 L972.569 1079.95 L972.797 1080.81 L973.025 1081.86 L973.253 1080.94 L973.48 1081.27 L973.708 1082.26 L973.936 1083.05 L974.164 1082.3 L974.391 1082.11 L974.619 1082.02 L974.847 1082.02 L975.075 1081.78 L975.303 1081.85 L975.53 1081.75 L975.758 1079.8 L975.986 1081.1 L976.214 1079.53 L976.441 1079.54 L976.669 1079.07 L976.897 1079.37 L977.125 1077.92 L977.352 1078.09 L977.58 1079.16 L977.808 1078.25 L978.036 1078.15 L978.264 1079.32 L978.491 1080.1 L978.719 1079.86 L978.947 1081.73 L979.175 1080.79 L979.402 1080.12 L979.63 1080.62 L979.858 1080.09 L980.086 1078.73 L980.313 1078.71 L980.541 1078.87 L980.769 1078.56 L980.997 1077.96 L981.225 1077.91 L981.452 1078.42 L981.68 1078.23 L981.908 1079.77 L982.136 1080.03 L982.363 1080.63 L982.591 1078.88 L982.819 1080.04 L983.047 1079.98 L983.274 1079.84 L983.502 1078.19 L983.73 1078.19 L983.958 1078.89 L984.186 1077.47 L984.413 1075.99 L984.641 1076.22 L984.869 1076.65 L985.097 1075.58 L985.324 1074.72 L985.552 1075.87 L985.78 1075.41 L986.008 1073.99 L986.236 1074.27 L986.463 1072.27 L986.691 1071.33 L986.919 1070.59 L987.147 1071.11 L987.374 1071.46 L987.602 1072.17 L987.83 1071.96 L988.058 1070.11 L988.285 1070.72 L988.513 1070.79 L988.741 1071.66 L988.969 1071.39 L989.197 1071.51 L989.424 1069.44 L989.652 1069.61 L989.88 1070.11 L990.108 1070.03 L990.335 1069.75 L990.563 1069.24 L990.791 1069.9 L991.019 1070.11 L991.246 1068.4 L991.474 1067.33 L991.702 1065.94 L991.93 1067.22 L992.158 1067.57 L992.385 1068 L992.613 1068.84 L992.841 1068.23 L993.069 1066.4 L993.296 1067.05 L993.524 1065.44 L993.752 1065.48 L993.98 1066.85 L994.207 1067.45 L994.435 1066.47 L994.663 1067.96 L994.891 1068.76 L995.119 1069.27 L995.346 1070.02 L995.574 1069.45 L995.802 1069.65 L996.03 1070.04 L996.257 1070.67 L996.485 1070.57 L996.713 1070.13 L996.941 1069.81 L997.169 1070.78 L997.396 1072.28 L997.624 1072.84 L997.852 1073.59 L998.08 1073.52 L998.307 1074.98 L998.535 1075.72 L998.763 1075.59 L998.991 1073.75 L999.218 1074.18 L999.446 1075.71 L999.674 1076.5 L999.902 1074.16 L1000.13 1074.5 L1000.36 1074.8 L1000.59 1075.02 L1000.81 1074.82 L1001.04 1074.43 L1001.27 1074.84 L1001.5 1074.74 L1001.72 1074.92 L1001.95 1072.88 L1002.18 1073.28 L1002.41 1072.96 L1002.64 1073 L1002.86 1071.87 L1003.09 1073.43 L1003.32 1072.6 L1003.55 1073.81 L1003.77 1073.27 L1004 1073.14 L1004.23 1073.42 L1004.46 1073.3 L1004.68 1072.92 L1004.91 1071.53 L1005.14 1073.13 L1005.37 1074 L1005.6 1074.47 L1005.82 1075.59 L1006.05 1074.71 L1006.28 1076.06 L1006.51 1076.7 L1006.73 1075.39 L1006.96 1074.21 L1007.19 1074.93 L1007.42 1074.84 L1007.65 1074.55 L1007.87 1075.01 L1008.1 1075.19 L1008.33 1075.44 L1008.56 1074.72 L1008.78 1075.02 L1009.01 1075.55 L1009.24 1075.01 L1009.47 1075.17 L1009.7 1075.17 L1009.92 1074.57 L1010.15 1073.87 L1010.38 1074.73 L1010.61 1074.9 L1010.83 1075 L1011.06 1076.6 L1011.29 1075.85 L1011.52 1076.71 L1011.75 1075.28 L1011.97 1075.53 L1012.2 1076.11 L1012.43 1076.64 L1012.66 1077.61 L1012.88 1075.8 L1013.11 1076.36 L1013.34 1077.02 L1013.57 1076.91 L1013.8 1075.99 L1014.02 1074.83 L1014.25 1075.12 L1014.48 1073.57 L1014.71 1073.48 L1014.93 1073.38 L1015.16 1073.15 L1015.39 1072.77 L1015.62 1073.37 L1015.85 1072.57 L1016.07 1072.77 L1016.3 1070.58 L1016.53 1070.24 L1016.76 1070.98 L1016.98 1071.58 L1017.21 1071.07 L1017.44 1070.72 L1017.67 1069.86 L1017.9 1070.18 L1018.12 1070.05 L1018.35 1069.15 L1018.58 1069.36 L1018.81 1069.46 L1019.03 1068.7 L1019.26 1067.52 L1019.49 1067.75 L1019.72 1068.26 L1019.95 1067.28 L1020.17 1067.47 L1020.4 1068.91 L1020.63 1068.19 L1020.86 1069.1 L1021.08 1069.9 L1021.31 1069.19 L1021.54 1070.88 L1021.77 1070.73 L1022 1070.61 L1022.22 1070.08 L1022.45 1071.23 L1022.68 1071.15 L1022.91 1070.96 L1023.13 1070.81 L1023.36 1071.89 L1023.59 1070.87 L1023.82 1070.99 L1024.05 1070.37 L1024.27 1070.46 L1024.5 1072.04 L1024.73 1072.97 L1024.96 1072.83 L1025.18 1072.5 L1025.41 1072.61 L1025.64 1071.02 L1025.87 1071.71 L1026.1 1070.96 L1026.32 1072.4 L1026.55 1073.04 L1026.78 1073.99 L1027.01 1074.55 L1027.23 1075.53 L1027.46 1076.62 L1027.69 1077.04 L1027.92 1076.08 L1028.15 1076.33 L1028.37 1077 L1028.6 1076.91 L1028.83 1076.83 L1029.06 1076.22 L1029.28 1075.42 L1029.51 1073.54 L1029.74 1073.64 L1029.97 1073.52 L1030.2 1072.85 L1030.42 1073.45 L1030.65 1073.86 L1030.88 1073.11 L1031.11 1072.69 L1031.33 1071.03 L1031.56 1070.92 L1031.79 1071.3 L1032.02 1070.84 L1032.25 1069.78 L1032.47 1070.3 L1032.7 1070.7 L1032.93 1069.45 L1033.16 1069.72 L1033.38 1069.21 L1033.61 1069.89 L1033.84 1068.6 L1034.07 1067.9 L1034.3 1068.93 L1034.52 1068.38 L1034.75 1068.76 L1034.98 1068.72 L1035.21 1067.9 L1035.43 1068.04 L1035.66 1067.98 L1035.89 1067.48 L1036.12 1066.43 L1036.35 1066.65 L1036.57 1068.26 L1036.8 1068.6 L1037.03 1068.3 L1037.26 1068.02 L1037.48 1068.49 L1037.71 1069.12 L1037.94 1069.79 L1038.17 1068.82 L1038.4 1069.11 L1038.62 1069.88 L1038.85 1069.71 L1039.08 1069.56 L1039.31 1070.45 L1039.53 1070.19 L1039.76 1070.22 L1039.99 1070.06 L1040.22 1070.84 L1040.44 1071.04 L1040.67 1072.75 L1040.9 1073.2 L1041.13 1073.01 L1041.36 1072.91 L1041.58 1073.97 L1041.81 1074.25 L1042.04 1072.96 L1042.27 1072.48 L1042.49 1071.92 L1042.72 1071.37 L1042.95 1071.81 L1043.18 1072.06 L1043.41 1072.81 L1043.63 1072.77 L1043.86 1072.84 L1044.09 1073.55 L1044.32 1073.69 L1044.54 1073.8 L1044.77 1073.67 L1045 1073.95 L1045.23 1073.15 L1045.46 1072 L1045.68 1072.88 L1045.91 1072.5 L1046.14 1072.04 L1046.37 1072.22 L1046.59 1073.05 L1046.82 1071.86 L1047.05 1071.45 L1047.28 1072.42 L1047.51 1072.01 L1047.73 1071.02 L1047.96 1071.13 L1048.19 1072.14 L1048.42 1071.04 L1048.64 1071.63 L1048.87 1071.84 L1049.1 1070.75 L1049.33 1070.51 L1049.56 1070.32 L1049.78 1070.17 L1050.01 1071.43 L1050.24 1070.97 L1050.47 1070.31 L1050.69 1069.87 L1050.92 1068.06 L1051.15 1069.19 L1051.38 1069.32 L1051.61 1069.86 L1051.83 1070.27 L1052.06 1070.4 L1052.29 1069.93 L1052.52 1070.06 L1052.74 1068.32 L1052.97 1067.04 L1053.2 1067.2 L1053.43 1067.16 L1053.66 1066.93 L1053.88 1066.66 L1054.11 1066.81 L1054.34 1066.39 L1054.57 1067.17 L1054.79 1068.31 L1055.02 1068.02 L1055.25 1068.34 L1055.48 1068.1 L1055.71 1067.77 L1055.93 1068.43 L1056.16 1068.64 L1056.39 1068.15 L1056.62 1067.65 L1056.84 1068.56 L1057.07 1068.74 L1057.3 1068.69 L1057.53 1067.69 L1057.76 1068.14 L1057.98 1069.41 L1058.21 1070.58 L1058.44 1069.84 L1058.67 1069.33 L1058.89 1069.62 L1059.12 1069.77 L1059.35 1070.16 L1059.58 1069.11 L1059.81 1068.55 L1060.03 1067.89 L1060.26 1068.32 L1060.49 1068.36 L1060.72 1068.2 L1060.94 1068.72 L1061.17 1069.5 L1061.4 1067.9 L1061.63 1068.67 L1061.86 1068.91 L1062.08 1068.82 L1062.31 1069.08 L1062.54 1069.11 L1062.77 1069.25 L1062.99 1070.99 L1063.22 1070.73 L1063.45 1071.08 L1063.68 1071.29 L1063.91 1071.43 L1064.13 1071.5 L1064.36 1071.75 L1064.59 1071.3 L1064.82 1070.8 L1065.04 1069.72 L1065.27 1069.57 L1065.5 1069.81 L1065.73 1069.44 L1065.96 1069.21 L1066.18 1069.03 L1066.41 1068.69 L1066.64 1066.38 L1066.87 1067.14 L1067.09 1066.75 L1067.32 1067.26 L1067.55 1066.09 L1067.78 1066.59 L1068.01 1067 L1068.23 1067.35 L1068.46 1067.47 L1068.69 1067.96 L1068.92 1067.11 L1069.14 1066.67 L1069.37 1067.26 L1069.6 1067.47 L1069.83 1068.42 L1070.06 1069.16 L1070.28 1070.27 L1070.51 1071.14 L1070.74 1070.65 L1070.97 1071.52 L1071.19 1073.05 L1071.42 1073.45 L1071.65 1073.3 L1071.88 1073.74 L1072.11 1074.04 L1072.33 1074.72 L1072.56 1075.11 L1072.79 1076.33 L1073.02 1076.52 L1073.24 1076.49 L1073.47 1076.74 L1073.7 1076.69 L1073.93 1077.09 L1074.16 1078.33 L1074.38 1077.4 L1074.61 1077.52 L1074.84 1077.34 L1075.07 1076.37 L1075.29 1077.56 L1075.52 1077.6 L1075.75 1078.44 L1075.98 1079.43 L1076.2 1079.9 L1076.43 1080.37 L1076.66 1078.87 L1076.89 1079.16 L1077.12 1080.59 L1077.34 1080.23 L1077.57 1079.87 L1077.8 1079.16 L1078.03 1080.71 L1078.25 1080.64 L1078.48 1079.96 L1078.71 1079.52 L1078.94 1080.23 L1079.17 1080.3 L1079.39 1079.12 L1079.62 1078.35 L1079.85 1076.48 L1080.08 1076.77 L1080.3 1076.79 L1080.53 1076.68 L1080.76 1075.97 L1080.99 1075.24 L1081.22 1075.25 L1081.44 1074.12 L1081.67 1074.52 L1081.9 1075.26 L1082.13 1075.73 L1082.35 1076.68 L1082.58 1076.05 L1082.81 1076.03 L1083.04 1076.1 L1083.27 1075.39 L1083.49 1075.1 L1083.72 1075.54 L1083.95 1075.74 L1084.18 1074.77 L1084.4 1073.99 L1084.63 1074.94 L1084.86 1074.41 L1085.09 1073.8 L1085.32 1073.12 L1085.54 1073.41 L1085.77 1072.17 L1086 1072.96 L1086.23 1072.01 L1086.45 1072.32 L1086.68 1071.56 L1086.91 1071.57 L1087.14 1072.57 L1087.37 1073.01 L1087.59 1072.31 L1087.82 1072.12 L1088.05 1073.09 L1088.28 1072.93 L1088.5 1073.59 L1088.73 1072.83 L1088.96 1072.79 L1089.19 1073.3 L1089.42 1073.1 L1089.64 1074.93 L1089.87 1074.37 L1090.1 1074.92 L1090.33 1075.39 L1090.55 1075.38 L1090.78 1075.35 L1091.01 1074.94 L1091.24 1075.23 L1091.47 1075.1 L1091.69 1075.49 L1091.92 1076.21 L1092.15 1076.8 L1092.38 1073.18 L1092.6 1073.91 L1092.83 1074.48 L1093.06 1075.54 L1093.29 1075.4 L1093.52 1076.08 L1093.74 1076.84 L1093.97 1076.26 L1094.2 1076.51 L1094.43 1076.87 L1094.65 1075.26 L1094.88 1074.21 L1095.11 1073.57 L1095.34 1074.64 L1095.57 1075.4 L1095.79 1075.24 L1096.02 1075.29 L1096.25 1073.47 L1096.48 1073.01 L1096.7 1072.86 L1096.93 1072.1 L1097.16 1072.21 L1097.39 1072.98 L1097.62 1072.91 L1097.84 1072.58 L1098.07 1073.11 L1098.3 1073.89 L1098.53 1075.05 L1098.75 1074.98 L1098.98 1074.91 L1099.21 1075.56 L1099.44 1075.27 L1099.67 1074.66 L1099.89 1073.91 L1100.12 1073.26 L1100.35 1073.56 L1100.58 1074.18 L1100.8 1074.98 L1101.03 1075.02 L1101.26 1073.46 L1101.49 1072.9 L1101.72 1072.9 L1101.94 1071.74 L1102.17 1069.77 L1102.4 1069.97 L1102.63 1070.38 L1102.85 1070.89 L1103.08 1069.47 L1103.31 1069.76 L1103.54 1070.57 L1103.77 1070.05 L1103.99 1070.59 L1104.22 1071.06 L1104.45 1070.8 L1104.68 1071.55 L1104.9 1071.88 L1105.13 1073.12 L1105.36 1072.99 L1105.59 1072.1 L1105.82 1071.56 L1106.04 1071.82 L1106.27 1071.87 L1106.5 1071.72 L1106.73 1071.66 L1106.95 1072.02 L1107.18 1072.4 L1107.41 1072.05 L1107.64 1073.52 L1107.87 1074.22 L1108.09 1074.55 L1108.32 1075.49 L1108.55 1074.71 L1108.78 1074.04 L1109 1074.36 L1109.23 1075.81 L1109.46 1076.29 L1109.69 1076.94 L1109.92 1076.18 L1110.14 1075.83 L1110.37 1075.01 L1110.6 1074.44 L1110.83 1075.73 L1111.05 1075.27 L1111.28 1076.31 L1111.51 1076.64 L1111.74 1076.23 L1111.97 1076.59 L1112.19 1076.8 L1112.42 1076.7 L1112.65 1077.56 L1112.88 1076.94 L1113.1 1076.17 L1113.33 1076.9 L1113.56 1077.39 L1113.79 1077.11 L1114.01 1076.86 L1114.24 1076.26 L1114.47 1076.88 L1114.7 1077.53 L1114.93 1078.25 L1115.15 1077.43 L1115.38 1077.5 L1115.61 1077.87 L1115.84 1077.48 L1116.06 1076.83 L1116.29 1076.09 L1116.52 1076.77 L1116.75 1077.19 L1116.98 1077.76 L1117.2 1078.15 L1117.43 1078.18 L1117.66 1078.36 L1117.89 1078.93 L1118.11 1079.22 L1118.34 1081.05 L1118.57 1081.42 L1118.8 1081.07 L1119.03 1081.57 L1119.25 1080.46 L1119.48 1079.54 L1119.71 1080.74 L1119.94 1081.15 L1120.16 1081.05 L1120.39 1080.57 L1120.62 1079.91 L1120.85 1079.23 L1121.08 1078.34 L1121.3 1079.64 L1121.53 1079.76 L1121.76 1079.58 L1121.99 1080.41 L1122.21 1079.74 L1122.44 1080.4 L1122.67 1081.42 L1122.9 1082.14 L1123.13 1082.47 L1123.35 1082.58 L1123.58 1082.62 L1123.81 1082.98 L1124.04 1083.49 L1124.26 1083 L1124.49 1083.39 L1124.72 1083.22 L1124.95 1083.05 L1125.18 1081.89 L1125.4 1082.96 L1125.63 1082.64 L1125.86 1083.64 L1126.09 1083.83 L1126.31 1084.08 L1126.54 1084.71 L1126.77 1085.29 L1127 1084.99 L1127.23 1085.52 L1127.45 1085.08 L1127.68 1085.43 L1127.91 1086.22 L1128.14 1084.09 L1128.36 1083.6 L1128.59 1082.74 L1128.82 1082.07 L1129.05 1082.71 L1129.28 1084.18 L1129.5 1084.68 L1129.73 1084.66 L1129.96 1083.69 L1130.19 1083.42 L1130.41 1084.12 L1130.64 1083.35 L1130.87 1083.5 L1131.1 1082.8 L1131.33 1081.48 L1131.55 1081.92 L1131.78 1082.35 L1132.01 1082.75 L1132.24 1083.17 L1132.46 1083.57 L1132.69 1084.01 L1132.92 1082.31 L1133.15 1083.03 L1133.38 1083.27 L1133.6 1084.87 L1133.83 1083.24 L1134.06 1081.96 L1134.29 1080.95 L1134.51 1081.25 L1134.74 1082.08 L1134.97 1081.18 L1135.2 1082 L1135.43 1081.19 L1135.65 1081.27 L1135.88 1081.97 L1136.11 1082.52 L1136.34 1082.09 L1136.56 1082.21 L1136.79 1081.87 L1137.02 1083.15 L1137.25 1083.51 L1137.48 1084.95 L1137.7 1083.87 L1137.93 1082.77 L1138.16 1082.19 L1138.39 1082.14 L1138.61 1081.2 L1138.84 1082.18 L1139.07 1081.5 L1139.3 1081.9 L1139.53 1082.79 L1139.75 1083.17 L1139.98 1082.89 L1140.21 1083.35 L1140.44 1083.57 L1140.66 1082.28 L1140.89 1082.96 L1141.12 1083.51 L1141.35 1083.43 L1141.58 1083.26 L1141.8 1082 L1142.03 1081.7 L1142.26 1080.9 L1142.49 1079.88 L1142.71 1079.69 L1142.94 1079.85 L1143.17 1079.84 L1143.4 1080.34 L1143.63 1080.86 L1143.85 1080.72 L1144.08 1079.73 L1144.31 1080.74 L1144.54 1081.1 L1144.76 1081.93 L1144.99 1081.27 L1145.22 1081.01 L1145.45 1081.77 L1145.68 1082.05 L1145.9 1082.11 L1146.13 1082.18 L1146.36 1081.04 L1146.59 1080.93 L1146.81 1080.63 L1147.04 1081.53 L1147.27 1081.61 L1147.5 1082.39 L1147.73 1080.84 L1147.95 1080.82 L1148.18 1081.35 L1148.41 1080.28 L1148.64 1080.44 L1148.86 1079.77 L1149.09 1079.6 L1149.32 1078.58 L1149.55 1078.56 L1149.77 1079.77 L1150 1079.66 L1150.23 1079.59 L1150.46 1079 L1150.69 1079.75 L1150.91 1079.55 L1151.14 1079.5 L1151.37 1079.6 L1151.6 1078.54 L1151.82 1079.44 L1152.05 1079.9 L1152.28 1080.53 L1152.51 1080.15 L1152.74 1080.41 L1152.96 1080.01 L1153.19 1081.42 L1153.42 1081.19 L1153.65 1081.83 L1153.87 1082.26 L1154.1 1083.48 L1154.33 1083.93 L1154.56 1083.97 L1154.79 1084.51 L1155.01 1085.41 L1155.24 1085.44 L1155.47 1085.17 L1155.7 1085.37 L1155.92 1086.13 L1156.15 1085.51 L1156.38 1086.43 L1156.61 1086.7 L1156.84 1087.13 L1157.06 1087.67 L1157.29 1086.93 L1157.52 1087.62 L1157.75 1087.9 L1157.97 1087.7 L1158.2 1088.61 L1158.43 1088.57 L1158.66 1088.52 L1158.89 1088.03 L1159.11 1086.88 L1159.34 1085.39 L1159.57 1085.83 L1159.8 1085.65 L1160.02 1085.22 L1160.25 1086.2 L1160.48 1087.5 L1160.71 1087 L1160.94 1086.58 L1161.16 1087.01 L1161.39 1087.33 L1161.62 1086.46 L1161.85 1086.51 L1162.07 1087.08 L1162.3 1087.62 L1162.53 1087.19 L1162.76 1086.98 L1162.99 1087.21 L1163.21 1086.47 L1163.44 1086.09 L1163.67 1086.36 L1163.9 1085.16 L1164.12 1084.95 L1164.35 1084.68 L1164.58 1084.34 L1164.81 1083.83 L1165.04 1083.96 L1165.26 1083.51 L1165.49 1082.79 L1165.72 1083.63 L1165.95 1083.16 L1166.17 1082.58 L1166.4 1083.7 L1166.63 1083.3 L1166.86 1082.96 L1167.09 1082.93 L1167.31 1082.46 L1167.54 1082.92 L1167.77 1083.59 L1168 1084.22 L1168.22 1084.27 L1168.45 1084.2 L1168.68 1083.29 L1168.91 1082.07 L1169.14 1081.53 L1169.36 1081.91 L1169.59 1081.34 L1169.82 1081.92 L1170.05 1081.64 L1170.27 1082.46 L1170.5 1083.87 L1170.73 1083.42 L1170.96 1083.41 L1171.19 1082.71 L1171.41 1083.74 L1171.64 1084.18 L1171.87 1083.88 L1172.1 1084.44 L1172.32 1083.56 L1172.55 1084.06 L1172.78 1084.28 L1173.01 1082.76 L1173.24 1081.85 L1173.46 1080.54 L1173.69 1080.05 L1173.92 1080.53 L1174.15 1079.73 L1174.37 1080.43 L1174.6 1080.48 L1174.83 1080.56 L1175.06 1081.14 L1175.29 1081.39 L1175.51 1080.54 L1175.74 1080.87 L1175.97 1081.69 L1176.2 1079.9 L1176.42 1079.54 L1176.65 1080.28 L1176.88 1081.13 L1177.11 1080.6 L1177.34 1080.69 L1177.56 1080.58 L1177.79 1080.37 L1178.02 1081.06 L1178.25 1080.46 L1178.47 1079.93 L1178.7 1079.88 L1178.93 1080.87 L1179.16 1080.62 L1179.39 1079.29 L1179.61 1078.72 L1179.84 1077.88 L1180.07 1078.43 L1180.3 1079.01 L1180.52 1077.45 L1180.75 1078.72 L1180.98 1079.12 L1181.21 1079.15 L1181.44 1079.32 L1181.66 1080.13 L1181.89 1080.12 L1182.12 1079.14 L1182.35 1079.16 L1182.57 1078.95 L1182.8 1078.7 L1183.03 1077.21 L1183.26 1077.23 L1183.49 1077.66 L1183.71 1077.93 L1183.94 1078.63 L1184.17 1079.9 L1184.4 1080.28 L1184.62 1079.82 L1184.85 1080.22 L1185.08 1081.49 L1185.31 1081.4 L1185.53 1081.71 L1185.76 1082.72 L1185.99 1082.99 L1186.22 1083.07 L1186.45 1082.81 L1186.67 1082.86 L1186.9 1081.33 L1187.13 1079.65 L1187.36 1081.28 L1187.58 1080.53 L1187.81 1080.69 L1188.04 1081.53 L1188.27 1081.42 L1188.5 1081.26 L1188.72 1082.44 L1188.95 1083.54 L1189.18 1083.2 L1189.41 1082.08 L1189.63 1081.27 L1189.86 1080.89 L1190.09 1082.35 L1190.32 1082.74 L1190.55 1083.33 L1190.77 1083.66 L1191 1084.5 L1191.23 1084.51 L1191.46 1083.74 L1191.68 1082.74 L1191.91 1081.41 L1192.14 1080.46 L1192.37 1080.71 L1192.6 1080.13 L1192.82 1081 L1193.05 1080.97 L1193.28 1080.42 L1193.51 1080.7 L1193.73 1080.26 L1193.96 1079.87 L1194.19 1081.3 L1194.42 1081.53 L1194.65 1082.65 L1194.87 1081.56 L1195.1 1082.7 L1195.33 1082.19 L1195.56 1083.74 L1195.78 1083.27 L1196.01 1083.19 L1196.24 1081.69 L1196.47 1082.37 L1196.7 1082.84 L1196.92 1082.9 L1197.15 1083.41 L1197.38 1083.12 L1197.61 1082.55 L1197.83 1082.21 L1198.06 1082.23 L1198.29 1082.31 L1198.52 1082.58 L1198.75 1082.38 L1198.97 1081.22 L1199.2 1081.1 L1199.43 1080.37 L1199.66 1081.09 L1199.88 1080.53 L1200.11 1081.25 L1200.34 1081.42 L1200.57 1082.27 L1200.8 1082.07 L1201.02 1082.25 L1201.25 1082.58 L1201.48 1082.37 L1201.71 1082.73 L1201.93 1082.19 L1202.16 1081.47 L1202.39 1081.41 L1202.62 1080.88 L1202.85 1080.91 L1203.07 1078.75 L1203.3 1079.22 L1203.53 1080.79 L1203.76 1081.03 L1203.98 1080.24 L1204.21 1079.65 L1204.44 1079.4 L1204.67 1078.85 L1204.9 1079.11 L1205.12 1078.52 L1205.35 1077.97 L1205.58 1078.34 L1205.81 1077.67 L1206.03 1078.29 L1206.26 1077.48 L1206.49 1077.67 L1206.72 1078 L1206.95 1077.87 L1207.17 1078.83 L1207.4 1078.29 L1207.63 1078.48 L1207.86 1079.25 L1208.08 1079.11 L1208.31 1079.07 L1208.54 1079.39 L1208.77 1079.9 L1209 1080.37 L1209.22 1080.6 L1209.45 1079.02 L1209.68 1079.45 L1209.91 1079.26 L1210.13 1080.25 L1210.36 1080.57 L1210.59 1079.94 L1210.82 1080.16 L1211.05 1079.76 L1211.27 1080 L1211.5 1080.75 L1211.73 1080.56 L1211.96 1080.45 L1212.18 1079.32 L1212.41 1078.96 L1212.64 1077.92 L1212.87 1078.01 L1213.1 1078.96 L1213.32 1078.92 L1213.55 1078.78 L1213.78 1077.73 L1214.01 1078.16 L1214.23 1077.74 L1214.46 1076.54 L1214.69 1076.41 L1214.92 1076.65 L1215.15 1075.43 L1215.37 1076 L1215.6 1076.09 L1215.83 1076.6 L1216.06 1076.32 L1216.28 1076.33 L1216.51 1077.05 L1216.74 1077.68 L1216.97 1078.26 L1217.2 1077.72 L1217.42 1077.77 L1217.65 1077.48 L1217.88 1077.41 L1218.11 1077.65 L1218.33 1077.86 L1218.56 1077.19 L1218.79 1076.64 L1219.02 1075.56 L1219.25 1075.92 L1219.47 1075.24 L1219.7 1075.49 L1219.93 1074.82 L1220.16 1074.92 L1220.38 1075.53 L1220.61 1075.73 L1220.84 1075.11 L1221.07 1075.1 L1221.29 1074.82 L1221.52 1074.48 L1221.75 1075.57 L1221.98 1075.88 L1222.21 1075.84 L1222.43 1075.37 L1222.66 1074.61 L1222.89 1074.89 L1223.12 1075.94 L1223.34 1076.45 L1223.57 1077.65 L1223.8 1078.52 L1224.03 1078.33 L1224.26 1078.03 L1224.48 1078.84 L1224.71 1079.2 L1224.94 1078.92 L1225.17 1079.86 L1225.39 1078.96 L1225.62 1078.9 L1225.85 1079.26 L1226.08 1079.29 L1226.31 1080.17 L1226.53 1079.88 L1226.76 1080.28 L1226.99 1080.24 L1227.22 1080.94 L1227.44 1081.92 L1227.67 1082.23 L1227.9 1082.71 L1228.13 1082.79 L1228.36 1082.77 L1228.58 1083.21 L1228.81 1082.43 L1229.04 1082.38 L1229.27 1082.09 L1229.49 1082.35 L1229.72 1082.84 L1229.95 1083.27 L1230.18 1083.55 L1230.41 1083.39 L1230.63 1083.32 L1230.86 1084.46 L1231.09 1084.5 L1231.32 1084.62 L1231.54 1084.08 L1231.77 1084.15 L1232 1084.19 L1232.23 1084.51 L1232.46 1083.94 L1232.68 1083.65 L1232.91 1085.37 L1233.14 1085.69 L1233.37 1086.47 L1233.59 1086.02 L1233.82 1086.49 L1234.05 1085.53 L1234.28 1085.87 L1234.51 1086.1 L1234.73 1086.39 L1234.96 1087.81 L1235.19 1088.36 L1235.42 1087.62 L1235.64 1087.14 L1235.87 1086.31 L1236.1 1086.45 L1236.33 1087.44 L1236.56 1087.48 L1236.78 1087.15 L1237.01 1087.51 L1237.24 1087.23 L1237.47 1087.02 L1237.69 1086.94 L1237.92 1086.57 L1238.15 1086.13 L1238.38 1084.45 L1238.61 1084.17 L1238.83 1085.36 L1239.06 1084.78 L1239.29 1083.9 L1239.52 1083.45 L1239.74 1084.3 L1239.97 1083.08 L1240.2 1082.89 L1240.43 1082.55 L1240.66 1082.24 L1240.88 1082.38 L1241.11 1082.16 L1241.34 1082.62 L1241.57 1082.56 L1241.79 1082.88 L1242.02 1082.4 L1242.25 1082.92 L1242.48 1081.6 L1242.71 1081.75 L1242.93 1082.06 L1243.16 1082.45 L1243.39 1081.74 L1243.62 1082.34 L1243.84 1083.17 L1244.07 1083.1 L1244.3 1083.38 L1244.53 1083.36 L1244.76 1083.6 L1244.98 1083.7 L1245.21 1082.9 L1245.44 1083.76 L1245.67 1083.49 L1245.89 1083.52 L1246.12 1083.19 L1246.35 1082.49 L1246.58 1081.44 L1246.81 1081.6 L1247.03 1080.31 L1247.26 1080.05 L1247.49 1080.38 L1247.72 1081.02 L1247.94 1080.35 L1248.17 1080.52 L1248.4 1081.19 L1248.63 1081.53 L1248.86 1082.08 L1249.08 1081.43 L1249.31 1082.1 L1249.54 1081.64 L1249.77 1081.66 L1249.99 1080.8 L1250.22 1081.07 L1250.45 1081.24 L1250.68 1082.11 L1250.91 1081.28 L1251.13 1081.04 L1251.36 1082.19 L1251.59 1083.06 L1251.82 1083.99 L1252.04 1084.03 L1252.27 1082.72 L1252.5 1082.54 L1252.73 1082.54 L1252.96 1082.66 L1253.18 1081.21 L1253.41 1080.72 L1253.64 1080.95 L1253.87 1080.07 L1254.09 1080.53 L1254.32 1080.29 L1254.55 1080.15 L1254.78 1080.04 L1255.01 1080.21 L1255.23 1079.94 L1255.46 1080.18 L1255.69 1080.22 L1255.92 1080.94 L1256.14 1081 L1256.37 1080.64 L1256.6 1081.29 L1256.83 1082.11 L1257.06 1082.65 L1257.28 1083.07 L1257.51 1082.34 L1257.74 1082.32 L1257.97 1082.46 L1258.19 1082.59 L1258.42 1082.65 L1258.65 1082.29 L1258.88 1081.81 L1259.1 1081.74 L1259.33 1081.84 L1259.56 1082.04 L1259.79 1082.02 L1260.02 1080.08 L1260.24 1079.53 L1260.47 1079.75 L1260.7 1079.08 L1260.93 1079.31 L1261.15 1079.66 L1261.38 1080.65 L1261.61 1079.92 L1261.84 1078.81 L1262.07 1079.14 L1262.29 1080.12 L1262.52 1080.43 L1262.75 1080 L1262.98 1079.74 L1263.2 1079.68 L1263.43 1079.85 L1263.66 1080.32 L1263.89 1080.42 L1264.12 1080.53 L1264.34 1079.83 L1264.57 1080 L1264.8 1080.19 L1265.03 1080.57 L1265.25 1081.7 L1265.48 1080.93 L1265.71 1080.41 L1265.94 1082.11 L1266.17 1083.46 L1266.39 1082.71 L1266.62 1082.87 L1266.85 1083.29 L1267.08 1083.3 L1267.3 1083.84 L1267.53 1084.75 L1267.76 1083.06 L1267.99 1083.83 L1268.22 1084.4 L1268.44 1084.36 L1268.67 1084.69 L1268.9 1086.12 L1269.13 1085.85 L1269.35 1086.16 L1269.58 1087.34 L1269.81 1087.24 L1270.04 1087.66 L1270.27 1086.62 L1270.49 1086.38 L1270.72 1085.65 L1270.95 1085.5 L1271.18 1086.35 L1271.4 1086.72 L1271.63 1086.85 L1271.86 1086.25 L1272.09 1085.37 L1272.32 1084.07 L1272.54 1084.72 L1272.77 1085.2 L1273 1084.09 L1273.23 1084.33 L1273.45 1084.81 L1273.68 1085.09 L1273.91 1085.37 L1274.14 1084.05 L1274.37 1083.87 L1274.59 1083.86 L1274.82 1084.46 L1275.05 1084.34 L1275.28 1084.81 L1275.5 1083.69 L1275.73 1083.75 L1275.96 1084.34 L1276.19 1083.29 L1276.42 1083.74 L1276.64 1083.61 L1276.87 1083.15 L1277.1 1083.82 L1277.33 1084.35 L1277.55 1084.65 L1277.78 1084.52 L1278.01 1083.6 L1278.24 1082.77 L1278.47 1083.16 L1278.69 1084.23 L1278.92 1084.13 L1279.15 1084.65 L1279.38 1084.43 L1279.6 1084.34 L1279.83 1083.58 L1280.06 1084.36 L1280.29 1083.57 L1280.52 1082.59 L1280.74 1083.64 L1280.97 1083.47 L1281.2 1082.24 L1281.43 1082.51 L1281.65 1082.53 L1281.88 1082.33 L1282.11 1082.98 L1282.34 1083.23 L1282.57 1083.57 L1282.79 1084.29 L1283.02 1083.57 L1283.25 1084.06 L1283.48 1083.76 L1283.7 1084.08 L1283.93 1084.47 L1284.16 1084.67 L1284.39 1085.45 L1284.62 1085.73 L1284.84 1085.68 L1285.07 1086.32 L1285.3 1086.64 L1285.53 1087.26 L1285.75 1087.67 L1285.98 1087.49 L1286.21 1088.01 L1286.44 1087.7 L1286.67 1088.03 L1286.89 1088.15 L1287.12 1088.24 L1287.35 1088.52 L1287.58 1088.16 L1287.8 1088.23 L1288.03 1087.43 L1288.26 1087.45 L1288.49 1086.95 L1288.72 1087.25 L1288.94 1087.21 L1289.17 1088.16 L1289.4 1088.03 L1289.63 1088.34 L1289.85 1087.6 L1290.08 1087.37 L1290.31 1088.43 L1290.54 1088.59 L1290.77 1088.43 L1290.99 1089.17 L1291.22 1088.12 L1291.45 1088.63 L1291.68 1089.29 L1291.9 1089.08 L1292.13 1089.94 L1292.36 1088.94 L1292.59 1088.73 L1292.82 1088.72 L1293.04 1088.65 L1293.27 1089.09 L1293.5 1088.37 L1293.73 1088.6 L1293.95 1089.08 L1294.18 1089.3 L1294.41 1088.88 L1294.64 1088.87 L1294.86 1088.73 L1295.09 1087.93 L1295.32 1088.5 L1295.55 1087.83 L1295.78 1088.43 L1296 1088.65 L1296.23 1088.42 L1296.46 1088.28 L1296.69 1086.39 L1296.91 1085.91 L1297.14 1086.16 L1297.37 1086.58 L1297.6 1086.15 L1297.83 1087.05 L1298.05 1085.48 L1298.28 1085.46 L1298.51 1085.72 L1298.74 1085.86 L1298.96 1086.22 L1299.19 1086.31 L1299.42 1086.16 L1299.65 1085.29 L1299.88 1084.51 L1300.1 1084.21 L1300.33 1084.27 L1300.56 1084.3 L1300.79 1084.19 L1301.01 1083.98 L1301.24 1084.85 L1301.47 1084.58 L1301.7 1084.2 L1301.93 1083.11 L1302.15 1083.05 L1302.38 1083.25 L1302.61 1083.59 L1302.84 1084.8 L1303.06 1085.91 L1303.29 1086.19 L1303.52 1086.19 L1303.75 1085.81 L1303.98 1086.09 L1304.2 1086.74 L1304.43 1086.36 L1304.66 1086.48 L1304.89 1087.55 L1305.11 1088.4 L1305.34 1087.58 L1305.57 1087.38 L1305.8 1087.79 L1306.03 1088.06 L1306.25 1087.34 L1306.48 1087.94 L1306.71 1088.59 L1306.94 1088.43 L1307.16 1088.41 L1307.39 1089.42 L1307.62 1088.81 L1307.85 1087.9 L1308.08 1087.61 L1308.3 1087.84 L1308.53 1087.19 L1308.76 1085.93 L1308.99 1085.82 L1309.21 1086.65 L1309.44 1086.59 L1309.67 1086.39 L1309.9 1086.34 L1310.13 1086.86 L1310.35 1086.89 L1310.58 1087.29 L1310.81 1088.02 L1311.04 1088.96 L1311.26 1089.33 L1311.49 1089.86 L1311.72 1090.02 L1311.95 1091.02 L1312.18 1091.75 L1312.4 1091.64 L1312.63 1092.11 L1312.86 1090.92 L1313.09 1090.89 L1313.31 1090.34 L1313.54 1090.98 L1313.77 1090.41 L1314 1090.43 L1314.23 1090.91 L1314.45 1089.98 L1314.68 1089.52 L1314.91 1089.47 L1315.14 1089.55 L1315.36 1090.16 L1315.59 1090.19 L1315.82 1089.81 L1316.05 1089.76 L1316.28 1089.96 L1316.5 1089.45 L1316.73 1090.14 L1316.96 1089.71 L1317.19 1090.04 L1317.41 1090.69 L1317.64 1090.45 L1317.87 1090.86 L1318.1 1090.83 L1318.33 1090.44 L1318.55 1089.24 L1318.78 1089.19 L1319.01 1089.18 L1319.24 1088.72 L1319.46 1088.64 L1319.69 1088.44 L1319.92 1087.75 L1320.15 1087.36 L1320.38 1087.13 L1320.6 1087.28 L1320.83 1086.76 L1321.06 1087.59 L1321.29 1087.73 L1321.51 1088.17 L1321.74 1088.42 L1321.97 1088.44 L1322.2 1087.52 L1322.43 1087.93 L1322.65 1087.34 L1322.88 1086.73 L1323.11 1086.62 L1323.34 1086.35 L1323.56 1085.86 L1323.79 1085.74 L1324.02 1087.03 L1324.25 1086.88 L1324.48 1086.71 L1324.7 1086.69 L1324.93 1086.04 L1325.16 1085.93 L1325.39 1086.39 L1325.61 1085.88 L1325.84 1086.47 L1326.07 1086.07 L1326.3 1086.7 L1326.53 1087 L1326.75 1087.24 L1326.98 1087.78 L1327.21 1088.74 L1327.44 1088.84 L1327.66 1089.02 L1327.89 1089.47 L1328.12 1088.92 L1328.35 1088.83 L1328.58 1089.26 L1328.8 1089.53 L1329.03 1089.68 L1329.26 1089.89 L1329.49 1090.56 L1329.71 1091.04 L1329.94 1091.06 L1330.17 1092.46 L1330.4 1092.93 L1330.62 1094.07 L1330.85 1094.25 L1331.08 1093.6 L1331.31 1094.23 L1331.54 1094.17 L1331.76 1094.47 L1331.99 1094.64 L1332.22 1094.86 L1332.45 1094.59 L1332.67 1094.22 L1332.9 1094.2 L1333.13 1094.9 L1333.36 1094.99 L1333.59 1095.46 L1333.81 1096.16 L1334.04 1095.91 L1334.27 1096.27 L1334.5 1096.24 L1334.72 1096.16 L1334.95 1096.84 L1335.18 1096.53 L1335.41 1097.17 L1335.64 1096.58 L1335.86 1096.92 L1336.09 1097.14 L1336.32 1096.75 L1336.55 1096.77 L1336.77 1096.71 L1337 1098.21 L1337.23 1098.88 L1337.46 1098.14 L1337.69 1097.97 L1337.91 1098.82 L1338.14 1098.98 L1338.37 1098.52 L1338.6 1098.83 L1338.82 1098.77 L1339.05 1099.22 L1339.28 1098.88 L1339.51 1100.33 L1339.74 1100.07 L1339.96 1099.94 L1340.19 1099.53 L1340.42 1098.71 L1340.65 1099.36 L1340.87 1098 L1341.1 1098.47 L1341.33 1099.45 L1341.56 1100.67 L1341.79 1100.74 L1342.01 1099.92 L1342.24 1100.08 L1342.47 1100.51 L1342.7 1100.78 L1342.92 1099.9 L1343.15 1099.48 L1343.38 1099.45 L1343.61 1100.07 L1343.84 1099.87 L1344.06 1099.4 L1344.29 1100.28 L1344.52 1099.64 L1344.75 1099.55 L1344.97 1100.47 L1345.2 1100.39 L1345.43 1101.15 L1345.66 1101.64 L1345.89 1101.6 L1346.11 1100.98 L1346.34 1100.98 L1346.57 1101.58 L1346.8 1102.01 L1347.02 1103.37 L1347.25 1103.92 L1347.48 1104.37 L1347.71 1104.2 L1347.94 1105.37 L1348.16 1105.81 L1348.39 1106.17 L1348.62 1106.33 L1348.85 1107.08 L1349.07 1107.06 L1349.3 1107.71 L1349.53 1108.95 L1349.76 1109.62 L1349.99 1109.36 L1350.21 1108.81 L1350.44 1108.73 L1350.67 1108.38 L1350.9 1108.23 L1351.12 1108.49 L1351.35 1107.95 L1351.58 1108.43 L1351.81 1108.73 L1352.04 1108.95 L1352.26 1109.42 L1352.49 1109.48 L1352.72 1109.18 L1352.95 1108.63 L1353.17 1107.72 L1353.4 1107.63 L1353.63 1107.9 L1353.86 1108.84 L1354.09 1108.49 L1354.31 1108.08 L1354.54 1108.14 L1354.77 1108.98 L1355 1108.36 L1355.22 1108.49 L1355.45 1108.78 L1355.68 1108.66 L1355.91 1109.35 L1356.14 1110.34 L1356.36 1109.13 L1356.59 1109.71 L1356.82 1108.74 L1357.05 1108.32 L1357.27 1107.83 L1357.5 1107.05 L1357.73 1107.32 L1357.96 1107.14 L1358.19 1107.41 L1358.41 1108.23 L1358.64 1107.42 L1358.87 1107.74 L1359.1 1108.05 L1359.32 1108.6 L1359.55 1108.86 L1359.78 1108.17 L1360.01 1107.7 L1360.24 1108.35 L1360.46 1108.58 L1360.69 1108.74 L1360.92 1108.73 L1361.15 1108.97 L1361.37 1108.27 L1361.6 1108.81 L1361.83 1109.55 L1362.06 1110.12 L1362.29 1109.63 L1362.51 1109.53 L1362.74 1109.73 L1362.97 1109.76 L1363.2 1109.02 L1363.42 1109.6 L1363.65 1108.61 L1363.88 1108.83 L1364.11 1108.05 L1364.34 1106.52 L1364.56 1106.77 L1364.79 1106.64 L1365.02 1106.93 L1365.25 1106.2 L1365.47 1106.31 L1365.7 1106.83 L1365.93 1107.06 L1366.16 1107.37 L1366.39 1106.91 L1366.61 1106.8 L1366.84 1107.38 L1367.07 1107.7 L1367.3 1107.25 L1367.52 1106.98 L1367.75 1106.89 L1367.98 1106.64 L1368.21 1106.66 L1368.43 1107.16 L1368.66 1107.79 L1368.89 1108.61 L1369.12 1108.94 L1369.35 1108.87 L1369.57 1108.79 L1369.8 1108.66 L1370.03 1108.85 L1370.26 1108.63 L1370.48 1109.2 L1370.71 1108.62 L1370.94 1108.22 L1371.17 1107.51 L1371.4 1108.71 L1371.62 1108.59 L1371.85 1107.92 L1372.08 1106.51 L1372.31 1106.63 L1372.53 1106.56 L1372.76 1106 L1372.99 1106.01 L1373.22 1106.31 L1373.45 1106.33 L1373.67 1106.5 L1373.9 1107.01 L1374.13 1107.84 L1374.36 1107.67 L1374.58 1107.44 L1374.81 1107.93 L1375.04 1107.36 L1375.27 1107.07 L1375.5 1107.39 L1375.72 1107.55 L1375.95 1107.62 L1376.18 1108.14 L1376.41 1108.08 L1376.63 1107.48 L1376.86 1107.52 L1377.09 1106.96 L1377.32 1107.37 L1377.55 1106.43 L1377.77 1106.73 L1378 1106.62 L1378.23 1105.86 L1378.46 1106.29 L1378.68 1106.26 L1378.91 1105.64 L1379.14 1105.2 L1379.37 1105.24 L1379.6 1104.68 L1379.82 1104.27 L1380.05 1103.63 L1380.28 1103.55 L1380.51 1104.24 L1380.73 1104.42 L1380.96 1103.33 L1381.19 1102.67 L1381.42 1102.35 L1381.65 1101.71 L1381.87 1101.56 L1382.1 1101.61 L1382.33 1100.75 L1382.56 1101.09 L1382.78 1101.88 L1383.01 1102.76 L1383.24 1103.16 L1383.47 1103.5 L1383.7 1103.17 L1383.92 1102.86 L1384.15 1102.55 L1384.38 1102.78 L1384.61 1101.62 L1384.83 1101.46 L1385.06 1101.38 L1385.29 1101.29 L1385.52 1100.49 L1385.75 1099.7 L1385.97 1099.04 L1386.2 1099.55 L1386.43 1099.63 L1386.66 1099.71 L1386.88 1099.03 L1387.11 1098.41 L1387.34 1097.74 L1387.57 1098.81 L1387.8 1098.48 L1388.02 1098.61 L1388.25 1098.65 L1388.48 1099.04 L1388.71 1099.24 L1388.93 1098.74 L1389.16 1099.28 L1389.39 1098.38 L1389.62 1097.97 L1389.85 1097.74 L1390.07 1097.81 L1390.3 1098.05 L1390.53 1098.14 L1390.76 1097.8 L1390.98 1098.13 L1391.21 1097.89 L1391.44 1097.6 L1391.67 1097.34 L1391.9 1097.24 L1392.12 1098.11 L1392.35 1097.81 L1392.58 1097.51 L1392.81 1098.45 L1393.03 1098.84 L1393.26 1098.64 L1393.49 1098.86 L1393.72 1099.1 L1393.95 1099.27 L1394.17 1099.02 L1394.4 1098.87 L1394.63 1099.26 L1394.86 1097.84 L1395.08 1097.98 L1395.31 1098.36 L1395.54 1097.49 L1395.77 1097.56 L1396 1097.7 L1396.22 1097.72 L1396.45 1097.34 L1396.68 1097.07 L1396.91 1097.54 L1397.13 1097.12 L1397.36 1097 L1397.59 1097.08 L1397.82 1097.44 L1398.05 1097.48 L1398.27 1097.85 L1398.5 1096.95 L1398.73 1096.94 L1398.96 1096.22 L1399.18 1096.51 L1399.41 1096.74 L1399.64 1096.86 L1399.87 1097.34 L1400.1 1096.76 L1400.32 1097.49 L1400.55 1097.3 L1400.78 1097.57 L1401.01 1097.79 L1401.23 1098.06 L1401.46 1098.18 L1401.69 1099.23 L1401.92 1099.37 L1402.15 1099.47 L1402.37 1099.85 L1402.6 1099.99 L1402.83 1100.37 L1403.06 1100.73 L1403.28 1100.5 L1403.51 1101.25 L1403.74 1100.42 L1403.97 1099.83 L1404.19 1099.28 L1404.42 1099.75 L1404.65 1099.44 L1404.88 1099.61 L1405.11 1099.21 L1405.33 1099.37 L1405.56 1098.81 L1405.79 1099.02 L1406.02 1098.81 L1406.24 1098.66 L1406.47 1098.86 L1406.7 1099.78 L1406.93 1100.59 L1407.16 1100.3 L1407.38 1100.05 L1407.61 1100.22 L1407.84 1100.17 L1408.07 1100.04 L1408.29 1100.51 L1408.52 1100.75 L1408.75 1100.7 L1408.98 1100.79 L1409.21 1100.31 L1409.43 1101 L1409.66 1101.14 L1409.89 1101.65 L1410.12 1102.4 L1410.34 1102.78 L1410.57 1102.72 L1410.8 1101.75 L1411.03 1100.34 L1411.26 1100.13 L1411.48 1099.11 L1411.71 1099.71 L1411.94 1099.18 L1412.17 1099.17 L1412.39 1099.13 L1412.62 1098.81 L1412.85 1098.59 L1413.08 1099 L1413.31 1099.64 L1413.53 1099.45 L1413.76 1098.55 L1413.99 1098.55 L1414.22 1097.92 L1414.44 1099 L1414.67 1099.1 L1414.9 1098.89 L1415.13 1098.6 L1415.36 1099.7 L1415.58 1099.78 L1415.81 1100.27 L1416.04 1099.71 L1416.27 1100.11 L1416.49 1100.16 L1416.72 1100.51 L1416.95 1100.97 L1417.18 1100.39 L1417.41 1100.49 L1417.63 1100.55 L1417.86 1100.44 L1418.09 1100.52 L1418.32 1100.21 L1418.54 1099.67 L1418.77 1099.03 L1419 1099.5 L1419.23 1100.62 L1419.46 1101.64 L1419.68 1101.97 L1419.91 1101.37 L1420.14 1101.84 L1420.37 1101.86 L1420.59 1102.44 L1420.82 1102.31 L1421.05 1102.83 L1421.28 1103.46 L1421.51 1103.45 L1421.73 1102.65 L1421.96 1102.05 L1422.19 1102.24 L1422.42 1101.73 L1422.64 1100.78 L1422.87 1100.81 L1423.1 1101.9 L1423.33 1101.8 L1423.56 1101.7 L1423.78 1101.48 L1424.01 1101.5 L1424.24 1100.44 L1424.47 1100.72 L1424.69 1101.68 L1424.92 1100.93 L1425.15 1100.93 L1425.38 1101.63 L1425.61 1102.12 L1425.83 1102.11 L1426.06 1102.57 L1426.29 1103.3 L1426.52 1103.01 L1426.74 1102.82 L1426.97 1101.86 L1427.2 1101.57 L1427.43 1101.71 L1427.66 1102.12 L1427.88 1102.7 L1428.11 1103.24 L1428.34 1103.28 L1428.57 1103.91 L1428.79 1104.24 L1429.02 1104.91 L1429.25 1104.8 L1429.48 1104.58 L1429.71 1105.36 L1429.93 1105.19 L1430.16 1104.83 L1430.39 1104.11 L1430.62 1103.65 L1430.84 1104.78 L1431.07 1104.24 L1431.3 1104.84 L1431.53 1105.23 L1431.76 1104.26 L1431.98 1103.32 L1432.21 1102.93 L1432.44 1104.09 L1432.67 1104.6 L1432.89 1104.21 L1433.12 1104.32 L1433.35 1103.97 L1433.58 1103.78 L1433.81 1103.02 L1434.03 1103.31 L1434.26 1104.1 L1434.49 1103.47 L1434.72 1102.86 L1434.94 1103.17 L1435.17 1102.07 L1435.4 1102.05 L1435.63 1102.2 L1435.86 1102.47 L1436.08 1102.74 L1436.31 1102.8 L1436.54 1102.36 L1436.77 1102.88 L1436.99 1103.44 L1437.22 1103.41 L1437.45 1103.29 L1437.68 1102.89 L1437.91 1102.66 L1438.13 1102.34 L1438.36 1101.89 L1438.59 1102.31 L1438.82 1102.72 L1439.04 1103.64 L1439.27 1103.78 L1439.5 1103.11 L1439.73 1103.03 L1439.95 1102.96 L1440.18 1103.34 L1440.41 1103.13 L1440.64 1103.13 L1440.87 1103.41 L1441.09 1103.45 L1441.32 1103.66 L1441.55 1103.56 L1441.78 1102.99 L1442 1103.39 L1442.23 1103.41 L1442.46 1103.8 L1442.69 1103.45 L1442.92 1103.12 L1443.14 1103.14 L1443.37 1103.29 L1443.6 1103.46 L1443.83 1103.47 L1444.05 1103.21 L1444.28 1103.36 L1444.51 1102.94 L1444.74 1102.18 L1444.97 1102.75 L1445.19 1102.7 L1445.42 1102.56 L1445.65 1102.57 L1445.88 1102.7 L1446.1 1102.12 L1446.33 1102.01 L1446.56 1101.73 L1446.79 1100.5 L1447.02 1100.86 L1447.24 1101.29 L1447.47 1101.03 L1447.7 1100.61 L1447.93 1101.09 L1448.15 1100.89 L1448.38 1101.2 L1448.61 1101.99 L1448.84 1102.72 L1449.07 1103.02 L1449.29 1103.1 L1449.52 1103.05 L1449.75 1102.96 L1449.98 1103.15 L1450.2 1103.45 L1450.43 1103.37 L1450.66 1102.55 L1450.89 1103.49 L1451.12 1104.55 L1451.34 1104.63 L1451.57 1103.65 L1451.8 1104.4 L1452.03 1104.49 L1452.25 1104.28 L1452.48 1103.79 L1452.71 1102.81 L1452.94 1102.06 L1453.17 1102.52 L1453.39 1103.09 L1453.62 1102.88 L1453.85 1102.55 L1454.08 1101.77 L1454.3 1101.94 L1454.53 1102.06 L1454.76 1102.13 L1454.99 1101.44 L1455.22 1102.14 L1455.44 1102.22 L1455.67 1101.25 L1455.9 1101.51 L1456.13 1101.97 L1456.35 1101.79 L1456.58 1101.98 L1456.81 1102.01 L1457.04 1102.13 L1457.27 1102.04 L1457.49 1101.17 L1457.72 1101.43 L1457.95 1101.54 L1458.18 1102.24 L1458.4 1102.43 L1458.63 1102.53 L1458.86 1102.34 L1459.09 1102.65 L1459.32 1103.38 L1459.54 1103.17 L1459.77 1102.87 L1460 1103.33 L1460.23 1103.89 L1460.45 1104.22 L1460.68 1104.16 L1460.91 1103.35 L1461.14 1102.53 L1461.37 1101.93 L1461.59 1101.78 L1461.82 1101.9 L1462.05 1102 L1462.28 1102.06 L1462.5 1101.12 L1462.73 1101.2 L1462.96 1101.54 L1463.19 1101.56 L1463.42 1100.91 L1463.64 1101.27 L1463.87 1101.48 L1464.1 1101.09 L1464.33 1101.48 L1464.55 1101.82 L1464.78 1101.65 L1465.01 1102.03 L1465.24 1102.45 L1465.47 1102.76 L1465.69 1102.43 L1465.92 1102.75 L1466.15 1102.76 L1466.38 1103.02 L1466.6 1102.97 L1466.83 1103.75 L1467.06 1103.9 L1467.29 1103.9 L1467.52 1104.1 L1467.74 1103.44 L1467.97 1103.57 L1468.2 1102.67 L1468.43 1102.1 L1468.65 1102.63 L1468.88 1101.61 L1469.11 1101.94 L1469.34 1102.32 L1469.57 1102.47 L1469.79 1102.75 L1470.02 1102.82 L1470.25 1102.43 L1470.48 1102.67 L1470.7 1102.71 L1470.93 1101.79 L1471.16 1101.86 L1471.39 1102.24 L1471.62 1102.02 L1471.84 1101.99 L1472.07 1101.5 L1472.3 1101.79 L1472.53 1102.51 L1472.75 1102.47 L1472.98 1103.54 L1473.21 1103.57 L1473.44 1103.84 L1473.67 1102.94 L1473.89 1103.25 L1474.12 1103.95 L1474.35 1104.08 L1474.58 1104.14 L1474.8 1104.22 L1475.03 1103.31 L1475.26 1103.43 L1475.49 1103.51 L1475.71 1103.14 L1475.94 1103.2 L1476.17 1102.87 L1476.4 1102.75 L1476.63 1102.92 L1476.85 1102.41 L1477.08 1102.41 L1477.31 1102.01 L1477.54 1102.91 L1477.76 1102.58 L1477.99 1102.18 L1478.22 1102.71 L1478.45 1102.84 L1478.68 1102.76 L1478.9 1101.8 L1479.13 1102.35 L1479.36 1102.85 L1479.59 1103.09 L1479.81 1103.35 L1480.04 1104.26 L1480.27 1104.43 L1480.5 1104.83 L1480.73 1104.82 L1480.95 1105.75 L1481.18 1106.28 L1481.41 1105.44 L1481.64 1105.88 L1481.86 1106.12 L1482.09 1107 L1482.32 1107.37 L1482.55 1107.66 L1482.78 1108.1 L1483 1107.75 L1483.23 1107.84 L1483.46 1107.13 L1483.69 1106.75 L1483.91 1107.05 L1484.14 1107.06 L1484.37 1106.8 L1484.6 1106.76 L1484.83 1107.26 L1485.05 1107.42 L1485.28 1107.07 L1485.51 1106.24 L1485.74 1106.81 L1485.96 1106.71 L1486.19 1107.09 L1486.42 1107.57 L1486.65 1107.56 L1486.88 1107.68 L1487.1 1107.98 L1487.33 1108.05 L1487.56 1107.53 L1487.79 1107.61 L1488.01 1108.34 L1488.24 1108.43 L1488.47 1108.06 L1488.7 1108.86 L1488.93 1109.3 L1489.15 1109.07 L1489.38 1109.41 L1489.61 1108.7 L1489.84 1109.36 L1490.06 1109.3 L1490.29 1109.49 L1490.52 1108.49 L1490.75 1109.16 L1490.98 1109.59 L1491.2 1110.12 L1491.43 1110.37 L1491.66 1110.25 L1491.89 1110.54 L1492.11 1110.32 L1492.34 1110.28 L1492.57 1110 L1492.8 1110.19 L1493.03 1110.92 L1493.25 1110.73 L1493.48 1110.57 L1493.71 1110.03 L1493.94 1109.22 L1494.16 1108.78 L1494.39 1108.22 L1494.62 1107.99 L1494.85 1108.38 L1495.08 1108.56 L1495.3 1109.02 L1495.53 1109 L1495.76 1108.86 L1495.99 1108.41 L1496.21 1108.62 L1496.44 1108.26 L1496.67 1108.43 L1496.9 1108.81 L1497.13 1108.64 L1497.35 1108.26 L1497.58 1108.77 L1497.81 1109.31 L1498.04 1109.53 L1498.26 1109.78 L1498.49 1110.15 L1498.72 1109.56 L1498.95 1109.69 L1499.18 1110.07 L1499.4 1110.23 L1499.63 1109.8 L1499.86 1109.83 L1500.09 1110.52 L1500.31 1110.2 L1500.54 1109.75 L1500.77 1110.24 L1501 1110.34 L1501.23 1111.07 L1501.45 1110.71 L1501.68 1110.93 L1501.91 1110.63 L1502.14 1110.56 L1502.36 1110.6 L1502.59 1110.01 L1502.82 1109.98 L1503.05 1110.52 L1503.28 1109.38 L1503.5 1108.57 L1503.73 1108.04 L1503.96 1107.95 L1504.19 1107.69 L1504.41 1108.32 L1504.64 1108.22 L1504.87 1108.18 L1505.1 1107.89 L1505.33 1106.86 L1505.55 1107.29 L1505.78 1107.54 L1506.01 1107.35 L1506.24 1107.94 L1506.46 1108.5 L1506.69 1108.65 L1506.92 1108.31 L1507.15 1107.84 L1507.38 1108.48 L1507.6 1109.42 L1507.83 1109.32 L1508.06 1109.82 L1508.29 1110.24 L1508.51 1110.74 L1508.74 1110.99 L1508.97 1111.22 L1509.2 1111.71 L1509.43 1112.57 L1509.65 1112.44 L1509.88 1111.6 L1510.11 1111.44 L1510.34 1111 L1510.56 1111.29 L1510.79 1111.39 L1511.02 1112.05 L1511.25 1111.54 L1511.48 1111.35 L1511.7 1111.4 L1511.93 1110.89 L1512.16 1111.19 L1512.39 1111.55 L1512.61 1112 L1512.84 1112.77 L1513.07 1113.18 L1513.3 1113.04 L1513.52 1113.66 L1513.75 1113.37 L1513.98 1112.21 L1514.21 1111.7 L1514.44 1111.71 L1514.66 1112.11 L1514.89 1112.19 L1515.12 1112.49 L1515.35 1112.27 L1515.57 1112.37 L1515.8 1112.86 L1516.03 1112.78 L1516.26 1112.76 L1516.49 1113.34 L1516.71 1113.96 L1516.94 1114.06 L1517.17 1113.74 L1517.4 1113.39 L1517.62 1113.12 L1517.85 1112.29 L1518.08 1111.98 L1518.31 1112.08 L1518.54 1111.6 L1518.76 1111.91 L1518.99 1112.02 L1519.22 1112.57 L1519.45 1113.18 L1519.67 1113.05 L1519.9 1112.62 L1520.13 1112.49 L1520.36 1112.74 L1520.59 1113.09 L1520.81 1112.46 L1521.04 1112.53 L1521.27 1112.67 L1521.5 1113.57 L1521.72 1113.18 L1521.95 1113.77 L1522.18 1114.1 L1522.41 1113.12 L1522.64 1113.32 L1522.86 1113.13 L1523.09 1113.26 L1523.32 1113.11 L1523.55 1113.21 L1523.77 1112.56 L1524 1112.09 L1524.23 1112.22 L1524.46 1112.43 L1524.69 1112.04 L1524.91 1112.17 L1525.14 1112.18 L1525.37 1112.13 L1525.6 1111.85 L1525.82 1111.47 L1526.05 1111.13 L1526.28 1110.88 L1526.51 1111.53 L1526.74 1111.35 L1526.96 1112.17 L1527.19 1112.84 L1527.42 1112.78 L1527.65 1112.35 L1527.87 1112.21 L1528.1 1111.05 L1528.33 1110.23 L1528.56 1110.18 L1528.79 1110.19 L1529.01 1109.95 L1529.24 1109.77 L1529.47 1109.73 L1529.7 1110.26 L1529.92 1110.33 L1530.15 1110.13 L1530.38 1110.49 L1530.61 1111.17 L1530.84 1111.66 L1531.06 1112.19 L1531.29 1111.61 L1531.52 1110.92 L1531.75 1111.3 L1531.97 1111.76 L1532.2 1111.99 L1532.43 1111.63 L1532.66 1112.19 L1532.89 1113.21 L1533.11 1112.91 L1533.34 1113.13 L1533.57 1113.25 L1533.8 1113.29 L1534.02 1113.32 L1534.25 1113.83 L1534.48 1114.33 L1534.71 1114.03 L1534.94 1113.63 L1535.16 1113.91 L1535.39 1114.58 L1535.62 1114.99 L1535.85 1115.08 L1536.07 1115.32 L1536.3 1115.64 L1536.53 1115.65 L1536.76 1115.98 L1536.99 1116.57 L1537.21 1117.27 L1537.44 1117.55 L1537.67 1118.33 L1537.9 1118.94 L1538.12 1119.12 L1538.35 1117.66 L1538.58 1117.65 L1538.81 1117.35 L1539.04 1117.22 L1539.26 1117.2 L1539.49 1117.4 L1539.72 1117.68 L1539.95 1117.87 L1540.17 1117.53 L1540.4 1117.96 L1540.63 1117.37 L1540.86 1117.23 L1541.09 1116.44 L1541.31 1116.54 L1541.54 1115.87 L1541.77 1116.05 L1542 1116.51 L1542.22 1116.92 L1542.45 1116.66 L1542.68 1116.95 L1542.91 1116.79 L1543.14 1116.91 L1543.36 1117.14 L1543.59 1116.98 L1543.82 1115.33 L1544.05 1115.67 L1544.27 1115.74 L1544.5 1116.06 L1544.73 1116.49 L1544.96 1117.36 L1545.19 1117.48 L1545.41 1117.2 L1545.64 1116.65 L1545.87 1116.07 L1546.1 1115.97 L1546.32 1116.36 L1546.55 1116.81 L1546.78 1115.64 L1547.01 1115.88 L1547.24 1114.83 L1547.46 1114.66 L1547.69 1114.83 L1547.92 1115.41 L1548.15 1115.77 L1548.37 1115.34 L1548.6 1115.27 L1548.83 1115.16 L1549.06 1115.13 L1549.28 1115.17 L1549.51 1114.6 L1549.74 1115.42 L1549.97 1116.45 L1550.2 1115.72 L1550.42 1116.02 L1550.65 1115.98 L1550.88 1116.06 L1551.11 1116.57 L1551.33 1115.72 L1551.56 1115.05 L1551.79 1115.2 L1552.02 1115.25 L1552.25 1115 L1552.47 1115.1 L1552.7 1115.13 L1552.93 1115.68 L1553.16 1115.43 L1553.38 1115.08 L1553.61 1115 L1553.84 1115.31 L1554.07 1114.38 L1554.3 1114.44 L1554.52 1113.64 L1554.75 1113.29 L1554.98 1112.81 L1555.21 1113.02 L1555.43 1113.68 L1555.66 1113.78 L1555.89 1113.98 L1556.12 1114.32 L1556.35 1114.31 L1556.57 1115.12 L1556.8 1114.95 L1557.03 1114.37 L1557.26 1114.6 L1557.48 1114.59 L1557.71 1114.84 L1557.94 1114.05 L1558.17 1112.77 L1558.4 1112.24 L1558.62 1112.57 L1558.85 1112.04 L1559.08 1112.69 L1559.31 1113.3 L1559.53 1112.96 L1559.76 1113.47 L1559.99 1113.66 L1560.22 1113.26 L1560.45 1113.8 L1560.67 1113.45 L1560.9 1113.19 L1561.13 1113.57 L1561.36 1113.64 L1561.58 1113.65 L1561.81 1113.72 L1562.04 1113.33 L1562.27 1113.72 L1562.5 1113.49 L1562.72 1112.71 L1562.95 1112.25 L1563.18 1112.42 L1563.41 1112.48 L1563.63 1112.05 L1563.86 1111.78 L1564.09 1110.99 L1564.32 1110.91 L1564.55 1111.35 L1564.77 1111.21 L1565 1110.56 L1565.23 1111.13 L1565.46 1111.21 L1565.68 1111.05 L1565.91 1111.03 L1566.14 1110.84 L1566.37 1111.47 L1566.6 1111.52 L1566.82 1111.34 L1567.05 1111.16 L1567.28 1111.3 L1567.51 1111.67 L1567.73 1111.23 L1567.96 1111.39 L1568.19 1110.88 L1568.42 1110.91 L1568.65 1110.3 L1568.87 1110.36 L1569.1 1110.19 L1569.33 1109.81 L1569.56 1110.48 L1569.78 1110.75 L1570.01 1110.64 L1570.24 1110.49 L1570.47 1110.2 L1570.7 1110.2 L1570.92 1110.49 L1571.15 1110.34 L1571.38 1110.56 L1571.61 1111.09 L1571.83 1111.43 L1572.06 1111.79 L1572.29 1111.9 L1572.52 1112.17 L1572.75 1111.99 L1572.97 1112.2 L1573.2 1112.49 L1573.43 1112.5 L1573.66 1112.48 L1573.88 1112.08 L1574.11 1111.78 L1574.34 1111.04 L1574.57 1110.47 L1574.8 1109.84 L1575.02 1109.55 L1575.25 1109.32 L1575.48 1108.69 L1575.71 1108.54 L1575.93 1107.71 L1576.16 1107.4 L1576.39 1107.31 L1576.62 1107.13 L1576.85 1107.32 L1577.07 1107.42 L1577.3 1106.95 L1577.53 1107.4 L1577.76 1106.96 L1577.98 1107.26 L1578.21 1107.4 L1578.44 1107.48 L1578.67 1107.76 L1578.9 1108.12 L1579.12 1108.03 L1579.35 1107.49 L1579.58 1107.74 L1579.81 1107.6 L1580.03 1107.94 L1580.26 1107.59 L1580.49 1106.85 L1580.72 1106.7 L1580.95 1106.74 L1581.17 1106.27 L1581.4 1105.52 L1581.63 1105.82 L1581.86 1104.62 L1582.08 1104.28 L1582.31 1104.38 L1582.54 1104.55 L1582.77 1104.57 L1583 1105.09 L1583.22 1105.24 L1583.45 1105 L1583.68 1104.76 L1583.91 1105.06 L1584.13 1104.98 L1584.36 1104.12 L1584.59 1104.96 L1584.82 1105.42 L1585.04 1105.59 L1585.27 1105.81 L1585.5 1104.9 L1585.73 1105.52 L1585.96 1105.26 L1586.18 1105.76 L1586.41 1105.75 L1586.64 1105.98 L1586.87 1106.04 L1587.09 1106.13 L1587.32 1105.66 L1587.55 1105.94 L1587.78 1106.25 L1588.01 1106.37 L1588.23 1106.03 L1588.46 1105.41 L1588.69 1105.65 L1588.92 1104.98 L1589.14 1104.66 L1589.37 1105.08 L1589.6 1105.55 L1589.83 1104.8 L1590.06 1104.68 L1590.28 1105.03 L1590.51 1104.43 L1590.74 1103.59 L1590.97 1103.59 L1591.19 1103.62 L1591.42 1103.52 L1591.65 1103.68 L1591.88 1103.43 L1592.11 1103.46 L1592.33 1102.98 L1592.56 1103.22 L1592.79 1103.25 L1593.02 1103.32 L1593.24 1103.63 L1593.47 1103.38 L1593.7 1103.92 L1593.93 1104.3 L1594.16 1104.13 L1594.38 1104.98 L1594.61 1104.77 L1594.84 1105.04 L1595.07 1105.21 L1595.29 1105.26 L1595.52 1105.36 L1595.75 1104.78 L1595.98 1104.35 L1596.21 1104.47 L1596.43 1105.33 L1596.66 1105.47 L1596.89 1105.87 L1597.12 1106.64 L1597.34 1106.61 L1597.57 1106.28 L1597.8 1105.86 L1598.03 1106 L1598.26 1105.31 L1598.48 1104.44 L1598.71 1104.5 L1598.94 1103.69 L1599.17 1103.69 L1599.39 1104.59 L1599.62 1104.03 L1599.85 1103.36 L1600.08 1103.18 L1600.31 1103.68 L1600.53 1103.74 L1600.76 1104.04 L1600.99 1104.05 L1601.22 1104.34 L1601.44 1104.01 L1601.67 1103.61 L1601.9 1103.6 L1602.13 1103.73 L1602.36 1102.27 L1602.58 1101.86 L1602.81 1102.81 L1603.04 1103.24 L1603.27 1102.79 L1603.49 1102.93 L1603.72 1102.98 L1603.95 1103.28 L1604.18 1103.3 L1604.41 1103.69 L1604.63 1104.09 L1604.86 1104.62 L1605.09 1103.55 L1605.32 1103.97 L1605.54 1104.1 L1605.77 1104.21 L1606 1104.29 L1606.23 1104.21 L1606.46 1104.23 L1606.68 1104.21 L1606.91 1104.8 L1607.14 1104.74 L1607.37 1104.53 L1607.59 1104.02 L1607.82 1104.24 L1608.05 1104.86 L1608.28 1104.89 L1608.51 1104.62 L1608.73 1104.88 L1608.96 1104.89 L1609.19 1104.75 L1609.42 1104.51 L1609.64 1105.1 L1609.87 1104.52 L1610.1 1105.09 L1610.33 1105.64 L1610.56 1105.75 L1610.78 1104.88 L1611.01 1104.23 L1611.24 1104.18 L1611.47 1103.67 L1611.69 1102.93 L1611.92 1102.74 L1612.15 1102.9 L1612.38 1102.84 L1612.61 1103.29 L1612.83 1103.36 L1613.06 1103.05 L1613.29 1102.86 L1613.52 1103.74 L1613.74 1104.17 L1613.97 1103.54 L1614.2 1103.36 L1614.43 1102.85 L1614.66 1102.5 L1614.88 1102.17 L1615.11 1102.49 L1615.34 1102.57 L1615.57 1101.77 L1615.79 1101.51 L1616.02 1100.97 L1616.25 1100.92 L1616.48 1101.71 L1616.71 1100.58 L1616.93 1100.4 L1617.16 1100.05 L1617.39 1100.09 L1617.62 1099.79 L1617.84 1099.48 L1618.07 1099.82 L1618.3 1099.87 L1618.53 1100.01 L1618.76 1100.52 L1618.98 1099.93 L1619.21 1099.14 L1619.44 1099.39 L1619.67 1099.8 L1619.89 1099.44 L1620.12 1099.42 L1620.35 1099.88 L1620.58 1100.4 L1620.81 1100.25 L1621.03 1100.05 L1621.26 1099.81 L1621.49 1100.16 L1621.72 1099.97 L1621.94 1099.93 L1622.17 1099.81 L1622.4 1100.86 L1622.63 1100.88 L1622.85 1101.4 L1623.08 1100.5 L1623.31 1100.58 L1623.54 1100.67 L1623.77 1101.03 L1623.99 1100.99 L1624.22 1101.02 L1624.45 1100.97 L1624.68 1101.03 L1624.9 1101.34 L1625.13 1101.82 L1625.36 1101.99 L1625.59 1101.59 L1625.82 1101.27 L1626.04 1101.47 L1626.27 1101.34 L1626.5 1101.32 L1626.73 1101.02 L1626.95 1100.19 L1627.18 1100.27 L1627.41 1100.4 L1627.64 1100.28 L1627.87 1100.11 L1628.09 1099.67 L1628.32 1100.24 L1628.55 1100.2 L1628.78 1100.92 L1629 1100.68 L1629.23 1101.27 L1629.46 1100.19 L1629.69 1100.41 L1629.92 1099.69 L1630.14 1099.34 L1630.37 1099.14 L1630.6 1099.53 L1630.83 1099 L1631.05 1099.03 L1631.28 1098.54 L1631.51 1098.43 L1631.74 1098.33 L1631.97 1098.23 L1632.19 1098.46 L1632.42 1098.6 L1632.65 1098.99 L1632.88 1099.47 L1633.1 1099.79 L1633.33 1099.96 L1633.56 1098.96 L1633.79 1098.56 L1634.02 1097.83 L1634.24 1098.01 L1634.47 1097.94 L1634.7 1097.74 L1634.93 1097.67 L1635.15 1097.73 L1635.38 1097.36 L1635.61 1097.78 L1635.84 1097.33 L1636.07 1097.57 L1636.29 1098.11 L1636.52 1097.93 L1636.75 1098.23 L1636.98 1098.25 L1637.2 1097.29 L1637.43 1097.62 L1637.66 1097.69 L1637.89 1097.72 L1638.12 1098.15 L1638.34 1098.05 L1638.57 1098.1 L1638.8 1097.99 L1639.03 1097.92 L1639.25 1096.93 L1639.48 1097.64 L1639.71 1097.95 L1639.94 1097.23 L1640.17 1096.15 L1640.39 1095.8 L1640.62 1095.47 L1640.85 1095.1 L1641.08 1095.44 L1641.3 1095.54 L1641.53 1095.91 L1641.76 1095.57 L1641.99 1095.6 L1642.22 1095.87 L1642.44 1095.6 L1642.67 1095.74 L1642.9 1096.12 L1643.13 1096.29 L1643.35 1095.58 L1643.58 1095.71 L1643.81 1094.9 L1644.04 1095.25 L1644.27 1095.86 L1644.49 1095.39 L1644.72 1095.22 L1644.95 1093.96 L1645.18 1093.65 L1645.4 1093.99 L1645.63 1094.79 L1645.86 1095.31 L1646.09 1094.88 L1646.32 1094.95 L1646.54 1095.43 L1646.77 1094.62 L1647 1095.25 L1647.23 1095.36 L1647.45 1094.07 L1647.68 1093.72 L1647.91 1094.12 L1648.14 1094.55 L1648.37 1094.66 L1648.59 1094.7 L1648.82 1095.11 L1649.05 1095.37 L1649.28 1095.42 L1649.5 1095.55 L1649.73 1094.88 L1649.96 1095.02 L1650.19 1095.01 L1650.42 1095 L1650.64 1095.53 L1650.87 1096.05 L1651.1 1096.09 L1651.33 1095.29 L1651.55 1096.02 L1651.78 1096.05 L1652.01 1096.52 L1652.24 1096.83 L1652.47 1096.93 L1652.69 1096.78 L1652.92 1097.09 L1653.15 1097.44 L1653.38 1097.74 L1653.6 1097.75 L1653.83 1098.18 L1654.06 1098.11 L1654.29 1098.65 L1654.52 1098.83 L1654.74 1098.33 L1654.97 1098.12 L1655.2 1098.03 L1655.43 1098.22 L1655.65 1098.78 L1655.88 1099.35 L1656.11 1099.7 L1656.34 1099.38 L1656.57 1099.72 L1656.79 1099.69 L1657.02 1099.86 L1657.25 1099.65 L1657.48 1100.04 L1657.7 1100.42 L1657.93 1100.72 L1658.16 1100.73 L1658.39 1100.92 L1658.61 1100.38 L1658.84 1100.59 L1659.07 1101.13 L1659.3 1101.54 L1659.53 1101.92 L1659.75 1102.58 L1659.98 1103.06 L1660.21 1102.68 L1660.44 1102.19 L1660.66 1101.77 L1660.89 1102.35 L1661.12 1102.77 L1661.35 1103.12 L1661.58 1103.25 L1661.8 1103.58 L1662.03 1103.75 L1662.26 1103.27 L1662.49 1103.5 L1662.71 1103.76 L1662.94 1102.76 L1663.17 1102.85 L1663.4 1103.34 L1663.63 1103.08 L1663.85 1103.57 L1664.08 1102.68 L1664.31 1102.38 L1664.54 1102.45 L1664.76 1103.22 L1664.99 1103.66 L1665.22 1103.77 L1665.45 1104.4 L1665.68 1104.2 L1665.9 1104.15 L1666.13 1103.86 L1666.36 1103.91 L1666.59 1103.99 L1666.81 1104.84 L1667.04 1104.34 L1667.27 1104.07 L1667.5 1104.29 L1667.73 1104.36 L1667.95 1104.54 L1668.18 1103.49 L1668.41 1104.21 L1668.64 1103.64 L1668.86 1104.21 L1669.09 1103.99 L1669.32 1104.32 L1669.55 1104.76 L1669.78 1105.44 L1670 1105.49 L1670.23 1106.12 L1670.46 1106.35 L1670.69 1105.91 L1670.91 1106.38 L1671.14 1106.46 L1671.37 1106.44 L1671.6 1105.93 L1671.83 1106.52 L1672.05 1106.83 L1672.28 1106.89 L1672.51 1106.48 L1672.74 1106.26 L1672.96 1106.53 L1673.19 1105.67 L1673.42 1106.44 L1673.65 1106.5 L1673.88 1106.08 L1674.1 1106.06 L1674.33 1106.26 L1674.56 1106.15 L1674.79 1105.82 L1675.01 1106.41 L1675.24 1105.6 L1675.47 1105.5 L1675.7 1105.13 L1675.93 1105.51 L1676.15 1105.72 L1676.38 1106.18 L1676.61 1105.27 L1676.84 1105.81 L1677.06 1105.86 L1677.29 1106.42 L1677.52 1105.94 L1677.75 1106.03 L1677.98 1105.99 L1678.2 1105.02 L1678.43 1105 L1678.66 1105.41 L1678.89 1105.45 L1679.11 1105.7 L1679.34 1105.52 L1679.57 1104.79 L1679.8 1105.29 L1680.03 1105.2 L1680.25 1104.9 L1680.48 1104.82 L1680.71 1104.57 L1680.94 1104.59 L1681.16 1104.45 L1681.39 1105.11 L1681.62 1105.36 L1681.85 1105.6 L1682.08 1105.9 L1682.3 1104.37 L1682.53 1104.36 L1682.76 1104.73 L1682.99 1105.1 L1683.21 1104.63 L1683.44 1104.7 L1683.67 1103.46 L1683.9 1103.62 L1684.13 1104.11 L1684.35 1103.77 L1684.58 1103.39 L1684.81 1103.57 L1685.04 1103.77 L1685.26 1103.78 L1685.49 1104.26 L1685.72 1104.09 L1685.95 1104.31 L1686.18 1104.8 L1686.4 1104.08 L1686.63 1104.65 L1686.86 1104.64 L1687.09 1103.76 L1687.31 1103.49 L1687.54 1103.37 L1687.77 1103.38 L1688 1103.53 L1688.23 1104.02 L1688.45 1102.9 L1688.68 1102.47 L1688.91 1103.04 L1689.14 1102.07 L1689.36 1101.73 L1689.59 1101.42 L1689.82 1100.4 L1690.05 1101.19 L1690.28 1100.86 L1690.5 1100.25 L1690.73 1100.49 L1690.96 1100.75 L1691.19 1100.63 L1691.41 1101.02 L1691.64 1101 L1691.87 1100.91 L1692.1 1101.28 L1692.33 1101.37 L1692.55 1101.57 L1692.78 1101.7 L1693.01 1101.41 L1693.24 1101.5 L1693.46 1101.39 L1693.69 1101 L1693.92 1101.19 L1694.15 1101.43 L1694.37 1101.68 L1694.6 1102.19 L1694.83 1101.03 L1695.06 1101.1 L1695.29 1101.36 L1695.51 1101.3 L1695.74 1101.01 L1695.97 1100.98 L1696.2 1101.46 L1696.42 1101.13 L1696.65 1101.04 L1696.88 1100.59 L1697.11 1100.83 L1697.34 1100.34 L1697.56 1099.99 L1697.79 1099.82 L1698.02 1099.98 L1698.25 1099.74 L1698.47 1099.89 L1698.7 1099.92 L1698.93 1099.86 L1699.16 1100.19 L1699.39 1100.54 L1699.61 1100.21 L1699.84 1099.75 L1700.07 1099.66 L1700.3 1099.85 L1700.52 1099.39 L1700.75 1099.69 L1700.98 1099.41 L1701.21 1099.48 L1701.44 1099.41 L1701.66 1099.83 L1701.89 1099.73 L1702.12 1099.49 L1702.35 1099.26 L1702.57 1099.07 L1702.8 1099.37 L1703.03 1099.08 L1703.26 1098.68 L1703.49 1099.67 L1703.71 1099.9 L1703.94 1100.27 L1704.17 1100.48 L1704.4 1100.14 L1704.62 1098.91 L1704.85 1099.11 L1705.08 1099.87 L1705.31 1100.27 L1705.54 1100.58 L1705.76 1100.11 L1705.99 1099.91 L1706.22 1100.11 L1706.45 1100.42 L1706.67 1101.04 L1706.9 1101 L1707.13 1100.36 L1707.36 1100.12 L1707.59 1099.88 L1707.81 1099.93 L1708.04 1100.03 L1708.27 1100.14 L1708.5 1100.62 L1708.72 1100.69 L1708.95 1100.42 L1709.18 1101.46 L1709.41 1101.21 L1709.64 1101.22 L1709.86 1100.86 L1710.09 1100.75 L1710.32 1100.57 L1710.55 1100.54 L1710.77 1100.5 L1711 1100 L1711.23 1099.79 L1711.46 1099.72 L1711.69 1100.09 L1711.91 1099.93 L1712.14 1100.25 L1712.37 1100.05 L1712.6 1098.92 L1712.82 1098.94 L1713.05 1099.85 L1713.28 1100.5 L1713.51 1100.02 L1713.74 1100.11 L1713.96 1099.49 L1714.19 1099.8 L1714.42 1099.88 L1714.65 1099.56 L1714.87 1099.36 L1715.1 1099.67 L1715.33 1099.27 L1715.56 1099.04 L1715.79 1099.3 L1716.01 1099.29 L1716.24 1099.7 L1716.47 1099.75 L1716.7 1099.67 L1716.92 1099.44 L1717.15 1099.5 L1717.38 1099.05 L1717.61 1099.15 L1717.84 1098.83 L1718.06 1098.68 L1718.29 1098.49 L1718.52 1099.07 L1718.75 1098.08 L1718.97 1098.6 L1719.2 1098.09 L1719.43 1098.42 L1719.66 1099.07 L1719.89 1099.36 L1720.11 1099.39 L1720.34 1100.16 L1720.57 1099.5 L1720.8 1099.12 L1721.02 1098.75 L1721.25 1098.95 L1721.48 1098.97 L1721.71 1098.35 L1721.94 1098.64 L1722.16 1099.12 L1722.39 1099.53 L1722.62 1099.36 L1722.85 1098.92 L1723.07 1099.1 L1723.3 1099.07 L1723.53 1098.93 L1723.76 1098.61 L1723.99 1098.41 L1724.21 1098.02 L1724.44 1098 L1724.67 1097.76 L1724.9 1097.67 L1725.12 1097.46 L1725.35 1097.84 L1725.58 1097.87 L1725.81 1098.16 L1726.04 1097.99 L1726.26 1098.2 L1726.49 1098.66 L1726.72 1098.4 L1726.95 1097.14 L1727.17 1097.43 L1727.4 1097.37 L1727.63 1097.61 L1727.86 1097.5 L1728.09 1097.99 L1728.31 1097.88 L1728.54 1098.34 L1728.77 1098.16 L1729 1098.48 L1729.22 1098.58 L1729.45 1098.32 L1729.68 1098.12 L1729.91 1097.27 L1730.14 1097.42 L1730.36 1097.03 L1730.59 1096.53 L1730.82 1096.3 L1731.05 1096.7 L1731.27 1096.77 L1731.5 1096.82 L1731.73 1096.64 L1731.96 1096.87 L1732.18 1097.35 L1732.41 1097.17 L1732.64 1096.96 L1732.87 1096.98 L1733.1 1097.81 L1733.32 1097.99 L1733.55 1098.48 L1733.78 1098.78 L1734.01 1098.78 L1734.23 1097.97 L1734.46 1097.88 L1734.69 1097.8 L1734.92 1097.91 L1735.15 1098.11 L1735.37 1098.02 L1735.6 1097.9 L1735.83 1096.98 L1736.06 1097.06 L1736.28 1097.69 L1736.51 1098.11 L1736.74 1097.66 L1736.97 1097.48 L1737.2 1096.84 L1737.42 1097.43 L1737.65 1097.27 L1737.88 1097.1 L1738.11 1097.79 L1738.33 1097.24 L1738.56 1097.08 L1738.79 1097.27 L1739.02 1097.28 L1739.25 1097.56 L1739.47 1097.69 L1739.7 1097.75 L1739.93 1097.77 L1740.16 1097.28 L1740.38 1096.89 L1740.61 1097.38 L1740.84 1098.1 L1741.07 1097.91 L1741.3 1097.93 L1741.52 1098.16 L1741.75 1097.89 L1741.98 1097.48 L1742.21 1097.9 L1742.43 1096.67 L1742.66 1096.64 L1742.89 1096.33 L1743.12 1096.17 L1743.35 1096.5 L1743.57 1096.76 L1743.8 1096.91 L1744.03 1097.29 L1744.26 1096.77 L1744.48 1097.09 L1744.71 1096.36 L1744.94 1096.81 L1745.17 1097.28 L1745.4 1097.86 L1745.62 1097.51 L1745.85 1096.92 L1746.08 1097.3 L1746.31 1097.44 L1746.53 1097.01 L1746.76 1096.82 L1746.99 1096.58 L1747.22 1095.59 L1747.45 1095.65 L1747.67 1096.21 L1747.9 1096.57 L1748.13 1097.27 L1748.36 1097.82 L1748.58 1098.24 L1748.81 1098.05 L1749.04 1098.62 L1749.27 1098.78 L1749.5 1098.89 L1749.72 1098.04 L1749.95 1097.76 L1750.18 1097.48 L1750.41 1097.87 L1750.63 1097.19 L1750.86 1097.18 L1751.09 1097.34 L1751.32 1097.62 L1751.55 1096.94 L1751.77 1097.23 L1752 1097.02 L1752.23 1097.37 L1752.46 1097.52 L1752.68 1097.85 L1752.91 1097.76 L1753.14 1097.58 L1753.37 1097.58 L1753.6 1097.72 L1753.82 1098.33 L1754.05 1098.48 L1754.28 1098.51 L1754.51 1098.6 L1754.73 1098.37 L1754.96 1098.88 L1755.19 1099.76 L1755.42 1100.51 L1755.65 1100.53 L1755.87 1101.33 L1756.1 1101.42 L1756.33 1100.64 L1756.56 1101.24 L1756.78 1100.63 L1757.01 1101.55 L1757.24 1101.66 L1757.47 1101.44 L1757.7 1101.89 L1757.92 1102.31 L1758.15 1101.99 L1758.38 1101.8 L1758.61 1101.89 L1758.83 1101.8 L1759.06 1102.04 L1759.29 1102.13 L1759.52 1101.24 L1759.75 1101.28 L1759.97 1101.44 L1760.2 1100.4 L1760.43 1100.94 L1760.66 1101.47 L1760.88 1101.03 L1761.11 1101.27 L1761.34 1101.85 L1761.57 1101.89 L1761.8 1101.85 L1762.02 1102.02 L1762.25 1101.76 L1762.48 1102.13 L1762.71 1101.57 L1762.93 1100.8 L1763.16 1100.56 L1763.39 1101.07 L1763.62 1100.9 L1763.85 1100.46 L1764.07 1100.19 L1764.3 1100.19 L1764.53 1100.41 L1764.76 1100.21 L1764.98 1099.82 L1765.21 1099.66 L1765.44 1099.5 L1765.67 1099.63 L1765.9 1099.92 L1766.12 1100.47 L1766.35 1100.69 L1766.58 1100.08 L1766.81 1100.58 L1767.03 1100.88 L1767.26 1100.57 L1767.49 1100.73 L1767.72 1101.01 L1767.94 1100.95 L1768.17 1100.91 L1768.4 1101.15 L1768.63 1101.31 L1768.86 1101.69 L1769.08 1101.17 L1769.31 1101.76 L1769.54 1101.76 L1769.77 1101.12 L1769.99 1101.47 L1770.22 1101.93 L1770.45 1102.12 L1770.68 1102.68 L1770.91 1103.21 L1771.13 1103.77 L1771.36 1104.14 L1771.59 1104.06 L1771.82 1103.38 L1772.04 1103.49 L1772.27 1103.47 L1772.5 1103.9 L1772.73 1103.8 L1772.96 1104.25 L1773.18 1103.91 L1773.41 1103.75 L1773.64 1104.22 L1773.87 1104.4 L1774.09 1104.57 L1774.32 1104.4 L1774.55 1103.83 L1774.78 1103.58 L1775.01 1103.77 L1775.23 1103.95 L1775.46 1103.81 L1775.69 1103.97 L1775.92 1103.59 L1776.14 1103.59 L1776.37 1103.26 L1776.6 1103.15 L1776.83 1103.73 L1777.06 1103.77 L1777.28 1104.04 L1777.51 1104.63 L1777.74 1104.03 L1777.97 1103.77 L1778.19 1104.03 L1778.42 1104.12 L1778.65 1104.48 L1778.88 1104.46 L1779.11 1104.2 L1779.33 1104.11 L1779.56 1103.11 L1779.79 1103.26 L1780.02 1103.4 L1780.24 1103.14 L1780.47 1103.59 L1780.7 1103.45 L1780.93 1104.23 L1781.16 1103.12 L1781.38 1103.66 L1781.61 1103.57 L1781.84 1103.97 L1782.07 1104.61 L1782.29 1104.76 L1782.52 1104.97 L1782.75 1104.1 L1782.98 1104.79 L1783.21 1104.91 L1783.43 1105.07 L1783.66 1104.79 L1783.89 1105.05 L1784.12 1104.64 L1784.34 1104.48 L1784.57 1103.8 L1784.8 1103.7 L1785.03 1102.84 L1785.26 1103.15 L1785.48 1103.13 L1785.71 1102.98 L1785.94 1102.34 L1786.17 1102.28 L1786.39 1102.34 L1786.62 1102.39 L1786.85 1101.89 L1787.08 1101.93 L1787.31 1101.93 L1787.53 1101.68 L1787.76 1101.56 L1787.99 1100.72 L1788.22 1100.21 L1788.44 1100.28 L1788.67 1100.18 L1788.9 1100.19 L1789.13 1100.05 L1789.36 1099.66 L1789.58 1099.49 L1789.81 1099.14 L1790.04 1098.81 L1790.27 1099.02 L1790.49 1099.37 L1790.72 1098.97 L1790.95 1099.01 L1791.18 1098.2 L1791.41 1098.68 L1791.63 1098.47 L1791.86 1098.43 L1792.09 1098.38 L1792.32 1098.88 L1792.54 1098.62 L1792.77 1098.59 L1793 1098.74 L1793.23 1099.67 L1793.46 1099.58 L1793.68 1099.28 L1793.91 1099.56 L1794.14 1099.27 L1794.37 1099.57 L1794.59 1099.39 L1794.82 1099.32 L1795.05 1099.55 L1795.28 1099.2 L1795.51 1099.32 L1795.73 1098.86 L1795.96 1098.96 L1796.19 1098.8 L1796.42 1099.29 L1796.64 1099.22 L1796.87 1099.66 L1797.1 1099.85 L1797.33 1100.13 L1797.56 1100.12 L1797.78 1099.74 L1798.01 1099.84 L1798.24 1099.64 L1798.47 1099 L1798.69 1099.34 L1798.92 1099.22 L1799.15 1099.31 L1799.38 1099.59 L1799.61 1099.11 L1799.83 1099.08 L1800.06 1099.76 L1800.29 1099.49 L1800.52 1099.22 L1800.74 1099.25 L1800.97 1099.47 L1801.2 1098.66 L1801.43 1098.97 L1801.66 1099.69 L1801.88 1100.16 L1802.11 1099.58 L1802.34 1099.88 L1802.57 1099.81 L1802.79 1099.92 L1803.02 1100.52 L1803.25 1099.93 L1803.48 1099.65 L1803.7 1100.11 L1803.93 1100.12 L1804.16 1099.71 L1804.39 1100.02 L1804.62 1100.18 L1804.84 1100.02 L1805.07 1099.5 L1805.3 1099.72 L1805.53 1100 L1805.75 1099.64 L1805.98 1099.73 L1806.21 1100.34 L1806.44 1100.18 L1806.67 1100.26 L1806.89 1099.66 L1807.12 1100.21 L1807.35 1100.18 L1807.58 1100.48 L1807.8 1100.73 L1808.03 1100.37 L1808.26 1100.09 L1808.49 1100.02 L1808.72 1100.57 L1808.94 1100.79 L1809.17 1100.93 L1809.4 1101.07 L1809.63 1101.11 L1809.85 1101.56 L1810.08 1101.9 L1810.31 1102.14 L1810.54 1102.27 L1810.77 1102.15 L1810.99 1102.2 L1811.22 1101.89 L1811.45 1102.13 L1811.68 1102.43 L1811.9 1102.36 L1812.13 1102.02 L1812.36 1102.52 L1812.59 1102.62 L1812.82 1103.15 L1813.04 1103.24 L1813.27 1102.47 L1813.5 1102.92 L1813.73 1103.15 L1813.95 1103.01 L1814.18 1102.93 L1814.41 1103.16 L1814.64 1103.38 L1814.87 1103.86 L1815.09 1104.34 L1815.32 1104.64 L1815.55 1104.57 L1815.78 1104.49 L1816 1105.06 L1816.23 1104.62 L1816.46 1104.89 L1816.69 1104.2 L1816.92 1103.9 L1817.14 1103.81 L1817.37 1103.92 L1817.6 1103.73 L1817.83 1104.3 L1818.05 1104.24 L1818.28 1104.17 L1818.51 1103.91 L1818.74 1103.27 L1818.97 1103.18 L1819.19 1103.1 L1819.42 1102.53 L1819.65 1102.06 L1819.88 1101.43 L1820.1 1101.44 L1820.33 1102.16 L1820.56 1102.01 L1820.79 1102.21 L1821.02 1101.96 L1821.24 1102.7 L1821.47 1102.62 L1821.7 1102.71 L1821.93 1102.18 L1822.15 1102.1 L1822.38 1101.74 L1822.61 1102.35 L1822.84 1102.4 L1823.07 1102.96 L1823.29 1103.7 L1823.52 1104.07 L1823.75 1104.59 L1823.98 1104.45 L1824.2 1103.92 L1824.43 1104.4 L1824.66 1104.2 L1824.89 1103.97 L1825.12 1103.14 L1825.34 1103.44 L1825.57 1103.6 L1825.8 1103.24 L1826.03 1103.21 L1826.25 1103.53 L1826.48 1103.26 L1826.71 1102.78 L1826.94 1102.9 L1827.17 1102.65 L1827.39 1102.25 L1827.62 1102.36 L1827.85 1101.34 L1828.08 1101.38 L1828.3 1101.49 L1828.53 1101.64 L1828.76 1100.81 L1828.99 1100.79 L1829.22 1099.77 L1829.44 1100.07 L1829.67 1100.47 L1829.9 1100.79 L1830.13 1100.89 L1830.35 1100.61 L1830.58 1100.51 L1830.81 1100.15 L1831.04 1100.71 L1831.27 1100.83 L1831.49 1101.58 L1831.72 1102.6 L1831.95 1103.11 L1832.18 1103.45 L1832.4 1104.18 L1832.63 1104.22 L1832.86 1104.73 L1833.09 1104.74 L1833.32 1104.72 L1833.54 1104.55 L1833.77 1104.93 L1834 1104.65 L1834.23 1104.63 L1834.45 1104.41 L1834.68 1104.49 L1834.91 1104.52 L1835.14 1104.98 L1835.37 1104.86 L1835.59 1104.89 L1835.82 1104.89 L1836.05 1104.59 L1836.28 1104.7 L1836.5 1105.29 L1836.73 1105.44 L1836.96 1105.62 L1837.19 1105.79 L1837.42 1106.32 L1837.64 1105.72 L1837.87 1106.04 L1838.1 1106.15 L1838.33 1106.53 L1838.55 1106.87 L1838.78 1107.11 L1839.01 1107.04 L1839.24 1107.63 L1839.46 1108.18 L1839.69 1108.07 L1839.92 1108.91 L1840.15 1108.69 L1840.38 1108.76 L1840.6 1108.45 L1840.83 1108.64 L1841.06 1108.24 L1841.29 1108.76 L1841.51 1109.42 L1841.74 1109 L1841.97 1109.35 L1842.2 1109.24 L1842.43 1108.86 L1842.65 1108.69 L1842.88 1108.2 L1843.11 1108.27 L1843.34 1108.48 L1843.56 1107.65 L1843.79 1107.75 L1844.02 1107.53 L1844.25 1107.75 L1844.48 1107.76 L1844.7 1107.89 L1844.93 1107.71 L1845.16 1107.95 L1845.39 1107.85 L1845.61 1108.29 L1845.84 1108.99 L1846.07 1109.09 L1846.3 1109.26 L1846.53 1109.45 L1846.75 1108.81 L1846.98 1108.77 L1847.21 1109.21 L1847.44 1109.12 L1847.66 1109.14 L1847.89 1109.58 L1848.12 1109.65 L1848.35 1109.43 L1848.58 1109.87 L1848.8 1109.45 L1849.03 1109.76 L1849.26 1109.58 L1849.49 1109.39 L1849.71 1108.85 L1849.94 1108.41 L1850.17 1108.73 L1850.4 1108.91 L1850.63 1107.59 L1850.85 1107.84 L1851.08 1107.48 L1851.31 1107.4 L1851.54 1107.19 L1851.76 1107.42 L1851.99 1107.43 L1852.22 1107.49 L1852.45 1107.23 L1852.68 1106.98 L1852.9 1107.08 L1853.13 1107.67 L1853.36 1107.5 L1853.59 1107.58 L1853.81 1108.18 L1854.04 1108.46 L1854.27 1108.25 L1854.5 1107.99 L1854.73 1107.67 L1854.95 1107.41 L1855.18 1107.79 L1855.41 1107.11 L1855.64 1107.52 L1855.86 1108.38 L1856.09 1108.54 L1856.32 1108.51 L1856.55 1108.22 L1856.78 1108.09 L1857 1107.05 L1857.23 1106.63 L1857.46 1106.08 L1857.69 1106.32 L1857.91 1106.2 L1858.14 1105.79 L1858.37 1106.18 L1858.6 1106.51 L1858.83 1105.85 L1859.05 1105.72 L1859.28 1105.71 L1859.51 1105.64 L1859.74 1106.05 L1859.96 1105.63 L1860.19 1106 L1860.42 1106.18 L1860.65 1106.42 L1860.88 1107.01 L1861.1 1106.52 L1861.33 1106.4 L1861.56 1106.75 L1861.79 1106.14 L1862.01 1106.42 L1862.24 1106.22 L1862.47 1106.42 L1862.7 1106.6 L1862.93 1106.3 L1863.15 1106.3 L1863.38 1106.28 L1863.61 1106.25 L1863.84 1105.73 L1864.06 1105.79 L1864.29 1106.15 L1864.52 1105.88 L1864.75 1105.95 L1864.98 1105.71 L1865.2 1105.81 L1865.43 1106.27 L1865.66 1106.11 L1865.89 1106.6 L1866.11 1106.65 L1866.34 1106.97 L1866.57 1107.3 L1866.8 1107.68 L1867.03 1108.22 L1867.25 1108.14 L1867.48 1108.46 L1867.71 1108.85 L1867.94 1109.02 L1868.16 1108.29 L1868.39 1108.95 L1868.62 1108.89 L1868.85 1109.42 L1869.08 1109.75 L1869.3 1110.54 L1869.53 1110.79 L1869.76 1110.32 L1869.99 1109.88 L1870.21 1109.68 L1870.44 1110 L1870.67 1109.97 L1870.9 1110.17 L1871.13 1109.92 L1871.35 1110.28 L1871.58 1110.08 L1871.81 1110.1 L1872.04 1110.38 L1872.26 1110 L1872.49 1109.74 L1872.72 1109.3 L1872.95 1108.94 L1873.18 1109.72 L1873.4 1109.84 L1873.63 1110.16 L1873.86 1110.75 L1874.09 1110.41 L1874.31 1110.27 L1874.54 1110.02 L1874.77 1109.79 L1875 1109.84 L1875.23 1110.05 L1875.45 1109.98 L1875.68 1110.1 L1875.91 1110.29 L1876.14 1110.2 L1876.36 1109.89 L1876.59 1109.73 L1876.82 1109.62 L1877.05 1109.68 L1877.27 1109.35 L1877.5 1109.94 L1877.73 1110.14 L1877.96 1110.43 L1878.19 1110.29 L1878.41 1109.22 L1878.64 1109.24 L1878.87 1109.53 L1879.1 1109.29 L1879.32 1109.63 L1879.55 1109.55 L1879.78 1109.53 L1880.01 1109.88 L1880.24 1109.99 L1880.46 1110.11 L1880.69 1110.22 L1880.92 1110.02 L1881.15 1110.28 L1881.37 1110.53 L1881.6 1110.66 L1881.83 1110.77 L1882.06 1111.01 L1882.29 1111.63 L1882.51 1112 L1882.74 1111.99 L1882.97 1111.46 L1883.2 1111.95 L1883.42 1112.18 L1883.65 1112.09 L1883.88 1112.4 L1884.11 1112.38 L1884.34 1112.41 L1884.56 1112.37 L1884.79 1112.58 L1885.02 1112.86 L1885.25 1113.32 L1885.47 1113.52 L1885.7 1112.76 L1885.93 1112.32 L1886.16 1113.1 L1886.39 1113.02 L1886.61 1113.42 L1886.84 1113.21 L1887.07 1113.11 L1887.3 1113.3 L1887.52 1113.75 L1887.75 1114.15 L1887.98 1113.22 L1888.21 1113.73 L1888.44 1113.38 L1888.66 1112.8 L1888.89 1112.7 L1889.12 1112.76 L1889.35 1112.89 L1889.57 1113.15 L1889.8 1113.17 L1890.03 1113.26 L1890.26 1113.44 L1890.49 1113.37 L1890.71 1112.62 L1890.94 1112.66 L1891.17 1113.1 L1891.4 1112.99 L1891.62 1112.71 L1891.85 1113.24 L1892.08 1113.82 L1892.31 1113.9 L1892.54 1114.08 L1892.76 1114.06 L1892.99 1113.92 L1893.22 1114.16 L1893.45 1114.44 L1893.67 1115.1 L1893.9 1114.56 L1894.13 1114.03 L1894.36 1113.95 L1894.59 1113.32 L1894.81 1113.31 L1895.04 1113.46 L1895.27 1113.52 L1895.5 1113.03 L1895.72 1113.43 L1895.95 1112.41 L1896.18 1112.51 L1896.41 1112.74 L1896.64 1112.51 L1896.86 1113.17 L1897.09 1113.54 L1897.32 1113.96 L1897.55 1113.94 L1897.77 1113.82 L1898 1112.96 L1898.23 1113.59 L1898.46 1113.86 L1898.69 1114.09 L1898.91 1113.67 L1899.14 1113.75 L1899.37 1113.78 L1899.6 1113.42 L1899.82 1113.24 L1900.05 1113.26 L1900.28 1113.5 L1900.51 1113.74 L1900.74 1113.7 L1900.96 1113.97 L1901.19 1114.53 L1901.42 1114.64 L1901.65 1114.79 L1901.87 1115.11 L1902.1 1115.17 L1902.33 1115.59 L1902.56 1116.01 L1902.79 1115.57 L1903.01 1115.32 L1903.24 1114.31 L1903.47 1114.39 L1903.7 1114.7 L1903.92 1114.49 L1904.15 1114.07 L1904.38 1114.01 L1904.61 1113.95 L1904.84 1113.28 L1905.06 1113.34 L1905.29 1113.21 L1905.52 1112.66 L1905.75 1113.18 L1905.97 1113.57 L1906.2 1113.5 L1906.43 1113.92 L1906.66 1113.9 L1906.89 1113.81 L1907.11 1114.26 L1907.34 1114.41 L1907.57 1113.61 L1907.8 1113.59 L1908.02 1114.24 L1908.25 1114.7 L1908.48 1114.48 L1908.71 1114.58 L1908.94 1114.68 L1909.16 1115.73 L1909.39 1115.36 L1909.62 1114.55 L1909.85 1114.47 L1910.07 1114.97 L1910.3 1114.18 L1910.53 1114.3 L1910.76 1114.41 L1910.99 1114.09 L1911.21 1113.86 L1911.44 1114.15 L1911.67 1114.18 L1911.9 1114.43 L1912.12 1114.48 L1912.35 1113.8 L1912.58 1113.5 L1912.81 1113.46 L1913.03 1113.53 L1913.26 1113.5 L1913.49 1113.82 L1913.72 1114.25 L1913.95 1114.28 L1914.17 1114.08 L1914.4 1114.72 L1914.63 1115.19 L1914.86 1115.96 L1915.08 1116.36 L1915.31 1116.13 L1915.54 1116.5 L1915.77 1116.39 L1916 1115.72 L1916.22 1114.97 L1916.45 1115.27 L1916.68 1114.86 L1916.91 1115.18 L1917.13 1114.86 L1917.36 1114.05 L1917.59 1114.02 L1917.82 1114.28 L1918.05 1114.7 L1918.27 1114.5 L1918.5 1114.94 L1918.73 1114.74 L1918.96 1115 L1919.18 1114.83 L1919.41 1114.41 L1919.64 1113.43 L1919.87 1113.18 L1920.1 1112.9 L1920.32 1112.82 L1920.55 1113.23 L1920.78 1112.63 L1921.01 1112.59 L1921.23 1112.58 L1921.46 1112.23 L1921.69 1112.37 L1921.92 1112.67 L1922.15 1112.5 L1922.37 1112.36 L1922.6 1112.25 L1922.83 1112.12 L1923.06 1112.43 L1923.28 1112.39 L1923.51 1111.85 L1923.74 1111.75 L1923.97 1112.15 L1924.2 1112.44 L1924.42 1112.56 L1924.65 1112.62 L1924.88 1112.37 L1925.11 1112.81 L1925.33 1112.63 L1925.56 1112.08 L1925.79 1112.41 L1926.02 1112.19 L1926.25 1112.25 L1926.47 1112.6 L1926.7 1112.15 L1926.93 1112.37 L1927.16 1112.76 L1927.38 1112.79 L1927.61 1112.54 L1927.84 1112.44 L1928.07 1112.65 L1928.3 1112.72 L1928.52 1112.52 L1928.75 1112.48 L1928.98 1112.27 L1929.21 1111.37 L1929.43 1111.07 L1929.66 1111.65 L1929.89 1112 L1930.12 1112.44 L1930.35 1111.84 L1930.57 1111.79 L1930.8 1111.27 L1931.03 1111.29 L1931.26 1111.49 L1931.48 1111.2 L1931.71 1111.61 L1931.94 1111.71 L1932.17 1111.85 L1932.4 1111.2 L1932.62 1110.43 L1932.85 1110.66 L1933.08 1110.66 L1933.31 1110.79 L1933.53 1110.73 L1933.76 1110.49 L1933.99 1110.66 L1934.22 1111.33 L1934.45 1111.32 L1934.67 1109.92 L1934.9 1109.88 L1935.13 1110.25 L1935.36 1110.33 L1935.58 1110.56 L1935.81 1109.93 L1936.04 1109.78 L1936.27 1109.85 L1936.5 1110.14 L1936.72 1109.49 L1936.95 1109.64 L1937.18 1109.83 L1937.41 1109.81 L1937.63 1109.99 L1937.86 1109.87 L1938.09 1110.31 L1938.32 1110.73 L1938.55 1110.39 L1938.77 1110.88 L1939 1110.99 L1939.23 1110.94 L1939.46 1111.13 L1939.68 1111.13 L1939.91 1111.36 L1940.14 1111.49 L1940.37 1111.44 L1940.6 1111.69 L1940.82 1111.39 L1941.05 1111.43 L1941.28 1111.6 L1941.51 1111.37 L1941.73 1111.34 L1941.96 1111.4 L1942.19 1110.68 L1942.42 1109.83 L1942.65 1109.17 L1942.87 1109.24 L1943.1 1109.67 L1943.33 1109.97 L1943.56 1109.85 L1943.78 1109.98 L1944.01 1109.87 L1944.24 1109.18 L1944.47 1109.17 L1944.7 1109.41 L1944.92 1109.37 L1945.15 1110.1 L1945.38 1109.44 L1945.61 1109.55 L1945.83 1109.89 L1946.06 1109.91 L1946.29 1109.78 L1946.52 1109.33 L1946.75 1109.05 L1946.97 1109.09 L1947.2 1109.28 L1947.43 1109.47 L1947.66 1108.99 L1947.88 1109.01 L1948.11 1109 L1948.34 1108.05 L1948.57 1108.5 L1948.79 1108.99 L1949.02 1109.14 L1949.25 1109.05 L1949.48 1108.78 L1949.71 1108.82 L1949.93 1109.27 L1950.16 1108.77 L1950.39 1108.79 L1950.62 1108.89 L1950.84 1108.4 L1951.07 1108.17 L1951.3 1108.33 L1951.53 1108.37 L1951.76 1108.06 L1951.98 1108.65 L1952.21 1108.62 L1952.44 1108.7 L1952.67 1108.86 L1952.89 1109.42 L1953.12 1109.98 L1953.35 1109.62 L1953.58 1109.3 L1953.81 1108.78 L1954.03 1108.71 L1954.26 1108.74 L1954.49 1109.03 L1954.72 1108.4 L1954.94 1108.19 L1955.17 1107.36 L1955.4 1106.93 L1955.63 1107.39 L1955.86 1107.34 L1956.08 1107.43 L1956.31 1107.61 L1956.54 1107.7 L1956.77 1107.78 L1956.99 1107.29 L1957.22 1106.97 L1957.45 1106.86 L1957.68 1107.15 L1957.91 1106.61 L1958.13 1105.93 L1958.36 1105.89 L1958.59 1106.35 L1958.82 1106.88 L1959.04 1107.01 L1959.27 1107.07 L1959.5 1106.79 L1959.73 1106.88 L1959.96 1107.11 L1960.18 1106.3 L1960.41 1106.41 L1960.64 1106.36 L1960.87 1105.53 L1961.09 1105.98 L1961.32 1106.07 L1961.55 1105.79 L1961.78 1105.7 L1962.01 1105.72 L1962.23 1105.15 L1962.46 1105.03 L1962.69 1105.04 L1962.92 1105.65 L1963.14 1105.33 L1963.37 1104.81 L1963.6 1104.56 L1963.83 1104.79 L1964.06 1105.13 L1964.28 1105.16 L1964.51 1105.07 L1964.74 1105.12 L1964.97 1105.03 L1965.19 1105.37 L1965.42 1105.16 L1965.65 1105.11 L1965.88 1105.23 L1966.11 1105.27 L1966.33 1105.18 L1966.56 1105.8 L1966.79 1106.09 L1967.02 1105.93 L1967.24 1106.15 L1967.47 1106.55 L1967.7 1106.48 L1967.93 1106.79 L1968.16 1106.88 L1968.38 1107.28 L1968.61 1107.75 L1968.84 1108.04 L1969.07 1107.91 L1969.29 1108.07 L1969.52 1108.01 L1969.75 1107.8 L1969.98 1107.84 L1970.21 1107.99 L1970.43 1108.29 L1970.66 1108.04 L1970.89 1108.41 L1971.12 1108.6 L1971.34 1108.94 L1971.57 1108.51 L1971.8 1109.02 L1972.03 1109.06 L1972.26 1108.84 L1972.48 1108.71 L1972.71 1108.19 L1972.94 1108.69 L1973.17 1108.64 L1973.39 1108.88 L1973.62 1109.17 L1973.85 1108.74 L1974.08 1108.75 L1974.31 1108.89 L1974.53 1108.97 L1974.76 1109.28 L1974.99 1109.91 L1975.22 1109.59 L1975.44 1109.51 L1975.67 1109.52 L1975.9 1109.34 L1976.13 1109.48 L1976.36 1109.26 L1976.58 1109.45 L1976.81 1109.49 L1977.04 1109.09 L1977.27 1108.95 L1977.49 1109.5 L1977.72 1109.59 L1977.95 1109.99 L1978.18 1110.38 L1978.41 1110.66 L1978.63 1111.03 L1978.86 1111.16 L1979.09 1111.56 L1979.32 1111.61 L1979.54 1111.96 L1979.77 1112.45 L1980 1112.16 L1980.23 1112.44 L1980.46 1113.35 L1980.68 1113.53 L1980.91 1113.42 L1981.14 1113.79 L1981.37 1113.57 L1981.59 1114.12 L1981.82 1114.37 L1982.05 1114.14 L1982.28 1114.39 L1982.51 1114.38 L1982.73 1114.37 L1982.96 1114.28 L1983.19 1113.73 L1983.42 1114.28 L1983.64 1114.71 L1983.87 1114.79 L1984.1 1114.43 L1984.33 1114.09 L1984.56 1113.77 L1984.78 1113.15 L1985.01 1113.82 L1985.24 1113.66 L1985.47 1113.6 L1985.69 1113.96 L1985.92 1113.67 L1986.15 1113.83 L1986.38 1113.98 L1986.6 1113.77 L1986.83 1114.26 L1987.06 1114.26 L1987.29 1113.92 L1987.52 1114.11 L1987.74 1114.49 L1987.97 1114.12 L1988.2 1114.43 L1988.43 1114.67 L1988.65 1114.84 L1988.88 1114.83 L1989.11 1115.14 L1989.34 1114.74 L1989.57 1115.05 L1989.79 1115.09 L1990.02 1114.59 L1990.25 1114.67 L1990.48 1115.01 L1990.7 1115.06 L1990.93 1115.04 L1991.16 1115.41 L1991.39 1115.65 L1991.62 1116.05 L1991.84 1115.74 L1992.07 1115.94 L1992.3 1115.96 L1992.53 1115.67 L1992.75 1115.65 L1992.98 1115.31 L1993.21 1114.73 L1993.44 1114.5 L1993.67 1114.09 L1993.89 1113.83 L1994.12 1113.99 L1994.35 1113.69 L1994.58 1113.61 L1994.8 1113.45 L1995.03 1113.54 L1995.26 1113.43 L1995.49 1113.68 L1995.72 1113.55 L1995.94 1113.44 L1996.17 1113.12 L1996.4 1112.26 L1996.63 1112.48 L1996.85 1113.24 L1997.08 1113.51 L1997.31 1113.35 L1997.54 1113.3 L1997.77 1113.28 L1997.99 1113.45 L1998.22 1112.93 L1998.45 1112.51 L1998.68 1112.83 L1998.9 1113.07 L1999.13 1112.78 L1999.36 1113.23 L1999.59 1112.33 L1999.82 1111.97 L2000.04 1111.87 L2000.27 1111.6 L2000.5 1111.38 L2000.73 1110.85 L2000.95 1110.52 L2001.18 1110.45 L2001.41 1110.47 L2001.64 1110.34 L2001.87 1110.44 L2002.09 1110.47 L2002.32 1110.89 L2002.55 1110.44 L2002.78 1110.56 L2003 1110.98 L2003.23 1111.08 L2003.46 1110.84 L2003.69 1110.73 L2003.92 1111.16 L2004.14 1111.05 L2004.37 1110.94 L2004.6 1110.92 L2004.83 1110.88 L2005.05 1111.2 L2005.28 1111.24 L2005.51 1111.07 L2005.74 1110.9 L2005.97 1111.42 L2006.19 1112.05 L2006.42 1111.73 L2006.65 1111.49 L2006.88 1111.33 L2007.1 1111.01 L2007.33 1110.41 L2007.56 1110.65 L2007.79 1111.08 L2008.02 1110.86 L2008.24 1111.1 L2008.47 1111.23 L2008.7 1111.07 L2008.93 1110.38 L2009.15 1110.8 L2009.38 1110.99 L2009.61 1110.07 L2009.84 1110.28 L2010.07 1110.19 L2010.29 1110.09 L2010.52 1110.21 L2010.75 1110.29 L2010.98 1110.45 L2011.2 1110.62 L2011.43 1110.67 L2011.66 1111.03 L2011.89 1110.97 L2012.12 1111.36 L2012.34 1110.99 L2012.57 1110.99 L2012.8 1110.56 L2013.03 1110.19 L2013.25 1110.38 L2013.48 1110.41 L2013.71 1110.62 L2013.94 1110.79 L2014.17 1111.14 L2014.39 1110.84 L2014.62 1110.43 L2014.85 1110.43 L2015.08 1110.46 L2015.3 1110.61 L2015.53 1110.37 L2015.76 1109.98 L2015.99 1110.31 L2016.22 1110.26 L2016.44 1110.56 L2016.67 1110.1 L2016.9 1110.82 L2017.13 1111.3 L2017.35 1110.81 L2017.58 1111.47 L2017.81 1111.09 L2018.04 1110.53 L2018.27 1109.76 L2018.49 1110.12 L2018.72 1109.51 L2018.95 1109.5 L2019.18 1109.06 L2019.4 1109.37 L2019.63 1108.53 L2019.86 1108.71 L2020.09 1108.08 L2020.32 1107.83 L2020.54 1108.35 L2020.77 1108.53 L2021 1108.03 L2021.23 1108.25 L2021.45 1108.79 L2021.68 1109.35 L2021.91 1109.1 L2022.14 1109.21 L2022.36 1109.41 L2022.59 1109.4 L2022.82 1109.27 L2023.05 1108.87 L2023.28 1109.17 L2023.5 1109.09 L2023.73 1109.34 L2023.96 1109.34 L2024.19 1109.26 L2024.41 1109.32 L2024.64 1109.91 L2024.87 1109.61 L2025.1 1109.77 L2025.33 1110.47 L2025.55 1110.75 L2025.78 1110.91 L2026.01 1110.55 L2026.24 1110.43 L2026.46 1110.18 L2026.69 1110.49 L2026.92 1110.73 L2027.15 1111.35 L2027.38 1111.04 L2027.6 1111.44 L2027.83 1111.64 L2028.06 1111.45 L2028.29 1111 L2028.51 1111.07 L2028.74 1111.04 L2028.97 1111.19 L2029.2 1111.27 L2029.43 1111.25 L2029.65 1111.69 L2029.88 1112.15 L2030.11 1111.92 L2030.34 1111.24 L2030.56 1111.54 L2030.79 1111.6 L2031.02 1111.16 L2031.25 1111.03 L2031.48 1111.72 L2031.7 1111.67 L2031.93 1112.09 L2032.16 1112.47 L2032.39 1112.45 L2032.61 1113.07 L2032.84 1113.68 L2033.07 1113.6 L2033.3 1113.54 L2033.53 1113.51 L2033.75 1113.46 L2033.98 1113.59 L2034.21 1113.46 L2034.44 1113.49 L2034.66 1113.58 L2034.89 1113.94 L2035.12 1113.95 L2035.35 1114.23 L2035.58 1114.78 L2035.8 1114.83 L2036.03 1114.93 L2036.26 1114.68 L2036.49 1114.41 L2036.71 1114.51 L2036.94 1114.62 L2037.17 1114.62 L2037.4 1114.67 L2037.63 1114.82 L2037.85 1115.1 L2038.08 1115.77 L2038.31 1115.97 L2038.54 1116.42 L2038.76 1116.24 L2038.99 1116.22 L2039.22 1116.58 L2039.45 1116.96 L2039.68 1116.86 L2039.9 1117.04 L2040.13 1116.53 L2040.36 1116.12 L2040.59 1116.63 L2040.81 1116.77 L2041.04 1116.38 L2041.27 1116.28 L2041.5 1116.4 L2041.73 1116.8 L2041.95 1117.32 L2042.18 1117.11 L2042.41 1117.11 L2042.64 1117.14 L2042.86 1117.57 L2043.09 1117.87 L2043.32 1117.49 L2043.55 1117.73 L2043.78 1117.79 L2044 1117.61 L2044.23 1117.33 L2044.46 1117.59 L2044.69 1116.84 L2044.91 1116.62 L2045.14 1116.38 L2045.37 1115.77 L2045.6 1116.11 L2045.83 1115.94 L2046.05 1116.3 L2046.28 1116.32 L2046.51 1116.68 L2046.74 1116.84 L2046.96 1116.75 L2047.19 1116.79 L2047.42 1116.73 L2047.65 1116.71 L2047.88 1116.95 L2048.1 1117.61 L2048.33 1117.95 L2048.56 1116.37 L2048.79 1116.25 L2049.01 1116.06 L2049.24 1115.78 L2049.47 1115.49 L2049.7 1115.48 L2049.93 1115.31 L2050.15 1115.11 L2050.38 1115.2 L2050.61 1115.48 L2050.84 1116.07 L2051.06 1116.44 L2051.29 1116.48 L2051.52 1115.81 L2051.75 1116.14 L2051.98 1115.86 L2052.2 1115.92 L2052.43 1116.1 L2052.66 1116.27 L2052.89 1116.11 L2053.11 1116.47 L2053.34 1116.6 L2053.57 1116.77 L2053.8 1116.55 L2054.03 1116.59 L2054.25 1116.17 L2054.48 1115.97 L2054.71 1116.07 L2054.94 1116.6 L2055.16 1116.75 L2055.39 1116.47 L2055.62 1116.27 L2055.85 1116.44 L2056.08 1116.1 L2056.3 1116.19 L2056.53 1116.07 L2056.76 1115.82 L2056.99 1115.72 L2057.21 1115.53 L2057.44 1115.65 L2057.67 1115.79 L2057.9 1115.74 L2058.12 1115.97 L2058.35 1115.41 L2058.58 1115.47 L2058.81 1115.94 L2059.04 1116.32 L2059.26 1116.76 L2059.49 1116.38 L2059.72 1116.79 L2059.95 1116.76 L2060.17 1117.24 L2060.4 1116.93 L2060.63 1117.21 L2060.86 1117.79 L2061.09 1117.86 L2061.31 1117.66 L2061.54 1117.04 L2061.77 1116.77 L2062 1116.13 L2062.22 1116.62 L2062.45 1116.85 L2062.68 1116.82 L2062.91 1117.04 L2063.14 1116.85 L2063.36 1116.48 L2063.59 1116.43 L2063.82 1116.94 L2064.05 1116.81 L2064.27 1116.01 L2064.5 1115.91 L2064.73 1115.84 L2064.96 1115.53 L2065.19 1115.74 L2065.41 1115.57 L2065.64 1115.9 L2065.87 1115.68 L2066.1 1116 L2066.32 1116.31 L2066.55 1116.76 L2066.78 1116.73 L2067.01 1117.29 L2067.24 1116.97 L2067.46 1116.62 L2067.69 1116.46 L2067.92 1116.83 L2068.15 1117.19 L2068.37 1117.24 L2068.6 1116.39 L2068.83 1116.76 L2069.06 1117.01 L2069.29 1117.15 L2069.51 1117.64 L2069.74 1117.72 L2069.97 1117.42 L2070.2 1116.63 L2070.42 1116.59 L2070.65 1116.49 L2070.88 1116.74 L2071.11 1117.13 L2071.34 1117.19 L2071.56 1117.03 L2071.79 1116.66 L2072.02 1116.79 L2072.25 1116.96 L2072.47 1116.96 L2072.7 1116.48 L2072.93 1116.44 L2073.16 1115.89 L2073.39 1115.75 L2073.61 1115.66 L2073.84 1115.4 L2074.07 1115.42 L2074.3 1114.91 L2074.52 1115.25 L2074.75 1114.88 L2074.98 1114.86 L2075.21 1114.78 L2075.44 1114.92 L2075.66 1115.3 L2075.89 1116.04 L2076.12 1115.79 L2076.35 1115.66 L2076.57 1115.39 L2076.8 1115.16 L2077.03 1115.19 L2077.26 1114.72 L2077.49 1114.6 L2077.71 1114.02 L2077.94 1113.92 L2078.17 1114.24 L2078.4 1114.2 L2078.62 1114.28 L2078.85 1114.81 L2079.08 1114.56 L2079.31 1114.91 L2079.54 1115.43 L2079.76 1115.75 L2079.99 1115.77 L2080.22 1115.76 L2080.45 1116.05 L2080.67 1116.35 L2080.9 1117.07 L2081.13 1117.72 L2081.36 1117.99 L2081.59 1118.1 L2081.81 1118.01 L2082.04 1117.45 L2082.27 1117.08 L2082.5 1116.97 L2082.72 1116.37 L2082.95 1116.38 L2083.18 1116.43 L2083.41 1116.45 L2083.64 1116.26 L2083.86 1116.48 L2084.09 1116.35 L2084.32 1116.63 L2084.55 1116.81 L2084.77 1116.72 L2085 1116.28 L2085.23 1116.5 L2085.46 1116.91 L2085.69 1116.59 L2085.91 1116.79 L2086.14 1117.38 L2086.37 1117.57 L2086.6 1117.47 L2086.82 1117.82 L2087.05 1118.08 L2087.28 1118.57 L2087.51 1118.45 L2087.74 1117.78 L2087.96 1117.72 L2088.19 1117.73 L2088.42 1117.82 L2088.65 1117.99 L2088.87 1117.59 L2089.1 1117.78 L2089.33 1117.78 L2089.56 1117.78 L2089.79 1117.91 L2090.01 1117.8 L2090.24 1118.03 L2090.47 1118.23 L2090.7 1118.35 L2090.92 1118.75 L2091.15 1119.23 L2091.38 1118.8 L2091.61 1118.49 L2091.84 1118.52 L2092.06 1118.36 L2092.29 1118.5 L2092.52 1118.08 L2092.75 1118.06 L2092.97 1117.88 L2093.2 1117.46 L2093.43 1116.84 L2093.66 1116.33 L2093.89 1116.02 L2094.11 1116.42 L2094.34 1116.44 L2094.57 1116.2 L2094.8 1116 L2095.02 1115.56 L2095.25 1115.5 L2095.48 1115.1 L2095.71 1114.96 L2095.93 1114.41 L2096.16 1114.18 L2096.39 1114.44 L2096.62 1114.98 L2096.85 1114.57 L2097.07 1115.29 L2097.3 1115.13 L2097.53 1115.48 L2097.76 1115.13 L2097.98 1115.54 L2098.21 1115.92 L2098.44 1116.03 L2098.67 1115.9 L2098.9 1116.15 L2099.12 1115.42 L2099.35 1115.67 L2099.58 1116.22 L2099.81 1116.43 L2100.03 1116.31 L2100.26 1115.57 L2100.49 1116.07 L2100.72 1115.28 L2100.95 1115.04 L2101.17 1115.11 L2101.4 1114.63 L2101.63 1114.82 L2101.86 1115.28 L2102.08 1115.14 L2102.31 1115.02 L2102.54 1115.34 L2102.77 1115.6 L2103 1116.15 L2103.22 1115.61 L2103.45 1115.62 L2103.68 1115.27 L2103.91 1115.32 L2104.13 1115.86 L2104.36 1116 L2104.59 1116.02 L2104.82 1116.05 L2105.05 1115.54 L2105.27 1115.44 L2105.5 1116.03 L2105.73 1115.92 L2105.96 1116.1 L2106.18 1115.73 L2106.41 1115.83 L2106.64 1116.3 L2106.87 1116.3 L2107.1 1116.39 L2107.32 1116.6 L2107.55 1116.79 L2107.78 1117.05 L2108.01 1117.37 L2108.23 1117.48 L2108.46 1117.64 L2108.69 1117.85 L2108.92 1117.91 L2109.15 1118.24 L2109.37 1118.32 L2109.6 1118.57 L2109.83 1118.52 L2110.06 1118.48 L2110.28 1118.75 L2110.51 1118.2 L2110.74 1118.53 L2110.97 1118.2 L2111.2 1118.45 L2111.42 1118.34 L2111.65 1118.4 L2111.88 1118.44 L2112.11 1118.98 L2112.33 1118.76 L2112.56 1119.18 L2112.79 1119.44 L2113.02 1119.9 L2113.25 1120.12 L2113.47 1120.16 L2113.7 1120.23 L2113.93 1119.74 L2114.16 1119.76 L2114.38 1120.29 L2114.61 1120.32 L2114.84 1119.64 L2115.07 1119.94 L2115.3 1120.38 L2115.52 1120.14 L2115.75 1120.37 L2115.98 1120.47 L2116.21 1120.63 L2116.43 1120.65 L2116.66 1121.36 L2116.89 1121.68 L2117.12 1121.84 L2117.35 1121.45 L2117.57 1121.44 L2117.8 1121.39 L2118.03 1121.42 L2118.26 1120.49 L2118.48 1120.69 L2118.71 1121.18 L2118.94 1120.94 L2119.17 1121.08 L2119.4 1121.15 L2119.62 1121.16 L2119.85 1121.13 L2120.08 1121.25 L2120.31 1121.63 L2120.53 1121.33 L2120.76 1121.78 L2120.99 1121.27 L2121.22 1120.75 L2121.45 1120.67 L2121.67 1120.39 L2121.9 1119.72 L2122.13 1119.32 L2122.36 1119.58 L2122.58 1119.78 L2122.81 1120.14 L2123.04 1120.45 L2123.27 1120.68 L2123.5 1120.48 L2123.72 1120.5 L2123.95 1121 L2124.18 1121.18 L2124.41 1120.96 L2124.63 1121.09 L2124.86 1120.91 L2125.09 1121.29 L2125.32 1121.53 L2125.55 1121.14 L2125.77 1121.73 L2126 1121.66 L2126.23 1121.91 L2126.46 1121.89 L2126.68 1121.95 L2126.91 1121.22 L2127.14 1120.96 L2127.37 1121.03 L2127.6 1121.23 L2127.82 1121.36 L2128.05 1121.74 L2128.28 1121.6 L2128.51 1121.42 L2128.73 1121.78 L2128.96 1121.75 L2129.19 1121.56 L2129.42 1121.17 L2129.65 1121.07 L2129.87 1120.58 L2130.1 1121.09 L2130.33 1121.06 L2130.56 1120.97 L2130.78 1120.91 L2131.01 1120.78 L2131.24 1120.78 L2131.47 1120.13 L2131.69 1119.93 L2131.92 1120.23 L2132.15 1120.46 L2132.38 1121.14 L2132.61 1121.87 L2132.83 1122.25 L2133.06 1122.6 L2133.29 1122.11 L2133.52 1121.82 L2133.74 1121.25 L2133.97 1121.3 L2134.2 1121.26 L2134.43 1121.12 L2134.66 1121.07 L2134.88 1121.21 L2135.11 1121.38 L2135.34 1121.14 L2135.57 1121.24 L2135.79 1121.15 L2136.02 1121.48 L2136.25 1121.73 L2136.48 1121.79 L2136.71 1121.87 L2136.93 1121.95 L2137.16 1122.05 L2137.39 1121.5 L2137.62 1121.52 L2137.84 1121.07 L2138.07 1120.43 L2138.3 1119.95 L2138.53 1120.64 L2138.76 1120.72 L2138.98 1120.65 L2139.21 1121.46 L2139.44 1121.34 L2139.67 1120.54 L2139.89 1121 L2140.12 1121.02 L2140.35 1121.21 L2140.58 1121.13 L2140.81 1121.23 L2141.03 1120.55 L2141.26 1120.86 L2141.49 1120.86 L2141.72 1121.03 L2141.94 1121.36 L2142.17 1121.3 L2142.4 1121.2 L2142.63 1120.82 L2142.86 1120.5 L2143.08 1119.33 L2143.31 1119.39 L2143.54 1119.5 L2143.77 1120.11 L2143.99 1118.74 L2144.22 1119.05 L2144.45 1119.25 L2144.68 1119 L2144.91 1119.52 L2145.13 1119.88 L2145.36 1119.18 L2145.59 1118.59 L2145.82 1118.43 L2146.04 1118.43 L2146.27 1118.94 L2146.5 1118.45 L2146.73 1118.14 L2146.96 1117.79 L2147.18 1117.4 L2147.41 1117.13 L2147.64 1117.05 L2147.87 1116.62 L2148.09 1116.4 L2148.32 1116.83 L2148.55 1116.76 L2148.78 1116.34 L2149.01 1116.47 L2149.23 1116.15 L2149.46 1115.28 L2149.69 1114.81 L2149.92 1114.81 L2150.14 1114.31 L2150.37 1114.07 L2150.6 1113.42 L2150.83 1113.32 L2151.06 1113.51 L2151.28 1113.68 L2151.51 1113.58 L2151.74 1113.98 L2151.97 1113.83 L2152.19 1113.99 L2152.42 1113.94 L2152.65 1114.12 L2152.88 1114.16 L2153.11 1114.23 L2153.33 1114.35 L2153.56 1114.69 L2153.79 1114.52 L2154.02 1114.35 L2154.24 1114.5 L2154.47 1114.1 L2154.7 1114.61 L2154.93 1114.46 L2155.16 1114.76 L2155.38 1114.61 L2155.61 1114.52 L2155.84 1114.65 L2156.07 1114.89 L2156.29 1115.34 L2156.52 1115.9 L2156.75 1116.01 L2156.98 1115.92 L2157.21 1115.78 L2157.43 1115.52 L2157.66 1115.23 L2157.89 1115.59 L2158.12 1115.16 L2158.34 1114.57 L2158.57 1114.75 L2158.8 1114.39 L2159.03 1114.46 L2159.26 1114.2 L2159.48 1113.87 L2159.71 1114.03 L2159.94 1114.11 L2160.17 1113.91 L2160.39 1113.75 L2160.62 1114.25 L2160.85 1113.77 L2161.08 1113.82 L2161.31 1113.65 L2161.53 1112.87 L2161.76 1112.64 L2161.99 1112.94 L2162.22 1112.55 L2162.44 1111.95 L2162.67 1112.07 L2162.9 1112.21 L2163.13 1112.47 L2163.36 1112.2 L2163.58 1112.29 L2163.81 1112.17 L2164.04 1111.89 L2164.27 1112.36 L2164.49 1112.34 L2164.72 1112 L2164.95 1112.12 L2165.18 1112.52 L2165.41 1112.18 L2165.63 1112.42 L2165.86 1112.24 L2166.09 1112.17 L2166.32 1112.09 L2166.54 1111.7 L2166.77 1111.52 L2167 1111.24 L2167.23 1111.63 L2167.45 1111.36 L2167.68 1111.8 L2167.91 1111.85 L2168.14 1112.41 L2168.37 1111.72 L2168.59 1110.54 L2168.82 1110.59 L2169.05 1111.14 L2169.28 1111.01 L2169.5 1110.77 L2169.73 1110.77 L2169.96 1111.06 L2170.19 1111.13 L2170.42 1111.44 L2170.64 1111.81 L2170.87 1111.9 L2171.1 1111.73 L2171.33 1111.97 L2171.55 1112.29 L2171.78 1112.39 L2172.01 1112.68 L2172.24 1113.13 L2172.47 1112.87 L2172.69 1112.84 L2172.92 1112.94 L2173.15 1113.15 L2173.38 1113.37 L2173.6 1113.42 L2173.83 1113.64 L2174.06 1113.54 L2174.29 1113.6 L2174.52 1114.15 L2174.74 1114.58 L2174.97 1114.78 L2175.2 1114.69 L2175.43 1114.99 L2175.65 1115.67 L2175.88 1115.89 L2176.11 1115.4 L2176.34 1115.01 L2176.57 1114.64 L2176.79 1114.9 L2177.02 1114.65 L2177.25 1114.77 L2177.48 1114.98 L2177.7 1114.59 L2177.93 1114.44 L2178.16 1114.32 L2178.39 1114.05 L2178.62 1113.9 L2178.84 1114.03 L2179.07 1113.94 L2179.3 1113.76 L2179.53 1113.61 L2179.75 1113.49 L2179.98 1112.97 L2180.21 1113.12 L2180.44 1113.04 L2180.67 1113.3 L2180.89 1114.06 L2181.12 1114.41 L2181.35 1114.99 L2181.58 1115.06 L2181.8 1114.86 L2182.03 1114.97 L2182.26 1115.25 L2182.49 1114.79 L2182.72 1114.99 L2182.94 1114.73 L2183.17 1114.67 L2183.4 1113.75 L2183.63 1113.42 L2183.85 1113.12 L2184.08 1113.28 L2184.31 1113.69 L2184.54 1114.05 L2184.77 1114.69 L2184.99 1114.46 L2185.22 1114.49 L2185.45 1114.72 L2185.68 1115.14 L2185.9 1115.01 L2186.13 1115.21 L2186.36 1114.83 L2186.59 1114.35 L2186.82 1114.93 L2187.04 1114.34 L2187.27 1114.24 L2187.5 1114.45 L2187.73 1114.21 L2187.95 1113.95 L2188.18 1114.11 L2188.41 1113.79 L2188.64 1113.83 L2188.87 1113.36 L2189.09 1113.54 L2189.32 1112.9 L2189.55 1113.25 L2189.78 1112.95 L2190 1113.36 L2190.23 1113.7 L2190.46 1113.21 L2190.69 1113.48 L2190.92 1113.36 L2191.14 1113.19 L2191.37 1113.35 L2191.6 1113.43 L2191.83 1113.84 L2192.05 1113.89 L2192.28 1114.11 L2192.51 1113.35 L2192.74 1113.43 L2192.97 1113.59 L2193.19 1113.85 L2193.42 1113.84 L2193.65 1114.05 L2193.88 1114.25 L2194.1 1114.12 L2194.33 1113.87 L2194.56 1114.04 L2194.79 1113.78 L2195.02 1114.05 L2195.24 1113.55 L2195.47 1113.28 L2195.7 1113.14 L2195.93 1113.33 L2196.15 1113.32 L2196.38 1113.22 L2196.61 1113.47 L2196.84 1113.82 L2197.07 1113.52 L2197.29 1113.91 L2197.52 1114.44 L2197.75 1114.55 L2197.98 1114.58 L2198.2 1114.36 L2198.43 1114.47 L2198.66 1114.51 L2198.89 1114.28 L2199.12 1114.39 L2199.34 1114.73 L2199.57 1115.08 L2199.8 1115.05 L2200.03 1115.02 L2200.25 1114.72 L2200.48 1114.78 L2200.71 1114.75 L2200.94 1114.69 L2201.17 1115.04 L2201.39 1115.3 L2201.62 1115.71 L2201.85 1115.3 L2202.08 1115.48 L2202.3 1115.8 L2202.53 1115.59 L2202.76 1115.8 L2202.99 1115.41 L2203.21 1115.33 L2203.44 1114.89 L2203.67 1114.9 L2203.9 1114.99 L2204.13 1115.04 L2204.35 1115.22 L2204.58 1115.03 L2204.81 1115 L2205.04 1114.92 L2205.26 1114.47 L2205.49 1114.57 L2205.72 1114.15 L2205.95 1114.42 L2206.18 1114.68 L2206.4 1114.94 L2206.63 1114.99 L2206.86 1114.84 L2207.09 1115.03 L2207.31 1114.97 L2207.54 1115.21 L2207.77 1115.36 L2208 1115 L2208.23 1114.72 L2208.45 1114.48 L2208.68 1114.63 L2208.91 1114.28 L2209.14 1114.62 L2209.36 1114.8 L2209.59 1114.66 L2209.82 1114.41 L2210.05 1114.71 L2210.28 1114.41 L2210.5 1114.23 L2210.73 1114.39 L2210.96 1114.41 L2211.19 1114.45 L2211.41 1114.79 L2211.64 1114.49 L2211.87 1114.75 L2212.1 1114.28 L2212.33 1114.11 L2212.55 1113.94 L2212.78 1113.35 L2213.01 1113.58 L2213.24 1113.24 L2213.46 1112.77 L2213.69 1112.55 L2213.92 1112.86 L2214.15 1113.33 L2214.38 1113.33 L2214.6 1112.78 L2214.83 1112.81 L2215.06 1112.98 L2215.29 1112.74 L2215.51 1112.99 L2215.74 1112.43 L2215.97 1112.18 L2216.2 1112.25 L2216.43 1112.4 L2216.65 1112.25 L2216.88 1112.25 L2217.11 1112.34 L2217.34 1112.2 L2217.56 1112.19 L2217.79 1111.89 L2218.02 1111.8 L2218.25 1111.87 L2218.48 1111.91 L2218.7 1111.65 L2218.93 1111.88 L2219.16 1112 L2219.39 1112.28 L2219.61 1112.23 L2219.84 1111.91 L2220.07 1112.08 L2220.3 1112.13 L2220.53 1112.39 L2220.75 1112.43 L2220.98 1112.29 L2221.21 1112.31 L2221.44 1112.09 L2221.66 1111.64 L2221.89 1112.02 L2222.12 1112.01 L2222.35 1111.97 L2222.58 1112.18 L2222.8 1111.34 L2223.03 1111.67 L2223.26 1111.94 L2223.49 1111.78 L2223.71 1111.25 L2223.94 1111.49 L2224.17 1111.55 L2224.4 1111.84 L2224.63 1111.72 L2224.85 1111.99 L2225.08 1111.37 L2225.31 1111.42 L2225.54 1111.4 L2225.76 1111.3 L2225.99 1111.08 L2226.22 1111.37 L2226.45 1111.64 L2226.68 1111.63 L2226.9 1111.44 L2227.13 1111.77 L2227.36 1112.32 L2227.59 1112.51 L2227.81 1112.27 L2228.04 1112.31 L2228.27 1112.45 L2228.5 1112.86 L2228.73 1112.82 L2228.95 1113.01 L2229.18 1113.41 L2229.41 1113.41 L2229.64 1113.2 L2229.86 1113.37 L2230.09 1113.5 L2230.32 1113.62 L2230.55 1113.83 L2230.78 1113.64 L2231 1112.99 L2231.23 1113.11 L2231.46 1113.03 L2231.69 1112.99 L2231.91 1112.71 L2232.14 1113.25 L2232.37 1113.56 L2232.6 1113.33 L2232.83 1113.72 L2233.05 1113.45 L2233.28 1113.34 L2233.51 1113.14 L2233.74 1112.69 L2233.96 1112.69 L2234.19 1112.71 L2234.42 1113 L2234.65 1113.1 L2234.88 1113.59 L2235.1 1113.63 L2235.33 1113.23 L2235.56 1113.45 L2235.79 1113.62 L2236.01 1113.66 L2236.24 1113.52 L2236.47 1113.49 L2236.7 1113.27 L2236.93 1113.62 L2237.15 1113.72 L2237.38 1113.74 L2237.61 1113.46 L2237.84 1113.5 L2238.06 1113.25 L2238.29 1113.34 L2238.52 1113.52 L2238.75 1113.55 L2238.98 1113.68 L2239.2 1113.73 L2239.43 1113.41 L2239.66 1113.58 L2239.89 1114.05 L2240.11 1113.78 L2240.34 1113.98 L2240.57 1113.86 L2240.8 1113.86 L2241.02 1114.28 L2241.25 1114.56 L2241.48 1115.11 L2241.71 1115.32 L2241.94 1115.78 L2242.16 1116.02 L2242.39 1116.33 L2242.62 1116.59 L2242.85 1116.43 L2243.07 1116.24 L2243.3 1116.59 L2243.53 1116.7 L2243.76 1116.44 L2243.99 1116.89 L2244.21 1117.34 L2244.44 1117.81 L2244.67 1117.91 L2244.9 1118.14 L2245.12 1118.14 L2245.35 1117.94 L2245.58 1118.21 L2245.81 1118.55 L2246.04 1118.73 L2246.26 1118.18 L2246.49 1117.93 L2246.72 1118.27 L2246.95 1118.16 L2247.17 1117.83 L2247.4 1117.89 L2247.63 1117.17 L2247.86 1117.08 L2248.09 1117.08 L2248.31 1117.15 L2248.54 1117.55 L2248.77 1117.2 L2249 1117.47 L2249.22 1117.57 L2249.45 1117.63 L2249.68 1117.91 L2249.91 1118.2 L2250.14 1118.06 L2250.36 1117.8 L2250.59 1117.38 L2250.82 1117.37 L2251.05 1117.14 L2251.27 1117.32 L2251.5 1117.15 L2251.73 1117.25 L2251.96 1117.76 L2252.19 1117.96 L2252.41 1118.47 L2252.64 1118.18 L2252.87 1117.88 L2253.1 1117.8 L2253.32 1117.67 L2253.55 1117.54 L2253.78 1117.05 L2254.01 1117.25 L2254.24 1117.18 L2254.46 1117.44 L2254.69 1117.95 L2254.92 1118.44 L2255.15 1118.3 L2255.37 1118.41 L2255.6 1117.44 L2255.83 1117.6 L2256.06 1117.62 L2256.29 1116.96 L2256.51 1116.97 L2256.74 1117.04 L2256.97 1117.26 L2257.2 1117.93 L2257.42 1117.49 L2257.65 1117.63 L2257.88 1117.53 L2258.11 1117.77 L2258.34 1117.78 L2258.56 1117.32 L2258.79 1117.15 L2259.02 1117.11 L2259.25 1117.34 L2259.47 1117.43 L2259.7 1117.96 L2259.93 1117.55 L2260.16 1118.26 L2260.39 1117.95 L2260.61 1118.63 L2260.84 1118.44 L2261.07 1118.93 L2261.3 1118.71 L2261.52 1118.69 L2261.75 1118.37 L2261.98 1118.11 L2262.21 1118.47 L2262.44 1118.4 L2262.66 1118.62 L2262.89 1119.04 L2263.12 1118.81 L2263.35 1119.27 L2263.57 1119.21 L2263.8 1118.84 L2264.03 1119.23 L2264.26 1119.16 L2264.49 1119.47 L2264.71 1119.43 L2264.94 1119.39 L2265.17 1118.52 L2265.4 1118.75 L2265.62 1118.7 L2265.85 1118.78 L2266.08 1118.61 L2266.31 1118.71 L2266.54 1118.76 L2266.76 1118.28 L2266.99 1117.67 L2267.22 1117.34 L2267.45 1117.26 L2267.67 1117.2 L2267.9 1117.77 L2268.13 1118.28 L2268.36 1118.84 L2268.59 1118.95 L2268.81 1119.22 L2269.04 1119.06 L2269.27 1118.95 L2269.5 1119 L2269.72 1119.08 L2269.95 1118.92 L2270.18 1118.79 L2270.41 1118.91 L2270.64 1119 L2270.86 1118.69 L2271.09 1119.21 L2271.32 1119.17 L2271.55 1119.02 L2271.77 1118.39 L2272 1118.33 L2272.23 1117.7 L2272.46 1117.54 L2272.69 1117.16 L2272.91 1117.48 L2273.14 1117.73 L2273.37 1117.83 L2273.6 1118.21 L2273.82 1117.92 L2274.05 1117.56 L2274.28 1117.4 L2274.51 1117.37 L2274.74 1117.52 L2274.96 1117.75 L2275.19 1117.33 L2275.42 1117 L2275.65 1116.73 L2275.87 1116.64 L2276.1 1116.55 L2276.33 1117.12 L2276.56 1117.2 L2276.78 1117.33 L2277.01 1117.16 L2277.24 1117.39 L2277.47 1117.53 L2277.7 1117.84 L2277.92 1117.98 L2278.15 1118.1 L2278.38 1118.08 L2278.61 1118.32 L2278.83 1118.72 L2279.06 1119.15 L2279.29 1118.84 L2279.52 1118.96 L2279.75 1119.06 L2279.97 1118.32 L2280.2 1118.88 L2280.43 1118.83 L2280.66 1119.06 L2280.88 1118.69 L2281.11 1118.71 L2281.34 1119.02 L2281.57 1118.99 L2281.8 1119.08 L2282.02 1119.35 L2282.25 1119.52 L2282.48 1119.71 L2282.71 1119.44 L2282.93 1119.52 L2283.16 1119.24 L2283.39 1119.03 L2283.62 1119.37 L2283.85 1119.39 L2284.07 1119.55 L2284.3 1119.66 L2284.53 1119.99 L2284.76 1119.41 L2284.98 1119.41 L2285.21 1119.33 L2285.44 1118.9 L2285.67 1119.14 L2285.9 1119.24 L2286.12 1119.04 L2286.35 1118.71 L2286.58 1118.83 L2286.81 1118.98 L2287.03 1119.03 L2287.26 1118.99 L2287.49 1119.01 L2287.72 1118.48 L2287.95 1118.55 L2288.17 1118.3 L2288.4 1118.28 L2288.63 1117.64 L2288.86 1117.89 L2289.08 1117.75 L2289.31 1117.84 L2289.54 1117.94 L2289.77 1117.87 L2290 1117.89 L2290.22 1117.83 L2290.45 1117.85 L2290.68 1117.75 L2290.91 1117.11 L2291.13 1117.24 L2291.36 1117.05 L2291.59 1116.68 L2291.82 1116.41 L2292.05 1116.16 L2292.27 1116.58 L2292.5 1116.13 L2292.73 1116.36 L2292.96 1116.32 L2293.18 1116.19 L2293.41 1116.44 L2293.64 1116.64 L2293.87 1117 L2294.1 1117.19 L2294.32 1117.22 L2294.55 1117.35 L2294.78 1117.45 L2295.01 1116.79 L2295.23 1116.89 L2295.46 1116.86 L2295.69 1116.6 L2295.92 1116.71 L2296.15 1116.74 L2296.37 1116.61 L2296.6 1116.85 L2296.83 1117.47 L2297.06 1117.8 L2297.28 1117.77 L2297.51 1117.9 L2297.74 1118.32 L2297.97 1118.16 L2298.2 1118.3 L2298.42 1118.18 L2298.65 1118.01 L2298.88 1117.86 L2299.11 1118.19 L2299.33 1118.54 L2299.56 1118.7 L2299.79 1118.75 L2300.02 1119.04 L2300.25 1119.27 L2300.47 1119.27 L2300.7 1119.24 L2300.93 1118.98 L2301.16 1118.75 L2301.38 1118.5 L2301.61 1118.19 L2301.84 1118.08 L2302.07 1118.15 L2302.3 1118.01 L2302.52 1118.2 L2302.75 1118.47 L2302.98 1118.8 L2303.21 1118.95 L2303.43 1118.63 L2303.66 1118.44 L2303.89 1118.53 L2304.12 1118.8 L2304.35 1118.28 L2304.57 1118.21 L2304.8 1118.67 L2305.03 1118.75 L2305.26 1117.95 L2305.48 1117.86 L2305.71 1117.82 L2305.94 1118.06 L2306.17 1118.09 L2306.4 1118.28 L2306.62 1118.83 L2306.85 1119.06 L2307.08 1118.74 L2307.31 1118.76 L2307.53 1119.25 L2307.76 1119.5 L2307.99 1119.28 L2308.22 1119.59 L2308.45 1119.91 L2308.67 1119.72 L2308.9 1119.79 L2309.13 1120.03 L2309.36 1120.35 L2309.58 1120.38 L2309.81 1120.97 L2310.04 1121.22 L2310.27 1121.24 L2310.5 1121.37 L2310.72 1120.92 L2310.95 1120.92 L2311.18 1121.31 L2311.41 1121.59 L2311.63 1121.53 L2311.86 1121.65 L2312.09 1121.82 L2312.32 1121.92 L2312.54 1122.01 L2312.77 1122.24 L2313 1122.1 L2313.23 1122.12 L2313.46 1122.75 L2313.68 1122.24 L2313.91 1122.11 L2314.14 1122.23 L2314.37 1122.09 L2314.59 1121.99 L2314.82 1122.08 L2315.05 1121.66 L2315.28 1122.06 L2315.51 1121.97 L2315.73 1122.09 L2315.96 1121.96 L2316.19 1122.13 L2316.42 1122.36 L2316.64 1122.58 L2316.87 1122.66 L2317.1 1123.26 L2317.33 1123.67 L2317.56 1123.59 L2317.78 1123.55 L2318.01 1123.37 L2318.24 1123.58 L2318.47 1123.65 L2318.69 1123.2 L2318.92 1123.19 L2319.15 1123.75 L2319.38 1124.01 L2319.61 1124.24 L2319.83 1124.55 L2320.06 1124.98 L2320.29 1124.49 L2320.52 1124.35 L2320.74 1124.55 L2320.97 1124.47 L2321.2 1124.18 L2321.43 1123.91 L2321.66 1123.19 L2321.88 1123.02 L2322.11 1123.32 L2322.34 1123.17 L2322.57 1122.95 L2322.79 1122.56 L2323.02 1122.6 L2323.25 1122.82 L2323.48 1122.77 L2323.71 1122.98 L2323.93 1123.32 L2324.16 1123.4 L2324.39 1123.2 L2324.62 1123.44 L2324.84 1123.48 L2325.07 1123.69 L2325.3 1123.47 L2325.53 1123.39 L2325.76 1123.16 L2325.98 1123.16 L2326.21 1123.28 L2326.44 1123.18 L2326.67 1123.46 L2326.89 1123.6 L2327.12 1123.71 L2327.35 1123.5 L2327.58 1123.26 L2327.81 1123.6 L2328.03 1123.86 L2328.26 1123.69 L2328.49 1124.04 L2328.72 1124.44 L2328.94 1124.45 L2329.17 1124.81 L2329.4 1124.13 L2329.63 1123.01 L2329.86 1123.08 L2330.08 1123.18 L2330.31 1123.2 L2330.54 1122.66 L2330.77 1122.46 L2330.99 1122.29 L2331.22 1122 L2331.45 1122.06 L2331.68 1121.78 L2331.91 1121.76 L2332.13 1121.88 L2332.36 1121.64 L2332.59 1121.81 L2332.82 1121.8 L2333.04 1121.48 L2333.27 1121.23 L2333.5 1121.19 L2333.73 1121.69 L2333.96 1121.84 L2334.18 1122.15 L2334.41 1122.27 L2334.64 1121.68 L2334.87 1121.82 L2335.09 1121.8 L2335.32 1121.66 L2335.55 1122.04 L2335.78 1121.92 L2336.01 1121.85 L2336.23 1121.45 L2336.46 1121.57 L2336.69 1121.07 L2336.92 1121.02 L2337.14 1121.37 L2337.37 1121.26 L2337.6 1121.3 L2337.83 1121.24 L2338.06 1121.29 L2338.28 1121.75 L2338.51 1121.38 L2338.74 1121.55 L2338.97 1121.34 L2339.19 1121.19 L2339.42 1120.93 L2339.65 1121 L2339.88 1120.64 L2340.11 1120.93 L2340.33 1121.1 L2340.56 1120.79 L2340.79 1121.24 L2341.02 1121.71 L2341.24 1121.42 L2341.47 1121.24 L2341.7 1121.66 L2341.93 1121.51 L2342.16 1121.78 L2342.38 1121.76 L2342.61 1122 L2342.84 1122.22 L2343.07 1122.38 L2343.29 1122.16 L2343.52 1121.63 L2343.75 1121.69 L2343.98 1120.67 L2344.21 1120.15 L2344.43 1120.01 L2344.66 1119.94 L2344.89 1120.03 L2345.12 1120.3 L2345.34 1120.57 L2345.57 1120.36 L2345.8 1120.57 L2346.03 1120.37 L2346.26 1120.56 L2346.48 1120.17 L2346.71 1120.24 L2346.94 1120.15 L2347.17 1120.18 L2347.39 1120.03 L2347.62 1120.23 L2347.85 1119.91 L2348.08 1119.88 L2348.31 1119.92 L2348.53 1119.89 L2348.76 1120.15 L2348.99 1119.97 L2349.22 1120.47 L2349.44 1120.47 L2349.67 1120.5 L2349.9 1120.55 L2350.13 1120.26 L2350.35 1120.15 L2350.58 1119.93 L2350.81 1120.29 L2351.04 1120.36 L2351.27 1120.49 L2351.49 1120.23 L2351.72 1120.09 L2351.95 1119.59 L2352.18 1119.57 L2352.4 1119.94 L2352.63 1120.22 L2352.86 1120.6 L2353.09 1120.47 L2353.32 1120.67 L2353.54 1120.83 L2353.77 1121.12 L2354 1121.22 L2354.23 1121.16 L2354.45 1121.1 L2354.68 1120.63 L2354.91 1120.9 L2355.14 1120.96 L2355.37 1120.84 L2355.59 1120.78 L2355.82 1120.36 L2356.05 1120.69 L2356.28 1120.83 L2356.5 1121.07 L2356.73 1121.1 L2356.96 1121.14 L2357.19 1121.28 L2357.42 1120.94 L2357.64 1120.76 L2357.87 1121.49 L2358.1 1122.06 L2358.33 1122.18 L2358.55 1121.81 L2358.78 1121.54 L2359.01 1121.8 L2359.24 1121.61 L2359.47 1121.3 L2359.69 1121.24 L2359.92 1121.51 L2360.15 1121.54 L2360.38 1121.5 L2360.6 1121.21 L2360.83 1121.58 L2361.06 1121.52 L2361.29 1121.78 L2361.52 1121.52 L2361.74 1121.58 L2361.97 1121.42 L2362.2 1121.9 L2362.43 1122.02 L2362.65 1122.04 L2362.88 1121.88 L2363.11 1122.1 L2363.34 1122.27 L2363.57 1122.59 L2363.79 1122.96 L2364.02 1123.04 L2364.25 1122.91 L2364.48 1123.16 L2364.7 1123.39 L2364.93 1123.57 L2365.16 1123.51 L2365.39 1123.45 L2365.62 1123.55 L2365.84 1123.43 L2366.07 1123.13 L2366.3 1123.19 L2366.53 1123.07 L2366.75 1122.55 L2366.98 1122.61 L2367.21 1122.62 L2367.44 1122.7 L2367.67 1123.04 L2367.89 1122.91 L2368.12 1122.84 L2368.35 1122.85 L2368.58 1122.77 L2368.8 1122.78 L2369.03 1123.12 L2369.26 1123.09 L2369.49 1123.47 L2369.72 1123.37 L2369.94 1123 L2370.17 1122.64 L2370.4 1122.6 L2370.63 1122.41 L2370.85 1122.03 L2371.08 1121.34 L2371.31 1121.69 L2371.54 1121.86 L2371.77 1121.61 L2371.99 1121.63 L2372.22 1121.59 L2372.45 1121.53 L2372.68 1121.62 L2372.9 1121.8 L2373.13 1121.85 L2373.36 1122.27 L2373.59 1121.98 L2373.82 1122.16 L2374.04 1122 L2374.27 1122.01 L2374.5 1122.24 L2374.73 1122.42 L2374.95 1122.52 L2375.18 1122.36 L2375.41 1122.73 L2375.64 1122.75 L2375.87 1122.48 L2376.09 1122.57 L2376.32 1122.9 L2376.55 1122.97 L2376.78 1123.09 L2377 1123.32 L2377.23 1123.23 L2377.46 1123.17 L2377.69 1122.96 L2377.92 1122.79 L2378.14 1122.55 L2378.37 1122.34 L2378.6 1122.57 L2378.83 1122.91 L2379.05 1122.76 L2379.28 1122.66 L2379.51 1122.32 L2379.74 1122.91 L2379.97 1123.22 L2380.19 1123.45 L2380.42 1123.87 L2380.65 1123.93 L2380.88 1124.18 L2381.1 1124.12 L2381.33 1124.56 L2381.56 1125.14 L2381.79 1125.67 L2382.02 1125.79 L2382.24 1125.82 L2382.47 1126.01 L2382.7 1125.39 L2382.93 1124.57 L2383.15 1124.83 L2383.38 1124.76 L2383.61 1125.03 L2383.84 1125.4 L2384.07 1125.54 L2384.29 1125.58 L2384.52 1125.8 L2384.75 1126.15 L2384.98 1125.98 L2385.2 1125.7 L2385.43 1125.9 L2385.66 1126.11 L2385.89 1126.18 L2386.11 1126.6 L2386.34 1126.75 L2386.57 1127 L2386.8 1127.36 L2387.03 1127.17 L2387.25 1127.34 L2387.48 1127.28 L2387.71 1127.17 L2387.94 1127.57 L2388.16 1127.91 L2388.39 1127.96 L2388.62 1127.75 L2388.85 1127.97 L2389.08 1128.19 L2389.3 1127.98 L2389.53 1127.87 L2389.76 1127.88 L2389.99 1127.72 L2390.21 1127.37 L2390.44 1127.06 L2390.67 1127.09 L2390.9 1127.36 L2391.13 1127.34 L2391.35 1127.38 L2391.58 1126.77 L2391.81 1126.81 L2392.04 1126.81 L2392.26 1126.64 L2392.49 1126.58 L2392.72 1126.85 L2392.95 1126.9 L2393.18 1126.48 L2393.4 1126.69 L2393.63 1126.87 L2393.86 1127.07 L2394.09 1126.44 L2394.31 1126.43 L2394.54 1126.56 L2394.77 1126.45 L2395 1126.51 L2395.23 1126.3 L2395.45 1126.47 L2395.68 1126.52 L2395.91 1126.34 L2396.14 1126.41 L2396.36 1126.33 L2396.59 1126.43 L2396.82 1126.43 L2397.05 1126.73 L2397.28 1126.96 L2397.5 1126.53 L2397.73 1125.68 L2397.96 1124.99 L2398.19 1125.2 L2398.41 1125.35 L2398.64 1125.67 L2398.87 1125.76 L2399.1 1125.67 L2399.33 1125.12 L2399.55 1125.27 L2399.78 1124.8 L2400.01 1125.2 L2400.24 1125.31 L2400.46 1125.57 L2400.69 1125.85 L2400.92 1125.53 L2401.15 1125.71 L2401.38 1126.1 L2401.6 1126.36 L2401.83 1125.91 L2402.06 1125.86 L2402.29 1125.64 L2402.51 1125.59 L2402.74 1125.27 L2402.97 1125.6 L2403.2 1125.84 L2403.43 1126.15 L2403.65 1126.07 L2403.88 1126.02 L2404.11 1126.16 L2404.34 1126.05 L2404.56 1126.03 L2404.79 1125.98 L2405.02 1125.38 L2405.25 1125.37 L2405.48 1125.47 L2405.7 1125.81 L2405.93 1126 L2406.16 1126.11 L2406.39 1125.53 L2406.61 1125.89 L2406.84 1125.82 L2407.07 1125.87 L2407.3 1126.14 L2407.53 1126.49 L2407.75 1126.83 L2407.98 1127.1 L2408.21 1126.79 L2408.44 1126.35 L2408.66 1126.16 L2408.89 1126.21 L2409.12 1126.45 L2409.35 1127.09 L2409.58 1126.93 L2409.8 1126.1 L2410.03 1126.36 L2410.26 1126.32 L2410.49 1126.32 L2410.71 1126.47 L2410.94 1126.11 L2411.17 1125.78 L2411.4 1125.45 L2411.63 1125.4 L2411.85 1125.08 L2412.08 1124.65 L2412.31 1124.98 L2412.54 1124.72 L2412.76 1124.62 L2412.99 1124.15 L2413.22 1123.89 L2413.45 1124.08 L2413.68 1124.15 L2413.9 1124.37 L2414.13 1124.76 L2414.36 1124.88 L2414.59 1124.97 L2414.81 1125.19 L2415.04 1125.15 L2415.27 1125.21 L2415.5 1125.29 L2415.73 1125.59 L2415.95 1125.21 L2416.18 1125.39 L2416.41 1125.54 L2416.64 1125.52 L2416.86 1125.6 L2417.09 1125.79 L2417.32 1125.73 L2417.55 1125.58 L2417.78 1125.46 L2418 1125.33 L2418.23 1124.75 L2418.46 1124.54 L2418.69 1124.81 L2418.91 1124.66 L2419.14 1124.59 L2419.37 1124.82 L2419.6 1124.81 L2419.83 1125.05 L2420.05 1125.08 L2420.28 1124.93 L2420.51 1124.82 L2420.74 1124.86 L2420.96 1124.99 L2421.19 1124.73 L2421.42 1125.19 L2421.65 1125.41 L2421.87 1125.67 L2422.1 1125.9 L2422.33 1125.54 L2422.56 1125.88 L2422.79 1125.6 L2423.01 1125.8 L2423.24 1125.59 L2423.47 1125.37 L2423.7 1125.84 L2423.92 1125.86 L2424.15 1126.12 L2424.38 1126.28 L2424.61 1126.66 L2424.84 1127.18 L2425.06 1127.18 L2425.29 1127.31 L2425.52 1127.07 L2425.75 1127.03 L2425.97 1127.18 L2426.2 1127.02 L2426.43 1127.22 L2426.66 1127.3 L2426.89 1127.47 L2427.11 1127.22 L2427.34 1127.3 L2427.57 1127.54 L2427.8 1127.7 L2428.02 1127.99 L2428.25 1128.48 L2428.48 1128.49 L2428.71 1128.24 L2428.94 1128.47 L2429.16 1128.79 L2429.39 1129.24 L2429.62 1129 L2429.85 1129.12 L2430.07 1129.15 L2430.3 1128.72 L2430.53 1128.77 L2430.76 1128.83 L2430.99 1129.21 L2431.21 1129.63 L2431.44 1129.72 L2431.67 1130.15 L2431.9 1130.02 L2432.12 1130.49 L2432.35 1130.38 L2432.58 1130.38 L2432.81 1130.01 L2433.04 1130.02 L2433.26 1130.27 L2433.49 1130.42 L2433.72 1130.66 L2433.95 1130.68 L2434.17 1130.59 L2434.4 1131.01 L2434.63 1130.88 L2434.86 1131.12 L2435.09 1131.08 L2435.31 1130.74 L2435.54 1131 L2435.77 1131.23 L2436 1130.99 L2436.22 1131.26 L2436.45 1130.9 L2436.68 1130.79 L2436.91 1130.74 L2437.14 1130.45 L2437.36 1130.47 L2437.59 1130.85 L2437.82 1130.73 L2438.05 1130.8 L2438.27 1130.82 L2438.5 1130.41 L2438.73 1130.65 L2438.96 1130.61 L2439.19 1130.92 L2439.41 1130.99 L2439.64 1131.05 L2439.87 1129.93 L2440.1 1129.97 L2440.32 1129.7 L2440.55 1129.17 L2440.78 1128.41 L2441.01 1128.4 L2441.24 1128.31 L2441.46 1128.03 L2441.69 1127.56 L2441.92 1127.44 L2442.15 1127.72 L2442.37 1127.87 L2442.6 1127.72 L2442.83 1126.98 L2443.06 1126.98 L2443.29 1127.13 L2443.51 1127.35 L2443.74 1126.98 L2443.97 1127.05 L2444.2 1127.3 L2444.42 1127.6 L2444.65 1127.38 L2444.88 1127.56 L2445.11 1127.57 L2445.34 1127.22 L2445.56 1127.53 L2445.79 1127.72 L2446.02 1127.53 L2446.25 1127.61 L2446.47 1127.49 L2446.7 1127.4 L2446.93 1127.39 L2447.16 1127.69 L2447.39 1127.51 L2447.61 1127.45 L2447.84 1127.92 L2448.07 1128.13 L2448.3 1127.79 L2448.52 1127.74 L2448.75 1127.77 L2448.98 1127.89 L2449.21 1127.61 L2449.44 1127.74 L2449.66 1127.73 L2449.89 1127.99 L2450.12 1127.58 L2450.35 1127.4 L2450.57 1127.21 L2450.8 1127.12 L2451.03 1127.13 L2451.26 1127 L2451.49 1126.97 L2451.71 1127.04 L2451.94 1127.09 L2452.17 1127.52 L2452.4 1127.72 L2452.62 1128.01 L2452.85 1128.2 L2453.08 1127.83 L2453.31 1127.35 L2453.54 1127.4 L2453.76 1127.46 L2453.99 1127.38 L2454.22 1126.96 L2454.45 1127.12 L2454.67 1127.3 L2454.9 1127.28 L2455.13 1127.03 L2455.36 1127.28 L2455.59 1127.7 L2455.81 1128.13 L2456.04 1128.22 L2456.27 1127.82 L2456.5 1127.9 L2456.72 1127.84 L2456.95 1128.12 L2457.18 1128.08 L2457.41 1128.31 L2457.64 1128.62 L2457.86 1128.82 L2458.09 1129.13 L2458.32 1129.4 L2458.55 1129.49 L2458.77 1128.98 L2459 1128.92 L2459.23 1129.05 L2459.46 1129.24 L2459.68 1129.63 L2459.91 1129.85 L2460.14 1129.65 L2460.37 1129.44 L2460.6 1129.15 L2460.82 1128.92 L2461.05 1129.22 L2461.28 1129.38 L2461.51 1128.85 L2461.73 1128.67 L2461.96 1129.02 L2462.19 1128.75 L2462.42 1128.33 L2462.65 1128.25 L2462.87 1127.42 L2463.1 1127.84 L2463.33 1127.93 L2463.56 1128.39 L2463.78 1128.45 L2464.01 1128.34 L2464.24 1128.26 L2464.47 1128.08 L2464.7 1127.91 L2464.92 1127.96 L2465.15 1127.88 L2465.38 1127.62 L2465.61 1127.61 L2465.83 1127.26 L2466.06 1127.34 L2466.29 1127.35 L2466.52 1127.73 L2466.75 1127.75 L2466.97 1127.29 L2467.2 1127.37 L2467.43 1127.08 L2467.66 1127.09 L2467.88 1126.61 L2468.11 1126.4 L2468.34 1126.4 L2468.57 1125.97 L2468.8 1125.74 L2469.02 1126.17 L2469.25 1126.36 L2469.48 1126.25 L2469.71 1126.3 L2469.93 1126.56 L2470.16 1126.86 L2470.39 1127.05 L2470.62 1127.17 L2470.85 1127.54 L2471.07 1126.84 L2471.3 1126.28 L2471.53 1126.13 L2471.76 1125.92 L2471.98 1125.98 L2472.21 1125.99 L2472.44 1126.43 L2472.67 1125.77 L2472.9 1125.69 L2473.12 1125.37 L2473.35 1125.52 L2473.58 1125.31 L2473.81 1126.02 L2474.03 1125.77 L2474.26 1125.64 L2474.49 1124.89 L2474.72 1124.81 L2474.95 1124.53 L2475.17 1124.69 L2475.4 1124.79 L2475.63 1125.13 L2475.86 1125.21 L2476.08 1125.59 L2476.31 1125.44 L2476.54 1125.67 L2476.77 1125.95 L2477 1126.31 L2477.22 1126.17 L2477.45 1126.02 L2477.68 1126.01 L2477.91 1126.07 L2478.13 1126.56 L2478.36 1126.34 L2478.59 1126.58 L2478.82 1126.96 L2479.05 1127.29 L2479.27 1127.35 L2479.5 1127.62 L2479.73 1127.84 L2479.96 1127.77 L2480.18 1127.51 L2480.41 1127.42 L2480.64 1127.44 L2480.87 1126.88 L2481.1 1127.08 L2481.32 1126.57 L2481.55 1126.35 L2481.78 1125.89 L2482.01 1125.93 L2482.23 1126.51 L2482.46 1126.28 L2482.69 1126.56 L2482.92 1126.92 L2483.15 1126.53 L2483.37 1126.7 L2483.6 1126.75 L2483.83 1126.57 L2484.06 1126.6 L2484.28 1126.99 L2484.51 1126.92 L2484.74 1126.49 L2484.97 1126.38 L2485.2 1126.42 L2485.42 1126.41 L2485.65 1126.8 L2485.88 1127.13 L2486.11 1126.87 L2486.33 1126.74 L2486.56 1126.83 L2486.79 1126.67 L2487.02 1125.89 L2487.25 1126.41 L2487.47 1126.51 L2487.7 1126.37 L2487.93 1126.16 L2488.16 1125.96 L2488.38 1125.74 L2488.61 1125.86 L2488.84 1125.65 L2489.07 1126.01 L2489.3 1125.67 L2489.52 1125.81 L2489.75 1126.29 L2489.98 1126.25 L2490.21 1126.2 L2490.43 1126.36 L2490.66 1126.74 L2490.89 1127.02 L2491.12 1126.81 L2491.35 1127.25 L2491.57 1127.23 L2491.8 1127.18 L2492.03 1127.39 L2492.26 1127.35 L2492.48 1127.41 L2492.71 1126.92 L2492.94 1126.86 L2493.17 1126.83 L2493.4 1126.47 L2493.62 1126.41 L2493.85 1126.37 L2494.08 1126.57 L2494.31 1126.51 L2494.53 1126.06 L2494.76 1125.64 L2494.99 1125.58 L2495.22 1125.53 L2495.44 1125.77 L2495.67 1125.52 L2495.9 1125.8 L2496.13 1126.02 L2496.36 1126.39 L2496.58 1126.48 L2496.81 1126.56 L2497.04 1126.6 L2497.27 1126.33 L2497.49 1126.32 L2497.72 1125.68 L2497.95 1125.64 L2498.18 1125.31 L2498.41 1124.61 L2498.63 1124.37 L2498.86 1124.68 L2499.09 1124.61 L2499.32 1124.52 L2499.54 1124.64 L2499.77 1124.46 L2500 1124.6 L2500.23 1124.16 L2500.46 1124.2 L2500.68 1124.25 L2500.91 1124.39 L2501.14 1124.71 L2501.37 1124.08 L2501.59 1123.57 L2501.82 1123.91 L2502.05 1124.11 L2502.28 1123.99 L2502.51 1124.24 L2502.73 1124.61 L2502.96 1124.72 L2503.19 1124.96 L2503.42 1125.16 L2503.64 1125.03 L2503.87 1125.34 L2504.1 1125.8 L2504.33 1125.86 L2504.56 1125.85 L2504.78 1125.92 L2505.01 1126.09 L2505.24 1126.72 L2505.47 1126.79 L2505.69 1127.16 L2505.92 1127.6 L2506.15 1127.18 L2506.38 1126.71 L2506.61 1126.98 L2506.83 1127.26 L2507.06 1127.13 L2507.29 1127.01 L2507.52 1126.51 L2507.74 1126.18 L2507.97 1125.6 L2508.2 1125.49 L2508.43 1126.14 L2508.66 1126.5 L2508.88 1125.56 L2509.11 1125.41 L2509.34 1125.5 L2509.57 1125.48 L2509.79 1125.52 L2510.02 1125.62 L2510.25 1125.82 L2510.48 1125.89 L2510.71 1126.18 L2510.93 1125.69 L2511.16 1125.69 L2511.39 1125.77 L2511.62 1125.82 L2511.84 1126.2 L2512.07 1126.56 L2512.3 1126.35 L2512.53 1126.33 L2512.76 1126.75 L2512.98 1126.95 L2513.21 1126.86 L2513.44 1126.83 L2513.67 1126.55 L2513.89 1126.12 L2514.12 1126.27 L2514.35 1126.41 L2514.58 1126.37 L2514.81 1126.09 L2515.03 1125.46 L2515.26 1125.38 L2515.49 1125.72 L2515.72 1125.12 L2515.94 1125.13 L2516.17 1125.31 L2516.4 1125.18 L2516.63 1125.35 L2516.86 1125.46 L2517.08 1125.63 L2517.31 1125.7 L2517.54 1125.72 L2517.77 1125.53 L2517.99 1125.53 L2518.22 1125.61 L2518.45 1125.76 L2518.68 1125.79 L2518.91 1125.83 L2519.13 1125.6 L2519.36 1125.75 L2519.59 1125.92 L2519.82 1126.39 L2520.04 1126.74 L2520.27 1126.23 L2520.5 1126.75 L2520.73 1126.91 L2520.96 1126.7 L2521.18 1126.93 L2521.41 1126.96 L2521.64 1127.03 L2521.87 1126.96 L2522.09 1126.92 L2522.32 1126.91 L2522.55 1126.65 L2522.78 1126.47 L2523.01 1126.3 L2523.23 1126.11 L2523.46 1126.44 L2523.69 1126.13 L2523.92 1126.12 L2524.14 1126.17 L2524.37 1125.86 L2524.6 1125.93 L2524.83 1125.39 L2525.06 1125.6 L2525.28 1126.02 L2525.51 1125.98 L2525.74 1125.72 L2525.97 1125.99 L2526.19 1125.84 L2526.42 1125.86 L2526.65 1126.17 L2526.88 1125.96 L2527.11 1126.37 L2527.33 1126.32 L2527.56 1126.52 L2527.79 1126.14 L2528.02 1125.97 L2528.24 1125.77 L2528.47 1125.77 L2528.7 1126.03 L2528.93 1126.26 L2529.16 1125.44 L2529.38 1125.58 L2529.61 1125.68 L2529.84 1125.69 L2530.07 1125.11 L2530.29 1125.21 L2530.52 1125.68 L2530.75 1125.41 L2530.98 1125.39 L2531.2 1125.38 L2531.43 1125.01 L2531.66 1124.96 L2531.89 1125.06 L2532.12 1125.4 L2532.34 1125.53 L2532.57 1124.88 L2532.8 1125.06 L2533.03 1125.22 L2533.25 1124.83 L2533.48 1125.27 L2533.71 1125.14 L2533.94 1125.44 L2534.17 1125.97 L2534.39 1125.8 L2534.62 1126.15 L2534.85 1126.09 L2535.08 1125.89 L2535.3 1125.86 L2535.53 1125.4 L2535.76 1124.97 L2535.99 1125.1 L2536.22 1125.13 L2536.44 1124.41 L2536.67 1124.26 L2536.9 1124.53 L2537.13 1124.4 L2537.35 1124.35 L2537.58 1124.43 L2537.81 1124.28 L2538.04 1124.35 L2538.27 1124.58 L2538.49 1125.14 L2538.72 1124.84 L2538.95 1124.78 L2539.18 1124.71 L2539.4 1124.92 L2539.63 1125.34 L2539.86 1125.36 L2540.09 1125.77 L2540.32 1125.53 L2540.54 1125.62 L2540.77 1125.44 L2541 1125.23 L2541.23 1125.39 L2541.45 1125.37 L2541.68 1125.4 L2541.91 1124.79 L2542.14 1124.61 L2542.37 1124.58 L2542.59 1124.53 L2542.82 1124.73 L2543.05 1124.77 L2543.28 1125.19 L2543.5 1125.47 L2543.73 1125.71 L2543.96 1125.59 L2544.19 1125.79 L2544.42 1125.61 L2544.64 1125.62 L2544.87 1125.72 L2545.1 1125.93 L2545.33 1125.66 L2545.55 1125.85 L2545.78 1125.82 L2546.01 1126.33 L2546.24 1126.21 L2546.47 1125.98 L2546.69 1126.23 L2546.92 1126.1 L2547.15 1126.18 L2547.38 1125.98 L2547.6 1126.16 L2547.83 1126.02 L2548.06 1125.89 L2548.29 1125.96 L2548.52 1126.22 L2548.74 1125.89 L2548.97 1125.5 L2549.2 1125.42 L2549.43 1125.64 L2549.65 1125.55 L2549.88 1125.37 L2550.11 1125.55 L2550.34 1125.36 L2550.57 1125.53 L2550.79 1125.43 L2551.02 1125.7 L2551.25 1125.5 L2551.48 1125.63 L2551.7 1125.57 L2551.93 1125.66 L2552.16 1125.87 L2552.39 1125.99 L2552.62 1126.34 L2552.84 1126.42 L2553.07 1126.59 L2553.3 1126.82 L2553.53 1126.79 L2553.75 1127 L2553.98 1126.46 L2554.21 1126.41 L2554.44 1126.4 L2554.67 1126.51 L2554.89 1126.32 L2555.12 1126.15 L2555.35 1126.28 L2555.58 1126.45 L2555.8 1126.94 L2556.03 1126.72 L2556.26 1126.64 L2556.49 1126.69 L2556.72 1126.35 L2556.94 1126.34 L2557.17 1126.46 L2557.4 1126.42 L2557.63 1126.4 L2557.85 1126.39 L2558.08 1126.19 L2558.31 1126.41 L2558.54 1126.69 L2558.77 1126.46 L2558.99 1125.7 L2559.22 1125.81 L2559.45 1125.63 L2559.68 1125.81 L2559.9 1125.72 L2560.13 1125.81 L2560.36 1125.68 L2560.59 1125.45 L2560.82 1125.3 L2561.04 1125.93 L2561.27 1126.03 L2561.5 1126.25 L2561.73 1125.7 L2561.95 1125.73 L2562.18 1125.3 L2562.41 1125.29 L2562.64 1125.52 L2562.87 1125.53 L2563.09 1125.73 L2563.32 1125.82 L2563.55 1125.55 L2563.78 1125.81 L2564 1126.31 L2564.23 1126.03 L2564.46 1126.18 L2564.69 1126.42 L2564.92 1126.64 L2565.14 1126.57 L2565.37 1126.73 L2565.6 1126.79 L2565.83 1126.72 L2566.05 1126.7 L2566.28 1126.4 L2566.51 1126.41 L2566.74 1126.3 L2566.96 1126.25 L2567.19 1126.3 L2567.42 1126.36 L2567.65 1126.66 L2567.88 1126.48 L2568.1 1126.7 L2568.33 1126.56 L2568.56 1126.19 L2568.79 1126.15 L2569.01 1126.35 L2569.24 1126.69 L2569.47 1126.88 L2569.7 1126.94 L2569.93 1126.78 L2570.15 1126.8 L2570.38 1127.36 L2570.61 1127.5 L2570.84 1128.04 L2571.06 1128.16 L2571.29 1128.01 L2571.52 1128.11 L2571.75 1128.17 L2571.98 1127.94 L2572.2 1127.72 L2572.43 1127.98 L2572.43 1024.7 L2572.2 1024.43 L2571.98 1024.65 L2571.75 1024.87 L2571.52 1024.8 L2571.29 1024.69 L2571.06 1024.83 L2570.84 1024.7 L2570.61 1024.17 L2570.38 1024.03 L2570.15 1023.48 L2569.93 1023.45 L2569.7 1023.6 L2569.47 1023.53 L2569.24 1023.33 L2569.01 1022.99 L2568.79 1022.79 L2568.56 1022.81 L2568.33 1023.19 L2568.1 1023.32 L2567.88 1023.09 L2567.65 1023.26 L2567.42 1022.96 L2567.19 1022.88 L2566.96 1022.83 L2566.74 1022.87 L2566.51 1022.97 L2566.28 1022.95 L2566.05 1023.24 L2565.83 1023.25 L2565.6 1023.31 L2565.37 1023.24 L2565.14 1023.08 L2564.92 1023.14 L2564.69 1022.91 L2564.46 1022.66 L2564.23 1022.5 L2564 1022.78 L2563.78 1022.28 L2563.55 1022.02 L2563.32 1022.28 L2563.09 1022.19 L2562.87 1021.98 L2562.64 1021.96 L2562.41 1021.72 L2562.18 1021.72 L2561.95 1022.15 L2561.73 1022.12 L2561.5 1022.67 L2561.27 1022.45 L2561.04 1022.33 L2560.82 1021.73 L2560.59 1021.86 L2560.36 1022.09 L2560.13 1022.22 L2559.9 1022.11 L2559.68 1022.19 L2559.45 1022.01 L2559.22 1022.18 L2558.99 1022.06 L2558.77 1022.85 L2558.54 1023.07 L2558.31 1022.79 L2558.08 1022.56 L2557.85 1022.76 L2557.63 1022.76 L2557.4 1022.76 L2557.17 1022.8 L2556.94 1022.66 L2556.72 1022.66 L2556.49 1023 L2556.26 1022.95 L2556.03 1023.01 L2555.8 1023.23 L2555.58 1022.75 L2555.35 1022.57 L2555.12 1022.43 L2554.89 1022.59 L2554.67 1022.77 L2554.44 1022.65 L2554.21 1022.65 L2553.98 1022.69 L2553.75 1023.24 L2553.53 1023.02 L2553.3 1023.05 L2553.07 1022.81 L2552.84 1022.63 L2552.62 1022.54 L2552.39 1022.19 L2552.16 1022.06 L2551.93 1021.84 L2551.7 1021.74 L2551.48 1021.79 L2551.25 1021.65 L2551.02 1021.84 L2550.79 1021.57 L2550.57 1021.66 L2550.34 1021.48 L2550.11 1021.66 L2549.88 1021.47 L2549.65 1021.65 L2549.43 1021.73 L2549.2 1021.5 L2548.97 1021.57 L2548.74 1021.96 L2548.52 1022.29 L2548.29 1022.02 L2548.06 1021.94 L2547.83 1022.07 L2547.6 1022.19 L2547.38 1022 L2547.15 1022.2 L2546.92 1022.11 L2546.69 1022.23 L2546.47 1021.97 L2546.24 1022.19 L2546.01 1022.31 L2545.78 1021.8 L2545.55 1021.83 L2545.33 1021.63 L2545.1 1021.89 L2544.87 1021.67 L2544.64 1021.56 L2544.42 1021.54 L2544.19 1021.72 L2543.96 1021.5 L2543.73 1021.62 L2543.5 1021.38 L2543.28 1021.09 L2543.05 1020.67 L2542.82 1020.62 L2542.59 1020.42 L2542.37 1020.45 L2542.14 1020.47 L2541.91 1020.64 L2541.68 1021.27 L2541.45 1021.23 L2541.23 1021.24 L2541 1021.08 L2540.77 1021.27 L2540.54 1021.44 L2540.32 1021.35 L2540.09 1021.58 L2539.86 1021.18 L2539.63 1021.14 L2539.4 1020.73 L2539.18 1020.51 L2538.95 1020.56 L2538.72 1020.62 L2538.49 1020.91 L2538.27 1020.36 L2538.04 1020.12 L2537.81 1020.05 L2537.58 1020.19 L2537.35 1020.1 L2537.13 1020.14 L2536.9 1020.26 L2536.67 1019.98 L2536.44 1020.13 L2536.22 1020.88 L2535.99 1020.84 L2535.76 1020.7 L2535.53 1021.13 L2535.3 1021.59 L2535.08 1021.61 L2534.85 1021.8 L2534.62 1021.85 L2534.39 1021.5 L2534.17 1021.66 L2533.94 1021.15 L2533.71 1020.84 L2533.48 1020.96 L2533.25 1020.52 L2533.03 1020.91 L2532.8 1020.74 L2532.57 1020.56 L2532.34 1021.22 L2532.12 1021.09 L2531.89 1020.74 L2531.66 1020.64 L2531.43 1020.68 L2531.2 1021.05 L2530.98 1021.04 L2530.75 1021.05 L2530.52 1021.32 L2530.29 1020.86 L2530.07 1020.75 L2529.84 1021.33 L2529.61 1021.32 L2529.38 1021.21 L2529.16 1021.06 L2528.93 1021.92 L2528.7 1021.68 L2528.47 1021.42 L2528.24 1021.41 L2528.02 1021.6 L2527.79 1021.77 L2527.56 1022.14 L2527.33 1021.93 L2527.11 1021.97 L2526.88 1021.56 L2526.65 1021.77 L2526.42 1021.45 L2526.19 1021.43 L2525.97 1021.57 L2525.74 1021.29 L2525.51 1021.54 L2525.28 1021.58 L2525.06 1021.16 L2524.83 1020.94 L2524.6 1021.5 L2524.37 1021.41 L2524.14 1021.72 L2523.92 1021.66 L2523.69 1021.65 L2523.46 1021.96 L2523.23 1021.63 L2523.01 1021.81 L2522.78 1021.97 L2522.55 1022.15 L2522.32 1022.4 L2522.09 1022.4 L2521.87 1022.43 L2521.64 1022.49 L2521.41 1022.4 L2521.18 1022.37 L2520.96 1022.13 L2520.73 1022.34 L2520.5 1022.16 L2520.27 1021.65 L2520.04 1022.17 L2519.82 1021.82 L2519.59 1021.36 L2519.36 1021.18 L2519.13 1021.02 L2518.91 1021.23 L2518.68 1021.19 L2518.45 1021.14 L2518.22 1020.99 L2517.99 1020.9 L2517.77 1020.89 L2517.54 1021.07 L2517.31 1021.04 L2517.08 1020.95 L2516.86 1020.78 L2516.63 1020.66 L2516.4 1020.48 L2516.17 1020.6 L2515.94 1020.41 L2515.72 1020.39 L2515.49 1021.01 L2515.26 1020.67 L2515.03 1020.74 L2514.81 1021.39 L2514.58 1021.66 L2514.35 1021.69 L2514.12 1021.55 L2513.89 1021.38 L2513.67 1021.81 L2513.44 1022.09 L2513.21 1022.11 L2512.98 1022.19 L2512.76 1021.99 L2512.53 1021.57 L2512.3 1021.58 L2512.07 1021.78 L2511.84 1021.42 L2511.62 1021.04 L2511.39 1020.97 L2511.16 1020.88 L2510.93 1020.88 L2510.71 1021.36 L2510.48 1021.07 L2510.25 1020.99 L2510.02 1020.79 L2509.79 1020.68 L2509.57 1020.63 L2509.34 1020.64 L2509.11 1020.54 L2508.88 1020.68 L2508.66 1021.67 L2508.43 1021.31 L2508.2 1020.68 L2507.97 1020.79 L2507.74 1021.38 L2507.52 1021.7 L2507.29 1022.21 L2507.06 1022.32 L2506.83 1022.44 L2506.61 1022.16 L2506.38 1021.88 L2506.15 1022.35 L2505.92 1022.78 L2505.69 1022.34 L2505.47 1021.97 L2505.24 1021.89 L2505.01 1021.28 L2504.78 1021.1 L2504.56 1021.02 L2504.33 1021.02 L2504.1 1020.95 L2503.87 1020.49 L2503.64 1020.18 L2503.42 1020.3 L2503.19 1020.09 L2502.96 1019.85 L2502.73 1019.73 L2502.51 1019.35 L2502.28 1019.09 L2502.05 1019.21 L2501.82 1019 L2501.59 1018.65 L2501.37 1019.18 L2501.14 1019.83 L2500.91 1019.5 L2500.68 1019.36 L2500.46 1019.3 L2500.23 1019.24 L2500 1019.69 L2499.77 1019.54 L2499.54 1019.71 L2499.32 1019.58 L2499.09 1019.66 L2498.86 1019.72 L2498.63 1019.4 L2498.41 1019.64 L2498.18 1020.37 L2497.95 1020.69 L2497.72 1020.72 L2497.49 1021.38 L2497.27 1021.38 L2497.04 1021.64 L2496.81 1021.59 L2496.58 1021.5 L2496.36 1021.4 L2496.13 1021.03 L2495.9 1020.8 L2495.67 1020.52 L2495.44 1020.76 L2495.22 1020.51 L2494.99 1020.56 L2494.76 1020.6 L2494.53 1021.03 L2494.31 1021.49 L2494.08 1021.53 L2493.85 1021.32 L2493.62 1021.35 L2493.4 1021.4 L2493.17 1021.76 L2492.94 1021.78 L2492.71 1021.83 L2492.48 1022.33 L2492.26 1022.26 L2492.03 1022.29 L2491.8 1022.07 L2491.57 1022.1 L2491.35 1022.12 L2491.12 1021.68 L2490.89 1021.88 L2490.66 1021.59 L2490.43 1021.22 L2490.21 1021.05 L2489.98 1021.08 L2489.75 1021.11 L2489.52 1020.64 L2489.3 1020.49 L2489.07 1020.83 L2488.84 1020.47 L2488.61 1020.67 L2488.38 1020.53 L2488.16 1020.76 L2487.93 1020.94 L2487.7 1021.14 L2487.47 1021.27 L2487.25 1021.17 L2487.02 1020.66 L2486.79 1021.47 L2486.56 1021.62 L2486.33 1021.52 L2486.11 1021.64 L2485.88 1021.9 L2485.65 1021.56 L2485.42 1021.17 L2485.2 1021.17 L2484.97 1021.12 L2484.74 1021.22 L2484.51 1021.65 L2484.28 1021.71 L2484.06 1021.32 L2483.83 1021.28 L2483.6 1021.45 L2483.37 1021.39 L2483.15 1021.22 L2482.92 1021.6 L2482.69 1021.24 L2482.46 1020.95 L2482.23 1021.18 L2482.01 1020.61 L2481.78 1020.56 L2481.55 1021.02 L2481.32 1021.24 L2481.1 1021.76 L2480.87 1021.55 L2480.64 1022.12 L2480.41 1022.09 L2480.18 1022.17 L2479.96 1022.42 L2479.73 1022.48 L2479.5 1022.26 L2479.27 1021.98 L2479.05 1021.91 L2478.82 1021.58 L2478.59 1021.19 L2478.36 1020.95 L2478.13 1021.16 L2477.91 1020.68 L2477.68 1020.61 L2477.45 1020.61 L2477.22 1020.75 L2477 1020.88 L2476.77 1020.52 L2476.54 1020.23 L2476.31 1019.99 L2476.08 1020.13 L2475.86 1019.75 L2475.63 1019.67 L2475.4 1019.32 L2475.17 1019.21 L2474.95 1019.04 L2474.72 1019.32 L2474.49 1019.38 L2474.26 1020.17 L2474.03 1020.29 L2473.81 1020.53 L2473.58 1019.84 L2473.35 1020.05 L2473.12 1019.88 L2472.9 1020.2 L2472.67 1020.27 L2472.44 1020.95 L2472.21 1020.52 L2471.98 1020.5 L2471.76 1020.43 L2471.53 1020.62 L2471.3 1020.77 L2471.07 1021.34 L2470.85 1022.06 L2470.62 1021.7 L2470.39 1021.57 L2470.16 1021.37 L2469.93 1021.06 L2469.71 1020.8 L2469.48 1020.74 L2469.25 1020.83 L2469.02 1020.63 L2468.8 1020.21 L2468.57 1020.43 L2468.34 1020.87 L2468.11 1020.85 L2467.88 1021.06 L2467.66 1021.54 L2467.43 1021.52 L2467.2 1021.8 L2466.97 1021.72 L2466.75 1022.18 L2466.52 1022.15 L2466.29 1021.77 L2466.06 1021.75 L2465.83 1021.65 L2465.61 1022 L2465.38 1022 L2465.15 1022.26 L2464.92 1022.32 L2464.7 1022.26 L2464.47 1022.43 L2464.24 1022.6 L2464.01 1022.67 L2463.78 1022.77 L2463.56 1022.69 L2463.33 1022.23 L2463.1 1022.14 L2462.87 1021.72 L2462.65 1022.59 L2462.42 1022.66 L2462.19 1023.08 L2461.96 1023.35 L2461.73 1023 L2461.51 1023.17 L2461.28 1023.7 L2461.05 1023.54 L2460.82 1023.24 L2460.6 1023.45 L2460.37 1023.74 L2460.14 1023.94 L2459.91 1024.13 L2459.68 1023.9 L2459.46 1023.52 L2459.23 1023.31 L2459 1023.18 L2458.77 1023.22 L2458.55 1023.75 L2458.32 1023.64 L2458.09 1023.37 L2457.86 1023.06 L2457.64 1022.84 L2457.41 1022.53 L2457.18 1022.29 L2456.95 1022.32 L2456.72 1022.04 L2456.5 1022.09 L2456.27 1022 L2456.04 1022.39 L2455.81 1022.3 L2455.59 1021.86 L2455.36 1021.45 L2455.13 1021.19 L2454.9 1021.44 L2454.67 1021.45 L2454.45 1021.26 L2454.22 1021.09 L2453.99 1021.51 L2453.76 1021.58 L2453.54 1021.51 L2453.31 1021.45 L2453.08 1021.93 L2452.85 1022.3 L2452.62 1022.1 L2452.4 1021.81 L2452.17 1021.6 L2451.94 1021.17 L2451.71 1021.11 L2451.49 1021.03 L2451.26 1021.05 L2451.03 1021.16 L2450.8 1021.14 L2450.57 1021.23 L2450.35 1021.4 L2450.12 1021.58 L2449.89 1021.99 L2449.66 1021.72 L2449.44 1021.72 L2449.21 1021.58 L2448.98 1021.86 L2448.75 1021.73 L2448.52 1021.68 L2448.3 1021.73 L2448.07 1022.07 L2447.84 1021.85 L2447.61 1021.37 L2447.39 1021.43 L2447.16 1021.6 L2446.93 1021.29 L2446.7 1021.29 L2446.47 1021.37 L2446.25 1021.48 L2446.02 1021.39 L2445.79 1021.57 L2445.56 1021.38 L2445.34 1021.06 L2445.11 1021.41 L2444.88 1021.39 L2444.65 1021.2 L2444.42 1021.41 L2444.2 1021.1 L2443.97 1020.85 L2443.74 1020.77 L2443.51 1021.13 L2443.29 1020.91 L2443.06 1020.75 L2442.83 1020.74 L2442.6 1021.51 L2442.37 1021.64 L2442.15 1021.49 L2441.92 1021.2 L2441.69 1021.31 L2441.46 1021.79 L2441.24 1022.06 L2441.01 1022.14 L2440.78 1022.14 L2440.55 1022.93 L2440.32 1023.47 L2440.1 1023.74 L2439.87 1023.69 L2439.64 1024.89 L2439.41 1024.82 L2439.19 1024.73 L2438.96 1024.42 L2438.73 1024.45 L2438.5 1024.2 L2438.27 1024.61 L2438.05 1024.59 L2437.82 1024.5 L2437.59 1024.62 L2437.36 1024.23 L2437.14 1024.2 L2436.91 1024.48 L2436.68 1024.52 L2436.45 1024.62 L2436.22 1024.99 L2436 1024.71 L2435.77 1024.94 L2435.54 1024.7 L2435.31 1024.44 L2435.09 1024.77 L2434.86 1024.8 L2434.63 1024.55 L2434.4 1024.68 L2434.17 1024.25 L2433.95 1024.33 L2433.72 1024.3 L2433.49 1024.06 L2433.26 1023.9 L2433.04 1023.64 L2432.81 1023.62 L2432.58 1023.98 L2432.35 1023.98 L2432.12 1024.08 L2431.9 1023.61 L2431.67 1023.73 L2431.44 1023.3 L2431.21 1023.21 L2430.99 1022.78 L2430.76 1022.41 L2430.53 1022.34 L2430.3 1022.27 L2430.07 1022.7 L2429.85 1022.66 L2429.62 1022.53 L2429.39 1022.76 L2429.16 1022.31 L2428.94 1022 L2428.71 1021.75 L2428.48 1022 L2428.25 1021.98 L2428.02 1021.49 L2427.8 1021.2 L2427.57 1021.03 L2427.34 1020.78 L2427.11 1020.69 L2426.89 1020.93 L2426.66 1020.75 L2426.43 1020.67 L2426.2 1020.46 L2425.97 1020.61 L2425.75 1020.45 L2425.52 1020.47 L2425.29 1020.71 L2425.06 1020.57 L2424.84 1020.56 L2424.61 1020.04 L2424.38 1019.66 L2424.15 1019.49 L2423.92 1019.22 L2423.7 1019.2 L2423.47 1018.73 L2423.24 1018.95 L2423.01 1019.15 L2422.79 1018.94 L2422.56 1019.21 L2422.33 1018.87 L2422.1 1019.22 L2421.87 1018.99 L2421.65 1018.72 L2421.42 1018.49 L2421.19 1018.04 L2420.96 1018.29 L2420.74 1018.15 L2420.51 1018.1 L2420.28 1018.2 L2420.05 1018.34 L2419.83 1018.29 L2419.6 1018.04 L2419.37 1018.05 L2419.14 1017.81 L2418.91 1017.87 L2418.69 1018.01 L2418.46 1017.73 L2418.23 1017.94 L2418 1018.52 L2417.78 1018.64 L2417.55 1018.75 L2417.32 1018.89 L2417.09 1018.95 L2416.86 1018.75 L2416.64 1018.65 L2416.41 1018.67 L2416.18 1018.51 L2415.95 1018.32 L2415.73 1018.69 L2415.5 1018.39 L2415.27 1018.3 L2415.04 1018.23 L2414.81 1018.26 L2414.59 1018.03 L2414.36 1017.93 L2414.13 1017.8 L2413.9 1017.41 L2413.68 1017.18 L2413.45 1017.09 L2413.22 1016.9 L2412.99 1017.15 L2412.76 1017.63 L2412.54 1017.71 L2412.31 1017.97 L2412.08 1017.63 L2411.85 1018.06 L2411.63 1018.38 L2411.4 1018.42 L2411.17 1018.75 L2410.94 1019.07 L2410.71 1019.43 L2410.49 1019.27 L2410.26 1019.26 L2410.03 1019.29 L2409.8 1019.02 L2409.58 1019.9 L2409.35 1020.04 L2409.12 1019.42 L2408.89 1019.18 L2408.66 1019.11 L2408.44 1019.3 L2408.21 1019.74 L2407.98 1020.04 L2407.75 1019.77 L2407.53 1019.43 L2407.3 1019.07 L2407.07 1018.79 L2406.84 1018.73 L2406.61 1018.79 L2406.39 1018.43 L2406.16 1019.02 L2405.93 1018.9 L2405.7 1018.7 L2405.48 1018.36 L2405.25 1018.25 L2405.02 1018.25 L2404.79 1018.86 L2404.56 1018.9 L2404.34 1018.91 L2404.11 1019.01 L2403.88 1018.85 L2403.65 1018.9 L2403.43 1018.96 L2403.2 1018.65 L2402.97 1018.4 L2402.74 1018.07 L2402.51 1018.39 L2402.29 1018.42 L2402.06 1018.64 L2401.83 1018.68 L2401.6 1019.12 L2401.38 1018.86 L2401.15 1018.47 L2400.92 1018.28 L2400.69 1018.59 L2400.46 1018.3 L2400.24 1018.04 L2400.01 1017.92 L2399.78 1017.53 L2399.55 1017.99 L2399.33 1017.84 L2399.1 1018.4 L2398.87 1018.48 L2398.64 1018.37 L2398.41 1018.05 L2398.19 1017.89 L2397.96 1017.67 L2397.73 1018.39 L2397.5 1019.28 L2397.28 1019.71 L2397.05 1019.47 L2396.82 1019.17 L2396.59 1019.15 L2396.36 1019.04 L2396.14 1019.11 L2395.91 1019.03 L2395.68 1019.2 L2395.45 1019.14 L2395.23 1018.96 L2395 1019.16 L2394.77 1019.09 L2394.54 1019.19 L2394.31 1019.05 L2394.09 1019.05 L2393.86 1019.69 L2393.63 1019.49 L2393.4 1019.29 L2393.18 1019.08 L2392.95 1019.5 L2392.72 1019.44 L2392.49 1019.16 L2392.26 1019.22 L2392.04 1019.37 L2391.81 1019.35 L2391.58 1019.31 L2391.35 1019.93 L2391.13 1019.88 L2390.9 1019.89 L2390.67 1019.61 L2390.44 1019.57 L2390.21 1019.88 L2389.99 1020.22 L2389.76 1020.38 L2389.53 1020.35 L2389.3 1020.46 L2389.08 1020.65 L2388.85 1020.43 L2388.62 1020.2 L2388.39 1020.4 L2388.16 1020.34 L2387.94 1020 L2387.71 1019.59 L2387.48 1019.69 L2387.25 1019.74 L2387.03 1019.57 L2386.8 1019.74 L2386.57 1019.38 L2386.34 1019.12 L2386.11 1018.96 L2385.89 1018.55 L2385.66 1018.46 L2385.43 1018.25 L2385.2 1018.03 L2384.98 1018.31 L2384.75 1018.47 L2384.52 1018.12 L2384.29 1017.89 L2384.07 1017.83 L2383.84 1017.69 L2383.61 1017.31 L2383.38 1017.03 L2383.15 1017.09 L2382.93 1016.83 L2382.7 1017.69 L2382.47 1018.32 L2382.24 1018.12 L2382.02 1018.08 L2381.79 1017.95 L2381.56 1017.43 L2381.33 1016.86 L2381.1 1016.43 L2380.88 1016.47 L2380.65 1016.21 L2380.42 1016.15 L2380.19 1015.72 L2379.97 1015.49 L2379.74 1015.18 L2379.51 1014.59 L2379.28 1014.93 L2379.05 1015.02 L2378.83 1015.16 L2378.6 1014.82 L2378.37 1014.58 L2378.14 1014.78 L2377.92 1015.01 L2377.69 1015.17 L2377.46 1015.37 L2377.23 1015.42 L2377 1015.5 L2376.78 1015.26 L2376.55 1015.13 L2376.32 1015.05 L2376.09 1014.72 L2375.87 1014.61 L2375.64 1014.87 L2375.41 1014.85 L2375.18 1014.47 L2374.95 1014.63 L2374.73 1014.52 L2374.5 1014.32 L2374.27 1014.08 L2374.04 1014.06 L2373.82 1014.21 L2373.59 1014.02 L2373.36 1014.31 L2373.13 1013.89 L2372.9 1013.83 L2372.68 1013.64 L2372.45 1013.54 L2372.22 1013.59 L2371.99 1013.61 L2371.77 1013.58 L2371.54 1013.82 L2371.31 1013.64 L2371.08 1013.29 L2370.85 1014.01 L2370.63 1014.39 L2370.4 1014.56 L2370.17 1014.59 L2369.94 1014.95 L2369.72 1015.32 L2369.49 1015.41 L2369.26 1015.02 L2369.03 1015.05 L2368.8 1014.7 L2368.58 1014.68 L2368.35 1014.74 L2368.12 1014.73 L2367.89 1014.78 L2367.67 1014.9 L2367.44 1014.56 L2367.21 1014.47 L2366.98 1014.44 L2366.75 1014.37 L2366.53 1014.9 L2366.3 1015.01 L2366.07 1014.94 L2365.84 1015.23 L2365.62 1015.35 L2365.39 1015.23 L2365.16 1015.28 L2364.93 1015.33 L2364.7 1015.14 L2364.48 1014.9 L2364.25 1014.64 L2364.02 1014.76 L2363.79 1014.67 L2363.57 1014.3 L2363.34 1013.98 L2363.11 1013.79 L2362.88 1013.56 L2362.65 1013.71 L2362.43 1013.68 L2362.2 1013.55 L2361.97 1013.08 L2361.74 1013.22 L2361.52 1013.16 L2361.29 1013.41 L2361.06 1013.14 L2360.83 1013.19 L2360.6 1012.81 L2360.38 1013.1 L2360.15 1013.13 L2359.92 1013.09 L2359.69 1012.81 L2359.47 1012.86 L2359.24 1013.16 L2359.01 1013.35 L2358.78 1013.07 L2358.55 1013.34 L2358.33 1013.71 L2358.1 1013.58 L2357.87 1013.02 L2357.64 1012.31 L2357.42 1012.48 L2357.19 1012.82 L2356.96 1012.67 L2356.73 1012.61 L2356.5 1012.58 L2356.28 1012.33 L2356.05 1012.18 L2355.82 1011.84 L2355.59 1012.27 L2355.37 1012.31 L2355.14 1012.42 L2354.91 1012.35 L2354.68 1012.07 L2354.45 1012.55 L2354.23 1012.59 L2354 1012.64 L2353.77 1012.53 L2353.54 1012.23 L2353.32 1012.07 L2353.09 1011.85 L2352.86 1011.97 L2352.63 1011.59 L2352.4 1011.31 L2352.18 1010.93 L2351.95 1010.95 L2351.72 1011.45 L2351.49 1011.58 L2351.27 1011.83 L2351.04 1011.69 L2350.81 1011.61 L2350.58 1011.25 L2350.35 1011.45 L2350.13 1011.55 L2349.9 1011.84 L2349.67 1011.77 L2349.44 1011.74 L2349.22 1011.72 L2348.99 1011.23 L2348.76 1011.4 L2348.53 1011.14 L2348.31 1011.15 L2348.08 1011.1 L2347.85 1011.12 L2347.62 1011.43 L2347.39 1011.22 L2347.17 1011.36 L2346.94 1011.31 L2346.71 1011.39 L2346.48 1011.31 L2346.26 1011.71 L2346.03 1011.51 L2345.8 1011.69 L2345.57 1011.48 L2345.34 1011.68 L2345.12 1011.4 L2344.89 1011.12 L2344.66 1011.02 L2344.43 1011.08 L2344.21 1011.21 L2343.98 1011.73 L2343.75 1012.83 L2343.52 1012.76 L2343.29 1013.29 L2343.07 1013.5 L2342.84 1013.34 L2342.61 1013.11 L2342.38 1012.85 L2342.16 1012.86 L2341.93 1012.58 L2341.7 1012.72 L2341.47 1012.31 L2341.24 1012.48 L2341.02 1012.76 L2340.79 1012.29 L2340.56 1011.85 L2340.33 1012.14 L2340.11 1011.97 L2339.88 1011.67 L2339.65 1012.03 L2339.42 1011.95 L2339.19 1012.19 L2338.97 1012.34 L2338.74 1012.54 L2338.51 1012.36 L2338.28 1012.73 L2338.06 1012.27 L2337.83 1012.21 L2337.6 1012.25 L2337.37 1012.2 L2337.14 1012.3 L2336.92 1011.95 L2336.69 1011.99 L2336.46 1012.49 L2336.23 1012.36 L2336.01 1012.76 L2335.78 1012.82 L2335.55 1012.93 L2335.32 1012.55 L2335.09 1012.68 L2334.87 1012.68 L2334.64 1012.53 L2334.41 1013.13 L2334.18 1013.01 L2333.96 1012.69 L2333.73 1012.53 L2333.5 1012.03 L2333.27 1012.07 L2333.04 1012.31 L2332.82 1012.61 L2332.59 1012.61 L2332.36 1012.44 L2332.13 1012.66 L2331.91 1012.53 L2331.68 1012.54 L2331.45 1012.82 L2331.22 1012.74 L2330.99 1013.03 L2330.77 1013.19 L2330.54 1013.38 L2330.31 1013.93 L2330.08 1013.9 L2329.86 1013.79 L2329.63 1013.7 L2329.4 1014.91 L2329.17 1015.61 L2328.94 1015.24 L2328.72 1015.22 L2328.49 1014.82 L2328.26 1014.47 L2328.03 1014.63 L2327.81 1014.36 L2327.58 1014.01 L2327.35 1014.25 L2327.12 1014.45 L2326.89 1014.33 L2326.67 1014.17 L2326.44 1013.89 L2326.21 1013.98 L2325.98 1013.85 L2325.76 1013.83 L2325.53 1014.05 L2325.3 1014.12 L2325.07 1014.33 L2324.84 1014.12 L2324.62 1014.06 L2324.39 1013.81 L2324.16 1014.01 L2323.93 1013.92 L2323.71 1013.58 L2323.48 1013.35 L2323.25 1013.39 L2323.02 1013.16 L2322.79 1013.11 L2322.57 1013.5 L2322.34 1013.71 L2322.11 1013.85 L2321.88 1013.54 L2321.66 1013.7 L2321.43 1014.44 L2321.2 1014.71 L2320.97 1015 L2320.74 1015.06 L2320.52 1014.85 L2320.29 1014.98 L2320.06 1015.48 L2319.83 1015.05 L2319.61 1014.73 L2319.38 1014.49 L2319.15 1014.22 L2318.92 1013.67 L2318.69 1013.67 L2318.47 1014.12 L2318.24 1014.04 L2318.01 1013.82 L2317.78 1013.99 L2317.56 1014.01 L2317.33 1014.08 L2317.1 1013.68 L2316.87 1013.09 L2316.64 1013 L2316.42 1012.77 L2316.19 1012.53 L2315.96 1012.35 L2315.73 1012.46 L2315.51 1012.34 L2315.28 1012.41 L2315.05 1012.01 L2314.82 1012.44 L2314.59 1012.33 L2314.37 1012.42 L2314.14 1012.55 L2313.91 1012.42 L2313.68 1012.54 L2313.46 1013.05 L2313.23 1012.44 L2313 1012.41 L2312.77 1012.54 L2312.54 1012.29 L2312.32 1012.19 L2312.09 1012.09 L2311.86 1011.9 L2311.63 1011.77 L2311.41 1011.81 L2311.18 1011.53 L2310.95 1011.14 L2310.72 1011.12 L2310.5 1011.58 L2310.27 1011.44 L2310.04 1011.41 L2309.81 1011.15 L2309.58 1010.57 L2309.36 1010.53 L2309.13 1010.2 L2308.9 1009.95 L2308.67 1009.87 L2308.45 1010.05 L2308.22 1009.73 L2307.99 1009.41 L2307.76 1009.62 L2307.53 1009.36 L2307.31 1008.87 L2307.08 1008.84 L2306.85 1009.16 L2306.62 1008.92 L2306.4 1008.38 L2306.17 1008.18 L2305.94 1008.14 L2305.71 1007.89 L2305.48 1007.92 L2305.26 1007.99 L2305.03 1008.83 L2304.8 1008.74 L2304.57 1008.29 L2304.35 1008.34 L2304.12 1008.87 L2303.89 1008.59 L2303.66 1008.49 L2303.43 1008.67 L2303.21 1008.98 L2302.98 1008.82 L2302.75 1008.49 L2302.52 1008.21 L2302.3 1008.01 L2302.07 1008.13 L2301.84 1008.05 L2301.61 1008.16 L2301.38 1008.45 L2301.16 1008.7 L2300.93 1008.92 L2300.7 1009.17 L2300.47 1009.19 L2300.25 1009.17 L2300.02 1008.94 L2299.79 1008.64 L2299.56 1008.58 L2299.33 1008.41 L2299.11 1008.06 L2298.88 1007.72 L2298.65 1007.85 L2298.42 1008.02 L2298.2 1008.13 L2297.97 1007.97 L2297.74 1008.12 L2297.51 1007.71 L2297.28 1007.56 L2297.06 1007.58 L2296.83 1007.25 L2296.6 1006.64 L2296.37 1006.39 L2296.15 1006.51 L2295.92 1006.47 L2295.69 1006.35 L2295.46 1006.6 L2295.23 1006.62 L2295.01 1006.5 L2294.78 1007.19 L2294.55 1007.08 L2294.32 1006.93 L2294.1 1006.88 L2293.87 1006.69 L2293.64 1006.32 L2293.41 1006.11 L2293.18 1005.85 L2292.96 1005.98 L2292.73 1006.01 L2292.5 1005.76 L2292.27 1006.21 L2292.05 1005.79 L2291.82 1006.04 L2291.59 1006.3 L2291.36 1006.66 L2291.13 1006.85 L2290.91 1006.71 L2290.68 1007.36 L2290.45 1007.45 L2290.22 1007.42 L2290 1007.46 L2289.77 1007.43 L2289.54 1007.49 L2289.31 1007.37 L2289.08 1007.27 L2288.86 1007.41 L2288.63 1007.15 L2288.4 1007.8 L2288.17 1007.81 L2287.95 1008.05 L2287.72 1007.97 L2287.49 1008.51 L2287.26 1008.48 L2287.03 1008.5 L2286.81 1008.43 L2286.58 1008.28 L2286.35 1008.14 L2286.12 1008.47 L2285.9 1008.66 L2285.67 1008.55 L2285.44 1008.3 L2285.21 1008.73 L2284.98 1008.8 L2284.76 1008.79 L2284.53 1009.38 L2284.3 1009.04 L2284.07 1008.92 L2283.85 1008.75 L2283.62 1008.72 L2283.39 1008.38 L2283.16 1008.57 L2282.93 1008.84 L2282.71 1008.75 L2282.48 1009.02 L2282.25 1008.81 L2282.02 1008.64 L2281.8 1008.36 L2281.57 1008.25 L2281.34 1008.27 L2281.11 1007.96 L2280.88 1007.92 L2280.66 1008.29 L2280.43 1008.05 L2280.2 1008.09 L2279.97 1007.54 L2279.75 1008.31 L2279.52 1008.19 L2279.29 1008.06 L2279.06 1008.37 L2278.83 1007.94 L2278.61 1007.53 L2278.38 1007.28 L2278.15 1007.29 L2277.92 1007.17 L2277.7 1007.01 L2277.47 1006.69 L2277.24 1006.55 L2277.01 1006.31 L2276.78 1006.46 L2276.56 1006.32 L2276.33 1006.23 L2276.1 1005.67 L2275.87 1005.75 L2275.65 1005.83 L2275.42 1006.08 L2275.19 1006.42 L2274.96 1006.83 L2274.74 1006.59 L2274.51 1006.44 L2274.28 1006.45 L2274.05 1006.6 L2273.82 1006.95 L2273.6 1007.24 L2273.37 1006.85 L2273.14 1006.74 L2272.91 1006.48 L2272.69 1006.16 L2272.46 1006.53 L2272.23 1006.69 L2272 1007.33 L2271.77 1007.38 L2271.55 1008.03 L2271.32 1008.16 L2271.09 1008.19 L2270.86 1007.68 L2270.64 1007.98 L2270.41 1007.88 L2270.18 1007.75 L2269.95 1007.86 L2269.72 1008.01 L2269.5 1007.92 L2269.27 1007.85 L2269.04 1007.96 L2268.81 1008.1 L2268.59 1007.82 L2268.36 1007.7 L2268.13 1007.15 L2267.9 1006.65 L2267.67 1006.08 L2267.45 1006.13 L2267.22 1006.2 L2266.99 1006.53 L2266.76 1007.15 L2266.54 1007.64 L2266.31 1007.57 L2266.08 1007.46 L2265.85 1007.62 L2265.62 1007.53 L2265.4 1007.56 L2265.17 1007.32 L2264.94 1008.24 L2264.71 1008.26 L2264.49 1008.29 L2264.26 1007.97 L2264.03 1008.03 L2263.8 1007.64 L2263.57 1008 L2263.35 1008.05 L2263.12 1007.6 L2262.89 1007.81 L2262.66 1007.39 L2262.44 1007.16 L2262.21 1007.23 L2261.98 1006.86 L2261.75 1007.11 L2261.52 1007.42 L2261.3 1007.43 L2261.07 1007.64 L2260.84 1007.15 L2260.61 1007.34 L2260.39 1006.67 L2260.16 1006.97 L2259.93 1006.28 L2259.7 1006.7 L2259.47 1006.17 L2259.25 1006.07 L2259.02 1005.83 L2258.79 1005.86 L2258.56 1006.02 L2258.34 1006.48 L2258.11 1006.45 L2257.88 1006.21 L2257.65 1006.29 L2257.42 1006.14 L2257.2 1006.59 L2256.97 1005.93 L2256.74 1005.7 L2256.51 1005.61 L2256.29 1005.6 L2256.06 1006.28 L2255.83 1006.24 L2255.6 1006.07 L2255.37 1007.09 L2255.15 1006.98 L2254.92 1007.1 L2254.69 1006.61 L2254.46 1006.1 L2254.24 1005.84 L2254.01 1005.9 L2253.78 1005.69 L2253.55 1006.18 L2253.32 1006.3 L2253.1 1006.41 L2252.87 1006.48 L2252.64 1006.78 L2252.41 1007.06 L2252.19 1006.56 L2251.96 1006.35 L2251.73 1005.85 L2251.5 1005.73 L2251.27 1005.89 L2251.05 1005.69 L2250.82 1005.92 L2250.59 1005.91 L2250.36 1006.33 L2250.14 1006.59 L2249.91 1006.71 L2249.68 1006.42 L2249.45 1006.13 L2249.22 1006.06 L2249 1005.94 L2248.77 1005.67 L2248.54 1006.02 L2248.31 1005.61 L2248.09 1005.53 L2247.86 1005.52 L2247.63 1005.59 L2247.4 1006.34 L2247.17 1006.27 L2246.95 1006.59 L2246.72 1006.68 L2246.49 1006.34 L2246.26 1006.58 L2246.04 1007.14 L2245.81 1006.95 L2245.58 1006.6 L2245.35 1006.33 L2245.12 1006.51 L2244.9 1006.5 L2244.67 1006.27 L2244.44 1006.15 L2244.21 1005.68 L2243.99 1005.23 L2243.76 1004.79 L2243.53 1005.04 L2243.3 1004.92 L2243.07 1004.56 L2242.85 1004.74 L2242.62 1004.89 L2242.39 1004.62 L2242.16 1004.31 L2241.94 1004.06 L2241.71 1003.6 L2241.48 1003.37 L2241.25 1002.83 L2241.02 1002.55 L2240.8 1002.13 L2240.57 1002.11 L2240.34 1002.22 L2240.11 1002.01 L2239.89 1002.27 L2239.66 1001.8 L2239.43 1001.62 L2239.2 1001.94 L2238.98 1001.87 L2238.75 1001.74 L2238.52 1001.69 L2238.29 1001.5 L2238.06 1001.39 L2237.84 1001.64 L2237.61 1001.59 L2237.38 1001.86 L2237.15 1001.82 L2236.93 1001.71 L2236.7 1001.36 L2236.47 1001.56 L2236.24 1001.58 L2236.01 1001.71 L2235.79 1001.66 L2235.56 1001.48 L2235.33 1001.25 L2235.1 1001.65 L2234.88 1001.59 L2234.65 1001.11 L2234.42 1001 L2234.19 1000.69 L2233.96 1000.66 L2233.74 1000.65 L2233.51 1001.1 L2233.28 1001.29 L2233.05 1001.38 L2232.83 1001.65 L2232.6 1001.26 L2232.37 1001.47 L2232.14 1001.16 L2231.91 1000.63 L2231.69 1000.9 L2231.46 1000.92 L2231.23 1001 L2231 1000.86 L2230.78 1001.53 L2230.55 1001.7 L2230.32 1001.48 L2230.09 1001.36 L2229.86 1001.21 L2229.64 1001.03 L2229.41 1001.23 L2229.18 1001.22 L2228.95 1000.82 L2228.73 1000.62 L2228.5 1000.64 L2228.27 1000.23 L2228.04 1000.08 L2227.81 1000.02 L2227.59 1000.26 L2227.36 1000.05 L2227.13 999.511 L2226.9 999.179 L2226.68 999.353 L2226.45 999.349 L2226.22 999.072 L2225.99 998.781 L2225.76 998.982 L2225.54 999.069 L2225.31 999.078 L2225.08 999.014 L2224.85 999.647 L2224.63 999.377 L2224.4 999.482 L2224.17 999.184 L2223.94 999.114 L2223.71 998.859 L2223.49 999.395 L2223.26 999.544 L2223.03 999.268 L2222.8 998.933 L2222.58 999.805 L2222.35 999.593 L2222.12 999.618 L2221.89 999.607 L2221.66 999.227 L2221.44 999.679 L2221.21 999.886 L2220.98 999.857 L2220.75 999.988 L2220.53 999.93 L2220.3 999.665 L2220.07 999.599 L2219.84 999.419 L2219.61 999.727 L2219.39 999.768 L2219.16 999.475 L2218.93 999.345 L2218.7 999.107 L2218.48 999.356 L2218.25 999.301 L2218.02 999.219 L2217.79 999.304 L2217.56 999.593 L2217.34 999.586 L2217.11 999.716 L2216.88 999.617 L2216.65 999.597 L2216.43 999.744 L2216.2 999.574 L2215.97 999.495 L2215.74 999.733 L2215.51 1000.31 L2215.29 1000.04 L2215.06 1000.27 L2214.83 1000.1 L2214.6 1000.05 L2214.38 1000.6 L2214.15 1000.6 L2213.92 1000.13 L2213.69 999.812 L2213.46 1000.01 L2213.24 1000.49 L2213.01 1000.83 L2212.78 1000.58 L2212.55 1001.18 L2212.33 1001.34 L2212.1 1001.5 L2211.87 1001.98 L2211.64 1001.71 L2211.41 1002 L2211.19 1001.65 L2210.96 1001.6 L2210.73 1001.56 L2210.5 1001.4 L2210.28 1001.57 L2210.05 1001.86 L2209.82 1001.55 L2209.59 1001.79 L2209.36 1001.92 L2209.14 1001.72 L2208.91 1001.38 L2208.68 1001.72 L2208.45 1001.56 L2208.23 1001.79 L2208 1002.07 L2207.77 1002.42 L2207.54 1002.26 L2207.31 1002.02 L2207.09 1002.06 L2206.86 1001.86 L2206.63 1001.99 L2206.4 1001.93 L2206.18 1001.66 L2205.95 1001.4 L2205.72 1001.12 L2205.49 1001.54 L2205.26 1001.42 L2205.04 1001.87 L2204.81 1001.94 L2204.58 1001.95 L2204.35 1002.14 L2204.13 1001.94 L2203.9 1001.88 L2203.67 1001.77 L2203.44 1001.76 L2203.21 1002.19 L2202.99 1002.26 L2202.76 1002.65 L2202.53 1002.43 L2202.3 1002.63 L2202.08 1002.3 L2201.85 1002.11 L2201.62 1002.51 L2201.39 1002.1 L2201.17 1001.84 L2200.94 1001.48 L2200.71 1001.53 L2200.48 1001.54 L2200.25 1001.47 L2200.03 1001.76 L2199.8 1001.78 L2199.57 1001.8 L2199.34 1001.44 L2199.12 1001.09 L2198.89 1000.97 L2198.66 1001.19 L2198.43 1001.14 L2198.2 1001.02 L2197.98 1001.23 L2197.75 1001.18 L2197.52 1001.06 L2197.29 1000.53 L2197.07 1000.14 L2196.84 1000.44 L2196.61 1000.08 L2196.38 999.816 L2196.15 999.908 L2195.93 999.906 L2195.7 999.703 L2195.47 999.827 L2195.24 1000.09 L2195.02 1000.6 L2194.79 1000.32 L2194.56 1000.56 L2194.33 1000.38 L2194.1 1000.62 L2193.88 1000.74 L2193.65 1000.53 L2193.42 1000.31 L2193.19 1000.31 L2192.97 1000.04 L2192.74 999.871 L2192.51 999.77 L2192.28 1000.57 L2192.05 1000.33 L2191.83 1000.27 L2191.6 999.854 L2191.37 999.767 L2191.14 999.595 L2190.92 999.754 L2190.69 999.861 L2190.46 999.582 L2190.23 1000.07 L2190 999.722 L2189.78 999.313 L2189.55 999.609 L2189.32 999.251 L2189.09 999.908 L2188.87 999.712 L2188.64 1000.18 L2188.41 1000.13 L2188.18 1000.44 L2187.95 1000.27 L2187.73 1000.53 L2187.5 1000.76 L2187.27 1000.53 L2187.04 1000.62 L2186.82 1001.22 L2186.59 1000.65 L2186.36 1001.14 L2186.13 1001.51 L2185.9 1001.29 L2185.68 1001.41 L2185.45 1000.99 L2185.22 1000.76 L2184.99 1000.71 L2184.77 1000.93 L2184.54 1000.31 L2184.31 999.936 L2184.08 999.523 L2183.85 999.351 L2183.63 999.648 L2183.4 999.967 L2183.17 1000.93 L2182.94 1000.98 L2182.72 1001.23 L2182.49 1001.02 L2182.26 1001.48 L2182.03 1001.19 L2181.8 1001.07 L2181.58 1001.26 L2181.35 1001.18 L2181.12 1000.6 L2180.89 1000.25 L2180.67 999.511 L2180.44 999.24 L2180.21 999.314 L2179.98 999.148 L2179.75 999.668 L2179.53 999.775 L2179.3 999.912 L2179.07 1000.09 L2178.84 1000.17 L2178.62 1000.02 L2178.39 1000.16 L2178.16 1000.42 L2177.93 1000.53 L2177.7 1000.66 L2177.48 1001.05 L2177.25 1000.83 L2177.02 1000.7 L2176.79 1000.93 L2176.57 1000.66 L2176.34 1001.03 L2176.11 1001.42 L2175.88 1001.91 L2175.65 1001.68 L2175.43 1001.01 L2175.2 1000.71 L2174.97 1000.79 L2174.74 1000.57 L2174.52 1000.14 L2174.29 999.601 L2174.06 999.531 L2173.83 999.616 L2173.6 999.385 L2173.38 999.316 L2173.15 999.09 L2172.92 998.867 L2172.69 998.758 L2172.47 998.774 L2172.24 999.024 L2172.01 998.567 L2171.78 998.272 L2171.55 998.157 L2171.33 997.832 L2171.1 997.584 L2170.87 997.745 L2170.64 997.639 L2170.42 997.267 L2170.19 996.946 L2169.96 996.861 L2169.73 996.565 L2169.5 996.549 L2169.28 996.78 L2169.05 996.896 L2168.82 996.359 L2168.59 996.294 L2168.37 997.559 L2168.14 998.267 L2167.91 997.711 L2167.68 997.646 L2167.45 997.214 L2167.23 997.467 L2167 997.079 L2166.77 997.345 L2166.54 997.516 L2166.32 997.896 L2166.09 997.969 L2165.86 998.027 L2165.63 998.196 L2165.41 997.94 L2165.18 998.278 L2164.95 997.871 L2164.72 997.737 L2164.49 998.078 L2164.27 998.079 L2164.04 997.608 L2163.81 997.881 L2163.58 997.994 L2163.36 997.881 L2163.13 998.151 L2162.9 997.875 L2162.67 997.721 L2162.44 997.59 L2162.22 998.201 L2161.99 998.59 L2161.76 998.278 L2161.53 998.5 L2161.31 999.307 L2161.08 999.466 L2160.85 999.404 L2160.62 999.889 L2160.39 999.39 L2160.17 999.536 L2159.94 999.728 L2159.71 999.634 L2159.48 999.464 L2159.26 999.784 L2159.03 1000.04 L2158.8 999.945 L2158.57 1000.31 L2158.34 1000.11 L2158.12 1000.71 L2157.89 1001.14 L2157.66 1000.77 L2157.43 1001.06 L2157.21 1001.3 L2156.98 1001.44 L2156.75 1001.51 L2156.52 1001.39 L2156.29 1000.84 L2156.07 1000.38 L2155.84 1000.14 L2155.61 999.997 L2155.38 1000.07 L2155.16 1000.21 L2154.93 999.9 L2154.7 1000.03 L2154.47 999.532 L2154.24 999.928 L2154.02 999.761 L2153.79 999.922 L2153.56 1000.08 L2153.33 999.736 L2153.11 999.596 L2152.88 999.514 L2152.65 999.456 L2152.42 999.267 L2152.19 999.307 L2151.97 999.128 L2151.74 999.274 L2151.51 998.864 L2151.28 998.954 L2151.06 998.772 L2150.83 998.574 L2150.6 998.661 L2150.37 999.321 L2150.14 999.547 L2149.92 1000.05 L2149.69 1000.04 L2149.46 1000.51 L2149.23 1001.42 L2149.01 1001.74 L2148.78 1001.59 L2148.55 1002.01 L2148.32 1002.07 L2148.09 1001.63 L2147.87 1001.84 L2147.64 1002.27 L2147.41 1002.33 L2147.18 1002.59 L2146.96 1002.98 L2146.73 1003.33 L2146.5 1003.63 L2146.27 1004.12 L2146.04 1003.61 L2145.82 1003.6 L2145.59 1003.75 L2145.36 1004.35 L2145.13 1005.06 L2144.91 1004.7 L2144.68 1004.18 L2144.45 1004.43 L2144.22 1004.21 L2143.99 1003.89 L2143.77 1005.39 L2143.54 1004.79 L2143.31 1004.67 L2143.08 1004.59 L2142.86 1005.84 L2142.63 1006.16 L2142.4 1006.54 L2142.17 1006.62 L2141.94 1006.67 L2141.72 1006.33 L2141.49 1006.15 L2141.26 1006.13 L2141.03 1005.82 L2140.81 1006.51 L2140.58 1006.4 L2140.35 1006.46 L2140.12 1006.26 L2139.89 1006.23 L2139.67 1005.78 L2139.44 1006.6 L2139.21 1006.7 L2138.98 1005.92 L2138.76 1005.99 L2138.53 1005.89 L2138.3 1005.21 L2138.07 1005.7 L2137.84 1006.35 L2137.62 1006.8 L2137.39 1006.77 L2137.16 1007.32 L2136.93 1007.21 L2136.71 1007.12 L2136.48 1007.02 L2136.25 1006.95 L2136.02 1006.68 L2135.79 1006.35 L2135.57 1006.43 L2135.34 1006.31 L2135.11 1006.55 L2134.88 1006.36 L2134.66 1006.21 L2134.43 1006.25 L2134.2 1006.38 L2133.97 1006.4 L2133.74 1006.33 L2133.52 1006.92 L2133.29 1007.19 L2133.06 1007.69 L2132.83 1007.32 L2132.61 1006.95 L2132.38 1006.23 L2132.15 1005.57 L2131.92 1005.33 L2131.69 1005.02 L2131.47 1005.21 L2131.24 1005.87 L2131.01 1005.86 L2130.78 1005.98 L2130.56 1006.02 L2130.33 1006.1 L2130.1 1006.12 L2129.87 1005.61 L2129.65 1006.1 L2129.42 1006.19 L2129.19 1006.57 L2128.96 1006.75 L2128.73 1006.77 L2128.51 1006.4 L2128.28 1006.57 L2128.05 1006.7 L2127.82 1006.31 L2127.6 1006.17 L2127.37 1005.96 L2127.14 1005.87 L2126.91 1006.12 L2126.68 1006.88 L2126.46 1006.8 L2126.23 1006.8 L2126 1006.54 L2125.77 1006.61 L2125.55 1006.02 L2125.32 1006.41 L2125.09 1006.15 L2124.86 1005.77 L2124.63 1005.94 L2124.41 1005.8 L2124.18 1006 L2123.95 1005.82 L2123.72 1005.31 L2123.5 1005.29 L2123.27 1005.47 L2123.04 1005.23 L2122.81 1004.91 L2122.58 1004.54 L2122.36 1004.34 L2122.13 1004.07 L2121.9 1004.47 L2121.67 1005.15 L2121.45 1005.42 L2121.22 1005.49 L2120.99 1006.01 L2120.76 1006.52 L2120.53 1006.07 L2120.31 1006.36 L2120.08 1005.98 L2119.85 1005.84 L2119.62 1005.86 L2119.4 1005.84 L2119.17 1005.75 L2118.94 1005.59 L2118.71 1005.83 L2118.48 1005.34 L2118.26 1005.13 L2118.03 1006.11 L2117.8 1006.06 L2117.57 1006.09 L2117.35 1006.1 L2117.12 1006.48 L2116.89 1006.31 L2116.66 1005.97 L2116.43 1005.29 L2116.21 1005.25 L2115.98 1005.07 L2115.75 1004.97 L2115.52 1004.72 L2115.3 1004.95 L2115.07 1004.51 L2114.84 1004.2 L2114.61 1004.9 L2114.38 1004.85 L2114.16 1004.33 L2113.93 1004.29 L2113.7 1004.79 L2113.47 1004.7 L2113.25 1004.65 L2113.02 1004.42 L2112.79 1003.96 L2112.56 1003.69 L2112.33 1003.26 L2112.11 1003.48 L2111.88 1002.94 L2111.65 1002.88 L2111.42 1002.81 L2111.2 1002.91 L2110.97 1002.65 L2110.74 1002.97 L2110.51 1002.63 L2110.28 1003.19 L2110.06 1002.91 L2109.83 1002.94 L2109.6 1002.97 L2109.37 1002.71 L2109.15 1002.61 L2108.92 1002.28 L2108.69 1002.2 L2108.46 1001.98 L2108.23 1001.81 L2108.01 1001.69 L2107.78 1001.36 L2107.55 1001.09 L2107.32 1000.89 L2107.1 1000.66 L2106.87 1000.56 L2106.64 1000.55 L2106.41 1000.08 L2106.18 999.967 L2105.96 1000.33 L2105.73 1000.14 L2105.5 1000.23 L2105.27 999.648 L2105.05 999.733 L2104.82 1000.24 L2104.59 1000.2 L2104.36 1000.16 L2104.13 1000.02 L2103.91 999.48 L2103.68 999.42 L2103.45 999.757 L2103.22 999.731 L2103 1000.28 L2102.77 999.731 L2102.54 999.46 L2102.31 999.135 L2102.08 999.243 L2101.86 999.375 L2101.63 998.914 L2101.4 998.711 L2101.17 999.191 L2100.95 999.105 L2100.72 999.328 L2100.49 1000.15 L2100.26 999.65 L2100.03 1000.42 L2099.81 1000.52 L2099.58 1000.3 L2099.35 999.753 L2099.12 999.498 L2098.9 1000.24 L2098.67 999.989 L2098.44 1000.11 L2098.21 999.982 L2097.98 999.595 L2097.76 999.184 L2097.53 999.525 L2097.3 999.172 L2097.07 999.312 L2096.85 998.614 L2096.62 999.021 L2096.39 998.485 L2096.16 998.212 L2095.93 998.435 L2095.71 998.99 L2095.48 999.112 L2095.25 999.506 L2095.02 999.552 L2094.8 999.993 L2094.57 1000.18 L2094.34 1000.41 L2094.11 1000.38 L2093.89 999.976 L2093.66 1000.27 L2093.43 1000.78 L2093.2 1001.42 L2092.97 1001.84 L2092.75 1002 L2092.52 1002.01 L2092.29 1002.42 L2092.06 1002.27 L2091.84 1002.41 L2091.61 1002.37 L2091.38 1002.67 L2091.15 1003.1 L2090.92 1002.62 L2090.7 1002.22 L2090.47 1002.08 L2090.24 1001.87 L2090.01 1001.63 L2089.79 1001.72 L2089.56 1001.59 L2089.33 1001.57 L2089.1 1001.55 L2088.87 1001.35 L2088.65 1001.75 L2088.42 1001.56 L2088.19 1001.46 L2087.96 1001.43 L2087.74 1001.48 L2087.51 1002.16 L2087.28 1002.27 L2087.05 1001.78 L2086.82 1001.51 L2086.6 1001.15 L2086.37 1001.25 L2086.14 1001.04 L2085.91 1000.46 L2085.69 1000.24 L2085.46 1000.56 L2085.23 1000.14 L2085 999.916 L2084.77 1000.35 L2084.55 1000.43 L2084.32 1000.23 L2084.09 999.941 L2083.86 1000.06 L2083.64 999.832 L2083.41 1000 L2083.18 999.975 L2082.95 999.906 L2082.72 999.887 L2082.5 1000.5 L2082.27 1000.59 L2082.04 1000.95 L2081.81 1001.52 L2081.59 1001.59 L2081.36 1001.47 L2081.13 1001.19 L2080.9 1000.55 L2080.67 999.848 L2080.45 999.539 L2080.22 999.242 L2079.99 999.241 L2079.76 999.205 L2079.54 998.875 L2079.31 998.358 L2079.08 997.999 L2078.85 998.244 L2078.62 997.718 L2078.4 997.617 L2078.17 997.65 L2077.94 997.313 L2077.71 997.407 L2077.49 997.987 L2077.26 998.092 L2077.03 998.565 L2076.8 998.524 L2076.57 998.736 L2076.35 999.004 L2076.12 999.116 L2075.89 999.351 L2075.66 998.635 L2075.44 998.245 L2075.21 998.093 L2074.98 998.164 L2074.75 998.171 L2074.52 998.527 L2074.3 998.184 L2074.07 998.693 L2073.84 998.659 L2073.61 998.911 L2073.39 998.98 L2073.16 999.111 L2072.93 999.661 L2072.7 999.689 L2072.47 1000.17 L2072.25 1000.16 L2072.02 999.976 L2071.79 999.83 L2071.56 1000.19 L2071.34 1000.34 L2071.11 1000.26 L2070.88 999.872 L2070.65 999.605 L2070.42 999.697 L2070.2 999.724 L2069.97 1000.53 L2069.74 1000.83 L2069.51 1000.73 L2069.29 1000.24 L2069.06 1000.1 L2068.83 999.829 L2068.6 999.456 L2068.37 1000.34 L2068.15 1000.27 L2067.92 999.903 L2067.69 999.531 L2067.46 999.679 L2067.24 1000.03 L2067.01 1000.34 L2066.78 999.779 L2066.55 999.794 L2066.32 999.343 L2066.1 999.026 L2065.87 998.693 L2065.64 998.906 L2065.41 998.568 L2065.19 998.72 L2064.96 998.499 L2064.73 998.805 L2064.5 998.859 L2064.27 998.94 L2064.05 999.768 L2063.82 999.89 L2063.59 999.377 L2063.36 999.411 L2063.14 999.777 L2062.91 999.957 L2062.68 999.718 L2062.45 999.741 L2062.22 999.501 L2062 999.01 L2061.77 999.661 L2061.54 999.919 L2061.31 1000.55 L2061.09 1000.73 L2060.86 1000.65 L2060.63 1000.08 L2060.4 999.788 L2060.17 1000.09 L2059.95 999.609 L2059.72 999.629 L2059.49 999.209 L2059.26 999.588 L2059.04 999.14 L2058.81 998.759 L2058.58 998.292 L2058.35 998.213 L2058.12 998.777 L2057.9 998.537 L2057.67 998.576 L2057.44 998.42 L2057.21 998.284 L2056.99 998.456 L2056.76 998.547 L2056.53 998.783 L2056.3 998.892 L2056.08 998.787 L2055.85 999.117 L2055.62 998.933 L2055.39 999.12 L2055.16 999.388 L2054.94 999.229 L2054.71 998.703 L2054.48 998.589 L2054.25 998.774 L2054.03 999.193 L2053.8 999.14 L2053.57 999.341 L2053.34 999.155 L2053.11 999.011 L2052.89 998.65 L2052.66 998.8 L2052.43 998.615 L2052.2 998.415 L2051.98 998.341 L2051.75 998.616 L2051.52 998.272 L2051.29 998.958 L2051.06 998.909 L2050.84 998.53 L2050.61 997.951 L2050.38 997.658 L2050.15 997.552 L2049.93 997.741 L2049.7 997.898 L2049.47 997.888 L2049.24 998.168 L2049.01 998.438 L2048.79 998.622 L2048.56 998.72 L2048.33 1000.47 L2048.1 1000.12 L2047.88 999.478 L2047.65 999.219 L2047.42 999.226 L2047.19 999.27 L2046.96 999.222 L2046.74 999.295 L2046.51 999.118 L2046.28 998.753 L2046.05 998.721 L2045.83 998.352 L2045.6 998.51 L2045.37 998.165 L2045.14 998.781 L2044.91 999.006 L2044.69 999.217 L2044.46 999.986 L2044.23 999.717 L2044 999.991 L2043.78 1000.16 L2043.55 1000.08 L2043.32 999.823 L2043.09 1000.2 L2042.86 999.889 L2042.64 999.458 L2042.41 999.416 L2042.18 999.402 L2041.95 999.592 L2041.73 999.074 L2041.5 998.671 L2041.27 998.542 L2041.04 998.62 L2040.81 999.007 L2040.59 998.851 L2040.36 998.34 L2040.13 998.751 L2039.9 999.261 L2039.68 999.065 L2039.45 999.156 L2039.22 998.77 L2038.99 998.402 L2038.76 998.411 L2038.54 998.569 L2038.31 998.122 L2038.08 997.909 L2037.85 997.252 L2037.63 996.963 L2037.4 996.798 L2037.17 996.734 L2036.94 996.718 L2036.71 996.596 L2036.49 996.476 L2036.26 996.732 L2036.03 996.974 L2035.8 996.864 L2035.58 996.797 L2035.35 996.247 L2035.12 995.956 L2034.89 995.929 L2034.66 995.564 L2034.44 995.464 L2034.21 995.415 L2033.98 995.536 L2033.75 995.39 L2033.53 995.422 L2033.3 995.441 L2033.07 995.485 L2032.84 995.546 L2032.61 994.947 L2032.39 994.337 L2032.16 994.343 L2031.93 993.955 L2031.7 993.525 L2031.48 993.56 L2031.25 992.89 L2031.02 993.009 L2030.79 993.442 L2030.56 993.367 L2030.34 993.056 L2030.11 993.752 L2029.88 993.971 L2029.65 993.512 L2029.43 993.065 L2029.2 993.073 L2028.97 992.978 L2028.74 992.809 L2028.51 992.83 L2028.29 992.743 L2028.06 993.189 L2027.83 993.36 L2027.6 993.15 L2027.38 992.744 L2027.15 993.048 L2026.92 992.434 L2026.69 992.188 L2026.46 991.863 L2026.24 992.099 L2026.01 992.209 L2025.78 992.559 L2025.55 992.392 L2025.33 992.097 L2025.1 991.409 L2024.87 991.235 L2024.64 991.527 L2024.41 990.949 L2024.19 990.868 L2023.96 990.934 L2023.73 990.924 L2023.5 990.662 L2023.28 990.72 L2023.05 990.411 L2022.82 990.804 L2022.59 990.924 L2022.36 990.916 L2022.14 990.701 L2021.91 990.584 L2021.68 990.819 L2021.45 990.265 L2021.23 989.729 L2021 989.495 L2020.77 989.998 L2020.54 989.805 L2020.32 989.279 L2020.09 989.517 L2019.86 990.158 L2019.63 989.965 L2019.4 990.842 L2019.18 990.52 L2018.95 990.958 L2018.72 990.95 L2018.49 991.573 L2018.27 991.205 L2018.04 992.003 L2017.81 992.56 L2017.58 992.938 L2017.35 992.291 L2017.13 992.783 L2016.9 992.295 L2016.67 991.595 L2016.44 992.055 L2016.22 991.741 L2015.99 991.781 L2015.76 991.438 L2015.53 991.829 L2015.3 992.048 L2015.08 991.89 L2014.85 991.847 L2014.62 991.829 L2014.39 992.234 L2014.17 992.522 L2013.94 992.161 L2013.71 991.979 L2013.48 991.758 L2013.25 991.717 L2013.03 991.511 L2012.8 991.874 L2012.57 992.303 L2012.34 992.282 L2012.12 992.65 L2011.89 992.252 L2011.66 992.291 L2011.43 991.931 L2011.2 991.858 L2010.98 991.682 L2010.75 991.508 L2010.52 991.412 L2010.29 991.273 L2010.07 991.361 L2009.84 991.433 L2009.61 991.215 L2009.38 992.169 L2009.15 991.97 L2008.93 991.548 L2008.7 992.244 L2008.47 992.39 L2008.24 992.254 L2008.02 992.001 L2007.79 992.202 L2007.56 991.771 L2007.33 991.517 L2007.1 992.131 L2006.88 992.438 L2006.65 992.588 L2006.42 992.816 L2006.19 993.12 L2005.97 992.496 L2005.74 991.979 L2005.51 992.14 L2005.28 992.293 L2005.05 992.243 L2004.83 991.908 L2004.6 991.934 L2004.37 991.937 L2004.14 992.034 L2003.92 992.126 L2003.69 991.69 L2003.46 991.785 L2003.23 992.017 L2003 991.897 L2002.78 991.473 L2002.55 991.34 L2002.32 991.794 L2002.09 991.366 L2001.87 991.324 L2001.64 991.208 L2001.41 991.322 L2001.18 991.284 L2000.95 991.34 L2000.73 991.664 L2000.5 992.19 L2000.27 992.399 L2000.04 992.66 L1999.82 992.739 L1999.59 993.096 L1999.36 994.028 L1999.13 993.583 L1998.9 993.858 L1998.68 993.601 L1998.45 993.274 L1998.22 993.694 L1997.99 994.211 L1997.77 994.029 L1997.54 994.031 L1997.31 994.068 L1997.08 994.209 L1996.85 993.933 L1996.63 993.193 L1996.4 992.955 L1996.17 993.852 L1995.94 994.163 L1995.72 994.255 L1995.49 994.37 L1995.26 994.111 L1995.03 994.205 L1994.8 994.101 L1994.58 994.249 L1994.35 994.307 L1994.12 994.604 L1993.89 994.428 L1993.67 994.675 L1993.44 995.076 L1993.21 995.295 L1992.98 995.879 L1992.75 996.21 L1992.53 996.221 L1992.3 996.496 L1992.07 996.465 L1991.84 996.25 L1991.62 996.542 L1991.39 996.142 L1991.16 995.89 L1990.93 995.515 L1990.7 995.52 L1990.48 995.446 L1990.25 995.102 L1990.02 995.009 L1989.79 995.51 L1989.57 995.449 L1989.34 995.131 L1989.11 995.52 L1988.88 995.201 L1988.65 995.2 L1988.43 995.01 L1988.2 994.758 L1987.97 994.44 L1987.74 994.801 L1987.52 994.417 L1987.29 994.218 L1987.06 994.547 L1986.83 994.532 L1986.6 994.036 L1986.38 994.23 L1986.15 994.073 L1985.92 993.897 L1985.69 994.178 L1985.47 993.812 L1985.24 993.846 L1985.01 993.994 L1984.78 993.334 L1984.56 993.968 L1984.33 994.28 L1984.1 994.611 L1983.87 994.963 L1983.64 994.87 L1983.42 994.434 L1983.19 993.888 L1982.96 994.438 L1982.73 994.509 L1982.51 994.503 L1982.28 994.497 L1982.05 994.239 L1981.82 994.459 L1981.59 994.193 L1981.37 993.642 L1981.14 993.856 L1980.91 993.477 L1980.68 993.574 L1980.46 993.375 L1980.23 992.497 L1980 992.212 L1979.77 992.493 L1979.54 991.998 L1979.32 991.639 L1979.09 991.572 L1978.86 991.166 L1978.63 991.027 L1978.41 990.648 L1978.18 990.353 L1977.95 989.96 L1977.72 989.553 L1977.49 989.444 L1977.27 988.897 L1977.04 989.024 L1976.81 989.417 L1976.58 989.361 L1976.36 989.155 L1976.13 989.37 L1975.9 989.207 L1975.67 989.38 L1975.44 989.351 L1975.22 989.416 L1974.99 989.724 L1974.76 989.1 L1974.53 988.785 L1974.31 988.689 L1974.08 988.533 L1973.85 988.507 L1973.62 988.929 L1973.39 988.628 L1973.17 988.381 L1972.94 988.409 L1972.71 987.913 L1972.48 988.428 L1972.26 988.552 L1972.03 988.753 L1971.8 988.697 L1971.57 988.188 L1971.34 988.609 L1971.12 988.264 L1970.89 988.058 L1970.66 987.684 L1970.43 987.915 L1970.21 987.613 L1969.98 987.442 L1969.75 987.39 L1969.52 987.586 L1969.29 987.624 L1969.07 987.454 L1968.84 987.571 L1968.61 987.27 L1968.38 986.798 L1968.16 986.393 L1967.93 986.285 L1967.7 985.965 L1967.47 986.018 L1967.24 985.615 L1967.02 985.382 L1966.79 985.519 L1966.56 985.218 L1966.33 984.612 L1966.11 984.683 L1965.88 984.625 L1965.65 984.492 L1965.42 984.527 L1965.19 984.724 L1964.97 984.368 L1964.74 984.448 L1964.51 984.382 L1964.28 984.451 L1964.06 984.411 L1963.83 984.054 L1963.6 983.816 L1963.37 984.051 L1963.14 984.569 L1962.92 984.88 L1962.69 984.281 L1962.46 984.258 L1962.23 984.359 L1962.01 984.934 L1961.78 984.892 L1961.55 984.973 L1961.32 985.242 L1961.09 985.13 L1960.87 984.682 L1960.64 985.537 L1960.41 985.568 L1960.18 985.44 L1959.96 986.278 L1959.73 986.035 L1959.5 985.937 L1959.27 986.203 L1959.04 986.129 L1958.82 985.977 L1958.59 985.453 L1958.36 984.984 L1958.13 985.007 L1957.91 985.707 L1957.68 986.243 L1957.45 985.944 L1957.22 986.043 L1956.99 986.354 L1956.77 986.841 L1956.54 986.739 L1956.31 986.637 L1956.08 986.442 L1955.86 986.337 L1955.63 986.374 L1955.4 985.907 L1955.17 986.333 L1954.94 987.191 L1954.72 987.382 L1954.49 988.021 L1954.26 987.721 L1954.03 987.674 L1953.81 987.727 L1953.58 988.253 L1953.35 988.558 L1953.12 988.908 L1952.89 988.356 L1952.67 987.795 L1952.44 987.628 L1952.21 987.528 L1951.98 987.542 L1951.76 986.957 L1951.53 987.256 L1951.3 987.2 L1951.07 987.026 L1950.84 987.245 L1950.62 987.732 L1950.39 987.615 L1950.16 987.579 L1949.93 988.075 L1949.71 987.626 L1949.48 987.563 L1949.25 987.828 L1949.02 987.894 L1948.79 987.735 L1948.57 987.245 L1948.34 986.782 L1948.11 987.785 L1947.88 987.771 L1947.66 987.734 L1947.43 988.221 L1947.2 988.007 L1946.97 987.809 L1946.75 987.748 L1946.52 988.017 L1946.29 988.463 L1946.06 988.579 L1945.83 988.543 L1945.61 988.191 L1945.38 988.066 L1945.15 988.744 L1944.92 988.025 L1944.7 988.046 L1944.47 987.796 L1944.24 987.792 L1944.01 988.493 L1943.78 988.588 L1943.56 988.442 L1943.33 988.547 L1943.1 988.232 L1942.87 987.801 L1942.65 987.712 L1942.42 988.387 L1942.19 989.267 L1941.96 990.001 L1941.73 989.928 L1941.51 989.944 L1941.28 990.16 L1941.05 989.973 L1940.82 989.911 L1940.6 990.203 L1940.37 989.938 L1940.14 989.978 L1939.91 989.828 L1939.68 989.586 L1939.46 989.572 L1939.23 989.366 L1939 989.399 L1938.77 989.273 L1938.55 988.776 L1938.32 989.105 L1938.09 988.686 L1937.86 988.237 L1937.63 988.343 L1937.41 988.154 L1937.18 988.155 L1936.95 987.944 L1936.72 987.785 L1936.5 988.44 L1936.27 988.137 L1936.04 988.054 L1935.81 988.189 L1935.58 988.828 L1935.36 988.581 L1935.13 988.487 L1934.9 988.111 L1934.67 988.132 L1934.45 989.651 L1934.22 989.645 L1933.99 988.987 L1933.76 988.8 L1933.53 989.03 L1933.31 989.073 L1933.08 988.919 L1932.85 988.909 L1932.62 988.662 L1932.4 989.46 L1932.17 990.115 L1931.94 989.959 L1931.71 989.846 L1931.48 989.43 L1931.26 989.707 L1931.03 989.486 L1930.8 989.451 L1930.57 989.979 L1930.35 990.011 L1930.12 990.614 L1929.89 990.166 L1929.66 989.814 L1929.43 989.237 L1929.21 989.521 L1928.98 990.464 L1928.75 990.658 L1928.52 990.682 L1928.3 990.868 L1928.07 990.774 L1927.84 990.55 L1927.61 990.638 L1927.38 990.874 L1927.16 990.829 L1926.93 990.428 L1926.7 990.199 L1926.47 990.645 L1926.25 990.284 L1926.02 990.207 L1925.79 990.412 L1925.56 990.073 L1925.33 990.618 L1925.11 990.784 L1924.88 990.34 L1924.65 990.58 L1924.42 990.5 L1924.2 990.371 L1923.97 990.066 L1923.74 989.656 L1923.51 989.739 L1923.28 990.278 L1923.06 990.31 L1922.83 989.986 L1922.6 990.099 L1922.37 990.19 L1922.15 990.32 L1921.92 990.473 L1921.69 990.162 L1921.46 990.002 L1921.23 990.35 L1921.01 990.341 L1920.78 990.359 L1920.55 990.971 L1920.32 990.552 L1920.1 990.616 L1919.87 990.879 L1919.64 991.116 L1919.41 992.142 L1919.18 992.555 L1918.96 992.712 L1918.73 992.439 L1918.5 992.625 L1918.27 992.178 L1918.05 992.365 L1917.82 991.944 L1917.59 991.662 L1917.36 991.684 L1917.13 992.514 L1916.91 992.82 L1916.68 992.492 L1916.45 992.892 L1916.22 992.583 L1916 993.357 L1915.77 994.034 L1915.54 994.126 L1915.31 993.752 L1915.08 993.967 L1914.86 993.554 L1914.63 992.812 L1914.4 992.332 L1914.17 991.704 L1913.95 991.888 L1913.72 991.839 L1913.49 991.406 L1913.26 991.076 L1913.03 991.09 L1912.81 990.997 L1912.58 991.028 L1912.35 991.313 L1912.12 992.003 L1911.9 991.939 L1911.67 991.674 L1911.44 991.623 L1911.21 991.319 L1910.99 991.538 L1910.76 991.854 L1910.53 991.722 L1910.3 991.59 L1910.07 992.403 L1909.85 991.901 L1909.62 991.958 L1909.39 992.801 L1909.16 993.158 L1908.94 992.157 L1908.71 992.043 L1908.48 991.919 L1908.25 992.132 L1908.02 991.665 L1907.8 991.023 L1907.57 991.031 L1907.34 991.846 L1907.11 991.682 L1906.89 991.23 L1906.66 991.306 L1906.43 991.307 L1906.2 990.881 L1905.97 990.93 L1905.75 990.534 L1905.52 990.012 L1905.29 990.563 L1905.06 990.681 L1904.84 990.598 L1904.61 991.288 L1904.38 991.328 L1904.15 991.374 L1903.92 991.778 L1903.7 991.979 L1903.47 991.66 L1903.24 991.556 L1903.01 992.625 L1902.79 992.857 L1902.56 993.288 L1902.33 992.87 L1902.1 992.443 L1901.87 992.36 L1901.65 992.031 L1901.42 991.867 L1901.19 991.741 L1900.96 991.182 L1900.74 990.9 L1900.51 990.923 L1900.28 990.672 L1900.05 990.417 L1899.82 990.378 L1899.6 990.538 L1899.37 990.897 L1899.14 990.848 L1898.91 990.743 L1898.69 991.156 L1898.46 990.919 L1898.23 990.631 L1898 990.015 L1897.77 990.898 L1897.55 991.005 L1897.32 991.001 L1897.09 990.579 L1896.86 990.197 L1896.64 989.551 L1896.41 989.766 L1896.18 989.518 L1895.95 989.405 L1895.72 990.478 L1895.5 990.073 L1895.27 990.556 L1895.04 990.475 L1894.81 990.317 L1894.59 990.303 L1894.36 990.948 L1894.13 991.006 L1893.9 991.531 L1893.67 992.072 L1893.45 991.425 L1893.22 991.135 L1892.99 990.882 L1892.76 991.007 L1892.54 991.003 L1892.31 990.805 L1892.08 990.715 L1891.85 990.133 L1891.62 989.606 L1891.4 989.874 L1891.17 989.969 L1890.94 989.519 L1890.71 989.461 L1890.49 990.237 L1890.26 990.284 L1890.03 990.095 L1889.8 989.987 L1889.57 989.941 L1889.35 989.668 L1889.12 989.525 L1888.89 989.447 L1888.66 989.53 L1888.44 990.11 L1888.21 990.45 L1887.98 989.947 L1887.75 990.906 L1887.52 990.506 L1887.3 990.046 L1887.07 989.843 L1886.84 989.929 L1886.61 990.122 L1886.39 989.712 L1886.16 989.773 L1885.93 989.011 L1885.7 989.448 L1885.47 990.225 L1885.25 990.013 L1885.02 989.545 L1884.79 989.257 L1884.56 989.033 L1884.34 989.055 L1884.11 989.002 L1883.88 989.013 L1883.65 988.69 L1883.42 988.761 L1883.2 988.515 L1882.97 988.027 L1882.74 988.557 L1882.51 988.545 L1882.29 988.17 L1882.06 987.547 L1881.83 987.296 L1881.6 987.176 L1881.37 987.028 L1881.15 986.765 L1880.92 986.484 L1880.69 986.677 L1880.46 986.549 L1880.24 986.414 L1880.01 986.285 L1879.78 985.923 L1879.55 985.92 L1879.32 985.986 L1879.1 985.635 L1878.87 985.866 L1878.64 985.559 L1878.41 985.524 L1878.19 986.653 L1877.96 986.77 L1877.73 986.467 L1877.5 986.252 L1877.27 985.673 L1877.05 985.985 L1876.82 985.906 L1876.59 986.003 L1876.36 986.147 L1876.14 986.446 L1875.91 986.518 L1875.68 986.316 L1875.45 986.174 L1875.23 986.231 L1875 985.998 L1874.77 985.938 L1874.54 986.146 L1874.31 986.386 L1874.09 986.507 L1873.86 986.835 L1873.63 986.249 L1873.4 985.916 L1873.18 985.787 L1872.95 985.023 L1872.72 985.373 L1872.49 985.806 L1872.26 986.051 L1872.04 986.427 L1871.81 986.135 L1871.58 986.097 L1871.35 986.276 L1871.13 985.904 L1870.9 986.148 L1870.67 985.927 L1870.44 985.938 L1870.21 985.607 L1869.99 985.789 L1869.76 986.232 L1869.53 986.695 L1869.3 986.431 L1869.08 985.659 L1868.85 985.315 L1868.62 984.789 L1868.39 984.828 L1868.16 984.179 L1867.94 984.923 L1867.71 984.738 L1867.48 984.342 L1867.25 984.004 L1867.03 984.07 L1866.8 983.53 L1866.57 983.143 L1866.34 982.802 L1866.11 982.472 L1865.89 982.399 L1865.66 981.905 L1865.43 982.055 L1865.2 981.584 L1864.98 981.472 L1864.75 981.699 L1864.52 981.609 L1864.29 981.861 L1864.06 981.491 L1863.84 981.412 L1863.61 981.931 L1863.38 981.949 L1863.15 981.946 L1862.93 981.933 L1862.7 982.215 L1862.47 982.027 L1862.24 981.807 L1862.01 981.99 L1861.79 981.697 L1861.56 982.312 L1861.33 981.955 L1861.1 982.06 L1860.88 982.545 L1860.65 981.955 L1860.42 981.701 L1860.19 981.507 L1859.96 981.13 L1859.74 981.538 L1859.51 981.119 L1859.28 981.168 L1859.05 981.164 L1858.83 981.279 L1858.6 981.949 L1858.37 981.608 L1858.14 981.206 L1857.91 981.607 L1857.69 981.71 L1857.46 981.455 L1857.23 982.013 L1857 982.418 L1856.78 983.508 L1856.55 983.62 L1856.32 983.897 L1856.09 983.914 L1855.86 983.736 L1855.64 982.904 L1855.41 982.488 L1855.18 983.175 L1854.95 982.786 L1854.73 983.033 L1854.5 983.339 L1854.27 983.588 L1854.04 983.777 L1853.81 983.488 L1853.59 982.893 L1853.36 982.792 L1853.13 982.947 L1852.9 982.356 L1852.68 982.238 L1852.45 982.482 L1852.22 982.724 L1851.99 982.648 L1851.76 982.617 L1851.54 982.373 L1851.31 982.57 L1851.08 982.625 L1850.85 982.973 L1850.63 982.712 L1850.4 984.136 L1850.17 983.932 L1849.94 983.604 L1849.71 984.04 L1849.49 984.579 L1849.26 984.752 L1849.03 984.913 L1848.8 984.587 L1848.58 985.001 L1848.35 984.554 L1848.12 984.765 L1847.89 984.679 L1847.66 984.228 L1847.44 984.188 L1847.21 984.263 L1846.98 983.813 L1846.75 983.837 L1846.53 984.483 L1846.3 984.282 L1846.07 984.098 L1845.84 983.979 L1845.61 983.292 L1845.39 982.838 L1845.16 982.927 L1844.93 982.67 L1844.7 982.828 L1844.48 982.689 L1844.25 982.656 L1844.02 982.418 L1843.79 982.623 L1843.56 982.504 L1843.34 983.367 L1843.11 983.141 L1842.88 983.047 L1842.65 983.538 L1842.43 983.686 L1842.2 984.056 L1841.97 984.149 L1841.74 983.792 L1841.51 984.2 L1841.29 983.548 L1841.06 983.034 L1840.83 983.419 L1840.6 983.211 L1840.38 983.51 L1840.15 983.428 L1839.92 983.627 L1839.69 982.811 L1839.46 982.903 L1839.24 982.349 L1839.01 981.768 L1838.78 981.82 L1838.55 981.564 L1838.33 981.211 L1838.1 980.823 L1837.87 980.697 L1837.64 980.363 L1837.42 980.963 L1837.19 980.433 L1836.96 980.249 L1836.73 980.047 L1836.5 979.889 L1836.28 979.296 L1836.05 979.172 L1835.82 979.458 L1835.59 979.438 L1835.37 979.394 L1835.14 979.495 L1834.91 979.023 L1834.68 978.975 L1834.45 978.877 L1834.23 979.08 L1834 979.086 L1833.77 979.355 L1833.54 978.967 L1833.32 979.112 L1833.09 979.113 L1832.86 979.087 L1832.63 978.579 L1832.4 978.517 L1832.18 977.796 L1831.95 977.447 L1831.72 976.932 L1831.49 975.962 L1831.27 975.221 L1831.04 975.085 L1830.81 974.528 L1830.58 974.876 L1830.35 974.962 L1830.13 975.225 L1829.9 975.102 L1829.67 974.769 L1829.44 974.363 L1829.22 974.049 L1828.99 975.123 L1828.76 975.117 L1828.53 975.975 L1828.3 975.806 L1828.08 975.685 L1827.85 975.626 L1827.62 976.693 L1827.39 976.56 L1827.17 976.956 L1826.94 977.194 L1826.71 977.047 L1826.48 977.529 L1826.25 977.786 L1826.03 977.45 L1825.8 977.459 L1825.57 977.811 L1825.34 977.634 L1825.12 977.324 L1824.89 978.177 L1824.66 978.393 L1824.43 978.576 L1824.2 978.086 L1823.98 978.62 L1823.75 978.742 L1823.52 978.222 L1823.29 977.839 L1823.07 977.114 L1822.84 976.554 L1822.61 976.481 L1822.38 975.88 L1822.15 976.223 L1821.93 976.285 L1821.7 976.82 L1821.47 976.708 L1821.24 976.77 L1821.02 976.048 L1820.79 976.281 L1820.56 976.065 L1820.33 976.198 L1820.1 975.491 L1819.88 975.454 L1819.65 976.09 L1819.42 976.558 L1819.19 977.127 L1818.97 977.192 L1818.74 977.266 L1818.51 977.91 L1818.28 978.152 L1818.05 978.205 L1817.83 978.25 L1817.6 977.675 L1817.37 977.853 L1817.14 977.721 L1816.92 977.792 L1816.69 978.082 L1816.46 978.781 L1816.23 978.502 L1816 978.927 L1815.78 978.36 L1815.55 978.419 L1815.32 978.474 L1815.09 978.16 L1814.87 977.673 L1814.64 977.19 L1814.41 976.952 L1814.18 976.71 L1813.95 976.766 L1813.73 976.894 L1813.5 976.651 L1813.27 976.194 L1813.04 976.974 L1812.82 976.865 L1812.59 976.34 L1812.36 976.214 L1812.13 975.712 L1811.9 976.043 L1811.68 976.092 L1811.45 975.777 L1811.22 975.53 L1810.99 975.827 L1810.77 975.752 L1810.54 975.851 L1810.31 975.703 L1810.08 975.449 L1809.85 975.095 L1809.63 974.647 L1809.4 974.589 L1809.17 974.431 L1808.94 974.268 L1808.72 974.038 L1808.49 973.487 L1808.26 973.532 L1808.03 973.798 L1807.8 974.145 L1807.58 973.887 L1807.35 973.571 L1807.12 973.583 L1806.89 973.027 L1806.67 973.634 L1806.44 973.531 L1806.21 973.677 L1805.98 973.07 L1805.75 972.962 L1805.53 973.312 L1805.3 973.019 L1805.07 972.776 L1804.84 973.3 L1804.62 973.436 L1804.39 973.262 L1804.16 972.944 L1803.93 973.34 L1803.7 973.309 L1803.48 972.846 L1803.25 973.108 L1803.02 973.697 L1802.79 973.101 L1802.57 972.974 L1802.34 973.028 L1802.11 972.709 L1801.88 973.296 L1801.66 972.821 L1801.43 972.11 L1801.2 971.791 L1800.97 972.62 L1800.74 972.383 L1800.52 972.334 L1800.29 972.592 L1800.06 972.849 L1799.83 972.176 L1799.61 972.181 L1799.38 972.662 L1799.15 972.368 L1798.92 972.258 L1798.69 972.358 L1798.47 972.001 L1798.24 972.651 L1798.01 972.835 L1797.78 972.713 L1797.56 973.087 L1797.33 973.077 L1797.1 972.785 L1796.87 972.571 L1796.64 972.127 L1796.42 972.181 L1796.19 971.688 L1795.96 971.831 L1795.73 971.706 L1795.51 972.165 L1795.28 972.019 L1795.05 972.363 L1794.82 972.118 L1794.59 972.166 L1794.37 972.327 L1794.14 972.011 L1793.91 972.292 L1793.68 972.001 L1793.46 972.287 L1793.23 972.357 L1793 971.454 L1792.77 971.286 L1792.54 971.294 L1792.32 971.539 L1792.09 971.04 L1791.86 971.071 L1791.63 971.092 L1791.41 971.285 L1791.18 970.796 L1790.95 971.63 L1790.72 971.568 L1790.49 971.957 L1790.27 971.594 L1790.04 971.369 L1789.81 971.689 L1789.58 972.029 L1789.36 972.182 L1789.13 972.559 L1788.9 972.676 L1788.67 972.653 L1788.44 972.728 L1788.22 972.641 L1787.99 973.143 L1787.76 974.014 L1787.53 974.113 L1787.31 974.35 L1787.08 974.333 L1786.85 974.264 L1786.62 974.769 L1786.39 974.7 L1786.17 974.614 L1785.94 974.659 L1785.71 975.305 L1785.48 975.432 L1785.26 975.437 L1785.03 975.107 L1784.8 975.995 L1784.57 976.08 L1784.34 976.769 L1784.12 976.908 L1783.89 977.311 L1783.66 977.039 L1783.43 977.298 L1783.21 977.117 L1782.98 976.982 L1782.75 976.298 L1782.52 977.195 L1782.29 976.969 L1782.07 976.808 L1781.84 976.167 L1781.61 975.761 L1781.38 975.834 L1781.16 975.291 L1780.93 976.46 L1780.7 975.695 L1780.47 975.818 L1780.24 975.36 L1780.02 975.599 L1779.79 975.439 L1779.56 975.272 L1779.33 976.323 L1779.11 976.395 L1778.88 976.633 L1778.65 976.633 L1778.42 976.264 L1778.19 976.152 L1777.97 975.876 L1777.74 976.119 L1777.51 976.722 L1777.28 976.137 L1777.06 975.854 L1776.83 975.793 L1776.6 975.21 L1776.37 975.306 L1776.14 975.622 L1775.92 975.604 L1775.69 975.968 L1775.46 975.79 L1775.23 975.916 L1775.01 975.712 L1774.78 975.51 L1774.55 975.741 L1774.32 976.314 L1774.09 976.461 L1773.87 976.27 L1773.64 976.079 L1773.41 975.597 L1773.18 975.739 L1772.96 976.067 L1772.73 975.611 L1772.5 975.693 L1772.27 975.252 L1772.04 975.259 L1771.82 975.129 L1771.59 975.816 L1771.36 975.874 L1771.13 975.493 L1770.91 974.936 L1770.68 974.398 L1770.45 973.837 L1770.22 973.627 L1769.99 973.168 L1769.77 972.804 L1769.54 973.451 L1769.31 973.424 L1769.08 972.836 L1768.86 973.357 L1768.63 972.962 L1768.4 972.784 L1768.17 972.528 L1767.94 972.55 L1767.72 972.588 L1767.49 972.296 L1767.26 972.117 L1767.03 972.408 L1766.81 972.099 L1766.58 971.592 L1766.35 972.202 L1766.12 971.964 L1765.9 971.413 L1765.67 971.112 L1765.44 970.963 L1765.21 971.107 L1764.98 971.244 L1764.76 971.624 L1764.53 971.807 L1764.3 971.572 L1764.07 971.55 L1763.85 971.808 L1763.62 972.233 L1763.39 972.392 L1763.16 971.873 L1762.93 972.094 L1762.71 972.879 L1762.48 973.44 L1762.25 973.059 L1762.02 973.304 L1761.8 973.116 L1761.57 973.133 L1761.34 973.073 L1761.11 972.5 L1760.88 972.242 L1760.66 972.672 L1760.43 972.138 L1760.2 971.595 L1759.97 972.678 L1759.75 972.506 L1759.52 972.446 L1759.29 973.366 L1759.06 973.253 L1758.83 972.998 L1758.61 973.064 L1758.38 972.957 L1758.15 973.131 L1757.92 973.433 L1757.7 973.002 L1757.47 972.547 L1757.24 972.754 L1757.01 972.617 L1756.78 971.729 L1756.56 972.348 L1756.33 971.742 L1756.1 972.544 L1755.87 972.429 L1755.65 971.653 L1755.42 971.614 L1755.19 970.874 L1754.96 970.021 L1754.73 969.499 L1754.51 969.719 L1754.28 969.607 L1754.05 969.552 L1753.82 969.388 L1753.6 968.783 L1753.37 968.62 L1753.14 968.598 L1752.91 968.762 L1752.68 968.836 L1752.46 968.49 L1752.23 968.317 L1752 967.962 L1751.77 968.152 L1751.55 967.844 L1751.32 968.528 L1751.09 968.24 L1750.86 968.056 L1750.63 968.053 L1750.41 968.737 L1750.18 968.339 L1749.95 968.6 L1749.72 968.862 L1749.5 969.733 L1749.27 969.604 L1749.04 969.429 L1748.81 968.855 L1748.58 969.027 L1748.36 968.598 L1748.13 968.044 L1747.9 967.352 L1747.67 966.985 L1747.45 966.422 L1747.22 966.34 L1746.99 967.378 L1746.76 967.6 L1746.53 967.77 L1746.31 968.184 L1746.08 968.027 L1745.85 967.639 L1745.62 968.227 L1745.4 968.57 L1745.17 967.982 L1744.94 967.505 L1744.71 967.049 L1744.48 967.797 L1744.26 967.463 L1744.03 967.978 L1743.8 967.584 L1743.57 967.41 L1743.35 967.137 L1743.12 966.795 L1742.89 966.932 L1742.66 967.228 L1742.43 967.243 L1742.21 968.543 L1741.98 968.113 L1741.75 968.518 L1741.52 968.773 L1741.3 968.518 L1741.07 968.481 L1740.84 968.654 L1740.61 967.941 L1740.38 967.449 L1740.16 967.828 L1739.93 968.304 L1739.7 968.266 L1739.47 968.184 L1739.25 968.038 L1739.02 967.744 L1738.79 967.717 L1738.56 967.509 L1738.33 967.647 L1738.11 968.19 L1737.88 967.506 L1737.65 967.66 L1737.42 967.804 L1737.2 967.212 L1736.97 967.855 L1736.74 968.019 L1736.51 968.459 L1736.28 968.03 L1736.06 967.406 L1735.83 967.3 L1735.6 968.258 L1735.37 968.351 L1735.15 968.423 L1734.92 968.211 L1734.69 968.073 L1734.46 968.133 L1734.23 968.208 L1734.01 969.032 L1733.78 969.011 L1733.55 968.702 L1733.32 968.198 L1733.1 968.003 L1732.87 967.19 L1732.64 967.155 L1732.41 967.342 L1732.18 967.506 L1731.96 967.024 L1731.73 966.773 L1731.5 966.933 L1731.27 966.862 L1731.05 966.776 L1730.82 966.358 L1730.59 966.58 L1730.36 967.074 L1730.14 967.446 L1729.91 967.281 L1729.68 968.153 L1729.45 968.328 L1729.22 968.571 L1729 968.459 L1728.77 968.12 L1728.54 968.283 L1728.31 967.816 L1728.09 967.903 L1727.86 967.404 L1727.63 967.501 L1727.4 967.243 L1727.17 967.279 L1726.95 966.98 L1726.72 968.312 L1726.49 968.553 L1726.26 968.084 L1726.04 967.86 L1725.81 968.011 L1725.58 967.702 L1725.35 967.651 L1725.12 967.26 L1724.9 967.459 L1724.67 967.524 L1724.44 967.744 L1724.21 967.748 L1723.99 968.126 L1723.76 968.302 L1723.53 968.605 L1723.3 968.732 L1723.07 968.741 L1722.85 968.537 L1722.62 968.968 L1722.39 969.119 L1722.16 968.701 L1721.94 968.217 L1721.71 967.908 L1721.48 968.533 L1721.25 968.493 L1721.02 968.276 L1720.8 968.63 L1720.57 968.997 L1720.34 969.662 L1720.11 968.9 L1719.89 968.856 L1719.66 968.547 L1719.43 967.906 L1719.2 967.557 L1718.97 968.067 L1718.75 967.533 L1718.52 968.562 L1718.29 967.982 L1718.06 968.157 L1717.84 968.288 L1717.61 968.595 L1717.38 968.47 L1717.15 968.917 L1716.92 968.837 L1716.7 969.044 L1716.47 969.105 L1716.24 969.037 L1716.01 968.615 L1715.79 968.598 L1715.56 968.325 L1715.33 968.532 L1715.1 968.921 L1714.87 968.595 L1714.65 968.779 L1714.42 969.083 L1714.19 968.98 L1713.96 968.657 L1713.74 969.282 L1713.51 969.169 L1713.28 969.647 L1713.05 968.997 L1712.82 968.116 L1712.6 968.076 L1712.37 969.258 L1712.14 969.438 L1711.91 969.106 L1711.69 969.242 L1711.46 968.86 L1711.23 968.912 L1711 969.104 L1710.77 969.601 L1710.55 969.62 L1710.32 969.63 L1710.09 969.782 L1709.86 969.88 L1709.64 970.222 L1709.41 970.189 L1709.18 970.419 L1708.95 969.423 L1708.72 969.672 L1708.5 969.582 L1708.27 969.098 L1708.04 968.965 L1707.81 968.844 L1707.59 968.776 L1707.36 969.001 L1707.13 969.223 L1706.9 969.864 L1706.67 969.879 L1706.45 969.266 L1706.22 968.937 L1705.99 968.716 L1705.76 968.896 L1705.54 969.36 L1705.31 969.032 L1705.08 968.62 L1704.85 967.878 L1704.62 967.66 L1704.4 968.965 L1704.17 969.287 L1703.94 969.06 L1703.71 968.671 L1703.49 968.43 L1703.26 967.468 L1703.03 967.857 L1702.8 968.135 L1702.57 967.814 L1702.35 967.991 L1702.12 968.197 L1701.89 968.418 L1701.66 968.504 L1701.44 968.071 L1701.21 968.121 L1700.98 968.03 L1700.75 968.294 L1700.52 967.971 L1700.3 968.43 L1700.07 968.214 L1699.84 968.283 L1699.61 968.738 L1699.39 969.049 L1699.16 968.684 L1698.93 968.341 L1698.7 968.38 L1698.47 968.332 L1698.25 968.165 L1698.02 968.381 L1697.79 968.208 L1697.56 968.349 L1697.34 968.693 L1697.11 969.177 L1696.88 968.91 L1696.65 969.35 L1696.42 969.422 L1696.2 969.738 L1695.97 969.248 L1695.74 969.255 L1695.51 969.537 L1695.29 969.576 L1695.06 969.291 L1694.83 969.201 L1694.6 970.428 L1694.37 969.905 L1694.15 969.645 L1693.92 969.384 L1693.69 969.177 L1693.46 969.548 L1693.24 969.638 L1693.01 969.526 L1692.78 969.801 L1692.55 969.649 L1692.33 969.437 L1692.1 969.327 L1691.87 968.943 L1691.64 969.006 L1691.41 969.009 L1691.19 968.605 L1690.96 968.7 L1690.73 968.428 L1690.5 968.167 L1690.28 968.781 L1690.05 969.096 L1689.82 968.314 L1689.59 969.381 L1689.36 969.67 L1689.14 969.996 L1688.91 971.001 L1688.68 970.435 L1688.45 970.848 L1688.23 972.02 L1688 971.522 L1687.77 971.354 L1687.54 971.327 L1687.31 971.423 L1687.09 971.673 L1686.86 972.576 L1686.63 972.573 L1686.4 971.993 L1686.18 972.73 L1685.95 972.228 L1685.72 971.992 L1685.49 972.14 L1685.26 971.651 L1685.04 971.62 L1684.81 971.4 L1684.58 971.206 L1684.35 971.566 L1684.13 971.888 L1683.9 971.391 L1683.67 971.217 L1683.44 972.531 L1683.21 972.438 L1682.99 972.9 L1682.76 972.517 L1682.53 972.134 L1682.3 972.123 L1682.08 973.771 L1681.85 973.458 L1681.62 973.195 L1681.39 972.934 L1681.16 972.279 L1680.94 972.394 L1680.71 972.352 L1680.48 972.584 L1680.25 972.639 L1680.03 972.922 L1679.8 972.998 L1679.57 972.489 L1679.34 973.223 L1679.11 973.387 L1678.89 973.123 L1678.66 973.057 L1678.43 972.635 L1678.2 972.63 L1677.98 973.634 L1677.75 973.654 L1677.52 973.542 L1677.29 974.012 L1677.06 973.455 L1676.84 973.383 L1676.61 972.832 L1676.38 973.776 L1676.15 973.309 L1675.93 973.074 L1675.7 972.686 L1675.47 973.037 L1675.24 973.12 L1675.01 973.943 L1674.79 973.35 L1674.56 973.668 L1674.33 973.759 L1674.1 973.535 L1673.88 973.534 L1673.65 973.945 L1673.42 973.863 L1673.19 973.108 L1672.96 973.989 L1672.74 973.697 L1672.51 973.897 L1672.28 974.295 L1672.05 974.209 L1671.83 973.891 L1671.6 973.297 L1671.37 973.802 L1671.14 973.796 L1670.91 973.698 L1670.69 973.214 L1670.46 973.643 L1670.23 973.393 L1670 972.77 L1669.78 972.698 L1669.55 972.018 L1669.32 971.567 L1669.09 971.223 L1668.86 971.424 L1668.64 970.856 L1668.41 971.416 L1668.18 970.702 L1667.95 971.806 L1667.73 971.601 L1667.5 971.515 L1667.27 971.273 L1667.04 971.524 L1666.81 972.02 L1666.59 971.186 L1666.36 971.086 L1666.13 971.011 L1665.9 971.285 L1665.68 971.315 L1665.45 971.49 L1665.22 970.861 L1664.99 970.736 L1664.76 970.279 L1664.54 969.519 L1664.31 969.432 L1664.08 969.714 L1663.85 970.631 L1663.63 970.136 L1663.4 970.369 L1663.17 969.877 L1662.94 969.766 L1662.71 970.795 L1662.49 970.518 L1662.26 970.268 L1662.03 970.748 L1661.8 970.553 L1661.58 970.208 L1661.35 970.057 L1661.12 969.688 L1660.89 969.262 L1660.66 968.675 L1660.44 969.09 L1660.21 969.571 L1659.98 969.931 L1659.75 969.442 L1659.53 968.787 L1659.3 968.39 L1659.07 967.975 L1658.84 967.425 L1658.61 967.199 L1658.39 967.729 L1658.16 967.521 L1657.93 967.484 L1657.7 967.167 L1657.48 966.778 L1657.25 966.371 L1657.02 966.566 L1656.79 966.377 L1656.57 966.38 L1656.34 966.031 L1656.11 966.332 L1655.88 965.965 L1655.65 965.393 L1655.43 964.824 L1655.2 964.617 L1654.97 964.686 L1654.74 964.874 L1654.52 965.363 L1654.29 965.17 L1654.06 964.62 L1653.83 964.673 L1653.6 964.232 L1653.38 964.191 L1653.15 963.878 L1652.92 963.515 L1652.69 963.185 L1652.47 963.311 L1652.24 963.192 L1652.01 962.863 L1651.78 962.389 L1651.55 962.339 L1651.33 961.615 L1651.1 962.425 L1650.87 962.363 L1650.64 961.834 L1650.42 961.305 L1650.19 961.29 L1649.96 961.276 L1649.73 961.117 L1649.5 961.784 L1649.28 961.641 L1649.05 961.562 L1648.82 961.292 L1648.59 960.862 L1648.37 960.801 L1648.14 960.667 L1647.91 960.228 L1647.68 959.816 L1647.45 960.145 L1647.23 961.515 L1647 961.383 L1646.77 960.753 L1646.54 961.585 L1646.32 961.093 L1646.09 961.004 L1645.86 961.417 L1645.63 960.89 L1645.4 960.111 L1645.18 959.752 L1644.95 960.048 L1644.72 961.373 L1644.49 961.52 L1644.27 961.99 L1644.04 961.377 L1643.81 961.006 L1643.58 961.833 L1643.35 961.689 L1643.13 962.405 L1642.9 962.205 L1642.67 961.817 L1642.44 961.65 L1642.22 961.91 L1641.99 961.622 L1641.76 961.569 L1641.53 961.893 L1641.3 961.507 L1641.08 961.38 L1640.85 961.031 L1640.62 961.383 L1640.39 961.691 L1640.17 962.026 L1639.94 963.152 L1639.71 963.879 L1639.48 963.555 L1639.25 962.851 L1639.03 963.872 L1638.8 963.923 L1638.57 964.011 L1638.34 963.936 L1638.12 964.012 L1637.89 963.579 L1637.66 963.517 L1637.43 963.434 L1637.2 963.079 L1636.98 964.074 L1636.75 964.038 L1636.52 963.712 L1636.29 963.877 L1636.07 963.325 L1635.84 963.065 L1635.61 963.51 L1635.38 963.08 L1635.15 963.432 L1634.93 963.347 L1634.7 963.398 L1634.47 963.574 L1634.24 963.621 L1634.02 963.419 L1633.79 964.164 L1633.56 964.545 L1633.33 965.58 L1633.1 965.389 L1632.88 965.054 L1632.65 964.568 L1632.42 964.161 L1632.19 963.999 L1631.97 963.747 L1631.74 963.825 L1631.51 963.906 L1631.28 963.997 L1631.05 964.468 L1630.83 964.419 L1630.6 964.94 L1630.37 964.534 L1630.14 964.718 L1629.92 965.048 L1629.69 965.775 L1629.46 965.538 L1629.23 966.663 L1629 966.074 L1628.78 966.287 L1628.55 965.574 L1628.32 965.592 L1628.09 965.017 L1627.87 965.448 L1627.64 965.597 L1627.41 965.69 L1627.18 965.543 L1626.95 965.439 L1626.73 966.283 L1626.5 966.57 L1626.27 966.567 L1626.04 966.669 L1625.82 966.456 L1625.59 966.756 L1625.36 967.138 L1625.13 966.949 L1624.9 966.459 L1624.68 966.136 L1624.45 966.053 L1624.22 966.073 L1623.99 966.027 L1623.77 966.039 L1623.54 965.667 L1623.31 965.553 L1623.08 965.447 L1622.85 966.376 L1622.63 965.849 L1622.4 965.806 L1622.17 964.793 L1621.94 964.891 L1621.72 964.905 L1621.49 965.077 L1621.26 964.708 L1621.03 964.924 L1620.81 965.105 L1620.58 965.234 L1620.35 964.709 L1620.12 964.232 L1619.89 964.232 L1619.67 964.572 L1619.44 964.148 L1619.21 963.885 L1618.98 964.684 L1618.76 965.272 L1618.53 964.755 L1618.3 964.587 L1618.07 964.522 L1617.84 964.163 L1617.62 964.454 L1617.39 964.739 L1617.16 964.673 L1616.93 965.004 L1616.71 965.165 L1616.48 966.343 L1616.25 965.566 L1616.02 965.598 L1615.79 966.123 L1615.57 966.367 L1615.34 967.186 L1615.11 967.075 L1614.88 966.741 L1614.66 967.058 L1614.43 967.39 L1614.2 967.892 L1613.97 968.044 L1613.74 968.672 L1613.52 968.233 L1613.29 967.376 L1613.06 967.537 L1612.83 967.833 L1612.61 967.734 L1612.38 967.275 L1612.15 967.312 L1611.92 967.136 L1611.69 967.303 L1611.47 968.047 L1611.24 968.552 L1611.01 968.579 L1610.78 969.221 L1610.56 970.115 L1610.33 969.985 L1610.1 969.425 L1609.87 968.852 L1609.64 969.433 L1609.42 968.833 L1609.19 969.05 L1608.96 969.172 L1608.73 969.138 L1608.51 968.857 L1608.28 969.106 L1608.05 969.054 L1607.82 968.436 L1607.59 968.195 L1607.37 968.692 L1607.14 968.885 L1606.91 968.917 L1606.68 968.327 L1606.46 968.323 L1606.23 968.281 L1606 968.333 L1605.77 968.234 L1605.54 968.101 L1605.32 967.943 L1605.09 967.513 L1604.86 968.63 L1604.63 968.094 L1604.41 967.678 L1604.18 967.272 L1603.95 967.226 L1603.72 966.91 L1603.49 966.835 L1603.27 966.668 L1603.04 967.11 L1602.81 966.67 L1602.58 965.74 L1602.36 966.145 L1602.13 967.71 L1601.9 967.548 L1601.67 967.542 L1601.44 967.923 L1601.22 968.232 L1600.99 967.925 L1600.76 967.89 L1600.53 967.576 L1600.31 967.494 L1600.08 966.984 L1599.85 967.135 L1599.62 967.81 L1599.39 968.361 L1599.17 967.488 L1598.94 967.459 L1598.71 968.287 L1598.48 968.204 L1598.26 969.09 L1598.03 969.789 L1597.8 969.627 L1597.57 970.03 L1597.34 970.339 L1597.12 970.345 L1596.89 969.585 L1596.66 969.17 L1596.43 969.008 L1596.21 968.17 L1595.98 968.027 L1595.75 968.434 L1595.52 969.014 L1595.29 968.886 L1595.07 968.817 L1594.84 968.626 L1594.61 968.339 L1594.38 968.525 L1594.16 967.691 L1593.93 967.84 L1593.7 967.437 L1593.47 966.89 L1593.24 967.122 L1593.02 966.791 L1592.79 966.702 L1592.56 966.651 L1592.33 966.391 L1592.11 966.853 L1591.88 966.799 L1591.65 967.035 L1591.42 966.851 L1591.19 966.923 L1590.97 966.871 L1590.74 966.848 L1590.51 967.708 L1590.28 968.298 L1590.06 967.936 L1589.83 968.034 L1589.6 968.789 L1589.37 968.308 L1589.14 967.876 L1588.92 968.169 L1588.69 968.848 L1588.46 968.582 L1588.23 969.198 L1588.01 969.524 L1587.78 969.38 L1587.55 969.057 L1587.32 968.755 L1587.09 969.207 L1586.87 969.1 L1586.64 969.015 L1586.41 968.76 L1586.18 968.746 L1585.96 968.242 L1585.73 968.472 L1585.5 967.857 L1585.27 968.787 L1585.04 968.548 L1584.82 968.351 L1584.59 967.877 L1584.36 967.056 L1584.13 967.935 L1583.91 967.99 L1583.68 967.676 L1583.45 967.894 L1583.22 968.111 L1583 967.934 L1582.77 967.405 L1582.54 967.361 L1582.31 967.17 L1582.08 967.045 L1581.86 967.365 L1581.63 968.625 L1581.4 968.305 L1581.17 969.067 L1580.95 969.521 L1580.72 969.457 L1580.49 969.586 L1580.26 970.33 L1580.03 970.666 L1579.81 970.306 L1579.58 970.43 L1579.35 970.155 L1579.12 970.692 L1578.9 970.749 L1578.67 970.373 L1578.44 970.075 L1578.21 969.973 L1577.98 969.811 L1577.76 969.487 L1577.53 969.913 L1577.3 969.455 L1577.07 969.916 L1576.85 969.788 L1576.62 969.572 L1576.39 969.738 L1576.16 969.8 L1575.93 970.087 L1575.71 970.93 L1575.48 971.064 L1575.25 971.687 L1575.02 971.899 L1574.8 972.169 L1574.57 972.793 L1574.34 973.357 L1574.11 974.098 L1573.88 974.387 L1573.66 974.764 L1573.43 974.76 L1573.2 974.724 L1572.97 974.419 L1572.75 974.189 L1572.52 974.34 L1572.29 974.052 L1572.06 973.918 L1571.83 973.542 L1571.61 973.186 L1571.38 972.646 L1571.15 972.407 L1570.92 972.526 L1570.7 972.221 L1570.47 972.195 L1570.24 972.465 L1570.01 972.59 L1569.78 972.678 L1569.56 972.383 L1569.33 971.715 L1569.1 972.082 L1568.87 972.233 L1568.65 972.149 L1568.42 972.749 L1568.19 972.698 L1567.96 973.195 L1567.73 973.011 L1567.51 973.437 L1567.28 973.046 L1567.05 972.889 L1566.82 973.047 L1566.6 973.204 L1566.37 973.122 L1566.14 972.496 L1565.91 972.656 L1565.68 972.658 L1565.46 972.786 L1565.23 972.686 L1565 972.112 L1564.77 972.757 L1564.55 972.873 L1564.32 972.42 L1564.09 972.48 L1563.86 973.276 L1563.63 973.526 L1563.41 973.942 L1563.18 973.859 L1562.95 973.66 L1562.72 974.103 L1562.5 974.898 L1562.27 975.104 L1562.04 974.698 L1561.81 975.071 L1561.58 974.976 L1561.36 974.946 L1561.13 974.853 L1560.9 974.456 L1560.67 974.686 L1560.45 975.02 L1560.22 974.475 L1559.99 974.855 L1559.76 974.643 L1559.53 974.122 L1559.31 974.446 L1559.08 973.831 L1558.85 973.183 L1558.62 973.694 L1558.4 973.347 L1558.17 973.873 L1557.94 975.22 L1557.71 976.018 L1557.48 975.747 L1557.26 975.735 L1557.03 975.478 L1556.8 976.052 L1556.57 976.203 L1556.35 975.399 L1556.12 975.387 L1555.89 975.025 L1555.66 974.804 L1555.43 974.68 L1555.21 974.025 L1554.98 973.786 L1554.75 974.254 L1554.52 974.585 L1554.3 975.403 L1554.07 975.312 L1553.84 976.272 L1553.61 975.944 L1553.38 975.992 L1553.16 976.33 L1552.93 976.554 L1552.7 975.994 L1552.47 975.944 L1552.25 975.816 L1552.02 976.047 L1551.79 975.967 L1551.56 975.799 L1551.33 976.461 L1551.11 977.328 L1550.88 976.804 L1550.65 976.699 L1550.42 976.723 L1550.2 976.397 L1549.97 977.136 L1549.74 976.135 L1549.51 975.327 L1549.28 975.885 L1549.06 975.822 L1548.83 975.828 L1548.6 975.915 L1548.37 975.963 L1548.15 976.376 L1547.92 975.995 L1547.69 975.404 L1547.46 975.22 L1547.24 975.358 L1547.01 976.451 L1546.78 976.19 L1546.55 977.413 L1546.32 976.944 L1546.1 976.543 L1545.87 976.617 L1545.64 977.186 L1545.41 977.731 L1545.19 977.991 L1544.96 977.847 L1544.73 976.987 L1544.5 976.543 L1544.27 976.202 L1544.05 976.11 L1543.82 975.756 L1543.59 977.537 L1543.36 977.676 L1543.14 977.422 L1542.91 977.281 L1542.68 977.408 L1542.45 977.106 L1542.22 977.339 L1542 976.911 L1541.77 976.439 L1541.54 976.235 L1541.31 976.909 L1541.09 976.784 L1540.86 977.578 L1540.63 977.691 L1540.4 978.282 L1540.17 977.831 L1539.95 978.155 L1539.72 977.945 L1539.49 977.638 L1539.26 977.414 L1539.04 977.41 L1538.81 977.513 L1538.58 977.801 L1538.35 977.776 L1538.12 979.341 L1537.9 979.142 L1537.67 978.518 L1537.44 977.754 L1537.21 977.446 L1536.99 976.752 L1536.76 976.157 L1536.53 975.801 L1536.3 975.766 L1536.07 975.432 L1535.85 975.167 L1535.62 975.047 L1535.39 974.622 L1535.16 973.958 L1534.94 973.658 L1534.71 974.037 L1534.48 974.311 L1534.25 973.799 L1534.02 973.278 L1533.8 973.231 L1533.57 973.157 L1533.34 973.02 L1533.11 972.776 L1532.89 973.052 L1532.66 972.064 L1532.43 971.489 L1532.2 971.833 L1531.97 971.585 L1531.75 971.109 L1531.52 970.708 L1531.29 971.396 L1531.06 971.975 L1530.84 971.437 L1530.61 970.931 L1530.38 970.254 L1530.15 969.875 L1529.92 970.052 L1529.7 969.954 L1529.47 969.415 L1529.24 969.425 L1529.01 969.58 L1528.79 969.798 L1528.56 969.761 L1528.33 969.782 L1528.1 970.619 L1527.87 971.827 L1527.65 971.942 L1527.42 972.357 L1527.19 972.394 L1526.96 971.717 L1526.74 970.912 L1526.51 971.071 L1526.28 970.414 L1526.05 970.641 L1525.82 970.966 L1525.6 971.326 L1525.37 971.578 L1525.14 971.608 L1524.91 971.565 L1524.69 971.413 L1524.46 971.783 L1524.23 971.552 L1524 971.394 L1523.77 971.853 L1523.55 972.495 L1523.32 972.374 L1523.09 972.495 L1522.86 972.348 L1522.64 972.509 L1522.41 972.283 L1522.18 973.298 L1521.95 972.948 L1521.72 972.344 L1521.5 972.718 L1521.27 971.835 L1521.04 971.669 L1520.81 971.573 L1520.59 972.203 L1520.36 971.833 L1520.13 971.556 L1519.9 971.667 L1519.67 972.084 L1519.45 972.185 L1519.22 971.572 L1518.99 971.005 L1518.76 970.876 L1518.54 970.539 L1518.31 971.012 L1518.08 970.882 L1517.85 971.17 L1517.62 972.009 L1517.4 972.262 L1517.17 972.594 L1516.94 972.885 L1516.71 972.761 L1516.49 972.135 L1516.26 971.548 L1516.03 971.543 L1515.8 971.602 L1515.57 971.093 L1515.35 970.966 L1515.12 971.17 L1514.89 970.843 L1514.66 970.741 L1514.44 970.322 L1514.21 970.286 L1513.98 970.785 L1513.75 971.995 L1513.52 972.265 L1513.3 971.64 L1513.07 971.747 L1512.84 971.324 L1512.61 970.562 L1512.39 970.09 L1512.16 969.719 L1511.93 969.395 L1511.7 969.891 L1511.48 969.812 L1511.25 969.981 L1511.02 970.48 L1510.79 969.816 L1510.56 969.689 L1510.34 969.38 L1510.11 969.797 L1509.88 969.934 L1509.65 970.791 L1509.43 970.891 L1509.2 970.042 L1508.97 969.543 L1508.74 969.286 L1508.51 969.014 L1508.29 968.506 L1508.06 968.067 L1507.83 967.55 L1507.6 967.622 L1507.38 966.711 L1507.15 966.06 L1506.92 966.519 L1506.69 966.834 L1506.46 966.662 L1506.24 966.088 L1506.01 965.491 L1505.78 965.657 L1505.55 965.387 L1505.33 964.938 L1505.1 966.001 L1504.87 966.275 L1504.64 966.284 L1504.41 966.354 L1504.19 965.721 L1503.96 965.956 L1503.73 966.02 L1503.5 966.536 L1503.28 967.36 L1503.05 968.551 L1502.82 967.999 L1502.59 967.998 L1502.36 968.577 L1502.14 968.509 L1501.91 968.561 L1501.68 968.834 L1501.45 968.59 L1501.23 968.928 L1501 968.201 L1500.77 968.079 L1500.54 967.574 L1500.31 968.01 L1500.09 968.309 L1499.86 967.619 L1499.63 967.555 L1499.4 967.967 L1499.18 967.782 L1498.95 967.384 L1498.72 967.232 L1498.49 967.816 L1498.26 967.426 L1498.04 967.145 L1497.81 966.902 L1497.58 966.351 L1497.35 965.834 L1497.13 966.194 L1496.9 966.337 L1496.67 965.939 L1496.44 965.744 L1496.21 966.082 L1495.99 965.848 L1495.76 966.284 L1495.53 966.396 L1495.3 966.39 L1495.08 965.91 L1494.85 965.706 L1494.62 965.299 L1494.39 965.508 L1494.16 966.051 L1493.94 966.476 L1493.71 967.301 L1493.48 967.824 L1493.25 967.962 L1493.03 968.124 L1492.8 967.391 L1492.57 967.176 L1492.34 967.436 L1492.11 967.447 L1491.89 967.644 L1491.66 967.331 L1491.43 967.429 L1491.2 967.152 L1490.98 966.611 L1490.75 966.162 L1490.52 965.489 L1490.29 966.516 L1490.06 966.302 L1489.84 966.337 L1489.61 965.676 L1489.38 966.381 L1489.15 966.018 L1488.93 966.222 L1488.7 965.774 L1488.47 964.976 L1488.24 965.323 L1488.01 965.209 L1487.79 964.483 L1487.56 964.374 L1487.33 964.885 L1487.1 964.786 L1486.88 964.463 L1486.65 964.318 L1486.42 964.294 L1486.19 963.803 L1485.96 963.405 L1485.74 963.48 L1485.51 962.894 L1485.28 963.743 L1485.05 964.068 L1484.83 963.877 L1484.6 963.367 L1484.37 963.377 L1484.14 963.613 L1483.91 963.58 L1483.69 963.254 L1483.46 963.614 L1483.23 964.327 L1483 964.211 L1482.78 964.541 L1482.55 964.084 L1482.32 963.766 L1482.09 963.382 L1481.86 962.516 L1481.64 962.244 L1481.41 961.79 L1481.18 962.636 L1480.95 962.102 L1480.73 961.185 L1480.5 961.168 L1480.27 960.746 L1480.04 960.557 L1479.81 959.66 L1479.59 959.373 L1479.36 959.117 L1479.13 958.6 L1478.9 958.04 L1478.68 959.015 L1478.45 959.073 L1478.22 958.916 L1477.99 958.373 L1477.76 958.75 L1477.54 959.065 L1477.31 958.18 L1477.08 958.557 L1476.85 958.529 L1476.63 959.023 L1476.4 958.827 L1476.17 958.92 L1475.94 959.228 L1475.71 959.146 L1475.49 959.491 L1475.26 959.386 L1475.03 959.234 L1474.8 960.164 L1474.58 960.056 L1474.35 959.974 L1474.12 959.813 L1473.89 959.11 L1473.67 958.78 L1473.44 959.695 L1473.21 959.396 L1472.98 959.338 L1472.75 958.307 L1472.53 958.313 L1472.3 957.596 L1472.07 957.283 L1471.84 957.756 L1471.62 957.755 L1471.39 957.95 L1471.16 957.549 L1470.93 957.455 L1470.7 958.399 L1470.48 958.33 L1470.25 958.061 L1470.02 958.426 L1469.79 958.334 L1469.57 958.032 L1469.34 957.855 L1469.11 957.45 L1468.88 957.097 L1468.65 958.151 L1468.43 957.606 L1468.2 958.171 L1467.97 959.078 L1467.74 958.926 L1467.52 959.581 L1467.29 959.353 L1467.06 959.324 L1466.83 959.153 L1466.6 958.374 L1466.38 958.397 L1466.15 958.11 L1465.92 958.079 L1465.69 957.734 L1465.47 958.04 L1465.24 957.708 L1465.01 957.27 L1464.78 956.869 L1464.55 957.014 L1464.33 956.649 L1464.1 956.235 L1463.87 956.613 L1463.64 956.374 L1463.42 955.99 L1463.19 956.638 L1462.96 956.591 L1462.73 956.228 L1462.5 956.122 L1462.28 957.077 L1462.05 956.993 L1461.82 956.862 L1461.59 956.718 L1461.37 956.84 L1461.14 957.434 L1460.91 958.259 L1460.68 959.075 L1460.45 959.103 L1460.23 958.754 L1460 958.186 L1459.77 957.704 L1459.54 957.982 L1459.32 958.165 L1459.09 957.433 L1458.86 957.106 L1458.63 957.263 L1458.4 957.139 L1458.18 956.919 L1457.95 956.218 L1457.72 956.084 L1457.49 955.797 L1457.27 956.674 L1457.04 956.741 L1456.81 956.596 L1456.58 956.529 L1456.35 956.314 L1456.13 956.467 L1455.9 955.995 L1455.67 955.711 L1455.44 956.695 L1455.22 956.591 L1454.99 955.892 L1454.76 956.574 L1454.53 956.476 L1454.3 956.333 L1454.08 956.132 L1453.85 956.914 L1453.62 957.218 L1453.39 957.403 L1453.17 956.827 L1452.94 956.345 L1452.71 957.094 L1452.48 958.1 L1452.25 958.573 L1452.03 958.755 L1451.8 958.64 L1451.57 957.895 L1451.34 958.893 L1451.12 958.782 L1450.89 957.751 L1450.66 956.827 L1450.43 957.66 L1450.2 957.713 L1449.98 957.389 L1449.75 957.174 L1449.52 957.229 L1449.29 957.25 L1449.07 957.141 L1448.84 956.818 L1448.61 956.088 L1448.38 955.304 L1448.15 954.969 L1447.93 955.144 L1447.7 954.644 L1447.47 955.052 L1447.24 955.281 L1447.02 954.832 L1446.79 954.448 L1446.56 955.733 L1446.33 955.992 L1446.1 956.069 L1445.88 956.643 L1445.65 956.487 L1445.42 956.447 L1445.19 956.556 L1444.97 956.58 L1444.74 955.996 L1444.51 956.757 L1444.28 957.157 L1444.05 956.98 L1443.83 957.214 L1443.6 957.179 L1443.37 956.982 L1443.14 956.803 L1442.92 956.753 L1442.69 957.057 L1442.46 957.388 L1442.23 956.972 L1442 956.93 L1441.78 956.509 L1441.55 957.062 L1441.32 957.133 L1441.09 956.902 L1440.87 956.828 L1440.64 956.52 L1440.41 956.499 L1440.18 956.676 L1439.95 956.277 L1439.73 956.312 L1439.5 956.363 L1439.27 957.038 L1439.04 956.869 L1438.82 955.962 L1438.59 955.529 L1438.36 955.092 L1438.13 955.524 L1437.91 955.812 L1437.68 956.018 L1437.45 956.402 L1437.22 956.493 L1436.99 956.495 L1436.77 955.915 L1436.54 955.388 L1436.31 955.805 L1436.08 955.713 L1435.86 955.419 L1435.63 955.125 L1435.4 954.95 L1435.17 954.932 L1434.94 956.073 L1434.72 955.742 L1434.49 956.343 L1434.26 956.954 L1434.03 956.168 L1433.81 955.855 L1433.58 956.62 L1433.35 956.777 L1433.12 957.105 L1432.89 956.965 L1432.67 957.337 L1432.44 956.811 L1432.21 955.687 L1431.98 956.06 L1431.76 957.014 L1431.53 958.003 L1431.3 957.596 L1431.07 956.987 L1430.84 957.507 L1430.62 956.411 L1430.39 956.853 L1430.16 957.575 L1429.93 957.912 L1429.71 958.053 L1429.48 957.27 L1429.25 957.471 L1429.02 957.549 L1428.79 956.867 L1428.57 956.514 L1428.34 955.876 L1428.11 955.812 L1427.88 955.255 L1427.66 954.661 L1427.43 954.233 L1427.2 954.067 L1426.97 954.333 L1426.74 955.31 L1426.52 955.474 L1426.29 955.731 L1426.06 955.005 L1425.83 954.525 L1425.61 954.502 L1425.38 954.002 L1425.15 953.293 L1424.92 953.267 L1424.69 954.016 L1424.47 953.075 L1424.24 952.767 L1424.01 953.859 L1423.78 953.805 L1423.56 954.003 L1423.33 954.068 L1423.1 954.143 L1422.87 953.08 L1422.64 953.021 L1422.42 953.989 L1422.19 954.478 L1421.96 954.266 L1421.73 954.851 L1421.51 955.658 L1421.28 955.638 L1421.05 954.997 L1420.82 954.458 L1420.59 954.567 L1420.37 953.971 L1420.14 953.918 L1419.91 953.431 L1419.68 954.023 L1419.46 953.664 L1419.23 952.674 L1419 951.585 L1418.77 951.101 L1418.54 951.725 L1418.32 952.254 L1418.09 952.538 L1417.86 952.427 L1417.63 952.51 L1417.41 952.416 L1417.18 952.291 L1416.95 952.85 L1416.72 952.371 L1416.49 952.006 L1416.27 951.927 L1416.04 951.497 L1415.81 952.049 L1415.58 951.543 L1415.36 951.429 L1415.13 950.364 L1414.9 950.624 L1414.67 950.809 L1414.44 950.681 L1414.22 949.622 L1413.99 950.243 L1413.76 950.217 L1413.53 951.124 L1413.31 951.29 L1413.08 950.639 L1412.85 950.21 L1412.62 950.404 L1412.39 950.695 L1412.17 950.707 L1411.94 950.682 L1411.71 951.202 L1411.48 950.594 L1411.26 951.639 L1411.03 951.817 L1410.8 953.306 L1410.57 954.291 L1410.34 954.322 L1410.12 953.916 L1409.89 953.166 L1409.66 952.646 L1409.43 952.469 L1409.21 951.782 L1408.98 952.242 L1408.75 952.116 L1408.52 952.134 L1408.29 951.873 L1408.07 951.387 L1407.84 951.483 L1407.61 951.505 L1407.38 951.307 L1407.16 951.529 L1406.93 951.786 L1406.7 950.985 L1406.47 950.079 L1406.24 949.845 L1406.02 949.971 L1405.79 950.153 L1405.56 949.909 L1405.33 950.46 L1405.11 950.263 L1404.88 950.645 L1404.65 950.451 L1404.42 950.728 L1404.19 950.239 L1403.97 950.778 L1403.74 951.348 L1403.51 952.186 L1403.28 951.438 L1403.06 951.639 L1402.83 951.256 L1402.6 950.85 L1402.37 950.684 L1402.15 950.282 L1401.92 950.153 L1401.69 949.977 L1401.46 948.955 L1401.23 948.809 L1401.01 948.507 L1400.78 948.258 L1400.55 947.959 L1400.32 948.125 L1400.1 947.388 L1399.87 947.957 L1399.64 947.461 L1399.41 947.304 L1399.18 947.047 L1398.96 946.736 L1398.73 947.447 L1398.5 947.43 L1398.27 948.344 L1398.05 947.95 L1397.82 947.877 L1397.59 947.494 L1397.36 947.385 L1397.13 947.477 L1396.91 947.875 L1396.68 947.378 L1396.45 947.624 L1396.22 947.977 L1396 947.932 L1395.77 947.763 L1395.54 947.654 L1395.31 948.535 L1395.08 948.135 L1394.86 947.967 L1394.63 949.46 L1394.4 949.044 L1394.17 949.172 L1393.95 949.39 L1393.72 949.187 L1393.49 948.924 L1393.26 948.675 L1393.03 948.843 L1392.81 948.43 L1392.58 947.509 L1392.35 947.779 L1392.12 948.048 L1391.9 947.185 L1391.67 947.256 L1391.44 947.49 L1391.21 947.751 L1390.98 947.961 L1390.76 947.605 L1390.53 947.923 L1390.3 947.802 L1390.07 947.533 L1389.85 947.429 L1389.62 947.637 L1389.39 948.026 L1389.16 948.931 L1388.93 948.372 L1388.71 948.858 L1388.48 948.631 L1388.25 948.213 L1388.02 948.149 L1387.8 947.982 L1387.57 948.289 L1387.34 947.247 L1387.11 947.904 L1386.88 948.508 L1386.66 949.188 L1386.43 949.073 L1386.2 948.962 L1385.97 948.437 L1385.75 949.085 L1385.52 949.872 L1385.29 950.678 L1385.06 950.74 L1384.83 950.786 L1384.61 950.919 L1384.38 952.109 L1384.15 951.854 L1383.92 952.135 L1383.7 952.423 L1383.47 952.729 L1383.24 952.359 L1383.01 951.932 L1382.78 951.062 L1382.56 950.27 L1382.33 949.912 L1382.1 950.773 L1381.87 950.691 L1381.65 950.81 L1381.42 951.441 L1381.19 951.732 L1380.96 952.384 L1380.73 953.503 L1380.51 953.293 L1380.28 952.595 L1380.05 952.644 L1379.82 953.28 L1379.6 953.66 L1379.37 954.205 L1379.14 954.132 L1378.91 954.557 L1378.68 955.156 L1378.46 955.154 L1378.23 954.705 L1378 955.467 L1377.77 955.546 L1377.55 955.214 L1377.32 956.17 L1377.09 955.733 L1376.86 956.285 L1376.63 956.209 L1376.41 956.796 L1376.18 956.819 L1375.95 956.289 L1375.72 956.183 L1375.5 955.991 L1375.27 955.645 L1375.04 955.909 L1374.81 956.46 L1374.58 955.952 L1374.36 956.151 L1374.13 956.293 L1373.9 955.469 L1373.67 954.941 L1373.45 954.736 L1373.22 954.688 L1372.99 954.36 L1372.76 954.314 L1372.53 954.86 L1372.31 954.9 L1372.08 954.75 L1371.85 956.231 L1371.62 956.895 L1371.4 956.983 L1371.17 955.821 L1370.94 956.522 L1370.71 956.893 L1370.48 957.457 L1370.26 956.876 L1370.03 957.069 L1369.8 956.843 L1369.57 956.942 L1369.35 956.993 L1369.12 957.029 L1368.89 956.672 L1368.66 955.855 L1368.43 955.209 L1368.21 954.694 L1367.98 954.639 L1367.75 954.868 L1367.52 954.918 L1367.3 955.16 L1367.07 955.591 L1366.84 955.245 L1366.61 954.646 L1366.39 954.722 L1366.16 955.168 L1365.93 954.828 L1365.7 954.565 L1365.47 954.028 L1365.25 953.888 L1365.02 954.613 L1364.79 954.298 L1364.56 954.395 L1364.34 954.111 L1364.11 955.734 L1363.88 956.519 L1363.65 956.269 L1363.42 957.276 L1363.2 956.683 L1362.97 957.412 L1362.74 957.351 L1362.51 957.117 L1362.29 957.192 L1362.06 957.66 L1361.83 957.072 L1361.6 956.331 L1361.37 955.769 L1361.15 956.466 L1360.92 956.19 L1360.69 956.167 L1360.46 955.98 L1360.24 955.722 L1360.01 955.061 L1359.78 955.506 L1359.55 956.185 L1359.32 955.894 L1359.1 955.334 L1358.87 954.997 L1358.64 954.648 L1358.41 955.453 L1358.19 954.641 L1357.96 954.339 L1357.73 954.489 L1357.5 954.188 L1357.27 954.965 L1357.05 955.437 L1356.82 955.83 L1356.59 956.812 L1356.36 956.217 L1356.14 957.477 L1355.91 956.501 L1355.68 955.8 L1355.45 955.887 L1355.22 955.574 L1355 955.408 L1354.77 956.015 L1354.54 955.181 L1354.31 955.084 L1354.09 955.467 L1353.86 955.79 L1353.63 954.861 L1353.4 954.566 L1353.17 954.622 L1352.95 955.536 L1352.72 956.068 L1352.49 956.346 L1352.26 956.248 L1352.04 955.756 L1351.81 955.508 L1351.58 955.176 L1351.35 954.675 L1351.12 955.203 L1350.9 954.913 L1350.67 955.033 L1350.44 955.352 L1350.21 955.403 L1349.99 955.926 L1349.76 956.163 L1349.53 955.478 L1349.3 954.279 L1349.07 953.624 L1348.85 953.605 L1348.62 952.849 L1348.39 952.656 L1348.16 952.274 L1347.94 951.808 L1347.71 950.67 L1347.48 950.808 L1347.25 950.339 L1347.02 949.769 L1346.8 948.467 L1346.57 948.018 L1346.34 947.402 L1346.11 947.362 L1345.89 947.972 L1345.66 947.982 L1345.43 947.464 L1345.2 946.707 L1344.97 946.754 L1344.75 945.844 L1344.52 945.892 L1344.29 946.519 L1344.06 945.645 L1343.84 946.091 L1343.61 946.258 L1343.38 945.633 L1343.15 945.626 L1342.92 946.02 L1342.7 946.909 L1342.47 946.603 L1342.24 946.155 L1342.01 945.961 L1341.79 946.78 L1341.56 946.678 L1341.33 945.494 L1341.1 944.53 L1340.87 944.033 L1340.65 945.462 L1340.42 944.799 L1340.19 945.615 L1339.96 946.005 L1339.74 946.106 L1339.51 946.329 L1339.28 944.944 L1339.05 945.259 L1338.82 944.787 L1338.6 944.812 L1338.37 944.473 L1338.14 944.907 L1337.91 944.717 L1337.69 943.868 L1337.46 944.01 L1337.23 944.743 L1337 944.063 L1336.77 942.637 L1336.55 942.656 L1336.32 942.606 L1336.09 942.972 L1335.86 942.721 L1335.64 942.35 L1335.41 942.922 L1335.18 942.273 L1334.95 942.556 L1334.72 941.864 L1334.5 941.907 L1334.27 941.908 L1334.04 941.513 L1333.81 941.735 L1333.59 941.023 L1333.36 940.538 L1333.13 940.412 L1332.9 939.699 L1332.67 939.688 L1332.45 940.034 L1332.22 940.273 L1331.99 940.014 L1331.76 939.816 L1331.54 939.491 L1331.31 939.51 L1331.08 938.865 L1330.85 939.508 L1330.62 939.294 L1330.4 938.181 L1330.17 937.694 L1329.94 936.349 L1329.71 936.301 L1329.49 935.794 L1329.26 935.114 L1329.03 934.877 L1328.8 934.688 L1328.58 934.392 L1328.35 933.937 L1328.12 933.992 L1327.89 934.518 L1327.66 934.048 L1327.44 933.833 L1327.21 933.699 L1326.98 932.751 L1326.75 932.191 L1326.53 931.918 L1326.3 931.595 L1326.07 930.952 L1325.84 931.325 L1325.61 930.713 L1325.39 931.199 L1325.16 930.724 L1324.93 930.8 L1324.7 931.43 L1324.48 931.417 L1324.25 931.554 L1324.02 931.668 L1323.79 930.424 L1323.56 930.51 L1323.34 930.98 L1323.11 931.213 L1322.88 931.294 L1322.65 931.883 L1322.43 932.458 L1322.2 932.026 L1321.97 932.945 L1321.74 932.892 L1321.51 932.617 L1321.29 932.147 L1321.06 931.973 L1320.83 931.145 L1320.6 931.647 L1320.38 931.458 L1320.15 931.663 L1319.92 932.026 L1319.69 932.695 L1319.46 932.871 L1319.24 932.912 L1319.01 933.343 L1318.78 933.32 L1318.55 933.339 L1318.33 934.581 L1318.1 934.936 L1317.87 934.931 L1317.64 934.496 L1317.41 934.707 L1317.19 934.039 L1316.96 933.683 L1316.73 934.084 L1316.5 933.386 L1316.28 933.877 L1316.05 933.639 L1315.82 933.656 L1315.59 934.013 L1315.36 933.942 L1315.14 933.314 L1314.91 933.202 L1314.68 933.214 L1314.45 933.657 L1314.23 934.595 L1314 934.086 L1313.77 934.036 L1313.54 934.589 L1313.31 933.935 L1313.09 934.459 L1312.86 934.458 L1312.63 935.676 L1312.4 935.19 L1312.18 935.259 L1311.95 934.525 L1311.72 933.537 L1311.49 933.344 L1311.26 932.797 L1311.04 932.393 L1310.81 931.462 L1310.58 930.728 L1310.35 930.292 L1310.13 930.233 L1309.9 929.692 L1309.67 929.701 L1309.44 929.868 L1309.21 929.894 L1308.99 929.068 L1308.76 929.14 L1308.53 930.445 L1308.3 931.081 L1308.08 930.823 L1307.85 931.073 L1307.62 931.989 L1307.39 932.59 L1307.16 931.587 L1306.94 931.575 L1306.71 931.704 L1306.48 931.036 L1306.25 930.418 L1306.03 931.132 L1305.8 930.828 L1305.57 930.393 L1305.34 930.558 L1305.11 931.375 L1304.89 930.527 L1304.66 929.473 L1304.43 929.317 L1304.2 929.672 L1303.98 929.009 L1303.75 928.698 L1303.52 929.05 L1303.29 929.008 L1303.06 928.698 L1302.84 927.611 L1302.61 926.437 L1302.38 926.068 L1302.15 925.835 L1301.93 925.862 L1301.7 926.967 L1301.47 927.323 L1301.24 927.557 L1301.01 926.695 L1300.79 926.871 L1300.56 926.946 L1300.33 926.877 L1300.1 926.778 L1299.88 927.054 L1299.65 927.823 L1299.42 928.697 L1299.19 928.812 L1298.96 928.689 L1298.74 928.298 L1298.51 928.121 L1298.28 927.826 L1298.05 927.809 L1297.83 929.472 L1297.6 928.583 L1297.37 928.982 L1297.14 928.537 L1296.91 928.252 L1296.69 928.71 L1296.46 930.745 L1296.23 930.849 L1296 931.051 L1295.78 930.801 L1295.55 930.174 L1295.32 930.833 L1295.09 930.244 L1294.86 931.044 L1294.64 931.141 L1294.41 931.123 L1294.18 931.513 L1293.95 931.255 L1293.73 930.754 L1293.5 930.491 L1293.27 931.201 L1293.04 930.73 L1292.82 930.764 L1292.59 930.743 L1292.36 930.918 L1292.13 931.932 L1291.9 931.068 L1291.68 931.243 L1291.45 930.572 L1291.22 930.043 L1290.99 931.105 L1290.77 930.358 L1290.54 930.482 L1290.31 930.286 L1290.08 929.245 L1289.85 929.443 L1289.63 930.171 L1289.4 929.827 L1289.17 929.929 L1288.94 928.98 L1288.72 928.982 L1288.49 928.651 L1288.26 929.129 L1288.03 929.077 L1287.8 929.863 L1287.58 929.761 L1287.35 930.091 L1287.12 929.777 L1286.89 929.649 L1286.67 929.498 L1286.44 929.142 L1286.21 929.419 L1285.98 928.867 L1285.75 929.015 L1285.53 928.578 L1285.3 927.945 L1285.07 927.588 L1284.84 926.935 L1284.62 926.944 L1284.39 926.633 L1284.16 925.847 L1283.93 925.615 L1283.7 925.195 L1283.48 924.845 L1283.25 925.115 L1283.02 924.595 L1282.79 925.309 L1282.57 924.576 L1282.34 924.207 L1282.11 923.922 L1281.88 923.257 L1281.65 923.416 L1281.43 923.364 L1281.2 923.063 L1280.97 924.332 L1280.74 924.46 L1280.52 923.427 L1280.29 924.417 L1280.06 925.206 L1279.83 924.412 L1279.6 925.162 L1279.38 925.22 L1279.15 925.403 L1278.92 924.862 L1278.69 924.923 L1278.47 923.874 L1278.24 923.456 L1278.01 924.285 L1277.78 925.206 L1277.55 925.297 L1277.33 924.963 L1277.1 924.409 L1276.87 923.725 L1276.64 924.157 L1276.42 924.249 L1276.19 923.777 L1275.96 924.849 L1275.73 924.238 L1275.5 924.143 L1275.28 925.279 L1275.05 924.788 L1274.82 924.866 L1274.59 924.252 L1274.37 924.225 L1274.14 924.37 L1273.91 925.735 L1273.68 925.42 L1273.45 925.108 L1273.23 924.602 L1273 924.329 L1272.77 925.466 L1272.54 924.962 L1272.32 924.294 L1272.09 925.635 L1271.86 926.522 L1271.63 927.096 L1271.4 926.934 L1271.18 926.536 L1270.95 925.677 L1270.72 925.796 L1270.49 926.515 L1270.27 926.717 L1270.04 927.776 L1269.81 927.321 L1269.58 927.383 L1269.35 926.231 L1269.13 925.891 L1268.9 926.132 L1268.67 924.76 L1268.44 924.396 L1268.22 924.394 L1267.99 923.804 L1267.76 923.031 L1267.53 924.828 L1267.3 923.918 L1267.08 923.355 L1266.85 923.305 L1266.62 922.858 L1266.39 922.659 L1266.17 923.405 L1265.94 922.096 L1265.71 920.491 L1265.48 920.989 L1265.25 921.746 L1265.03 920.643 L1264.8 920.227 L1264.57 920.006 L1264.34 919.801 L1264.12 920.481 L1263.89 920.339 L1263.66 920.196 L1263.43 919.708 L1263.2 919.501 L1262.98 919.519 L1262.75 919.75 L1262.52 920.145 L1262.29 919.803 L1262.07 918.833 L1261.84 918.472 L1261.61 919.597 L1261.38 920.322 L1261.15 919.339 L1260.93 918.952 L1260.7 918.686 L1260.47 919.344 L1260.24 919.09 L1260.02 919.616 L1259.79 921.714 L1259.56 921.694 L1259.33 921.46 L1259.1 921.318 L1258.88 921.352 L1258.65 921.803 L1258.42 922.139 L1258.19 922.042 L1257.97 921.871 L1257.74 921.694 L1257.51 921.678 L1257.28 922.392 L1257.06 921.941 L1256.83 921.384 L1256.6 920.555 L1256.37 919.887 L1256.14 920.213 L1255.92 920.112 L1255.69 919.384 L1255.46 919.303 L1255.23 919.026 L1255.01 919.265 L1254.78 919.055 L1254.55 919.133 L1254.32 919.234 L1254.09 919.437 L1253.87 918.951 L1253.64 919.824 L1253.41 919.566 L1253.18 920.022 L1252.96 921.545 L1252.73 921.381 L1252.5 921.348 L1252.27 921.494 L1252.04 922.843 L1251.82 922.768 L1251.59 921.844 L1251.36 920.968 L1251.13 919.838 L1250.91 920.048 L1250.68 920.869 L1250.45 919.995 L1250.22 919.79 L1249.99 919.488 L1249.77 920.347 L1249.54 920.289 L1249.31 920.713 L1249.08 920.026 L1248.86 920.662 L1248.63 920.086 L1248.4 919.716 L1248.17 919.024 L1247.94 918.823 L1247.72 919.472 L1247.49 918.813 L1247.26 918.45 L1247.03 918.673 L1246.81 920.005 L1246.58 919.812 L1246.35 920.871 L1246.12 921.561 L1245.89 921.859 L1245.67 921.789 L1245.44 922.024 L1245.21 921.161 L1244.98 921.954 L1244.76 921.812 L1244.53 921.536 L1244.3 921.512 L1244.07 921.203 L1243.84 921.23 L1243.62 920.395 L1243.39 919.773 L1243.16 920.467 L1242.93 920.045 L1242.71 919.702 L1242.48 919.521 L1242.25 920.884 L1242.02 920.342 L1241.79 920.79 L1241.57 920.434 L1241.34 920.457 L1241.11 919.969 L1240.88 920.155 L1240.66 919.974 L1240.43 920.249 L1240.2 920.555 L1239.97 920.704 L1239.74 921.964 L1239.52 921.11 L1239.29 921.53 L1239.06 922.404 L1238.83 922.959 L1238.61 921.796 L1238.38 922.039 L1238.15 923.816 L1237.92 924.227 L1237.69 924.569 L1237.47 924.609 L1237.24 924.783 L1237.01 925.027 L1236.78 924.635 L1236.56 924.928 L1236.33 924.849 L1236.1 923.866 L1235.87 923.686 L1235.64 924.514 L1235.42 924.959 L1235.19 925.684 L1234.96 925.115 L1234.73 923.749 L1234.51 923.415 L1234.28 923.157 L1234.05 922.783 L1233.82 923.745 L1233.59 923.242 L1233.37 923.664 L1233.14 922.878 L1232.91 922.517 L1232.68 920.896 L1232.46 921.143 L1232.23 921.694 L1232 921.334 L1231.77 921.255 L1231.54 921.147 L1231.32 921.66 L1231.09 921.5 L1230.86 921.422 L1230.63 920.302 L1230.41 920.334 L1230.18 920.455 L1229.95 920.141 L1229.72 919.678 L1229.49 919.164 L1229.27 918.864 L1229.04 919.114 L1228.81 919.13 L1228.58 919.897 L1228.36 919.425 L1228.13 919.409 L1227.9 919.286 L1227.67 918.779 L1227.44 918.438 L1227.22 917.461 L1226.99 916.747 L1226.76 916.742 L1226.53 916.311 L1226.31 916.569 L1226.08 915.681 L1225.85 915.609 L1225.62 915.222 L1225.39 915.24 L1225.17 916.131 L1224.94 915.2 L1224.71 915.441 L1224.48 915.043 L1224.26 914.229 L1224.03 914.491 L1223.8 914.645 L1223.57 913.764 L1223.34 912.595 L1223.12 912.055 L1222.89 911.017 L1222.66 910.705 L1222.43 911.448 L1222.21 911.889 L1221.98 911.887 L1221.75 911.543 L1221.52 910.464 L1221.29 910.77 L1221.07 911.018 L1220.84 910.984 L1220.61 911.579 L1220.38 911.344 L1220.16 910.712 L1219.93 910.573 L1219.7 911.216 L1219.47 910.931 L1219.25 911.592 L1219.02 911.201 L1218.79 912.287 L1218.56 912.819 L1218.33 913.463 L1218.11 913.215 L1217.88 912.941 L1217.65 912.972 L1217.42 913.226 L1217.2 913.132 L1216.97 913.641 L1216.74 913.039 L1216.51 912.388 L1216.28 911.647 L1216.06 911.601 L1215.83 911.842 L1215.6 911.302 L1215.37 911.17 L1215.15 910.582 L1214.92 911.833 L1214.69 911.548 L1214.46 911.64 L1214.23 912.866 L1214.01 913.254 L1213.78 912.799 L1213.55 913.853 L1213.32 913.954 L1213.1 913.958 L1212.87 913.009 L1212.64 912.875 L1212.41 913.925 L1212.18 914.248 L1211.96 915.398 L1211.73 915.469 L1211.5 915.619 L1211.27 914.855 L1211.05 914.581 L1210.82 914.937 L1210.59 914.679 L1210.36 915.292 L1210.13 914.932 L1209.91 913.951 L1209.68 914.095 L1209.45 913.633 L1209.22 915.3 L1209 915.035 L1208.77 914.527 L1208.54 913.987 L1208.31 913.639 L1208.08 913.637 L1207.86 913.734 L1207.63 912.954 L1207.4 912.719 L1207.17 913.229 L1206.95 912.276 L1206.72 912.358 L1206.49 911.997 L1206.26 911.769 L1206.03 912.563 L1205.81 911.923 L1205.58 912.573 L1205.35 912.163 L1205.12 912.684 L1204.9 913.249 L1204.67 912.956 L1204.44 913.471 L1204.21 913.688 L1203.98 914.25 L1203.76 915.03 L1203.53 914.75 L1203.3 913.249 L1203.07 912.746 L1202.85 915.102 L1202.62 915.033 L1202.39 915.534 L1202.16 915.547 L1201.93 916.248 L1201.71 916.76 L1201.48 916.37 L1201.25 916.536 L1201.02 916.166 L1200.8 915.95 L1200.57 916.104 L1200.34 915.253 L1200.11 915.041 L1199.88 914.301 L1199.66 914.833 L1199.43 914.099 L1199.2 914.811 L1198.97 914.891 L1198.75 916.064 L1198.52 916.231 L1198.29 915.924 L1198.06 915.797 L1197.83 915.738 L1197.61 916.044 L1197.38 916.583 L1197.15 916.834 L1196.92 916.294 L1196.7 916.189 L1196.47 915.695 L1196.24 914.989 L1196.01 916.56 L1195.78 916.594 L1195.56 917.038 L1195.33 915.549 L1195.1 916.023 L1194.87 914.902 L1194.65 916.008 L1194.42 914.905 L1194.19 914.632 L1193.96 913.254 L1193.73 913.608 L1193.51 914.011 L1193.28 913.696 L1193.05 914.216 L1192.82 914.202 L1192.6 913.32 L1192.37 913.881 L1192.14 913.587 L1191.91 914.543 L1191.68 915.912 L1191.46 916.914 L1191.23 917.671 L1191 917.613 L1190.77 916.763 L1190.55 916.397 L1190.32 915.78 L1190.09 915.363 L1189.86 913.949 L1189.63 914.293 L1189.41 915.097 L1189.18 916.229 L1188.95 916.536 L1188.72 915.442 L1188.5 914.283 L1188.27 914.399 L1188.04 914.474 L1187.81 913.624 L1187.58 913.422 L1187.36 914.158 L1187.13 912.599 L1186.9 914.37 L1186.67 915.977 L1186.45 915.878 L1186.22 916.098 L1185.99 915.977 L1185.76 915.67 L1185.53 914.665 L1185.31 914.313 L1185.08 914.366 L1184.85 913.126 L1184.62 912.691 L1184.4 913.114 L1184.17 912.698 L1183.94 911.456 L1183.71 910.736 L1183.49 910.426 L1183.26 909.97 L1183.03 909.907 L1182.8 911.453 L1182.57 911.663 L1182.35 911.829 L1182.12 911.77 L1181.89 912.754 L1181.66 912.722 L1181.44 911.894 L1181.21 911.689 L1180.98 911.612 L1180.75 911.173 L1180.52 909.94 L1180.3 911.571 L1180.07 910.958 L1179.84 910.384 L1179.61 911.215 L1179.39 911.75 L1179.16 913.128 L1178.93 913.334 L1178.7 912.346 L1178.47 912.355 L1178.25 912.854 L1178.02 913.419 L1177.79 912.712 L1177.56 912.884 L1177.34 912.948 L1177.11 912.818 L1176.88 913.315 L1176.65 912.455 L1176.42 911.697 L1176.2 912.024 L1175.97 913.917 L1175.74 913.084 L1175.51 912.723 L1175.29 913.558 L1175.06 913.27 L1174.83 912.657 L1174.6 912.542 L1174.37 912.449 L1174.15 911.721 L1173.92 912.507 L1173.69 911.995 L1173.46 912.457 L1173.24 913.807 L1173.01 914.703 L1172.78 916.297 L1172.55 916.034 L1172.32 915.497 L1172.1 916.374 L1171.87 915.782 L1171.64 916.043 L1171.41 915.568 L1171.19 914.544 L1170.96 915.227 L1170.73 915.192 L1170.5 915.601 L1170.27 914.242 L1170.05 913.408 L1169.82 913.65 L1169.59 913.033 L1169.36 913.578 L1169.14 913.162 L1168.91 913.67 L1168.68 914.918 L1168.45 915.82 L1168.22 915.841 L1168 915.749 L1167.77 915.101 L1167.54 914.4 L1167.31 913.909 L1167.09 914.34 L1166.86 914.33 L1166.63 914.628 L1166.4 914.993 L1166.17 913.886 L1165.95 914.443 L1165.72 914.876 L1165.49 914.026 L1165.26 914.721 L1165.04 915.137 L1164.81 914.958 L1164.58 915.436 L1164.35 915.742 L1164.12 915.964 L1163.9 916.131 L1163.67 917.359 L1163.44 917.048 L1163.21 917.387 L1162.99 918.107 L1162.76 917.833 L1162.53 918.001 L1162.3 918.396 L1162.07 917.829 L1161.85 917.229 L1161.62 917.137 L1161.39 917.988 L1161.16 917.637 L1160.94 917.165 L1160.71 917.552 L1160.48 918.02 L1160.25 916.744 L1160.02 915.765 L1159.8 916.16 L1159.57 916.292 L1159.34 915.825 L1159.11 917.364 L1158.89 918.533 L1158.66 918.986 L1158.43 918.997 L1158.2 918.991 L1157.97 918.071 L1157.75 918.227 L1157.52 917.906 L1157.29 917.197 L1157.06 917.915 L1156.84 917.345 L1156.61 916.881 L1156.38 916.568 L1156.15 915.64 L1155.92 916.234 L1155.7 915.46 L1155.47 915.214 L1155.24 915.443 L1155.01 915.371 L1154.79 914.459 L1154.56 913.891 L1154.33 913.802 L1154.1 913.317 L1153.87 912.118 L1153.65 911.656 L1153.42 910.989 L1153.19 911.172 L1152.96 909.806 L1152.74 910.165 L1152.51 909.861 L1152.28 910.209 L1152.05 909.549 L1151.82 909.053 L1151.6 908.148 L1151.37 909.211 L1151.14 909.071 L1150.91 909.073 L1150.69 909.224 L1150.46 908.458 L1150.23 909.019 L1150 909.043 L1149.77 909.101 L1149.55 907.919 L1149.32 907.884 L1149.09 908.911 L1148.86 909.036 L1148.64 909.684 L1148.41 909.474 L1148.18 910.553 L1147.95 909.986 L1147.73 909.964 L1147.5 911.585 L1147.27 910.79 L1147.04 910.658 L1146.81 909.751 L1146.59 910.009 L1146.36 910.078 L1146.13 911.23 L1145.9 911.115 L1145.68 911.003 L1145.45 910.685 L1145.22 909.906 L1144.99 910.117 L1144.76 910.75 L1144.54 909.913 L1144.31 909.508 L1144.08 908.499 L1143.85 909.492 L1143.63 909.583 L1143.4 909.033 L1143.17 908.492 L1142.94 908.461 L1142.71 908.259 L1142.49 908.402 L1142.26 909.425 L1142.03 910.198 L1141.8 910.463 L1141.58 911.743 L1141.35 911.873 L1141.12 911.91 L1140.89 911.321 L1140.66 910.617 L1140.44 911.939 L1140.21 911.674 L1139.98 911.18 L1139.75 911.421 L1139.53 910.995 L1139.3 910.101 L1139.07 909.656 L1138.84 910.315 L1138.61 909.331 L1138.39 910.262 L1138.16 910.268 L1137.93 910.81 L1137.7 911.918 L1137.48 913.001 L1137.25 911.606 L1137.02 911.206 L1136.79 909.952 L1136.56 910.247 L1136.34 910.086 L1136.11 910.474 L1135.88 909.894 L1135.65 909.168 L1135.43 909.041 L1135.2 909.835 L1134.97 908.999 L1134.74 909.895 L1134.51 909.043 L1134.29 908.704 L1134.06 909.715 L1133.83 911.019 L1133.6 912.731 L1133.38 911.192 L1133.15 910.91 L1132.92 910.168 L1132.69 911.953 L1132.46 911.475 L1132.24 911.042 L1132.01 910.575 L1131.78 910.14 L1131.55 909.675 L1131.33 909.19 L1131.1 910.55 L1130.87 911.227 L1130.64 911.028 L1130.41 911.776 L1130.19 911.052 L1129.96 911.279 L1129.73 912.245 L1129.5 912.217 L1129.28 911.678 L1129.05 910.252 L1128.82 909.586 L1128.59 910.23 L1128.36 911.079 L1128.14 911.532 L1127.91 913.832 L1127.68 913.021 L1127.45 912.629 L1127.23 913.033 L1127 912.464 L1126.77 912.721 L1126.54 912.116 L1126.31 911.449 L1126.09 911.158 L1125.86 910.926 L1125.63 909.918 L1125.4 910.196 L1125.18 909.135 L1124.95 910.309 L1124.72 910.43 L1124.49 910.55 L1124.26 910.124 L1124.04 910.576 L1123.81 910.026 L1123.58 909.629 L1123.35 909.544 L1123.13 909.384 L1122.9 909.01 L1122.67 908.265 L1122.44 907.242 L1122.21 906.557 L1121.99 907.2 L1121.76 906.359 L1121.53 906.488 L1121.3 906.316 L1121.08 905.044 L1120.85 905.922 L1120.62 906.572 L1120.39 907.205 L1120.16 907.65 L1119.94 907.701 L1119.71 907.251 L1119.48 906.065 L1119.25 906.975 L1119.03 908.095 L1118.8 907.559 L1118.57 907.866 L1118.34 907.452 L1118.11 905.717 L1117.89 905.38 L1117.66 904.776 L1117.43 904.553 L1117.2 904.476 L1116.98 904.044 L1116.75 903.436 L1116.52 902.976 L1116.29 902.273 L1116.06 902.989 L1115.84 903.606 L1115.61 903.951 L1115.38 903.547 L1115.15 903.428 L1114.93 904.224 L1114.7 903.483 L1114.47 902.801 L1114.24 902.148 L1114.01 902.715 L1113.79 902.921 L1113.56 903.157 L1113.33 902.629 L1113.1 901.877 L1112.88 902.622 L1112.65 903.207 L1112.42 902.331 L1112.19 902.388 L1111.97 902.129 L1111.74 901.731 L1111.51 902.092 L1111.28 901.717 L1111.05 900.684 L1110.83 901.105 L1110.6 899.83 L1110.37 900.365 L1110.14 901.164 L1109.92 901.473 L1109.69 902.206 L1109.46 901.534 L1109.23 901.011 L1109 899.604 L1108.78 899.24 L1108.55 899.877 L1108.32 900.635 L1108.09 899.683 L1107.87 899.316 L1107.64 898.585 L1107.41 897.163 L1107.18 897.465 L1106.95 897.042 L1106.73 896.638 L1106.5 896.655 L1106.27 896.754 L1106.04 896.656 L1105.82 896.348 L1105.59 896.848 L1105.36 897.725 L1105.13 897.803 L1104.9 896.585 L1104.68 896.209 L1104.45 895.432 L1104.22 895.653 L1103.99 895.141 L1103.77 894.562 L1103.54 895.044 L1103.31 894.215 L1103.08 893.878 L1102.85 895.345 L1102.63 894.793 L1102.4 894.344 L1102.17 894.097 L1101.94 896.203 L1101.72 897.371 L1101.49 897.323 L1101.26 897.841 L1101.03 899.466 L1100.8 899.379 L1100.58 898.557 L1100.35 897.903 L1100.12 897.557 L1099.89 898.181 L1099.67 898.902 L1099.44 899.474 L1099.21 899.722 L1098.98 899.04 L1098.75 899.058 L1098.53 899.078 L1098.3 897.928 L1098.07 897.127 L1097.84 896.556 L1097.62 896.842 L1097.39 896.865 L1097.16 896.068 L1096.93 895.914 L1096.7 896.646 L1096.48 896.74 L1096.25 897.159 L1096.02 899.09 L1095.79 898.986 L1095.57 899.1 L1095.34 898.316 L1095.11 897.242 L1094.88 897.848 L1094.65 898.901 L1094.43 900.58 L1094.2 900.169 L1093.97 899.877 L1093.74 900.414 L1093.52 899.636 L1093.29 898.921 L1093.06 899.015 L1092.83 897.951 L1092.6 897.353 L1092.38 896.595 L1092.15 900.84 L1091.92 900.208 L1091.69 899.468 L1091.47 899.028 L1091.24 899.107 L1091.01 898.773 L1090.78 899.145 L1090.55 899.124 L1090.33 899.079 L1090.1 898.569 L1089.87 897.985 L1089.64 898.506 L1089.42 896.764 L1089.19 896.921 L1088.96 896.367 L1088.73 896.354 L1088.5 897.09 L1088.28 896.399 L1088.05 896.513 L1087.82 895.531 L1087.59 895.674 L1087.37 896.339 L1087.14 895.862 L1086.91 894.854 L1086.68 894.79 L1086.45 895.527 L1086.23 895.166 L1086 896.112 L1085.77 895.297 L1085.54 896.548 L1085.32 896.212 L1085.09 896.866 L1084.86 897.432 L1084.63 897.929 L1084.4 896.969 L1084.18 897.72 L1083.95 898.682 L1083.72 898.43 L1083.49 897.949 L1083.27 898.195 L1083.04 898.868 L1082.81 898.752 L1082.58 898.718 L1082.35 899.312 L1082.13 898.348 L1081.9 897.844 L1081.67 897.071 L1081.44 896.628 L1081.22 897.763 L1080.99 897.706 L1080.76 898.398 L1080.53 899.085 L1080.3 899.142 L1080.08 899.075 L1079.85 898.734 L1079.62 900.716 L1079.39 901.451 L1079.17 902.649 L1078.94 902.521 L1078.71 901.785 L1078.48 902.176 L1078.25 902.833 L1078.03 902.847 L1077.8 901.343 L1077.57 902.023 L1077.34 902.34 L1077.12 902.653 L1076.89 901.252 L1076.66 900.919 L1076.43 902.466 L1076.2 901.955 L1075.98 901.446 L1075.75 900.444 L1075.52 899.581 L1075.29 899.49 L1075.07 898.315 L1074.84 899.266 L1074.61 899.397 L1074.38 899.23 L1074.16 900.145 L1073.93 898.911 L1073.7 898.467 L1073.47 898.471 L1073.24 898.171 L1073.02 898.147 L1072.79 897.908 L1072.56 896.695 L1072.33 896.259 L1072.11 895.548 L1071.88 895.2 L1071.65 894.722 L1071.42 894.82 L1071.19 894.372 L1070.97 892.889 L1070.74 891.998 L1070.51 892.45 L1070.28 891.56 L1070.06 890.449 L1069.83 889.681 L1069.6 888.72 L1069.37 888.454 L1069.14 887.832 L1068.92 888.227 L1068.69 889.05 L1068.46 888.518 L1068.23 888.354 L1068.01 887.956 L1067.78 887.497 L1067.55 886.959 L1067.32 888.133 L1067.09 887.584 L1066.87 887.93 L1066.64 887.138 L1066.41 889.648 L1066.18 889.941 L1065.96 890.068 L1065.73 890.246 L1065.5 890.567 L1065.27 890.277 L1065.04 890.377 L1064.82 891.448 L1064.59 891.908 L1064.36 892.307 L1064.13 892.014 L1063.91 891.889 L1063.68 891.701 L1063.45 891.437 L1063.22 891.039 L1062.99 891.249 L1062.77 889.575 L1062.54 889.385 L1062.31 889.309 L1062.08 888.989 L1061.86 889.031 L1061.63 888.745 L1061.4 887.943 L1061.17 889.608 L1060.94 888.802 L1060.72 888.234 L1060.49 888.346 L1060.26 888.247 L1060.03 887.773 L1059.81 888.402 L1059.58 888.924 L1059.35 889.962 L1059.12 889.527 L1058.89 889.326 L1058.67 888.98 L1058.44 889.446 L1058.21 890.16 L1057.98 888.997 L1057.76 887.74 L1057.53 887.241 L1057.3 888.229 L1057.07 888.224 L1056.84 887.993 L1056.62 887.069 L1056.39 887.526 L1056.16 887.964 L1055.93 887.701 L1055.71 887.008 L1055.48 887.288 L1055.25 887.478 L1055.02 887.111 L1054.79 887.348 L1054.57 886.212 L1054.34 885.406 L1054.11 885.768 L1053.88 885.57 L1053.66 885.788 L1053.43 885.965 L1053.2 885.948 L1052.97 885.736 L1052.74 887.033 L1052.52 888.858 L1052.29 888.668 L1052.06 889.093 L1051.83 888.915 L1051.61 888.455 L1051.38 887.87 L1051.15 887.686 L1050.92 886.563 L1050.69 888.456 L1050.47 888.852 L1050.24 889.474 L1050.01 889.891 L1049.78 888.636 L1049.56 888.738 L1049.33 888.872 L1049.1 889.064 L1048.87 890.148 L1048.64 889.881 L1048.42 889.252 L1048.19 890.352 L1047.96 889.324 L1047.73 889.16 L1047.51 890.145 L1047.28 890.507 L1047.05 889.514 L1046.82 889.881 L1046.59 891.075 L1046.37 890.22 L1046.14 889.986 L1045.91 890.404 L1045.68 890.733 L1045.46 889.832 L1045.23 890.974 L1045 891.75 L1044.77 891.417 L1044.54 891.487 L1044.32 891.322 L1044.09 891.129 L1043.86 890.389 L1043.63 890.269 L1043.41 890.25 L1043.18 889.471 L1042.95 889.164 L1042.72 888.678 L1042.49 889.183 L1042.27 889.703 L1042.04 890.139 L1041.81 891.434 L1041.58 891.11 L1041.36 890.042 L1041.13 890.086 L1040.9 890.222 L1040.67 889.724 L1040.44 888.072 L1040.22 887.823 L1039.99 887.008 L1039.76 887.116 L1039.53 887.029 L1039.31 887.24 L1039.08 886.326 L1038.85 886.42 L1038.62 886.532 L1038.4 885.734 L1038.17 885.39 L1037.94 886.35 L1037.71 885.642 L1037.48 884.969 L1037.26 884.449 L1037.03 884.681 L1036.8 884.926 L1036.57 884.537 L1036.35 882.978 L1036.12 882.701 L1035.89 883.738 L1035.66 884.197 L1035.43 884.194 L1035.21 884.001 L1034.98 884.789 L1034.75 884.775 L1034.52 884.349 L1034.3 884.845 L1034.07 883.807 L1033.84 884.467 L1033.61 885.77 L1033.38 885.056 L1033.16 885.521 L1032.93 885.194 L1032.7 886.454 L1032.47 886.01 L1032.25 885.446 L1032.02 886.492 L1031.79 886.906 L1031.56 886.469 L1031.33 886.529 L1031.11 888.242 L1030.88 888.621 L1030.65 889.333 L1030.42 888.868 L1030.2 888.228 L1029.97 888.861 L1029.74 888.929 L1029.51 888.772 L1029.28 890.74 L1029.06 891.515 L1028.83 892.081 L1028.6 892.106 L1028.37 892.131 L1028.15 891.423 L1027.92 891.121 L1027.69 892.067 L1027.46 891.592 L1027.23 890.499 L1027.01 889.5 L1026.78 888.897 L1026.55 887.924 L1026.32 887.251 L1026.1 885.834 L1025.87 886.552 L1025.64 885.819 L1025.41 887.464 L1025.18 887.299 L1024.96 887.576 L1024.73 887.66 L1024.5 886.708 L1024.27 885.167 L1024.05 885.019 L1023.82 885.594 L1023.59 885.425 L1023.36 886.424 L1023.13 885.332 L1022.91 885.429 L1022.68 885.561 L1022.45 885.581 L1022.22 884.431 L1022 884.916 L1021.77 884.972 L1021.54 885.062 L1021.31 883.431 L1021.08 884.103 L1020.86 883.273 L1020.63 882.343 L1020.4 883.024 L1020.17 881.605 L1019.95 881.359 L1019.72 882.324 L1019.49 881.761 L1019.26 881.484 L1019.03 882.659 L1018.81 883.387 L1018.58 883.226 L1018.35 882.964 L1018.12 883.834 L1017.9 883.901 L1017.67 883.531 L1017.44 884.359 L1017.21 884.657 L1016.98 885.114 L1016.76 884.475 L1016.53 883.698 L1016.3 883.98 L1016.07 886.331 L1015.85 886.068 L1015.62 886.836 L1015.39 886.2 L1015.16 886.522 L1014.93 886.696 L1014.71 886.732 L1014.48 886.77 L1014.25 888.359 L1014.02 888.015 L1013.8 889.166 L1013.57 890.062 L1013.34 890.115 L1013.11 889.418 L1012.88 888.807 L1012.66 890.701 L1012.43 889.707 L1012.2 889.131 L1011.97 888.514 L1011.75 888.205 L1011.52 889.66 L1011.29 888.768 L1011.06 889.483 L1010.83 887.928 L1010.61 887.763 L1010.38 887.542 L1010.15 886.648 L1009.92 887.317 L1009.7 887.868 L1009.47 887.807 L1009.24 887.592 L1009.01 888.082 L1008.78 887.503 L1008.56 887.151 L1008.33 887.826 L1008.1 887.523 L1007.87 887.277 L1007.65 886.775 L1007.42 887.006 L1007.19 887.029 L1006.96 886.278 L1006.73 887.447 L1006.51 888.769 L1006.28 888.085 L1006.05 886.751 L1005.82 887.602 L1005.6 886.472 L1005.37 885.958 L1005.14 885.055 L1004.91 883.501 L1004.68 884.903 L1004.46 885.232 L1004.23 885.289 L1004 884.951 L1003.77 885.02 L1003.55 885.512 L1003.32 884.302 L1003.09 885.102 L1002.86 883.578 L1002.64 884.698 L1002.41 884.594 L1002.18 884.866 L1001.95 884.412 L1001.72 886.572 L1001.5 886.324 L1001.27 886.368 L1001.04 885.904 L1000.81 886.241 L1000.59 886.379 L1000.36 886.101 L1000.13 885.741 L999.902 885.354 L999.674 887.866 L999.446 887.043 L999.218 885.542 L998.991 885.067 L998.763 886.988 L998.535 887.055 L998.307 886.281 L998.08 884.84 L997.852 884.848 L997.624 884.061 L997.396 883.452 L997.169 881.986 L996.941 880.997 L996.713 881.249 L996.485 881.645 L996.257 881.677 L996.03 881.003 L995.802 880.561 L995.574 880.302 L995.346 880.825 L995.119 880.036 L994.891 879.475 L994.663 878.637 L994.435 877.179 L994.207 878.131 L993.98 877.489 L993.752 876.135 L993.524 876.029 L993.296 877.684 L993.069 876.995 L992.841 878.897 L992.613 879.464 L992.385 878.588 L992.158 878.105 L991.93 877.702 L991.702 876.426 L991.474 877.835 L991.246 878.887 L991.019 880.661 L990.791 880.385 L990.563 879.686 L990.335 880.141 L990.108 880.359 L989.88 880.376 L989.652 879.825 L989.424 879.598 L989.197 881.79 L988.969 881.605 L988.741 881.822 L988.513 880.922 L988.285 880.787 L988.058 880.132 L987.83 882.062 L987.602 882.209 L987.374 881.456 L987.147 881.057 L986.919 880.486 L986.691 881.183 L986.463 882.091 L986.236 884.197 L986.008 883.859 L985.78 885.305 L985.552 885.705 L985.324 884.549 L985.097 885.377 L984.869 886.432 L984.641 885.945 L984.413 885.657 L984.186 887.164 L983.958 888.595 L983.73 887.853 L983.502 887.793 L983.274 889.491 L983.047 889.568 L982.819 889.57 L982.591 888.397 L982.363 890.219 L982.136 889.57 L981.908 889.254 L981.68 887.742 L981.452 887.87 L981.225 887.306 L980.997 887.296 L980.769 887.847 L980.541 888.091 L980.313 887.867 L980.086 887.828 L979.858 889.201 L979.63 889.68 L979.402 889.126 L979.175 889.742 L978.947 890.662 L978.719 888.863 L978.491 889.037 L978.264 888.221 L978.036 887.044 L977.808 887.078 L977.58 887.955 L977.352 886.872 L977.125 886.641 L976.897 888.112 L976.669 887.751 L976.441 888.167 L976.214 888.092 L975.986 889.695 L975.758 888.403 L975.53 890.441 L975.303 890.478 L975.075 890.346 L974.847 890.524 L974.619 890.465 L974.391 890.486 L974.164 890.617 L973.936 891.319 L973.708 890.492 L973.48 889.475 L973.253 889.084 L973.025 889.979 L972.797 888.905 L972.569 888.018 L972.341 889.171 L972.114 889.096 L971.886 889.846 L971.658 888.466 L971.43 888.816 L971.203 890.059 L970.975 889.162 L970.747 889.253 L970.519 888.426 L970.292 887.405 L970.064 888.367 L969.836 888.056 L969.608 887.278 L969.38 888.105 L969.153 888.182 L968.925 888.046 L968.697 887.32 L968.469 888.39 L968.242 887.511 L968.014 887.425 L967.786 886.92 L967.558 886.02 L967.331 884.833 L967.103 883.83 L966.875 883.207 L966.647 885.194 L966.419 884.673 L966.192 884.459 L965.964 884.861 L965.736 885.268 L965.508 884.097 L965.281 882.83 L965.053 881.805 L964.825 881.048 L964.597 880.113 L964.37 880.971 L964.142 880.708 L963.914 880.21 L963.686 880.407 L963.458 878.761 L963.231 878.235 L963.003 877.545 L962.775 878.063 L962.547 878.962 L962.32 878.492 L962.092 878.077 L961.864 877.857 L961.636 877.816 L961.408 878.511 L961.181 878.018 L960.953 878.36 L960.725 880.12 L960.497 877.924 L960.27 878.172 L960.042 877.035 L959.814 876.619 L959.586 876.545 L959.359 876.71 L959.131 876.552 L958.903 877.574 L958.675 876.848 L958.447 876.413 L958.22 875.77 L957.992 876.399 L957.764 875.88 L957.536 875.633 L957.309 876.208 L957.081 876.352 L956.853 877.954 L956.625 877.512 L956.398 878.803 L956.17 879.451 L955.942 878.222 L955.714 877.436 L955.486 877.199 L955.259 877.432 L955.031 877.376 L954.803 880.244 L954.575 881.202 L954.348 881.457 L954.12 880.897 L953.892 881.433 L953.664 882.283 L953.437 881.281 L953.209 880.218 L952.981 879.586 L952.753 879.044 L952.525 878.009 L952.298 877.961 L952.07 877.001 L951.842 878.282 L951.614 878.401 L951.387 877.527 L951.159 878.612 L950.931 879.645 L950.703 879.018 L950.475 877.487 L950.248 876.516 L950.02 875.403 L949.792 875.651 L949.564 874.67 L949.337 874.825 L949.109 875.343 L948.881 873.898 L948.653 872.167 L948.426 871.173 L948.198 872.601 L947.97 871.481 L947.742 870.534 L947.514 870.014 L947.287 873.327 L947.059 872.922 L946.831 876.093 L946.603 876.405 L946.376 876.612 L946.148 877.038 L945.92 877.239 L945.692 877.952 L945.465 878.808 L945.237 878.857 L945.009 879.599 L944.781 879.405 L944.553 879.856 L944.326 880.771 L944.098 881.331 L943.87 880.969 L943.642 881.335 L943.415 880.388 L943.187 881.474 L942.959 880.381 L942.731 880.539 L942.504 880.942 L942.276 881.907 L942.048 881.254 L941.82 880.93 L941.592 879.241 L941.365 879.309 L941.137 879.66 L940.909 879.901 L940.681 880.207 L940.454 879.042 L940.226 879.072 L939.998 879.567 L939.77 879.752 L939.542 878.761 L939.315 878.146 L939.087 877.2 L938.859 876.024 L938.631 876.071 L938.404 876.102 L938.176 875.935 L937.948 876.176 L937.72 876.383 L937.493 877.263 L937.265 876.505 L937.037 876.085 L936.809 874.935 L936.581 874.848 L936.354 875.073 L936.126 874.829 L935.898 874.18 L935.67 874.103 L935.443 874.407 L935.215 874.084 L934.987 877.036 L934.759 878.117 L934.532 877.934 L934.304 877.535 L934.076 876.866 L933.848 876.245 L933.62 876.399 L933.393 877.699 L933.165 875.444 L932.937 874.182 L932.709 873.196 L932.482 873.64 L932.254 873.838 L932.026 874.23 L931.798 874.05 L931.571 874.351 L931.343 873.575 L931.115 873.647 L930.887 872.382 L930.659 871.66 L930.432 871.333 L930.204 869.841 L929.976 869.391 L929.748 868.401 L929.521 867.646 L929.293 867.249 L929.065 865.759 L928.837 864.691 L928.609 864.104 L928.382 864.922 L928.154 864.651 L927.926 865.675 L927.698 865.728 L927.471 864.466 L927.243 865.419 L927.015 864.773 L926.787 863.762 L926.56 863.118 L926.332 863.584 L926.104 862.048 L925.876 864.437 L925.648 864.379 L925.421 864.064 L925.193 863.83 L924.965 863.686 L924.737 862.418 L924.51 862.393 L924.282 862.014 L924.054 860.517 L923.826 861.053 L923.599 862.896 L923.371 864.503 L923.143 865.14 L922.915 865.028 L922.687 864.963 L922.46 867.054 L922.232 865.804 L922.004 864.717 L921.776 865.337 L921.549 863.948 L921.321 863.118 L921.093 862.083 L920.865 862.947 L920.638 862.094 L920.41 860.586 L920.182 860.315 L919.954 860.024 L919.726 860.099 L919.499 859.981 L919.271 861.203 L919.043 860.932 L918.815 861.133 L918.588 861.484 L918.36 860.534 L918.132 859.152 L917.904 858.451 L917.676 857.391 L917.449 857.261 L917.221 859.827 L916.993 859.743 L916.765 858.439 L916.538 858.341 L916.31 858.458 L916.082 857.307 L915.854 857.016 L915.627 855.599 L915.399 854.958 L915.171 856.055 L914.943 855.699 L914.715 854.041 L914.488 854.728 L914.26 852.949 L914.032 851.424 L913.804 852.908 L913.577 853.146 L913.349 853.703 L913.121 853.06 L912.893 856.009 L912.666 857.512 L912.438 856.956 L912.21 855.993 L911.982 855.81 L911.754 854.481 L911.527 853.055 L911.299 851.095 L911.071 850.595 L910.843 850.241 L910.616 850.787 L910.388 851.038 L910.16 851.816 L909.932 851.561 L909.705 851.18 L909.477 850.283 L909.249 850.59 L909.021 850.731 L908.793 849.648 L908.566 849.403 L908.338 847.663 L908.11 847.788 L907.882 849.442 L907.655 848.975 L907.427 848.056 L907.199 847.953 L906.971 849.614 L906.743 849.268 L906.516 849.675 L906.288 850.204 L906.06 849.556 L905.832 848.197 L905.605 847.945 L905.377 846.248 L905.149 845.828 L904.921 845.596 L904.694 845.93 L904.466 844.792 L904.238 843.105 L904.01 842.62 L903.782 842.884 L903.555 843.353 L903.327 843.041 L903.099 842.002 L902.871 843.313 L902.644 844.797 L902.416 845.452 L902.188 845.664 L901.96 845.268 L901.733 845.127 L901.505 845.018 L901.277 847.889 L901.049 848.816 L900.821 849.85 L900.594 849.035 L900.366 849.645 L900.138 848.336 L899.91 848.095 L899.683 847.803 L899.455 848.267 L899.227 849.15 L898.999 851.45 L898.772 851.895 L898.544 850.66 L898.316 850.804 L898.088 850.907 L897.86 849.948 L897.633 853.389 L897.405 853.878 L897.177 855.478 L896.949 855.845 L896.722 855.055 L896.494 854.758 L896.266 854.99 L896.038 853.821 L895.81 853.502 L895.583 851.939 L895.355 852.905 L895.127 853.257 L894.899 852.293 L894.672 853.137 L894.444 854.807 L894.216 854.499 L893.988 854.208 L893.761 855.594 L893.533 856.189 L893.305 857.743 L893.077 856.872 L892.849 857.766 L892.622 861.257 L892.394 861.085 L892.166 859.776 L891.938 860.514 L891.711 860.32 L891.483 860.444 L891.255 860.044 L891.027 858.953 L890.8 857.73 L890.572 860.154 L890.344 860.179 L890.116 862.176 L889.888 861.354 L889.661 860.687 L889.433 862.026 L889.205 861.353 L888.977 863.61 L888.75 862.994 L888.522 864.321 L888.294 863.849 L888.066 864.561 L887.839 865.07 L887.611 865.867 L887.383 865.735 L887.155 864.973 L886.927 863.084 L886.7 861.584 L886.472 861.951 L886.244 864.78 L886.016 865.176 L885.789 864.308 L885.561 864.928 L885.333 865.481 L885.105 865.204 L884.877 864.417 L884.65 864.387 L884.422 866.65 L884.194 866.015 L883.966 865.62 L883.739 866.019 L883.511 865.878 L883.283 868.283 L883.055 867.557 L882.828 867.278 L882.6 869.776 L882.372 868.934 L882.144 868.113 L881.916 866.37 L881.689 864.583 L881.461 862.68 L881.233 861.552 L881.005 861.385 L880.778 861.743 L880.55 861.335 L880.322 861.413 L880.094 862.783 L879.867 862.164 L879.639 861.861 L879.411 860.693 L879.183 860.264 L878.955 860.523 L878.728 859.223 L878.5 858.399 L878.272 857.431 L878.044 856.522 L877.817 857.555 L877.589 857.606 L877.361 857.282 L877.133 857.883 L876.906 859.685 L876.678 857.621 L876.45 856.336 L876.222 855.191 L875.994 855.139 L875.767 855.642 L875.539 854.587 L875.311 854.799 L875.083 853.684 L874.856 853.896 L874.628 854.707 L874.4 856.733 L874.172 854.59 L873.944 853.08 L873.717 853.085 L873.489 853.426 L873.261 854.75 L873.033 854.862 L872.806 853.743 L872.578 854.627 L872.35 853.866 L872.122 853.712 L871.895 854.96 L871.667 856.132 L871.439 855.034 L871.211 854.726 L870.983 854.854 L870.756 854.108 L870.528 854.196 L870.3 853.529 L870.072 852.557 L869.845 852.252 L869.617 851.611 L869.389 850.396 L869.161 849.977 L868.934 850.998 L868.706 850.835 L868.478 851.738 L868.25 851.839 L868.022 853.105 L867.795 851.675 L867.567 851.699 L867.339 852.438 L867.111 851.752 L866.884 853.002 L866.656 853.17 L866.428 854.038 L866.2 854.037 L865.973 852.704 L865.745 851.693 L865.517 853.296 L865.289 851.669 L865.061 853.909 L864.834 852.757 L864.606 853.669 L864.378 854.061 L864.15 853.32 L863.923 853.263 L863.695 853.102 L863.467 852.809 L863.239 854.269 L863.011 854.599 L862.784 854.157 L862.556 853.958 L862.328 854.486 L862.1 854.24 L861.873 854.044 L861.645 855.131 L861.417 854.104 L861.189 853.311 L860.962 852.858 L860.734 851.344 L860.506 850.88 L860.278 850.999 L860.05 849.376 L859.823 848.159 L859.595 847.085 L859.367 847.169 L859.139 844.442 L858.912 844.145 L858.684 844.004 L858.456 848.378 L858.228 847.905 L858.001 846.034 L857.773 846.044 L857.545 847.745 L857.317 849.325 L857.089 848.953 L856.862 850.29 L856.634 849.413 L856.406 849.354 L856.178 849.673 L855.951 850.393 L855.723 848.943 L855.495 848.135 L855.267 847.417 L855.04 845.863 L854.812 846.055 L854.584 846.537 L854.356 846.316 L854.128 846.782 L853.901 846.109 L853.673 847.284 L853.445 846.145 L853.217 848.303 L852.99 848.274 L852.762 849.425 L852.534 848.471 L852.306 847.843 L852.078 846.305 L851.851 847.734 L851.623 846.875 L851.395 847.111 L851.167 846.269 L850.94 845.307 L850.712 845.045 L850.484 844.312 L850.256 843.804 L850.029 843.473 L849.801 842.865 L849.573 842.336 L849.345 843.682 L849.117 842.691 L848.89 843.543 L848.662 842.068 L848.434 841.758 L848.206 843.011 L847.979 842.346 L847.751 841.94 L847.523 840.937 L847.295 843.907 L847.068 843.625 L846.84 842.406 L846.612 842.133 L846.384 842.111 L846.156 841.119 L845.929 842.164 L845.701 841.394 L845.473 841.036 L845.245 841.074 L845.018 841.004 L844.79 840.949 L844.562 841.393 L844.334 841.626 L844.107 841.196 L843.879 841.556 L843.651 839.988 L843.423 839.625 L843.195 840.915 L842.968 840.515 L842.74 839.856 L842.512 838.449 L842.284 838.133 L842.057 838.245 L841.829 836.578 L841.601 838.01 L841.373 837.212 L841.145 839.042 L840.918 839.673 L840.69 839.3 L840.462 839.779 L840.234 839.003 L840.007 837.149 L839.779 835.955 L839.551 834.813 L839.323 835.307 L839.096 835.641 L838.868 834.32 L838.64 835.448 L838.412 836.251 L838.184 836.646 L837.957 836.326 L837.729 837.321 L837.501 838.141 L837.273 838.825 L837.046 837.555 L836.818 836.99 L836.59 837.598 L836.362 836.053 L836.135 835.907 L835.907 835.544 L835.679 835.814 L835.451 835.602 L835.223 834.626 L834.996 834.306 L834.768 834.291 L834.54 834.688 L834.312 835.377 L834.085 833.567 L833.857 831.886 L833.629 832.639 L833.401 832.292 L833.174 831.839 L832.946 832.651 L832.718 832.506 L832.49 831.094 L832.262 834.929 L832.035 834.474 L831.807 834.842 L831.579 833.334 L831.351 832.747 L831.124 830.93 L830.896 829.953 L830.668 830.833 L830.44 832.392 L830.212 832.7 L829.985 832.309 L829.757 830.878 L829.529 831.596 L829.301 831.79 L829.074 832.928 L828.846 834.767 L828.618 833.282 L828.39 834.261 L828.163 833.934 L827.935 832.432 L827.707 832.185 L827.479 831.538 L827.251 831.907 L827.024 832.786 L826.796 831.325 L826.568 832.399 L826.34 831.93 L826.113 830.743 L825.885 829.859 L825.657 829.915 L825.429 830.52 L825.202 828.788 L824.974 827.438 L824.746 828.661 L824.518 830.575 L824.29 828.886 L824.063 827.66 L823.835 826.992 L823.607 826.172 L823.379 823.863 L823.152 823.534 L822.924 823.299 L822.696 821.751 L822.468 822.316 L822.241 823.556 L822.013 824.85 L821.785 826.943 L821.557 828.56 L821.329 831.181 L821.102 831.34 L820.874 829.819 L820.646 831.221 L820.418 831.308 L820.191 829.908 L819.963 830.546 L819.735 830.007 L819.507 829.638 L819.279 829.861 L819.052 829.085 L818.824 829.309 L818.596 829.517 L818.368 830.136 L818.141 829.563 L817.913 828.621 L817.685 831.222 L817.457 829.165 L817.23 828.621 L817.002 828.003 L816.774 828.52 L816.546 827.262 L816.318 826.782 L816.091 826.679 L815.863 827.016 L815.635 827.669 L815.407 829.001 L815.18 829.687 L814.952 828.579 L814.724 829.018 L814.496 829.821 L814.269 828.798 L814.041 827.52 L813.813 831.172 L813.585 831.161 L813.357 829.647 L813.13 827.117 L812.902 828.379 L812.674 828.55 L812.446 829.091 L812.219 828.005 L811.991 828.71 L811.763 828.853 L811.535 829.037 L811.308 828.807 L811.08 830.345 L810.852 830.786 L810.624 829.135 L810.396 831.221 L810.169 830.52 L809.941 830.851 L809.713 830.17 L809.485 830.731 L809.258 832.848 L809.03 830.814 L808.802 829.587 L808.574 828.852 L808.346 828.102 L808.119 829.118 L807.891 828.259 L807.663 826.772 L807.435 825.233 L807.208 826.766 L806.98 826.359 L806.752 825.581 L806.524 824.225 L806.297 823.934 L806.069 825.416 L805.841 824.993 L805.613 827.299 L805.385 827.033 L805.158 826.801 L804.93 826.225 L804.702 824.697 L804.474 825.707 L804.247 824.584 L804.019 825.958 L803.791 825.557 L803.563 823.881 L803.336 825.847 L803.108 827.41 L802.88 826.413 L802.652 827.075 L802.424 825.617 L802.197 826.037 L801.969 829.76 L801.741 828.062 L801.513 827.466 L801.286 829.017 L801.058 827.702 L800.83 827.874 L800.602 829.075 L800.375 828.311 L800.147 828.962 L799.919 830.636 L799.691 831.698 L799.463 829.943 L799.236 830.855 L799.008 832.415 L798.78 830.217 L798.552 829.447 L798.325 828.345 L798.097 828.552 L797.869 828.823 L797.641 830.75 L797.413 830.919 L797.186 830.431 L796.958 832.52 L796.73 834.525 L796.502 836.871 L796.275 834.742 L796.047 834.933 L795.819 833.29 L795.591 834.019 L795.364 835.99 L795.136 835.628 L794.908 836.807 L794.68 836.311 L794.452 836.62 L794.225 836.817 L793.997 838.387 L793.769 838.123 L793.541 837.537 L793.314 836.906 L793.086 836.897 L792.858 836.363 L792.63 837.421 L792.403 837.799 L792.175 836.595 L791.947 835.895 L791.719 833.982 L791.491 833.053 L791.264 833.866 L791.036 834.752 L790.808 837.475 L790.58 837.564 L790.353 837.514 L790.125 836.931 L789.897 834.672 L789.669 832.996 L789.442 831.639 L789.214 831.081 L788.986 830.211 L788.758 828.87 L788.53 828.284 L788.303 827.785 L788.075 828.32 L787.847 827.646 L787.619 826.06 L787.392 826.042 L787.164 827.083 L786.936 828.523 L786.708 829.967 L786.481 830.273 L786.253 830.585 L786.025 830.236 L785.797 829.054 L785.569 830.477 L785.342 829.735 L785.114 829.316 L784.886 829.174 L784.658 829.185 L784.431 829.508 L784.203 830.555 L783.975 831.742 L783.747 833.265 L783.519 834.153 L783.292 835.304 L783.064 833.918 L782.836 834.186 L782.608 833.256 L782.381 831.365 L782.153 831.023 L781.925 830.7 L781.697 830.173 L781.47 830.443 L781.242 832.217 L781.014 835.283 L780.786 837.363 L780.558 837.434 L780.331 839.075 L780.103 839.612 L779.875 840.931 L779.647 841.36 L779.42 841.005 L779.192 840.15 L778.964 841.443 L778.736 841.177 L778.509 842.402 L778.281 842.358 L778.053 843.072 L777.825 842.194 L777.597 841.205 L777.37 841.992 L777.142 845.997 L776.914 846.454 L776.686 845.633 L776.459 844.943 L776.231 847.238 L776.003 846.133 L775.775 845.533 L775.548 844.674 L775.32 845.357 L775.092 844.574 L774.864 843.942 L774.636 843.287 L774.409 844.157 L774.181 843.436 L773.953 841.655 L773.725 839.528 L773.498 839.65 L773.27 838.072 L773.042 837.844 L772.814 836.565 L772.586 837.747 L772.359 836.414 L772.131 835.042 L771.903 833.931 L771.675 834.936 L771.448 833.376 L771.22 831.645 L770.992 830.893 L770.764 830.32 L770.537 829.64 L770.309 829.034 L770.081 829.025 L769.853 829.791 L769.625 831.272 L769.398 832.338 L769.17 834.376 L768.942 833.204 L768.714 834.732 L768.487 837.891 L768.259 836.375 L768.031 836.946 L767.803 837.29 L767.576 835.214 L767.348 835.746 L767.12 835.857 L766.892 837.202 L766.664 838.484 L766.437 836.993 L766.209 838.735 L765.981 839.737 L765.753 840.562 L765.526 839.441 L765.298 838.237 L765.07 836.954 L764.842 836.202 L764.615 836.163 L764.387 836.161 L764.159 834.535 L763.931 834.204 L763.703 832.717 L763.476 832.133 L763.248 833.5 L763.02 836.334 L762.792 834.591 L762.565 836.698 L762.337 837.68 L762.109 837.037 L761.881 836.817 L761.653 836.603 L761.426 836.863 L761.198 834.679 L760.97 833.985 L760.742 831.997 L760.515 831.841 L760.287 830.735 L760.059 831.355 L759.831 831.567 L759.604 831.246 L759.376 833.478 L759.148 833.5 L758.92 833.565 L758.692 831.833 L758.465 836.118 L758.237 835.69 L758.009 835.311 L757.781 834.159 L757.554 832.132 L757.326 835.237 L757.098 834.344 L756.87 833.724 L756.643 834.024 L756.415 832.512 L756.187 832.248 L755.959 833.425 L755.731 832.14 L755.504 831.247 L755.276 829.422 L755.048 828.933 L754.82 828.826 L754.593 828.858 L754.365 829.988 L754.137 830.499 L753.909 831.856 L753.682 833.859 L753.454 832.727 L753.226 832.275 L752.998 831.815 L752.77 830.023 L752.543 829.293 L752.315 828.758 L752.087 829.192 L751.859 829.065 L751.632 829.744 L751.404 829.218 L751.176 828.263 L750.948 825.231 L750.72 827.548 L750.493 828.022 L750.265 828.039 L750.037 829.718 L749.809 829.936 L749.582 828.295 L749.354 828.807 L749.126 828.681 L748.898 826.085 L748.671 827.509 L748.443 827.046 L748.215 825.184 L747.987 825.587 L747.759 824.481 L747.532 825.911 L747.304 825.935 L747.076 826.447 L746.848 827.507 L746.621 825.992 L746.393 827.06 L746.165 827.623 L745.937 828.645 L745.71 829.383 L745.482 826.911 L745.254 825.484 L745.026 823.622 L744.798 823.814 L744.571 824.608 L744.343 822.426 L744.115 824.718 L743.887 825.302 L743.66 823.328 L743.432 821.997 L743.204 823.368 L742.976 824.533 L742.749 823.21 L742.521 822.513 L742.293 822.77 L742.065 823.387 L741.837 823.254 L741.61 820.538 L741.382 818.666 L741.154 817.721 L740.926 817.783 L740.699 819.213 L740.471 820.684 L740.243 822.938 L740.015 827.085 L739.787 827.795 L739.56 828.588 L739.332 826.68 L739.104 827.556 L738.876 827.185 L738.649 826.66 L738.421 826.969 L738.193 825.536 L737.965 828.196 L737.738 828.137 L737.51 827.131 L737.282 826.54 L737.054 824.636 L736.826 824.532 L736.599 824.337 L736.371 823.39 L736.143 823.401 L735.915 824.286 L735.688 823.678 L735.46 825.653 L735.232 825.278 L735.004 828.252 L734.777 825.905 L734.549 824.886 L734.321 827.183 L734.093 827.138 L733.865 827.326 L733.638 825.206 L733.41 824.591 L733.182 824.386 L732.954 823.926 L732.727 824.572 L732.499 823.945 L732.271 823.726 L732.043 822.089 L731.816 821.52 L731.588 821.179 L731.36 820.893 L731.132 820.495 L730.904 820.277 L730.677 823.076 L730.449 823.09 L730.221 824.057 L729.993 821.945 L729.766 822.432 L729.538 821.855 L729.31 821.088 L729.082 819.23 L728.854 821.27 L728.627 821.043 L728.399 821.509 L728.171 821.695 L727.943 820.4 L727.716 820.274 L727.488 819.498 L727.26 820.834 L727.032 821.405 L726.805 820.251 L726.577 821.964 L726.349 823.215 L726.121 823.627 L725.893 824.734 L725.666 827.129 L725.438 827.961 L725.21 827.857 L724.982 828.541 L724.755 829.129 L724.527 828.768 L724.299 830.825 L724.071 830.112 L723.844 828.444 L723.616 828.797 L723.388 831.046 L723.16 830.329 L722.932 830.195 L722.705 829.008 L722.477 828.091 L722.249 827.349 L722.021 827.768 L721.794 825.934 L721.566 825.65 L721.338 826.615 L721.11 829.015 L720.883 828.018 L720.655 828.768 L720.427 827.672 L720.199 827.864 L719.971 826.151 L719.744 824.746 L719.516 829.611 L719.288 829.589 L719.06 829.715 L718.833 828.829 L718.605 828.315 L718.377 826.347 L718.149 823.856 L717.921 825.01 L717.694 823.926 L717.466 824.686 L717.238 826.141 L717.01 825.807 L716.783 825.004 L716.555 825.974 L716.327 823.838 L716.099 822.949 L715.872 823.668 L715.644 822.068 L715.416 821.83 L715.188 822.413 L714.96 821.614 L714.733 822.394 L714.505 820.222 L714.277 820.413 L714.049 819.198 L713.822 819.021 L713.594 820.665 L713.366 821.108 L713.138 818.233 L712.911 822.859 L712.683 822.352 L712.455 820.919 L712.227 823.576 L711.999 823.403 L711.772 820.776 L711.544 820.51 L711.316 819.519 L711.088 820.192 L710.861 818.799 L710.633 818.727 L710.405 818.185 L710.177 817.968 L709.95 819.617 L709.722 819.554 L709.494 818.6 L709.266 819.624 L709.038 817.79 L708.811 816.926 L708.583 816.955 L708.355 815.522 L708.127 814.835 L707.9 815.419 L707.672 815.994 L707.444 815.025 L707.216 814.649 L706.988 815.639 L706.761 815.995 L706.533 815.645 L706.305 817.268 L706.077 816.296 L705.85 814.929 L705.622 818.335 L705.394 820.525 L705.166 819.049 L704.939 818.771 L704.711 821.248 L704.483 822.615 L704.255 823.963 L704.027 824.958 L703.8 824.166 L703.572 825.262 L703.344 825.862 L703.116 826.157 L702.889 826.835 L702.661 824.607 L702.433 822.415 L702.205 821.882 L701.978 818.669 L701.75 818.592 L701.522 819.074 L701.294 817.971 L701.066 815.59 L700.839 812.718 L700.611 810.769 L700.383 808.543 L700.155 809.508 L699.928 809.944 L699.7 808.089 L699.472 807.602 L699.244 807.009 L699.017 808.166 L698.789 810.738 L698.561 812.795 L698.333 814.15 L698.105 813.003 L697.878 814.369 L697.65 814.201 L697.422 815.764 L697.194 814.42 L696.967 814.157 L696.739 812.656 L696.511 811.334 L696.283 809.071 L696.055 809.034 L695.828 809.402 L695.6 807.105 L695.372 806.416 L695.144 805.957 L694.917 807.068 L694.689 810.927 L694.461 817.654 L694.233 817.491 L694.006 817.238 L693.778 815.614 L693.55 812.841 L693.322 813.957 L693.094 813.114 L692.867 812.982 L692.639 811.757 L692.411 812.249 L692.183 810.797 L691.956 810.469 L691.728 810.768 L691.5 809.298 L691.272 809.577 L691.045 810.853 L690.817 810.124 L690.589 806.87 L690.361 804.731 L690.133 804.466 L689.906 805.967 L689.678 807.243 L689.45 806.52 L689.222 805.149 L688.995 804.22 L688.767 803.185 L688.539 801.299 L688.311 801.844 L688.084 803.187 L687.856 802.49 L687.628 802.37 L687.4 803.371 L687.172 801.978 L686.945 801.745 L686.717 803.072 L686.489 804.837 L686.261 802.452 L686.034 802.796 L685.806 800.889 L685.578 803.984 L685.35 805.056 L685.122 803.735 L684.895 803.16 L684.667 801.399 L684.439 806.071 L684.211 805.631 L683.984 803.372 L683.756 801.732 L683.528 800.424 L683.3 803.33 L683.073 801.276 L682.845 800.288 L682.617 799.646 L682.389 802.157 L682.161 804.251 L681.934 802.231 L681.706 800.035 L681.478 799.558 L681.25 799.076 L681.023 799.077 L680.795 797.41 L680.567 797.182 L680.339 798.125 L680.112 795.84 L679.884 794.91 L679.656 795.639 L679.428 794.681 L679.2 794.114 L678.973 792.973 L678.745 791.914 L678.517 791.014 L678.289 791.411 L678.062 790.558 L677.834 792.449 L677.606 793.339 L677.378 791.724 L677.151 790.658 L676.923 792.65 L676.695 791.759 L676.467 791.235 L676.239 791.874 L676.012 791.446 L675.784 791.573 L675.556 789.667 L675.328 788.414 L675.101 790.72 L674.873 788.29 L674.645 787.46 L674.417 786.428 L674.189 785.845 L673.962 786.362 L673.734 786.821 L673.506 788.578 L673.278 790.281 L673.051 790.976 L672.823 792.371 L672.595 791.432 L672.367 793.432 L672.14 792.545 L671.912 795.607 L671.684 795.218 L671.456 794.876 L671.228 795.658 L671.001 794.822 L670.773 796.194 L670.545 796.061 L670.317 793.062 L670.09 792.026 L669.862 790.755 L669.634 790.134 L669.406 789.051 L669.179 788.953 L668.951 787.013 L668.723 786.285 L668.495 787.681 L668.267 786.453 L668.04 787.672 L667.812 785.349 L667.584 784.181 L667.356 785.625 L667.129 784.148 L666.901 783.458 L666.673 781.06 L666.445 782.911 L666.218 789.124 L665.99 787.421 L665.762 785.737 L665.534 786.467 L665.306 785.767 L665.079 782.914 L664.851 781.707 L664.623 780.072 L664.395 780.299 L664.168 780.857 L663.94 782.47 L663.712 783.894 L663.484 785.663 L663.256 786.791 L663.029 788.496 L662.801 788.447 L662.573 785.659 L662.345 785.1 L662.118 785.754 L661.89 785.691 L661.662 783.784 L661.434 781.513 L661.207 780.56 L660.979 780.618 L660.751 780.589 L660.523 779.46 L660.295 781.777 L660.068 782.74 L659.84 783.018 L659.612 781.686 L659.384 780.734 L659.157 779.326 L658.929 776.768 L658.701 779.789 L658.473 779.876 L658.246 782.092 L658.018 781.732 L657.79 780.985 L657.562 779.428 L657.334 780.635 L657.107 780.165 L656.879 779.38 L656.651 778.858 L656.423 780.001 L656.196 781.878 L655.968 780.624 L655.74 779.831 L655.512 781.809 L655.285 782.685 L655.057 783.489 L654.829 784.55 L654.601 786.014 L654.373 790.677 L654.146 792.509 L653.918 792.713 L653.69 791.874 L653.462 790.542 L653.235 791.461 L653.007 790.743 L652.779 791.33 L652.551 790.187 L652.323 791.013 L652.096 787.656 L651.868 788.467 L651.64 786.95 L651.412 787.267 L651.185 789.628 L650.957 787.858 L650.729 787.884 L650.501 786.106 L650.274 786.388 L650.046 786.451 L649.818 785.768 L649.59 785.897 L649.362 785.661 L649.135 785.634 L648.907 789.049 L648.679 787.534 L648.451 785.449 L648.224 787.804 L647.996 788.503 L647.768 791.511 L647.54 791.669 L647.313 795.191 L647.085 792.67 L646.857 793.084 L646.629 792.866 L646.401 792.57 L646.174 792.295 L645.946 791.743 L645.718 792.66 L645.49 793.279 L645.263 796.884 L645.035 795.556 L644.807 796.625 L644.579 799.058 L644.352 799.712 L644.124 798.288 L643.896 799.53 L643.668 799.588 L643.44 802.07 L643.213 802.863 L642.985 802.65 L642.757 799.642 L642.529 798.113 L642.302 799.059 L642.074 801.149 L641.846 802.241 L641.618 804.722 L641.39 804.628 L641.163 805.014 L640.935 803.105 L640.707 807.336 L640.479 808.247 L640.252 805.567 L640.024 805.968 L639.796 806.527 L639.568 811.681 L639.341 813.233 L639.113 810.514 L638.885 809.817 L638.657 809.14 L638.429 812.187 L638.202 813.333 L637.974 814.601 L637.746 815.641 L637.518 813.943 L637.291 813.34 L637.063 814.09 L636.835 814.11 L636.607 816.236 L636.38 816.59 L636.152 817.902 L635.924 820.573 L635.696 821.39 L635.468 819.716 L635.241 818.042 L635.013 817.467 L634.785 818.112 L634.557 818.484 L634.33 816.26 L634.102 814.649 L633.874 812.061 L633.646 810.094 L633.419 810.229 L633.191 810.542 L632.963 809.711 L632.735 812.6 L632.507 813.141 L632.28 814.191 L632.052 813.706 L631.824 814.832 L631.596 814.177 L631.369 815.341 L631.141 814.052 L630.913 815.539 L630.685 815.407 L630.457 816.566 L630.23 820.102 L630.002 824.045 L629.774 826.323 L629.546 825.811 L629.319 829.607 L629.091 827.022 L628.863 826.651 L628.635 827.209 L628.408 827.758 L628.18 829.001 L627.952 832.056 L627.724 830.441 L627.496 827.984 L627.269 835.65 L627.041 832.162 L626.813 833.519 L626.585 839.73 L626.358 842.672 L626.13 843.265 L625.902 843.753 L625.674 842.418 L625.447 841.591 L625.219 841.162 L624.991 845.668 L624.763 844.051 L624.535 844.518 L624.308 846.84 L624.08 845.927 L623.852 844.919 L623.624 844.269 L623.397 843.057 L623.169 842.166 L622.941 842.684 L622.713 843.372 L622.486 840.86 L622.258 841.518 L622.03 840.817 L621.802 841.116 L621.574 839.512 L621.347 839.92 L621.119 842.44 L620.891 841.475 L620.663 841.252 L620.436 842.886 L620.208 844.471 L619.98 846.051 L619.752 848.632 L619.524 850.045 L619.297 847.357 L619.069 851.397 L618.841 850.188 L618.613 854.583 L618.386 853.014 L618.158 851.606 L617.93 853.747 L617.702 855.716 L617.475 853.177 L617.247 853.533 L617.019 855.298 L616.791 854.445 L616.563 852.572 L616.336 850.818 L616.108 854.01 L615.88 854.811 L615.652 852.614 L615.425 852.529 L615.197 853.094 L614.969 853.433 L614.741 851.75 L614.514 848.547 L614.286 849.578 L614.058 849.552 L613.83 850.231 L613.602 852.557 L613.375 850.989 L613.147 848.901 L612.919 852.925 L612.691 849.907 L612.464 847.709 L612.236 850.551 L612.008 848.973 L611.78 848.968 L611.553 849.216 L611.325 849.36 L611.097 846.461 L610.869 845.47 L610.641 844.933 L610.414 842.683 L610.186 842.32 L609.958 841.221 L609.73 838.571 L609.503 838.839 L609.275 838.393 L609.047 841.537 L608.819 841.154 L608.591 842.641 L608.364 845.394 L608.136 846.58 L607.908 847.049 L607.68 846.604 L607.453 848.849 L607.225 846.922 L606.997 848.892 L606.769 846.823 L606.542 846.36 L606.314 845.151 L606.086 844.271 L605.858 847.302 L605.63 846.309 L605.403 845.08 L605.175 843.191 L604.947 842.572 L604.719 839.174 L604.492 839.443 L604.264 840.435 L604.036 841.863 L603.808 840.361 L603.581 841.948 L603.353 839.299 L603.125 841.552 L602.897 842.22 L602.669 843.357 L602.442 846.096 L602.214 841.409 L601.986 841.272 L601.758 840.8 L601.531 839.632 L601.303 837.494 L601.075 837.651 L600.847 835.047 L600.62 837.548 L600.392 839.382 L600.164 842.327 L599.936 842.903 L599.708 841.427 L599.481 846.827 L599.253 846.645 L599.025 845.632 L598.797 843.89 L598.57 842.437 L598.342 843.482 L598.114 841.921 L597.886 843.73 L597.658 842.348 L597.431 840.353 L597.203 842.46 L596.975 846.976 L596.747 847.413 L596.52 845.778 L596.292 845.462 L596.064 844.916 L595.836 849.764 L595.609 850.442 L595.381 851.762 L595.153 855.013 L594.925 853.305 L594.697 852.914 L594.47 851.311 L594.242 848.906 L594.014 849.27 L593.786 847.664 L593.559 846.578 L593.331 848.956 L593.103 848.452 L592.875 848.439 L592.648 850.319 L592.42 849.436 L592.192 847.248 L591.964 845.194 L591.736 842.897 L591.509 842.655 L591.281 840.036 L591.053 839.147 L590.825 836.154 L590.598 835.794 L590.37 834.982 L590.142 835.847 L589.914 838.78 L589.687 842.255 L589.459 846.901 L589.231 847.231 L589.003 844.797 L588.775 846.953 L588.548 849.009 L588.32 847.198 L588.092 846.579 L587.864 847.112 L587.637 846.404 L587.409 843.735 L587.181 845.58 L586.953 848.041 L586.725 845.46 L586.498 844.053 L586.27 841.586 L586.042 841.298 L585.814 841.284 L585.587 838.753 L585.359 839.865 L585.131 842.993 L584.903 842.487 L584.676 844.849 L584.448 841.511 L584.22 839.596 L583.992 838.862 L583.764 837.711 L583.537 836.916 L583.309 834.99 L583.081 834.837 L582.853 834.411 L582.626 832.906 L582.398 839.758 L582.17 836.854 L581.942 839.061 L581.715 836.641 L581.487 837.501 L581.259 840.377 L581.031 846.338 L580.803 845.418 L580.576 844.951 L580.348 849.204 L580.12 853.279 L579.892 858.648 L579.665 858.599 L579.437 859.273 L579.209 857.008 L578.981 862.954 L578.754 860.905 L578.526 861.3 L578.298 858.991 L578.07 856.932 L577.842 857.039 L577.615 854.675 L577.387 854.101 L577.159 854.995 L576.931 850.416 L576.704 850.183 L576.476 845.901 L576.248 846.835 L576.02 844.714 L575.792 845.535 L575.565 843.081 L575.337 842.766 L575.109 844.431 L574.881 845.229 L574.654 846.037 L574.426 846.952 L574.198 843.17 L573.97 845.499 L573.743 844.084 L573.515 840.356 L573.287 835.661 L573.059 835.905 L572.831 835.151 L572.604 834.529 L572.376 838.283 L572.148 839.729 L571.92 838.451 L571.693 834.286 L571.465 829.752 L571.237 830.126 L571.009 829.927 L570.782 831.695 L570.554 829.896 L570.326 829.679 L570.098 826.392 L569.87 824.926 L569.643 829.711 L569.415 830.667 L569.187 827.999 L568.959 827.015 L568.732 825.388 L568.504 827.632 L568.276 828.76 L568.048 829.054 L567.821 829.609 L567.593 828.671 L567.365 827.378 L567.137 827.397 L566.909 830.343 L566.682 830.336 L566.454 828.527 L566.226 826.224 L565.998 824.622 L565.771 823.785 L565.543 822.694 L565.315 826.077 L565.087 827.132 L564.859 828.597 L564.632 828.44 L564.404 826.957 L564.176 829.548 L563.948 832.179 L563.721 832.087 L563.493 832.726 L563.265 829.928 L563.037 830.407 L562.81 829.838 L562.582 828.685 L562.354 827.707 L562.126 827.066 L561.898 823.824 L561.671 823.098 L561.443 823.26 L561.215 822.582 L560.987 820.158 L560.76 821.435 L560.532 822.244 L560.304 816.985 L560.076 815.696 L559.849 814.01 L559.621 818.14 L559.393 816.406 L559.165 817.068 L558.937 821.839 L558.71 819.214 L558.482 820.818 L558.254 818.69 L558.026 818.306 L557.799 816.259 L557.571 814.631 L557.343 813.581 L557.115 810.269 L556.888 809.3 L556.66 807.532 L556.432 804.115 L556.204 802.235 L555.976 800.992 L555.749 800.594 L555.521 799.066 L555.293 796.231 L555.065 794.403 L554.838 792.806 L554.61 789.186 L554.382 787.667 L554.154 787.902 L553.926 789.692 L553.699 787.48 L553.471 786.428 L553.243 784.6 L553.015 787.849 L552.788 788.42 L552.56 786.676 L552.332 787.67 L552.104 783.955 L551.877 781.796 L551.649 783.911 L551.421 782.71 L551.193 785.371 L550.965 782.631 L550.738 781.705 L550.51 786.008 L550.282 788.308 L550.054 787.873 L549.827 790.659 L549.599 794.058 L549.371 792.578 L549.143 790.746 L548.916 794.931 L548.688 792.39 L548.46 792.552 L548.232 790.678 L548.004 792.467 L547.777 792.295 L547.549 790.885 L547.321 790.275 L547.093 790.779 L546.866 789.871 L546.638 791.47 L546.41 788.024 L546.182 785.394 L545.955 783.715 L545.727 784.835 L545.499 784.871 L545.271 785.177 L545.043 783.99 L544.816 784.213 L544.588 782.575 L544.36 787.324 L544.132 791.399 L543.905 788.136 L543.677 789.335 L543.449 786.981 L543.221 788.66 L542.993 789.803 L542.766 787.521 L542.538 787.97 L542.31 787.739 L542.082 786.144 L541.855 786.096 L541.627 782.644 L541.399 781.253 L541.171 777.532 L540.944 776.731 L540.716 774.293 L540.488 771.461 L540.26 768.431 L540.032 765.195 L539.805 763.904 L539.577 764.868 L539.349 763.259 L539.121 762.759 L538.894 761.904 L538.666 760.304 L538.438 757.93 L538.21 755.933 L537.983 753.777 L537.755 754.907 L537.527 750.729 L537.299 753.87 L537.071 747.962 L536.844 745.226 L536.616 743.326 L536.388 742.951 L536.16 747.779 L535.933 749.401 L535.705 748.593 L535.477 751.451 L535.249 750.769 L535.022 746.823 L534.794 748.139 L534.566 744.548 L534.338 743.301 L534.11 746.81 L533.883 748.35 L533.655 751.536 L533.427 754.787 L533.199 759.816 L532.972 758.061 L532.744 753.378 L532.516 752.919 L532.288 751.858 L532.06 750.486 L531.833 752.956 L531.605 756.6 L531.377 759.512 L531.149 760.085 L530.922 764.251 L530.694 766.296 L530.466 765.588 L530.238 765.089 L530.011 764.468 L529.783 763.284 L529.555 763.67 L529.327 761.927 L529.099 765.221 L528.872 765.52 L528.644 764.568 L528.416 763.71 L528.188 758.871 L527.961 765.944 L527.733 761.753 L527.505 761.212 L527.277 764.216 L527.05 764.043 L526.822 760.768 L526.594 769.764 L526.366 768.948 L526.138 766.808 L525.911 767.378 L525.683 767.394 L525.455 764.61 L525.227 773.409 L525 772.388 L524.772 769.047 L524.544 768.028 L524.316 765.613 L524.089 768.401 L523.861 773.726 L523.633 775.448 L523.405 779.954 L523.177 782.836 L522.95 783.345 L522.722 781.855 L522.494 786.33 L522.266 792.392 L522.039 789.797 L521.811 789.232 L521.583 788.811 L521.355 790.493 L521.127 786.764 L520.9 784.664 L520.672 784.352 L520.444 784.48 L520.216 781.783 L519.989 782.289 L519.761 780.813 L519.533 781.483 L519.305 778.126 L519.078 776.626 L518.85 773.865 L518.622 773.319 L518.394 773.211 L518.166 772.026 L517.939 769.643 L517.711 771.366 L517.483 772.675 L517.255 776.011 L517.028 774.071 L516.8 776.385 L516.572 776.75 L516.344 776.704 L516.117 775.532 L515.889 772.956 L515.661 774.292 L515.433 771.876 L515.205 773.346 L514.978 774.775 L514.75 775.31 L514.522 776.999 L514.294 776.156 L514.067 778.929 L513.839 778.669 L513.611 781.886 L513.383 778.628 L513.156 774.972 L512.928 777.835 L512.7 778.993 L512.472 780.259 L512.244 776.489 L512.017 780.571 L511.789 777.26 L511.561 775.08 L511.333 778.333 L511.106 781.882 L510.878 782.409 L510.65 782.881 L510.422 788.613 L510.194 789.079 L509.967 787.146 L509.739 786.127 L509.511 784.397 L509.283 787.236 L509.056 787.065 L508.828 788.915 L508.6 784.755 L508.372 782.996 L508.145 780.934 L507.917 776.959 L507.689 773.651 L507.461 776.98 L507.233 779.263 L507.006 776.45 L506.778 772.44 L506.55 777.097 L506.322 777.016 L506.095 773.463 L505.867 771.559 L505.639 770.354 L505.411 768.588 L505.184 769.14 L504.956 769.035 L504.728 765.305 L504.5 765.094 L504.272 763.968 L504.045 760.538 L503.817 761.32 L503.589 759.517 L503.361 757.709 L503.134 752.579 L502.906 752.834 L502.678 754.536 L502.45 755.277 L502.223 757.761 L501.995 755.33 L501.767 752.852 L501.539 758.15 L501.311 757.505 L501.084 757.183 L500.856 752.923 L500.628 753.216 L500.4 755.325 L500.173 751.289 L499.945 756.093 L499.717 751.16 L499.489 747.484 L499.261 745.574 L499.034 749.253 L498.806 749.531 L498.578 751.772 L498.35 750.084 L498.123 748.398 L497.895 748.614 L497.667 755.72 L497.439 758.372 L497.212 755.149 L496.984 752.096 L496.756 751.02 L496.528 751.327 L496.3 755.214 L496.073 753.861 L495.845 753.769 L495.617 755.615 L495.389 753.074 L495.162 757.974 L494.934 762.723 L494.706 760.995 L494.478 758.749 L494.251 759.523 L494.023 756.663 L493.795 757.737 L493.567 756.996 L493.339 756.536 L493.112 759.647 L492.884 757.042 L492.656 759.496 L492.428 762.814 L492.201 765.319 L491.973 765.012 L491.745 767.08 L491.517 765.801 L491.29 761.563 L491.062 765.088 L490.834 763.513 L490.606 763.582 L490.378 762.254 L490.151 760.036 L489.923 756.867 L489.695 757.222 L489.467 760.703 L489.24 767.846 L489.012 767.259 L488.784 770.042 L488.556 784.219 L488.328 781.162 L488.101 780.751 L487.873 787.086 L487.645 785.635 L487.417 783.532 L487.19 781.499 L486.962 777.205 L486.734 773.017 L486.506 773.162 L486.279 775.225 L486.051 772.721 L485.823 773.274 L485.595 773.345 L485.367 769.741 L485.14 769.005 L484.912 768.664 L484.684 769.027 L484.456 771.241 L484.229 767.587 L484.001 765.83 L483.773 764.005 L483.545 764.729 L483.318 769.057 L483.09 770.832 L482.862 771.588 L482.634 767.323 L482.406 763.807 L482.179 771.407 L481.951 768.476 L481.723 766.625 L481.495 762.913 L481.268 765.423 L481.04 759.92 L480.812 759.872 L480.584 758.773 L480.357 758.055 L480.129 756.627 L479.901 759.268 L479.673 761.26 L479.445 763.469 L479.218 761.762 L478.99 764.384 L478.762 760.316 L478.534 758.547 L478.307 769.252 L478.079 765.805 L477.851 765.826 L477.623 766.131 L477.395 767.955 L477.168 766.026 L476.94 764.775 L476.712 760.071 L476.484 761.3 L476.257 760.008 L476.029 760.124 L475.801 758.432 L475.573 755.796 L475.346 754.08 L475.118 749.692 L474.89 751.454 L474.662 757.165 L474.434 756.948 L474.207 758.501 L473.979 756.084 L473.751 754.386 L473.523 752.903 L473.296 750.681 L473.068 756.366 L472.84 759.26 L472.612 758.774 L472.385 757.853 L472.157 760.532 L471.929 758.533 L471.701 755.986 L471.473 756.883 L471.246 756.732 L471.018 753.622 L470.79 753.279 L470.562 757.991 L470.335 759.673 L470.107 759.321 L469.879 754.695 L469.651 755.643 L469.424 755.757 L469.196 753.729 L468.968 755.896 L468.74 752.082 L468.512 747.409 L468.285 743.851 L468.057 747.218 L467.829 741.003 L467.601 743.123 L467.374 743.349 L467.146 738.472 L466.918 735.301 L466.69 736.419 L466.462 734.377 L466.235 734.161 L466.007 730.25 L465.779 724.689 L465.551 723.764 L465.324 725.823 L465.096 723.231 L464.868 720.359 L464.64 717.633 L464.413 717.84 L464.185 712.342 L463.957 712.821 L463.729 712.865 L463.501 714.171 L463.274 712.597 L463.046 718.762 L462.818 721.972 L462.59 721.921 L462.363 720.895 L462.135 720.71 L461.907 722.806 L461.679 721.626 L461.452 721.554 L461.224 722.161 L460.996 722.73 L460.768 722.663 L460.54 727.483 L460.313 726.221 L460.085 721.508 L459.857 718.114 L459.629 721.547 L459.402 715.146 L459.174 716.203 L458.946 717.737 L458.718 714.733 L458.491 715.018 L458.263 723.962 L458.035 721.753 L457.807 717.177 L457.579 717.906 L457.352 718.103 L457.124 720.291 L456.896 714.551 L456.668 714.7 L456.441 720.884 L456.213 721.074 L455.985 727.5 L455.757 724.62 L455.53 728.286 L455.302 735.446 L455.074 737.12 L454.846 737.668 L454.618 734.566 L454.391 734.196 L454.163 732.906 L453.935 731.805 L453.707 734.82 L453.48 736.095 L453.252 734.892 L453.024 731.151 L452.796 728.885 L452.568 726.341 L452.341 720.558 L452.113 721.289 L451.885 726.93 L451.657 725.646 L451.43 724.897 L451.202 720.612 L450.974 715.832 L450.746 722.216 L450.519 730.798 L450.291 730.862 L450.063 732.279 L449.835 729.801 L449.607 725.324 L449.38 722.94 L449.152 727.724 L448.924 726.894 L448.696 733.258 L448.469 733.189 L448.241 736.172 L448.013 731.847 L447.785 730.547 L447.558 732.201 L447.33 724.17 L447.102 724.691 L446.874 725.434 L446.646 725.828 L446.419 729.024 L446.191 724.71 L445.963 728.766 L445.735 729.775 L445.508 729.075 L445.28 726.181 L445.052 729.147 L444.824 729.735 L444.597 727.234 L444.369 722.735 L444.141 715.269 L443.913 719.693 L443.685 714.711 L443.458 712.919 L443.23 717.497 L443.002 711.108 L442.774 711.248 L442.547 706.578 L442.319 705.252 L442.091 705.263 L441.863 715.84 L441.635 711.583 L441.408 719.062 L441.18 716.64 L440.952 721.622 L440.724 715.94 L440.497 715.676 L440.269 711.742 L440.041 707.735 L439.813 713.621 L439.586 706.608 L439.358 706.222 L439.13 706.335 L438.902 704.643 L438.674 702.959 L438.447 699.262 L438.219 699.99 L437.991 698.418 L437.763 705.838 L437.536 708.551 L437.308 707.473 L437.08 709.717 L436.852 710.343 L436.625 713.054 L436.397 712.397 L436.169 708.624 L435.941 707.973 L435.713 707.043 L435.486 702.952 L435.258 701.874 L435.03 705.542 L434.802 711.008 L434.575 712.582 L434.347 707.707 L434.119 702.435 L433.891 698.991 L433.664 695.721 L433.436 694.259 L433.208 687.485 L432.98 685.855 L432.752 683.405 L432.525 687.77 L432.297 691.728 L432.069 691.212 L431.841 694.318 L431.614 692.299 L431.386 690.845 L431.158 684.759 L430.93 679.248 L430.702 682.417 L430.475 694.767 L430.247 689.923 L430.019 687.014 L429.791 684.972 L429.564 687.501 L429.336 684.816 L429.108 678.724 L428.88 671.506 L428.653 670.568 L428.425 665.596 L428.197 664.741 L427.969 660.202 L427.741 655.167 L427.514 655.073 L427.286 655.635 L427.058 656.587 L426.83 655.062 L426.603 655.417 L426.375 654.866 L426.147 650.489 L425.919 650.2 L425.692 647.615 L425.464 640.558 L425.236 641.48 L425.008 640.158 L424.78 640.781 L424.553 640.867 L424.325 647.425 L424.097 646.874 L423.869 646.579 L423.642 646.806 L423.414 646.288 L423.186 643.644 L422.958 642.555 L422.731 634.937 L422.503 636.866 L422.275 637.247 L422.047 634.724 L421.819 635.989 L421.592 637.836 L421.364 635.844 L421.136 631.601 L420.908 631.271 L420.681 627.943 L420.453 627.715 L420.225 620.042 L419.997 622.057 L419.769 620.299 L419.542 617.244 L419.314 617.361 L419.086 622.44 L418.858 620.797 L418.631 616.358 L418.403 610.585 L418.175 610.149 L417.947 615.083 L417.72 616.52 L417.492 619.553 L417.264 628.961 L417.036 643.994 L416.808 647.479 L416.581 652.85 L416.353 655.749 L416.125 662.169 L415.897 663.801 L415.67 664.356 L415.442 667.007 L415.214 662.136 L414.986 659.288 L414.759 653.418 L414.531 647.333 L414.303 641.183 L414.075 639.14 L413.847 636.291 L413.62 635.14 L413.392 629.723 L413.164 641.928 L412.936 645.476 L412.709 645.101 L412.481 643.828 L412.253 638.463 L412.025 637.31 L411.798 640.88 L411.57 640.533 L411.342 635.44 L411.114 631.934 L410.886 653.521 L410.659 649.819 L410.431 656.821 L410.203 662.327 L409.975 661.609 L409.748 653.43 L409.52 656.636 L409.292 656.965 L409.064 660.77 L408.836 652.814 L408.609 649.534 L408.381 652.625 L408.153 649.339 L407.925 653.262 L407.698 651.593 L407.47 648.875 L407.242 652.407 L407.014 643.928 L406.787 646.888 L406.559 649.929 L406.331 654.054 L406.103 649.031 L405.875 649.039 L405.648 651.945 L405.42 654.915 L405.192 648.642 L404.964 644.387 L404.737 641.891 L404.509 639.706 L404.281 628.546 L404.053 634.063 L403.826 633.157 L403.598 636.191 L403.37 638.5 L403.142 636.08 L402.914 628.063 L402.687 625.68 L402.459 628.799 L402.231 626.806 L402.003 636.627 L401.776 636.325 L401.548 645.059 L401.32 647.256 L401.092 647.972 L400.865 649.806 L400.637 643.159 L400.409 650.736 L400.181 651.049 L399.953 646.808 L399.726 640.973 L399.498 633.37 L399.27 629.52 L399.042 624.284 L398.815 620.274 L398.587 619.11 L398.359 623.294 L398.131 617.164 L397.903 613.431 L397.676 612.985 L397.448 612.182 L397.22 606.278 L396.992 602.274 L396.765 603.347 L396.537 604.597 L396.309 612.874 L396.081 610.754 L395.854 607.798 L395.626 607.795 L395.398 607.037 L395.17 622.686 L394.942 629.723 L394.715 630.971 L394.487 625.137 L394.259 637.02 L394.031 631.168 L393.804 629.388 L393.576 623.957 L393.348 618.935 L393.12 614.17 L392.893 628.699 L392.665 632.143 L392.437 638.689 L392.209 643.932 L391.981 643.98 L391.754 641.542 L391.526 647.02 L391.298 643.533 L391.07 636.223 L390.843 638.042 L390.615 653.207 L390.387 643.41 L390.159 650.161 L389.932 652.271 L389.704 645.413 L389.476 633.76 L389.248 633.187 L389.02 632.38 L388.793 631.789 L388.565 629.41 L388.337 624.529 L388.109 623.207 L387.882 619.086 L387.654 623.2 L387.426 620.294 L387.198 607.206 L386.97 610.607 L386.743 606.061 L386.515 599.773 L386.287 606.098 L386.059 601.701 L385.832 599.12 L385.604 591.509 L385.376 592.986 L385.148 586.324 L384.921 589.771 L384.693 581.554 L384.465 588.076 L384.237 589.468 L384.009 584.52 L383.782 597.515 L383.554 587.701 L383.326 589.513 L383.098 587.27 L382.871 584.114 L382.643 575.704 L382.415 569.111 L382.187 574.799 L381.96 566.89 L381.732 575.518 L381.504 570.634 L381.276 572.331 L381.048 574.413 L380.821 567.575 L380.593 570.424 L380.365 577.907 L380.137 573.4 L379.91 566.72 L379.682 576.973 L379.454 582.527 L379.226 582.466 L378.999 573.794 L378.771 568.035 L378.543 567.883 L378.315 558.673 L378.087 558.004 L377.86 560.764 L377.632 554.383 L377.404 542.636 L377.176 543.702 L376.949 542.908 L376.721 547.961 L376.493 539.327 L376.265 532.715 L376.037 540.879 L375.81 538.011 L375.582 535.981 L375.354 530.148 L375.126 536.312 L374.899 534.989 L374.671 531.541 L374.443 537.706 L374.215 523.145 L373.988 518.663 L373.76 528.298 L373.532 519.042 L373.304 528.952 L373.076 526.511 L372.849 527.165 L372.621 522.122 L372.393 518.385 L372.165 519.941 L371.938 533.558 L371.71 544.46 L371.482 535.394 L371.254 533.829 L371.027 519.655 L370.799 524.979 L370.571 541.368 L370.343 540.316 L370.115 536.395 L369.888 544.061 L369.66 554.133 L369.432 556.903 L369.204 548.536 L368.977 547.681 L368.749 553.441 L368.521 550.803 L368.293 547.025 L368.066 552.433 L367.838 546.272 L367.61 547.819 L367.382 547.814 L367.154 541.689 L366.927 544.965 L366.699 534.555 L366.471 529.373 L366.243 554.337 L366.016 546.94 L365.788 546.97 L365.56 565.454 L365.332 554.107 L365.104 548.594 L364.877 553.272 L364.649 542.586 L364.421 539.134 L364.193 548.077 L363.966 550.457 L363.738 573.313 L363.51 569.666 L363.282 558.309 L363.055 544.294 L362.827 539.845 L362.599 551.739 L362.371 556.707 L362.143 551.001 L361.916 541.617 L361.688 557.063 L361.46 562.995 L361.232 590.179 L361.005 586.172 L360.777 568.281 L360.549 570.876 L360.321 572.015 L360.094 561.611 L359.866 567.678 L359.638 556.336 L359.41 551.032 L359.182 555.421 L358.955 568.386 L358.727 555.308 L358.499 538.376 L358.271 527.726 L358.044 529.953 L357.816 534.922 L357.588 552.907 L357.36 544.841 L357.133 560.506 L356.905 562.541 L356.677 564.997 L356.449 584.974 L356.221 585.531 L355.994 606.427 L355.766 597.61 L355.538 592.419 L355.31 589.633 L355.083 584.892 L354.855 579.614 L354.627 585.645 L354.399 586.981 L354.171 581.611 L353.944 572.466 L353.716 571.66 L353.488 568.451 L353.26 565.341 L353.033 549.858 L352.805 543.398 L352.577 542.005 L352.349 554.792 L352.122 554.538 L351.894 555.365 L351.666 549.562 L351.438 534.35 L351.21 533.384 L350.983 537.764 L350.755 541.643 L350.527 548.99 L350.299 596.154 L350.072 601.107 L349.844 589.112 L349.616 588.186 L349.388 609.583 L349.161 613.483 L348.933 614.261 L348.705 600.947 L348.477 609.234 L348.249 605.456 L348.022 595.25 L347.794 582.664 L347.566 568.098 L347.338 539.781 L347.111 539.442 L346.883 525.085 L346.655 529.545 L346.427 535.172 L346.2 536.372 L345.972 530.2 L345.744 522.422 L345.516 510.131 L345.288 496.426 L345.061 500.574 L344.833 482.069 L344.605 475.299 L344.377 485.643 L344.15 476.136 L343.922 498.343 L343.694 517.958 L343.466 514.086 L343.238 540.46 L343.011 530.963 L342.783 535.49 L342.555 552.346 L342.327 537.242 L342.1 523.896 L341.872 529.073 L341.644 529.497 L341.416 512.259 L341.189 524.029 L340.961 514.921 L340.733 512.427 L340.505 543.796 L340.277 554.528 L340.05 548.153 L339.822 562.549 L339.594 550.458 L339.366 540.624 L339.139 526.735 L338.911 516.607 L338.683 514.516 L338.455 508.235 L338.228 504.123 L338 486.738 L337.772 492.865 L337.544 495.087 L337.316 475.262 L337.089 505.291 L336.861 502.289 L336.633 514.094 L336.405 492.624 L336.178 501.373 L335.95 514.369 L335.722 520.215 L335.494 537.64 L335.267 535.21 L335.039 550.563 L334.811 555.867 L334.583 580.001 L334.355 570.852 L334.128 559.805 L333.9 576.501 L333.672 572.634 L333.444 569.648 L333.217 592.243 L332.989 580.94 L332.761 557.261 L332.533 549.457 L332.305 529.747 L332.078 538.256 L331.85 531.79 L331.622 510.363 L331.394 537.502 L331.167 530.966 L330.939 531.848 L330.711 511.83 L330.483 517.895 L330.256 537.556 L330.028 555.993 L329.8 558.06 L329.572 572.22 L329.344 553.817 L329.117 553.93 L328.889 528.126 L328.661 521.234 L328.433 508.448 L328.206 543.465 L327.978 544.837 L327.75 524.692 L327.522 510.054 L327.295 492.702 L327.067 477.643 L326.839 494.735 L326.611 479.479 L326.383 474.335 L326.156 458.03 L325.928 485.042 L325.7 476.39 L325.472 470.061 L325.245 460.335 L325.017 450.022 L324.789 451.827 L324.561 456.274 L324.334 437.184 L324.106 421.866 L323.878 506.252 L323.65 489.801 L323.422 529.033 L323.195 584.432 L322.967 594.861 L322.739 561.916 L322.511 527.312 L322.284 499.372 L322.056 470.599 L321.828 485.283 L321.6 469.369 L321.372 454.692 L321.145 456.795 L320.917 475.771 L320.689 485.057 L320.461 489.787 L320.234 493.899 L320.006 492.907 L319.778 489.679 L319.55 506.12 L319.323 502.306 L319.095 519.772 L318.867 509.859 L318.639 501.67 L318.411 486.424 L318.184 488.185 L317.956 455.266 L317.728 424.224 L317.5 400.202 L317.273 410.543 L317.045 400.073 L316.817 392.181 L316.589 352.892 L316.362 358.584 L316.134 428.417 L315.906 447.039 L315.678 523.088 L315.45 530.49 L315.223 518.71 L314.995 509.369 L314.767 573.789 L314.539 548.012 L314.312 562.829 L314.084 553.112 L313.856 603.855 L313.628 564.757 L313.401 602.083 L313.173 576.279 L312.945 533.554 L312.717 504.078 L312.489 508.026 L312.262 475.428 L312.034 414.174 L311.806 407.888 L311.578 381.848 L311.351 344.894 L311.123 351.307 L310.895 343.538 L310.667 326.058 L310.439 343.191 L310.212 285.233 L309.984 272.701 L309.756 266.904 L309.528 282.861 L309.301 372.622 L309.073 362.518 L308.845 340.203 L308.617 438.155 L308.39 430.076 L308.162 401.831 L307.934 432.786 L307.706 381.976 L307.478 473.481 L307.251 485.88 L307.023 512.692 L306.795 497.974 L306.567 522.486 L306.34 545.06 L306.112 532.554 L305.884 515.158 L305.656 474.197 L305.429 547.986 L305.201 495.241 L304.973 412.36 L304.745 363.691 L304.517 320.3 L304.29 215.789 L304.062 170.355 L303.834 152.448 L303.606 62.554 L303.379 56.1091 L303.151 3.32255 L302.923 -40.6062 L302.695 -119.685 L302.468 -132.241 L302.24 -160.758 L302.012 -282.607 L301.784 -173.829 L301.556 -267.811 L301.329 -339.332 L301.101 -155.431 L300.873 -186.584 L300.645 -124.274 L300.418 129.782 L300.19 354.587 L299.962 198.078 L299.734 35.7392 L299.506 -7.20897 L299.279 -134.031 L299.051 -299.468 L298.823 -332.972 L298.595 -452.841 L298.368 -570.613 L298.14 -319.899 L297.912 122.954 L297.684 -94.8991 L297.457 -146.035 L297.229 -295.895 L297.001 -171.639 L296.773 -565.114 L296.545 -112.768 L296.318 636.9 L296.09 357.814 L295.862 -82.2439 L295.634 -681.079 L295.407 1275.79 L295.179 1474.21 L294.951 1945.39  Z" fill="#009af9" fill-rule="evenodd" fill-opacity="0.5"/>
<polyline clip-path="url(#clip992)" style="stroke:#009af9; stroke-linecap:round; stroke-linejoin:round; stroke-width:4; stroke-opacity:1; fill:none" points="294.951,1945.39 295.179,1786.21 295.407,2301.28 295.634,1339.2 295.862,1524.05 296.09,1730.15 296.318,2423.46 296.545,1826.74 296.773,1367.63 297.001,1634.41 297.229,1400.73 297.457,1402.77 297.684,1335.83 297.912,1475.57 298.14,1119.65 298.368,865.284 298.595,897.419 298.823,943.228 299.051,909.505 299.279,1041.88 299.506,1122.38 299.734,1112.92 299.962,1275.64 300.19,1429.38 300.418,1231.34 300.645,1015.49 300.873,924.547 301.101,915.43 301.329,745.997 301.556,783.257 301.784,851.54 302.012,735.793 302.24,852.423 302.468,850.695 302.695,835.235 302.923,894.7 303.151,913.766 303.379,944.286 303.606,928.23 303.834,1012.05 304.062,1008.75 304.29,1035.68 304.517,1153.58 304.745,1179.38 304.973,1212.33 305.201,1293.73 305.429,1333.1 305.656,1257.09 305.884,1283.67 306.112,1285.55 306.34,1283.16 306.567,1249.29 306.795,1214.24 307.023,1215.57 307.251,1179.48 307.478,1156.12 307.706,1072.34 307.934,1116.69 308.162,1078.25 308.39,1096.04 308.617,1093.14 308.845,1006.52 309.073,1018.6 309.301,1018.38 309.528,937.884 309.756,913.67 309.984,909.786 310.212,912.909 310.439,972.711 310.667,948.354 310.895,957.255 311.123,956.442 311.351,942.323 311.578,974.517 311.806,993.825 312.034,992.357 312.262,1062.28 312.489,1089.81 312.717,1078.85 312.945,1103.08 313.173,1144.59 313.401,1164.81 313.628,1125.88 313.856,1163.02 314.084,1113.96 314.312,1117.15 314.539,1097.3 314.767,1118.37 314.995,1059.87 315.223,1063.1 315.45,1068.99 315.678,1056.28 315.906,990.157 316.134,967.55 316.362,905.639 316.589,894.647 316.817,933.781 317.045,936.138 317.273,941.26 317.5,926.393 317.728,946.697 317.956,975.707 318.184,1007.24 318.411,1000.63 318.639,1011.39 318.867,1014.79 319.095,1020.07 319.323,999.576 319.55,998.809 319.778,979.355 320.006,978.157 320.234,974.84 320.461,966.719 320.689,958.09 320.917,945.338 321.145,924.146 321.372,918.179 321.6,929.441 321.828,942.152 322.056,924.904 322.284,953.342 322.511,980.736 322.739,1017.32 322.967,1051.52 323.195,1038.18 323.422,989.264 323.65,952.152 323.878,965.75 324.106,897.183 324.334,909.442 324.561,926.04 324.789,918.27 325.017,913.057 325.245,920.11 325.472,926.594 325.7,929.592 325.928,935.051 326.156,907.92 326.383,921.805 326.611,923.758 326.839,936.566 327.067,917.878 327.295,930.552 327.522,945.891 327.75,958.183 327.978,976.91 328.206,972.669 328.433,939.633 328.661,949.997 328.889,954.096 329.117,980.137 329.344,977.248 329.572,994.183 329.8,978.412 330.028,973.725 330.256,954.337 330.483,933.939 330.711,925.558 330.939,944.669 331.167,941.254 331.394,945.31 331.622,918.931 331.85,939.923 332.078,943.97 332.305,933.515 332.533,952.508 332.761,958.038 332.989,982.107 333.217,991.45 333.444,968.944 333.672,969.59 333.9,971.154 334.128,953.664 334.355,962.856 334.583,970.011 334.811,946.405 335.039,939.149 335.267,922.914 335.494,923.172 335.722,905.213 335.95,897.53 336.178,883.424 336.405,873.118 336.633,894.92 336.861,881.908 337.089,882.869 337.316,854.818 337.544,874.605 337.772,870.458 338,862.649 338.228,879.52 338.455,881.697 338.683,886.139 338.911,886.31 339.139,894.925 339.366,907.798 339.594,916.128 339.822,926.99 340.05,911.954 340.277,916.621 340.505,904.824 340.733,876.061 340.961,876.761 341.189,884.402 341.416,871.729 341.644,888.774 341.872,886.635 342.1,879.974 342.327,892.456 342.555,907.014 342.783,890.038 343.011,884.03 343.238,892.234 343.466,867.542 343.694,869.793 343.922,850.567 344.15,829.221 344.377,837.477 344.605,826.218 344.833,831.544 345.061,850.427 345.288,844.871 345.516,857.956 345.744,869.419 345.972,875.883 346.2,880.649 346.427,877.969 346.655,871.101 346.883,865.334 347.111,879.315 347.338,878.186 347.566,911.198 347.794,925.424 348.022,937.34 348.249,946.568 348.477,948.929 348.705,939.689 348.933,952.513 349.161,950.344 349.388,945.189 349.616,924.782 349.844,924.315 350.072,935.669 350.299,929.557 350.527,889.973 350.755,881.631 350.983,876.52 351.21,870.948 351.438,870.556 351.666,885.741 351.894,890.323 352.122,888.19 352.349,887.129 352.577,874.026 352.805,874.114 353.033,879.445 353.26,895.068 353.488,896.918 353.716,898.879 353.944,898.425 354.171,906.727 354.399,910.96 354.627,908.428 354.855,901.461 355.083,905.626 355.31,909.238 355.538,910.839 355.766,914.942 355.994,922.968 356.221,903.256 356.449,901.542 356.677,882.581 356.905,879.036 357.133,875.903 357.36,860.531 357.588,867.771 357.816,850.474 358.044,844.579 358.271,841.289 358.499,851.451 358.727,869.201 358.955,882.2 359.182,869.166 359.41,863.845 359.638,868.163 359.866,879.16 360.094,872.297 360.321,882.232 360.549,880.047 360.777,876.465 361.005,895.569 361.232,898.571 361.46,874.063 361.688,867.351 361.916,852.262 362.143,861.077 362.371,865.879 362.599,860.082 362.827,848.057 363.055,851.554 363.282,865.821 363.51,876.93 363.738,879.606 363.966,858.525 364.193,855.202 364.421,845.806 364.649,848.297 364.877,858.665 365.104,853.187 365.332,857.854 365.56,869.011 365.788,851.494 366.016,850.496 366.243,857.211 366.471,834.537 366.699,838.871 366.927,848.963 367.154,844.837 367.382,850.193 367.61,849.257 367.838,846.808 368.066,852.222 368.293,846.123 368.521,849.031 368.749,850.77 368.977,844.362 369.204,844.309 369.432,852.176 369.66,848.59 369.888,838.315 370.115,830.19 370.343,833.288 370.571,833.459 370.799,817.806 371.027,811.841 371.254,826.57 371.482,827.268 371.71,835.97 371.938,825 372.165,811.693 372.393,809.318 372.621,812.259 372.849,816.58 373.076,815.098 373.304,816.719 373.532,806.653 373.76,815.616 373.988,805.8 374.215,809.553 374.443,824.868 374.671,818.183 374.899,820.864 375.126,821.377 375.354,814.707 375.582,819.922 375.81,821.163 376.037,823.267 376.265,814.802 376.493,820.878 376.721,829.203 376.949,823.586 377.176,823.604 377.404,821.789 377.632,833.753 377.86,839.604 378.087,836.162 378.315,836.073 378.543,845.092 378.771,844.494 378.999,849.693 379.226,858.115 379.454,857.437 379.682,851.401 379.91,841.149 380.137,847.368 380.365,851.252 380.593,843.469 380.821,839.982 381.048,846.39 381.276,843.646 381.504,841.275 381.732,845.582 381.96,836.798 382.187,844.416 382.415,838.295 382.643,844.461 382.871,852.657 383.098,855.165 383.326,856.736 383.554,854.281 383.782,864.11 384.009,851.568 384.237,855.975 384.465,853.936 384.693,847.08 384.921,855.091 385.148,851.091 385.376,857.374 385.604,855.265 385.832,862.61 386.059,864.561 386.287,868.412 386.515,861.76 386.743,867.653 386.97,871.671 387.198,867.741 387.426,881.604 387.654,883.908 387.882,879.312 388.109,882.892 388.337,883.585 388.565,887.98 388.793,889.756 389.02,889.723 389.248,889.91 389.476,889.865 389.704,902.01 389.932,908.576 390.159,905.907 390.387,898.914 390.615,908.846 390.843,894.599 391.07,892.215 391.298,899.298 391.526,902.248 391.754,896.426 391.981,898.292 392.209,897.652 392.437,892.055 392.665,885.269 392.893,881.353 393.12,867.663 393.348,871.984 393.576,876.583 393.804,881.626 394.031,882.837 394.259,888.345 394.487,876.891 394.715,882.382 394.942,880.586 395.17,873.378 395.398,858.79 395.626,858.979 395.854,858.418 396.081,860.856 396.309,862.434 396.537,854.13 396.765,852.347 396.992,850.738 397.22,854.287 397.448,859.884 397.676,860.138 397.903,860.038 398.131,863.311 398.359,869.167 398.587,864.606 398.815,865.236 399.042,868.811 399.27,873.702 399.498,877.112 399.726,884.632 399.953,890.182 400.181,894.013 400.409,893.181 400.637,885.545 400.865,892.002 401.092,889.691 401.32,888.471 401.548,885.814 401.776,877.165 402.003,876.956 402.231,867.361 402.459,868.863 402.687,865.336 402.914,867.244 403.142,875.277 403.37,877.225 403.598,874.477 403.826,871.04 404.053,871.452 404.281,865.708 404.509,877.502 404.737,879.218 404.964,881.257 405.192,885.146 405.42,891.233 405.648,887.869 405.875,884.568 406.103,884.08 406.331,888.806 406.559,884.364 406.787,880.942 407.014,877.599 407.242,886.215 407.47,882.334 407.698,884.624 407.925,885.836 408.153,881.594 408.381,884.484 408.609,881.027 408.836,883.914 409.064,891.946 409.292,887.823 409.52,887.04 409.748,883.483 409.975,891.782 410.203,892.047 410.431,886.359 410.659,879.32 410.886,882.666 411.114,863.648 411.342,866.783 411.57,871.618 411.798,871.516 412.025,867.624 412.253,868.335 412.481,873.476 412.709,874.31 412.936,874.243 413.164,870.381 413.392,858.854 413.62,864.058 413.847,864.776 414.075,867.24 414.303,868.87 414.531,874.893 414.759,880.845 414.986,886.56 415.214,889.03 415.442,893.653 415.67,890.652 415.897,889.679 416.125,887.657 416.353,881.174 416.581,877.942 416.808,872.412 417.036,868.63 417.264,854.809 417.492,845.689 417.72,842.333 417.947,840.507 418.175,835.381 418.403,835.401 418.631,841.032 418.858,845.208 419.086,846.453 419.314,841.2 419.542,840.677 419.769,843.388 419.997,844.757 420.225,842.385 420.453,850.181 420.681,850.006 420.908,853.012 421.136,852.942 421.364,856.924 421.592,858.541 421.819,856.34 422.047,854.705 422.275,856.876 422.503,856.107 422.731,853.833 422.958,861.59 423.186,862.295 423.414,864.599 423.642,864.731 423.869,864.122 424.097,864.035 424.325,864.203 424.553,857.646 424.78,857.182 425.008,856.189 425.236,857.142 425.464,855.858 425.692,862.996 425.919,865.252 426.147,865.168 426.375,869.328 426.603,869.509 426.83,868.789 427.058,869.958 427.286,868.654 427.514,867.734 427.741,867.465 427.969,872.353 428.197,876.701 428.425,877.197 428.653,882.02 428.88,882.601 429.108,889.954 429.336,896.023 429.564,898.403 429.791,895.59 430.019,897.304 430.247,899.922 430.475,904.615 430.702,893.121 430.93,889.706 431.158,895.135 431.386,901.206 431.614,902.322 431.841,904.018 432.069,900.667 432.297,900.836 432.525,896.69 432.752,892.168 432.98,894.317 433.208,895.62 433.436,902.486 433.664,903.619 433.891,906.632 434.119,909.832 434.347,915.014 434.575,919.76 434.802,917.882 435.03,912.362 435.258,908.498 435.486,909.247 435.713,913.146 435.941,913.746 436.169,914.066 436.397,917.628 436.625,917.956 436.852,914.999 437.08,914.053 437.308,911.542 437.536,912.3 437.763,909.345 437.991,902.096 438.219,903.36 438.447,902.32 438.674,905.811 438.902,907.192 439.13,908.583 439.358,908.153 439.586,908.221 439.813,915.41 440.041,909.537 440.269,913.367 440.497,917.119 440.724,917.07 440.952,922.746 441.18,917.695 441.408,919.848 441.635,912.565 441.863,916.673 442.091,906.73 442.319,906.409 442.547,907.435 442.774,911.994 443.002,911.548 443.23,918.032 443.458,913.357 443.685,914.863 443.913,919.771 444.141,915.24 444.369,922.976 444.597,927.354 444.824,929.6 445.052,928.716 445.28,925.545 445.508,928.205 445.735,928.607 445.963,927.314 446.191,923.129 446.419,927.315 446.646,923.932 446.874,923.246 447.102,922.217 447.33,921.408 447.558,929.836 447.785,927.923 448.013,928.939 448.241,933.144 448.469,929.968 448.696,929.747 448.924,923.476 449.152,924.019 449.38,919.175 449.607,921.316 449.835,925.692 450.063,927.933 450.291,926.256 450.519,925.908 450.746,917.703 450.974,911.422 451.202,916.135 451.43,920.307 451.657,920.775 451.885,921.786 452.113,916.173 452.341,915.17 452.568,921.009 452.796,923.325 453.024,925.352 453.252,928.941 453.48,929.874 453.707,928.344 453.935,925.153 454.163,925.985 454.391,927.01 454.618,927.105 454.846,930.018 455.074,929.203 455.302,927.292 455.53,920.345 455.757,916.551 455.985,919.231 456.213,912.932 456.441,912.474 456.668,906.391 456.896,905.975 457.124,911.785 457.352,909.386 457.579,908.924 457.807,907.938 458.035,912.453 458.263,914.436 458.491,905.952 458.718,905.405 458.946,908.224 459.174,906.459 459.402,905.155 459.629,911.731 459.857,908.164 460.085,911.402 460.313,916.074 460.54,917.086 460.768,912.243 460.996,912.051 461.224,911.229 461.452,910.371 461.679,910.186 461.907,911.118 462.135,908.819 462.363,908.749 462.59,909.526 462.818,909.323 463.046,905.974 463.274,899.931 463.501,901.27 463.729,899.736 463.957,899.442 464.185,898.717 464.413,904.284 464.64,903.829 464.868,906.372 465.096,909.071 465.324,911.474 465.551,909.221 465.779,909.903 466.007,915.548 466.235,919.364 466.462,919.334 466.69,921.166 466.918,919.822 467.146,922.844 467.374,927.727 467.601,927.259 467.829,924.953 468.057,931.354 468.285,927.872 468.512,931.312 468.74,935.971 468.968,939.689 469.196,937.34 469.424,939.163 469.651,938.811 469.879,937.639 470.107,942.251 470.335,942.366 470.562,940.484 470.79,935.77 471.018,935.877 471.246,938.844 471.473,938.761 471.701,937.643 471.929,940.016 472.157,941.816 472.385,938.989 472.612,939.684 472.84,939.939 473.068,936.911 473.296,931.327 473.523,933.364 473.751,934.634 473.979,936.126 474.207,938.369 474.434,936.62 474.662,936.609 474.89,931.007 475.118,929.06 475.346,933.426 475.573,934.939 475.801,937.417 476.029,938.907 476.257,938.568 476.484,939.649 476.712,938.218 476.94,942.94 477.168,943.98 477.395,945.719 477.623,943.718 477.851,943.195 478.079,942.954 478.307,946.303 478.534,936.433 478.762,938.008 478.99,942.032 479.218,939.273 479.445,940.786 479.673,938.419 479.901,936.259 480.129,933.485 480.357,934.711 480.584,935.216 480.812,936.108 481.04,935.94 481.268,941.575 481.495,938.926 481.723,942.568 481.951,944.236 482.179,947.039 482.406,939.799 482.634,943.23 482.862,947.48 483.09,946.52 483.318,944.575 483.545,940.241 483.773,939.315 484.001,940.958 484.229,942.531 484.456,946.117 484.684,943.754 484.912,943.185 485.14,943.318 485.367,943.848 485.595,947.382 485.823,947.105 486.051,946.351 486.279,948.709 486.506,946.495 486.734,946.146 486.962,950.322 487.19,954.616 487.417,956.482 487.645,958.424 487.873,959.688 488.101,953.567 488.328,953.775 488.556,956.726 488.784,944.111 489.012,941.216 489.24,941.601 489.467,934.771 489.695,931.225 489.923,930.67 490.151,933.741 490.378,935.804 490.606,936.944 490.834,936.674 491.062,938.07 491.29,934.487 491.517,938.725 491.745,939.816 491.973,937.603 492.201,937.711 492.428,935.085 492.656,931.696 492.884,929.119 493.112,931.595 493.339,928.4 493.567,928.665 493.795,929.212 494.023,927.96 494.251,930.708 494.478,929.75 494.706,931.85 494.934,933.411 495.162,928.716 495.389,923.885 495.617,926.298 495.845,924.303 496.073,924.202 496.3,925.379 496.528,921.472 496.756,920.975 496.984,921.869 497.212,924.83 497.439,927.974 497.667,925.218 497.895,918.442 498.123,918.038 498.35,919.56 498.578,921.085 498.806,918.719 499.034,918.254 499.261,914.543 499.489,916.301 499.717,919.94 499.945,924.974 500.173,920.238 500.4,924.272 500.628,922.033 500.856,921.556 501.084,925.838 501.311,925.975 501.539,926.437 501.767,921.259 501.995,923.615 502.223,925.922 502.45,923.331 502.678,922.415 502.906,920.568 503.134,920.132 503.361,925.397 503.589,927.053 503.817,928.705 504.045,927.752 504.272,931.131 504.5,932.086 504.728,932.116 504.956,935.823 505.184,935.747 505.411,935.021 505.639,936.637 505.867,937.676 506.095,939.437 506.322,942.954 506.55,942.857 506.778,938.265 507.006,942.285 507.233,945.006 507.461,942.612 507.689,939.237 507.917,942.492 508.145,946.474 508.372,948.402 508.6,950.015 508.828,954.204 509.056,952.223 509.283,952.219 509.511,949.304 509.739,950.889 509.967,951.742 510.194,953.538 510.422,952.903 510.65,947.358 510.878,946.717 511.106,946.022 511.333,942.45 511.561,939.152 511.789,941.209 512.017,944.473 512.244,940.413 512.472,944.178 512.7,942.763 512.928,941.454 513.156,938.522 513.383,942.163 513.611,945.372 513.839,942.11 514.067,942.2 514.294,939.353 514.522,940.033 514.75,938.213 514.978,937.516 515.205,935.948 515.433,934.341 515.661,936.652 515.889,935.174 516.117,937.657 516.344,938.675 516.572,938.554 516.8,938.027 517.028,935.615 517.255,937.43 517.483,934.063 517.711,932.615 517.939,930.767 518.166,933.049 518.394,934.082 518.622,934.026 518.85,934.411 519.078,937.095 519.305,938.454 519.533,941.781 519.761,940.957 519.989,942.293 520.216,941.63 520.444,944.248 520.672,943.961 520.9,944.111 521.127,946.099 521.355,949.836 521.583,948.033 521.811,948.295 522.039,948.702 522.266,951.215 522.494,945.402 522.722,941.002 522.95,942.356 523.177,941.694 523.405,938.757 523.633,934.33 523.861,932.491 524.089,927.331 524.316,924.483 524.544,926.806 524.772,927.676 525,930.994 525.227,931.868 525.455,923.716 525.683,926.432 525.911,926.26 526.138,925.54 526.366,927.574 526.594,928.239 526.822,919.924 527.05,923.17 527.277,923.187 527.505,920.14 527.733,920.527 527.961,924.783 528.188,918.095 528.416,923.078 528.644,923.787 528.872,924.591 529.099,924.14 529.327,920.825 529.555,922.446 529.783,921.909 530.011,922.953 530.238,923.423 530.466,923.771 530.694,924.33 530.922,922.188 531.149,918.076 531.377,917.357 531.605,914.399 531.833,910.765 532.06,908.222 532.288,909.461 532.516,910.381 532.744,910.69 532.972,915.507 533.199,917.143 533.427,912.256 533.655,908.986 533.883,905.776 534.11,904.119 534.338,900.611 534.566,901.724 534.794,905.324 535.022,903.884 535.249,907.876 535.477,908.412 535.705,905.51 535.933,906.174 536.16,904.442 536.388,899.737 536.616,899.965 536.844,901.758 537.071,904.434 537.299,910.673 537.527,907.507 537.755,911.76 537.983,910.502 538.21,912.564 538.438,914.459 538.666,916.75 538.894,918.232 539.121,918.947 539.349,919.303 539.577,920.795 539.805,919.7 540.032,920.863 540.26,924.083 540.488,927.08 540.716,929.862 540.944,932.226 541.171,932.889 541.399,936.64 541.627,937.908 541.855,941.365 542.082,941.269 542.31,942.749 542.538,942.837 542.766,942.251 542.993,944.45 543.221,943.185 543.449,941.404 543.677,943.681 543.905,942.363 544.132,945.617 544.36,941.601 544.588,936.978 544.816,938.505 545.043,938.143 545.271,939.204 545.499,938.76 545.727,938.584 545.955,937.345 546.182,938.915 546.41,941.489 546.638,944.946 546.866,943.244 547.093,944.021 547.321,943.384 547.549,943.859 547.777,945.153 548.004,945.188 548.232,943.305 548.46,945.082 548.688,944.785 548.916,947.266 549.143,943.158 549.371,944.892 549.599,946.259 549.827,942.869 550.054,940.048 550.282,940.348 550.51,937.984 550.738,933.77 550.965,934.568 551.193,937.266 551.421,934.562 551.649,935.643 551.877,933.454 552.104,935.534 552.332,939.294 552.56,938.182 552.788,939.827 553.015,939.128 553.243,935.879 553.471,937.613 553.699,938.542 553.926,940.68 554.154,938.803 554.382,938.437 554.61,939.849 554.838,943.507 555.065,945.001 555.293,946.735 555.521,949.54 555.749,950.962 555.976,951.23 556.204,952.359 556.432,954.149 556.66,957.586 556.888,959.26 557.115,960.108 557.343,963.432 557.571,964.363 557.799,965.891 558.026,967.859 558.254,968.114 558.482,970.167 558.71,968.471 558.937,971.053 559.165,966.429 559.393,965.646 559.621,967.286 559.849,963.239 560.076,964.829 560.304,966.009 560.532,971.533 560.76,970.607 560.987,969.227 561.215,971.596 561.443,972.151 561.671,971.863 561.898,972.467 562.126,975.719 562.354,976.236 562.582,977.097 562.81,978.138 563.037,978.584 563.265,977.984 563.493,980.756 563.721,980 563.948,979.966 564.176,977.303 564.404,974.676 564.632,976.059 564.859,976.092 565.087,974.534 565.315,973.373 565.543,970.013 565.771,970.993 565.998,971.713 566.226,973.221 566.454,975.466 566.682,977.191 566.909,977.076 567.137,974.121 567.365,973.98 567.593,975.169 567.821,975.994 568.048,975.322 568.276,974.909 568.504,973.68 568.732,971.383 568.959,972.92 569.187,973.793 569.415,976.432 569.643,975.37 569.87,970.746 570.098,972.116 570.326,975.427 570.554,975.524 570.782,977.242 571.009,975.398 571.237,975.478 571.465,974.988 571.693,979.694 571.92,983.983 572.148,985.159 572.376,983.626 572.604,979.933 572.831,980.438 573.059,981.079 573.287,980.718 573.515,985.612 573.743,989.413 573.97,990.732 574.198,988.359 574.426,992.22 574.654,991.201 574.881,990.285 575.109,989.38 575.337,987.638 575.565,987.836 575.792,990.25 576.02,989.323 576.248,991.383 576.476,990.347 576.704,994.773 576.931,994.889 577.159,999.652 577.387,998.656 577.615,999.116 577.842,1001.44 578.07,1001.21 578.298,1003.21 578.526,1005.47 578.754,1004.96 578.981,1006.95 579.209,1001.31 579.437,1003.53 579.665,1002.74 579.892,1002.68 580.12,997.546 580.348,993.567 580.576,989.427 580.803,989.78 581.031,990.595 581.259,984.944 581.487,982.063 581.715,981.101 581.942,983.482 582.17,981.228 582.398,984.129 582.626,977.71 582.853,979.127 583.081,979.441 583.309,979.479 583.537,981.337 583.764,982.025 583.992,983.077 584.22,983.703 584.448,985.55 584.676,988.926 584.903,986.528 585.131,986.923 585.359,983.811 585.587,982.606 585.814,985.108 586.042,985.01 586.27,985.185 586.498,987.62 586.725,988.938 586.953,991.495 587.181,989.005 587.409,987.097 587.637,989.749 587.864,990.35 588.092,989.711 588.32,990.223 588.548,991.964 588.775,989.857 589.003,987.656 589.231,990.057 589.459,989.619 589.687,985.133 589.914,981.706 590.142,978.778 590.37,977.815 590.598,978.524 590.825,978.775 591.053,981.78 591.281,982.567 591.509,985.168 591.736,985.301 591.964,987.558 592.192,989.557 592.42,991.698 592.648,992.48 592.875,990.542 593.103,990.447 593.331,990.845 593.559,988.437 593.786,989.428 594.014,990.958 594.242,990.489 594.47,992.863 594.697,994.39 594.925,994.675 595.153,996.312 595.381,993.095 595.609,991.694 595.836,990.918 596.064,986.258 596.292,986.701 596.52,986.91 596.747,988.472 596.975,987.933 597.203,983.57 597.431,981.42 597.658,983.359 597.886,984.659 598.114,982.792 598.342,984.277 598.57,983.144 598.797,984.517 599.025,986.192 599.253,987.111 599.481,987.189 599.708,982.045 599.936,983.443 600.164,982.768 600.392,979.836 600.62,977.947 600.847,975.428 601.075,978.02 601.303,977.76 601.531,979.853 601.758,980.933 601.986,981.302 602.214,981.337 602.442,986.251 602.669,983.512 602.897,982.291 603.125,981.528 603.353,979.243 603.581,981.884 603.808,980.232 604.036,981.659 604.264,980.159 604.492,979.08 604.719,978.71 604.947,982.171 605.175,982.691 605.403,984.524 605.63,985.668 605.858,986.57 606.086,983.563 606.314,984.35 606.542,985.474 606.769,985.837 606.997,987.862 607.225,985.847 607.453,987.721 607.68,985.446 607.908,985.792 608.136,985.227 608.364,983.961 608.591,981.213 608.819,979.659 609.047,979.942 609.275,976.833 609.503,977.18 609.73,976.814 609.958,979.461 610.186,980.475 610.414,980.74 610.641,982.959 610.869,983.399 611.097,984.301 611.325,987.219 611.553,986.978 611.78,986.633 612.008,986.538 612.236,988.049 612.464,985.221 612.691,987.385 612.919,990.435 613.147,986.526 613.375,988.574 613.602,990.075 613.83,987.728 614.058,986.959 614.286,986.888 614.514,985.776 614.741,989.029 614.969,990.651 615.197,990.218 615.425,989.561 615.652,989.549 615.88,991.715 616.108,990.828 616.336,987.678 616.563,989.376 616.791,991.199 617.019,991.963 617.247,990.148 617.475,989.699 617.702,992.231 617.93,990.223 618.158,988.052 618.386,989.389 618.613,990.894 618.841,986.656 619.069,987.788 619.297,983.869 619.524,986.563 619.752,985.084 619.98,982.502 620.208,980.864 620.436,979.221 620.663,977.533 620.891,977.661 621.119,978.542 621.347,976.017 621.574,975.518 621.802,977.06 622.03,976.669 622.258,977.28 622.486,976.536 622.713,979.043 622.941,978.269 623.169,977.662 623.397,978.468 623.624,979.605 623.852,980.166 624.08,981.093 624.308,981.922 624.535,979.584 624.763,979.029 624.991,980.586 625.219,976.255 625.447,976.593 625.674,977.334 625.902,978.599 626.13,978.023 626.358,977.344 626.585,974.431 626.813,968.61 627.041,967.19 627.269,970.767 627.496,963.714 627.724,966.163 627.952,967.719 628.18,964.701 628.408,963.39 628.635,962.753 628.863,962.109 629.091,962.389 629.319,964.977 629.546,961.282 629.774,961.704 630.002,959.41 630.23,955.583 630.457,952.124 630.685,950.895 630.913,950.935 631.141,949.39 631.369,950.609 631.596,949.374 631.824,949.942 632.052,948.746 632.28,949.142 632.507,948.019 632.735,947.392 632.963,944.529 633.191,945.277 633.419,944.876 633.646,944.651 633.874,946.581 634.102,949.175 634.33,950.73 634.557,952.933 634.785,952.474 635.013,951.747 635.241,952.236 635.468,953.858 635.696,955.48 635.924,954.585 636.152,951.925 636.38,950.551 636.607,950.111 636.835,947.962 637.063,947.853 637.291,947.024 637.518,947.542 637.746,949.191 637.974,948.08 638.202,946.749 638.429,945.536 638.657,942.53 638.885,943.124 639.113,943.738 639.341,946.477 639.568,944.874 639.796,939.978 640.024,939.336 640.252,938.851 640.479,941.548 640.707,940.564 640.935,936.485 641.163,938.356 641.39,937.885 641.618,937.892 641.846,935.413 642.074,934.252 642.302,932.14 642.529,931.121 642.757,932.595 642.985,935.65 643.213,935.777 643.44,934.908 643.668,932.428 643.896,932.284 644.124,930.98 644.352,932.344 644.579,931.612 644.807,929.178 645.035,928.042 645.263,929.307 645.49,925.795 645.718,925.096 645.946,924.108 646.174,924.578 646.401,924.767 646.629,924.978 646.857,925.111 647.085,924.615 647.313,927.145 647.54,923.709 647.768,923.467 647.996,920.501 648.224,919.726 648.451,917.368 648.679,919.429 648.907,920.891 649.135,917.553 649.362,917.496 649.59,917.647 649.818,917.434 650.046,918.038 650.274,917.891 650.501,917.527 650.729,919.265 650.957,919.155 651.185,920.884 651.412,918.522 651.64,918.123 651.868,919.588 652.096,918.705 652.323,922.153 652.551,921.255 652.779,922.331 653.007,921.668 653.235,922.308 653.462,921.321 653.69,922.593 653.918,923.358 654.146,923.072 654.373,921.209 654.601,916.755 654.829,915.242 655.057,914.117 655.285,913.242 655.512,912.297 655.74,910.295 655.968,911.013 656.196,912.206 656.423,910.3 656.651,909.097 656.879,909.54 657.107,910.25 657.334,910.64 657.562,909.375 657.79,910.885 658.018,911.557 658.246,911.837 658.473,909.613 658.701,909.446 658.929,906.474 659.157,909.049 659.384,910.403 659.612,911.286 659.84,912.561 660.068,912.205 660.295,911.176 660.523,908.859 660.751,909.924 660.979,909.872 661.207,909.735 661.434,910.619 661.662,912.886 661.89,914.765 662.118,914.748 662.345,914.022 662.573,914.505 662.801,917.332 663.029,917.301 663.256,915.562 663.484,914.376 663.712,912.576 663.94,911.104 664.168,909.452 664.395,908.821 664.623,908.516 664.851,910.111 665.079,911.259 665.306,914.158 665.534,914.785 665.762,913.986 665.99,915.631 666.218,917.297 666.445,911.508 666.673,909.631 666.901,912.036 667.129,912.654 667.356,914.082 667.584,912.593 667.812,913.701 668.04,916.027 668.267,914.753 668.495,915.924 668.723,914.481 668.951,915.138 669.179,917.055 669.406,917.075 669.634,918.096 669.862,918.643 670.09,919.859 670.317,920.832 670.545,923.894 670.773,923.949 671.001,922.53 671.228,923.298 671.456,922.45 671.684,922.716 671.912,923.029 672.14,920.026 672.367,920.847 672.595,918.831 672.823,919.704 673.051,918.264 673.278,917.501 673.506,915.767 673.734,913.982 673.962,913.451 674.189,912.862 674.417,913.374 674.645,914.344 674.873,915.106 675.101,917.551 675.328,915.249 675.556,916.448 675.784,918.333 676.012,918.13 676.239,918.485 676.467,917.777 676.695,918.229 676.923,919.055 677.151,917.049 677.378,918.055 677.606,919.633 677.834,918.681 678.062,916.771 678.289,917.558 678.517,917.09 678.745,917.926 678.973,918.926 679.2,920.009 679.428,920.506 679.656,921.402 679.884,920.608 680.112,921.475 680.339,923.766 680.567,922.764 680.795,922.918 681.023,924.552 681.25,924.477 681.478,924.888 681.706,925.294 681.934,927.49 682.161,929.498 682.389,927.398 682.617,924.909 682.845,925.483 683.073,926.41 683.3,928.455 683.528,925.601 683.756,926.86 683.984,928.466 684.211,930.731 684.439,931.099 684.667,926.658 684.895,928.391 685.122,928.898 685.35,930.17 685.578,929.045 685.806,926.017 686.034,927.907 686.261,927.493 686.489,929.893 686.717,928.104 686.945,926.734 687.172,926.894 687.4,928.243 687.628,927.187 687.856,927.235 688.084,927.865 688.311,926.48 688.539,925.869 688.767,927.737 688.995,928.714 689.222,929.583 689.45,930.91 689.678,931.568 689.906,930.247 690.133,928.71 690.361,928.905 690.589,931.042 690.817,934.399 691.045,935.064 691.272,933.743 691.5,933.395 691.728,934.825 691.956,934.458 692.183,934.715 692.411,936.127 692.639,935.57 692.867,936.745 693.094,936.806 693.322,937.588 693.55,936.422 693.778,939.249 694.006,940.842 694.233,941.025 694.461,941.118 694.689,934.918 694.917,931.202 695.144,930.041 695.372,930.431 695.6,931.056 695.828,933.365 696.055,932.93 696.283,932.897 696.511,935.17 696.739,936.447 696.967,937.911 697.194,938.104 697.422,939.405 697.65,937.812 697.878,937.909 698.105,936.504 698.333,937.601 698.561,936.206 698.789,934.146 699.017,931.604 699.244,930.401 699.472,930.929 699.7,931.349 699.928,933.188 700.155,932.686 700.383,931.668 700.611,933.903 700.839,935.841 701.066,938.779 701.294,941.18 701.522,942.233 701.75,941.686 701.978,941.694 702.205,945.009 702.433,945.478 702.661,947.676 702.889,949.914 703.116,949.176 703.344,948.814 703.572,948.152 703.8,947.008 704.027,947.74 704.255,946.694 704.483,945.308 704.711,943.903 704.939,941.453 705.166,941.664 705.394,943.103 705.622,940.92 705.85,937.618 706.077,938.944 706.305,939.862 706.533,938.213 706.761,938.496 706.988,938.075 707.216,937.035 707.444,937.345 707.672,938.26 707.9,937.623 708.127,936.979 708.355,937.604 708.583,939.001 708.811,938.905 709.038,939.712 709.266,941.532 709.494,940.459 709.722,941.359 709.95,941.354 710.177,939.683 710.405,939.834 710.633,940.313 710.861,940.318 711.088,941.674 711.316,940.943 711.544,941.882 711.772,942.082 711.999,944.756 712.227,944.863 712.455,942.247 712.683,943.645 712.911,944.089 713.138,939.702 713.366,942.649 713.594,942.143 713.822,940.477 714.049,940.589 714.277,941.759 714.505,941.503 714.733,943.685 714.96,942.85 715.188,943.593 715.416,942.95 715.644,943.123 715.872,944.697 716.099,943.922 716.327,944.757 716.555,946.9 716.783,945.881 717.01,946.628 717.238,946.899 717.466,945.413 717.694,944.599 717.921,945.635 718.149,944.439 718.377,946.967 718.605,948.932 718.833,949.384 719.06,950.216 719.288,950.026 719.516,949.984 719.744,945.39 719.971,946.761 720.199,948.455 720.427,948.2 720.655,949.249 720.883,948.444 721.11,949.392 721.338,947.017 721.566,946.005 721.794,946.225 722.021,948.048 722.249,947.568 722.477,948.254 722.705,949.119 722.932,950.263 723.16,950.334 723.388,950.993 723.616,948.761 723.844,948.346 724.071,949.994 724.299,950.651 724.527,948.598 724.755,948.897 724.982,948.252 725.21,947.514 725.438,947.554 725.666,946.672 725.893,944.303 726.121,943.154 726.349,942.683 726.577,941.396 726.805,939.667 727.032,940.779 727.26,940.151 727.488,938.782 727.716,939.504 727.943,939.568 728.171,940.825 728.399,940.577 728.627,940.054 728.854,940.218 729.082,938.184 729.31,940.034 729.538,940.747 729.766,941.266 729.993,940.722 730.221,942.844 730.449,941.832 730.677,941.756 730.904,939.016 731.132,939.172 731.36,939.511 731.588,939.735 731.816,940.017 732.043,940.527 732.271,942.145 732.499,942.303 732.727,942.874 732.954,942.174 733.182,942.575 733.41,942.719 733.638,943.279 733.865,945.411 734.093,945.162 734.321,945.147 734.549,942.872 734.777,943.846 735.004,946.224 735.232,943.326 735.46,943.642 735.688,941.669 735.915,942.221 736.143,941.289 736.371,941.218 736.599,942.117 736.826,942.252 737.054,942.295 737.282,944.197 737.51,944.733 737.738,945.693 737.965,945.691 738.193,943.082 738.421,944.488 738.649,944.12 738.876,944.589 739.104,944.901 739.332,943.979 739.56,945.887 739.787,945.045 740.015,944.285 740.243,940.333 740.471,938.101 740.699,936.607 740.926,935.151 741.154,935.03 741.382,935.928 741.61,937.798 741.837,940.58 742.065,940.654 742.293,939.984 742.521,939.67 742.749,940.314 742.976,941.605 743.204,940.404 743.432,939.006 743.66,940.304 743.887,942.284 744.115,941.648 744.343,939.38 744.571,941.583 744.798,940.741 745.026,940.491 745.254,942.35 745.482,943.751 745.71,946.268 745.937,945.481 746.165,944.419 746.393,943.803 746.621,942.696 746.848,944.19 747.076,943.091 747.304,942.525 747.532,942.442 747.759,940.988 747.987,942.055 748.215,941.597 748.443,943.457 748.671,943.865 748.898,942.417 749.126,945.071 749.354,945.139 749.582,944.574 749.809,946.2 750.037,945.926 750.265,944.236 750.493,944.161 750.72,943.633 750.948,941.345 751.176,944.481 751.404,945.393 751.632,945.864 751.859,945.136 752.087,945.205 752.315,944.717 752.543,945.199 752.77,945.879 752.998,947.666 753.226,948.071 753.454,948.468 753.682,949.563 753.909,947.568 754.137,946.186 754.365,945.623 754.593,944.458 754.82,944.368 755.048,944.418 755.276,944.853 755.504,946.677 755.731,947.525 755.959,948.779 756.187,947.57 756.415,947.777 756.643,949.27 756.87,948.915 757.098,949.483 757.326,950.332 757.554,947.321 757.781,949.362 758.009,950.477 758.237,950.802 758.465,951.175 758.692,947.11 758.92,948.835 759.148,948.713 759.376,948.635 759.604,946.428 759.831,946.694 760.059,946.426 760.287,945.757 760.515,946.826 760.742,946.926 760.97,948.925 761.198,949.57 761.426,951.779 761.653,951.465 761.881,951.623 762.109,951.788 762.337,952.381 762.565,951.36 762.792,949.271 763.02,951.009 763.248,948.246 763.476,946.856 763.703,947.389 763.931,948.856 764.159,949.133 764.387,950.747 764.615,950.693 764.842,950.677 765.07,951.382 765.298,952.636 765.526,953.807 765.753,954.893 765.981,954.025 766.209,952.987 766.437,951.24 766.664,952.712 766.892,951.405 767.12,950.036 767.348,949.87 767.576,949.289 767.803,951.384 768.031,950.988 768.259,950.368 768.487,951.867 768.714,948.811 768.942,947.269 769.17,948.407 769.398,946.384 769.625,945.283 769.853,943.785 770.081,942.976 770.309,942.93 770.537,943.487 770.764,944.119 770.992,944.643 771.22,945.349 771.448,947.076 771.675,948.621 771.903,947.581 772.131,948.657 772.359,950.005 772.586,951.313 772.814,950.102 773.042,951.354 773.27,951.528 773.498,953.093 773.725,952.918 773.953,955.069 774.181,956.85 774.409,957.525 774.636,956.615 774.864,957.223 775.092,957.807 775.32,958.546 775.548,957.818 775.775,958.634 776.003,959.186 776.231,960.257 776.459,957.996 776.686,958.639 776.914,959.417 777.142,958.911 777.37,955.1 777.597,954.271 777.825,955.222 778.053,956.059 778.281,955.302 778.509,955.292 778.736,954.041 778.964,954.255 779.192,952.938 779.42,953.751 779.647,954.054 779.875,953.577 780.103,952.235 780.331,951.651 780.558,950.004 780.786,949.881 781.014,947.821 781.242,944.852 781.47,943.078 781.697,942.758 781.925,943.237 782.153,943.507 782.381,943.799 782.608,945.699 782.836,946.59 783.064,946.271 783.292,947.637 783.519,946.458 783.747,945.531 783.975,943.996 784.203,942.783 784.431,941.704 784.658,941.331 784.886,941.268 785.114,941.358 785.342,941.727 785.569,942.425 785.797,940.985 786.025,942.138 786.253,942.438 786.481,942.076 786.708,941.72 786.936,940.261 787.164,938.805 787.392,937.733 787.619,937.699 787.847,939.277 788.075,939.906 788.303,939.326 788.53,939.776 788.758,940.315 788.986,941.635 789.214,942.467 789.442,942.977 789.669,944.313 789.897,945.987 790.125,948.285 790.353,948.823 790.58,948.821 790.808,948.682 791.036,946.029 791.264,945.107 791.491,944.256 791.719,945.147 791.947,947.073 792.175,947.73 792.403,948.907 792.63,948.481 792.858,947.393 793.086,947.88 793.314,947.838 793.541,948.425 793.769,948.965 793.997,949.18 794.225,947.603 794.452,947.356 794.68,946.998 794.908,947.448 795.136,946.244 795.364,946.557 795.591,944.602 795.819,943.833 796.047,945.473 796.275,945.233 796.502,947.392 796.73,945.088 796.958,943.102 797.186,941.036 797.413,941.478 797.641,941.26 797.869,939.346 798.097,939.027 798.325,938.771 798.552,939.843 798.78,940.573 799.008,942.808 799.236,941.241 799.463,940.295 799.691,942.053 799.919,940.962 800.147,939.288 800.375,938.595 800.602,939.319 800.83,938.094 801.058,937.874 801.286,939.167 801.513,937.61 801.741,938.162 801.969,939.861 802.197,936.308 802.424,935.843 802.652,937.288 802.88,936.585 803.108,937.548 803.336,935.979 803.563,934.03 803.791,935.706 804.019,936.06 804.247,934.67 804.474,935.765 804.702,934.724 804.93,936.244 805.158,936.776 805.385,936.959 805.613,937.176 805.841,934.911 806.069,935.288 806.297,933.796 806.524,934.039 806.752,935.378 806.98,936.116 807.208,936.477 807.435,934.937 807.663,936.468 807.891,937.945 808.119,938.766 808.346,937.721 808.574,938.432 808.802,939.126 809.03,940.33 809.258,942.39 809.485,940.301 809.713,939.698 809.941,940.338 810.169,939.961 810.396,940.621 810.624,938.562 810.852,940.212 811.08,939.727 811.308,938.182 811.535,938.365 811.763,938.133 811.991,937.943 812.219,937.2 812.446,938.257 812.674,937.675 812.902,937.456 813.13,936.175 813.357,938.776 813.585,940.283 813.813,940.245 814.041,936.76 814.269,938.017 814.496,939.009 814.724,938.171 814.952,937.689 815.18,938.769 815.407,938.045 815.635,936.698 815.863,936.005 816.091,935.623 816.318,935.679 816.546,936.114 816.774,937.351 817.002,936.793 817.23,937.369 817.457,937.87 817.685,939.957 817.913,937.422 818.141,938.331 818.368,938.862 818.596,938.203 818.824,937.95 819.052,937.679 819.279,938.418 819.507,938.149 819.735,938.472 819.963,938.969 820.191,938.292 820.418,939.679 820.646,939.546 820.874,938.132 821.102,939.647 821.329,939.442 821.557,936.89 821.785,935.272 822.013,933.208 822.241,931.898 822.468,930.639 822.696,930.033 822.924,931.578 823.152,931.767 823.379,932.05 823.607,934.411 823.835,935.195 824.063,935.824 824.29,937.029 824.518,938.723 824.746,936.827 824.974,935.585 825.202,936.92 825.429,938.659 825.657,938.015 825.885,937.913 826.113,938.763 826.34,939.928 826.568,940.354 826.796,939.255 827.024,940.707 827.251,939.798 827.479,939.385 827.707,939.993 827.935,940.194 828.163,941.69 828.39,941.972 828.618,940.965 828.846,942.444 829.074,940.618 829.301,939.458 829.529,939.219 829.757,938.466 829.985,939.887 830.212,940.234 830.44,939.883 830.668,938.322 830.896,937.411 831.124,938.358 831.351,940.19 831.579,940.737 831.807,942.239 832.035,941.829 832.262,942.242 832.49,938.599 832.718,940 832.946,940.099 833.174,939.254 833.401,939.666 833.629,939.968 833.857,939.181 834.085,940.868 834.312,942.693 834.54,941.968 834.768,941.528 834.996,941.499 835.223,941.775 835.451,942.721 835.679,942.889 835.907,942.575 836.135,942.895 836.362,942.997 836.59,944.539 836.818,943.894 837.046,944.419 837.273,945.672 837.501,944.953 837.729,944.101 837.957,943.08 838.184,943.357 838.412,942.921 838.64,942.085 838.868,940.937 839.096,942.243 839.323,941.867 839.551,941.334 839.779,942.454 840.007,943.628 840.234,945.502 840.462,946.243 840.69,945.724 840.918,946.055 841.145,945.388 841.373,943.573 841.601,944.337 841.829,942.898 842.057,944.571 842.284,944.415 842.512,944.689 842.74,946.087 842.968,946.709 843.195,947.067 843.423,945.764 843.651,946.084 843.879,947.653 844.107,947.252 844.334,947.641 844.562,947.366 844.79,946.882 845.018,946.893 845.245,946.918 845.473,946.837 845.701,947.154 845.929,947.889 846.156,946.822 846.384,947.787 846.612,947.766 846.84,947.995 847.068,949.197 847.295,949.436 847.523,946.573 847.751,947.549 847.979,947.915 848.206,948.543 848.434,947.276 848.662,947.544 848.89,949.015 849.117,948.133 849.345,949.098 849.573,947.743 849.801,948.233 850.029,948.804 850.256,949.092 850.484,949.562 850.712,950.26 850.94,950.48 851.167,951.415 851.395,952.226 851.623,951.948 851.851,952.777 852.078,951.343 852.306,952.88 852.534,953.472 852.762,954.399 852.99,953.23 853.217,953.217 853.445,951.098 853.673,952.217 853.901,951.026 854.128,951.663 854.356,951.159 854.584,951.338 854.812,950.819 855.04,950.584 855.267,952.141 855.495,952.824 855.723,953.602 855.951,955.047 856.178,954.295 856.406,953.936 856.634,953.952 856.862,954.801 857.089,953.454 857.317,953.786 857.545,952.21 857.773,950.519 858.001,950.466 858.228,952.361 858.456,952.796 858.684,948.69 858.912,948.788 859.139,949.044 859.367,951.876 859.595,951.751 859.823,952.803 860.05,954.004 860.278,955.634 860.506,955.473 860.734,955.898 860.962,957.411 861.189,957.827 861.417,958.588 861.645,959.591 861.873,958.485 862.1,958.639 862.328,958.845 862.556,958.28 862.784,958.437 863.011,958.841 863.239,958.471 863.467,957.01 863.695,957.262 863.923,957.382 864.15,957.397 864.378,958.105 864.606,957.675 864.834,956.738 865.061,957.872 865.289,955.679 865.517,957.314 865.745,955.717 865.973,956.705 866.2,958.028 866.428,957.988 866.656,957.093 866.884,956.885 867.111,955.623 867.339,956.276 867.567,955.506 867.795,955.441 868.022,956.867 868.25,955.59 868.478,955.448 868.706,954.52 868.934,954.642 869.161,953.601 869.389,953.981 869.617,955.181 869.845,955.789 870.072,956.054 870.3,957.001 870.528,957.635 870.756,957.507 870.983,958.221 871.211,958.053 871.439,958.321 871.667,959.401 871.895,958.214 872.122,956.954 872.35,957.068 872.578,957.798 872.806,956.889 873.033,957.989 873.261,957.838 873.489,956.506 873.717,956.127 873.944,956.082 874.172,957.594 874.4,959.786 874.628,957.793 874.856,956.955 875.083,956.704 875.311,957.801 875.539,957.549 875.767,958.584 875.994,958.046 876.222,958.058 876.45,959.187 876.678,960.462 876.906,962.568 877.133,960.785 877.361,960.152 877.589,960.437 877.817,960.345 878.044,959.294 878.272,960.177 878.5,961.122 878.728,961.917 878.955,963.208 879.183,962.911 879.411,963.302 879.639,964.456 879.867,964.721 880.094,965.305 880.322,963.931 880.55,963.814 880.778,964.184 881.005,963.789 881.233,963.917 881.461,965.028 881.689,966.961 881.916,968.771 882.144,970.531 882.372,971.325 882.6,972.14 882.828,969.712 883.055,969.953 883.283,970.648 883.511,968.307 883.739,968.408 883.966,967.973 884.194,968.331 884.422,968.933 884.65,966.723 884.877,966.713 885.105,967.472 885.333,967.71 885.561,967.124 885.789,966.473 886.016,967.315 886.244,966.883 886.472,964.154 886.7,963.751 886.927,965.255 887.155,967.174 887.383,967.906 887.611,967.999 887.839,967.175 888.066,966.633 888.294,965.892 888.522,966.329 888.75,964.996 888.977,965.58 889.205,963.375 889.433,964.017 889.661,962.672 889.888,963.308 890.116,964.103 890.344,962.14 890.572,962.075 890.8,959.717 891.027,960.929 891.255,962.002 891.483,962.366 891.711,962.204 891.938,962.36 892.166,961.594 892.394,962.896 892.622,963.029 892.849,959.708 893.077,958.79 893.305,959.636 893.533,958.089 893.761,957.463 893.988,956.074 894.216,956.328 894.444,956.598 894.672,954.941 894.899,954.074 895.127,955.015 895.355,954.627 895.583,953.641 895.81,955.212 896.038,955.495 896.266,956.65 896.494,956.381 896.722,956.64 896.949,957.404 897.177,957.001 897.405,955.41 897.633,954.889 897.86,951.612 898.088,952.549 898.316,952.409 898.544,952.227 898.772,953.452 898.999,952.973 899.227,950.73 899.455,949.824 899.683,949.327 899.91,949.581 900.138,949.785 900.366,951.088 900.594,950.447 900.821,951.236 901.049,950.186 901.277,949.237 901.505,946.472 901.733,946.543 901.96,946.647 902.188,947.007 902.416,946.759 902.644,946.074 902.871,944.594 903.099,943.278 903.327,944.299 903.555,944.574 903.782,944.073 904.01,943.772 904.238,944.223 904.466,945.928 904.694,947.052 904.921,946.683 905.149,946.878 905.377,947.263 905.605,948.978 905.832,949.193 906.06,950.549 906.288,951.167 906.516,950.607 906.743,950.166 906.971,950.477 907.199,948.83 907.427,948.895 907.655,949.792 907.882,950.225 908.11,948.585 908.338,948.423 908.566,950.185 908.793,950.393 909.021,951.46 909.249,951.283 909.477,950.941 909.705,951.815 909.932,952.161 910.16,952.379 910.388,951.577 910.616,951.29 910.843,950.713 911.071,951.032 911.299,951.499 911.527,953.499 911.754,954.926 911.982,956.251 912.21,956.398 912.438,957.342 912.666,957.865 912.893,956.369 913.121,953.536 913.349,954.149 913.577,953.562 913.804,953.288 914.032,951.809 914.26,953.342 914.488,955.147 914.715,954.433 914.943,956.107 915.171,956.429 915.399,955.318 915.627,955.93 915.854,957.348 916.082,957.603 916.31,958.743 916.538,958.59 916.765,958.65 916.993,959.951 917.221,959.998 917.449,957.514 917.676,957.607 917.904,958.651 918.132,959.325 918.36,960.707 918.588,961.637 918.815,961.252 919.043,961.015 919.271,961.251 919.499,960.021 919.726,960.103 919.954,959.992 920.182,960.248 920.41,960.483 920.638,961.999 920.865,962.829 921.093,961.943 921.321,962.963 921.549,963.769 921.776,965.158 922.004,964.511 922.232,965.583 922.46,966.827 922.687,964.781 922.915,964.809 923.143,964.885 923.371,964.221 923.599,962.626 923.826,960.81 924.054,960.245 924.282,961.749 924.51,962.095 924.737,962.084 924.965,963.346 925.193,963.455 925.421,963.653 925.648,963.934 925.876,963.956 926.104,961.635 926.332,963.181 926.56,962.684 926.787,963.3 927.015,964.294 927.243,964.911 927.471,963.941 927.698,965.197 927.926,965.109 928.154,964.069 928.382,964.306 928.609,963.466 928.837,964.023 929.065,965.077 929.293,966.575 929.521,966.939 929.748,967.668 929.976,968.641 930.204,969.059 930.432,970.559 930.659,970.852 930.887,971.548 931.115,972.808 931.343,972.701 931.571,973.452 931.798,973.118 932.026,973.263 932.254,972.839 932.482,972.607 932.709,972.131 932.937,973.101 933.165,974.358 933.393,976.683 933.62,975.38 933.848,975.191 934.076,975.784 934.304,976.426 934.532,976.793 934.759,976.94 934.987,975.848 935.215,973.016 935.443,973.306 935.67,972.97 935.898,973.011 936.126,973.633 936.354,973.842 936.581,973.583 936.809,973.635 937.037,974.775 937.265,975.164 937.493,975.897 937.72,974.998 937.948,974.758 938.176,974.483 938.404,974.615 938.631,974.55 938.859,974.468 939.087,975.635 939.315,976.564 939.542,977.15 939.77,978.125 939.998,977.906 940.226,977.383 940.454,977.318 940.681,978.474 940.909,978.136 941.137,977.862 941.365,977.479 941.592,977.376 941.82,979.088 942.048,979.38 942.276,980.006 942.504,979.025 942.731,978.591 942.959,978.399 943.187,979.481 943.415,978.384 943.642,979.313 943.87,978.916 944.098,979.246 944.326,978.658 944.553,977.726 944.781,977.245 945.009,977.405 945.237,976.641 945.465,976.558 945.692,975.683 945.92,974.946 946.148,974.712 946.376,974.256 946.603,974.016 946.831,973.672 947.059,970.647 947.287,971.021 947.514,967.869 947.742,968.36 947.97,969.289 948.198,970.399 948.426,968.977 948.653,969.956 948.881,971.713 949.109,973.166 949.337,972.62 949.564,972.431 949.792,973.397 950.02,973.116 950.248,974.219 950.475,975.174 950.703,976.718 950.931,977.318 951.159,976.273 951.387,975.177 951.614,976.032 951.842,975.879 952.07,974.597 952.298,975.541 952.525,975.555 952.753,976.577 952.981,977.09 953.209,977.696 953.437,978.747 953.664,979.735 953.892,978.866 954.12,978.302 954.348,978.834 954.575,978.547 954.803,977.575 955.031,974.822 955.259,974.845 955.486,974.58 955.714,974.784 955.942,975.548 956.17,976.773 956.398,976.1 956.625,974.809 956.853,975.22 957.081,973.635 957.309,973.458 957.536,972.856 957.764,973.071 957.992,973.562 958.22,972.907 958.447,973.525 958.675,973.93 958.903,974.632 959.131,973.598 959.359,973.723 959.586,973.526 959.814,973.566 960.042,973.952 960.27,975.081 960.497,974.802 960.725,977.066 960.953,975.333 961.181,974.961 961.408,975.424 961.636,974.706 961.864,974.714 962.092,974.902 962.32,975.287 962.547,975.728 962.775,974.813 963.003,974.267 963.231,974.933 963.458,975.431 963.686,977.1 963.914,976.87 964.142,977.341 964.37,977.571 964.597,976.696 964.825,977.615 965.053,978.349 965.281,979.362 965.508,980.629 965.736,981.794 965.964,981.357 966.192,980.927 966.419,981.108 966.647,981.601 966.875,979.657 967.103,980.254 967.331,981.244 967.558,982.427 967.786,983.309 968.014,983.786 968.242,983.841 968.469,984.701 968.697,983.622 968.925,984.325 969.153,984.429 969.38,984.32 969.608,983.474 969.836,984.231 970.064,984.512 970.292,983.536 970.519,984.545 970.747,985.353 970.975,985.23 971.203,986.11 971.43,984.865 971.658,984.486 971.886,985.872 972.114,985.102 972.341,985.144 972.569,983.986 972.797,984.856 973.025,985.921 973.253,985.009 973.48,985.371 973.708,986.376 973.936,987.185 974.164,986.46 974.391,986.298 974.619,986.245 974.847,986.272 975.075,986.063 975.303,986.163 975.53,986.094 975.758,984.103 975.986,985.397 976.214,983.813 976.441,983.856 976.669,983.412 976.897,983.743 977.125,982.283 977.352,982.483 977.58,983.557 977.808,982.664 978.036,982.598 978.264,983.771 978.491,984.568 978.719,984.363 978.947,986.198 979.175,985.264 979.402,984.624 979.63,985.152 979.858,984.646 980.086,983.28 980.313,983.287 980.541,983.479 980.769,983.205 980.997,982.629 981.225,982.608 981.452,983.146 981.68,982.987 981.908,984.514 982.136,984.8 982.363,985.426 982.591,983.637 982.819,984.806 983.047,984.773 983.274,984.664 983.502,982.991 983.73,983.02 983.958,983.741 984.186,982.319 984.413,980.826 984.641,981.084 984.869,981.543 985.097,980.48 985.324,979.635 985.552,980.787 985.78,980.359 986.008,978.923 986.236,979.232 986.463,977.18 986.691,976.258 986.919,975.54 987.147,976.085 987.374,976.455 987.602,977.188 987.83,977.011 988.058,975.122 988.285,975.754 988.513,975.857 988.741,976.743 988.969,976.495 989.197,976.65 989.424,974.519 989.652,974.716 989.88,975.241 990.108,975.193 990.335,974.945 990.563,974.464 990.791,975.141 991.019,975.387 991.246,973.644 991.474,972.584 991.702,971.184 991.93,972.462 992.158,972.837 992.385,973.293 992.613,974.153 992.841,973.562 993.069,971.699 993.296,972.366 993.524,970.735 993.752,970.81 993.98,972.171 994.207,972.79 994.435,971.826 994.663,973.297 994.891,974.118 995.119,974.654 995.346,975.424 995.574,974.877 995.802,975.106 996.03,975.521 996.257,976.172 996.485,976.11 996.713,975.687 996.941,975.406 997.169,976.384 997.396,977.863 997.624,978.449 997.852,979.218 998.08,979.179 998.307,980.633 998.535,981.388 998.763,981.29 998.991,979.411 999.218,979.859 999.446,981.377 999.674,982.183 999.902,979.759 1000.13,980.119 1000.36,980.45 1000.59,980.699 1000.81,980.532 1001.04,980.166 1001.27,980.604 1001.5,980.53 1001.72,980.748 1001.95,978.649 1002.18,979.075 1002.41,978.775 1002.64,978.849 1002.86,977.725 1003.09,979.267 1003.32,978.449 1003.55,979.659 1003.77,979.143 1004,979.044 1004.23,979.354 1004.46,979.266 1004.68,978.909 1004.91,977.517 1005.14,979.092 1005.37,979.981 1005.6,980.469 1005.82,981.595 1006.05,980.729 1006.28,982.07 1006.51,982.733 1006.73,981.416 1006.96,980.246 1007.19,980.978 1007.42,980.925 1007.65,980.665 1007.87,981.141 1008.1,981.358 1008.33,981.633 1008.56,980.938 1008.78,981.262 1009.01,981.818 1009.24,981.303 1009.47,981.489 1009.7,981.52 1009.92,980.946 1010.15,980.257 1010.38,981.137 1010.61,981.329 1010.83,981.465 1011.06,983.041 1011.29,982.308 1011.52,983.185 1011.75,981.743 1011.97,982.024 1012.2,982.619 1012.43,983.172 1012.66,984.155 1012.88,982.303 1013.11,982.891 1013.34,983.568 1013.57,983.486 1013.8,982.577 1014.02,981.423 1014.25,981.74 1014.48,980.172 1014.71,980.105 1014.93,980.039 1015.16,979.836 1015.39,979.487 1015.62,980.102 1015.85,979.317 1016.07,979.552 1016.3,977.278 1016.53,976.968 1016.76,977.727 1016.98,978.346 1017.21,977.863 1017.44,977.539 1017.67,976.696 1017.9,977.039 1018.12,976.943 1018.35,976.059 1018.58,976.293 1018.81,976.425 1019.03,975.68 1019.26,974.504 1019.49,974.753 1019.72,975.293 1019.95,974.318 1020.17,974.536 1020.4,975.969 1020.63,975.268 1020.86,976.187 1021.08,977.001 1021.31,976.31 1021.54,977.969 1021.77,977.85 1022,977.765 1022.22,977.256 1022.45,978.405 1022.68,978.356 1022.91,978.195 1023.13,978.069 1023.36,979.157 1023.59,978.15 1023.82,978.29 1024.05,977.693 1024.27,977.813 1024.5,979.376 1024.73,980.317 1024.96,980.204 1025.18,979.9 1025.41,980.037 1025.64,978.418 1025.87,979.133 1026.1,978.397 1026.32,979.828 1026.55,980.481 1026.78,981.444 1027.01,982.025 1027.23,983.016 1027.46,984.105 1027.69,984.556 1027.92,983.599 1028.15,983.874 1028.37,984.564 1028.6,984.51 1028.83,984.456 1029.06,983.868 1029.28,983.078 1029.51,981.158 1029.74,981.287 1029.97,981.19 1030.2,980.537 1030.42,981.157 1030.65,981.598 1030.88,980.868 1031.11,980.464 1031.33,978.781 1031.56,978.693 1031.79,979.105 1032.02,978.667 1032.25,977.615 1032.47,978.157 1032.7,978.576 1032.93,977.321 1033.16,977.621 1033.38,977.133 1033.61,977.828 1033.84,976.532 1034.07,975.853 1034.3,976.885 1034.52,976.367 1034.75,976.768 1034.98,976.753 1035.21,975.951 1035.43,976.115 1035.66,976.091 1035.89,975.608 1036.12,974.566 1036.35,974.816 1036.57,976.398 1036.8,976.763 1037.03,976.49 1037.26,976.232 1037.48,976.729 1037.71,977.383 1037.94,978.072 1038.17,977.104 1038.4,977.422 1038.62,978.205 1038.85,978.066 1039.08,977.944 1039.31,978.847 1039.53,978.609 1039.76,978.668 1039.99,978.532 1040.22,979.333 1040.44,979.555 1040.67,981.239 1040.9,981.713 1041.13,981.55 1041.36,981.478 1041.58,982.542 1041.81,982.84 1042.04,981.552 1042.27,981.092 1042.49,980.551 1042.72,980.024 1042.95,980.486 1043.18,980.767 1043.41,981.53 1043.63,981.522 1043.86,981.615 1044.09,982.338 1044.32,982.504 1044.54,982.641 1044.77,982.544 1045,982.851 1045.23,982.061 1045.46,980.918 1045.68,981.808 1045.91,981.454 1046.14,981.013 1046.37,981.221 1046.59,982.063 1046.82,980.871 1047.05,980.48 1047.28,981.466 1047.51,981.079 1047.73,980.088 1047.96,980.225 1048.19,981.248 1048.42,980.146 1048.64,980.755 1048.87,980.995 1049.1,979.909 1049.33,979.691 1049.56,979.53 1049.78,979.401 1050.01,980.662 1050.24,980.222 1050.47,979.581 1050.69,979.162 1050.92,977.314 1051.15,978.436 1051.38,978.594 1051.61,979.158 1051.83,979.594 1052.06,979.746 1052.29,979.297 1052.52,979.461 1052.74,977.676 1052.97,976.387 1053.2,976.572 1053.43,976.562 1053.66,976.359 1053.88,976.115 1054.11,976.287 1054.34,975.9 1054.57,976.693 1054.79,977.829 1055.02,977.567 1055.25,977.909 1055.48,977.693 1055.71,977.388 1055.93,978.064 1056.16,978.3 1056.39,977.84 1056.62,977.361 1056.84,978.275 1057.07,978.481 1057.3,978.459 1057.53,977.465 1057.76,977.942 1057.98,979.205 1058.21,980.371 1058.44,979.642 1058.67,979.153 1058.89,979.475 1059.12,979.65 1059.35,980.061 1059.58,979.019 1059.81,978.477 1060.03,977.831 1060.26,978.282 1060.49,978.353 1060.72,978.215 1060.94,978.763 1061.17,979.556 1061.4,977.921 1061.63,978.71 1061.86,978.97 1062.08,978.902 1062.31,979.197 1062.54,979.246 1062.77,979.411 1062.99,981.12 1063.22,980.885 1063.45,981.259 1063.68,981.498 1063.91,981.66 1064.13,981.759 1064.36,982.026 1064.59,981.605 1064.82,981.123 1065.04,980.05 1065.27,979.924 1065.5,980.189 1065.73,979.844 1065.96,979.641 1066.18,979.488 1066.41,979.171 1066.64,976.758 1066.87,977.537 1067.09,977.168 1067.32,977.696 1067.55,976.525 1067.78,977.043 1068.01,977.48 1068.23,977.854 1068.46,977.992 1068.69,978.503 1068.92,977.669 1069.14,977.252 1069.37,977.855 1069.6,978.096 1069.83,979.05 1070.06,979.804 1070.28,980.916 1070.51,981.796 1070.74,981.322 1070.97,982.204 1071.19,983.71 1071.42,984.135 1071.65,984.011 1071.88,984.468 1072.11,984.792 1072.33,985.487 1072.56,985.901 1072.79,987.119 1073.02,987.334 1073.24,987.331 1073.47,987.606 1073.7,987.577 1073.93,987.998 1074.16,989.239 1074.38,988.317 1074.61,988.458 1074.84,988.302 1075.07,987.344 1075.29,988.524 1075.52,988.589 1075.75,989.442 1075.98,990.439 1076.2,990.927 1076.43,991.417 1076.66,989.895 1076.89,990.204 1077.12,991.622 1077.34,991.286 1077.57,990.945 1077.8,990.25 1078.03,991.779 1078.25,991.739 1078.48,991.066 1078.71,990.653 1078.94,991.374 1079.17,991.476 1079.39,990.283 1079.62,989.534 1079.85,987.607 1080.08,987.924 1080.3,987.966 1080.53,987.884 1080.76,987.182 1080.99,986.475 1081.22,986.506 1081.44,985.373 1081.67,985.794 1081.9,986.554 1082.13,987.038 1082.35,987.996 1082.58,987.384 1082.81,987.392 1083.04,987.483 1083.27,986.795 1083.49,986.524 1083.72,986.984 1083.95,987.212 1084.18,986.245 1084.4,985.481 1084.63,986.435 1084.86,985.919 1085.09,985.335 1085.32,984.665 1085.54,984.977 1085.77,983.734 1086,984.538 1086.23,983.586 1086.45,983.924 1086.68,983.174 1086.91,983.212 1087.14,984.217 1087.37,984.673 1087.59,983.994 1087.82,983.825 1088.05,984.802 1088.28,984.664 1088.5,985.339 1088.73,984.59 1088.96,984.578 1089.19,985.113 1089.42,984.931 1089.64,986.718 1089.87,986.178 1090.1,986.743 1090.33,987.233 1090.55,987.253 1090.78,987.249 1091.01,986.855 1091.24,987.166 1091.47,987.062 1091.69,987.481 1091.92,988.207 1092.15,988.822 1092.38,984.889 1092.6,985.634 1092.83,986.214 1093.06,987.278 1093.29,987.159 1093.52,987.859 1093.74,988.624 1093.97,988.07 1094.2,988.338 1094.43,988.727 1094.65,987.082 1094.88,986.028 1095.11,985.405 1095.34,986.48 1095.57,987.251 1095.79,987.112 1096.02,987.192 1096.25,985.313 1096.48,984.872 1096.7,984.754 1096.93,984.009 1097.16,984.139 1097.39,984.924 1097.62,984.876 1097.84,984.567 1098.07,985.12 1098.3,985.91 1098.53,987.064 1098.75,987.019 1098.98,986.976 1099.21,987.642 1099.44,987.372 1099.67,986.782 1099.89,986.048 1100.12,985.408 1100.35,985.732 1100.58,986.37 1100.8,987.182 1101.03,987.244 1101.26,985.649 1101.49,985.113 1101.72,985.136 1101.94,983.973 1102.17,981.933 1102.4,982.157 1102.63,982.585 1102.85,983.118 1103.08,981.672 1103.31,981.987 1103.54,982.806 1103.77,982.304 1103.99,982.865 1104.22,983.358 1104.45,983.114 1104.68,983.879 1104.9,984.234 1105.13,985.459 1105.36,985.357 1105.59,984.473 1105.82,983.954 1106.04,984.239 1106.27,984.312 1106.5,984.189 1106.73,984.148 1106.95,984.531 1107.18,984.933 1107.41,984.608 1107.64,986.052 1107.87,986.77 1108.09,987.114 1108.32,988.062 1108.55,987.292 1108.78,986.641 1109,986.982 1109.23,988.41 1109.46,988.914 1109.69,989.571 1109.92,988.825 1110.14,988.495 1110.37,987.685 1110.6,987.133 1110.83,988.42 1111.05,987.98 1111.28,989.011 1111.51,989.365 1111.74,988.983 1111.97,989.359 1112.19,989.595 1112.42,989.514 1112.65,990.382 1112.88,989.781 1113.1,989.024 1113.33,989.764 1113.56,990.273 1113.79,990.015 1114.01,989.786 1114.24,989.202 1114.47,989.84 1114.7,990.508 1114.93,991.236 1115.15,990.431 1115.38,990.526 1115.61,990.908 1115.84,990.542 1116.06,989.91 1116.29,989.181 1116.52,989.872 1116.75,990.311 1116.98,990.903 1117.2,991.315 1117.43,991.368 1117.66,991.568 1117.89,992.155 1118.11,992.47 1118.34,994.251 1118.57,994.645 1118.8,994.316 1119.03,994.835 1119.25,993.718 1119.48,992.803 1119.71,993.996 1119.94,994.427 1120.16,994.352 1120.39,993.888 1120.62,993.24 1120.85,992.576 1121.08,991.691 1121.3,992.976 1121.53,993.124 1121.76,992.972 1121.99,993.804 1122.21,993.147 1122.44,993.819 1122.67,994.841 1122.9,995.574 1123.13,995.927 1123.35,996.064 1123.58,996.125 1123.81,996.501 1124.04,997.034 1124.26,996.563 1124.49,996.969 1124.72,996.826 1124.95,996.681 1125.18,995.514 1125.4,996.577 1125.63,996.276 1125.86,997.284 1126.09,997.493 1126.31,997.762 1126.54,998.415 1126.77,999.004 1127,998.725 1127.23,999.278 1127.45,998.854 1127.68,999.225 1127.91,1000.03 1128.14,997.812 1128.36,997.341 1128.59,996.485 1128.82,995.827 1129.05,996.479 1129.28,997.928 1129.5,998.45 1129.73,998.454 1129.96,997.486 1130.19,997.236 1130.41,997.948 1130.64,997.19 1130.87,997.366 1131.1,996.676 1131.33,995.333 1131.55,995.799 1131.78,996.246 1132.01,996.66 1132.24,997.108 1132.46,997.522 1132.69,997.981 1132.92,996.241 1133.15,996.971 1133.38,997.231 1133.6,998.802 1133.83,997.129 1134.06,995.838 1134.29,994.827 1134.51,995.145 1134.74,995.99 1134.97,995.089 1135.2,995.917 1135.43,995.113 1135.65,995.217 1135.88,995.932 1136.11,996.495 1136.34,996.087 1136.56,996.226 1136.79,995.91 1137.02,997.176 1137.25,997.557 1137.48,998.973 1137.7,997.893 1137.93,996.789 1138.16,996.23 1138.39,996.201 1138.61,995.266 1138.84,996.249 1139.07,995.577 1139.3,996.003 1139.53,996.892 1139.75,997.298 1139.98,997.035 1140.21,997.511 1140.44,997.754 1140.66,996.447 1140.89,997.14 1141.12,997.712 1141.35,997.653 1141.58,997.5 1141.8,996.232 1142.03,995.947 1142.26,995.164 1142.49,994.142 1142.71,993.976 1142.94,994.156 1143.17,994.164 1143.4,994.688 1143.63,995.221 1143.85,995.107 1144.08,994.114 1144.31,995.123 1144.54,995.508 1144.76,996.338 1144.99,995.691 1145.22,995.458 1145.45,996.228 1145.68,996.525 1145.9,996.614 1146.13,996.706 1146.36,995.561 1146.59,995.469 1146.81,995.19 1147.04,996.093 1147.27,996.202 1147.5,996.989 1147.73,995.401 1147.95,995.401 1148.18,995.951 1148.41,994.875 1148.64,995.064 1148.86,994.403 1149.09,994.256 1149.32,993.23 1149.55,993.242 1149.77,994.433 1150,994.353 1150.23,994.306 1150.46,993.729 1150.69,994.486 1150.91,994.313 1151.14,994.288 1151.37,994.405 1151.6,993.345 1151.82,994.246 1152.05,994.724 1152.28,995.371 1152.51,995.003 1152.74,995.286 1152.96,994.909 1153.19,996.295 1153.42,996.091 1153.65,996.745 1153.87,997.189 1154.1,998.399 1154.33,998.866 1154.56,998.932 1154.79,999.485 1155.01,1000.39 1155.24,1000.44 1155.47,1000.19 1155.7,1000.42 1155.92,1001.18 1156.15,1000.57 1156.38,1001.5 1156.61,1001.79 1156.84,1002.24 1157.06,1002.79 1157.29,1002.06 1157.52,1002.76 1157.75,1003.06 1157.97,1002.88 1158.2,1003.8 1158.43,1003.78 1158.66,1003.75 1158.89,1003.28 1159.11,1002.12 1159.34,1000.61 1159.57,1001.06 1159.8,1000.91 1160.02,1000.49 1160.25,1001.47 1160.48,1002.76 1160.71,1002.28 1160.94,1001.87 1161.16,1002.33 1161.39,1002.66 1161.62,1001.8 1161.85,1001.87 1162.07,1002.46 1162.3,1003.01 1162.53,1002.59 1162.76,1002.4 1162.99,1002.66 1163.21,1001.93 1163.44,1001.57 1163.67,1001.86 1163.9,1000.64 1164.12,1000.46 1164.35,1000.21 1164.58,999.887 1164.81,999.393 1165.04,999.55 1165.26,999.116 1165.49,998.41 1165.72,999.254 1165.95,998.804 1166.17,998.232 1166.4,999.346 1166.63,998.962 1166.86,998.644 1167.09,998.633 1167.31,998.184 1167.54,998.658 1167.77,999.348 1168,999.983 1168.22,1000.05 1168.45,1000.01 1168.68,999.105 1168.91,997.87 1169.14,997.347 1169.36,997.744 1169.59,997.184 1169.82,997.787 1170.05,997.526 1170.27,998.353 1170.5,999.733 1170.73,999.307 1170.96,999.32 1171.19,998.626 1171.41,999.652 1171.64,1000.11 1171.87,999.83 1172.1,1000.41 1172.32,999.526 1172.55,1000.05 1172.78,1000.29 1173.01,998.73 1173.24,997.831 1173.46,996.5 1173.69,996.021 1173.92,996.516 1174.15,995.724 1174.37,996.441 1174.6,996.513 1174.83,996.606 1175.06,997.206 1175.29,997.473 1175.51,996.632 1175.74,996.975 1175.97,997.802 1176.2,995.964 1176.42,995.619 1176.65,996.368 1176.88,997.223 1177.11,996.711 1177.34,996.819 1177.56,996.733 1177.79,996.541 1178.02,997.237 1178.25,996.658 1178.47,996.144 1178.7,996.113 1178.93,997.102 1179.16,996.875 1179.39,995.518 1179.61,994.969 1179.84,994.132 1180.07,994.691 1180.3,995.292 1180.52,993.697 1180.75,994.944 1180.98,995.365 1181.21,995.422 1181.44,995.606 1181.66,996.428 1181.89,996.438 1182.12,995.454 1182.35,995.493 1182.57,995.306 1182.8,995.075 1183.03,993.56 1183.26,993.603 1183.49,994.041 1183.71,994.332 1183.94,995.042 1184.17,996.299 1184.4,996.697 1184.62,996.257 1184.85,996.675 1185.08,997.93 1185.31,997.855 1185.53,998.189 1185.76,999.196 1185.99,999.483 1186.22,999.584 1186.45,999.343 1186.67,999.421 1186.9,997.849 1187.13,996.126 1187.36,997.721 1187.58,996.977 1187.81,997.158 1188.04,998.004 1188.27,997.907 1188.5,997.771 1188.72,998.94 1188.95,1000.04 1189.18,999.715 1189.41,998.59 1189.63,997.781 1189.86,997.418 1190.09,998.858 1190.32,999.258 1190.55,999.862 1190.77,1000.21 1191,1001.05 1191.23,1001.09 1191.46,1000.33 1191.68,999.327 1191.91,997.979 1192.14,997.022 1192.37,997.297 1192.6,996.723 1192.82,997.602 1193.05,997.594 1193.28,997.059 1193.51,997.355 1193.73,996.935 1193.96,996.563 1194.19,997.966 1194.42,998.219 1194.65,999.33 1194.87,998.23 1195.1,999.36 1195.33,998.87 1195.56,1000.39 1195.78,999.931 1196.01,999.876 1196.24,998.338 1196.47,999.034 1196.7,999.513 1196.92,999.597 1197.15,1000.12 1197.38,999.852 1197.61,999.299 1197.83,998.974 1198.06,999.012 1198.29,999.119 1198.52,999.406 1198.75,999.22 1198.97,998.056 1199.2,997.956 1199.43,997.236 1199.66,997.96 1199.88,997.414 1200.11,998.146 1200.34,998.338 1200.57,999.185 1200.8,999.01 1201.02,999.206 1201.25,999.558 1201.48,999.372 1201.71,999.745 1201.93,999.218 1202.16,998.508 1202.39,998.474 1202.62,997.959 1202.85,998.007 1203.07,995.749 1203.3,996.236 1203.53,997.77 1203.76,998.031 1203.98,997.245 1204.21,996.669 1204.44,996.433 1204.67,995.904 1204.9,996.178 1205.12,995.6 1205.35,995.065 1205.58,995.457 1205.81,994.797 1206.03,995.425 1206.26,994.626 1206.49,994.833 1206.72,995.177 1206.95,995.075 1207.17,996.028 1207.4,995.503 1207.63,995.719 1207.86,996.492 1208.08,996.375 1208.31,996.357 1208.54,996.686 1208.77,997.212 1209,997.704 1209.22,997.951 1209.45,996.325 1209.68,996.771 1209.91,996.607 1210.13,997.59 1210.36,997.932 1210.59,997.307 1210.82,997.546 1211.05,997.173 1211.27,997.428 1211.5,998.185 1211.73,998.015 1211.96,997.923 1212.18,996.783 1212.41,996.443 1212.64,995.397 1212.87,995.511 1213.1,996.461 1213.32,996.436 1213.55,996.316 1213.78,995.266 1214.01,995.705 1214.23,995.301 1214.46,994.089 1214.69,993.977 1214.92,994.242 1215.15,993.007 1215.37,993.583 1215.6,993.694 1215.83,994.22 1216.06,993.961 1216.28,993.986 1216.51,994.719 1216.74,995.36 1216.97,995.949 1217.2,995.427 1217.42,995.5 1217.65,995.227 1217.88,995.176 1218.11,995.432 1218.33,995.661 1218.56,995.006 1218.79,994.461 1219.02,993.382 1219.25,993.756 1219.47,993.085 1219.7,993.352 1219.93,992.698 1220.16,992.818 1220.38,993.438 1220.61,993.654 1220.84,993.047 1221.07,993.061 1221.29,992.794 1221.52,992.471 1221.75,993.557 1221.98,993.884 1222.21,993.866 1222.43,993.41 1222.66,992.66 1222.89,992.954 1223.12,993.997 1223.34,994.524 1223.57,995.705 1223.8,996.584 1224.03,996.411 1224.26,996.13 1224.48,996.94 1224.71,997.321 1224.94,997.062 1225.17,997.994 1225.39,997.1 1225.62,997.063 1225.85,997.433 1226.08,997.485 1226.31,998.372 1226.53,998.095 1226.76,998.511 1226.99,998.495 1227.22,999.201 1227.44,1000.18 1227.67,1000.5 1227.9,1001 1228.13,1001.1 1228.36,1001.1 1228.58,1001.55 1228.81,1000.78 1229.04,1000.74 1229.27,1000.48 1229.49,1000.76 1229.72,1001.26 1229.95,1001.71 1230.18,1002 1230.41,1001.86 1230.63,1001.81 1230.86,1002.94 1231.09,1003 1231.32,1003.14 1231.54,1002.61 1231.77,1002.7 1232,1002.76 1232.23,1003.1 1232.46,1002.54 1232.68,1002.28 1232.91,1003.94 1233.14,1004.29 1233.37,1005.07 1233.59,1004.63 1233.82,1005.12 1234.05,1004.16 1234.28,1004.52 1234.51,1004.76 1234.73,1005.07 1234.96,1006.46 1235.19,1007.02 1235.42,1006.29 1235.64,1005.83 1235.87,1005 1236.1,1005.16 1236.33,1006.15 1236.56,1006.2 1236.78,1005.89 1237.01,1006.27 1237.24,1006.01 1237.47,1005.82 1237.69,1005.76 1237.92,1005.4 1238.15,1004.97 1238.38,1003.24 1238.61,1002.98 1238.83,1004.16 1239.06,1003.59 1239.29,1002.72 1239.52,1002.28 1239.74,1003.13 1239.97,1001.89 1240.2,1001.72 1240.43,1001.4 1240.66,1001.11 1240.88,1001.27 1241.11,1001.06 1241.34,1001.54 1241.57,1001.5 1241.79,1001.84 1242.02,1001.37 1242.25,1001.9 1242.48,1000.56 1242.71,1000.72 1242.93,1001.05 1243.16,1001.46 1243.39,1000.75 1243.62,1001.37 1243.84,1002.2 1244.07,1002.15 1244.3,1002.44 1244.53,1002.45 1244.76,1002.71 1244.98,1002.83 1245.21,1002.03 1245.44,1002.89 1245.67,1002.64 1245.89,1002.69 1246.12,1002.38 1246.35,1001.68 1246.58,1000.63 1246.81,1000.8 1247.03,999.49 1247.26,999.25 1247.49,999.596 1247.72,1000.25 1247.94,999.588 1248.17,999.77 1248.4,1000.45 1248.63,1000.81 1248.86,1001.37 1249.08,1000.73 1249.31,1001.41 1249.54,1000.97 1249.77,1001.01 1249.99,1000.15 1250.22,1000.43 1250.45,1000.62 1250.68,1001.49 1250.91,1000.66 1251.13,1000.44 1251.36,1001.58 1251.59,1002.45 1251.82,1003.38 1252.04,1003.44 1252.27,1002.11 1252.5,1001.94 1252.73,1001.96 1252.96,1002.1 1253.18,1000.61 1253.41,1000.14 1253.64,1000.38 1253.87,999.511 1254.09,999.983 1254.32,999.762 1254.55,999.643 1254.78,999.545 1255.01,999.737 1255.23,999.48 1255.46,999.74 1255.69,999.803 1255.92,1000.52 1256.14,1000.61 1256.37,1000.26 1256.6,1000.92 1256.83,1001.75 1257.06,1002.29 1257.28,1002.73 1257.51,1002.01 1257.74,1002.01 1257.97,1002.17 1258.19,1002.32 1258.42,1002.4 1258.65,1002.04 1258.88,1001.58 1259.1,1001.53 1259.33,1001.65 1259.56,1001.87 1259.79,1001.87 1260.02,999.848 1260.24,999.31 1260.47,999.547 1260.7,998.881 1260.93,999.129 1261.15,999.501 1261.38,1000.49 1261.61,999.757 1261.84,998.642 1262.07,998.988 1262.29,999.961 1262.52,1000.29 1262.75,999.877 1262.98,999.629 1263.2,999.592 1263.43,999.781 1263.66,1000.26 1263.89,1000.38 1264.12,1000.5 1264.34,999.817 1264.57,1000 1264.8,1000.21 1265.03,1000.61 1265.25,1001.72 1265.48,1000.96 1265.71,1000.45 1265.94,1002.1 1266.17,1003.43 1266.39,1002.68 1266.62,1002.86 1266.85,1003.3 1267.08,1003.33 1267.3,1003.88 1267.53,1004.79 1267.76,1003.05 1267.99,1003.82 1268.22,1004.4 1268.44,1004.38 1268.67,1004.73 1268.9,1006.13 1269.13,1005.87 1269.35,1006.19 1269.58,1007.36 1269.81,1007.28 1270.04,1007.72 1270.27,1006.67 1270.49,1006.45 1270.72,1005.72 1270.95,1005.59 1271.18,1006.44 1271.4,1006.83 1271.63,1006.97 1271.86,1006.39 1272.09,1005.5 1272.32,1004.18 1272.54,1004.84 1272.77,1005.33 1273,1004.21 1273.23,1004.47 1273.45,1004.96 1273.68,1005.25 1273.91,1005.55 1274.14,1004.21 1274.37,1004.05 1274.59,1004.06 1274.82,1004.66 1275.05,1004.56 1275.28,1005.04 1275.5,1003.92 1275.73,1004 1275.96,1004.6 1276.19,1003.53 1276.42,1003.99 1276.64,1003.88 1276.87,1003.44 1277.1,1004.11 1277.33,1004.66 1277.55,1004.97 1277.78,1004.86 1278.01,1003.94 1278.24,1003.11 1278.47,1003.52 1278.69,1004.58 1278.92,1004.5 1279.15,1005.03 1279.38,1004.83 1279.6,1004.75 1279.83,1003.99 1280.06,1004.78 1280.29,1003.99 1280.52,1003.01 1280.74,1004.05 1280.97,1003.9 1281.2,1002.65 1281.43,1002.94 1281.65,1002.97 1281.88,1002.79 1282.11,1003.45 1282.34,1003.72 1282.57,1004.07 1282.79,1004.8 1283.02,1004.08 1283.25,1004.59 1283.48,1004.3 1283.7,1004.64 1283.93,1005.04 1284.16,1005.26 1284.39,1006.04 1284.62,1006.33 1284.84,1006.31 1285.07,1006.95 1285.3,1007.29 1285.53,1007.92 1285.75,1008.34 1285.98,1008.18 1286.21,1008.72 1286.44,1008.42 1286.67,1008.76 1286.89,1008.9 1287.12,1009.01 1287.35,1009.31 1287.58,1008.96 1287.8,1009.04 1288.03,1008.26 1288.26,1008.29 1288.49,1007.8 1288.72,1008.11 1288.94,1008.09 1289.17,1009.05 1289.4,1008.93 1289.63,1009.26 1289.85,1008.52 1290.08,1008.31 1290.31,1009.36 1290.54,1009.54 1290.77,1009.39 1290.99,1010.14 1291.22,1009.08 1291.45,1009.6 1291.68,1010.26 1291.9,1010.07 1292.13,1010.94 1292.36,1009.93 1292.59,1009.74 1292.82,1009.74 1293.04,1009.69 1293.27,1010.15 1293.5,1009.43 1293.73,1009.68 1293.95,1010.17 1294.18,1010.41 1294.41,1010 1294.64,1010 1294.86,1009.89 1295.09,1009.09 1295.32,1009.67 1295.55,1009 1295.78,1009.62 1296,1009.85 1296.23,1009.63 1296.46,1009.51 1296.69,1007.55 1296.91,1007.08 1297.14,1007.35 1297.37,1007.78 1297.6,1007.37 1297.83,1008.26 1298.05,1006.64 1298.28,1006.64 1298.51,1006.92 1298.74,1007.08 1298.96,1007.46 1299.19,1007.56 1299.42,1007.43 1299.65,1006.56 1299.88,1005.78 1300.1,1005.49 1300.33,1005.57 1300.56,1005.63 1300.79,1005.53 1301.01,1005.34 1301.24,1006.2 1301.47,1005.95 1301.7,1005.58 1301.93,1004.49 1302.15,1004.44 1302.38,1004.66 1302.61,1005.01 1302.84,1006.2 1303.06,1007.3 1303.29,1007.6 1303.52,1007.62 1303.75,1007.26 1303.98,1007.55 1304.2,1008.21 1304.43,1007.84 1304.66,1007.98 1304.89,1009.04 1305.11,1009.89 1305.34,1009.07 1305.57,1008.89 1305.8,1009.31 1306.03,1009.6 1306.25,1008.88 1306.48,1009.49 1306.71,1010.15 1306.94,1010 1307.16,1010 1307.39,1011.01 1307.62,1010.4 1307.85,1009.48 1308.08,1009.22 1308.3,1009.46 1308.53,1008.82 1308.76,1007.54 1308.99,1007.45 1309.21,1008.27 1309.44,1008.23 1309.67,1008.04 1309.9,1008.02 1310.13,1008.55 1310.35,1008.59 1310.58,1009.01 1310.81,1009.74 1311.04,1010.68 1311.26,1011.07 1311.49,1011.6 1311.72,1011.78 1311.95,1012.77 1312.18,1013.5 1312.4,1013.42 1312.63,1013.89 1312.86,1012.69 1313.09,1012.67 1313.31,1012.14 1313.54,1012.79 1313.77,1012.22 1314,1012.26 1314.23,1012.75 1314.45,1011.82 1314.68,1011.36 1314.91,1011.33 1315.14,1011.43 1315.36,1012.05 1315.59,1012.1 1315.82,1011.73 1316.05,1011.7 1316.28,1011.92 1316.5,1011.42 1316.73,1012.11 1316.96,1011.7 1317.19,1012.04 1317.41,1012.7 1317.64,1012.47 1317.87,1012.89 1318.1,1012.88 1318.33,1012.51 1318.55,1011.29 1318.78,1011.25 1319.01,1011.26 1319.24,1010.82 1319.46,1010.76 1319.69,1010.56 1319.92,1009.89 1320.15,1009.51 1320.38,1009.29 1320.6,1009.46 1320.83,1008.95 1321.06,1009.78 1321.29,1009.94 1321.51,1010.4 1321.74,1010.65 1321.97,1010.69 1322.2,1009.77 1322.43,1010.19 1322.65,1009.61 1322.88,1009.01 1323.11,1008.91 1323.34,1008.67 1323.56,1008.18 1323.79,1008.08 1324.02,1009.35 1324.25,1009.22 1324.48,1009.06 1324.7,1009.06 1324.93,1008.42 1325.16,1008.33 1325.39,1008.79 1325.61,1008.29 1325.84,1008.9 1326.07,1008.51 1326.3,1009.15 1326.53,1009.46 1326.75,1009.71 1326.98,1010.26 1327.21,1011.22 1327.44,1011.34 1327.66,1011.53 1327.89,1011.99 1328.12,1011.46 1328.35,1011.38 1328.58,1011.83 1328.8,1012.11 1329.03,1012.28 1329.26,1012.5 1329.49,1013.18 1329.71,1013.67 1329.94,1013.7 1330.17,1015.08 1330.4,1015.55 1330.62,1016.68 1330.85,1016.88 1331.08,1016.23 1331.31,1016.87 1331.54,1016.83 1331.76,1017.14 1331.99,1017.32 1332.22,1017.57 1332.45,1017.31 1332.67,1016.95 1332.9,1016.95 1333.13,1017.66 1333.36,1017.77 1333.59,1018.24 1333.81,1018.95 1334.04,1018.71 1334.27,1019.09 1334.5,1019.07 1334.72,1019.01 1334.95,1019.7 1335.18,1019.4 1335.41,1020.04 1335.64,1019.46 1335.86,1019.82 1336.09,1020.06 1336.32,1019.68 1336.55,1019.71 1336.77,1019.68 1337,1021.14 1337.23,1021.81 1337.46,1021.08 1337.69,1020.92 1337.91,1021.77 1338.14,1021.94 1338.37,1021.5 1338.6,1021.82 1338.82,1021.78 1339.05,1022.24 1339.28,1021.91 1339.51,1023.33 1339.74,1023.09 1339.96,1022.97 1340.19,1022.57 1340.42,1021.75 1340.65,1022.41 1340.87,1021.01 1341.1,1021.5 1341.33,1022.47 1341.56,1023.67 1341.79,1023.76 1342.01,1022.94 1342.24,1023.12 1342.47,1023.56 1342.7,1023.85 1342.92,1022.96 1343.15,1022.55 1343.38,1022.54 1343.61,1023.16 1343.84,1022.98 1344.06,1022.52 1344.29,1023.4 1344.52,1022.76 1344.75,1022.7 1344.97,1023.61 1345.2,1023.55 1345.43,1024.31 1345.66,1024.81 1345.89,1024.79 1346.11,1024.17 1346.34,1024.19 1346.57,1024.8 1346.8,1025.24 1347.02,1026.57 1347.25,1027.13 1347.48,1027.59 1347.71,1027.43 1347.94,1028.59 1348.16,1029.04 1348.39,1029.41 1348.62,1029.59 1348.85,1030.34 1349.07,1030.34 1349.3,1030.99 1349.53,1032.21 1349.76,1032.89 1349.99,1032.64 1350.21,1032.11 1350.44,1032.04 1350.67,1031.71 1350.9,1031.57 1351.12,1031.85 1351.35,1031.31 1351.58,1031.8 1351.81,1032.12 1352.04,1032.35 1352.26,1032.83 1352.49,1032.91 1352.72,1032.62 1352.95,1032.08 1353.17,1031.17 1353.4,1031.1 1353.63,1031.38 1353.86,1032.31 1354.09,1031.98 1354.31,1031.58 1354.54,1031.66 1354.77,1032.5 1355,1031.88 1355.22,1032.03 1355.45,1032.33 1355.68,1032.23 1355.91,1032.93 1356.14,1033.91 1356.36,1032.67 1356.59,1033.26 1356.82,1032.29 1357.05,1031.88 1357.27,1031.4 1357.5,1030.62 1357.73,1030.91 1357.96,1030.74 1358.19,1031.03 1358.41,1031.84 1358.64,1031.04 1358.87,1031.37 1359.1,1031.69 1359.32,1032.24 1359.55,1032.52 1359.78,1031.84 1360.01,1031.38 1360.24,1032.04 1360.46,1032.28 1360.69,1032.45 1360.92,1032.46 1361.15,1032.72 1361.37,1032.02 1361.6,1032.57 1361.83,1033.31 1362.06,1033.89 1362.29,1033.41 1362.51,1033.32 1362.74,1033.54 1362.97,1033.59 1363.2,1032.85 1363.42,1033.44 1363.65,1032.44 1363.88,1032.68 1364.11,1031.89 1364.34,1030.31 1364.56,1030.58 1364.79,1030.47 1365.02,1030.77 1365.25,1030.04 1365.47,1030.17 1365.7,1030.7 1365.93,1030.94 1366.16,1031.27 1366.39,1030.81 1366.61,1030.72 1366.84,1031.31 1367.07,1031.65 1367.3,1031.2 1367.52,1030.95 1367.75,1030.88 1367.98,1030.64 1368.21,1030.68 1368.43,1031.18 1368.66,1031.82 1368.89,1032.64 1369.12,1032.98 1369.35,1032.93 1369.57,1032.87 1369.8,1032.75 1370.03,1032.96 1370.26,1032.75 1370.48,1033.33 1370.71,1032.75 1370.94,1032.37 1371.17,1031.67 1371.4,1032.85 1371.62,1032.74 1371.85,1032.08 1372.08,1030.63 1372.31,1030.77 1372.53,1030.71 1372.76,1030.16 1372.99,1030.19 1373.22,1030.5 1373.45,1030.53 1373.67,1030.72 1373.9,1031.24 1374.13,1032.07 1374.36,1031.91 1374.58,1031.7 1374.81,1032.19 1375.04,1031.63 1375.27,1031.36 1375.5,1031.69 1375.72,1031.87 1375.95,1031.96 1376.18,1032.48 1376.41,1032.44 1376.63,1031.84 1376.86,1031.9 1377.09,1031.35 1377.32,1031.77 1377.55,1030.82 1377.77,1031.14 1378,1031.05 1378.23,1030.28 1378.46,1030.72 1378.68,1030.71 1378.91,1030.1 1379.14,1029.66 1379.37,1029.72 1379.6,1029.17 1379.82,1028.78 1380.05,1028.14 1380.28,1028.07 1380.51,1028.77 1380.73,1028.96 1380.96,1027.86 1381.19,1027.2 1381.42,1026.9 1381.65,1026.26 1381.87,1026.13 1382.1,1026.19 1382.33,1025.33 1382.56,1025.68 1382.78,1026.47 1383.01,1027.34 1383.24,1027.76 1383.47,1028.12 1383.7,1027.8 1383.92,1027.5 1384.15,1027.2 1384.38,1027.44 1384.61,1026.27 1384.83,1026.12 1385.06,1026.06 1385.29,1025.98 1385.52,1025.18 1385.75,1024.39 1385.97,1023.74 1386.2,1024.26 1386.43,1024.35 1386.66,1024.45 1386.88,1023.77 1387.11,1023.16 1387.34,1022.5 1387.57,1023.55 1387.8,1023.23 1388.02,1023.38 1388.25,1023.43 1388.48,1023.84 1388.71,1024.05 1388.93,1023.55 1389.16,1024.1 1389.39,1023.21 1389.62,1022.8 1389.85,1022.58 1390.07,1022.67 1390.3,1022.93 1390.53,1023.03 1390.76,1022.7 1390.98,1023.04 1391.21,1022.82 1391.44,1022.54 1391.67,1022.3 1391.9,1022.21 1392.12,1023.08 1392.35,1022.8 1392.58,1022.51 1392.81,1023.44 1393.03,1023.84 1393.26,1023.66 1393.49,1023.89 1393.72,1024.14 1393.95,1024.33 1394.17,1024.1 1394.4,1023.95 1394.63,1024.36 1394.86,1022.9 1395.08,1023.06 1395.31,1023.45 1395.54,1022.57 1395.77,1022.66 1396,1022.82 1396.22,1022.85 1396.45,1022.48 1396.68,1022.22 1396.91,1022.71 1397.13,1022.3 1397.36,1022.19 1397.59,1022.29 1397.82,1022.66 1398.05,1022.72 1398.27,1023.1 1398.5,1022.19 1398.73,1022.19 1398.96,1021.48 1399.18,1021.78 1399.41,1022.02 1399.64,1022.16 1399.87,1022.65 1400.1,1022.07 1400.32,1022.81 1400.55,1022.63 1400.78,1022.91 1401.01,1023.15 1401.23,1023.44 1401.46,1023.57 1401.69,1024.6 1401.92,1024.76 1402.15,1024.88 1402.37,1025.27 1402.6,1025.42 1402.83,1025.81 1403.06,1026.19 1403.28,1025.97 1403.51,1026.72 1403.74,1025.88 1403.97,1025.31 1404.19,1024.76 1404.42,1025.24 1404.65,1024.95 1404.88,1025.13 1405.11,1024.73 1405.33,1024.92 1405.56,1024.36 1405.79,1024.59 1406.02,1024.39 1406.24,1024.25 1406.47,1024.47 1406.7,1025.38 1406.93,1026.19 1407.16,1025.92 1407.38,1025.68 1407.61,1025.86 1407.84,1025.83 1408.07,1025.72 1408.29,1026.19 1408.52,1026.44 1408.75,1026.41 1408.98,1026.52 1409.21,1026.05 1409.43,1026.73 1409.66,1026.89 1409.89,1027.41 1410.12,1028.16 1410.34,1028.55 1410.57,1028.5 1410.8,1027.53 1411.03,1026.08 1411.26,1025.89 1411.48,1024.85 1411.71,1025.46 1411.94,1024.93 1412.17,1024.94 1412.39,1024.91 1412.62,1024.61 1412.85,1024.4 1413.08,1024.82 1413.31,1025.46 1413.53,1025.28 1413.76,1024.38 1413.99,1024.39 1414.22,1023.77 1414.44,1024.84 1414.67,1024.96 1414.9,1024.76 1415.13,1024.48 1415.36,1025.56 1415.58,1025.66 1415.81,1026.16 1416.04,1025.6 1416.27,1026.02 1416.49,1026.08 1416.72,1026.44 1416.95,1026.91 1417.18,1026.34 1417.41,1026.45 1417.63,1026.53 1417.86,1026.43 1418.09,1026.53 1418.32,1026.23 1418.54,1025.7 1418.77,1025.07 1419,1025.54 1419.23,1026.65 1419.46,1027.65 1419.68,1028 1419.91,1027.4 1420.14,1027.88 1420.37,1027.92 1420.59,1028.51 1420.82,1028.38 1421.05,1028.91 1421.28,1029.55 1421.51,1029.55 1421.73,1028.75 1421.96,1028.16 1422.19,1028.36 1422.42,1027.86 1422.64,1026.9 1422.87,1026.94 1423.1,1028.02 1423.33,1027.93 1423.56,1027.85 1423.78,1027.64 1424.01,1027.68 1424.24,1026.6 1424.47,1026.9 1424.69,1027.85 1424.92,1027.1 1425.15,1027.11 1425.38,1027.82 1425.61,1028.31 1425.83,1028.32 1426.06,1028.79 1426.29,1029.51 1426.52,1029.24 1426.74,1029.07 1426.97,1028.1 1427.2,1027.82 1427.43,1027.97 1427.66,1028.39 1427.88,1028.98 1428.11,1029.53 1428.34,1029.58 1428.57,1030.21 1428.79,1030.55 1429.02,1031.23 1429.25,1031.14 1429.48,1030.92 1429.71,1031.71 1429.93,1031.55 1430.16,1031.2 1430.39,1030.48 1430.62,1030.03 1430.84,1031.14 1431.07,1030.61 1431.3,1031.22 1431.53,1031.62 1431.76,1030.64 1431.98,1029.69 1432.21,1029.31 1432.44,1030.45 1432.67,1030.97 1432.89,1030.59 1433.12,1030.71 1433.35,1030.37 1433.58,1030.2 1433.81,1029.44 1434.03,1029.74 1434.26,1030.52 1434.49,1029.91 1434.72,1029.3 1434.94,1029.62 1435.17,1028.5 1435.4,1028.5 1435.63,1028.66 1435.86,1028.94 1436.08,1029.23 1436.31,1029.3 1436.54,1028.88 1436.77,1029.4 1436.99,1029.97 1437.22,1029.95 1437.45,1029.85 1437.68,1029.45 1437.91,1029.23 1438.13,1028.93 1438.36,1028.49 1438.59,1028.92 1438.82,1029.34 1439.04,1030.26 1439.27,1030.41 1439.5,1029.73 1439.73,1029.67 1439.95,1029.62 1440.18,1030.01 1440.41,1029.82 1440.64,1029.82 1440.87,1030.12 1441.09,1030.18 1441.32,1030.4 1441.55,1030.31 1441.78,1029.75 1442,1030.16 1442.23,1030.19 1442.46,1030.6 1442.69,1030.25 1442.92,1029.94 1443.14,1029.97 1443.37,1030.14 1443.6,1030.32 1443.83,1030.34 1444.05,1030.1 1444.28,1030.26 1444.51,1029.85 1444.74,1029.09 1444.97,1029.67 1445.19,1029.63 1445.42,1029.51 1445.65,1029.53 1445.88,1029.67 1446.1,1029.09 1446.33,1029 1446.56,1028.73 1446.79,1027.47 1447.02,1027.85 1447.24,1028.29 1447.47,1028.04 1447.7,1027.63 1447.93,1028.12 1448.15,1027.93 1448.38,1028.25 1448.61,1029.04 1448.84,1029.77 1449.07,1030.08 1449.29,1030.17 1449.52,1030.14 1449.75,1030.07 1449.98,1030.27 1450.2,1030.58 1450.43,1030.52 1450.66,1029.69 1450.89,1030.62 1451.12,1031.66 1451.34,1031.76 1451.57,1030.77 1451.8,1031.52 1452.03,1031.62 1452.25,1031.43 1452.48,1030.94 1452.71,1029.95 1452.94,1029.2 1453.17,1029.67 1453.39,1030.24 1453.62,1030.05 1453.85,1029.73 1454.08,1028.95 1454.3,1029.14 1454.53,1029.27 1454.76,1029.35 1454.99,1028.67 1455.22,1029.37 1455.44,1029.45 1455.67,1028.48 1455.9,1028.75 1456.13,1029.22 1456.35,1029.05 1456.58,1029.25 1456.81,1029.31 1457.04,1029.44 1457.27,1029.36 1457.49,1028.48 1457.72,1028.76 1457.95,1028.88 1458.18,1029.58 1458.4,1029.79 1458.63,1029.9 1458.86,1029.72 1459.09,1030.04 1459.32,1030.77 1459.54,1030.58 1459.77,1030.29 1460,1030.76 1460.23,1031.32 1460.45,1031.66 1460.68,1031.62 1460.91,1030.81 1461.14,1029.98 1461.37,1029.39 1461.59,1029.25 1461.82,1029.38 1462.05,1029.5 1462.28,1029.57 1462.5,1028.62 1462.73,1028.71 1462.96,1029.07 1463.19,1029.1 1463.42,1028.45 1463.64,1028.82 1463.87,1029.05 1464.1,1028.66 1464.33,1029.06 1464.55,1029.42 1464.78,1029.26 1465.01,1029.65 1465.24,1030.08 1465.47,1030.4 1465.69,1030.08 1465.92,1030.42 1466.15,1030.43 1466.38,1030.71 1466.6,1030.67 1466.83,1031.45 1467.06,1031.61 1467.29,1031.63 1467.52,1031.84 1467.74,1031.18 1467.97,1031.32 1468.2,1030.42 1468.43,1029.85 1468.65,1030.39 1468.88,1029.35 1469.11,1029.69 1469.34,1030.09 1469.57,1030.25 1469.79,1030.54 1470.02,1030.62 1470.25,1030.25 1470.48,1030.5 1470.7,1030.56 1470.93,1029.62 1471.16,1029.7 1471.39,1030.09 1471.62,1029.89 1471.84,1029.87 1472.07,1029.39 1472.3,1029.69 1472.53,1030.41 1472.75,1030.39 1472.98,1031.44 1473.21,1031.48 1473.44,1031.77 1473.67,1030.86 1473.89,1031.18 1474.12,1031.88 1474.35,1032.03 1474.58,1032.1 1474.8,1032.19 1475.03,1031.27 1475.26,1031.41 1475.49,1031.5 1475.71,1031.14 1475.94,1031.21 1476.17,1030.89 1476.4,1030.79 1476.63,1030.97 1476.85,1030.47 1477.08,1030.48 1477.31,1030.1 1477.54,1030.99 1477.76,1030.66 1477.99,1030.28 1478.22,1030.81 1478.45,1030.96 1478.68,1030.89 1478.9,1029.92 1479.13,1030.48 1479.36,1030.99 1479.59,1031.23 1479.81,1031.5 1480.04,1032.41 1480.27,1032.59 1480.5,1033 1480.73,1033 1480.95,1033.93 1481.18,1034.46 1481.41,1033.61 1481.64,1034.06 1481.86,1034.32 1482.09,1035.19 1482.32,1035.57 1482.55,1035.87 1482.78,1036.32 1483,1035.98 1483.23,1036.08 1483.46,1035.37 1483.69,1035 1483.91,1035.32 1484.14,1035.34 1484.37,1035.09 1484.6,1035.06 1484.83,1035.57 1485.05,1035.74 1485.28,1035.41 1485.51,1034.57 1485.74,1035.15 1485.96,1035.06 1486.19,1035.45 1486.42,1035.93 1486.65,1035.94 1486.88,1036.07 1487.1,1036.38 1487.33,1036.47 1487.56,1035.95 1487.79,1036.05 1488.01,1036.77 1488.24,1036.87 1488.47,1036.52 1488.7,1037.32 1488.93,1037.76 1489.15,1037.54 1489.38,1037.89 1489.61,1037.19 1489.84,1037.85 1490.06,1037.8 1490.29,1038 1490.52,1036.99 1490.75,1037.66 1490.98,1038.1 1491.2,1038.64 1491.43,1038.9 1491.66,1038.79 1491.89,1039.09 1492.11,1038.88 1492.34,1038.86 1492.57,1038.59 1492.8,1038.79 1493.03,1039.52 1493.25,1039.35 1493.48,1039.2 1493.71,1038.67 1493.94,1037.85 1494.16,1037.41 1494.39,1036.87 1494.62,1036.64 1494.85,1037.04 1495.08,1037.23 1495.3,1037.71 1495.53,1037.7 1495.76,1037.57 1495.99,1037.13 1496.21,1037.35 1496.44,1037 1496.67,1037.19 1496.9,1037.57 1497.13,1037.42 1497.35,1037.05 1497.58,1037.56 1497.81,1038.1 1498.04,1038.34 1498.26,1038.6 1498.49,1038.99 1498.72,1038.4 1498.95,1038.54 1499.18,1038.92 1499.4,1039.1 1499.63,1038.68 1499.86,1038.73 1500.09,1039.42 1500.31,1039.11 1500.54,1038.66 1500.77,1039.16 1501,1039.27 1501.23,1040 1501.45,1039.65 1501.68,1039.88 1501.91,1039.6 1502.14,1039.53 1502.36,1039.59 1502.59,1039 1502.82,1038.99 1503.05,1039.54 1503.28,1038.37 1503.5,1037.55 1503.73,1037.03 1503.96,1036.95 1504.19,1036.7 1504.41,1037.34 1504.64,1037.25 1504.87,1037.23 1505.1,1036.94 1505.33,1035.9 1505.55,1036.34 1505.78,1036.6 1506.01,1036.42 1506.24,1037.01 1506.46,1037.58 1506.69,1037.74 1506.92,1037.42 1507.15,1036.95 1507.38,1037.6 1507.6,1038.52 1507.83,1038.43 1508.06,1038.94 1508.29,1039.38 1508.51,1039.88 1508.74,1040.14 1508.97,1040.38 1509.2,1040.88 1509.43,1041.73 1509.65,1041.62 1509.88,1040.77 1510.11,1040.62 1510.34,1040.19 1510.56,1040.49 1510.79,1040.6 1511.02,1041.27 1511.25,1040.76 1511.48,1040.58 1511.7,1040.65 1511.93,1040.14 1512.16,1040.46 1512.39,1040.82 1512.61,1041.28 1512.84,1042.05 1513.07,1042.46 1513.3,1042.34 1513.52,1042.96 1513.75,1042.68 1513.98,1041.5 1514.21,1040.99 1514.44,1041.02 1514.66,1041.43 1514.89,1041.52 1515.12,1041.83 1515.35,1041.62 1515.57,1041.73 1515.8,1042.23 1516.03,1042.16 1516.26,1042.15 1516.49,1042.74 1516.71,1043.36 1516.94,1043.47 1517.17,1043.17 1517.4,1042.83 1517.62,1042.56 1517.85,1041.73 1518.08,1041.43 1518.31,1041.55 1518.54,1041.07 1518.76,1041.39 1518.99,1041.51 1519.22,1042.07 1519.45,1042.68 1519.67,1042.57 1519.9,1042.14 1520.13,1042.02 1520.36,1042.29 1520.59,1042.65 1520.81,1042.02 1521.04,1042.1 1521.27,1042.25 1521.5,1043.14 1521.72,1042.76 1521.95,1043.36 1522.18,1043.7 1522.41,1042.7 1522.64,1042.91 1522.86,1042.74 1523.09,1042.88 1523.32,1042.74 1523.55,1042.85 1523.77,1042.21 1524,1041.74 1524.23,1041.89 1524.46,1042.11 1524.69,1041.73 1524.91,1041.87 1525.14,1041.9 1525.37,1041.85 1525.6,1041.59 1525.82,1041.22 1526.05,1040.89 1526.28,1040.65 1526.51,1041.3 1526.74,1041.13 1526.96,1041.94 1527.19,1042.62 1527.42,1042.57 1527.65,1042.15 1527.87,1042.02 1528.1,1040.83 1528.33,1040 1528.56,1039.97 1528.79,1039.99 1529.01,1039.76 1529.24,1039.6 1529.47,1039.57 1529.7,1040.11 1529.92,1040.19 1530.15,1040 1530.38,1040.37 1530.61,1041.05 1530.84,1041.55 1531.06,1042.08 1531.29,1041.5 1531.52,1040.81 1531.75,1041.21 1531.97,1041.67 1532.2,1041.91 1532.43,1041.56 1532.66,1042.13 1532.89,1043.13 1533.11,1042.84 1533.34,1043.08 1533.57,1043.2 1533.8,1043.26 1534.02,1043.3 1534.25,1043.81 1534.48,1044.32 1534.71,1044.03 1534.94,1043.65 1535.16,1043.94 1535.39,1044.6 1535.62,1045.02 1535.85,1045.12 1536.07,1045.38 1536.3,1045.7 1536.53,1045.72 1536.76,1046.07 1536.99,1046.66 1537.21,1047.36 1537.44,1047.65 1537.67,1048.42 1537.9,1049.04 1538.12,1049.23 1538.35,1047.72 1538.58,1047.73 1538.81,1047.43 1539.04,1047.31 1539.26,1047.31 1539.49,1047.52 1539.72,1047.81 1539.95,1048.01 1540.17,1047.68 1540.4,1048.12 1540.63,1047.53 1540.86,1047.4 1541.09,1046.61 1541.31,1046.73 1541.54,1046.05 1541.77,1046.24 1542,1046.71 1542.22,1047.13 1542.45,1046.89 1542.68,1047.18 1542.91,1047.04 1543.14,1047.17 1543.36,1047.41 1543.59,1047.26 1543.82,1045.55 1544.05,1045.89 1544.27,1045.97 1544.5,1046.3 1544.73,1046.74 1544.96,1047.61 1545.19,1047.74 1545.41,1047.47 1545.64,1046.92 1545.87,1046.34 1546.1,1046.26 1546.32,1046.65 1546.55,1047.11 1546.78,1045.92 1547.01,1046.17 1547.24,1045.09 1547.46,1044.94 1547.69,1045.11 1547.92,1045.7 1548.15,1046.07 1548.37,1045.65 1548.6,1045.59 1548.83,1045.49 1549.06,1045.48 1549.28,1045.53 1549.51,1044.96 1549.74,1045.78 1549.97,1046.79 1550.2,1046.06 1550.42,1046.37 1550.65,1046.34 1550.88,1046.43 1551.11,1046.95 1551.33,1046.09 1551.56,1045.43 1551.79,1045.58 1552.02,1045.65 1552.25,1045.41 1552.47,1045.52 1552.7,1045.56 1552.93,1046.12 1553.16,1045.88 1553.38,1045.53 1553.61,1045.47 1553.84,1045.79 1554.07,1044.85 1554.3,1044.92 1554.52,1044.11 1554.75,1043.77 1554.98,1043.3 1555.21,1043.52 1555.43,1044.18 1555.66,1044.29 1555.89,1044.5 1556.12,1044.85 1556.35,1044.85 1556.57,1045.66 1556.8,1045.5 1557.03,1044.92 1557.26,1045.17 1557.48,1045.17 1557.71,1045.43 1557.94,1044.64 1558.17,1043.32 1558.4,1042.79 1558.62,1043.13 1558.85,1042.61 1559.08,1043.26 1559.31,1043.87 1559.53,1043.54 1559.76,1044.06 1559.99,1044.26 1560.22,1043.87 1560.45,1044.41 1560.67,1044.07 1560.9,1043.82 1561.13,1044.21 1561.36,1044.29 1561.58,1044.31 1561.81,1044.39 1562.04,1044.01 1562.27,1044.41 1562.5,1044.19 1562.72,1043.4 1562.95,1042.95 1563.18,1043.14 1563.41,1043.21 1563.63,1042.79 1563.86,1042.53 1564.09,1041.74 1564.32,1041.67 1564.55,1042.11 1564.77,1041.98 1565,1041.34 1565.23,1041.91 1565.46,1042 1565.68,1041.86 1565.91,1041.84 1566.14,1041.67 1566.37,1042.29 1566.6,1042.36 1566.82,1042.2 1567.05,1042.03 1567.28,1042.17 1567.51,1042.55 1567.73,1042.12 1567.96,1042.29 1568.19,1041.79 1568.42,1041.83 1568.65,1041.23 1568.87,1041.3 1569.1,1041.14 1569.33,1040.76 1569.56,1041.43 1569.78,1041.71 1570.01,1041.61 1570.24,1041.48 1570.47,1041.2 1570.7,1041.21 1570.92,1041.51 1571.15,1041.37 1571.38,1041.6 1571.61,1042.14 1571.83,1042.49 1572.06,1042.85 1572.29,1042.98 1572.52,1043.25 1572.75,1043.09 1572.97,1043.31 1573.2,1043.6 1573.43,1043.63 1573.66,1043.62 1573.88,1043.24 1574.11,1042.94 1574.34,1042.2 1574.57,1041.63 1574.8,1041.01 1575.02,1040.73 1575.25,1040.5 1575.48,1039.88 1575.71,1039.73 1575.93,1038.9 1576.16,1038.6 1576.39,1038.53 1576.62,1038.35 1576.85,1038.55 1577.07,1038.67 1577.3,1038.2 1577.53,1038.65 1577.76,1038.22 1577.98,1038.54 1578.21,1038.69 1578.44,1038.78 1578.67,1039.06 1578.9,1039.43 1579.12,1039.36 1579.35,1038.82 1579.58,1039.09 1579.81,1038.95 1580.03,1039.3 1580.26,1038.96 1580.49,1038.22 1580.72,1038.08 1580.95,1038.13 1581.17,1037.67 1581.4,1036.91 1581.63,1037.22 1581.86,1035.99 1582.08,1035.66 1582.31,1035.77 1582.54,1035.95 1582.77,1035.99 1583,1036.51 1583.22,1036.68 1583.45,1036.45 1583.68,1036.22 1583.91,1036.52 1584.13,1036.46 1584.36,1035.59 1584.59,1036.42 1584.82,1036.88 1585.04,1037.07 1585.27,1037.3 1585.5,1036.38 1585.73,1036.99 1585.96,1036.75 1586.18,1037.25 1586.41,1037.25 1586.64,1037.5 1586.87,1037.57 1587.09,1037.67 1587.32,1037.21 1587.55,1037.5 1587.78,1037.81 1588.01,1037.95 1588.23,1037.61 1588.46,1036.99 1588.69,1037.25 1588.92,1036.57 1589.14,1036.27 1589.37,1036.7 1589.6,1037.17 1589.83,1036.42 1590.06,1036.31 1590.28,1036.66 1590.51,1036.07 1590.74,1035.22 1590.97,1035.23 1591.19,1035.27 1591.42,1035.19 1591.65,1035.36 1591.88,1035.11 1592.11,1035.16 1592.33,1034.69 1592.56,1034.94 1592.79,1034.98 1593.02,1035.05 1593.24,1035.38 1593.47,1035.13 1593.7,1035.68 1593.93,1036.07 1594.16,1035.91 1594.38,1036.75 1594.61,1036.56 1594.84,1036.83 1595.07,1037.01 1595.29,1037.07 1595.52,1037.19 1595.75,1036.6 1595.98,1036.19 1596.21,1036.32 1596.43,1037.17 1596.66,1037.32 1596.89,1037.73 1597.12,1038.49 1597.34,1038.47 1597.57,1038.16 1597.8,1037.75 1598.03,1037.9 1598.26,1037.2 1598.48,1036.32 1598.71,1036.39 1598.94,1035.57 1599.17,1035.59 1599.39,1036.47 1599.62,1035.92 1599.85,1035.25 1600.08,1035.08 1600.31,1035.59 1600.53,1035.66 1600.76,1035.96 1600.99,1035.99 1601.22,1036.28 1601.44,1035.97 1601.67,1035.58 1601.9,1035.57 1602.13,1035.72 1602.36,1034.21 1602.58,1033.8 1602.81,1034.74 1603.04,1035.17 1603.27,1034.73 1603.49,1034.88 1603.72,1034.95 1603.95,1035.25 1604.18,1035.29 1604.41,1035.68 1604.63,1036.09 1604.86,1036.63 1605.09,1035.53 1605.32,1035.95 1605.54,1036.1 1605.77,1036.22 1606,1036.31 1606.23,1036.25 1606.46,1036.28 1606.68,1036.27 1606.91,1036.86 1607.14,1036.81 1607.37,1036.61 1607.59,1036.11 1607.82,1036.34 1608.05,1036.96 1608.28,1037 1608.51,1036.74 1608.73,1037.01 1608.96,1037.03 1609.19,1036.9 1609.42,1036.67 1609.64,1037.27 1609.87,1036.69 1610.1,1037.26 1610.33,1037.81 1610.56,1037.93 1610.78,1037.05 1611.01,1036.41 1611.24,1036.37 1611.47,1035.86 1611.69,1035.12 1611.92,1034.94 1612.15,1035.1 1612.38,1035.06 1612.61,1035.51 1612.83,1035.6 1613.06,1035.29 1613.29,1035.12 1613.52,1035.99 1613.74,1036.42 1613.97,1035.79 1614.2,1035.63 1614.43,1035.12 1614.66,1034.78 1614.88,1034.45 1615.11,1034.78 1615.34,1034.88 1615.57,1034.07 1615.79,1033.81 1616.02,1033.29 1616.25,1033.24 1616.48,1034.02 1616.71,1032.87 1616.93,1032.7 1617.16,1032.36 1617.39,1032.42 1617.62,1032.12 1617.84,1031.82 1618.07,1032.17 1618.3,1032.23 1618.53,1032.38 1618.76,1032.9 1618.98,1032.31 1619.21,1031.51 1619.44,1031.77 1619.67,1032.18 1619.89,1031.84 1620.12,1031.82 1620.35,1032.3 1620.58,1032.82 1620.81,1032.68 1621.03,1032.49 1621.26,1032.26 1621.49,1032.62 1621.72,1032.44 1621.94,1032.41 1622.17,1032.3 1622.4,1033.33 1622.63,1033.37 1622.85,1033.89 1623.08,1032.97 1623.31,1033.07 1623.54,1033.17 1623.77,1033.53 1623.99,1033.51 1624.22,1033.54 1624.45,1033.51 1624.68,1033.58 1624.9,1033.9 1625.13,1034.38 1625.36,1034.56 1625.59,1034.17 1625.82,1033.87 1626.04,1034.07 1626.27,1033.95 1626.5,1033.95 1626.73,1033.65 1626.95,1032.81 1627.18,1032.91 1627.41,1033.04 1627.64,1032.94 1627.87,1032.78 1628.09,1032.34 1628.32,1032.92 1628.55,1032.89 1628.78,1033.6 1629,1033.38 1629.23,1033.97 1629.46,1032.86 1629.69,1033.09 1629.92,1032.37 1630.14,1032.03 1630.37,1031.83 1630.6,1032.23 1630.83,1031.71 1631.05,1031.75 1631.28,1031.27 1631.51,1031.17 1631.74,1031.08 1631.97,1030.99 1632.19,1031.23 1632.42,1031.38 1632.65,1031.78 1632.88,1032.26 1633.1,1032.59 1633.33,1032.77 1633.56,1031.75 1633.79,1031.36 1634.02,1030.62 1634.24,1030.81 1634.47,1030.76 1634.7,1030.57 1634.93,1030.51 1635.15,1030.58 1635.38,1030.22 1635.61,1030.65 1635.84,1030.2 1636.07,1030.45 1636.29,1030.99 1636.52,1030.82 1636.75,1031.14 1636.98,1031.16 1637.2,1030.18 1637.43,1030.53 1637.66,1030.6 1637.89,1030.65 1638.12,1031.08 1638.34,1030.99 1638.57,1031.06 1638.8,1030.96 1639.03,1030.89 1639.25,1029.89 1639.48,1030.6 1639.71,1030.91 1639.94,1030.19 1640.17,1029.09 1640.39,1028.74 1640.62,1028.43 1640.85,1028.07 1641.08,1028.41 1641.3,1028.52 1641.53,1028.9 1641.76,1028.57 1641.99,1028.61 1642.22,1028.89 1642.44,1028.62 1642.67,1028.78 1642.9,1029.16 1643.13,1029.35 1643.35,1028.64 1643.58,1028.77 1643.81,1027.95 1644.04,1028.31 1644.27,1028.93 1644.49,1028.45 1644.72,1028.29 1644.95,1027.01 1645.18,1026.7 1645.4,1027.05 1645.63,1027.84 1645.86,1028.36 1646.09,1027.94 1646.32,1028.02 1646.54,1028.51 1646.77,1027.68 1647,1028.31 1647.23,1028.44 1647.45,1027.1 1647.68,1026.77 1647.91,1027.17 1648.14,1027.61 1648.37,1027.73 1648.59,1027.78 1648.82,1028.2 1649.05,1028.46 1649.28,1028.53 1649.5,1028.66 1649.73,1028 1649.96,1028.15 1650.19,1028.15 1650.42,1028.15 1650.64,1028.68 1650.87,1029.21 1651.1,1029.26 1651.33,1028.45 1651.55,1029.18 1651.78,1029.22 1652.01,1029.69 1652.24,1030.01 1652.47,1030.12 1652.69,1029.98 1652.92,1030.3 1653.15,1030.66 1653.38,1030.96 1653.6,1030.99 1653.83,1031.43 1654.06,1031.36 1654.29,1031.91 1654.52,1032.09 1654.74,1031.6 1654.97,1031.4 1655.2,1031.32 1655.43,1031.52 1655.65,1032.09 1655.88,1032.66 1656.11,1033.02 1656.34,1032.71 1656.57,1033.05 1656.79,1033.03 1657.02,1033.21 1657.25,1033.01 1657.48,1033.41 1657.7,1033.79 1657.93,1034.1 1658.16,1034.13 1658.39,1034.32 1658.61,1033.79 1658.84,1034.01 1659.07,1034.55 1659.3,1034.96 1659.53,1035.35 1659.75,1036.01 1659.98,1036.49 1660.21,1036.13 1660.44,1035.64 1660.66,1035.22 1660.89,1035.81 1661.12,1036.23 1661.35,1036.59 1661.58,1036.73 1661.8,1037.07 1662.03,1037.25 1662.26,1036.77 1662.49,1037.01 1662.71,1037.28 1662.94,1036.27 1663.17,1036.37 1663.4,1036.85 1663.63,1036.61 1663.85,1037.1 1664.08,1036.2 1664.31,1035.91 1664.54,1035.98 1664.76,1036.75 1664.99,1037.2 1665.22,1037.31 1665.45,1037.94 1665.68,1037.76 1665.9,1037.72 1666.13,1037.44 1666.36,1037.5 1666.59,1037.59 1666.81,1038.43 1667.04,1037.93 1667.27,1037.67 1667.5,1037.9 1667.73,1037.98 1667.95,1038.17 1668.18,1037.09 1668.41,1037.81 1668.64,1037.25 1668.86,1037.82 1669.09,1037.61 1669.32,1037.94 1669.55,1038.39 1669.78,1039.07 1670,1039.13 1670.23,1039.75 1670.46,1040 1670.69,1039.56 1670.91,1040.04 1671.14,1040.13 1671.37,1040.12 1671.6,1039.61 1671.83,1040.21 1672.05,1040.52 1672.28,1040.59 1672.51,1040.19 1672.74,1039.98 1672.96,1040.26 1673.19,1039.39 1673.42,1040.15 1673.65,1040.22 1673.88,1039.81 1674.1,1039.8 1674.33,1040.01 1674.56,1039.91 1674.79,1039.58 1675.01,1040.18 1675.24,1039.36 1675.47,1039.27 1675.7,1038.91 1675.93,1039.29 1676.15,1039.52 1676.38,1039.98 1676.61,1039.05 1676.84,1039.6 1677.06,1039.66 1677.29,1040.21 1677.52,1039.74 1677.75,1039.84 1677.98,1039.81 1678.2,1038.82 1678.43,1038.82 1678.66,1039.23 1678.89,1039.29 1679.11,1039.54 1679.34,1039.37 1679.57,1038.64 1679.8,1039.15 1680.03,1039.06 1680.25,1038.77 1680.48,1038.7 1680.71,1038.46 1680.94,1038.49 1681.16,1038.37 1681.39,1039.02 1681.62,1039.28 1681.85,1039.53 1682.08,1039.83 1682.3,1038.25 1682.53,1038.25 1682.76,1038.62 1682.99,1039 1683.21,1038.53 1683.44,1038.62 1683.67,1037.34 1683.9,1037.5 1684.13,1038 1684.35,1037.67 1684.58,1037.3 1684.81,1037.48 1685.04,1037.69 1685.26,1037.72 1685.49,1038.2 1685.72,1038.04 1685.95,1038.27 1686.18,1038.77 1686.4,1038.03 1686.63,1038.61 1686.86,1038.61 1687.09,1037.72 1687.31,1037.46 1687.54,1037.35 1687.77,1037.37 1688,1037.52 1688.23,1038.02 1688.45,1036.87 1688.68,1036.46 1688.91,1037.02 1689.14,1036.03 1689.36,1035.7 1689.59,1035.4 1689.82,1034.35 1690.05,1035.14 1690.28,1034.82 1690.5,1034.21 1690.73,1034.46 1690.96,1034.72 1691.19,1034.62 1691.41,1035.02 1691.64,1035 1691.87,1034.93 1692.1,1035.31 1692.33,1035.4 1692.55,1035.61 1692.78,1035.75 1693.01,1035.47 1693.24,1035.57 1693.46,1035.47 1693.69,1035.09 1693.92,1035.29 1694.15,1035.54 1694.37,1035.79 1694.6,1036.31 1694.83,1035.11 1695.06,1035.19 1695.29,1035.47 1695.51,1035.42 1695.74,1035.13 1695.97,1035.11 1696.2,1035.6 1696.42,1035.27 1696.65,1035.19 1696.88,1034.75 1697.11,1035.01 1697.34,1034.52 1697.56,1034.17 1697.79,1034.02 1698.02,1034.18 1698.25,1033.95 1698.47,1034.11 1698.7,1034.15 1698.93,1034.1 1699.16,1034.43 1699.39,1034.79 1699.61,1034.47 1699.84,1034.01 1700.07,1033.93 1700.3,1034.14 1700.52,1033.68 1700.75,1033.99 1700.98,1033.72 1701.21,1033.8 1701.44,1033.74 1701.66,1034.17 1701.89,1034.07 1702.12,1033.84 1702.35,1033.63 1702.57,1033.44 1702.8,1033.75 1703.03,1033.47 1703.26,1033.07 1703.49,1034.05 1703.71,1034.28 1703.94,1034.67 1704.17,1034.88 1704.4,1034.55 1704.62,1033.29 1704.85,1033.5 1705.08,1034.24 1705.31,1034.65 1705.54,1034.97 1705.76,1034.5 1705.99,1034.31 1706.22,1034.52 1706.45,1034.84 1706.67,1035.46 1706.9,1035.43 1707.13,1034.79 1707.36,1034.56 1707.59,1034.33 1707.81,1034.39 1708.04,1034.5 1708.27,1034.62 1708.5,1035.1 1708.72,1035.18 1708.95,1034.92 1709.18,1035.94 1709.41,1035.7 1709.64,1035.72 1709.86,1035.37 1710.09,1035.26 1710.32,1035.1 1710.55,1035.08 1710.77,1035.05 1711,1034.55 1711.23,1034.35 1711.46,1034.29 1711.69,1034.66 1711.91,1034.52 1712.14,1034.84 1712.37,1034.65 1712.6,1033.5 1712.82,1033.53 1713.05,1034.42 1713.28,1035.08 1713.51,1034.59 1713.74,1034.7 1713.96,1034.07 1714.19,1034.39 1714.42,1034.48 1714.65,1034.17 1714.87,1033.98 1715.1,1034.29 1715.33,1033.9 1715.56,1033.68 1715.79,1033.95 1716.01,1033.95 1716.24,1034.37 1716.47,1034.43 1716.7,1034.36 1716.92,1034.14 1717.15,1034.21 1717.38,1033.76 1717.61,1033.87 1717.84,1033.56 1718.06,1033.42 1718.29,1033.23 1718.52,1033.81 1718.75,1032.8 1718.97,1033.34 1719.2,1032.82 1719.43,1033.17 1719.66,1033.81 1719.89,1034.11 1720.11,1034.14 1720.34,1034.91 1720.57,1034.25 1720.8,1033.88 1721.02,1033.52 1721.25,1033.72 1721.48,1033.75 1721.71,1033.13 1721.94,1033.43 1722.16,1033.91 1722.39,1034.32 1722.62,1034.16 1722.85,1033.73 1723.07,1033.92 1723.3,1033.9 1723.53,1033.77 1723.76,1033.45 1723.99,1033.27 1724.21,1032.89 1724.44,1032.87 1724.67,1032.64 1724.9,1032.57 1725.12,1032.36 1725.35,1032.74 1725.58,1032.78 1725.81,1033.09 1726.04,1032.93 1726.26,1033.14 1726.49,1033.61 1726.72,1033.36 1726.95,1032.06 1727.17,1032.35 1727.4,1032.31 1727.63,1032.56 1727.86,1032.45 1728.09,1032.95 1728.31,1032.85 1728.54,1033.31 1728.77,1033.14 1729,1033.47 1729.22,1033.57 1729.45,1033.32 1729.68,1033.14 1729.91,1032.28 1730.14,1032.43 1730.36,1032.05 1730.59,1031.56 1730.82,1031.33 1731.05,1031.74 1731.27,1031.82 1731.5,1031.88 1731.73,1031.71 1731.96,1031.95 1732.18,1032.43 1732.41,1032.25 1732.64,1032.06 1732.87,1032.08 1733.1,1032.91 1733.32,1033.09 1733.55,1033.59 1733.78,1033.89 1734.01,1033.9 1734.23,1033.09 1734.46,1033 1734.69,1032.93 1734.92,1033.06 1735.15,1033.27 1735.37,1033.18 1735.6,1033.08 1735.83,1032.14 1736.06,1032.23 1736.28,1032.86 1736.51,1033.28 1736.74,1032.84 1736.97,1032.67 1737.2,1032.03 1737.42,1032.62 1737.65,1032.46 1737.88,1032.3 1738.11,1032.99 1738.33,1032.44 1738.56,1032.3 1738.79,1032.5 1739.02,1032.51 1739.25,1032.8 1739.47,1032.94 1739.7,1033.01 1739.93,1033.03 1740.16,1032.56 1740.38,1032.17 1740.61,1032.66 1740.84,1033.38 1741.07,1033.2 1741.3,1033.22 1741.52,1033.47 1741.75,1033.21 1741.98,1032.8 1742.21,1033.22 1742.43,1031.96 1742.66,1031.93 1742.89,1031.63 1743.12,1031.48 1743.35,1031.82 1743.57,1032.08 1743.8,1032.25 1744.03,1032.64 1744.26,1032.12 1744.48,1032.45 1744.71,1031.7 1744.94,1032.16 1745.17,1032.63 1745.4,1033.22 1745.62,1032.87 1745.85,1032.28 1746.08,1032.66 1746.31,1032.81 1746.53,1032.39 1746.76,1032.21 1746.99,1031.98 1747.22,1030.96 1747.45,1031.04 1747.67,1031.6 1747.9,1031.96 1748.13,1032.65 1748.36,1033.21 1748.58,1033.63 1748.81,1033.45 1749.04,1034.02 1749.27,1034.19 1749.5,1034.31 1749.72,1033.45 1749.95,1033.18 1750.18,1032.91 1750.41,1033.3 1750.63,1032.62 1750.86,1032.62 1751.09,1032.79 1751.32,1033.07 1751.55,1032.39 1751.77,1032.69 1752,1032.49 1752.23,1032.84 1752.46,1033.01 1752.68,1033.35 1752.91,1033.26 1753.14,1033.09 1753.37,1033.1 1753.6,1033.25 1753.82,1033.86 1754.05,1034.01 1754.28,1034.06 1754.51,1034.16 1754.73,1033.93 1754.96,1034.45 1755.19,1035.32 1755.42,1036.06 1755.65,1036.09 1755.87,1036.88 1756.1,1036.98 1756.33,1036.19 1756.56,1036.8 1756.78,1036.18 1757.01,1037.08 1757.24,1037.21 1757.47,1036.99 1757.7,1037.44 1757.92,1037.87 1758.15,1037.56 1758.38,1037.38 1758.61,1037.48 1758.83,1037.4 1759.06,1037.65 1759.29,1037.75 1759.52,1036.84 1759.75,1036.89 1759.97,1037.06 1760.2,1036 1760.43,1036.54 1760.66,1037.07 1760.88,1036.64 1761.11,1036.89 1761.34,1037.46 1761.57,1037.51 1761.8,1037.48 1762.02,1037.66 1762.25,1037.41 1762.48,1037.78 1762.71,1037.22 1762.93,1036.45 1763.16,1036.22 1763.39,1036.73 1763.62,1036.56 1763.85,1036.14 1764.07,1035.87 1764.3,1035.88 1764.53,1036.11 1764.76,1035.92 1764.98,1035.53 1765.21,1035.39 1765.44,1035.23 1765.67,1035.37 1765.9,1035.66 1766.12,1036.21 1766.35,1036.44 1766.58,1035.84 1766.81,1036.34 1767.03,1036.64 1767.26,1036.34 1767.49,1036.51 1767.72,1036.8 1767.94,1036.75 1768.17,1036.72 1768.4,1036.97 1768.63,1037.14 1768.86,1037.52 1769.08,1037 1769.31,1037.59 1769.54,1037.61 1769.77,1036.96 1769.99,1037.32 1770.22,1037.78 1770.45,1037.98 1770.68,1038.54 1770.91,1039.08 1771.13,1039.63 1771.36,1040.01 1771.59,1039.94 1771.82,1039.26 1772.04,1039.38 1772.27,1039.36 1772.5,1039.8 1772.73,1039.7 1772.96,1040.16 1773.18,1039.82 1773.41,1039.67 1773.64,1040.15 1773.87,1040.33 1774.09,1040.52 1774.32,1040.36 1774.55,1039.79 1774.78,1039.55 1775.01,1039.74 1775.23,1039.94 1775.46,1039.8 1775.69,1039.97 1775.92,1039.6 1776.14,1039.61 1776.37,1039.28 1776.6,1039.18 1776.83,1039.76 1777.06,1039.81 1777.28,1040.09 1777.51,1040.67 1777.74,1040.07 1777.97,1039.82 1778.19,1040.09 1778.42,1040.19 1778.65,1040.56 1778.88,1040.55 1779.11,1040.3 1779.33,1040.22 1779.56,1039.19 1779.79,1039.35 1780.02,1039.5 1780.24,1039.25 1780.47,1039.71 1780.7,1039.57 1780.93,1040.35 1781.16,1039.21 1781.38,1039.75 1781.61,1039.67 1781.84,1040.07 1782.07,1040.71 1782.29,1040.86 1782.52,1041.08 1782.75,1040.2 1782.98,1040.89 1783.21,1041.01 1783.43,1041.18 1783.66,1040.92 1783.89,1041.18 1784.12,1040.77 1784.34,1040.62 1784.57,1039.94 1784.8,1039.85 1785.03,1038.97 1785.26,1039.29 1785.48,1039.28 1785.71,1039.14 1785.94,1038.5 1786.17,1038.45 1786.39,1038.52 1786.62,1038.58 1786.85,1038.07 1787.08,1038.13 1787.31,1038.14 1787.53,1037.9 1787.76,1037.79 1787.99,1036.93 1788.22,1036.43 1788.44,1036.5 1788.67,1036.42 1788.9,1036.43 1789.13,1036.31 1789.36,1035.92 1789.58,1035.76 1789.81,1035.42 1790.04,1035.09 1790.27,1035.31 1790.49,1035.66 1790.72,1035.27 1790.95,1035.32 1791.18,1034.5 1791.41,1034.98 1791.63,1034.78 1791.86,1034.75 1792.09,1034.71 1792.32,1035.21 1792.54,1034.96 1792.77,1034.94 1793,1035.1 1793.23,1036.02 1793.46,1035.94 1793.68,1035.64 1793.91,1035.93 1794.14,1035.64 1794.37,1035.95 1794.59,1035.78 1794.82,1035.72 1795.05,1035.96 1795.28,1035.61 1795.51,1035.74 1795.73,1035.28 1795.96,1035.4 1796.19,1035.25 1796.42,1035.74 1796.64,1035.67 1796.87,1036.11 1797.1,1036.32 1797.33,1036.6 1797.56,1036.6 1797.78,1036.22 1798.01,1036.34 1798.24,1036.14 1798.47,1035.5 1798.69,1035.85 1798.92,1035.74 1799.15,1035.84 1799.38,1036.13 1799.61,1035.64 1799.83,1035.63 1800.06,1036.31 1800.29,1036.04 1800.52,1035.78 1800.74,1035.82 1800.97,1036.05 1801.2,1035.23 1801.43,1035.54 1801.66,1036.26 1801.88,1036.73 1802.11,1036.14 1802.34,1036.45 1802.57,1036.39 1802.79,1036.51 1803.02,1037.11 1803.25,1036.52 1803.48,1036.25 1803.7,1036.71 1803.93,1036.73 1804.16,1036.33 1804.39,1036.64 1804.62,1036.81 1804.84,1036.66 1805.07,1036.14 1805.3,1036.37 1805.53,1036.66 1805.75,1036.3 1805.98,1036.4 1806.21,1037.01 1806.44,1036.85 1806.67,1036.95 1806.89,1036.34 1807.12,1036.9 1807.35,1036.88 1807.58,1037.19 1807.8,1037.44 1808.03,1037.08 1808.26,1036.81 1808.49,1036.76 1808.72,1037.31 1808.94,1037.53 1809.17,1037.68 1809.4,1037.83 1809.63,1037.88 1809.85,1038.33 1810.08,1038.67 1810.31,1038.92 1810.54,1039.06 1810.77,1038.95 1810.99,1039.02 1811.22,1038.71 1811.45,1038.95 1811.68,1039.26 1811.9,1039.2 1812.13,1038.86 1812.36,1039.36 1812.59,1039.48 1812.82,1040.01 1813.04,1040.11 1813.27,1039.33 1813.5,1039.79 1813.73,1040.02 1813.95,1039.89 1814.18,1039.82 1814.41,1040.06 1814.64,1040.29 1814.87,1040.77 1815.09,1041.25 1815.32,1041.56 1815.55,1041.49 1815.78,1041.43 1816,1041.99 1816.23,1041.56 1816.46,1041.84 1816.69,1041.14 1816.92,1040.84 1817.14,1040.76 1817.37,1040.89 1817.6,1040.7 1817.83,1041.28 1818.05,1041.22 1818.28,1041.16 1818.51,1040.91 1818.74,1040.27 1818.97,1040.19 1819.19,1040.11 1819.42,1039.54 1819.65,1039.07 1819.88,1038.44 1820.1,1038.47 1820.33,1039.18 1820.56,1039.04 1820.79,1039.25 1821.02,1039.01 1821.24,1039.73 1821.47,1039.66 1821.7,1039.77 1821.93,1039.23 1822.15,1039.16 1822.38,1038.81 1822.61,1039.41 1822.84,1039.48 1823.07,1040.04 1823.29,1040.77 1823.52,1041.15 1823.75,1041.67 1823.98,1041.54 1824.2,1041 1824.43,1041.49 1824.66,1041.3 1824.89,1041.07 1825.12,1040.23 1825.34,1040.54 1825.57,1040.71 1825.8,1040.35 1826.03,1040.33 1826.25,1040.66 1826.48,1040.4 1826.71,1039.91 1826.94,1040.05 1827.17,1039.8 1827.39,1039.4 1827.62,1039.53 1827.85,1038.48 1828.08,1038.53 1828.3,1038.65 1828.53,1038.81 1828.76,1037.96 1828.99,1037.96 1829.22,1036.91 1829.44,1037.22 1829.67,1037.62 1829.9,1037.94 1830.13,1038.06 1830.35,1037.79 1830.58,1037.69 1830.81,1037.34 1831.04,1037.9 1831.27,1038.02 1831.49,1038.77 1831.72,1039.76 1831.95,1040.28 1832.18,1040.62 1832.4,1041.35 1832.63,1041.4 1832.86,1041.91 1833.09,1041.92 1833.32,1041.91 1833.54,1041.76 1833.77,1042.14 1834,1041.87 1834.23,1041.85 1834.45,1041.64 1834.68,1041.73 1834.91,1041.77 1835.14,1042.24 1835.37,1042.13 1835.59,1042.16 1835.82,1042.17 1836.05,1041.88 1836.28,1042 1836.5,1042.59 1836.73,1042.74 1836.96,1042.94 1837.19,1043.11 1837.42,1043.64 1837.64,1043.04 1837.87,1043.37 1838.1,1043.49 1838.33,1043.87 1838.55,1044.22 1838.78,1044.47 1839.01,1044.41 1839.24,1044.99 1839.46,1045.54 1839.69,1045.44 1839.92,1046.27 1840.15,1046.06 1840.38,1046.13 1840.6,1045.83 1840.83,1046.03 1841.06,1045.64 1841.29,1046.15 1841.51,1046.81 1841.74,1046.4 1841.97,1046.75 1842.2,1046.65 1842.43,1046.27 1842.65,1046.11 1842.88,1045.62 1843.11,1045.71 1843.34,1045.92 1843.56,1045.07 1843.79,1045.19 1844.02,1044.97 1844.25,1045.2 1844.48,1045.23 1844.7,1045.36 1844.93,1045.19 1845.16,1045.44 1845.39,1045.34 1845.61,1045.79 1845.84,1046.49 1846.07,1046.6 1846.3,1046.77 1846.53,1046.96 1846.75,1046.32 1846.98,1046.29 1847.21,1046.74 1847.44,1046.65 1847.66,1046.68 1847.89,1047.13 1848.12,1047.21 1848.35,1046.99 1848.58,1047.43 1848.8,1047.02 1849.03,1047.34 1849.26,1047.17 1849.49,1046.99 1849.71,1046.45 1849.94,1046.01 1850.17,1046.33 1850.4,1046.53 1850.63,1045.15 1850.85,1045.4 1851.08,1045.05 1851.31,1044.99 1851.54,1044.78 1851.76,1045.02 1851.99,1045.04 1852.22,1045.11 1852.45,1044.86 1852.68,1044.61 1852.9,1044.72 1853.13,1045.31 1853.36,1045.15 1853.59,1045.24 1853.81,1045.83 1854.04,1046.12 1854.27,1045.92 1854.5,1045.66 1854.73,1045.35 1854.95,1045.1 1855.18,1045.48 1855.41,1044.8 1855.64,1045.21 1855.86,1046.06 1856.09,1046.23 1856.32,1046.2 1856.55,1045.92 1856.78,1045.8 1857,1044.73 1857.23,1044.32 1857.46,1043.77 1857.69,1044.01 1857.91,1043.9 1858.14,1043.5 1858.37,1043.9 1858.6,1044.23 1858.83,1043.57 1859.05,1043.44 1859.28,1043.44 1859.51,1043.38 1859.74,1043.79 1859.96,1043.38 1860.19,1043.75 1860.42,1043.94 1860.65,1044.19 1860.88,1044.78 1861.1,1044.29 1861.33,1044.18 1861.56,1044.53 1861.79,1043.92 1862.01,1044.2 1862.24,1044.01 1862.47,1044.23 1862.7,1044.41 1862.93,1044.12 1863.15,1044.12 1863.38,1044.12 1863.61,1044.09 1863.84,1043.57 1864.06,1043.64 1864.29,1044 1864.52,1043.74 1864.75,1043.83 1864.98,1043.59 1865.2,1043.7 1865.43,1044.16 1865.66,1044.01 1865.89,1044.5 1866.11,1044.56 1866.34,1044.89 1866.57,1045.22 1866.8,1045.61 1867.03,1046.15 1867.25,1046.07 1867.48,1046.4 1867.71,1046.79 1867.94,1046.97 1868.16,1046.23 1868.39,1046.89 1868.62,1046.84 1868.85,1047.37 1869.08,1047.7 1869.3,1048.49 1869.53,1048.74 1869.76,1048.28 1869.99,1047.83 1870.21,1047.64 1870.44,1047.97 1870.67,1047.95 1870.9,1048.16 1871.13,1047.91 1871.35,1048.28 1871.58,1048.09 1871.81,1048.12 1872.04,1048.4 1872.26,1048.03 1872.49,1047.77 1872.72,1047.34 1872.95,1046.98 1873.18,1047.76 1873.4,1047.88 1873.63,1048.2 1873.86,1048.79 1874.09,1048.46 1874.31,1048.33 1874.54,1048.08 1874.77,1047.87 1875,1047.92 1875.23,1048.14 1875.45,1048.08 1875.68,1048.21 1875.91,1048.4 1876.14,1048.32 1876.36,1048.02 1876.59,1047.87 1876.82,1047.76 1877.05,1047.83 1877.27,1047.51 1877.5,1048.09 1877.73,1048.3 1877.96,1048.6 1878.19,1048.47 1878.41,1047.37 1878.64,1047.4 1878.87,1047.7 1879.1,1047.46 1879.32,1047.81 1879.55,1047.73 1879.78,1047.73 1880.01,1048.08 1880.24,1048.2 1880.46,1048.33 1880.69,1048.45 1880.92,1048.25 1881.15,1048.52 1881.37,1048.78 1881.6,1048.92 1881.83,1049.03 1882.06,1049.28 1882.29,1049.9 1882.51,1050.27 1882.74,1050.28 1882.97,1049.75 1883.2,1050.23 1883.42,1050.47 1883.65,1050.39 1883.88,1050.71 1884.11,1050.69 1884.34,1050.73 1884.56,1050.7 1884.79,1050.92 1885.02,1051.2 1885.25,1051.67 1885.47,1051.87 1885.7,1051.1 1885.93,1050.66 1886.16,1051.44 1886.39,1051.37 1886.61,1051.77 1886.84,1051.57 1887.07,1051.48 1887.3,1051.67 1887.52,1052.13 1887.75,1052.53 1887.98,1051.59 1888.21,1052.09 1888.44,1051.74 1888.66,1051.16 1888.89,1051.07 1889.12,1051.14 1889.35,1051.28 1889.57,1051.54 1889.8,1051.58 1890.03,1051.68 1890.26,1051.86 1890.49,1051.81 1890.71,1051.04 1890.94,1051.09 1891.17,1051.54 1891.4,1051.43 1891.62,1051.16 1891.85,1051.69 1892.08,1052.27 1892.31,1052.35 1892.54,1052.54 1892.76,1052.54 1892.99,1052.4 1893.22,1052.65 1893.45,1052.93 1893.67,1053.58 1893.9,1053.04 1894.13,1052.52 1894.36,1052.45 1894.59,1051.81 1894.81,1051.82 1895.04,1051.97 1895.27,1052.04 1895.5,1051.55 1895.72,1051.95 1895.95,1050.91 1896.18,1051.01 1896.41,1051.25 1896.64,1051.03 1896.86,1051.68 1897.09,1052.06 1897.32,1052.48 1897.55,1052.47 1897.77,1052.36 1898,1051.49 1898.23,1052.11 1898.46,1052.39 1898.69,1052.62 1898.91,1052.2 1899.14,1052.3 1899.37,1052.34 1899.6,1051.98 1899.82,1051.81 1900.05,1051.84 1900.28,1052.09 1900.51,1052.33 1900.74,1052.3 1900.96,1052.58 1901.19,1053.14 1901.42,1053.25 1901.65,1053.41 1901.87,1053.73 1902.1,1053.81 1902.33,1054.23 1902.56,1054.65 1902.79,1054.21 1903.01,1053.97 1903.24,1052.93 1903.47,1053.03 1903.7,1053.34 1903.92,1053.13 1904.15,1052.72 1904.38,1052.67 1904.61,1052.62 1904.84,1051.94 1905.06,1052.01 1905.29,1051.88 1905.52,1051.34 1905.75,1051.86 1905.97,1052.25 1906.2,1052.19 1906.43,1052.61 1906.66,1052.6 1906.89,1052.52 1907.11,1052.97 1907.34,1053.13 1907.57,1052.32 1907.8,1052.31 1908.02,1052.95 1908.25,1053.42 1908.48,1053.2 1908.71,1053.31 1908.94,1053.42 1909.16,1054.44 1909.39,1054.08 1909.62,1053.25 1909.85,1053.19 1910.07,1053.69 1910.3,1052.89 1910.53,1053.01 1910.76,1053.13 1910.99,1052.81 1911.21,1052.59 1911.44,1052.88 1911.67,1052.93 1911.9,1053.19 1912.12,1053.24 1912.35,1052.56 1912.58,1052.27 1912.81,1052.23 1913.03,1052.31 1913.26,1052.29 1913.49,1052.61 1913.72,1053.04 1913.95,1053.08 1914.17,1052.89 1914.4,1053.52 1914.63,1054 1914.86,1054.76 1915.08,1055.16 1915.31,1054.94 1915.54,1055.31 1915.77,1055.21 1916,1054.54 1916.22,1053.78 1916.45,1054.08 1916.68,1053.68 1916.91,1054 1917.13,1053.69 1917.36,1052.87 1917.59,1052.84 1917.82,1053.11 1918.05,1053.53 1918.27,1053.34 1918.5,1053.78 1918.73,1053.59 1918.96,1053.86 1919.18,1053.69 1919.41,1053.28 1919.64,1052.27 1919.87,1052.03 1920.1,1051.76 1920.32,1051.69 1920.55,1052.1 1920.78,1051.49 1921.01,1051.47 1921.23,1051.47 1921.46,1051.11 1921.69,1051.27 1921.92,1051.57 1922.15,1051.41 1922.37,1051.27 1922.6,1051.17 1922.83,1051.05 1923.06,1051.37 1923.28,1051.33 1923.51,1050.79 1923.74,1050.7 1923.97,1051.11 1924.2,1051.41 1924.42,1051.53 1924.65,1051.6 1924.88,1051.35 1925.11,1051.8 1925.33,1051.62 1925.56,1051.08 1925.79,1051.41 1926.02,1051.2 1926.25,1051.27 1926.47,1051.62 1926.7,1051.18 1926.93,1051.4 1927.16,1051.8 1927.38,1051.83 1927.61,1051.59 1927.84,1051.49 1928.07,1051.71 1928.3,1051.8 1928.52,1051.6 1928.75,1051.57 1928.98,1051.37 1929.21,1050.44 1929.43,1050.15 1929.66,1050.73 1929.89,1051.08 1930.12,1051.53 1930.35,1050.93 1930.57,1050.89 1930.8,1050.36 1931.03,1050.39 1931.26,1050.6 1931.48,1050.32 1931.71,1050.73 1931.94,1050.83 1932.17,1050.98 1932.4,1050.33 1932.62,1049.55 1932.85,1049.79 1933.08,1049.79 1933.31,1049.93 1933.53,1049.88 1933.76,1049.65 1933.99,1049.83 1934.22,1050.49 1934.45,1050.49 1934.67,1049.03 1934.9,1049 1935.13,1049.37 1935.36,1049.45 1935.58,1049.7 1935.81,1049.06 1936.04,1048.92 1936.27,1048.99 1936.5,1049.29 1936.72,1048.64 1936.95,1048.79 1937.18,1048.99 1937.41,1048.98 1937.63,1049.17 1937.86,1049.05 1938.09,1049.5 1938.32,1049.92 1938.55,1049.58 1938.77,1050.08 1939,1050.2 1939.23,1050.15 1939.46,1050.35 1939.68,1050.36 1939.91,1050.59 1940.14,1050.74 1940.37,1050.69 1940.6,1050.95 1940.82,1050.65 1941.05,1050.7 1941.28,1050.88 1941.51,1050.66 1941.73,1050.64 1941.96,1050.7 1942.19,1049.97 1942.42,1049.11 1942.65,1048.44 1942.87,1048.52 1943.1,1048.95 1943.33,1049.26 1943.56,1049.15 1943.78,1049.28 1944.01,1049.18 1944.24,1048.49 1944.47,1048.48 1944.7,1048.73 1944.92,1048.7 1945.15,1049.42 1945.38,1048.75 1945.61,1048.87 1945.83,1049.22 1946.06,1049.24 1946.29,1049.12 1946.52,1048.67 1946.75,1048.4 1946.97,1048.45 1947.2,1048.64 1947.43,1048.85 1947.66,1048.36 1947.88,1048.39 1948.11,1048.39 1948.34,1047.41 1948.57,1047.87 1948.79,1048.36 1949.02,1048.52 1949.25,1048.44 1949.48,1048.17 1949.71,1048.23 1949.93,1048.67 1950.16,1048.18 1950.39,1048.2 1950.62,1048.31 1950.84,1047.82 1951.07,1047.6 1951.3,1047.77 1951.53,1047.81 1951.76,1047.51 1951.98,1048.1 1952.21,1048.07 1952.44,1048.17 1952.67,1048.33 1952.89,1048.89 1953.12,1049.44 1953.35,1049.09 1953.58,1048.78 1953.81,1048.25 1954.03,1048.19 1954.26,1048.23 1954.49,1048.52 1954.72,1047.89 1954.94,1047.69 1955.17,1046.85 1955.4,1046.42 1955.63,1046.88 1955.86,1046.84 1956.08,1046.94 1956.31,1047.12 1956.54,1047.22 1956.77,1047.31 1956.99,1046.82 1957.22,1046.51 1957.45,1046.4 1957.68,1046.69 1957.91,1046.16 1958.13,1045.47 1958.36,1045.43 1958.59,1045.9 1958.82,1046.43 1959.04,1046.57 1959.27,1046.64 1959.5,1046.37 1959.73,1046.46 1959.96,1046.69 1960.18,1045.87 1960.41,1045.99 1960.64,1045.95 1960.87,1045.11 1961.09,1045.55 1961.32,1045.66 1961.55,1045.38 1961.78,1045.29 1962.01,1045.33 1962.23,1044.76 1962.46,1044.65 1962.69,1044.66 1962.92,1045.26 1963.14,1044.95 1963.37,1044.43 1963.6,1044.19 1963.83,1044.42 1964.06,1044.77 1964.28,1044.8 1964.51,1044.73 1964.74,1044.78 1964.97,1044.7 1965.19,1045.05 1965.42,1044.84 1965.65,1044.8 1965.88,1044.93 1966.11,1044.98 1966.33,1044.9 1966.56,1045.51 1966.79,1045.8 1967.02,1045.66 1967.24,1045.89 1967.47,1046.28 1967.7,1046.22 1967.93,1046.54 1968.16,1046.64 1968.38,1047.04 1968.61,1047.51 1968.84,1047.81 1969.07,1047.68 1969.29,1047.84 1969.52,1047.8 1969.75,1047.6 1969.98,1047.64 1970.21,1047.8 1970.43,1048.1 1970.66,1047.86 1970.89,1048.23 1971.12,1048.43 1971.34,1048.77 1971.57,1048.35 1971.8,1048.86 1972.03,1048.91 1972.26,1048.7 1972.48,1048.57 1972.71,1048.05 1972.94,1048.55 1973.17,1048.51 1973.39,1048.75 1973.62,1049.05 1973.85,1048.62 1974.08,1048.64 1974.31,1048.79 1974.53,1048.88 1974.76,1049.19 1974.99,1049.82 1975.22,1049.5 1975.44,1049.43 1975.67,1049.45 1975.9,1049.27 1976.13,1049.43 1976.36,1049.21 1976.58,1049.4 1976.81,1049.45 1977.04,1049.06 1977.27,1048.92 1977.49,1049.47 1977.72,1049.57 1977.95,1049.98 1978.18,1050.37 1978.41,1050.65 1978.63,1051.03 1978.86,1051.16 1979.09,1051.56 1979.32,1051.62 1979.54,1051.98 1979.77,1052.47 1980,1052.19 1980.23,1052.47 1980.46,1053.36 1980.68,1053.55 1980.91,1053.45 1981.14,1053.82 1981.37,1053.6 1981.59,1054.16 1981.82,1054.42 1982.05,1054.19 1982.28,1054.44 1982.51,1054.44 1982.73,1054.44 1982.96,1054.36 1983.19,1053.81 1983.42,1054.36 1983.64,1054.79 1983.87,1054.88 1984.1,1054.52 1984.33,1054.19 1984.56,1053.87 1984.78,1053.24 1985.01,1053.91 1985.24,1053.75 1985.47,1053.71 1985.69,1054.07 1985.92,1053.78 1986.15,1053.95 1986.38,1054.1 1986.6,1053.9 1986.83,1054.4 1987.06,1054.41 1987.29,1054.07 1987.52,1054.26 1987.74,1054.64 1987.97,1054.28 1988.2,1054.59 1988.43,1054.84 1988.65,1055.02 1988.88,1055.01 1989.11,1055.33 1989.34,1054.94 1989.57,1055.25 1989.79,1055.3 1990.02,1054.8 1990.25,1054.89 1990.48,1055.23 1990.7,1055.29 1990.93,1055.28 1991.16,1055.65 1991.39,1055.9 1991.62,1056.29 1991.84,1056 1992.07,1056.2 1992.3,1056.23 1992.53,1055.95 1992.75,1055.93 1992.98,1055.59 1993.21,1055.01 1993.44,1054.79 1993.67,1054.38 1993.89,1054.13 1994.12,1054.3 1994.35,1054 1994.58,1053.93 1994.8,1053.78 1995.03,1053.87 1995.26,1053.77 1995.49,1054.02 1995.72,1053.9 1995.94,1053.8 1996.17,1053.49 1996.4,1052.61 1996.63,1052.84 1996.85,1053.59 1997.08,1053.86 1997.31,1053.71 1997.54,1053.66 1997.77,1053.65 1997.99,1053.83 1998.22,1053.31 1998.45,1052.89 1998.68,1053.21 1998.9,1053.46 1999.13,1053.18 1999.36,1053.63 1999.59,1052.71 1999.82,1052.35 2000.04,1052.27 2000.27,1052 2000.5,1051.78 2000.73,1051.26 2000.95,1050.93 2001.18,1050.87 2001.41,1050.9 2001.64,1050.78 2001.87,1050.88 2002.09,1050.92 2002.32,1051.34 2002.55,1050.89 2002.78,1051.01 2003,1051.44 2003.23,1051.55 2003.46,1051.31 2003.69,1051.21 2003.92,1051.64 2004.14,1051.54 2004.37,1051.44 2004.6,1051.43 2004.83,1051.39 2005.05,1051.72 2005.28,1051.77 2005.51,1051.61 2005.74,1051.44 2005.97,1051.96 2006.19,1052.58 2006.42,1052.28 2006.65,1052.04 2006.88,1051.88 2007.1,1051.57 2007.33,1050.96 2007.56,1051.21 2007.79,1051.64 2008.02,1051.43 2008.24,1051.68 2008.47,1051.81 2008.7,1051.65 2008.93,1050.97 2009.15,1051.39 2009.38,1051.58 2009.61,1050.64 2009.84,1050.86 2010.07,1050.78 2010.29,1050.68 2010.52,1050.81 2010.75,1050.9 2010.98,1051.07 2011.2,1051.24 2011.43,1051.3 2011.66,1051.66 2011.89,1051.61 2012.12,1052.01 2012.34,1051.64 2012.57,1051.65 2012.8,1051.22 2013.03,1050.85 2013.25,1051.05 2013.48,1051.08 2013.71,1051.3 2013.94,1051.47 2014.17,1051.83 2014.39,1051.54 2014.62,1051.13 2014.85,1051.14 2015.08,1051.18 2015.3,1051.33 2015.53,1051.1 2015.76,1050.71 2015.99,1051.05 2016.22,1051 2016.44,1051.31 2016.67,1050.85 2016.9,1051.55 2017.13,1052.04 2017.35,1051.55 2017.58,1052.2 2017.81,1051.82 2018.04,1051.27 2018.27,1050.48 2018.49,1050.85 2018.72,1050.23 2018.95,1050.23 2019.18,1049.79 2019.4,1050.11 2019.63,1049.25 2019.86,1049.43 2020.09,1048.8 2020.32,1048.55 2020.54,1049.08 2020.77,1049.27 2021,1048.76 2021.23,1048.99 2021.45,1049.53 2021.68,1050.08 2021.91,1049.84 2022.14,1049.95 2022.36,1050.16 2022.59,1050.16 2022.82,1050.04 2023.05,1049.64 2023.28,1049.94 2023.5,1049.88 2023.73,1050.13 2023.96,1050.14 2024.19,1050.06 2024.41,1050.14 2024.64,1050.72 2024.87,1050.42 2025.1,1050.59 2025.33,1051.28 2025.55,1051.57 2025.78,1051.73 2026.01,1051.38 2026.24,1051.26 2026.46,1051.02 2026.69,1051.34 2026.92,1051.58 2027.15,1052.2 2027.38,1051.89 2027.6,1052.29 2027.83,1052.5 2028.06,1052.32 2028.29,1051.87 2028.51,1051.95 2028.74,1051.92 2028.97,1052.09 2029.2,1052.17 2029.43,1052.16 2029.65,1052.6 2029.88,1053.06 2030.11,1052.84 2030.34,1052.15 2030.56,1052.45 2030.79,1052.52 2031.02,1052.09 2031.25,1051.96 2031.48,1052.64 2031.7,1052.6 2031.93,1053.02 2032.16,1053.41 2032.39,1053.39 2032.61,1054.01 2032.84,1054.61 2033.07,1054.54 2033.3,1054.49 2033.53,1054.47 2033.75,1054.43 2033.98,1054.56 2034.21,1054.44 2034.44,1054.48 2034.66,1054.57 2034.89,1054.93 2035.12,1054.95 2035.35,1055.24 2035.58,1055.79 2035.8,1055.85 2036.03,1055.95 2036.26,1055.7 2036.49,1055.44 2036.71,1055.55 2036.94,1055.67 2037.17,1055.68 2037.4,1055.73 2037.63,1055.89 2037.85,1056.18 2038.08,1056.84 2038.31,1057.05 2038.54,1057.49 2038.76,1057.33 2038.99,1057.31 2039.22,1057.68 2039.45,1058.06 2039.68,1057.96 2039.9,1058.15 2040.13,1057.64 2040.36,1057.23 2040.59,1057.74 2040.81,1057.89 2041.04,1057.5 2041.27,1057.41 2041.5,1057.53 2041.73,1057.94 2041.95,1058.45 2042.18,1058.26 2042.41,1058.26 2042.64,1058.3 2042.86,1058.73 2043.09,1059.03 2043.32,1058.66 2043.55,1058.91 2043.78,1058.97 2044,1058.8 2044.23,1058.52 2044.46,1058.79 2044.69,1058.03 2044.91,1057.81 2045.14,1057.58 2045.37,1056.97 2045.6,1057.31 2045.83,1057.15 2046.05,1057.51 2046.28,1057.54 2046.51,1057.9 2046.74,1058.07 2046.96,1057.99 2047.19,1058.03 2047.42,1057.98 2047.65,1057.96 2047.88,1058.22 2048.1,1058.87 2048.33,1059.21 2048.56,1057.54 2048.79,1057.44 2049.01,1057.25 2049.24,1056.97 2049.47,1056.69 2049.7,1056.69 2049.93,1056.53 2050.15,1056.33 2050.38,1056.43 2050.61,1056.72 2050.84,1057.3 2051.06,1057.68 2051.29,1057.72 2051.52,1057.04 2051.75,1057.38 2051.98,1057.1 2052.2,1057.16 2052.43,1057.36 2052.66,1057.54 2052.89,1057.38 2053.11,1057.74 2053.34,1057.88 2053.57,1058.05 2053.8,1057.85 2054.03,1057.89 2054.25,1057.47 2054.48,1057.28 2054.71,1057.39 2054.94,1057.92 2055.16,1058.07 2055.39,1057.79 2055.62,1057.6 2055.85,1057.78 2056.08,1057.44 2056.3,1057.54 2056.53,1057.43 2056.76,1057.18 2056.99,1057.09 2057.21,1056.91 2057.44,1057.04 2057.67,1057.19 2057.9,1057.14 2058.12,1057.37 2058.35,1056.81 2058.58,1056.88 2058.81,1057.35 2059.04,1057.73 2059.26,1058.17 2059.49,1057.79 2059.72,1058.21 2059.95,1058.18 2060.17,1058.67 2060.4,1058.36 2060.63,1058.65 2060.86,1059.22 2061.09,1059.29 2061.31,1059.11 2061.54,1058.48 2061.77,1058.22 2062,1057.57 2062.22,1058.06 2062.45,1058.3 2062.68,1058.27 2062.91,1058.5 2063.14,1058.31 2063.36,1057.94 2063.59,1057.9 2063.82,1058.42 2064.05,1058.29 2064.27,1057.47 2064.5,1057.39 2064.73,1057.32 2064.96,1057.01 2065.19,1057.23 2065.41,1057.07 2065.64,1057.4 2065.87,1057.19 2066.1,1057.51 2066.32,1057.83 2066.55,1058.28 2066.78,1058.26 2067.01,1058.81 2067.24,1058.5 2067.46,1058.15 2067.69,1057.99 2067.92,1058.36 2068.15,1058.73 2068.37,1058.79 2068.6,1057.92 2068.83,1058.29 2069.06,1058.55 2069.29,1058.69 2069.51,1059.19 2069.74,1059.27 2069.97,1058.98 2070.2,1058.18 2070.42,1058.14 2070.65,1058.04 2070.88,1058.31 2071.11,1058.69 2071.34,1058.77 2071.56,1058.61 2071.79,1058.25 2072.02,1058.38 2072.25,1058.56 2072.47,1058.56 2072.7,1058.08 2072.93,1058.05 2073.16,1057.5 2073.39,1057.36 2073.61,1057.29 2073.84,1057.03 2074.07,1057.06 2074.3,1056.55 2074.52,1056.89 2074.75,1056.53 2074.98,1056.51 2075.21,1056.44 2075.44,1056.58 2075.66,1056.97 2075.89,1057.69 2076.12,1057.45 2076.35,1057.33 2076.57,1057.06 2076.8,1056.84 2077.03,1056.88 2077.26,1056.4 2077.49,1056.29 2077.71,1055.72 2077.94,1055.61 2078.17,1055.95 2078.4,1055.91 2078.62,1056 2078.85,1056.53 2079.08,1056.28 2079.31,1056.63 2079.54,1057.15 2079.76,1057.48 2079.99,1057.51 2080.22,1057.5 2080.45,1057.79 2080.67,1058.1 2080.9,1058.81 2081.13,1059.46 2081.36,1059.73 2081.59,1059.85 2081.81,1059.76 2082.04,1059.2 2082.27,1058.83 2082.5,1058.74 2082.72,1058.13 2082.95,1058.14 2083.18,1058.2 2083.41,1058.23 2083.64,1058.05 2083.86,1058.27 2084.09,1058.14 2084.32,1058.43 2084.55,1058.62 2084.77,1058.53 2085,1058.1 2085.23,1058.32 2085.46,1058.73 2085.69,1058.41 2085.91,1058.63 2086.14,1059.21 2086.37,1059.41 2086.6,1059.31 2086.82,1059.66 2087.05,1059.93 2087.28,1060.42 2087.51,1060.31 2087.74,1059.63 2087.96,1059.57 2088.19,1059.59 2088.42,1059.69 2088.65,1059.87 2088.87,1059.47 2089.1,1059.66 2089.33,1059.68 2089.56,1059.68 2089.79,1059.81 2090.01,1059.71 2090.24,1059.95 2090.47,1060.16 2090.7,1060.28 2090.92,1060.69 2091.15,1061.16 2091.38,1060.74 2091.61,1060.43 2091.84,1060.47 2092.06,1060.32 2092.29,1060.46 2092.52,1060.04 2092.75,1060.03 2092.97,1059.86 2093.2,1059.44 2093.43,1058.81 2093.66,1058.3 2093.89,1058 2094.11,1058.4 2094.34,1058.43 2094.57,1058.19 2094.8,1058 2095.02,1057.55 2095.25,1057.5 2095.48,1057.1 2095.71,1056.98 2095.93,1056.42 2096.16,1056.2 2096.39,1056.46 2096.62,1057 2096.85,1056.59 2097.07,1057.3 2097.3,1057.15 2097.53,1057.5 2097.76,1057.16 2097.98,1057.57 2098.21,1057.95 2098.44,1058.07 2098.67,1057.95 2098.9,1058.19 2099.12,1057.46 2099.35,1057.71 2099.58,1058.26 2099.81,1058.47 2100.03,1058.36 2100.26,1057.61 2100.49,1058.11 2100.72,1057.3 2100.95,1057.07 2101.17,1057.15 2101.4,1056.67 2101.63,1056.87 2101.86,1057.33 2102.08,1057.19 2102.31,1057.08 2102.54,1057.4 2102.77,1057.66 2103,1058.22 2103.22,1057.67 2103.45,1057.69 2103.68,1057.35 2103.91,1057.4 2104.13,1057.94 2104.36,1058.08 2104.59,1058.11 2104.82,1058.15 2105.05,1057.64 2105.27,1057.54 2105.5,1058.13 2105.73,1058.03 2105.96,1058.22 2106.18,1057.85 2106.41,1057.96 2106.64,1058.43 2106.87,1058.43 2107.1,1058.52 2107.32,1058.75 2107.55,1058.94 2107.78,1059.2 2108.01,1059.53 2108.23,1059.64 2108.46,1059.81 2108.69,1060.02 2108.92,1060.1 2109.15,1060.42 2109.37,1060.51 2109.6,1060.77 2109.83,1060.73 2110.06,1060.69 2110.28,1060.97 2110.51,1060.41 2110.74,1060.75 2110.97,1060.43 2111.2,1060.68 2111.42,1060.58 2111.65,1060.64 2111.88,1060.69 2112.11,1061.23 2112.33,1061.01 2112.56,1061.43 2112.79,1061.7 2113.02,1062.16 2113.25,1062.38 2113.47,1062.43 2113.7,1062.51 2113.93,1062.02 2114.16,1062.04 2114.38,1062.57 2114.61,1062.61 2114.84,1061.92 2115.07,1062.22 2115.3,1062.66 2115.52,1062.43 2115.75,1062.67 2115.98,1062.77 2116.21,1062.94 2116.43,1062.97 2116.66,1063.66 2116.89,1063.99 2117.12,1064.16 2117.35,1063.78 2117.57,1063.77 2117.8,1063.73 2118.03,1063.77 2118.26,1062.81 2118.48,1063.01 2118.71,1063.51 2118.94,1063.27 2119.17,1063.41 2119.4,1063.49 2119.62,1063.51 2119.85,1063.48 2120.08,1063.62 2120.31,1064 2120.53,1063.7 2120.76,1064.15 2120.99,1063.64 2121.22,1063.12 2121.45,1063.04 2121.67,1062.77 2121.9,1062.09 2122.13,1061.7 2122.36,1061.96 2122.58,1062.16 2122.81,1062.53 2123.04,1062.84 2123.27,1063.07 2123.5,1062.88 2123.72,1062.91 2123.95,1063.41 2124.18,1063.59 2124.41,1063.38 2124.63,1063.51 2124.86,1063.34 2125.09,1063.72 2125.32,1063.97 2125.55,1063.58 2125.77,1064.17 2126,1064.1 2126.23,1064.36 2126.46,1064.34 2126.68,1064.41 2126.91,1063.67 2127.14,1063.42 2127.37,1063.49 2127.6,1063.7 2127.82,1063.84 2128.05,1064.22 2128.28,1064.09 2128.51,1063.91 2128.73,1064.28 2128.96,1064.25 2129.19,1064.07 2129.42,1063.68 2129.65,1063.59 2129.87,1063.1 2130.1,1063.61 2130.33,1063.58 2130.56,1063.49 2130.78,1063.44 2131.01,1063.32 2131.24,1063.32 2131.47,1062.67 2131.69,1062.48 2131.92,1062.78 2132.15,1063.01 2132.38,1063.68 2132.61,1064.41 2132.83,1064.78 2133.06,1065.15 2133.29,1064.65 2133.52,1064.37 2133.74,1063.79 2133.97,1063.85 2134.2,1063.82 2134.43,1063.69 2134.66,1063.64 2134.88,1063.78 2135.11,1063.96 2135.34,1063.72 2135.57,1063.84 2135.79,1063.75 2136.02,1064.08 2136.25,1064.34 2136.48,1064.41 2136.71,1064.49 2136.93,1064.58 2137.16,1064.68 2137.39,1064.13 2137.62,1064.16 2137.84,1063.71 2138.07,1063.07 2138.3,1062.58 2138.53,1063.26 2138.76,1063.35 2138.98,1063.28 2139.21,1064.08 2139.44,1063.97 2139.67,1063.16 2139.89,1063.62 2140.12,1063.64 2140.35,1063.83 2140.58,1063.76 2140.81,1063.87 2141.03,1063.19 2141.26,1063.49 2141.49,1063.5 2141.72,1063.68 2141.94,1064.02 2142.17,1063.96 2142.4,1063.87 2142.63,1063.49 2142.86,1063.17 2143.08,1061.96 2143.31,1062.03 2143.54,1062.14 2143.77,1062.75 2143.99,1061.32 2144.22,1061.63 2144.45,1061.84 2144.68,1061.59 2144.91,1062.11 2145.13,1062.47 2145.36,1061.76 2145.59,1061.17 2145.82,1061.02 2146.04,1061.02 2146.27,1061.53 2146.5,1061.04 2146.73,1060.73 2146.96,1060.38 2147.18,1060 2147.41,1059.73 2147.64,1059.66 2147.87,1059.23 2148.09,1059.01 2148.32,1059.45 2148.55,1059.39 2148.78,1058.96 2149.01,1059.1 2149.23,1058.78 2149.46,1057.9 2149.69,1057.43 2149.92,1057.43 2150.14,1056.93 2150.37,1056.7 2150.6,1056.04 2150.83,1055.95 2151.06,1056.14 2151.28,1056.32 2151.51,1056.22 2151.74,1056.63 2151.97,1056.48 2152.19,1056.65 2152.42,1056.6 2152.65,1056.79 2152.88,1056.84 2153.11,1056.91 2153.33,1057.04 2153.56,1057.38 2153.79,1057.22 2154.02,1057.05 2154.24,1057.22 2154.47,1056.82 2154.7,1057.32 2154.93,1057.18 2155.16,1057.49 2155.38,1057.34 2155.61,1057.26 2155.84,1057.39 2156.07,1057.64 2156.29,1058.09 2156.52,1058.64 2156.75,1058.76 2156.98,1058.68 2157.21,1058.54 2157.43,1058.29 2157.66,1058 2157.89,1058.37 2158.12,1057.93 2158.34,1057.34 2158.57,1057.53 2158.8,1057.17 2159.03,1057.25 2159.26,1056.99 2159.48,1056.67 2159.71,1056.83 2159.94,1056.92 2160.17,1056.72 2160.39,1056.57 2160.62,1057.07 2160.85,1056.59 2161.08,1056.64 2161.31,1056.48 2161.53,1055.68 2161.76,1055.46 2161.99,1055.76 2162.22,1055.37 2162.44,1054.77 2162.67,1054.89 2162.9,1055.04 2163.13,1055.31 2163.36,1055.04 2163.58,1055.14 2163.81,1055.02 2164.04,1054.75 2164.27,1055.22 2164.49,1055.21 2164.72,1054.87 2164.95,1054.99 2165.18,1055.4 2165.41,1055.06 2165.63,1055.31 2165.86,1055.13 2166.09,1055.07 2166.32,1054.99 2166.54,1054.61 2166.77,1054.43 2167,1054.16 2167.23,1054.55 2167.45,1054.29 2167.68,1054.72 2167.91,1054.78 2168.14,1055.34 2168.37,1054.64 2168.59,1053.42 2168.82,1053.48 2169.05,1054.02 2169.28,1053.89 2169.5,1053.66 2169.73,1053.67 2169.96,1053.96 2170.19,1054.04 2170.42,1054.35 2170.64,1054.72 2170.87,1054.82 2171.1,1054.66 2171.33,1054.9 2171.55,1055.22 2171.78,1055.33 2172.01,1055.62 2172.24,1056.08 2172.47,1055.82 2172.69,1055.8 2172.92,1055.9 2173.15,1056.12 2173.38,1056.34 2173.6,1056.4 2173.83,1056.63 2174.06,1056.54 2174.29,1056.6 2174.52,1057.14 2174.74,1057.57 2174.97,1057.78 2175.2,1057.7 2175.43,1058 2175.65,1058.68 2175.88,1058.9 2176.11,1058.41 2176.34,1058.02 2176.57,1057.65 2176.79,1057.92 2177.02,1057.68 2177.25,1057.8 2177.48,1058.01 2177.7,1057.63 2177.93,1057.48 2178.16,1057.37 2178.39,1057.1 2178.62,1056.96 2178.84,1057.1 2179.07,1057.01 2179.3,1056.83 2179.53,1056.69 2179.75,1056.58 2179.98,1056.06 2180.21,1056.22 2180.44,1056.14 2180.67,1056.4 2180.89,1057.15 2181.12,1057.5 2181.35,1058.09 2181.58,1058.16 2181.8,1057.96 2182.03,1058.08 2182.26,1058.36 2182.49,1057.91 2182.72,1058.11 2182.94,1057.86 2183.17,1057.8 2183.4,1056.86 2183.63,1056.54 2183.85,1056.24 2184.08,1056.4 2184.31,1056.81 2184.54,1057.18 2184.77,1057.81 2184.99,1057.58 2185.22,1057.62 2185.45,1057.85 2185.68,1058.28 2185.9,1058.15 2186.13,1058.36 2186.36,1057.99 2186.59,1057.5 2186.82,1058.07 2187.04,1057.48 2187.27,1057.39 2187.5,1057.6 2187.73,1057.37 2187.95,1057.11 2188.18,1057.27 2188.41,1056.96 2188.64,1057.01 2188.87,1056.53 2189.09,1056.73 2189.32,1056.08 2189.55,1056.43 2189.78,1056.13 2190,1056.54 2190.23,1056.89 2190.46,1056.4 2190.69,1056.67 2190.92,1056.56 2191.14,1056.39 2191.37,1056.56 2191.6,1056.64 2191.83,1057.06 2192.05,1057.11 2192.28,1057.34 2192.51,1056.56 2192.74,1056.65 2192.97,1056.81 2193.19,1057.08 2193.42,1057.08 2193.65,1057.29 2193.88,1057.49 2194.1,1057.37 2194.33,1057.12 2194.56,1057.3 2194.79,1057.05 2195.02,1057.33 2195.24,1056.82 2195.47,1056.55 2195.7,1056.42 2195.93,1056.62 2196.15,1056.61 2196.38,1056.52 2196.61,1056.78 2196.84,1057.13 2197.07,1056.83 2197.29,1057.22 2197.52,1057.75 2197.75,1057.86 2197.98,1057.91 2198.2,1057.69 2198.43,1057.8 2198.66,1057.85 2198.89,1057.62 2199.12,1057.74 2199.34,1058.08 2199.57,1058.44 2199.8,1058.41 2200.03,1058.39 2200.25,1058.09 2200.48,1058.16 2200.71,1058.14 2200.94,1058.09 2201.17,1058.44 2201.39,1058.7 2201.62,1059.11 2201.85,1058.71 2202.08,1058.89 2202.3,1059.22 2202.53,1059.01 2202.76,1059.22 2202.99,1058.83 2203.21,1058.76 2203.44,1058.32 2203.67,1058.34 2203.9,1058.44 2204.13,1058.49 2204.35,1058.68 2204.58,1058.49 2204.81,1058.47 2205.04,1058.4 2205.26,1057.94 2205.49,1058.06 2205.72,1057.64 2205.95,1057.91 2206.18,1058.17 2206.4,1058.43 2206.63,1058.49 2206.86,1058.35 2207.09,1058.55 2207.31,1058.5 2207.54,1058.73 2207.77,1058.89 2208,1058.53 2208.23,1058.26 2208.45,1058.02 2208.68,1058.17 2208.91,1057.83 2209.14,1058.17 2209.36,1058.36 2209.59,1058.22 2209.82,1057.98 2210.05,1058.28 2210.28,1057.99 2210.5,1057.81 2210.73,1057.97 2210.96,1058.01 2211.19,1058.05 2211.41,1058.39 2211.64,1058.1 2211.87,1058.37 2212.1,1057.89 2212.33,1057.72 2212.55,1057.56 2212.78,1056.96 2213.01,1057.2 2213.24,1056.87 2213.46,1056.39 2213.69,1056.18 2213.92,1056.5 2214.15,1056.97 2214.38,1056.97 2214.6,1056.41 2214.83,1056.45 2215.06,1056.62 2215.29,1056.39 2215.51,1056.65 2215.74,1056.08 2215.97,1055.84 2216.2,1055.91 2216.43,1056.07 2216.65,1055.92 2216.88,1055.94 2217.11,1056.03 2217.34,1055.89 2217.56,1055.89 2217.79,1055.6 2218.02,1055.51 2218.25,1055.58 2218.48,1055.63 2218.7,1055.38 2218.93,1055.61 2219.16,1055.74 2219.39,1056.02 2219.61,1055.98 2219.84,1055.67 2220.07,1055.84 2220.3,1055.9 2220.53,1056.16 2220.75,1056.21 2220.98,1056.07 2221.21,1056.1 2221.44,1055.88 2221.66,1055.43 2221.89,1055.81 2222.12,1055.82 2222.35,1055.78 2222.58,1055.99 2222.8,1055.14 2223.03,1055.47 2223.26,1055.74 2223.49,1055.59 2223.71,1055.05 2223.94,1055.3 2224.17,1055.37 2224.4,1055.66 2224.63,1055.55 2224.85,1055.82 2225.08,1055.19 2225.31,1055.25 2225.54,1055.23 2225.76,1055.14 2225.99,1054.93 2226.22,1055.22 2226.45,1055.49 2226.68,1055.49 2226.9,1055.31 2227.13,1055.64 2227.36,1056.18 2227.59,1056.39 2227.81,1056.14 2228.04,1056.19 2228.27,1056.34 2228.5,1056.75 2228.73,1056.72 2228.95,1056.91 2229.18,1057.31 2229.41,1057.32 2229.64,1057.11 2229.86,1057.29 2230.09,1057.43 2230.32,1057.55 2230.55,1057.77 2230.78,1057.59 2231,1056.92 2231.23,1057.05 2231.46,1056.98 2231.69,1056.94 2231.91,1056.67 2232.14,1057.2 2232.37,1057.52 2232.6,1057.29 2232.83,1057.69 2233.05,1057.41 2233.28,1057.32 2233.51,1057.12 2233.74,1056.67 2233.96,1056.68 2234.19,1056.7 2234.42,1057 2234.65,1057.11 2234.88,1057.59 2235.1,1057.64 2235.33,1057.24 2235.56,1057.47 2235.79,1057.64 2236.01,1057.68 2236.24,1057.55 2236.47,1057.52 2236.7,1057.32 2236.93,1057.66 2237.15,1057.77 2237.38,1057.8 2237.61,1057.53 2237.84,1057.57 2238.06,1057.32 2238.29,1057.42 2238.52,1057.6 2238.75,1057.65 2238.98,1057.78 2239.2,1057.84 2239.43,1057.51 2239.66,1057.69 2239.89,1058.16 2240.11,1057.9 2240.34,1058.1 2240.57,1057.98 2240.8,1057.99 2241.02,1058.41 2241.25,1058.69 2241.48,1059.24 2241.71,1059.46 2241.94,1059.92 2242.16,1060.16 2242.39,1060.47 2242.62,1060.74 2242.85,1060.58 2243.07,1060.4 2243.3,1060.76 2243.53,1060.87 2243.76,1060.62 2243.99,1061.06 2244.21,1061.51 2244.44,1061.98 2244.67,1062.09 2244.9,1062.32 2245.12,1062.32 2245.35,1062.14 2245.58,1062.41 2245.81,1062.75 2246.04,1062.94 2246.26,1062.38 2246.49,1062.13 2246.72,1062.48 2246.95,1062.37 2247.17,1062.05 2247.4,1062.12 2247.63,1061.38 2247.86,1061.3 2248.09,1061.31 2248.31,1061.38 2248.54,1061.78 2248.77,1061.43 2249,1061.71 2249.22,1061.82 2249.45,1061.88 2249.68,1062.17 2249.91,1062.46 2250.14,1062.33 2250.36,1062.06 2250.59,1061.64 2250.82,1061.65 2251.05,1061.41 2251.27,1061.6 2251.5,1061.44 2251.73,1061.55 2251.96,1062.05 2252.19,1062.26 2252.41,1062.77 2252.64,1062.48 2252.87,1062.18 2253.1,1062.11 2253.32,1061.99 2253.55,1061.86 2253.78,1061.37 2254.01,1061.57 2254.24,1061.51 2254.46,1061.77 2254.69,1062.28 2254.92,1062.77 2255.15,1062.64 2255.37,1062.75 2255.6,1061.76 2255.83,1061.92 2256.06,1061.95 2256.29,1061.28 2256.51,1061.29 2256.74,1061.37 2256.97,1061.59 2257.2,1062.26 2257.42,1061.81 2257.65,1061.96 2257.88,1061.87 2258.11,1062.11 2258.34,1062.13 2258.56,1061.67 2258.79,1061.5 2259.02,1061.47 2259.25,1061.7 2259.47,1061.8 2259.7,1062.33 2259.93,1061.91 2260.16,1062.61 2260.39,1062.31 2260.61,1062.98 2260.84,1062.79 2261.07,1063.28 2261.3,1063.07 2261.52,1063.05 2261.75,1062.74 2261.98,1062.49 2262.21,1062.85 2262.44,1062.78 2262.66,1063 2262.89,1063.42 2263.12,1063.21 2263.35,1063.66 2263.57,1063.61 2263.8,1063.24 2264.03,1063.63 2264.26,1063.56 2264.49,1063.88 2264.71,1063.85 2264.94,1063.81 2265.17,1062.92 2265.4,1063.15 2265.62,1063.12 2265.85,1063.2 2266.08,1063.04 2266.31,1063.14 2266.54,1063.2 2266.76,1062.71 2266.99,1062.1 2267.22,1061.77 2267.45,1061.7 2267.67,1061.64 2267.9,1062.21 2268.13,1062.71 2268.36,1063.27 2268.59,1063.38 2268.81,1063.66 2269.04,1063.51 2269.27,1063.4 2269.5,1063.46 2269.72,1063.54 2269.95,1063.39 2270.18,1063.27 2270.41,1063.39 2270.64,1063.49 2270.86,1063.19 2271.09,1063.7 2271.32,1063.67 2271.55,1063.52 2271.77,1062.89 2272,1062.83 2272.23,1062.19 2272.46,1062.04 2272.69,1061.66 2272.91,1061.98 2273.14,1062.23 2273.37,1062.34 2273.6,1062.73 2273.82,1062.43 2274.05,1062.08 2274.28,1061.92 2274.51,1061.91 2274.74,1062.05 2274.96,1062.29 2275.19,1061.88 2275.42,1061.54 2275.65,1061.28 2275.87,1061.19 2276.1,1061.11 2276.33,1061.67 2276.56,1061.76 2276.78,1061.9 2277.01,1061.73 2277.24,1061.97 2277.47,1062.11 2277.7,1062.42 2277.92,1062.57 2278.15,1062.7 2278.38,1062.68 2278.61,1062.92 2278.83,1063.33 2279.06,1063.76 2279.29,1063.45 2279.52,1063.57 2279.75,1063.68 2279.97,1062.93 2280.2,1063.49 2280.43,1063.44 2280.66,1063.68 2280.88,1063.3 2281.11,1063.33 2281.34,1063.64 2281.57,1063.62 2281.8,1063.72 2282.02,1064 2282.25,1064.17 2282.48,1064.36 2282.71,1064.1 2282.93,1064.18 2283.16,1063.91 2283.39,1063.71 2283.62,1064.05 2283.85,1064.07 2284.07,1064.24 2284.3,1064.35 2284.53,1064.69 2284.76,1064.1 2284.98,1064.11 2285.21,1064.03 2285.44,1063.6 2285.67,1063.84 2285.9,1063.95 2286.12,1063.75 2286.35,1063.43 2286.58,1063.56 2286.81,1063.7 2287.03,1063.76 2287.26,1063.74 2287.49,1063.76 2287.72,1063.22 2287.95,1063.3 2288.17,1063.06 2288.4,1063.04 2288.63,1062.39 2288.86,1062.65 2289.08,1062.51 2289.31,1062.61 2289.54,1062.71 2289.77,1062.65 2290,1062.67 2290.22,1062.63 2290.45,1062.65 2290.68,1062.56 2290.91,1061.91 2291.13,1062.05 2291.36,1061.85 2291.59,1061.49 2291.82,1061.23 2292.05,1060.97 2292.27,1061.39 2292.5,1060.94 2292.73,1061.18 2292.96,1061.15 2293.18,1061.02 2293.41,1061.28 2293.64,1061.48 2293.87,1061.84 2294.1,1062.04 2294.32,1062.08 2294.55,1062.21 2294.78,1062.32 2295.01,1061.65 2295.23,1061.76 2295.46,1061.73 2295.69,1061.47 2295.92,1061.59 2296.15,1061.63 2296.37,1061.5 2296.6,1061.75 2296.83,1062.36 2297.06,1062.69 2297.28,1062.67 2297.51,1062.8 2297.74,1063.22 2297.97,1063.06 2298.2,1063.21 2298.42,1063.1 2298.65,1062.93 2298.88,1062.79 2299.11,1063.13 2299.33,1063.47 2299.56,1063.64 2299.79,1063.69 2300.02,1063.99 2300.25,1064.22 2300.47,1064.23 2300.7,1064.2 2300.93,1063.95 2301.16,1063.72 2301.38,1063.47 2301.61,1063.18 2301.84,1063.06 2302.07,1063.14 2302.3,1063.01 2302.52,1063.21 2302.75,1063.48 2302.98,1063.81 2303.21,1063.97 2303.43,1063.65 2303.66,1063.46 2303.89,1063.56 2304.12,1063.83 2304.35,1063.31 2304.57,1063.25 2304.8,1063.71 2305.03,1063.79 2305.26,1062.97 2305.48,1062.89 2305.71,1062.86 2305.94,1063.1 2306.17,1063.14 2306.4,1063.33 2306.62,1063.88 2306.85,1064.11 2307.08,1063.79 2307.31,1063.81 2307.53,1064.31 2307.76,1064.56 2307.99,1064.34 2308.22,1064.66 2308.45,1064.98 2308.67,1064.8 2308.9,1064.87 2309.13,1065.12 2309.36,1065.44 2309.58,1065.48 2309.81,1066.06 2310.04,1066.31 2310.27,1066.34 2310.5,1066.47 2310.72,1066.02 2310.95,1066.03 2311.18,1066.42 2311.41,1066.7 2311.63,1066.65 2311.86,1066.78 2312.09,1066.95 2312.32,1067.05 2312.54,1067.15 2312.77,1067.39 2313,1067.25 2313.23,1067.28 2313.46,1067.9 2313.68,1067.39 2313.91,1067.27 2314.14,1067.39 2314.37,1067.26 2314.59,1067.16 2314.82,1067.26 2315.05,1066.83 2315.28,1067.24 2315.51,1067.15 2315.73,1067.28 2315.96,1067.16 2316.19,1067.33 2316.42,1067.56 2316.64,1067.79 2316.87,1067.87 2317.1,1068.47 2317.33,1068.87 2317.56,1068.8 2317.78,1068.77 2318.01,1068.6 2318.24,1068.81 2318.47,1068.88 2318.69,1068.43 2318.92,1068.43 2319.15,1068.99 2319.38,1069.25 2319.61,1069.49 2319.83,1069.8 2320.06,1070.23 2320.29,1069.74 2320.52,1069.6 2320.74,1069.81 2320.97,1069.73 2321.2,1069.45 2321.43,1069.18 2321.66,1068.44 2321.88,1068.28 2322.11,1068.58 2322.34,1068.44 2322.57,1068.22 2322.79,1067.84 2323.02,1067.88 2323.25,1068.11 2323.48,1068.06 2323.71,1068.28 2323.93,1068.62 2324.16,1068.71 2324.39,1068.51 2324.62,1068.75 2324.84,1068.8 2325.07,1069.01 2325.3,1068.79 2325.53,1068.72 2325.76,1068.5 2325.98,1068.5 2326.21,1068.63 2326.44,1068.54 2326.67,1068.81 2326.89,1068.97 2327.12,1069.08 2327.35,1068.87 2327.58,1068.64 2327.81,1068.98 2328.03,1069.24 2328.26,1069.08 2328.49,1069.43 2328.72,1069.83 2328.94,1069.84 2329.17,1070.21 2329.4,1069.52 2329.63,1068.35 2329.86,1068.43 2330.08,1068.54 2330.31,1068.57 2330.54,1068.02 2330.77,1067.83 2330.99,1067.66 2331.22,1067.37 2331.45,1067.44 2331.68,1067.16 2331.91,1067.14 2332.13,1067.27 2332.36,1067.04 2332.59,1067.21 2332.82,1067.2 2333.04,1066.89 2333.27,1066.65 2333.5,1066.61 2333.73,1067.11 2333.96,1067.26 2334.18,1067.58 2334.41,1067.7 2334.64,1067.11 2334.87,1067.25 2335.09,1067.24 2335.32,1067.11 2335.55,1067.49 2335.78,1067.37 2336.01,1067.31 2336.23,1066.91 2336.46,1067.03 2336.69,1066.53 2336.92,1066.49 2337.14,1066.83 2337.37,1066.73 2337.6,1066.77 2337.83,1066.72 2338.06,1066.78 2338.28,1067.24 2338.51,1066.87 2338.74,1067.05 2338.97,1066.84 2339.19,1066.69 2339.42,1066.44 2339.65,1066.52 2339.88,1066.16 2340.11,1066.45 2340.33,1066.62 2340.56,1066.32 2340.79,1066.76 2341.02,1067.23 2341.24,1066.95 2341.47,1066.77 2341.7,1067.19 2341.93,1067.04 2342.16,1067.32 2342.38,1067.31 2342.61,1067.56 2342.84,1067.78 2343.07,1067.94 2343.29,1067.73 2343.52,1067.2 2343.75,1067.26 2343.98,1066.2 2344.21,1065.68 2344.43,1065.55 2344.66,1065.48 2344.89,1065.57 2345.12,1065.85 2345.34,1066.13 2345.57,1065.92 2345.8,1066.13 2346.03,1065.94 2346.26,1066.13 2346.48,1065.74 2346.71,1065.81 2346.94,1065.73 2347.17,1065.77 2347.39,1065.62 2347.62,1065.83 2347.85,1065.51 2348.08,1065.49 2348.31,1065.54 2348.53,1065.52 2348.76,1065.77 2348.99,1065.6 2349.22,1066.1 2349.44,1066.1 2349.67,1066.14 2349.9,1066.2 2350.13,1065.9 2350.35,1065.8 2350.58,1065.59 2350.81,1065.95 2351.04,1066.03 2351.27,1066.16 2351.49,1065.9 2351.72,1065.77 2351.95,1065.27 2352.18,1065.25 2352.4,1065.62 2352.63,1065.9 2352.86,1066.29 2353.09,1066.16 2353.32,1066.37 2353.54,1066.53 2353.77,1066.83 2354,1066.93 2354.23,1066.87 2354.45,1066.83 2354.68,1066.35 2354.91,1066.63 2355.14,1066.69 2355.37,1066.57 2355.59,1066.53 2355.82,1066.1 2356.05,1066.43 2356.28,1066.58 2356.5,1066.83 2356.73,1066.86 2356.96,1066.91 2357.19,1067.05 2357.42,1066.71 2357.64,1066.54 2357.87,1067.25 2358.1,1067.82 2358.33,1067.95 2358.55,1067.58 2358.78,1067.31 2359.01,1067.58 2359.24,1067.38 2359.47,1067.08 2359.69,1067.03 2359.92,1067.3 2360.15,1067.34 2360.38,1067.3 2360.6,1067.01 2360.83,1067.38 2361.06,1067.33 2361.29,1067.59 2361.52,1067.34 2361.74,1067.4 2361.97,1067.25 2362.2,1067.72 2362.43,1067.85 2362.65,1067.87 2362.88,1067.72 2363.11,1067.95 2363.34,1068.13 2363.57,1068.45 2363.79,1068.82 2364.02,1068.9 2364.25,1068.78 2364.48,1069.03 2364.7,1069.27 2364.93,1069.45 2365.16,1069.39 2365.39,1069.34 2365.62,1069.45 2365.84,1069.33 2366.07,1069.03 2366.3,1069.1 2366.53,1068.99 2366.75,1068.46 2366.98,1068.53 2367.21,1068.55 2367.44,1068.63 2367.67,1068.97 2367.89,1068.84 2368.12,1068.78 2368.35,1068.79 2368.58,1068.72 2368.8,1068.74 2369.03,1069.09 2369.26,1069.06 2369.49,1069.44 2369.72,1069.35 2369.94,1068.98 2370.17,1068.62 2370.4,1068.58 2370.63,1068.4 2370.85,1068.02 2371.08,1067.32 2371.31,1067.67 2371.54,1067.84 2371.77,1067.6 2371.99,1067.62 2372.22,1067.59 2372.45,1067.53 2372.68,1067.63 2372.9,1067.81 2373.13,1067.87 2373.36,1068.29 2373.59,1068 2373.82,1068.18 2374.04,1068.03 2374.27,1068.04 2374.5,1068.28 2374.73,1068.47 2374.95,1068.57 2375.18,1068.41 2375.41,1068.79 2375.64,1068.81 2375.87,1068.54 2376.09,1068.64 2376.32,1068.98 2376.55,1069.05 2376.78,1069.18 2377,1069.41 2377.23,1069.33 2377.46,1069.27 2377.69,1069.06 2377.92,1068.9 2378.14,1068.66 2378.37,1068.46 2378.6,1068.69 2378.83,1069.04 2379.05,1068.89 2379.28,1068.8 2379.51,1068.46 2379.74,1069.05 2379.97,1069.35 2380.19,1069.59 2380.42,1070.01 2380.65,1070.07 2380.88,1070.33 2381.1,1070.28 2381.33,1070.71 2381.56,1071.28 2381.79,1071.81 2382.02,1071.93 2382.24,1071.97 2382.47,1072.17 2382.7,1071.54 2382.93,1070.7 2383.15,1070.96 2383.38,1070.89 2383.61,1071.17 2383.84,1071.55 2384.07,1071.68 2384.29,1071.74 2384.52,1071.96 2384.75,1072.31 2384.98,1072.14 2385.2,1071.86 2385.43,1072.08 2385.66,1072.28 2385.89,1072.37 2386.11,1072.78 2386.34,1072.93 2386.57,1073.19 2386.8,1073.55 2387.03,1073.37 2387.25,1073.54 2387.48,1073.49 2387.71,1073.38 2387.94,1073.79 2388.16,1074.13 2388.39,1074.18 2388.62,1073.97 2388.85,1074.2 2389.08,1074.42 2389.3,1074.22 2389.53,1074.11 2389.76,1074.13 2389.99,1073.97 2390.21,1073.63 2390.44,1073.31 2390.67,1073.35 2390.9,1073.63 2391.13,1073.61 2391.35,1073.65 2391.58,1073.04 2391.81,1073.08 2392.04,1073.09 2392.26,1072.93 2392.49,1072.87 2392.72,1073.15 2392.95,1073.2 2393.18,1072.78 2393.4,1072.99 2393.63,1073.18 2393.86,1073.38 2394.09,1072.75 2394.31,1072.74 2394.54,1072.88 2394.77,1072.77 2395,1072.83 2395.23,1072.63 2395.45,1072.8 2395.68,1072.86 2395.91,1072.69 2396.14,1072.76 2396.36,1072.68 2396.59,1072.79 2396.82,1072.8 2397.05,1073.1 2397.28,1073.34 2397.5,1072.91 2397.73,1072.04 2397.96,1071.33 2398.19,1071.55 2398.41,1071.7 2398.64,1072.02 2398.87,1072.12 2399.1,1072.03 2399.33,1071.48 2399.55,1071.63 2399.78,1071.16 2400.01,1071.56 2400.24,1071.67 2400.46,1071.94 2400.69,1072.22 2400.92,1071.91 2401.15,1072.09 2401.38,1072.48 2401.6,1072.74 2401.83,1072.29 2402.06,1072.25 2402.29,1072.03 2402.51,1071.99 2402.74,1071.67 2402.97,1072 2403.2,1072.24 2403.43,1072.55 2403.65,1072.49 2403.88,1072.44 2404.11,1072.59 2404.34,1072.48 2404.56,1072.46 2404.79,1072.42 2405.02,1071.81 2405.25,1071.81 2405.48,1071.92 2405.7,1072.26 2405.93,1072.45 2406.16,1072.56 2406.39,1071.98 2406.61,1072.34 2406.84,1072.28 2407.07,1072.33 2407.3,1072.61 2407.53,1072.96 2407.75,1073.3 2407.98,1073.57 2408.21,1073.26 2408.44,1072.83 2408.66,1072.63 2408.89,1072.7 2409.12,1072.93 2409.35,1073.57 2409.58,1073.42 2409.8,1072.56 2410.03,1072.82 2410.26,1072.79 2410.49,1072.8 2410.71,1072.95 2410.94,1072.59 2411.17,1072.27 2411.4,1071.94 2411.63,1071.89 2411.85,1071.57 2412.08,1071.14 2412.31,1071.48 2412.54,1071.22 2412.76,1071.13 2412.99,1070.65 2413.22,1070.4 2413.45,1070.59 2413.68,1070.67 2413.9,1070.89 2414.13,1071.28 2414.36,1071.41 2414.59,1071.5 2414.81,1071.73 2415.04,1071.69 2415.27,1071.76 2415.5,1071.84 2415.73,1072.14 2415.95,1071.76 2416.18,1071.95 2416.41,1072.1 2416.64,1072.08 2416.86,1072.18 2417.09,1072.37 2417.32,1072.31 2417.55,1072.17 2417.78,1072.05 2418,1071.93 2418.23,1071.34 2418.46,1071.13 2418.69,1071.41 2418.91,1071.27 2419.14,1071.2 2419.37,1071.43 2419.6,1071.43 2419.83,1071.67 2420.05,1071.71 2420.28,1071.56 2420.51,1071.46 2420.74,1071.51 2420.96,1071.64 2421.19,1071.38 2421.42,1071.84 2421.65,1072.07 2421.87,1072.33 2422.1,1072.56 2422.33,1072.21 2422.56,1072.54 2422.79,1072.27 2423.01,1072.48 2423.24,1072.27 2423.47,1072.05 2423.7,1072.52 2423.92,1072.54 2424.15,1072.81 2424.38,1072.97 2424.61,1073.35 2424.84,1073.87 2425.06,1073.88 2425.29,1074.01 2425.52,1073.77 2425.75,1073.74 2425.97,1073.89 2426.2,1073.74 2426.43,1073.95 2426.66,1074.02 2426.89,1074.2 2427.11,1073.96 2427.34,1074.04 2427.57,1074.29 2427.8,1074.45 2428.02,1074.74 2428.25,1075.23 2428.48,1075.25 2428.71,1075 2428.94,1075.24 2429.16,1075.55 2429.39,1076 2429.62,1075.76 2429.85,1075.89 2430.07,1075.92 2430.3,1075.5 2430.53,1075.56 2430.76,1075.62 2430.99,1075.99 2431.21,1076.42 2431.44,1076.51 2431.67,1076.94 2431.9,1076.82 2432.12,1077.28 2432.35,1077.18 2432.58,1077.18 2432.81,1076.82 2433.04,1076.83 2433.26,1077.08 2433.49,1077.24 2433.72,1077.48 2433.95,1077.5 2434.17,1077.42 2434.4,1077.84 2434.63,1077.71 2434.86,1077.96 2435.09,1077.92 2435.31,1077.59 2435.54,1077.85 2435.77,1078.09 2436,1077.85 2436.22,1078.13 2436.45,1077.76 2436.68,1077.65 2436.91,1077.61 2437.14,1077.32 2437.36,1077.35 2437.59,1077.74 2437.82,1077.61 2438.05,1077.7 2438.27,1077.71 2438.5,1077.31 2438.73,1077.55 2438.96,1077.52 2439.19,1077.82 2439.41,1077.9 2439.64,1077.97 2439.87,1076.81 2440.1,1076.85 2440.32,1076.58 2440.55,1076.05 2440.78,1075.27 2441.01,1075.27 2441.24,1075.19 2441.46,1074.91 2441.69,1074.43 2441.92,1074.32 2442.15,1074.6 2442.37,1074.75 2442.6,1074.61 2442.83,1073.86 2443.06,1073.86 2443.29,1074.02 2443.51,1074.24 2443.74,1073.88 2443.97,1073.95 2444.2,1074.2 2444.42,1074.51 2444.65,1074.29 2444.88,1074.48 2445.11,1074.49 2445.34,1074.14 2445.56,1074.45 2445.79,1074.65 2446.02,1074.46 2446.25,1074.55 2446.47,1074.43 2446.7,1074.35 2446.93,1074.34 2447.16,1074.64 2447.39,1074.47 2447.61,1074.41 2447.84,1074.88 2448.07,1075.1 2448.3,1074.76 2448.52,1074.71 2448.75,1074.75 2448.98,1074.87 2449.21,1074.59 2449.44,1074.73 2449.66,1074.72 2449.89,1074.99 2450.12,1074.58 2450.35,1074.4 2450.57,1074.22 2450.8,1074.13 2451.03,1074.15 2451.26,1074.03 2451.49,1074 2451.71,1074.08 2451.94,1074.13 2452.17,1074.56 2452.4,1074.76 2452.62,1075.05 2452.85,1075.25 2453.08,1074.88 2453.31,1074.4 2453.54,1074.46 2453.76,1074.52 2453.99,1074.44 2454.22,1074.03 2454.45,1074.19 2454.67,1074.38 2454.9,1074.36 2455.13,1074.11 2455.36,1074.36 2455.59,1074.78 2455.81,1075.22 2456.04,1075.31 2456.27,1074.91 2456.5,1074.99 2456.72,1074.94 2456.95,1075.22 2457.18,1075.18 2457.41,1075.42 2457.64,1075.73 2457.86,1075.94 2458.09,1076.25 2458.32,1076.52 2458.55,1076.62 2458.77,1076.1 2459,1076.05 2459.23,1076.18 2459.46,1076.38 2459.68,1076.77 2459.91,1076.99 2460.14,1076.8 2460.37,1076.59 2460.6,1076.3 2460.82,1076.08 2461.05,1076.38 2461.28,1076.54 2461.51,1076.01 2461.73,1075.84 2461.96,1076.19 2462.19,1075.92 2462.42,1075.5 2462.65,1075.42 2462.87,1074.57 2463.1,1074.99 2463.33,1075.08 2463.56,1075.54 2463.78,1075.61 2464.01,1075.51 2464.24,1075.43 2464.47,1075.26 2464.7,1075.08 2464.92,1075.14 2465.15,1075.07 2465.38,1074.81 2465.61,1074.81 2465.83,1074.45 2466.06,1074.55 2466.29,1074.56 2466.52,1074.94 2466.75,1074.97 2466.97,1074.51 2467.2,1074.59 2467.43,1074.3 2467.66,1074.31 2467.88,1073.84 2468.11,1073.63 2468.34,1073.64 2468.57,1073.2 2468.8,1072.98 2469.02,1073.4 2469.25,1073.6 2469.48,1073.5 2469.71,1073.55 2469.93,1073.81 2470.16,1074.11 2470.39,1074.31 2470.62,1074.43 2470.85,1074.8 2471.07,1074.09 2471.3,1073.53 2471.53,1073.38 2471.76,1073.18 2471.98,1073.24 2472.21,1073.25 2472.44,1073.69 2472.67,1073.02 2472.9,1072.94 2473.12,1072.62 2473.35,1072.79 2473.58,1072.57 2473.81,1073.27 2474.03,1073.03 2474.26,1072.91 2474.49,1072.14 2474.72,1072.06 2474.95,1071.79 2475.17,1071.95 2475.4,1072.05 2475.63,1072.4 2475.86,1072.48 2476.08,1072.86 2476.31,1072.72 2476.54,1072.95 2476.77,1073.24 2477,1073.59 2477.22,1073.46 2477.45,1073.32 2477.68,1073.31 2477.91,1073.37 2478.13,1073.86 2478.36,1073.64 2478.59,1073.88 2478.82,1074.27 2479.05,1074.6 2479.27,1074.67 2479.5,1074.94 2479.73,1075.16 2479.96,1075.09 2480.18,1074.84 2480.41,1074.76 2480.64,1074.78 2480.87,1074.22 2481.1,1074.42 2481.32,1073.91 2481.55,1073.69 2481.78,1073.23 2482.01,1073.27 2482.23,1073.84 2482.46,1073.62 2482.69,1073.9 2482.92,1074.26 2483.15,1073.87 2483.37,1074.04 2483.6,1074.1 2483.83,1073.92 2484.06,1073.96 2484.28,1074.35 2484.51,1074.28 2484.74,1073.86 2484.97,1073.75 2485.2,1073.79 2485.42,1073.79 2485.65,1074.18 2485.88,1074.51 2486.11,1074.25 2486.33,1074.13 2486.56,1074.23 2486.79,1074.07 2487.02,1073.27 2487.25,1073.79 2487.47,1073.89 2487.7,1073.76 2487.93,1073.55 2488.16,1073.36 2488.38,1073.14 2488.61,1073.26 2488.84,1073.06 2489.07,1073.42 2489.3,1073.08 2489.52,1073.23 2489.75,1073.7 2489.98,1073.66 2490.21,1073.62 2490.43,1073.79 2490.66,1074.17 2490.89,1074.45 2491.12,1074.25 2491.35,1074.69 2491.57,1074.67 2491.8,1074.62 2492.03,1074.84 2492.26,1074.81 2492.48,1074.87 2492.71,1074.37 2492.94,1074.32 2493.17,1074.29 2493.4,1073.93 2493.62,1073.88 2493.85,1073.85 2494.08,1074.05 2494.31,1074 2494.53,1073.54 2494.76,1073.12 2494.99,1073.07 2495.22,1073.02 2495.44,1073.26 2495.67,1073.02 2495.9,1073.3 2496.13,1073.53 2496.36,1073.89 2496.58,1073.99 2496.81,1074.07 2497.04,1074.12 2497.27,1073.85 2497.49,1073.85 2497.72,1073.2 2497.95,1073.16 2498.18,1072.84 2498.41,1072.12 2498.63,1071.88 2498.86,1072.2 2499.09,1072.14 2499.32,1072.05 2499.54,1072.17 2499.77,1072 2500,1072.15 2500.23,1071.7 2500.46,1071.75 2500.68,1071.81 2500.91,1071.94 2501.14,1072.27 2501.37,1071.63 2501.59,1071.11 2501.82,1071.46 2502.05,1071.66 2502.28,1071.54 2502.51,1071.8 2502.73,1072.17 2502.96,1072.29 2503.19,1072.53 2503.42,1072.73 2503.64,1072.61 2503.87,1072.92 2504.1,1073.37 2504.33,1073.44 2504.56,1073.44 2504.78,1073.51 2505.01,1073.69 2505.24,1074.31 2505.47,1074.38 2505.69,1074.75 2505.92,1075.19 2506.15,1074.77 2506.38,1074.3 2506.61,1074.57 2506.83,1074.85 2507.06,1074.73 2507.29,1074.61 2507.52,1074.1 2507.74,1073.78 2507.97,1073.2 2508.2,1073.08 2508.43,1073.72 2508.66,1074.09 2508.88,1073.12 2509.11,1072.98 2509.34,1073.07 2509.57,1073.06 2509.79,1073.1 2510.02,1073.2 2510.25,1073.41 2510.48,1073.48 2510.71,1073.77 2510.93,1073.29 2511.16,1073.28 2511.39,1073.37 2511.62,1073.43 2511.84,1073.81 2512.07,1074.17 2512.3,1073.97 2512.53,1073.95 2512.76,1074.37 2512.98,1074.57 2513.21,1074.48 2513.44,1074.46 2513.67,1074.18 2513.89,1073.75 2514.12,1073.91 2514.35,1074.05 2514.58,1074.02 2514.81,1073.74 2515.03,1073.1 2515.26,1073.03 2515.49,1073.37 2515.72,1072.75 2515.94,1072.77 2516.17,1072.96 2516.4,1072.83 2516.63,1073.01 2516.86,1073.12 2517.08,1073.29 2517.31,1073.37 2517.54,1073.39 2517.77,1073.21 2517.99,1073.22 2518.22,1073.3 2518.45,1073.45 2518.68,1073.49 2518.91,1073.53 2519.13,1073.31 2519.36,1073.46 2519.59,1073.64 2519.82,1074.1 2520.04,1074.45 2520.27,1073.94 2520.5,1074.45 2520.73,1074.63 2520.96,1074.41 2521.18,1074.65 2521.41,1074.68 2521.64,1074.76 2521.87,1074.7 2522.09,1074.66 2522.32,1074.66 2522.55,1074.4 2522.78,1074.22 2523.01,1074.05 2523.23,1073.87 2523.46,1074.2 2523.69,1073.89 2523.92,1073.89 2524.14,1073.94 2524.37,1073.64 2524.6,1073.71 2524.83,1073.16 2525.06,1073.38 2525.28,1073.8 2525.51,1073.76 2525.74,1073.5 2525.97,1073.78 2526.19,1073.64 2526.42,1073.65 2526.65,1073.97 2526.88,1073.76 2527.11,1074.17 2527.33,1074.13 2527.56,1074.33 2527.79,1073.95 2528.02,1073.78 2528.24,1073.59 2528.47,1073.59 2528.7,1073.86 2528.93,1074.09 2529.16,1073.25 2529.38,1073.4 2529.61,1073.5 2529.84,1073.51 2530.07,1072.93 2530.29,1073.03 2530.52,1073.5 2530.75,1073.23 2530.98,1073.22 2531.2,1073.21 2531.43,1072.84 2531.66,1072.8 2531.89,1072.9 2532.12,1073.24 2532.34,1073.38 2532.57,1072.72 2532.8,1072.9 2533.03,1073.07 2533.25,1072.68 2533.48,1073.12 2533.71,1072.99 2533.94,1073.29 2534.17,1073.82 2534.39,1073.65 2534.62,1074 2534.85,1073.95 2535.08,1073.75 2535.3,1073.72 2535.53,1073.26 2535.76,1072.84 2535.99,1072.97 2536.22,1073.01 2536.44,1072.27 2536.67,1072.12 2536.9,1072.4 2537.13,1072.27 2537.35,1072.22 2537.58,1072.31 2537.81,1072.16 2538.04,1072.24 2538.27,1072.47 2538.49,1073.03 2538.72,1072.73 2538.95,1072.67 2539.18,1072.61 2539.4,1072.83 2539.63,1073.24 2539.86,1073.27 2540.09,1073.68 2540.32,1073.44 2540.54,1073.53 2540.77,1073.35 2541,1073.16 2541.23,1073.32 2541.45,1073.3 2541.68,1073.33 2541.91,1072.71 2542.14,1072.54 2542.37,1072.51 2542.59,1072.47 2542.82,1072.67 2543.05,1072.72 2543.28,1073.14 2543.5,1073.43 2543.73,1073.66 2543.96,1073.55 2544.19,1073.76 2544.42,1073.57 2544.64,1073.59 2544.87,1073.7 2545.1,1073.91 2545.33,1073.65 2545.55,1073.84 2545.78,1073.81 2546.01,1074.32 2546.24,1074.2 2546.47,1073.97 2546.69,1074.23 2546.92,1074.11 2547.15,1074.19 2547.38,1073.99 2547.6,1074.18 2547.83,1074.05 2548.06,1073.92 2548.29,1073.99 2548.52,1074.26 2548.74,1073.92 2548.97,1073.54 2549.2,1073.46 2549.43,1073.69 2549.65,1073.6 2549.88,1073.42 2550.11,1073.6 2550.34,1073.42 2550.57,1073.59 2550.79,1073.5 2551.02,1073.77 2551.25,1073.57 2551.48,1073.71 2551.7,1073.65 2551.93,1073.75 2552.16,1073.96 2552.39,1074.09 2552.62,1074.44 2552.84,1074.53 2553.07,1074.7 2553.3,1074.93 2553.53,1074.9 2553.75,1075.12 2553.98,1074.57 2554.21,1074.53 2554.44,1074.52 2554.67,1074.64 2554.89,1074.46 2555.12,1074.29 2555.35,1074.43 2555.58,1074.6 2555.8,1075.08 2556.03,1074.86 2556.26,1074.8 2556.49,1074.85 2556.72,1074.5 2556.94,1074.5 2557.17,1074.63 2557.4,1074.59 2557.63,1074.58 2557.85,1074.58 2558.08,1074.37 2558.31,1074.6 2558.54,1074.88 2558.77,1074.66 2558.99,1073.88 2559.22,1073.99 2559.45,1073.82 2559.68,1074 2559.9,1073.92 2560.13,1074.02 2560.36,1073.89 2560.59,1073.66 2560.82,1073.51 2561.04,1074.13 2561.27,1074.24 2561.5,1074.46 2561.73,1073.91 2561.95,1073.94 2562.18,1073.51 2562.41,1073.51 2562.64,1073.74 2562.87,1073.75 2563.09,1073.96 2563.32,1074.05 2563.55,1073.78 2563.78,1074.04 2564,1074.54 2564.23,1074.27 2564.46,1074.42 2564.69,1074.66 2564.92,1074.89 2565.14,1074.82 2565.37,1074.98 2565.6,1075.05 2565.83,1074.99 2566.05,1074.97 2566.28,1074.67 2566.51,1074.69 2566.74,1074.59 2566.96,1074.54 2567.19,1074.59 2567.42,1074.66 2567.65,1074.96 2567.88,1074.78 2568.1,1075.01 2568.33,1074.87 2568.56,1074.5 2568.79,1074.47 2569.01,1074.67 2569.24,1075.01 2569.47,1075.2 2569.7,1075.27 2569.93,1075.11 2570.15,1075.14 2570.38,1075.7 2570.61,1075.84 2570.84,1076.37 2571.06,1076.5 2571.29,1076.35 2571.52,1076.45 2571.75,1076.52 2571.98,1076.3 2572.2,1076.07 2572.43,1076.34 "/>
</svg>

Figure 4: Monte Carlo mean and 95% confidence interval of the simulated
temperature in 2100 as the number of samples increases.

1.  Here we can see the benefits of setting the variable `n_samples`
    above; if we had hard-coded the value and needed to change it, we
    would also have needed to change it in all three of these lines.
2.  There are more efficient ways to do this updating, for example
    `T_est[i] = (T_est[i-1] * (i-1) + T_est[i]) / i`, which avoids
    re-computing the average for the first `i-1` simulations, but in
    this case it’s fast enough.
3.  We can’t compute the standard deviation for a single value, so we
    just start with `i == 2`.

If we wanted tighter confidence intervals, we could run the simulation
longer (though recall the $1/\sqrt{n}$ error law, which means to get an
order of magnitude reduction, we’d need approximately 100,000 samples),
but it looks from <a href="#fig-mc-est" class="quarto-xref">Figure 4</a>
as though the estimate had stabilized around 8,000-9,000 samples, though
ultimately it depends on the desired level of precision in both the
estimate and the confidence interval. In this case, regardless, we’re
looking at an estimated temperature anomaly of around $4.7^\circ C$ once
we exceed 5,000 samples, and it isn’t clear what more precision
necessarily would get us.

### Problem 3 (10 points)

**This problem is only required for students in BEE 5750**.

A factory is planning a third wastewater discharge into the river
downstream of the second plant. This discharge would consist of 5
m<sup>3</sup>/day of wastewater with a dissolved oxygen content of 4.5
mg/L and CBOD and NBOD levels of 50 and 45 mg/L, respectively.

**In this problem**:

-   Assume that the treatment plan for waste stream 2 that you
    identified in Problem 1 is still in place for the existing
    discharges. If the third discharge will not be treated, under the
    original inflow conditions (7.5 mg/L DO), how far downstream from
    the second discharge does this third discharge need to be placed to
    keep the river concentration from dropping below 4 mg/L?

#### Solution

Based on how we solved Problem 1, we need to modify our solution to
include a third discharge with an unknown distance from the second
discharge.

> **Tip**
>
> We could have solved Problem 1 in a more modular way, which included
> multiple discharges (using a vector of tuples instead of explicitly
> specifying `waste1` and `waste2`, which would have made this solution
> simpler).

``` julia
# do_simulate: function to simulate dissolved oxygen concentrations over the entire river
# inputs:
#   - d: distance between waste stream 2 and waste stream 3
#   - inflow: tuple with inflow properties: (Volume, DO, CBOD, NBOD)
#   - waste1: tuple with waste stream 1 properties: (Volume, DO, CBOD, NBOD)
#   - waste2: tuple with waste stream 2 properties: (Volume, DO, CBOD, NBOD)
#   - waste3: tuple with waste stream 3 properties: (Volume, DO, CBOD, NBOD)
#   - C₀, B₀, N₀: initial conditions for DO, CBOD, and NBOD, respectively (mg/L)
#   - U: velocity of river (km/d)
#   - Cₛ = saturation oxygen concentration (mg/L)
#   - ka, kc, kn: reaeration, CBOD decay, and NBOD decay rates, respectively (d^{-1})
function do_simulate2(d, inflow, waste1, waste2, waste3, U, Cₛ, ka, kb, kn)
    # set up ranges for each box/segment
    x₁ = 0:1:15
    x₂ = (15:1:15+d) .- 15 
    x₃ = (15+d:1:50) .- (15 + d)


    V_inflow, C_inflow, B_inflow, N_inflow = inflow
    V_ws1, C_ws1, B_ws1, N_ws1 = waste1
    V_ws2, C_ws2, B_ws2, N_ws2 = waste2
    V_ws3, C_ws3, B_ws3, N_ws3 = waste2

    # initialize storage for final C, B, and N
    # need to store d=1 so the total length should be d+1
    C = zeros(51)
    B = zeros(51)
    N = zeros(51)

    # compute initial conditions for first segment
    C₀ = mix_concentration(V_inflow, C_inflow, V_ws1, C_ws1)
    B₀ = mix_concentration(V_inflow, B_inflow, V_ws1, B_ws1)
    N₀ = mix_concentration(V_inflow, N_inflow, V_ws1, N_ws1)

    # conduct first segment simulation
    (C₁, B₁, N₁) = dissolved_oxygen(x₁, C₀, B₀, N₀, U, Cₛ, ka, kb, kn)
    C[1:15] = C₁[1:end-1] 
    B[1:15] = B₁[1:end-1]
    N[1:15] = N₁[1:end-1]

    # compute initial conditions for second segment
    C₀₁ = mix_concentration(V_inflow + V_ws1, C₁[end], V_ws2, C_ws2)
    B₀₁ = mix_concentration(V_inflow + V_ws1, B₁[end], V_ws2, B_ws2)
    N₀₁ = mix_concentration(V_inflow + V_ws1, N₁[end], V_ws2, N_ws2)

    # conduct second segment simulation
    (C₂, B₂, N₂) = dissolved_oxygen(x₂, C₀₁, B₀₁, N₀₁, U, Cₛ, ka, kb, kn)
    C[16:16+d] = C₂ 
    B[16:16+d] = B₂
    N[16:16+d] = N₂

    # compute initial conditions for third segment
    C₀₂ = mix_concentration(V_inflow + V_ws1 + V_ws2, C₂[end], V_ws3, C_ws3)
    B₀₂ = mix_concentration(V_inflow + V_ws1 + V_ws2, B₂[end], V_ws3, B_ws3)
    N₀₂ = mix_concentration(V_inflow + V_ws1 + V_ws2, N₂[end], V_ws3, N_ws3)

    # conduct second segment simulation
    (C₃, B₃, N₃) = dissolved_oxygen(x₃, C₀₂, B₀₂, N₀₂, U, Cₛ, ka, kb, kn)
    C[16+d:end] = C₃ 
    B[16+d:end] = B₃
    N[16+d:end] = N₃


    return (C, B, N)
end

# set up treatment level for waste stream 2 based on solution for Problem 1
B_ws2 = (1 - 0.08) * 45.0
N_ws2 = (1 - 0.08) * 35.0
waste2 = (V_ws2, C_ws2, B_ws2, N_ws2)

# set parameters for waste stream 3
C_ws3 = 4.5
B_ws3 = 50.0
N_ws3 = 45.0
V_ws3 = 5.0 * 1_000 # convert from m^3/day to L/day
waste3 = (V_ws3, C_ws3, B_ws3, N_ws3)
```

    (5000.0, 4.5, 50.0, 45.0)

1.  This is related to the issue in Problem 1; if your answer is
    different because of this issue with scaling and units you should
    receive full credit so long as your approach was clear.

Now we can evaluate `do_simulate2` over the different values of `d` to
find the minimum value of `C`.

``` julia
# three_discharge_minC: function to find the minimum DO concentration based on the three discharge model
# inputs:
#   - d: distance between waste stream 2 and waste stream 3
#   - inflow: tuple with inflow properties: (Volume, DO, CBOD, NBOD)
#   - waste1: tuple with waste stream 1 properties: (Volume, DO, CBOD, NBOD)
#   - waste2: tuple with waste stream 2 properties: (Volume, DO, CBOD, NBOD)
#   - waste3: tuple with waste stream 3 properties: (Volume, DO, CBOD, NBOD)
#   - C₀, B₀, N₀: initial conditions for DO, CBOD, and NBOD, respectively (mg/L)
#   - U: velocity of river (km/d)
#   - Cₛ = saturation oxygen concentration (mg/L)
#   - ka, kc, kn: reaeration, CBOD decay, and NBOD decay rates, respectively (d^{-1})
function three_discharge_minC(d, inflow, waste1, waste2, waste3, U, Cₛ, ka, kb, kn)
    C, B, N = do_simulate2(d, inflow, waste1, waste2, waste3, U, Cₛ, ka, kb, kn)
    return minimum(C)
end

# evaluate over all distances from 1 to 35 m downstream using broadcasting and anonymous functions
d = 1:1:35
minC_d = (dist -> three_discharge_minC(dist, inflow, waste1, waste2, waste3, U, Cₛ, ka, kb, kn)).(d)
idx = findfirst(minC_d .>= 4.0)
min_dist = d[idx]
@show min_dist;
```

    min_dist = 14

So, without treatment, the third discharge should be placed 14 m
downstream from the second discharge.

## References

List any external references consulted, including classmates.